In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.optimize import minimize_scalar

from src.rent_vs_buy import (
    RentVsBuy,
    rent_vs_buy_breakeven_objective_closure,
    render_plotly_html,
    rent_vs_buy_objective_closure,
)

## Setup
We can use `RentVsBuy`'s `calculate` function to simulate a financial scenario:

In [2]:
# We will reuse these kwargs later on.
kwargs = dict(
    home_price=1_000_000,  # Value of the home
    years=40,  # Compute earnings over 40-year period
    mortgage_rate=0.06342,  # Mortgage rate
    downpayment=0.2,  # Downpayment amount
    pmi=0.005,  # PMI expressed as a percentage of the total loan value
    length_of_mortgage=30,  # 30-year fixed mortgage
    home_price_growth_rate=0.08,  # Annual appreciation of the home
    rent_growth_rate=0.01,  # How much rent increases annually
    investment_return_rate=0.08,  # How much can we make in the market annually?
    investment_tax_rate=0.15,  # Long term capital gains tax
    inflation_rate=0.02,  # How much does inflation go up each year?
    filing_jointly=True,  # Tax filing status
    property_tax_rate=0.008,  # Annual tax rate on the home
    marginal_tax_rate=0.02,  # Tax bracket based on income
    costs_of_buying_home=0.01,
    costs_of_selling_home=0.06,  # Seller fee
    maintenance_rate=0.005,  # How much we put away each year for maintenance (roofs, appliances, etc.)
    home_owners_insurance_rate=0.004,  # Insurance on the home.
    monthly_utilities=0,  # Utilities for the home (usually a wash with renting utilities)
    monthly_common_fees=350,  # HOA fees, etc.
    monthly_rent=3550,  # How much is the current rent?
    security_deposit=1,  # Proportion of monthly rent paid in the first month
    brokers_fee=0.00,  # Brokers fee (if applicable)
    renters_insurance_rate=0.01,  # Rental insurance
)
# Create our RentVsBuy calculator.
rent_vs_buy = RentVsBuy().calculate(**kwargs)
df = rent_vs_buy.df
df

annual_per  per    home_value  first_month_home_value        ppmt  \
0             0    1  1.006434e+06            1.006434e+06  771.627534   
1             0    2  1.012909e+06            1.006434e+06  775.591638   
2             0    3  1.019427e+06            1.006434e+06  779.576107   
3             0    4  1.025986e+06            1.006434e+06  783.581046   
4             0    5  1.032587e+06            1.006434e+06  787.606559   
..          ...  ...           ...                     ...         ...   
475          39  476  2.117430e+07            2.024472e+07    0.000000   
476          39  477  2.131053e+07            2.024472e+07    0.000000   
477          39  478  2.144764e+07            2.024472e+07    0.000000   
478          39  479  2.158564e+07            2.024472e+07    0.000000   
479          39  480  2.172452e+07            2.024472e+07    0.000000   

            ipmt          pmt  maintenance    insurance  property_taxes  ...  \
0    4109.862808  4881.490342   418.389567   334.864534      668.508349  ...   
1    4105.898704  4881.490342   418.389567   334.864534      668.508349  ...   
2    4101.914235  4881.490342   418.389567   334.864534      668.508349  ...   
3    4097.909296  4881.490342   418.389567   334.864534      668.508349  ...   
4    4093.883783  4881.490342   418.389567   334.864534      668.508349  ...   
..           ...          ...          ...          ...             ...  ...   
475     0.000000     0.000000  8416.030678  6735.899789    13447.244450  ...   
476     0.000000     0.000000  8416.030678  6735.899789    13447.244450  ...   
477     0.000000     0.000000  8416.030678  6735.899789    13447.244450  ...   
478     0.000000     0.000000  8416.030678  6735.899789    13447.244450  ...   
479     0.000000     0.000000  8416.030678  6735.899789    13447.244450  ...   

     total_rent_assets  home_opportunity_cost  rental_opportunity_cost  \
0                  0.0          209517.752792             0.000000e+00   
1                  0.0            3067.752792             0.000000e+00   
2                  0.0            3067.752792             0.000000e+00   
3                  0.0            3067.752792             0.000000e+00   
4                  0.0            3067.752792             0.000000e+00   
..                 ...                    ...                      ...   
475                0.0           24071.369332             0.000000e+00   
476                0.0           24071.369332             0.000000e+00   
477                0.0           24071.369332             0.000000e+00   
478                0.0           24071.369332             0.000000e+00   
479             3550.0               0.000000             2.039343e+07   

     home_opportunity_cost_fv  rental_opportunity_cost_fv  \
0                4.522575e+06               -0.000000e+00   
1                6.579607e+04               -0.000000e+00   
2                6.537544e+04               -0.000000e+00   
3                6.495750e+04               -0.000000e+00   
4                6.454224e+04               -0.000000e+00   
..                        ...                         ...   
475              2.469688e+04               -0.000000e+00   
476              2.453899e+04               -0.000000e+00   
477              2.438212e+04               -0.000000e+00   
478              2.422625e+04               -0.000000e+00   
479             -0.000000e+00                2.039343e+07   

     home_opportunity_cost_fv_post_tax  rental_opportunity_cost_fv_post_tax  \
0                         3.844188e+06                         0.000000e+00   
1                         5.592666e+04                         0.000000e+00   
2                         5.556913e+04                         0.000000e+00   
3                         5.521388e+04                         0.000000e+00   
4                         5.486090e+04                         0.000000e+00   
..                                 ...                     

## Calculations
**High level overview**

First, model the cash flows over the life of the home. These cashflows should incorporate any inflation or appreciation that occurs with time.  For this we need:
- Value of the home
- Liabilities of the home
- Assets the home
- Liabilities of renting
- Assets of renting

Next, we can calculate the opportunity cost of either renting or buying. This is the difference in money (i.e. $8000 monthly house payment - $3000 monthly rent payment) that we can invest for the remaining time of the home, appreciated at market rates.

Below, we show the output of these various calculations for the given scenario.

### Home Value
How much will the home be worth when we sell it?

In [3]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Home Value": df.home_value,
                "Initial Home Price": df.home_value[0],
            }
        )
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="8642a03d-b7da-43a3-b42e-1072886bb84d" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("8642a03d-b7da-43a3-b42e-1072886bb84d")) {                    Plotly.newPlot(                        "8642a03d-b7da-43a3-b42e-1072886bb84d",                        [{"hovertemplate":"variable=Home Value\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Home Value","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Home Value","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[1006434.0301100034,1012909.4569634633,1019426.5469082735,1025985.5680060182,1032586.7900429977,1039230.4845413263,1045916.9247700985,1052646.3857566314,1059419.144297776,1066235.478971302,1073095.6701473573,1079999.9999999995,1086948.7525188033,1093942.2135205402,1100980.670660935,1108064.4134464993,1115193.7332464373,1122368.9233046318,1129590.278751706,1136858.0966171618,1144172.6758415978,1151534.3172890057,1158943.3237591456,1166399.9999999993,1173904.6527203075,1181457.590602183,1189059.1243138094,1196709.5665222188,1204409.2319061519,1212158.4371690021,1219957.5010518422,1227806.7443465344,1235706.4899089253,1243657.0626721259,1251658.7896598768,1259711.9999999988,1267817.0249379314,1275974.197850357,1284183.854258914,1292446.3318439957,1300761.9704586437,1309131.112142522,1317554.1011359892,1326031.2838942567,1334563.009101639,1343149.6276858957,1351791.4928326667,1360488.9599999983,1369242.3869329658,1378052.1336783853,1386918.5625996266,1395842.0383915151,1404822.928095335,1413861.6011139234,1422958.429226868,1432113.786605797,1441328.0498297696,1450601.5979007669,1459934.8122592794,1469328.0767999978,1478781.7778876026,1488296.3043726557,1497872.0476075963,1507509.401462836,1517208.762342961,1526970.5292030366,1536795.103565017,1546682.8895342601,1556634.2938161506,1566649.7257328278,1576729.5972400212,1586874.322943997,1597084.3201186103,1607360.0087224678,1617701.8114162034,1628110.1535798623,1638585.4633303976,1649128.1715392792,1659738.7118502178,1670417.5206970004,1681165.0373214423,1691981.7037914535,1702867.9650192223,1713824.2687795165,1724851.0657280984,1735948.8094202646,1747117.9563294991,1758358.965866251,1769672.300396829,1781058.4252624211,1792517.8087982347,1804050.9223527599,1815658.2403071572,1827340.240094769,1839097.4022207598,1850930.2102818773,1862839.1509863457,1874824.7141738853,1886887.3928358585,1899027.6831355505,1911246.0844285747,1923543.0992834142,1935919.233502093,1948374.9961409802,1960910.899531729,1973527.4593023502,1986225.19439842,1999004.6271044267,2011866.283065253,2024810.6913077955,2037838.3842627269,2050949.8977863938,2064145.7711828598,2077426.5472260867,2090792.77218226,2104244.995832258,2117783.771494267,2131409.6560465377,2145123.209950293,2158924.99727278,2172815.5857104724,2186795.5466124183,2200865.4550037445,2215025.8896093047,2229277.4328774884,2243620.671004173,2258056.19395684,2272584.595498838,2287206.4732138077,2301922.42853026,2316733.066746316,2331638.9970546025,2346640.8325673095,2361739.190341411,2376934.6914040428,2392227.9607780483,2407619.627507686,2423110.3246845063,2438700.6894733864,2454391.363138744,2470182.9910709113,2486076.22281268,2502071.7120860205,2518170.1168189696,2534372.0991726937,2550678.3255687235,2567089.466716366,2583606.1976402914,2600229.1977083003,2616959.150659266,2633796.7446312564,2650742.672189843,2667797.6303565837,2684962.320637693,2702237.4490529015,2719623.7261644863,2737121.8671065085,2754732.5916142203,2772456.6240536743,2790294.693451514,2808247.533524964,2826315.882712006,2844500.484201756,2862802.0859650294,2881221.4407851095,2899759.306288708,2918416.444977132,2937193.6242576446,2956091.616475028,2975111.1989433574,2994253.1539779673,3013518.268927634,3032907.33620696,3052421.153328966,3072060.5229378957,3091826.2528422307,3111719.156047917,3131740.0507918037,3151889.760575302,3172169.114198255,3192578.9457930294,3213120.0948588247,3233793.406296204,3254599.730441844,3275539.9231035155,3296614.845595282,3317825.3647729266,3339172.3530696086,3360656.6885317494,3382279.254855147,3404040.941421325,3425942.6433341145,3447985.261456471,3470169.70244753,3492496.8787998985,3514967.70887719,3537583.1169517958,3560344.0332429036,3583251.3939547595,3606306.1413151766,3629509.2236142885,3652861.595243558,3676364.21673503,3700018.0548008424,3723824.0823729876,3747783.278643331,3771896.6291038897,3796165.1255873647,3820589.7663079384,3845171.555902335,3869911.505471139,3894810.632620389,3919869.9615034303,3945090.522863041,3970473.3540738313,3996019.4991849083,4021730.008962825,4047605.940934797,4073648.3594322,4099858.3356343526,4126236.947612573,4152785.2803745205,4179504.4259088286,4206395.483230019,4233459.558423704,4260697.764692083,4288111.222399737,4315701.0591197,4343468.40967985,4371414.4162095785,4399540.228186774,4427847.002485099,4456335.903421577,4485008.102804481,4513864.779981534,4542907.121888419,4572136.323097598,4601553.585867449,4631160.120191714,4660957.143849274,4690945.882454237,4721127.569506343,4751503.446441716,4782074.762683906,4812842.775695302,4843808.751028838,4874973.962380055,4906339.691639491,4937907.228945405,4969677.872736843,5001652.92980705,5033833.715357216,5066221.553050574,5098817.77506685,5131623.722157051,5164640.7436986165,5197870.197750924,5231313.451111143,5264971.879370458,5298846.866970649,5332939.807261036,5367252.102555789,5401785.164191613,5436540.412585791,5471519.277294618,5506723.197072196,5542153.619929614,5577812.003194504,5613699.813570997,5649818.527200033,5686169.629720094,5722754.6163283,5759574.991841917,5796632.27076025,5833927.97732694,5871463.645592652,5909240.819478186,5947261.05283797,5985525.909523981,6024036.963450063,6062795.798656674,6101804.009376034,6141063.2000977,6180574.985634562,6220340.991189268,6260362.852421069,6300642.2155130925,6341180.737240062,6381980.085036439,6423041.937065005,6464367.982285896,6505959.920526067,6547819.462549207,6589948.330126114,6632348.256105513,6675020.9844853245,6717968.270484408,6761191.880614752,6804693.592754139,6848475.196219265,6892538.491839352,6936885.292030205,6981517.4208687665,7026436.71416815,7071645.019553142,7117144.196536202,7162936.116593951,7209022.663244149,7255405.732123158,7302087.23106393,7349069.0801744675,7396353.211916804,7443941.571186498,7491836.115392618,7540038.814538266,7588551.651301599,7637376.62111739,7686515.732259096,7735971.005921466,7785744.476303678,7835838.190693009,7886254.209549042,7936994.606588422,7988061.468870147,8039456.896881415,8091183.004624025,8143241.919701325,8195635.783405726,8248366.75080678,8301436.990839821,8354848.686395181,8408604.03440797,8462705.245948447,8517154.546312962,8571954.175115494,8627106.386379756,8682613.448631926,8738477.644993944,8794701.273277428,8851286.64607818,8908236.090871317,8965551.950107004,9023236.581306791,9081292.357160605,9139721.66562432,9198526.910017999,9257710.509124732,9317274.897290133,9377222.524522478,9437555.856593458,9498277.375139618,9559389.577764431,9620894.978141021,9682796.106115563,9745095.507811334,9807795.745733451,9870899.398874262,9934409.062819434,9998327.349854706,10062656.889073342,10127400.326484272,10192560.325120931,10258139.565150786,10324140.743985584,10390566.5763923,10457419.794604803,10524703.148436237,10592419.405392123,10660571.350784201,10729161.787844988,10798193.53784308,10867669.440199206,10937592.352603013,11007965.151130604,11078790.730362846,11150072.003504427,11221811.90250368,11294013.378173186,11366679.400311131,11439812.957823489,11513417.058846934,11587494.73087258,11662049.020870524,11737082.99541514,11812599.74081125,11888602.363221047,11965093.98879187,12042077.76378478,12119556.85470397,12197534.448427035,12276013.752336018,12354997.994449364,12434490.423554685,12514494.309342384,12595012.94254016,12676049.635048347,12757607.720076144,12839690.552278727,12922301.507895214,13005443.984887555,13089121.403080286,13173337.204301195,13258094.852522897,13343397.834005311,13429249.657439055,13515653.854089772,13602613.977943368,13690133.60585221,13778216.337682232,13866865.796461022,13956085.62852683,14045879.503678557,14136251.115326704,14227204.180645285,14318742.440724725,14410869.660725731,14503589.630034177,14596906.162416948,14690823.096178833,14785344.294320382,14880473.644696806,14976215.0601779,15072572.478808971,15169549.863972837,15267151.204552835,15365380.515096905,15464241.835982697,15563739.233583786,15663876.800436905,15764658.6554103,15866088.943873135,15968171.837866008,16070911.536272546,16174312.264992125,16278378.277113685,16383113.853090659,16488523.30091706,16594610.956304653,16701381.182861308,16808838.372270484,16916986.94447185,17025831.347843118,17135376.059382983,17245625.584895283,17356584.459174346,17468257.24619149,17580648.539282773,17693762.961337905,17807605.164990418,17922179.83280902,18037491.67749021,18153545.44205212,18270345.900029596,18387897.855670564,18506206.144133613,18625275.6316869,18745111.21590829,18865717.825886805,18987100.42242539,19109263.998244934,19232213.57818965,19355954.219433732,19480491.01168942,19605829.077416282,19731973.572031956,19858929.6841242,19986702.6356643,20115297.68222185,20244720.113180947,20374975.251957744,20506068.456219412,20638005.118104525,20770790.66444481,20904430.556988426,21038930.292624567,21174295.403609578,21310531.457794506,21447644.05885413,21585638.846517436,21724521.49679959],"yaxis":"y","type":"scatter"},{"hovertemplate":"variable=Initial Home Price\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Initial Home Price","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Initial Home Price","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034,1006434.0301100034],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### House Liabilities
What will we need to pay to upkeep the home?

In [4]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Monthly Fees": df.monthly_common_fees,
                "Insurance": df.insurance,
                "Property Taxes": df.property_taxes,
                "Maintenance": df.maintenance,
                "Closing Costs": df.buying_closing_costs,
                "Mortgage Principal Payment": df.ppmt,
                "Mortgage Interest Payment": df.ipmt,
                "Total Mortgage Payment": df.pmt,
                "Utilities": df.home_monthly_utilities,
                "Downpayment": df.down_fee,
                "PMI": df.pmi,
                "Sellers Fee": df.sellers_fee,
                "Loan Payoff": df.loan_payoff,
            }
        ),
        log_y=True,
        markers=True,
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="7730965c-2456-492d-ad3c-506854c74870" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("7730965c-2456-492d-ad3c-506854c74870")) {                    Plotly.newPlot(                        "7730965c-2456-492d-ad3c-506854c74870",                        [{"hovertemplate":"variable=Monthly Fees\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Monthly Fees","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Monthly Fees","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,357.0000000000002,357.0000000000002,357.0000000000002,357.0000000000002,357.0000000000002,357.0000000000002,357.0000000000002,357.0000000000002,357.0000000000002,357.0000000000002,357.0000000000002,357.0000000000002,364.14000000000044,364.14000000000044,364.14000000000044,364.14000000000044,364.14000000000044,364.14000000000044,364.14000000000044,364.14000000000044,364.14000000000044,364.14000000000044,364.14000000000044,364.14000000000044,371.42280000000073,371.42280000000073,371.42280000000073,371.42280000000073,371.42280000000073,371.42280000000073,371.42280000000073,371.42280000000073,371.42280000000073,371.42280000000073,371.42280000000073,371.42280000000073,378.851256000001,378.851256000001,378.851256000001,378.851256000001,378.851256000001,378.851256000001,378.851256000001,378.851256000001,378.851256000001,378.851256000001,378.851256000001,378.851256000001,386.4282811200013,386.4282811200013,386.4282811200013,386.4282811200013,386.4282811200013,386.4282811200013,386.4282811200013,386.4282811200013,386.4282811200013,386.4282811200013,386.4282811200013,386.4282811200013,394.15684674240157,394.15684674240157,394.15684674240157,394.15684674240157,394.15684674240157,394.15684674240157,394.15684674240157,394.15684674240157,394.15684674240157,394.15684674240157,394.15684674240157,394.15684674240157,402.0399836772499,402.0399836772499,402.0399836772499,402.0399836772499,402.0399836772499,402.0399836772499,402.0399836772499,402.0399836772499,402.0399836772499,402.0399836772499,402.0399836772499,402.0399836772499,410.0807833507952,410.0807833507952,410.0807833507952,410.0807833507952,410.0807833507952,410.0807833507952,410.0807833507952,410.0807833507952,410.0807833507952,410.0807833507952,410.0807833507952,410.0807833507952,418.2823990178114,418.2823990178114,418.2823990178114,418.2823990178114,418.2823990178114,418.2823990178114,418.2823990178114,418.2823990178114,418.2823990178114,418.2823990178114,418.2823990178114,418.2823990178114,426.64804699816784,426.64804699816784,426.64804699816784,426.64804699816784,426.64804699816784,426.64804699816784,426.64804699816784,426.64804699816784,426.64804699816784,426.64804699816784,426.64804699816784,426.64804699816784,435.1810079381315,435.1810079381315,435.1810079381315,435.1810079381315,435.1810079381315,435.1810079381315,435.1810079381315,435.1810079381315,435.1810079381315,435.1810079381315,435.1810079381315,435.1810079381315,443.8846280968944,443.8846280968944,443.8846280968944,443.8846280968944,443.8846280968944,443.8846280968944,443.8846280968944,443.8846280968944,443.8846280968944,443.8846280968944,443.8846280968944,443.8846280968944,452.7623206588326,452.7623206588326,452.7623206588326,452.7623206588326,452.7623206588326,452.7623206588326,452.7623206588326,452.7623206588326,452.7623206588326,452.7623206588326,452.7623206588326,452.7623206588326,461.8175670720096,461.8175670720096,461.8175670720096,461.8175670720096,461.8175670720096,461.8175670720096,461.8175670720096,461.8175670720096,461.8175670720096,461.8175670720096,461.8175670720096,461.8175670720096,471.0539184134501,471.0539184134501,471.0539184134501,471.0539184134501,471.0539184134501,471.0539184134501,471.0539184134501,471.0539184134501,471.0539184134501,471.0539184134501,471.0539184134501,471.0539184134501,480.4749967817194,480.4749967817194,480.4749967817194,480.4749967817194,480.4749967817194,480.4749967817194,480.4749967817194,480.4749967817194,480.4749967817194,480.4749967817194,480.4749967817194,480.4749967817194,490.0844967173541,490.0844967173541,490.0844967173541,490.0844967173541,490.0844967173541,490.0844967173541,490.0844967173541,490.0844967173541,490.0844967173541,490.0844967173541,490.0844967173541,490.0844967173541,499.88618665170156,499.88618665170156,499.88618665170156,499.88618665170156,499.88618665170156,499.88618665170156,499.88618665170156,499.88618665170156,499.88618665170156,499.88618665170156,499.88618665170156,499.88618665170156,509.88391038473594,509.88391038473594,509.88391038473594,509.88391038473594,509.88391038473594,509.88391038473594,509.88391038473594,509.88391038473594,509.88391038473594,509.88391038473594,509.88391038473594,509.88391038473594,520.081588592431,520.081588592431,520.081588592431,520.081588592431,520.081588592431,520.081588592431,520.081588592431,520.081588592431,520.081588592431,520.081588592431,520.081588592431,520.081588592431,530.48322036428,530.48322036428,530.48322036428,530.48322036428,530.48322036428,530.48322036428,530.48322036428,530.48322036428,530.48322036428,530.48322036428,530.48322036428,530.48322036428,541.0928847715659,541.0928847715659,541.0928847715659,541.0928847715659,541.0928847715659,541.0928847715659,541.0928847715659,541.0928847715659,541.0928847715659,541.0928847715659,541.0928847715659,541.0928847715659,551.9147424669976,551.9147424669976,551.9147424669976,551.9147424669976,551.9147424669976,551.9147424669976,551.9147424669976,551.9147424669976,551.9147424669976,551.9147424669976,551.9147424669976,551.9147424669976,562.9530373163379,562.9530373163379,562.9530373163379,562.9530373163379,562.9530373163379,562.9530373163379,562.9530373163379,562.9530373163379,562.9530373163379,562.9530373163379,562.9530373163379,562.9530373163379,574.2120980626652,574.2120980626652,574.2120980626652,574.2120980626652,574.2120980626652,574.2120980626652,574.2120980626652,574.2120980626652,574.2120980626652,574.2120980626652,574.2120980626652,574.2120980626652,585.6963400239188,585.6963400239188,585.6963400239188,585.6963400239188,585.6963400239188,585.6963400239188,585.6963400239188,585.6963400239188,585.6963400239188,585.6963400239188,585.6963400239188,585.6963400239188,597.4102668243976,597.4102668243976,597.4102668243976,597.4102668243976,597.4102668243976,597.4102668243976,597.4102668243976,597.4102668243976,597.4102668243976,597.4102668243976,597.4102668243976,597.4102668243976,609.358472160886,609.358472160886,609.358472160886,609.358472160886,609.358472160886,609.358472160886,609.358472160886,609.358472160886,609.358472160886,609.358472160886,609.358472160886,609.358472160886,621.545641604104,621.545641604104,621.545641604104,621.545641604104,621.545641604104,621.545641604104,621.545641604104,621.545641604104,621.545641604104,621.545641604104,621.545641604104,621.545641604104,633.9765544361866,633.9765544361866,633.9765544361866,633.9765544361866,633.9765544361866,633.9765544361866,633.9765544361866,633.9765544361866,633.9765544361866,633.9765544361866,633.9765544361866,633.9765544361866,646.6560855249107,646.6560855249107,646.6560855249107,646.6560855249107,646.6560855249107,646.6560855249107,646.6560855249107,646.6560855249107,646.6560855249107,646.6560855249107,646.6560855249107,646.6560855249107,659.5892072354094,659.5892072354094,659.5892072354094,659.5892072354094,659.5892072354094,659.5892072354094,659.5892072354094,659.5892072354094,659.5892072354094,659.5892072354094,659.5892072354094,659.5892072354094,672.780991380118,672.780991380118,672.780991380118,672.780991380118,672.780991380118,672.780991380118,672.780991380118,672.780991380118,672.780991380118,672.780991380118,672.780991380118,672.780991380118,686.2366112077208,686.2366112077208,686.2366112077208,686.2366112077208,686.2366112077208,686.2366112077208,686.2366112077208,686.2366112077208,686.2366112077208,686.2366112077208,686.2366112077208,686.2366112077208,699.9613434318757,699.9613434318757,699.9613434318757,699.9613434318757,699.9613434318757,699.9613434318757,699.9613434318757,699.9613434318757,699.9613434318757,699.9613434318757,699.9613434318757,699.9613434318757,713.9605703005137,713.9605703005137,713.9605703005137,713.9605703005137,713.9605703005137,713.9605703005137,713.9605703005137,713.9605703005137,713.9605703005137,713.9605703005137,713.9605703005137,713.9605703005137,728.2397817065245,728.2397817065245,728.2397817065245,728.2397817065245,728.2397817065245,728.2397817065245,728.2397817065245,728.2397817065245,728.2397817065245,728.2397817065245,728.2397817065245,728.2397817065245,742.8045773406554,742.8045773406554,742.8045773406554,742.8045773406554,742.8045773406554,742.8045773406554,742.8045773406554,742.8045773406554,742.8045773406554,742.8045773406554,742.8045773406554,742.8045773406554,757.6606688874691,757.6606688874691,757.6606688874691,757.6606688874691,757.6606688874691,757.6606688874691,757.6606688874691,757.6606688874691,757.6606688874691,757.6606688874691,757.6606688874691,757.6606688874691],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Insurance\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Insurance","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Insurance","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[334.86453422477456,334.86453422477456,334.86453422477456,334.86453422477456,334.86453422477456,334.86453422477456,334.86453422477456,334.86453422477456,334.86453422477456,334.86453422477456,334.86453422477456,334.86453422477456,361.65369696275644,361.65369696275644,361.65369696275644,361.65369696275644,361.65369696275644,361.65369696275644,361.65369696275644,361.65369696275644,361.65369696275644,361.65369696275644,361.65369696275644,361.65369696275644,390.58599271977687,390.58599271977687,390.58599271977687,390.58599271977687,390.58599271977687,390.58599271977687,390.58599271977687,390.58599271977687,390.58599271977687,390.58599271977687,390.58599271977687,390.58599271977687,421.83287213735883,421.83287213735883,421.83287213735883,421.83287213735883,421.83287213735883,421.83287213735883,421.83287213735883,421.83287213735883,421.83287213735883,421.83287213735883,421.83287213735883,421.83287213735883,455.5795019083475,455.5795019083475,455.5795019083475,455.5795019083475,455.5795019083475,455.5795019083475,455.5795019083475,455.5795019083475,455.5795019083475,455.5795019083475,455.5795019083475,455.5795019083475,492.0258620610151,492.0258620610151,492.0258620610151,492.0258620610151,492.0258620610151,492.0258620610151,492.0258620610151,492.0258620610151,492.0258620610151,492.0258620610151,492.0258620610151,492.0258620610151,531.3879310258961,531.3879310258961,531.3879310258961,531.3879310258961,531.3879310258961,531.3879310258961,531.3879310258961,531.3879310258961,531.3879310258961,531.3879310258961,531.3879310258961,531.3879310258961,573.8989655079677,573.8989655079677,573.8989655079677,573.8989655079677,573.8989655079677,573.8989655079677,573.8989655079677,573.8989655079677,573.8989655079677,573.8989655079677,573.8989655079677,573.8989655079677,619.8108827486049,619.8108827486049,619.8108827486049,619.8108827486049,619.8108827486049,619.8108827486049,619.8108827486049,619.8108827486049,619.8108827486049,619.8108827486049,619.8108827486049,619.8108827486049,669.395753368493,669.395753368493,669.395753368493,669.395753368493,669.395753368493,669.395753368493,669.395753368493,669.395753368493,669.395753368493,669.395753368493,669.395753368493,669.395753368493,722.9474136379723,722.9474136379723,722.9474136379723,722.9474136379723,722.9474136379723,722.9474136379723,722.9474136379723,722.9474136379723,722.9474136379723,722.9474136379723,722.9474136379723,722.9474136379723,780.7832067290099,780.7832067290099,780.7832067290099,780.7832067290099,780.7832067290099,780.7832067290099,780.7832067290099,780.7832067290099,780.7832067290099,780.7832067290099,780.7832067290099,780.7832067290099,843.2458632673305,843.2458632673305,843.2458632673305,843.2458632673305,843.2458632673305,843.2458632673305,843.2458632673305,843.2458632673305,843.2458632673305,843.2458632673305,843.2458632673305,843.2458632673305,910.7055323287167,910.7055323287167,910.7055323287167,910.7055323287167,910.7055323287167,910.7055323287167,910.7055323287167,910.7055323287167,910.7055323287167,910.7055323287167,910.7055323287167,910.7055323287167,983.5619749150136,983.5619749150136,983.5619749150136,983.5619749150136,983.5619749150136,983.5619749150136,983.5619749150136,983.5619749150136,983.5619749150136,983.5619749150136,983.5619749150136,983.5619749150136,1062.2469329082144,1062.2469329082144,1062.2469329082144,1062.2469329082144,1062.2469329082144,1062.2469329082144,1062.2469329082144,1062.2469329082144,1062.2469329082144,1062.2469329082144,1062.2469329082144,1062.2469329082144,1147.2266875408714,1147.2266875408714,1147.2266875408714,1147.2266875408714,1147.2266875408714,1147.2266875408714,1147.2266875408714,1147.2266875408714,1147.2266875408714,1147.2266875408714,1147.2266875408714,1147.2266875408714,1239.0048225441406,1239.0048225441406,1239.0048225441406,1239.0048225441406,1239.0048225441406,1239.0048225441406,1239.0048225441406,1239.0048225441406,1239.0048225441406,1239.0048225441406,1239.0048225441406,1239.0048225441406,1338.1252083476713,1338.1252083476713,1338.1252083476713,1338.1252083476713,1338.1252083476713,1338.1252083476713,1338.1252083476713,1338.1252083476713,1338.1252083476713,1338.1252083476713,1338.1252083476713,1338.1252083476713,1445.1752250154846,1445.1752250154846,1445.1752250154846,1445.1752250154846,1445.1752250154846,1445.1752250154846,1445.1752250154846,1445.1752250154846,1445.1752250154846,1445.1752250154846,1445.1752250154846,1445.1752250154846,1560.7892430167233,1560.7892430167233,1560.7892430167233,1560.7892430167233,1560.7892430167233,1560.7892430167233,1560.7892430167233,1560.7892430167233,1560.7892430167233,1560.7892430167233,1560.7892430167233,1560.7892430167233,1685.6523824580604,1685.6523824580604,1685.6523824580604,1685.6523824580604,1685.6523824580604,1685.6523824580604,1685.6523824580604,1685.6523824580604,1685.6523824580604,1685.6523824580604,1685.6523824580604,1685.6523824580604,1820.5045730547047,1820.5045730547047,1820.5045730547047,1820.5045730547047,1820.5045730547047,1820.5045730547047,1820.5045730547047,1820.5045730547047,1820.5045730547047,1820.5045730547047,1820.5045730547047,1820.5045730547047,1966.1449388990804,1966.1449388990804,1966.1449388990804,1966.1449388990804,1966.1449388990804,1966.1449388990804,1966.1449388990804,1966.1449388990804,1966.1449388990804,1966.1449388990804,1966.1449388990804,1966.1449388990804,2123.436534011006,2123.436534011006,2123.436534011006,2123.436534011006,2123.436534011006,2123.436534011006,2123.436534011006,2123.436534011006,2123.436534011006,2123.436534011006,2123.436534011006,2123.436534011006,2293.311456731886,2293.311456731886,2293.311456731886,2293.311456731886,2293.311456731886,2293.311456731886,2293.311456731886,2293.311456731886,2293.311456731886,2293.311456731886,2293.311456731886,2293.311456731886,2476.776373270436,2476.776373270436,2476.776373270436,2476.776373270436,2476.776373270436,2476.776373270436,2476.776373270436,2476.776373270436,2476.776373270436,2476.776373270436,2476.776373270436,2476.776373270436,2674.91848313207,2674.91848313207,2674.91848313207,2674.91848313207,2674.91848313207,2674.91848313207,2674.91848313207,2674.91848313207,2674.91848313207,2674.91848313207,2674.91848313207,2674.91848313207,2888.911961782635,2888.911961782635,2888.911961782635,2888.911961782635,2888.911961782635,2888.911961782635,2888.911961782635,2888.911961782635,2888.911961782635,2888.911961782635,2888.911961782635,2888.911961782635,3120.0249187252452,3120.0249187252452,3120.0249187252452,3120.0249187252452,3120.0249187252452,3120.0249187252452,3120.0249187252452,3120.0249187252452,3120.0249187252452,3120.0249187252452,3120.0249187252452,3120.0249187252452,3369.6269122232634,3369.6269122232634,3369.6269122232634,3369.6269122232634,3369.6269122232634,3369.6269122232634,3369.6269122232634,3369.6269122232634,3369.6269122232634,3369.6269122232634,3369.6269122232634,3369.6269122232634,3639.197065201124,3639.197065201124,3639.197065201124,3639.197065201124,3639.197065201124,3639.197065201124,3639.197065201124,3639.197065201124,3639.197065201124,3639.197065201124,3639.197065201124,3639.197065201124,3930.3328304172123,3930.3328304172123,3930.3328304172123,3930.3328304172123,3930.3328304172123,3930.3328304172123,3930.3328304172123,3930.3328304172123,3930.3328304172123,3930.3328304172123,3930.3328304172123,3930.3328304172123,4244.759456850587,4244.759456850587,4244.759456850587,4244.759456850587,4244.759456850587,4244.759456850587,4244.759456850587,4244.759456850587,4244.759456850587,4244.759456850587,4244.759456850587,4244.759456850587,4584.340213398634,4584.340213398634,4584.340213398634,4584.340213398634,4584.340213398634,4584.340213398634,4584.340213398634,4584.340213398634,4584.340213398634,4584.340213398634,4584.340213398634,4584.340213398634,4951.087430470523,4951.087430470523,4951.087430470523,4951.087430470523,4951.087430470523,4951.087430470523,4951.087430470523,4951.087430470523,4951.087430470523,4951.087430470523,4951.087430470523,4951.087430470523,5347.174424908163,5347.174424908163,5347.174424908163,5347.174424908163,5347.174424908163,5347.174424908163,5347.174424908163,5347.174424908163,5347.174424908163,5347.174424908163,5347.174424908163,5347.174424908163,5774.948378900815,5774.948378900815,5774.948378900815,5774.948378900815,5774.948378900815,5774.948378900815,5774.948378900815,5774.948378900815,5774.948378900815,5774.948378900815,5774.948378900815,5774.948378900815,6236.944249212878,6236.944249212878,6236.944249212878,6236.944249212878,6236.944249212878,6236.944249212878,6236.944249212878,6236.944249212878,6236.944249212878,6236.944249212878,6236.944249212878,6236.944249212878,6735.8997891499075,6735.8997891499075,6735.8997891499075,6735.8997891499075,6735.8997891499075,6735.8997891499075,6735.8997891499075,6735.8997891499075,6735.8997891499075,6735.8997891499075,6735.8997891499075,6735.8997891499075],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Property Taxes\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Property Taxes","line":{"color":"#00cc96","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Property Taxes","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[668.5083493197047,668.5083493197047,668.5083493197047,668.5083493197047,668.5083493197047,668.5083493197047,668.5083493197047,668.5083493197047,668.5083493197047,668.5083493197047,668.5083493197047,668.5083493197047,721.9890172652808,721.9890172652808,721.9890172652808,721.9890172652808,721.9890172652808,721.9890172652808,721.9890172652808,721.9890172652808,721.9890172652808,721.9890172652808,721.9890172652808,721.9890172652808,779.7481386465032,779.7481386465032,779.7481386465032,779.7481386465032,779.7481386465032,779.7481386465032,779.7481386465032,779.7481386465032,779.7481386465032,779.7481386465032,779.7481386465032,779.7481386465032,842.127989738223,842.127989738223,842.127989738223,842.127989738223,842.127989738223,842.127989738223,842.127989738223,842.127989738223,842.127989738223,842.127989738223,842.127989738223,842.127989738223,909.4982289172808,909.4982289172808,909.4982289172808,909.4982289172808,909.4982289172808,909.4982289172808,909.4982289172808,909.4982289172808,909.4982289172808,909.4982289172808,909.4982289172808,909.4982289172808,982.2580872306629,982.2580872306629,982.2580872306629,982.2580872306629,982.2580872306629,982.2580872306629,982.2580872306629,982.2580872306629,982.2580872306629,982.2580872306629,982.2580872306629,982.2580872306629,1060.8387342091155,1060.8387342091155,1060.8387342091155,1060.8387342091155,1060.8387342091155,1060.8387342091155,1060.8387342091155,1060.8387342091155,1060.8387342091155,1060.8387342091155,1060.8387342091155,1060.8387342091155,1145.7058329458443,1145.7058329458443,1145.7058329458443,1145.7058329458443,1145.7058329458443,1145.7058329458443,1145.7058329458443,1145.7058329458443,1145.7058329458443,1145.7058329458443,1145.7058329458443,1145.7058329458443,1237.3622995815115,1237.3622995815115,1237.3622995815115,1237.3622995815115,1237.3622995815115,1237.3622995815115,1237.3622995815115,1237.3622995815115,1237.3622995815115,1237.3622995815115,1237.3622995815115,1237.3622995815115,1336.3512835480321,1336.3512835480321,1336.3512835480321,1336.3512835480321,1336.3512835480321,1336.3512835480321,1336.3512835480321,1336.3512835480321,1336.3512835480321,1336.3512835480321,1336.3512835480321,1336.3512835480321,1443.2593862318743,1443.2593862318743,1443.2593862318743,1443.2593862318743,1443.2593862318743,1443.2593862318743,1443.2593862318743,1443.2593862318743,1443.2593862318743,1443.2593862318743,1443.2593862318743,1443.2593862318743,1558.7201371304238,1558.7201371304238,1558.7201371304238,1558.7201371304238,1558.7201371304238,1558.7201371304238,1558.7201371304238,1558.7201371304238,1558.7201371304238,1558.7201371304238,1558.7201371304238,1558.7201371304238,1683.4177481008574,1683.4177481008574,1683.4177481008574,1683.4177481008574,1683.4177481008574,1683.4177481008574,1683.4177481008574,1683.4177481008574,1683.4177481008574,1683.4177481008574,1683.4177481008574,1683.4177481008574,1818.0911679489254,1818.0911679489254,1818.0911679489254,1818.0911679489254,1818.0911679489254,1818.0911679489254,1818.0911679489254,1818.0911679489254,1818.0911679489254,1818.0911679489254,1818.0911679489254,1818.0911679489254,1963.5384613848387,1963.5384613848387,1963.5384613848387,1963.5384613848387,1963.5384613848387,1963.5384613848387,1963.5384613848387,1963.5384613848387,1963.5384613848387,1963.5384613848387,1963.5384613848387,1963.5384613848387,2120.621538295625,2120.621538295625,2120.621538295625,2120.621538295625,2120.621538295625,2120.621538295625,2120.621538295625,2120.621538295625,2120.621538295625,2120.621538295625,2120.621538295625,2120.621538295625,2290.271261359275,2290.271261359275,2290.271261359275,2290.271261359275,2290.271261359275,2290.271261359275,2290.271261359275,2290.271261359275,2290.271261359275,2290.271261359275,2290.271261359275,2290.271261359275,2473.492962268016,2473.492962268016,2473.492962268016,2473.492962268016,2473.492962268016,2473.492962268016,2473.492962268016,2473.492962268016,2473.492962268016,2473.492962268016,2473.492962268016,2473.492962268016,2671.372399249456,2671.372399249456,2671.372399249456,2671.372399249456,2671.372399249456,2671.372399249456,2671.372399249456,2671.372399249456,2671.372399249456,2671.372399249456,2671.372399249456,2671.372399249456,2885.082191189412,2885.082191189412,2885.082191189412,2885.082191189412,2885.082191189412,2885.082191189412,2885.082191189412,2885.082191189412,2885.082191189412,2885.082191189412,2885.082191189412,2885.082191189412,3115.8887664845643,3115.8887664845643,3115.8887664845643,3115.8887664845643,3115.8887664845643,3115.8887664845643,3115.8887664845643,3115.8887664845643,3115.8887664845643,3115.8887664845643,3115.8887664845643,3115.8887664845643,3365.159867803328,3365.159867803328,3365.159867803328,3365.159867803328,3365.159867803328,3365.159867803328,3365.159867803328,3365.159867803328,3365.159867803328,3365.159867803328,3365.159867803328,3365.159867803328,3634.372657227593,3634.372657227593,3634.372657227593,3634.372657227593,3634.372657227593,3634.372657227593,3634.372657227593,3634.372657227593,3634.372657227593,3634.372657227593,3634.372657227593,3634.372657227593,3925.1224698057995,3925.1224698057995,3925.1224698057995,3925.1224698057995,3925.1224698057995,3925.1224698057995,3925.1224698057995,3925.1224698057995,3925.1224698057995,3925.1224698057995,3925.1224698057995,3925.1224698057995,4239.132267390262,4239.132267390262,4239.132267390262,4239.132267390262,4239.132267390262,4239.132267390262,4239.132267390262,4239.132267390262,4239.132267390262,4239.132267390262,4239.132267390262,4239.132267390262,4578.262848781482,4578.262848781482,4578.262848781482,4578.262848781482,4578.262848781482,4578.262848781482,4578.262848781482,4578.262848781482,4578.262848781482,4578.262848781482,4578.262848781482,4578.262848781482,4944.523876683999,4944.523876683999,4944.523876683999,4944.523876683999,4944.523876683999,4944.523876683999,4944.523876683999,4944.523876683999,4944.523876683999,4944.523876683999,4944.523876683999,4944.523876683999,5340.085786818717,5340.085786818717,5340.085786818717,5340.085786818717,5340.085786818717,5340.085786818717,5340.085786818717,5340.085786818717,5340.085786818717,5340.085786818717,5340.085786818717,5340.085786818717,5767.292649764213,5767.292649764213,5767.292649764213,5767.292649764213,5767.292649764213,5767.292649764213,5767.292649764213,5767.292649764213,5767.292649764213,5767.292649764213,5767.292649764213,5767.292649764213,6228.676061745348,6228.676061745348,6228.676061745348,6228.676061745348,6228.676061745348,6228.676061745348,6228.676061745348,6228.676061745348,6228.676061745348,6228.676061745348,6228.676061745348,6228.676061745348,6726.970146684974,6726.970146684974,6726.970146684974,6726.970146684974,6726.970146684974,6726.970146684974,6726.970146684974,6726.970146684974,6726.970146684974,6726.970146684974,6726.970146684974,6726.970146684974,7265.1277584197705,7265.1277584197705,7265.1277584197705,7265.1277584197705,7265.1277584197705,7265.1277584197705,7265.1277584197705,7265.1277584197705,7265.1277584197705,7265.1277584197705,7265.1277584197705,7265.1277584197705,7846.3379790933495,7846.3379790933495,7846.3379790933495,7846.3379790933495,7846.3379790933495,7846.3379790933495,7846.3379790933495,7846.3379790933495,7846.3379790933495,7846.3379790933495,7846.3379790933495,7846.3379790933495,8474.045017420814,8474.045017420814,8474.045017420814,8474.045017420814,8474.045017420814,8474.045017420814,8474.045017420814,8474.045017420814,8474.045017420814,8474.045017420814,8474.045017420814,8474.045017420814,9151.968618814477,9151.968618814477,9151.968618814477,9151.968618814477,9151.968618814477,9151.968618814477,9151.968618814477,9151.968618814477,9151.968618814477,9151.968618814477,9151.968618814477,9151.968618814477,9884.126108319631,9884.126108319631,9884.126108319631,9884.126108319631,9884.126108319631,9884.126108319631,9884.126108319631,9884.126108319631,9884.126108319631,9884.126108319631,9884.126108319631,9884.126108319631,10674.8561969852,10674.8561969852,10674.8561969852,10674.8561969852,10674.8561969852,10674.8561969852,10674.8561969852,10674.8561969852,10674.8561969852,10674.8561969852,10674.8561969852,10674.8561969852,11528.844692744013,11528.844692744013,11528.844692744013,11528.844692744013,11528.844692744013,11528.844692744013,11528.844692744013,11528.844692744013,11528.844692744013,11528.844692744013,11528.844692744013,11528.844692744013,12451.15226816353,12451.15226816353,12451.15226816353,12451.15226816353,12451.15226816353,12451.15226816353,12451.15226816353,12451.15226816353,12451.15226816353,12451.15226816353,12451.15226816353,12451.15226816353,13447.24444961661,13447.24444961661,13447.24444961661,13447.24444961661,13447.24444961661,13447.24444961661,13447.24444961661,13447.24444961661,13447.24444961661,13447.24444961661,13447.24444961661,13447.24444961661],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Maintenance\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Maintenance","line":{"color":"#ab63fa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Maintenance","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[418.3895665571823,418.3895665571823,418.3895665571823,418.3895665571823,418.3895665571823,418.3895665571823,418.3895665571823,418.3895665571823,418.3895665571823,418.3895665571823,418.3895665571823,418.3895665571823,451.8607318817568,451.8607318817568,451.8607318817568,451.8607318817568,451.8607318817568,451.8607318817568,451.8607318817568,451.8607318817568,451.8607318817568,451.8607318817568,451.8607318817568,451.8607318817568,488.0095904322973,488.0095904322973,488.0095904322973,488.0095904322973,488.0095904322973,488.0095904322973,488.0095904322973,488.0095904322973,488.0095904322973,488.0095904322973,488.0095904322973,488.0095904322973,527.0503576668808,527.0503576668808,527.0503576668808,527.0503576668808,527.0503576668808,527.0503576668808,527.0503576668808,527.0503576668808,527.0503576668808,527.0503576668808,527.0503576668808,527.0503576668808,569.2143862802312,569.2143862802312,569.2143862802312,569.2143862802312,569.2143862802312,569.2143862802312,569.2143862802312,569.2143862802312,569.2143862802312,569.2143862802312,569.2143862802312,569.2143862802312,614.7515371826495,614.7515371826495,614.7515371826495,614.7515371826495,614.7515371826495,614.7515371826495,614.7515371826495,614.7515371826495,614.7515371826495,614.7515371826495,614.7515371826495,614.7515371826495,663.9316601572613,663.9316601572613,663.9316601572613,663.9316601572613,663.9316601572613,663.9316601572613,663.9316601572613,663.9316601572613,663.9316601572613,663.9316601572613,663.9316601572613,663.9316601572613,717.0461929698419,717.0461929698419,717.0461929698419,717.0461929698419,717.0461929698419,717.0461929698419,717.0461929698419,717.0461929698419,717.0461929698419,717.0461929698419,717.0461929698419,717.0461929698419,774.409888407429,774.409888407429,774.409888407429,774.409888407429,774.409888407429,774.409888407429,774.409888407429,774.409888407429,774.409888407429,774.409888407429,774.409888407429,774.409888407429,836.3626794800231,836.3626794800231,836.3626794800231,836.3626794800231,836.3626794800231,836.3626794800231,836.3626794800231,836.3626794800231,836.3626794800231,836.3626794800231,836.3626794800231,836.3626794800231,903.2716938384247,903.2716938384247,903.2716938384247,903.2716938384247,903.2716938384247,903.2716938384247,903.2716938384247,903.2716938384247,903.2716938384247,903.2716938384247,903.2716938384247,903.2716938384247,975.5334293454982,975.5334293454982,975.5334293454982,975.5334293454982,975.5334293454982,975.5334293454982,975.5334293454982,975.5334293454982,975.5334293454982,975.5334293454982,975.5334293454982,975.5334293454982,1053.5761036931378,1053.5761036931378,1053.5761036931378,1053.5761036931378,1053.5761036931378,1053.5761036931378,1053.5761036931378,1053.5761036931378,1053.5761036931378,1053.5761036931378,1053.5761036931378,1053.5761036931378,1137.8621919885886,1137.8621919885886,1137.8621919885886,1137.8621919885886,1137.8621919885886,1137.8621919885886,1137.8621919885886,1137.8621919885886,1137.8621919885886,1137.8621919885886,1137.8621919885886,1137.8621919885886,1228.8911673476753,1228.8911673476753,1228.8911673476753,1228.8911673476753,1228.8911673476753,1228.8911673476753,1228.8911673476753,1228.8911673476753,1228.8911673476753,1228.8911673476753,1228.8911673476753,1228.8911673476753,1327.202460735489,1327.202460735489,1327.202460735489,1327.202460735489,1327.202460735489,1327.202460735489,1327.202460735489,1327.202460735489,1327.202460735489,1327.202460735489,1327.202460735489,1327.202460735489,1433.3786575943277,1433.3786575943277,1433.3786575943277,1433.3786575943277,1433.3786575943277,1433.3786575943277,1433.3786575943277,1433.3786575943277,1433.3786575943277,1433.3786575943277,1433.3786575943277,1433.3786575943277,1548.0489502018736,1548.0489502018736,1548.0489502018736,1548.0489502018736,1548.0489502018736,1548.0489502018736,1548.0489502018736,1548.0489502018736,1548.0489502018736,1548.0489502018736,1548.0489502018736,1548.0489502018736,1671.8928662180226,1671.8928662180226,1671.8928662180226,1671.8928662180226,1671.8928662180226,1671.8928662180226,1671.8928662180226,1671.8928662180226,1671.8928662180226,1671.8928662180226,1671.8928662180226,1671.8928662180226,1805.644295515464,1805.644295515464,1805.644295515464,1805.644295515464,1805.644295515464,1805.644295515464,1805.644295515464,1805.644295515464,1805.644295515464,1805.644295515464,1805.644295515464,1805.644295515464,1950.0958391567008,1950.0958391567008,1950.0958391567008,1950.0958391567008,1950.0958391567008,1950.0958391567008,1950.0958391567008,1950.0958391567008,1950.0958391567008,1950.0958391567008,1950.0958391567008,1950.0958391567008,2106.103506289236,2106.103506289236,2106.103506289236,2106.103506289236,2106.103506289236,2106.103506289236,2106.103506289236,2106.103506289236,2106.103506289236,2106.103506289236,2106.103506289236,2106.103506289236,2274.591786792374,2274.591786792374,2274.591786792374,2274.591786792374,2274.591786792374,2274.591786792374,2274.591786792374,2274.591786792374,2274.591786792374,2274.591786792374,2274.591786792374,2274.591786792374,2456.5591297357632,2456.5591297357632,2456.5591297357632,2456.5591297357632,2456.5591297357632,2456.5591297357632,2456.5591297357632,2456.5591297357632,2456.5591297357632,2456.5591297357632,2456.5591297357632,2456.5591297357632,2653.083860114624,2653.083860114624,2653.083860114624,2653.083860114624,2653.083860114624,2653.083860114624,2653.083860114624,2653.083860114624,2653.083860114624,2653.083860114624,2653.083860114624,2653.083860114624,2865.3305689237927,2865.3305689237927,2865.3305689237927,2865.3305689237927,2865.3305689237927,2865.3305689237927,2865.3305689237927,2865.3305689237927,2865.3305689237927,2865.3305689237927,2865.3305689237927,2865.3305689237927,3094.557014437695,3094.557014437695,3094.557014437695,3094.557014437695,3094.557014437695,3094.557014437695,3094.557014437695,3094.557014437695,3094.557014437695,3094.557014437695,3094.557014437695,3094.557014437695,3342.1215755927096,3342.1215755927096,3342.1215755927096,3342.1215755927096,3342.1215755927096,3342.1215755927096,3342.1215755927096,3342.1215755927096,3342.1215755927096,3342.1215755927096,3342.1215755927096,3342.1215755927096,3609.4913016401256,3609.4913016401256,3609.4913016401256,3609.4913016401256,3609.4913016401256,3609.4913016401256,3609.4913016401256,3609.4913016401256,3609.4913016401256,3609.4913016401256,3609.4913016401256,3609.4913016401256,3898.250605771335,3898.250605771335,3898.250605771335,3898.250605771335,3898.250605771335,3898.250605771335,3898.250605771335,3898.250605771335,3898.250605771335,3898.250605771335,3898.250605771335,3898.250605771335,4210.11065423304,4210.11065423304,4210.11065423304,4210.11065423304,4210.11065423304,4210.11065423304,4210.11065423304,4210.11065423304,4210.11065423304,4210.11065423304,4210.11065423304,4210.11065423304,4546.919506571683,4546.919506571683,4546.919506571683,4546.919506571683,4546.919506571683,4546.919506571683,4546.919506571683,4546.919506571683,4546.919506571683,4546.919506571683,4546.919506571683,4546.919506571683,4910.673067097416,4910.673067097416,4910.673067097416,4910.673067097416,4910.673067097416,4910.673067097416,4910.673067097416,4910.673067097416,4910.673067097416,4910.673067097416,4910.673067097416,4910.673067097416,5303.526912465207,5303.526912465207,5303.526912465207,5303.526912465207,5303.526912465207,5303.526912465207,5303.526912465207,5303.526912465207,5303.526912465207,5303.526912465207,5303.526912465207,5303.526912465207,5727.809065462422,5727.809065462422,5727.809065462422,5727.809065462422,5727.809065462422,5727.809065462422,5727.809065462422,5727.809065462422,5727.809065462422,5727.809065462422,5727.809065462422,5727.809065462422,6186.033790699414,6186.033790699414,6186.033790699414,6186.033790699414,6186.033790699414,6186.033790699414,6186.033790699414,6186.033790699414,6186.033790699414,6186.033790699414,6186.033790699414,6186.033790699414,6680.916493955365,6680.916493955365,6680.916493955365,6680.916493955365,6680.916493955365,6680.916493955365,6680.916493955365,6680.916493955365,6680.916493955365,6680.916493955365,6680.916493955365,6680.916493955365,7215.389813471792,7215.389813471792,7215.389813471792,7215.389813471792,7215.389813471792,7215.389813471792,7215.389813471792,7215.389813471792,7215.389813471792,7215.389813471792,7215.389813471792,7215.389813471792,7792.620998549533,7792.620998549533,7792.620998549533,7792.620998549533,7792.620998549533,7792.620998549533,7792.620998549533,7792.620998549533,7792.620998549533,7792.620998549533,7792.620998549533,7792.620998549533,8416.030678433493,8416.030678433493,8416.030678433493,8416.030678433493,8416.030678433493,8416.030678433493,8416.030678433493,8416.030678433493,8416.030678433493,8416.030678433493,8416.030678433493,8416.030678433493],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Closing Costs\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Closing Costs","line":{"color":"#FFA15A","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Closing Costs","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[10000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Mortgage Principal Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Mortgage Principal Payment","line":{"color":"#19d3f3","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Mortgage Principal Payment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[771.6275341269902,775.5916382568676,779.5761072919067,783.5810458533151,787.6065590997746,791.6527527302014,795.7197329865198,799.8076066564581,803.916481076345,808.0464641339308,812.1976642712225,816.3701904873292,820.5641523413228,824.7796599551175,829.0168240163584,833.275755781332,837.5565670778819,841.8593703083488,846.1842784525247,850.5314050706097,854.9008643062048,859.2927708893035,863.7072401393029,868.1443879680346,872.6043308828089,877.0871859894701,881.593070995475,886.1221042129832,890.6744045619603,895.250091573304,899.8492853919824,904.472106780187,909.1186771205039,913.7891184191021,918.4835533089363,923.2021050529665,927.9448975473956,932.712055324921,937.5037035580071,942.3199680621692,947.1609752992786,952.0268523808818,956.9177270715413,961.8337277921851,966.7749836234852,971.7416243092403,976.7337802597876,981.7515825554246,986.7951629498498,991.8646538736266,996.9601884376557,1002.0819004366704,1007.2299243527577,1012.4043953588757,1017.6054493224169,1022.8332228087643,1028.087853084885,1033.369478122931,1038.6782366038624,1044.0142679210876,1049.3777121841276,1054.7687102222917,1060.1874035883702,1065.6339345623628,1071.1084461552086,1076.6110821125344,1082.1419869184433,1087.7013057992954,1093.2891847275282,1098.905770425487,1104.5512103692777,1110.2256527926438,1115.929246690845,1121.6621418245873,1127.4244887239433,1133.2164386923073,1139.0381438103695,1144.8897569401106,1150.7714317288096,1156.6833226130852,1162.6255848229457,1168.59837438587,1174.601848130897,1180.6361636927513,1186.7014795159762,1192.797954859101,1198.9257497988137,1205.0850252341716,1211.275942890822,1217.4986653252508,1223.7533559290491,1230.0401789332059,1236.3592994124156,1242.71088328942,1249.0950973393565,1255.5121091941437,1261.962087346878,1268.445201156264,1274.9616208510538,1281.5115175345213,1288.0950631889564,1294.712430680177,1301.3637937620688,1308.0493270811485,1314.7692061811504,1321.5236075076332,1328.3127084126177,1335.1366871592345,1341.9957229264164,1348.8899958135935,1355.8196868454256,1362.7849779765593,1369.7860520963986,1376.8230930339128,1383.8962855624573,1391.0058154046328,1398.151869237157,1405.3346346957665,1412.5543003801436,1419.8110558588724,1427.1050916744075,1434.4365993480887,1441.8057713851617,1449.2128012798312,1456.6578835203509,1464.1412135941214,1471.662987992826,1479.2234042175928,1486.8226607841762,1494.4609572281702,1502.1384941102497,1509.85547302144,1517.6120965883974,1525.4085684787433,1533.2450934064063,1541.1218771369954,1549.0391264932086,1556.997049360259,1564.9958546913294,1573.0357525130703,1581.1169539311063,1589.2396711355786,1597.4041174067206,1605.610507120457,1613.8590557540315,1622.149979891662,1630.4834972302383,1638.8598265850233,1647.2791878954076,1655.7418022306847,1664.247891795851,1672.7976799374483,1681.3913911494146,1690.029251078996,1698.7114865326535,1707.4383254820355,1716.20999706995,1725.02673161639,1733.888760624579,1742.7963167870425,1751.7496339917325,1760.7489473281535,1769.7944930935428,1778.8865087990794,1788.025233176109,1797.2109061824235,1806.4437690085515,1815.7240640841032,1825.0520350841243,1834.4279269354997,1843.8519858233867,1853.3244591976759,1862.8455957794854,1872.415645567702,1882.0348598455334,1891.7034911871156,1901.421793464137,1911.1900218525102,1921.0084328390722,1930.8772842283147,1940.7968351491581,1950.767346061747,1960.7890787643041,1970.8622963999896,1980.9872634638182,1991.164245809602,2001.3935106569343,2011.6753265981984,2022.009963605628,2032.3976930383951,2042.838787649724,2053.333521594073,2063.8821704343154,2074.485011148981,2085.1423221395353,2095.854383237675,2106.6214757126913,2117.4438822788456,2128.3218871027952,2139.255775811054,2150.2458354974956,2161.2923547308865,2172.3956235624673,2183.555933533566,2194.7735776832574,2206.0488505560475,2217.3820482096207,2228.7734682226055,2240.2234097023875,2251.7321732929695,2263.3000611828534,2274.927377112988,2286.6144263847464,2298.361515867917,2310.1689540087973,2322.0370508382616,2333.9661179799264,2345.9564686583085,2358.008417707073,2370.1222815772803,2382.2983783457116,2394.537027723205,2406.8385510630656,2419.2032713694935,2431.631513306062,2444.123603204258,2456.679869072032,2469.300640602423,2481.9862491822087,2494.7370279006154,2507.5533115580524,2520.435436674909,2533.3837415003927,2546.39856602141,2559.480251971483,2572.629142839744,2585.8455838799296,2599.1299221194704,2612.482506368576,2625.9036872294255,2639.3938171053424,2652.9532502100697,2666.582342577059,2680.28145206883,2694.0509383863473,2707.891163078481,2721.8024895515127,2735.785283078636,2749.839910809591,2763.966741780283,2778.166146922472,2792.4384990735125,2806.7841729861602,2821.2035453383896,2835.6969947432954,2850.2649017590334,2864.9076488988044,2879.625620640917,2894.4192034388643,2909.288785731479,2924.2347579531347,2939.257512543987,2954.3574439602903,2969.534948684751,2984.790425236939,3000.124274183746,3015.5368981499114,3031.0287018285835,3046.600091991962,3062.25147750196,3077.9832693209532,3093.7958805225653,3109.6897263025185,3125.665223989521,3141.7227930562476,3157.8628551303345,3174.08583400546,3190.392155652479,3206.7822482305746,3223.2565420985497,3239.815469826087,3256.459466205132,3273.1889682612855,3290.0044152653068,3306.906248744619,3323.8949124949318,3340.9708525918713,3358.1345174026983,3375.386357598086,3392.726826163953,3410.156378413354,3427.6754719984374,3445.2845669224557,3462.9841255518577,3480.774612628413,3498.6564952814265,3516.6302430399974,3534.696327845356,3552.8552240632403,3571.107408496369,3589.453360396952,3607.8935614792645,3626.428495932327,3645.0586504325706,3663.784514156674,3682.6065787943517,3701.525338561304,3720.5412902121675,3739.6549330535904,3758.866768957304,3778.1773023733313,3797.587040343208,3817.096492513324,3836.706171148278,3856.4165911443497,3876.228270043003,3896.141728044484,3916.1574880214857,3936.276075532857,3956.498018837422,3976.823848907847,3997.254099444574,4017.789306889842,4038.430010441769,4059.176752068512,4080.0300765224965,4100.990531354721,4122.058666929124,4143.235036437061,4164.520195911806,4185.914704243144,4207.419123192089,4229.034017405576,4250.759954431342,4272.597504732792,4294.54724170398,4316.609741684694,4338.785583975547,4361.07535085323,4383.479627585766,4405.999002447893,4428.634066736524,4451.385414786243,4474.253643984924,4497.239354789434,4520.343150741373,4543.565638482939,4566.907427772845,4590.369131502329,4613.951365711273,4637.654749604337,4661.479905567251,4685.427459183133,4709.498039248951,4733.692277791982,4758.010810086445,4782.454274670173,4807.023313361386,4831.71857127552,4856.5406968421885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Mortgage Interest Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Mortgage Interest Payment","line":{"color":"#FF6692","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Mortgage Interest Payment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[4109.862807695208,4105.898703565331,4101.914234530292,4097.909295968883,4093.883782722424,4089.837589091997,4085.7706088356786,4081.6827351657403,4077.5738607458534,4073.4438776882675,4069.292677550976,4065.120151334869,4060.9261894808756,4056.710681867081,4052.47351780584,4048.2145860408664,4043.9337747443165,4039.6309715138495,4035.3060633696737,4030.9589367515887,4026.5894775159936,4022.197570932895,4017.7831016828954,4013.345953854164,4008.8860109393895,4004.4031558327283,3999.8972708267233,3995.368237609215,3990.815937260238,3986.2402502488944,3981.641056430216,3977.0182350420114,3972.3716647016945,3967.7012234030963,3963.006788513262,3958.288236769232,3953.545444274803,3948.7782864972773,3943.9866382641912,3939.170373760029,3934.32936652292,3929.4634894413166,3924.572614750657,3919.6566140300133,3914.715358198713,3909.748717512958,3904.7565615624108,3899.7387592667737,3894.6951788723486,3889.625687948572,3884.5301533845427,3879.408441385528,3874.2604174694407,3869.0859464633227,3863.8848924997815,3858.657119013434,3853.4024887373134,3848.1208636992674,3842.812105218336,3837.476073901111,3832.1126296380708,3826.7216315999067,3821.302938233828,3815.8564072598356,3810.3818956669897,3804.879259709664,3799.348354903755,3793.789036022903,3788.20115709467,3782.5845713967115,3776.9391314529207,3771.2646890295546,3765.5610951313533,3759.828199997611,3754.065853098255,3748.273903129891,3742.452198011829,3736.600584882088,3730.718910093389,3724.807019209113,3718.8647569992527,3712.8919674363283,3706.8884936913014,3700.854178129447,3694.788862306222,3688.6923869630973,3682.5645920233846,3676.405316588027,3670.2143989313763,3663.9916764969475,3657.7369858931493,3651.4501628889925,3645.131042409783,3638.7794585327783,3632.395244482842,3625.9782326280547,3619.5282544753204,3613.0451406659345,3606.5287209711446,3599.978824287677,3593.395278633242,3586.7779111420214,3580.1265480601296,3573.44101474105,3566.721135641048,3559.966734314565,3553.1776334095807,3546.353654662964,3539.494618895782,3532.600346008605,3525.6706549767728,3518.705363845639,3511.7042897257998,3504.6672487882856,3497.594056259741,3490.4845264175656,3483.3384725850415,3476.155707126432,3468.936041442055,3461.679285963326,3454.385250147791,3447.0537424741096,3439.6845704370367,3432.277540542367,3424.8324583018475,3417.349128228077,3409.8273538293724,3402.2669376046056,3394.6676810380222,3387.029384594028,3379.3518477119487,3371.6348688007583,3363.878245233801,3356.081773343455,3348.245248415792,3340.368464685203,3332.4512153289897,3324.4932924619393,3316.494487130869,3308.454589309128,3300.373387891092,3292.2506706866197,3284.086224415478,3275.8798347017414,3267.631286068167,3259.3403619305363,3251.00684459196,3242.630515237175,3234.2111539267908,3225.7485395915137,3217.2424500263473,3208.69266188475,3200.0989506727838,3191.4610907432025,3182.778855289545,3174.052016340163,3165.2803447522483,3156.4636102058084,3147.6015811976195,3138.694025035156,3129.740707830466,3120.741394494045,3111.6958487286556,3102.603833023119,3093.4651086460894,3084.279435639775,3075.046572813647,3065.766277738095,3056.438306738074,3047.0624148866987,3037.6383559988117,3028.1658826245225,3018.644746042713,3009.0746962544963,2999.455481976665,2989.786850635083,2980.0685483580614,2970.300319969688,2960.4819089831262,2950.6130575938837,2940.6935066730402,2930.7229957604513,2920.7012630578943,2910.6280454222087,2900.50307835838,2890.3260960125963,2880.096831165264,2869.815015224,2859.4803782165704,2849.0926487838033,2838.6515541724743,2828.1568202281255,2817.608171387883,2807.0053306732175,2796.348019682663,2785.6359585845235,2774.868866109507,2764.0464595433527,2753.168454719403,2742.2345660111446,2731.244506324703,2720.197987091312,2709.094718259731,2697.9344082886323,2686.716764138941,2675.441491266151,2664.1082936125777,2652.716873599593,2641.266932119811,2629.758168529229,2618.190280639345,2606.5629647092105,2594.875915437452,2583.128825954281,2571.321387813401,2559.4532909839368,2547.524223842272,2535.53387316389,2523.4819241151254,2511.368060244918,2499.1919634764868,2486.9533140989934,2474.651790759133,2462.287070452705,2449.8588285161363,2437.3667386179404,2424.8104727501664,2412.1897012197755,2399.5040926399897,2386.753313921583,2373.937030264146,2361.0549051472894,2348.1066003218057,2335.0917758007886,2322.0100898507153,2308.8611989824544,2295.644757942269,2282.360419702728,2269.007835453622,2255.586654592773,2242.096524716856,2228.5370916121287,2214.9079992451393,2201.2088897533686,2187.439403435851,2173.5991787437174,2159.6878522706857,2145.7050587435624,2131.6504310126074,2117.5236000419154,2103.3241948997265,2089.051842748686,2074.706168836038,2060.2867964838088,2045.7933470789033,2031.225440063165,2016.582692923394,2001.8647211812813,1987.0711383833338,1972.2015560907191,1957.2555838690637,1942.2328292782113,1927.132897861908,1911.9553931374476,1896.6999165852594,1881.3660676384527,1865.9534436722872,1850.4616399936149,1834.8902498302364,1819.2388643202382,1803.5070725012451,1787.694461299633,1771.8006155196802,1755.8251178326775,1739.767548765951,1723.6274866918636,1707.404507816738,1691.0981861697196,1674.708093591624,1658.2337997236486,1641.6748719961113,1625.0308756170666,1608.301373560913,1591.4859265568916,1574.5840930775794,1557.5954293272666,1540.519489230327,1523.3558244194999,1506.1039842241128,1488.7635156582455,1471.3339634088443,1453.8148698237612,1436.2057748997427,1418.506216270341,1400.7157291937858,1382.833846540772,1364.860098782201,1346.7940139768423,1328.6351177589581,1310.3829333258295,1292.0369814252463,1273.596780342934,1255.0618458898712,1236.4316913896278,1217.7058276655246,1198.883763027847,1179.965003260894,1160.949051610031,1141.835408768608,1122.6235728648946,1103.313039448867,1083.9033014789902,1064.3938493088747,1044.7841706739205,1025.0737506778485,1005.2620717791957,985.3486137777143,965.3328538007127,945.2142662893416,924.9923229847765,904.6664929143512,884.2362423776243,863.7010349323566,843.0603313804295,822.3135897536864,801.4602652997019,780.499810467478,759.4316748930744,738.2553053851373,716.9701459103926,695.575637579054,674.0712186301097,652.4563244166224,630.7303873908565,608.8928370894062,586.9431001182184,564.880600137504,542.7047578466513,520.4149909689688,498.01071423643185,475.49133937430514,452.85627508567467,430.10492703595537,407.2366978372745,384.2509870327641,361.14719108082465,337.92470333925974,314.5829140493531,291.1212103198695,267.5389761109256,243.8355922178621,220.01043625494694,196.06288263906558,171.99230257324743,147.79806403021703,123.4795317357533,99.03606715202524,74.46702846081261,49.77177054667877,24.94964498000966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Total Mortgage Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Total Mortgage Payment","line":{"color":"#B6E880","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Total Mortgage Payment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,4881.490341822198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Utilities\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Utilities","line":{"color":"#FF97FF","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Utilities","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Downpayment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Downpayment","line":{"color":"#FECB52","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Downpayment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=PMI\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"PMI","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"PMI","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Sellers Fee\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Sellers Fee","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Sellers Fee","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1303471.2898079753],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Loan Payoff\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Loan Payoff","line":{"color":"#00cc96","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Loan Payoff","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9.313225746154785e-10],"yaxis":"y","type":"scattergl"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"},"type":"log"},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### House Assets
What money will the home bring in? This includes a tax credit for paying interest on a house (limited to 750K over the lifetime of the loan) and the sale of the house in the last month (shown on the log-scale).

In [5]:
render_plotly_html(px.line(pd.DataFrame({"Total Home Assets": df.total_home_assets}), log_y=True, markers=True))

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="93d57229-6222-4757-93d4-e212d4cb3104" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("93d57229-6222-4757-93d4-e212d4cb3104")) {                    Plotly.newPlot(                        "93d57229-6222-4757-93d4-e212d4cb3104",                        [{"hovertemplate":"variable=Total Home Assets\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Total Home Assets","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Total Home Assets","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.23772463909576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.384453055473166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.450793627964245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.43239109230866,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.3246288320427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.12261260071556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.82115321771224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.41474817270774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.89756206965929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.263405836864013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.50571462495149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.617524309729463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.591446511532995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.41964203712824,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0937926442650783,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21724521.49679959],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"},"type":"log"},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### Rent Liabilities
What will renting cost us? Mostly, this is the rent payment. But also includes some other expenses that do not occur when buying a home.

In [6]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Rent Payment": df.rent,
                "Renters Insurance": df.renters_insurance,
                "Security Deposit": df.security_deposit_cost,
                "Brokers Fee": df.brokers_fee_cost,
            }
        ),
        markers=True,
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="0299e3b2-4fe8-4190-899b-7f0e6783aaff" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("0299e3b2-4fe8-4190-899b-7f0e6783aaff")) {                    Plotly.newPlot(                        "0299e3b2-4fe8-4190-899b-7f0e6783aaff",                        [{"hovertemplate":"variable=Rent Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Rent Payment","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Rent Payment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3585.499999999997,3585.499999999997,3585.499999999997,3585.499999999997,3585.499999999997,3585.499999999997,3585.499999999997,3585.499999999997,3585.499999999997,3585.499999999997,3585.499999999997,3585.499999999997,3621.3549999999937,3621.3549999999937,3621.3549999999937,3621.3549999999937,3621.3549999999937,3621.3549999999937,3621.3549999999937,3621.3549999999937,3621.3549999999937,3621.3549999999937,3621.3549999999937,3621.3549999999937,3657.5685499999904,3657.5685499999904,3657.5685499999904,3657.5685499999904,3657.5685499999904,3657.5685499999904,3657.5685499999904,3657.5685499999904,3657.5685499999904,3657.5685499999904,3657.5685499999904,3657.5685499999904,3694.1442354999876,3694.1442354999876,3694.1442354999876,3694.1442354999876,3694.1442354999876,3694.1442354999876,3694.1442354999876,3694.1442354999876,3694.1442354999876,3694.1442354999876,3694.1442354999876,3694.1442354999876,3731.085677854984,3731.085677854984,3731.085677854984,3731.085677854984,3731.085677854984,3731.085677854984,3731.085677854984,3731.085677854984,3731.085677854984,3731.085677854984,3731.085677854984,3731.085677854984,3768.39653463353,3768.39653463353,3768.39653463353,3768.39653463353,3768.39653463353,3768.39653463353,3768.39653463353,3768.39653463353,3768.39653463353,3768.39653463353,3768.39653463353,3768.39653463353,3806.080499979862,3806.080499979862,3806.080499979862,3806.080499979862,3806.080499979862,3806.080499979862,3806.080499979862,3806.080499979862,3806.080499979862,3806.080499979862,3806.080499979862,3806.080499979862,3844.1413049796574,3844.1413049796574,3844.1413049796574,3844.1413049796574,3844.1413049796574,3844.1413049796574,3844.1413049796574,3844.1413049796574,3844.1413049796574,3844.1413049796574,3844.1413049796574,3844.1413049796574,3882.5827180294505,3882.5827180294505,3882.5827180294505,3882.5827180294505,3882.5827180294505,3882.5827180294505,3882.5827180294505,3882.5827180294505,3882.5827180294505,3882.5827180294505,3882.5827180294505,3882.5827180294505,3921.408545209742,3921.408545209742,3921.408545209742,3921.408545209742,3921.408545209742,3921.408545209742,3921.408545209742,3921.408545209742,3921.408545209742,3921.408545209742,3921.408545209742,3921.408545209742,3960.622630661836,3960.622630661836,3960.622630661836,3960.622630661836,3960.622630661836,3960.622630661836,3960.622630661836,3960.622630661836,3960.622630661836,3960.622630661836,3960.622630661836,3960.622630661836,4000.228856968451,4000.228856968451,4000.228856968451,4000.228856968451,4000.228856968451,4000.228856968451,4000.228856968451,4000.228856968451,4000.228856968451,4000.228856968451,4000.228856968451,4000.228856968451,4040.231145538132,4040.231145538132,4040.231145538132,4040.231145538132,4040.231145538132,4040.231145538132,4040.231145538132,4040.231145538132,4040.231145538132,4040.231145538132,4040.231145538132,4040.231145538132,4080.6334569935098,4080.6334569935098,4080.6334569935098,4080.6334569935098,4080.6334569935098,4080.6334569935098,4080.6334569935098,4080.6334569935098,4080.6334569935098,4080.6334569935098,4080.6334569935098,4080.6334569935098,4121.439791563441,4121.439791563441,4121.439791563441,4121.439791563441,4121.439791563441,4121.439791563441,4121.439791563441,4121.439791563441,4121.439791563441,4121.439791563441,4121.439791563441,4121.439791563441,4162.654189479072,4162.654189479072,4162.654189479072,4162.654189479072,4162.654189479072,4162.654189479072,4162.654189479072,4162.654189479072,4162.654189479072,4162.654189479072,4162.654189479072,4162.654189479072,4204.280731373859,4204.280731373859,4204.280731373859,4204.280731373859,4204.280731373859,4204.280731373859,4204.280731373859,4204.280731373859,4204.280731373859,4204.280731373859,4204.280731373859,4204.280731373859,4246.323538687594,4246.323538687594,4246.323538687594,4246.323538687594,4246.323538687594,4246.323538687594,4246.323538687594,4246.323538687594,4246.323538687594,4246.323538687594,4246.323538687594,4246.323538687594,4288.786774074466,4288.786774074466,4288.786774074466,4288.786774074466,4288.786774074466,4288.786774074466,4288.786774074466,4288.786774074466,4288.786774074466,4288.786774074466,4288.786774074466,4288.786774074466,4331.674641815206,4331.674641815206,4331.674641815206,4331.674641815206,4331.674641815206,4331.674641815206,4331.674641815206,4331.674641815206,4331.674641815206,4331.674641815206,4331.674641815206,4331.674641815206,4374.991388233355,4374.991388233355,4374.991388233355,4374.991388233355,4374.991388233355,4374.991388233355,4374.991388233355,4374.991388233355,4374.991388233355,4374.991388233355,4374.991388233355,4374.991388233355,4418.741302115685,4418.741302115685,4418.741302115685,4418.741302115685,4418.741302115685,4418.741302115685,4418.741302115685,4418.741302115685,4418.741302115685,4418.741302115685,4418.741302115685,4418.741302115685,4462.928715136838,4462.928715136838,4462.928715136838,4462.928715136838,4462.928715136838,4462.928715136838,4462.928715136838,4462.928715136838,4462.928715136838,4462.928715136838,4462.928715136838,4462.928715136838,4507.558002288202,4507.558002288202,4507.558002288202,4507.558002288202,4507.558002288202,4507.558002288202,4507.558002288202,4507.558002288202,4507.558002288202,4507.558002288202,4507.558002288202,4507.558002288202,4552.63358231108,4552.63358231108,4552.63358231108,4552.63358231108,4552.63358231108,4552.63358231108,4552.63358231108,4552.63358231108,4552.63358231108,4552.63358231108,4552.63358231108,4552.63358231108,4598.159918134187,4598.159918134187,4598.159918134187,4598.159918134187,4598.159918134187,4598.159918134187,4598.159918134187,4598.159918134187,4598.159918134187,4598.159918134187,4598.159918134187,4598.159918134187,4644.141517315526,4644.141517315526,4644.141517315526,4644.141517315526,4644.141517315526,4644.141517315526,4644.141517315526,4644.141517315526,4644.141517315526,4644.141517315526,4644.141517315526,4644.141517315526,4690.582932488676,4690.582932488676,4690.582932488676,4690.582932488676,4690.582932488676,4690.582932488676,4690.582932488676,4690.582932488676,4690.582932488676,4690.582932488676,4690.582932488676,4690.582932488676,4737.488761813559,4737.488761813559,4737.488761813559,4737.488761813559,4737.488761813559,4737.488761813559,4737.488761813559,4737.488761813559,4737.488761813559,4737.488761813559,4737.488761813559,4737.488761813559,4784.863649431691,4784.863649431691,4784.863649431691,4784.863649431691,4784.863649431691,4784.863649431691,4784.863649431691,4784.863649431691,4784.863649431691,4784.863649431691,4784.863649431691,4784.863649431691,4832.7122859260035,4832.7122859260035,4832.7122859260035,4832.7122859260035,4832.7122859260035,4832.7122859260035,4832.7122859260035,4832.7122859260035,4832.7122859260035,4832.7122859260035,4832.7122859260035,4832.7122859260035,4881.039408785258,4881.039408785258,4881.039408785258,4881.039408785258,4881.039408785258,4881.039408785258,4881.039408785258,4881.039408785258,4881.039408785258,4881.039408785258,4881.039408785258,4881.039408785258,4929.849802873107,4929.849802873107,4929.849802873107,4929.849802873107,4929.849802873107,4929.849802873107,4929.849802873107,4929.849802873107,4929.849802873107,4929.849802873107,4929.849802873107,4929.849802873107,4979.148300901833,4979.148300901833,4979.148300901833,4979.148300901833,4979.148300901833,4979.148300901833,4979.148300901833,4979.148300901833,4979.148300901833,4979.148300901833,4979.148300901833,4979.148300901833,5028.939783910848,5028.939783910848,5028.939783910848,5028.939783910848,5028.939783910848,5028.939783910848,5028.939783910848,5028.939783910848,5028.939783910848,5028.939783910848,5028.939783910848,5028.939783910848,5079.229181749952,5079.229181749952,5079.229181749952,5079.229181749952,5079.229181749952,5079.229181749952,5079.229181749952,5079.229181749952,5079.229181749952,5079.229181749952,5079.229181749952,5079.229181749952,5130.021473567446,5130.021473567446,5130.021473567446,5130.021473567446,5130.021473567446,5130.021473567446,5130.021473567446,5130.021473567446,5130.021473567446,5130.021473567446,5130.021473567446,5130.021473567446,5181.321688303117,5181.321688303117,5181.321688303117,5181.321688303117,5181.321688303117,5181.321688303117,5181.321688303117,5181.321688303117,5181.321688303117,5181.321688303117,5181.321688303117,5181.321688303117,5233.1349051861425,5233.1349051861425,5233.1349051861425,5233.1349051861425,5233.1349051861425,5233.1349051861425,5233.1349051861425,5233.1349051861425,5233.1349051861425,5233.1349051861425,5233.1349051861425,5233.1349051861425],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Renters Insurance\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Renters Insurance","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Renters Insurance","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.85499999999997,35.85499999999997,35.85499999999997,35.85499999999997,35.85499999999997,35.85499999999997,35.85499999999997,35.85499999999997,35.85499999999997,35.85499999999997,35.85499999999997,35.85499999999997,36.213549999999934,36.213549999999934,36.213549999999934,36.213549999999934,36.213549999999934,36.213549999999934,36.213549999999934,36.213549999999934,36.213549999999934,36.213549999999934,36.213549999999934,36.213549999999934,36.575685499999906,36.575685499999906,36.575685499999906,36.575685499999906,36.575685499999906,36.575685499999906,36.575685499999906,36.575685499999906,36.575685499999906,36.575685499999906,36.575685499999906,36.575685499999906,36.94144235499988,36.94144235499988,36.94144235499988,36.94144235499988,36.94144235499988,36.94144235499988,36.94144235499988,36.94144235499988,36.94144235499988,36.94144235499988,36.94144235499988,36.94144235499988,37.31085677854984,37.31085677854984,37.31085677854984,37.31085677854984,37.31085677854984,37.31085677854984,37.31085677854984,37.31085677854984,37.31085677854984,37.31085677854984,37.31085677854984,37.31085677854984,37.6839653463353,37.6839653463353,37.6839653463353,37.6839653463353,37.6839653463353,37.6839653463353,37.6839653463353,37.6839653463353,37.6839653463353,37.6839653463353,37.6839653463353,37.6839653463353,38.06080499979862,38.06080499979862,38.06080499979862,38.06080499979862,38.06080499979862,38.06080499979862,38.06080499979862,38.06080499979862,38.06080499979862,38.06080499979862,38.06080499979862,38.06080499979862,38.44141304979657,38.44141304979657,38.44141304979657,38.44141304979657,38.44141304979657,38.44141304979657,38.44141304979657,38.44141304979657,38.44141304979657,38.44141304979657,38.44141304979657,38.44141304979657,38.8258271802945,38.8258271802945,38.8258271802945,38.8258271802945,38.8258271802945,38.8258271802945,38.8258271802945,38.8258271802945,38.8258271802945,38.8258271802945,38.8258271802945,38.8258271802945,39.21408545209742,39.21408545209742,39.21408545209742,39.21408545209742,39.21408545209742,39.21408545209742,39.21408545209742,39.21408545209742,39.21408545209742,39.21408545209742,39.21408545209742,39.21408545209742,39.60622630661836,39.60622630661836,39.60622630661836,39.60622630661836,39.60622630661836,39.60622630661836,39.60622630661836,39.60622630661836,39.60622630661836,39.60622630661836,39.60622630661836,39.60622630661836,40.00228856968451,40.00228856968451,40.00228856968451,40.00228856968451,40.00228856968451,40.00228856968451,40.00228856968451,40.00228856968451,40.00228856968451,40.00228856968451,40.00228856968451,40.00228856968451,40.402311455381316,40.402311455381316,40.402311455381316,40.402311455381316,40.402311455381316,40.402311455381316,40.402311455381316,40.402311455381316,40.402311455381316,40.402311455381316,40.402311455381316,40.402311455381316,40.806334569935096,40.806334569935096,40.806334569935096,40.806334569935096,40.806334569935096,40.806334569935096,40.806334569935096,40.806334569935096,40.806334569935096,40.806334569935096,40.806334569935096,40.806334569935096,41.21439791563441,41.21439791563441,41.21439791563441,41.21439791563441,41.21439791563441,41.21439791563441,41.21439791563441,41.21439791563441,41.21439791563441,41.21439791563441,41.21439791563441,41.21439791563441,41.62654189479072,41.62654189479072,41.62654189479072,41.62654189479072,41.62654189479072,41.62654189479072,41.62654189479072,41.62654189479072,41.62654189479072,41.62654189479072,41.62654189479072,41.62654189479072,42.04280731373859,42.04280731373859,42.04280731373859,42.04280731373859,42.04280731373859,42.04280731373859,42.04280731373859,42.04280731373859,42.04280731373859,42.04280731373859,42.04280731373859,42.04280731373859,42.463235386875944,42.463235386875944,42.463235386875944,42.463235386875944,42.463235386875944,42.463235386875944,42.463235386875944,42.463235386875944,42.463235386875944,42.463235386875944,42.463235386875944,42.463235386875944,42.88786774074466,42.88786774074466,42.88786774074466,42.88786774074466,42.88786774074466,42.88786774074466,42.88786774074466,42.88786774074466,42.88786774074466,42.88786774074466,42.88786774074466,42.88786774074466,43.316746418152064,43.316746418152064,43.316746418152064,43.316746418152064,43.316746418152064,43.316746418152064,43.316746418152064,43.316746418152064,43.316746418152064,43.316746418152064,43.316746418152064,43.316746418152064,43.74991388233355,43.74991388233355,43.74991388233355,43.74991388233355,43.74991388233355,43.74991388233355,43.74991388233355,43.74991388233355,43.74991388233355,43.74991388233355,43.74991388233355,43.74991388233355,44.18741302115686,44.18741302115686,44.18741302115686,44.18741302115686,44.18741302115686,44.18741302115686,44.18741302115686,44.18741302115686,44.18741302115686,44.18741302115686,44.18741302115686,44.18741302115686,44.62928715136838,44.62928715136838,44.62928715136838,44.62928715136838,44.62928715136838,44.62928715136838,44.62928715136838,44.62928715136838,44.62928715136838,44.62928715136838,44.62928715136838,44.62928715136838,45.07558002288202,45.07558002288202,45.07558002288202,45.07558002288202,45.07558002288202,45.07558002288202,45.07558002288202,45.07558002288202,45.07558002288202,45.07558002288202,45.07558002288202,45.07558002288202,45.526335823110806,45.526335823110806,45.526335823110806,45.526335823110806,45.526335823110806,45.526335823110806,45.526335823110806,45.526335823110806,45.526335823110806,45.526335823110806,45.526335823110806,45.526335823110806,45.98159918134187,45.98159918134187,45.98159918134187,45.98159918134187,45.98159918134187,45.98159918134187,45.98159918134187,45.98159918134187,45.98159918134187,45.98159918134187,45.98159918134187,45.98159918134187,46.44141517315526,46.44141517315526,46.44141517315526,46.44141517315526,46.44141517315526,46.44141517315526,46.44141517315526,46.44141517315526,46.44141517315526,46.44141517315526,46.44141517315526,46.44141517315526,46.90582932488677,46.90582932488677,46.90582932488677,46.90582932488677,46.90582932488677,46.90582932488677,46.90582932488677,46.90582932488677,46.90582932488677,46.90582932488677,46.90582932488677,46.90582932488677,47.37488761813559,47.37488761813559,47.37488761813559,47.37488761813559,47.37488761813559,47.37488761813559,47.37488761813559,47.37488761813559,47.37488761813559,47.37488761813559,47.37488761813559,47.37488761813559,47.84863649431691,47.84863649431691,47.84863649431691,47.84863649431691,47.84863649431691,47.84863649431691,47.84863649431691,47.84863649431691,47.84863649431691,47.84863649431691,47.84863649431691,47.84863649431691,48.32712285926004,48.32712285926004,48.32712285926004,48.32712285926004,48.32712285926004,48.32712285926004,48.32712285926004,48.32712285926004,48.32712285926004,48.32712285926004,48.32712285926004,48.32712285926004,48.810394087852586,48.810394087852586,48.810394087852586,48.810394087852586,48.810394087852586,48.810394087852586,48.810394087852586,48.810394087852586,48.810394087852586,48.810394087852586,48.810394087852586,48.810394087852586,49.298498028731075,49.298498028731075,49.298498028731075,49.298498028731075,49.298498028731075,49.298498028731075,49.298498028731075,49.298498028731075,49.298498028731075,49.298498028731075,49.298498028731075,49.298498028731075,49.79148300901833,49.79148300901833,49.79148300901833,49.79148300901833,49.79148300901833,49.79148300901833,49.79148300901833,49.79148300901833,49.79148300901833,49.79148300901833,49.79148300901833,49.79148300901833,50.28939783910848,50.28939783910848,50.28939783910848,50.28939783910848,50.28939783910848,50.28939783910848,50.28939783910848,50.28939783910848,50.28939783910848,50.28939783910848,50.28939783910848,50.28939783910848,50.79229181749952,50.79229181749952,50.79229181749952,50.79229181749952,50.79229181749952,50.79229181749952,50.79229181749952,50.79229181749952,50.79229181749952,50.79229181749952,50.79229181749952,50.79229181749952,51.300214735674466,51.300214735674466,51.300214735674466,51.300214735674466,51.300214735674466,51.300214735674466,51.300214735674466,51.300214735674466,51.300214735674466,51.300214735674466,51.300214735674466,51.300214735674466,51.813216883031174,51.813216883031174,51.813216883031174,51.813216883031174,51.813216883031174,51.813216883031174,51.813216883031174,51.813216883031174,51.813216883031174,51.813216883031174,51.813216883031174,51.813216883031174,52.33134905186142,52.33134905186142,52.33134905186142,52.33134905186142,52.33134905186142,52.33134905186142,52.33134905186142,52.33134905186142,52.33134905186142,52.33134905186142,52.33134905186142,52.33134905186142],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Security Deposit\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Security Deposit","line":{"color":"#00cc96","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Security Deposit","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[3550.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Brokers Fee\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Brokers Fee","line":{"color":"#ab63fa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Brokers Fee","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### Rent Assets
The only positive cash flow in renting is getting back the security deposit in the last month.

In [7]:
render_plotly_html(
    px.line(
        pd.DataFrame({"Rent Assets": df.total_rent_assets}),
        markers=True,
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="9af44fd8-beda-4721-b5d8-fdf97430bf10" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("9af44fd8-beda-4721-b5d8-fdf97430bf10")) {                    Plotly.newPlot(                        "9af44fd8-beda-4721-b5d8-fdf97430bf10",                        [{"hovertemplate":"variable=Rent Assets\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Rent Assets","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Rent Assets","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3550.0],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### Opportunity Costs
If we didn't buy the house, how much could we earn in the market after we pay the rent?

```python
opportunity_cost = (total_home_liability - total_home_assets) - (total_rent_liability - total_rent_assets)
```

We take the future value of each month's cash flow (to the last month's period) and then apply a capital gains tax penalty for selling the asset at the end of the period. Then we compute:

```python
buy_vs_rent = rental_cumulative_opportunity - home_cumulative_opportunity
```

In the last month to see the total advantage of owning a home vs. renting.

In [8]:
df[
    [
        "home_opportunity_cost",
        "rental_opportunity_cost",
        "home_opportunity_cost_fv",
        "rental_opportunity_cost_fv",
        "home_opportunity_cost_fv_post_tax",
        "rental_opportunity_cost_fv_post_tax",
        "home_cumulative_opportunity",
        "rental_cumulative_opportunity",
        "buy_vs_rent",
    ]
]

home_opportunity_cost  rental_opportunity_cost  home_opportunity_cost_fv  \
0            209517.752792             0.000000e+00              4.522575e+06   
1              3067.752792             0.000000e+00              6.579607e+04   
2              3067.752792             0.000000e+00              6.537544e+04   
3              3067.752792             0.000000e+00              6.495750e+04   
4              3067.752792             0.000000e+00              6.454224e+04   
..                     ...                      ...                       ...   
475           24071.369332             0.000000e+00              2.469688e+04   
476           24071.369332             0.000000e+00              2.453899e+04   
477           24071.369332             0.000000e+00              2.438212e+04   
478           24071.369332             0.000000e+00              2.422625e+04   
479               0.000000             2.039343e+07             -0.000000e+00   

     rental_opportunity_cost_fv  home_opportunity_cost_fv_post_tax  \
0                 -0.000000e+00                       3.844188e+06   
1                 -0.000000e+00                       5.592666e+04   
2                 -0.000000e+00                       5.556913e+04   
3                 -0.000000e+00                       5.521388e+04   
4                 -0.000000e+00                       5.486090e+04   
..                          ...                                ...   
475               -0.000000e+00                       2.407137e+04   
476               -0.000000e+00                       2.407137e+04   
477               -0.000000e+00                       2.407137e+04   
478               -0.000000e+00                       2.407137e+04   
479                2.039343e+07                       0.000000e+00   

     rental_opportunity_cost_fv_post_tax  home_cumulative_opportunity  \
0                           0.000000e+00                 3.844188e+06   
1                           0.000000e+00                 3.900115e+06   
2                           0.000000e+00                 3.955684e+06   
3                           0.000000e+00                 4.010898e+06   
4                           0.000000e+00                 4.065759e+06   
..                                   ...                          ...   
475                         0.000000e+00                 1.869184e+07   
476                         0.000000e+00                 1.871591e+07   
477                         0.000000e+00                 1.873998e+07   
478                         0.000000e+00                 1.876405e+07   
479                         2.039343e+07                 1.876405e+07   

     rental_cumulative_opportunity   buy_vs_rent  
0                     0.000000e+00 -3.844188e+06  
1                     0.000000e+00 -3.900115e+06  
2                     0.000000e+00 -3.955684e+06  
3                     0.000000e+00 -4.010898e+06  
4                     0.000000e+00 -4.065759e+06  
..                             ...           ...  
475                   0.000000e+00 -1.869184e+07  
476                   0.000000e+00 -1.871591e+07  
477                   0.000000e+00 -1.873998e+07  
478                   0.000000e+00 -1.876405e+07  
479                   2.039343e+07  1.629379e+06  

[480 rows x 9 columns]

Giving a final opportunity of:

In [9]:
print(df.buy_vs_rent.iloc[-1])

1629379.2057856917


## Optimize

In addition to visualizing the cost curves, we can also use `scipy.optimize.minimize_scalar` to answer questions. If we only wanted to stay in the house for 5 years, what would be the optimal mortgage rate to breakeven on our investment?

In [10]:
optimize_kwargs = kwargs.copy()
optimize_kwargs["years"] = 5

In [11]:
minimize_scalar(
    fun=rent_vs_buy_breakeven_objective_closure(
        scalar="mortgage_rate", **optimize_kwargs
    ),
    bounds=[0.0001, 0.1],
)

 message: Solution found.
 success: True
  status: 0
     fun: 22922.953811162966
       x: 0.09999339591456094
     nit: 20
    nfev: 20

We can also maximize the house advantage by optimizing the downpayment for a certain amount of years:

In [12]:
optimization_per_year = {}
for i in range(1, 21):
    optimize_kwargs = kwargs.copy()
    optimize_kwargs["years"] = i
    optimization_per_year[i] = minimize_scalar(
        fun=rent_vs_buy_objective_closure(
            scalar="downpayment", maximize=True, **optimize_kwargs
        ),
        bounds=[0.0, 1.0],
    )
optimization_per_year_df = pd.DataFrame.from_dict(optimization_per_year).T
optimization_per_year_df.fun *= -1  # Need to reverse sign since we are minimizing a negative.
optimization_per_year_df

fun status success          message  \
1    [26964.48510717007]      0    True  Solution found.   
2   [128926.34276358329]      0    True  Solution found.   
3   [172551.85680377204]      0    True  Solution found.   
4   [202354.96091943118]      0    True  Solution found.   
5   [233335.62611138006]      0    True  Solution found.   
6   [265469.21249106294]      0    True  Solution found.   
7    [298719.4248776252]      0    True  Solution found.   
8   [333036.60481286934]      0    True  Solution found.   
9   [376206.82638946804]      0    True  Solution found.   
10  [433993.59103876434]      0    True  Solution found.   
11   [494044.5414490332]      0    True  Solution found.   
12   [556333.1830919643]      0    True  Solution found.   
13   [620814.8996148093]      0    True  Solution found.   
14   [687423.3580196153]      0    True  Solution found.   
15   [756067.1608767435]      0    True  Solution found.   
16   [826629.2744868086]      0    True  Solution found.   
17   [898961.5180192709]      0    True  Solution found.   
18   [972878.1042906116]      0    True  Solution found.   
19  [1048149.1716485564]      0    True  Solution found.   
20  [1124504.6932380316]      0    True  Solution found.   

                        x nfev nit  
1    [0.9999940391390134]   25  25  
2    [0.9999940391390134]   25  25  
3    [0.9999940391390134]   25  25  
4    [0.9999940391390134]   25  25  
5    [0.9999940391390134]   25  25  
6    [0.9999940391390134]   25  25  
7    [0.9999940391390134]   25  25  
8    [0.9999940391390134]   25  25  
9   [0.19922839055952876]   25  25  
10  [0.19922839055952876]   25  25  
11  [0.19922839055952876]   25  25  
12  [0.19923026553936787]   26  26  
13  [0.19923038553560887]   26  26  
14  [0.19923010355766244]   25  25  
15  [0.19923271224139022]   26  26  
16   [0.1992311375063031]   25  25  
17  [0.19923041437219008]   26  26  
18   [0.1992279115320856]   26  26  
19   [0.1992306541336122]   26  26  
20  [0.19922875344144916]   26  26

## Vectorize
Since `numpy-financial` is a vectorized library, we can actually see the breakdown for a range of values. For example, we can visualize a 2-d surface of the total opportunity.
### `home_price` and `downpayment`

In [13]:
_x, _y = np.linspace(100_000, 2_000_000, 100), np.linspace(0.00, 1.0, 100)
x, y = np.meshgrid(_x, _y, indexing="ij")
x = x.flatten()
y = y.flatten()
surface_kwargs = kwargs.copy()
surface_kwargs.update({"home_price": x, "downpayment": y})
render_plotly_html(
    px.imshow(
        RentVsBuy().calculate(**surface_kwargs).value.reshape((100, 100)).T,
        aspect="auto",
        labels=dict(x="Home Price", y="Downpayment", color="Opportunity"),
        x=_x,
        y=_y,
        title="Opportunity Surface",
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="e4108a79-6586-4af7-9ebc-33f88677f840" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("e4108a79-6586-4af7-9ebc-33f88677f840")) {                    Plotly.newPlot(                        "e4108a79-6586-4af7-9ebc-33f88677f840",                        [{"coloraxis":"coloraxis","name":"0","x":[100000.0,119191.9191919192,138383.8383838384,157575.75757575757,176767.67676767678,195959.59595959596,215151.51515151514,234343.43434343435,253535.35353535353,272727.2727272727,291919.1919191919,311111.1111111111,330303.0303030303,349494.9494949495,368686.8686868687,387878.78787878784,407070.70707070705,426262.62626262626,445454.5454545454,464646.4646464646,483838.38383838383,503030.303030303,522222.2222222222,541414.1414141414,560606.0606060605,579797.9797979798,598989.898989899,618181.8181818181,637373.7373737374,656565.6565656565,675757.5757575757,694949.494949495,714141.4141414141,733333.3333333333,752525.2525252525,771717.1717171717,790909.0909090908,810101.0101010101,829292.9292929292,848484.8484848484,867676.7676767677,886868.6868686868,906060.606060606,925252.5252525252,944444.4444444444,963636.3636363635,982828.2828282828,1002020.202020202,1021212.1212121211,1040404.0404040404,1059595.9595959596,1078787.878787879,1097979.797979798,1117171.717171717,1136363.6363636362,1155555.5555555555,1174747.4747474748,1193939.3939393938,1213131.313131313,1232323.2323232323,1251515.1515151514,1270707.0707070706,1289898.98989899,1309090.909090909,1328282.8282828282,1347474.7474747475,1366666.6666666665,1385858.5858585858,1405050.505050505,1424242.424242424,1443434.3434343433,1462626.2626262626,1481818.1818181816,1501010.101010101,1520202.0202020202,1539393.9393939392,1558585.8585858585,1577777.7777777778,1596969.6969696968,1616161.616161616,1635353.5353535353,1654545.4545454544,1673737.3737373736,1692929.292929293,1712121.212121212,1731313.1313131312,1750505.0505050505,1769696.9696969695,1788888.8888888888,1808080.808080808,1827272.727272727,1846464.6464646463,1865656.5656565656,1884848.4848484846,1904040.404040404,1923232.3232323232,1942424.2424242422,1961616.1616161615,1980808.0808080807,2000000.0],"y":[0.0,0.010101010101010102,0.020202020202020204,0.030303030303030304,0.04040404040404041,0.05050505050505051,0.06060606060606061,0.07070707070707072,0.08080808080808081,0.09090909090909091,0.10101010101010102,0.11111111111111112,0.12121212121212122,0.13131313131313133,0.14141414141414144,0.15151515151515152,0.16161616161616163,0.17171717171717174,0.18181818181818182,0.19191919191919193,0.20202020202020204,0.21212121212121213,0.22222222222222224,0.23232323232323235,0.24242424242424243,0.25252525252525254,0.26262626262626265,0.27272727272727276,0.2828282828282829,0.29292929292929293,0.30303030303030304,0.31313131313131315,0.32323232323232326,0.33333333333333337,0.3434343434343435,0.3535353535353536,0.36363636363636365,0.37373737373737376,0.38383838383838387,0.393939393939394,0.4040404040404041,0.4141414141414142,0.42424242424242425,0.43434343434343436,0.4444444444444445,0.4545454545454546,0.4646464646464647,0.4747474747474748,0.48484848484848486,0.494949494949495,0.5050505050505051,0.5151515151515152,0.5252525252525253,0.5353535353535354,0.5454545454545455,0.5555555555555556,0.5656565656565657,0.5757575757575758,0.5858585858585859,0.595959595959596,0.6060606060606061,0.6161616161616162,0.6262626262626263,0.6363636363636365,0.6464646464646465,0.6565656565656566,0.6666666666666667,0.6767676767676768,0.686868686868687,0.696969696969697,0.7070707070707072,0.7171717171717172,0.7272727272727273,0.7373737373737375,0.7474747474747475,0.7575757575757577,0.7676767676767677,0.7777777777777778,0.787878787878788,0.797979797979798,0.8080808080808082,0.8181818181818182,0.8282828282828284,0.8383838383838385,0.8484848484848485,0.8585858585858587,0.8686868686868687,0.8787878787878789,0.888888888888889,0.8989898989898991,0.9090909090909092,0.9191919191919192,0.9292929292929294,0.9393939393939394,0.9494949494949496,0.9595959595959597,0.9696969696969697,0.9797979797979799,0.98989898989899,1.0],"z":[[8982743.083833225,8821657.73648692,8660572.389140613,8499487.04179431,8338401.694447994,8177316.3471016865,8016230.999755379,7855145.652409075,7694060.305062762,7532974.957716456,7371889.610370147,7210804.263023838,7049718.915677532,6888633.568331222,6727548.220984913,6566462.873638606,6405377.526292296,6244298.854424784,6083261.347282397,5922259.461006481,5761288.758838667,5600345.003787599,5439421.909001392,5278516.551994216,5117628.199450839,4956752.678176095,4795887.519061738,4635030.537332745,4474185.372110222,4313346.215021225,4152512.625974482,3991687.2403783295,3830864.050370332,3670048.866102362,3509234.990098452,3348426.8712251876,3187621.309395451,3026817.5100356694,2866019.341369208,2705221.172702793,2544423.004036369,2383593.5002466813,2222787.93841693,2061910.1060631834,1901096.2300592363,1740282.3540553227,1579339.3902581707,1418516.2002501264,1257693.010242153,1096674.434086468,935840.8450397737,775007.2559929676,614173.6669462323,453056.24154673517,292211.0763241686,131365.91110163927,-29479.254120934755,-190324.41934341565,-351569.0597762838,-512427.07786496356,-673285.0959536396,-834143.1140423678,-995001.1321310773,-1155859.150219757,-1317265.2764423713,-1478137.5316889808,-1639009.7869355977,-1799882.0421822928,-1960754.2974288315,-2121626.5526755154,-2283219.4781418554,-2444107.47113299,-2604995.464124121,-2765883.4571152404,-2926771.4501063935,-3087659.4430975765,-3248547.4360886216,-3409435.4290798493,-3570323.4220710024,-3732183.440406952,-3893088.7974141575,-4053994.15442124,-4214899.51142849,-4375804.868435606,-4536710.225442827,-4697615.582450002,-4858520.939457133,-5019426.296464361,-5180331.65347144,-5341237.010478668,-5503444.331651568,-5664368.814773813,-5825293.297896124,-5986217.781018451,-6147142.264140815,-6308066.747263163,-6468991.2303854525,-6629915.713507704,-6790840.196629994,-6951764.679752365],[8982105.649022112,8820897.965701958,8659690.282381807,8498482.599061647,8337274.915741487,8176067.232421327,8014859.549101165,7853651.865781004,7692444.182460847,7531236.499140693,7370028.815820534,7208821.132500375,7047613.449180218,6886405.765860058,6725198.082539898,6563990.399219741,6402782.715899583,6241575.032579427,6080406.949284284,5919275.294337327,5758174.357038335,5597099.730279005,5436047.480286033,5275014.100671436,5113996.470715963,4952991.817508777,4791997.681599437,4631015.031281763,4470041.95856926,4309074.039150951,4148115.5711172335,3987161.502951624,3826213.328915857,3665269.5657413453,3504330.6116288137,3343394.037704302,3182463.313784389,3021532.5898644514,2860607.794684075,2699684.389249064,2538760.9838140253,2377809.694184715,2216878.970264815,2055948.246344868,1894941.0705033801,1734002.1163908541,1573063.1622783095,1411990.7183790691,1251042.5443433262,1090094.3703075089,928946.210443005,767987.7424092069,607029.274375502,446070.8063418418,284823.8493295945,123853.92205113918,-37116.0052273348,-198085.9325057678,-359055.8597842045,-520429.7788835168,-681412.4292011745,-842395.0795188658,-1003377.7298365571,-1164360.3801542073,-1325881.1854624487,-1486877.92912836,-1647874.6727942675,-1808871.4164602347,-1969868.160126146,-2130864.903792076,-2291861.647458043,-2453582.684120532,-2614595.006563902,-2775607.3290072083,-2936619.651450541,-3097631.9738938734,-3258644.296337139,-3419656.6187805273,-3580668.9412238523,-3741681.263667185,-3903668.0300477035,-4064697.5411129296,-4225727.0521782115,-4386756.5632434115,-4547786.074308716,-4708815.58537396,-4869845.096439183,-5030874.607504517,-5191904.11856968,-5352933.629634932,-5513963.140700147,-5676295.154838242,-5837343.59882547,-5998392.042812869,-6159440.486800209,-6320488.930787504,-6481537.374774799,-6642585.818762071,-6803634.262749411,-6964682.7067368105],[8981474.829471514,8820146.079772955,8658817.330074396,8497488.580375852,8336159.8306772895,8174831.080978733,8013502.331280176,7852173.581581619,7690844.831883059,7529516.082184506,7368187.332485949,7206858.582787393,7045529.833088838,6884201.083390279,6722872.333691722,6561543.583993166,6400214.834294606,6238886.084596051,6077582.019264827,5916321.865242271,5755092.485249929,5593889.496604011,5432708.986267234,5271547.465304547,5110401.827532426,4949269.31198827,4788149.349498492,4627041.455928197,4465940.708961463,4304847.289222559,4143763.2193840574,3982681.738244444,3821609.8497763574,3660538.7772907484,3499476.0146650467,3338413.252039328,3177357.6388701126,3016303.0224554557,2855251.1078866236,2694203.735278463,2533156.3626702633,2372108.990062084,2211029.9403821677,2049975.323967494,1888848.3884079233,1727785.6257821992,1566722.8631565198,1405531.522754766,1244459.6342866868,1083387.7458185405,922315.8573505171,761039.5934539959,599957.516028773,438875.43860355765,277511.79574990645,116418.37601105124,-44675.04372789711,-205768.46346678957,-366861.8832057193,-528350.958509896,-689456.9714610949,-850562.9844123013,-1011668.9973635338,-1172775.0103146993,-1333881.0232658833,-1495529.1876028664,-1656649.1500926167,-1817769.1125824153,-1978889.0750721693,-2140009.037561983,-2301129.0000517443,-2462248.9625415243,-2624096.5228706338,-2785231.9051706344,-2946367.2874707393,-3107502.6697707847,-3268638.0520708077,-3429773.4343709014,-3590908.8166709766,-3752044.198971018,-3913179.581271138,-4075291.475312412,-4236443.8708404,-4397596.266368225,-4558748.661896169,-4719901.057424024,-4881053.452951856,-5042205.848479807,-5203358.244007677,-5364510.639535509,-5525663.035063423,-5686815.430591263,-5849270.481712751,-6010441.616969638,-6171612.752226613,-6332783.887483448,-6493955.02274029,-6655126.157997064,-6816297.293253943,-6977468.42851086],[8981176.661769371,8819790.687966365,8658404.714163356,8497018.74036037,8335632.766557347,8174246.792754338,8012860.818951332,7851474.845148327,7690088.871345316,7528702.897542311,7367316.923739302,7205930.949936295,7044544.976133289,6883159.002330281,6721773.028527271,6560387.054724266,6399001.080921256,6237615.107118254,6076245.666538926,5914914.620193818,5753620.110788936,5592352.602530385,5431107.674123737,5269881.854086924,5108672.050772115,4947477.450889263,4786296.176125229,4625124.2738196775,4463959.69487267,4302806.624947252,4141658.7877177615,3980516.9431749526,3819381.975251628,3658250.214625152,3497124.706863964,3336001.977999015,3174882.93766639,3013768.271030953,2852653.6043954976,2691545.5818402786,2530438.0843328964,2369330.5868255123,2208194.9378537703,2047080.2712183334,1885965.6045828946,1724774.6893496625,1563651.9604847543,1402529.2316198014,1241273.805227995,1080142.0446015038,919010.2839750685,757680.0523185916,596538.2077757493,435396.36323298514,274254.5186901316,112826.80688652024,-48326.26303894445,-209479.332964357,-370632.40288972855,-531785.4728151597,-693338.325435698,-854503.8587464988,-1015669.3920572959,-1176834.9253680632,-1338000.4586788155,-1499165.991989579,-1660877.3854619488,-1822056.7245016769,-1983236.0635413267,-2144415.4025810286,-2305594.741620686,-2466774.0806603953,-2627953.4197000824,-2789863.3434874564,-2951057.943370361,-3112252.5432531983,-3273447.1431359723,-3434641.7430188917,-3595836.342901744,-3757030.942784559,-3918225.54266743,-4079420.142550215,-4241592.9711904,-4402804.408906937,-4564015.846623518,-4725227.284340113,-4886438.722056627,-5047650.159773238,-5208861.597489774,-5370073.03520637,-5531284.4729229435,-5692495.91063945,-5853707.348355956,-6016221.207800917,-6177451.192053452,-6338681.176305883,-6499911.16055835,-6661141.144810677,-6822371.129063204,-6983601.113315769],[8980896.874007605,8819457.203563457,8658017.5331193,8496577.862675153,8335138.192230992,8173698.521786829,8012258.851342681,7850819.180898528,7689379.51045437,7527939.8400102155,7366500.169566062,7205060.499121906,7043620.828677752,6882181.158233596,6720741.48778944,6559301.817345286,6397862.14690113,6236422.4764569765,6074991.548438132,5913599.037967848,5752239.741459319,5590909.005589301,5429602.674243246,5268317.040328517,5107048.802023713,4945795.023069122,4784553.096738227,4623320.713160712,4462099.186953684,4300886.637048103,4139678.55989095,3978480.475693859,3817285.359492991,3656097.2541714497,3494911.7127555534,3333732.545114694,3172553.605081985,3011382.4156890716,2850211.2262961287,2689043.379411308,2527879.28446801,2366715.1895247716,2205551.0945815034,2044355.2793315947,1883184.0899386518,1721940.8687061518,1560761.7010653242,1399582.533424452,1238275.6743482389,1077087.5690266676,915899.4637052082,754711.3583835922,593320.7075549029,432122.6233578548,270924.53916069865,109726.45496366173,-51761.564464662224,-212970.7571132481,-374179.949761875,-535389.1424104832,-696598.3350591138,-858211.1778788157,-1019432.7040858716,-1180654.2302929126,-1341875.7564999312,-1503097.2827069536,-1664854.4283129051,-1826089.6164392084,-1987324.8045654558,-2148559.99269177,-2309795.1808180697,-2471030.3689443804,-2632265.557070639,-2793500.7451968566,-2955469.1870451234,-3116719.477047451,-3277969.7670497335,-3439220.0570521243,-3600470.347054459,-3761720.6370568164,-3922970.927059099,-4084221.2170614004,-4245471.507063776,-4407701.392051078,-4568968.344493076,-4730235.296935014,-4891502.24937693,-5052769.201818913,-5214036.154260837,-5375303.10670279,-5536570.05914475,-5697837.011586599,-5859103.964028522,-6020370.916470595,-6182939.670389414,-6344224.976174198,-6505510.281958908,-6666795.587743625,-6828080.893528417,-6989366.199313208],[8980635.500709036,8819145.667712634,8657655.834716214,8496166.001719827,8334676.168723411,8173186.335727004,8011696.502730602,7850206.669734198,7688716.836737788,7527227.003741383,7365737.170744978,7204247.337748571,7042757.504752169,6881267.671755761,6719777.838759354,6558288.00576295,6396798.172766539,6235308.339770138,6073819.458400665,5912369.017894107,5750951.845676618,5589563.314965735,5428199.292200697,5266856.089293586,5105530.4202737585,4944219.361934572,4782920.318125727,4621632.815001736,4460357.002350746,4299087.552190155,4137825.77090721,3976571.9794317447,3815321.189669287,3654079.3331145067,3492838.417186806,3331605.2198049463,3170373.147477079,3009145.736099541,2847921.5580380373,2686697.4209124744,2525480.262622187,2364263.1043319236,2203045.9460416716,2041800.6677305773,1880576.4896690696,1719352.311607577,1558052.4241983145,1396820.351870425,1235588.2795425914,1074224.678270962,912983.7623433098,751742.846415516,590305.3944198489,429054.6046574302,267803.8148948997,106553.02513248846,-54980.542448967695,-216242.32373185828,-377504.1050147787,-538765.8862976953,-700027.6675806344,-861684.5031252466,-1022958.4881396927,-1184232.4731541164,-1345506.4581684694,-1506780.4431829192,-1668054.4281973317,-1829867.3099575303,-1991154.8130814843,-2152442.3162054867,-2313729.819329459,-2475017.322453443,-2636304.8255774565,-2797592.3287013844,-2959600.49367838,-3120902.9397113174,-3282205.385744162,-3443507.831777137,-3604810.2778100893,-3766112.7238429673,-3927415.169875905,-4088717.615908712,-4250020.061941765,-4411322.50797461,-4573605.564432144,-4734924.497510508,-4896243.430588931,-5057562.363667458,-5218881.296745807,-5380200.229824193,-5541519.162902683,-5702838.095981084,-5864157.029059485,-6025475.962137967,-6186794.895216413,-6349414.623134114,-6510751.716362238,-6672088.80959031,-6833425.902818553,-6994762.996046737],[8980392.559128828,8818856.100980662,8657319.642832495,8495783.184684344,8334246.7265361715,8172710.268388005,8011173.81023985,7849637.352091684,7688100.893943519,7526564.435795359,7365027.977647196,7203491.51949903,7041955.061350869,6880418.6032027025,6718882.14505454,6557345.686906377,6395809.228758212,6234272.77061005,6072736.31246189,5911224.640148106,5749753.129622138,5588310.341470896,5426892.164687374,5265494.930188362,5104115.367854011,4942750.56753073,4781399.502867712,4620061.6629232215,4458732.2966799885,4297409.483664883,4136098.6511481274,3974791.574303312,3813491.6163759585,3652196.577992223,3490906.388858827,3329620.1352308225,3168338.6956164315,3007058.3720466346,2845784.742717022,2684511.1133874785,2523241.168514481,2361974.484277703,2200707.8000409584,2039441.1158041991,1878142.9667395614,1716869.3374099806,1555524.2994723767,1394242.8598579727,1232961.42024358,1071553.551857803,910263.3627244905,748973.1735909767,587682.9844575785,426192.4999015108,264892.5419741012,103592.58404682577,-57707.37388059497,-219293.8268039748,-380604.6593207158,-541915.4918374531,-703226.3243542053,-864537.1568709873,-1026246.521644529,-1187569.4280658476,-1348892.3344871104,-1510215.2409084253,-1671538.147329688,-1832861.0537510328,-1994725.8466455564,-2156062.1273668036,-2317398.408087969,-2478734.688809175,-2640070.969530359,-2801407.2502514943,-2962743.530972723,-3124802.665619783,-3286153.7302827463,-3447504.7949457876,-3608855.8596088067,-3770206.9242718033,-3931557.9889348373,-4092909.0535977483,-4254260.118260846,-4415611.182923824,-4577927.211011313,-4739294.587325722,-4900661.963640168,-5062029.339954615,-5223396.716268994,-5384764.092583418,-5546131.468897849,-5707498.845212258,-5868866.221526608,-6030233.597841114,-6191600.9741555825,-6352968.350469984,-6515635.128599837,-6677020.471870802,-6838405.815141916,-6999791.158413082],[8980168.048092421,8818588.501967583,8657008.955842735,8495429.409717897,8333849.8635930335,8172270.31746819,8010690.771343351,7849111.225218502,7687531.679093654,7525952.132968807,7364372.586843963,7202793.040719117,7041213.49459427,6879633.948469423,6718054.402344577,6556474.85621973,6394895.310094882,6233315.76397004,6071736.217845196,5910172.922284672,5748645.474620606,5587150.079196428,5425681.285569492,5264233.556653662,5102803.638179862,4941391.355804529,4779992.486929522,4618604.445115531,4457225.062455123,4295856.771801993,4134496.07718507,3973139.7306096014,3811793.6632288042,3650448.9801912103,3489113.0550272,3327777.2823281214,3166450.0126022734,3005122.742876414,2843800.770592682,2682481.227170093,2521161.99195355,2359849.3189453483,2198536.645937154,2037223.9729289748,1875883.5100571327,1714563.9666345268,1553244.4232119583,1391850.0456177928,1230522.7758919783,1069195.5061660856,907738.252403155,746402.327239085,585066.4020750634,423730.4769110903,262190.70705115795,100845.11813404039,-60500.470783151686,-221846.05970029533,-383481.62692851946,-544837.9735040143,-706194.320079498,-867550.6666550077,-1028907.01323051,-1190665.1104040444,-1352033.4010571688,-1513401.691710256,-1674769.9823633656,-1836138.273016557,-1997506.5636696033,-2159419.4429041147,-2320800.9640473984,-2482182.4851907417,-2643564.0063341,-2804945.5274773836,-2966327.0486207493,-3127708.5697640777,-3289814.818971839,-3451210.9650898725,-3612607.111207895,-3774003.2573259287,-3935399.4034439884,-4096795.549561955,-4258191.6956800595,-4419587.84179806,-4580983.987916119,-4743345.586715825,-4904757.869091123,-5066170.151466534,-5227582.433841832,-5388994.716217227,-5550406.998592548,-5711819.280967757,-5873231.563343108,-6034643.845718496,-6196056.128093898,-6357468.410469279,-6518880.69284454,-6681590.5976252705,-6843020.653764196,-7004450.709903203],[8980327.269650966,8818778.281198965,8657229.29274697,8495680.304294994,8334131.315842988,8172582.327390994,8011033.338938999,7849484.350487,7687935.3620350035,7526386.373583011,7364837.385131014,7203288.39667902,7041739.408227028,6880190.41977503,6718641.431323035,6557092.442871041,6395543.454419043,6233994.465967048,6072445.477515054,5910904.613171024,5749399.648729058,5587925.703401776,5426478.413972662,5265053.8847794905,5103648.642295991,4942259.59390276,4780883.990474162,4619519.392440199,4458166.276524277,4296823.609776745,4135486.605302993,3974158.3903183043,3812836.1034617107,3651518.529204268,3490207.4058746938,3328899.3903295975,3167596.305566283,3006296.8513785023,2844999.0657455698,2683707.253879506,2522415.4420134127,2361127.3578388356,2199842.341708746,2038557.3255786672,1877272.309448555,1715956.3826830834,1554664.570817031,1393372.7589509413,1232002.8553129397,1070703.4011251442,909403.9469374232,747971.1416083723,586663.1260632463,425355.110518232,263849.46979798377,102531.89554059505,-58785.678716856986,-220103.25297430158,-381703.4142684154,-543031.6292530783,-704359.8442377932,-865688.0592224933,-1027016.2742071822,-1188737.3842267022,-1350077.4134620503,-1511417.442697458,-1672757.471932821,-1834097.5011682883,-1995437.5304036103,-2156777.559639048,-2318664.5919445753,-2480017.7078604773,-2641370.823776424,-2802723.939692322,-2964077.0556082986,-3125430.1715242118,-3286783.287440099,-3448862.6876249425,-3610230.2695484534,-3771597.851471979,-3932965.4333954677,-4094333.0153189376,-4255700.597242482,-4417068.179165974,-4578435.761089578,-4739803.343013026,-4902136.516087212,-5063520.058873974,-5224903.601660676,-5386287.144447438,-5547670.687234171,-5709054.230020836,-5870437.772807494,-6031821.3155943155,-6193204.858381093,-6354588.401167758,-6515971.943954453,-6677355.4867411405,-6838739.029527873,-7001435.609494522],[8980522.778283255,8819011.311690003,8657499.845096735,8495988.378503466,8334476.911910182,8172965.4453169145,8011453.978723648,7849942.512130377,7688431.045537103,7526919.578943837,7365408.112350561,7203896.645757295,7042385.179164026,6880873.712570754,6719362.245977482,6557850.779384211,6396339.3127909405,6234827.846197673,6073316.379604401,5911804.913011126,5750329.393628751,5588884.976543959,5427467.320711297,5266072.55115157,5104697.21161195,4943338.223721589,4781992.849925272,4620660.796382138,4459340.913354561,4298028.696215592,4136723.892583793,3975429.226862857,3814137.684716312,3652855.0893044733,3491574.8261156306,3330301.6843753513,3169029.5962955263,3007764.9218316805,2846500.247367846,2685241.170911733,2523984.054788055,2362727.215426892,2201476.8203607835,2040226.4252946973,1878976.0302285515,1717698.4741696864,1556441.3580460325,1395184.2419223674,1233853.5027296059,1072588.82826576,911324.1538019851,749931.4165309332,588658.2747906186,427385.1330504194,266111.991310101,104637.99024397135,-56644.605167936534,-217927.20057974383,-379209.795991607,-540778.1109618843,-702071.230169896,-863364.3493779674,-1024657.4685860202,-1185950.5877940133,-1347639.4308973178,-1508944.234529104,-1670249.0381608717,-1831553.8417927101,-1992858.6454244703,-2154163.449056331,-2316004.439756781,-2477322.186259445,-2638639.9327621013,-2799957.679264765,-2961275.4257673956,-3122593.172270052,-3283910.918772675,-3445228.665275339,-3607273.934317097,-3768605.987860199,-3929938.0414033122,-4091270.0949463695,-4252602.148489587,-4413934.202032648,-4575266.255575806,-4736598.309118934,-4897930.362662017,-5060227.795036741,-5221575.634048916,-5382923.4730611965,-5544271.312073395,-5705619.15108557,-5866966.990097821,-6028314.829110101,-6189662.668122351,-6351010.507134631,-6512358.34614677,-6673706.185158975,-6835054.024171226,-6996401.863183513],[8980755.031086244,8819288.138263237,8657821.245440232,8496354.352617241,8334887.459794224,8173420.566971221,8011953.674148223,7850486.781325224,7689019.888502216,7527552.995679216,7366086.102856208,7204619.210033205,7043152.317210205,6881685.424387201,6720218.531564198,6558751.638741193,6397284.745918192,6235817.85309519,6074350.960272189,5912884.067449177,5751438.919381656,5590029.083999208,5428648.113674545,5267290.155659806,5105951.770975589,4944629.895499652,4783324.812173467,4622032.329995954,4460749.747878715,4299475.032259861,4138212.506143192,3976955.416835904,3815704.4453161005,3654461.106656272,3493220.5420015715,3331987.539495757,3170756.3234718163,3009530.6571826246,2848307.814354267,2687086.856664812,2525871.488195069,2364656.1197253205,2203444.2234682795,2042235.5013776831,1881026.779287111,1719818.0571964905,1558579.277376648,1397363.908906918,1236148.5404372215,1074856.5432426184,913633.7004143596,752410.8575859405,591056.8670889363,429825.65106504783,268594.4350410849,107168.68425282463,-54071.88040189445,-215312.44505657628,-376553.00971129537,-537793.5743659772,-699322.7572387904,-860573.7287586704,-1021824.7002784722,-1183075.6717982814,-1344326.643318057,-1505975.9079397954,-1667238.4340564385,-1828500.9601731673,-1989763.486289762,-2151026.012406487,-2312288.538523119,-2474089.2347681858,-2635364.559946064,-2796639.8851238266,-2957915.210301716,-3119190.535479568,-3280465.860657364,-3441741.185835179,-3603016.511013042,-3765020.2790658996,-3926309.7523170933,-4087599.2255682275,-4248888.698819473,-4410178.172070615,-4571467.645321816,-4732757.118573055,-4894046.591824196,-5055336.065075457,-5217590.354050733,-5378895.437376998,-5540200.520703137,-5701505.60402929,-5862810.687355496,-6024115.770681798,-6185420.85400793,-6346725.937334143,-6508031.020660341,-6669336.103986494,-6830641.18731273,-6991946.27063895],[8981423.52599597,8820084.93017584,8658746.33435572,8497407.738535607,8336069.142715472,8174730.546895342,8013391.951075222,7852053.355255096,7690714.759434969,7529376.163614846,7368037.567794721,7206698.971974596,7045360.376154472,6884021.780334343,6722683.184514221,6561344.588694095,6400005.992873971,6238667.397053843,6077328.80123372,5915990.205413591,5754664.892234608,5593373.110144305,5432110.257037601,5270872.221135725,5109655.333450624,4948456.324629871,4787272.2857921915,4626100.632997028,4464939.075022725,4303789.8944023065,4142649.0587876644,3981513.2691266257,3820388.1685276423,3659266.2327909525,3498151.4221260175,3337039.949864272,3175934.3827893212,3014830.394246891,2853733.106286645,2692635.8183264285,2531544.09301324,2370454.195430046,2209364.2978469077,2048280.9185697287,1887197.5926872417,1726114.2668047957,1565030.9409223571,1403914.809930969,1242824.9123478085,1081662.9387241676,920565.6507640183,759468.3628037423,598245.3095896952,437139.742514804,276034.1754397452,114928.60836486146,-46374.412502963096,-207489.22316788137,-368604.03383285925,-529718.8444977924,-690833.6551627293,-852239.748843316,-1013364.849442333,-1174489.9500413276,-1335615.05064027,-1496740.151239276,-1658265.8544929698,-1819402.3798619024,-1980538.9052307978,-2141675.430599738,-2302811.9559686296,-2463948.4813375436,-2625624.872275319,-2786774.052895665,-2947923.2335161306,-3109072.414136503,-3270221.5947568975,-3431370.775377307,-3592519.9559977204,-3753669.136618141,-3915547.362515282,-4076710.532241866,-4237873.701968595,-4399036.871695131,-4560200.041421831,-4721363.211148471,-4882526.380875118,-5043689.550601818,-5204852.720328383,-5366015.89005512,-5528158.396291234,-5689337.000698768,-5850515.605106227,-6011694.209513828,-6172872.813921414,-6334051.418329023,-6495230.022736527,-6656408.627143972,-6817587.2315516025,-6978765.835959181],[8982154.140356928,8820955.763454553,8659757.386552183,8498559.00964982,8337360.632747449,8176162.255845078,8014963.878942713,7853765.502040343,7692567.1251379745,7531368.748235609,7370170.371333241,7208971.99443087,7047773.617528504,6886575.240626137,6725376.863723768,6564178.4868214,6402980.109919038,6241781.733016666,6080583.356114301,5919384.979211925,5758191.4228363605,5597031.478246344,5435900.56998165,5274794.608761602,5113709.944410435,4952643.3231178485,4791591.848649014,4630552.947151058,4469528.002863834,4308513.221086467,4147505.3883297443,3986506.785623463,3825515.5124665517,3664528.204703266,3503549.766807573,3342572.6320472434,3181603.750493681,3020635.7542825546,2859673.550435651,2698713.739258442,2537755.693877831,2376803.1890960746,2215850.6843143236,2054901.0975527987,1893955.0897933282,1733009.0820338838,1572063.074274458,1411088.1604055539,1250135.6556238234,1089183.1508420222,928155.8163088672,767196.0051315725,606236.1939543188,445147.8149043247,284179.81869314983,123211.82248207554,-37756.17372910306,-198924.33267981187,-359901.4674401581,-520878.6022004448,-681855.7369607538,-843116.4126691781,-1004103.7204324752,-1165091.0281957462,-1326078.335959021,-1487065.643722348,-1648052.9514855966,-1809442.9118025564,-1970441.5145087652,-2131440.1172151007,-2292438.7199213505,-2453437.3226276077,-2614435.9253339283,-2775975.8014300168,-2936986.9155781604,-3097998.0297262557,-3259009.1438743286,-3420020.258022446,-3581031.3721705973,-3742042.486318659,-3903053.6004667915,-4064794.044359006,-4225818.988646284,-4386843.932933457,-4547868.8772207275,-4708893.821507961,-4869918.765795127,-5030943.7100824565,-5191968.65436966,-5352993.598656885,-5514018.542944096,-5676021.383662872,-5837061.587236851,-5998101.790811017,-6159141.994385034,-6320182.197959028,-6481222.401533067,-6642262.605106987,-6803302.808681063,-6964343.012255184],[8983377.603778368,8822414.03298719,8661450.462196002,8500486.891404836,8339523.320613637,8178559.749822452,8017596.179031282,7856632.608240097,7695669.03744892,7534705.466657742,7373741.895866558,7212778.325075376,7051814.7542841965,6890851.183493019,6729887.612701835,6568924.041910653,6407960.471119478,6246996.900328293,6086033.329537115,5925069.758745926,5764106.187954752,5603169.026470904,5442264.650241653,5281385.350896856,5120527.497073053,4959687.8505020775,4798864.020349934,4638056.312048412,4477259.832933819,4316472.425440589,4155693.9649582077,3994926.089884989,3834162.4956557136,3673407.567921547,3512656.925126251,3351912.0534639806,3191171.106222024,3030435.268068148,2869701.1569955107,2708973.4097947106,2548245.6625938546,2387523.106753206,2226802.5819663163,2066082.0571794398,1905367.2724708132,1744653.1700278036,1583939.0675848164,1423224.9651417844,1262479.4332449473,1101758.908458028,941038.3836711943,780240.4433847778,619512.6961839758,458784.94898321107,297926.0194522701,137190.18129842728,-23545.656855478883,-184281.49500934035,-345219.99476803467,-505964.8664302863,-666709.7380925491,-827454.6097548418,-988485.3363694325,-1149240.2641035728,-1309995.1918377131,-1470750.1195719019,-1631505.047306031,-1792653.2527228855,-1953419.3455730192,-2114185.4384232685,-2274951.5312734805,-2435717.6241237037,-2596483.7169739828,-2757249.8098241314,-2918558.2962659337,-3079336.7567483336,-3240115.217230633,-3400893.677713018,-3561672.138195418,-3722450.598677758,-3883229.059160143,-4044007.51964242,-4205515.27640228,-4366307.408056617,-4527099.539711028,-4687891.671365462,-4848683.803019747,-5009475.934674181,-5170268.066328511,-5331060.1979829,-5491852.329637289,-5652644.461291574,-5814412.698511459,-5975219.914058596,-6136027.129605681,-6296834.34515278,-6457641.560699828,-6618448.776246816,-6779255.991793938,-6940063.207341112],[8984697.736719567,8823987.524775682,8663277.312831806,8502567.100887941,8341856.888944055,8181146.677000173,8020436.465056296,7859726.253112411,7699016.041168531,7538305.829224653,7377595.617280774,7216885.405336892,7056175.193393015,6895464.981449138,6734754.769505257,6574044.557561375,6413334.345617499,6252624.133673616,6091913.9217297435,5931203.709785854,5770493.497841984,5609799.713124522,5449137.262829901,5288501.8104310455,5127889.4834588505,4967296.828374904,4806720.769612935,4646158.572416692,4485607.809134748,4325067.042849323,4164539.074581243,4004017.1984858084,3843502.1682277825,3682995.840951368,3522491.845982449,3361997.490164468,3201503.4343363773,3041017.902609743,2880532.7752492223,2720053.3065591697,2559576.1760708652,2399100.356110761,2238630.3640548736,2078160.371998947,1917692.4450570717,1757228.8006666303,1596765.1562761962,1436301.5118857287,1275810.4117193185,1115340.419663377,954870.4276075251,794327.7406996563,633850.6102113388,473373.4797230698,312770.73748130724,152285.61012085155,-8199.517239674926,-168684.64460015297,-329365.7938815318,-489859.84970960766,-650353.9055376984,-810847.9613658153,-971342.0171939097,-1132124.0081164725,-1292628.0030853935,-1453131.9980543181,-1613635.9930232204,-1774139.987992186,-1935038.9210618027,-2095553.951319836,-2256068.9815778807,-2416584.011835929,-2577099.042093996,-2737614.0723519996,-2898129.102610085,-3059187.359042026,-3219714.6131224744,-3380241.867203038,-3540769.1212835982,-3701296.3753640912,-3861823.6294445954,-4022350.8835250586,-4182878.137605652,-4344134.336562552,-4504675.102847986,-4665215.869133413,-4825756.635418788,-4986297.401704244,-5146838.167989582,-5307378.934275024,-5467919.700560436,-5628460.466845848,-5789001.2331312,-5950515.963327847,-6111071.638111897,-6271627.312895954,-6432182.9876798615,-6592738.662463814,-6753294.337247826,-6913850.012031928],[8986585.162116716,8826237.18332987,8665889.204543011,8505541.225756181,8345193.246969316,8184845.2681824695,8024497.289395623,7864149.310608767,7703801.331821918,7543453.353035067,7383105.3742482215,7222757.395461367,7062409.416674517,6902061.437887668,6741713.459100818,6581365.480313962,6421017.501527119,6260669.5227402635,6100321.543953419,5939973.565166562,5779625.586379715,5619285.237046975,5458976.33152517,5298694.557748089,5138436.0637233285,4978197.412854039,4817975.543384792,4657767.731603585,4497572.471892368,4337391.031857295,4177217.705952054,4017050.7031441517,3856895.6097880267,3696744.2586581465,3536600.070764195,3376460.8201613575,3216326.454477191,3056196.179067204,2896070.6368097924,2735946.593858974,2575828.954392923,2415711.3149268944,2255598.1596079245,2095487.574592702,1935376.9895774536,1775271.0676991493,1615166.755671002,1455062.44364281,1294958.1316146776,1134824.0645013414,974713.4794861749,814602.8944709003,654417.2808004841,494299.64133450016,334182.0018684119,173936.41843422502,13810.87617680803,-146314.66608054936,-306440.20833795145,-466763.9562590532,-626898.3219432235,-787032.6876274534,-947167.0533116721,-1107301.4189958088,-1267725.5660545118,-1427869.753948439,-1588013.9418424182,-1748158.129736375,-1908302.3176302835,-2068842.8443892598,-2228997.9377453886,-2389153.0311015137,-2549308.1244577244,-2709463.21781379,-2869618.311169978,-3029773.4045261294,-3190472.269019101,-3350639.4423880763,-3510806.615756996,-3670973.7891259342,-3831140.962494891,-3991308.1358637214,-4151475.309232764,-4311642.482601643,-4472537.931511857,-4632718.458118558,-4792898.984725215,-4953079.511332035,-5113260.0379387215,-5273440.56454543,-5433621.091152169,-5593801.617758781,-5753982.144365422,-5914162.670972228,-6075314.669047743,-6235509.928758971,-6395705.188470103,-6555900.448181249,-6716095.7078925595,-6876290.967603818],[8988615.799689766,8828657.53922502,8668699.278760273,8508741.018295527,8348782.75783077,8188824.497366016,8028866.236901269,7868907.976436518,7708949.715971772,7548991.455507018,7389033.195042267,7229074.934577515,7069116.674112769,6909158.413648015,6749200.153183265,6589241.892718512,6429283.63225377,6269325.371789014,6109367.111324268,5949408.85085951,5789450.590394763,5629492.329930013,5469563.286027898,5309662.676037651,5149785.500125404,4989928.338635839,4830088.1436243905,4670264.749125213,4510454.842949938,4350655.3028282,4190864.104450902,4031084.4969586246,3871311.78880642,3711544.1175172385,3551786.003334254,3392029.3431116585,3232282.1527364664,3072534.9623612463,2912796.1468189433,2753057.674829742,2593324.3514937293,2433593.688215023,2273863.544269869,2114139.851460401,1954416.1586509384,1794693.379335776,1634975.88483496,1475258.3903341405,1315540.8958333395,1155823.401332505,996074.0017942637,836350.3089847565,676626.6161752865,516825.72887056693,357095.0655918382,197364.40231318027,37503.65101319179,-122234.82097599655,-281973.29296522215,-441711.7649543956,-601650.4162686542,-761397.606643863,-921144.7970190868,-1080891.9873942845,-1240920.8662832193,-1400677.7619370893,-1560434.6575909853,-1720191.553244952,-1879948.4488988332,-2039705.3445527442,-2199859.720182039,-2359627.3914711736,-2519395.062760424,-2679162.734049525,-2838930.4053387456,-2998698.0766278952,-3158465.747917056,-3318777.447686758,-3478557.055179052,-3638336.662671309,-3798116.2701635994,-3957895.8776558414,-4117675.4851482064,-4277455.092640445,-4437234.700132802,-4597741.595896751,-4757534.397659726,-4917327.199422732,-5077120.001185618,-5236912.802948616,-5396705.604711577,-5556498.4064744115,-5716291.208237357,-5876084.010000363,-6035876.811763339,-6196638.241764359,-6356445.6012376845,-6516252.960710958,-6676060.32018435,-6835867.679657757],[8991310.812630534,8831869.776871597,8672428.741112655,8512987.705353724,8353546.669594767,8194105.633835822,8034664.598076879,7875223.562317933,7715782.526558988,7556341.4908000445,7396900.455041102,7237459.419282152,7078018.383523208,6918577.347764265,6759136.312005321,6599695.276246371,6440254.240487435,6280813.204728486,6121372.168969544,5961931.133210592,5802490.097451657,5643049.061692705,5483625.58379759,5324231.415806958,5164862.473667573,5005515.112091763,4846186.081766379,4686872.490523546,4527571.768118963,4368281.634294309,4209004.063758818,4049735.5345967077,3890472.547471117,3731219.8046326283,3571971.528318027,3412729.4312878605,3253492.45617684,3094259.947494317,2935031.2747766376,2775807.3794393316,2616584.3793104663,2457368.1986027844,2298152.017895134,2138939.2781951763,1979729.9839751832,1820520.6897552088,1661314.6830171049,1502111.5124273524,1342908.341837611,1183705.1712478362,1024474.2186553776,865264.9244353399,706055.6302153505,546774.0301107466,387557.8494030386,228341.66869539768,69125.48798769712,-90222.73663254455,-249446.63196985424,-408670.527307149,-567894.4226444773,-727320.2608864941,-886552.7695690282,-1045785.2782515734,-1205017.7869340926,-1364533.4796171822,-1523775.576647319,-1683017.6736775003,-1842259.7707076818,-2001501.8677378185,-2160743.9647680074,-2320384.4232300334,-2479637.16606858,-2638889.908907015,-2798142.6517455317,-2957395.3945840225,-3116648.137422476,-3275900.8802610077,-3435697.6213042885,-3594962.1565361954,-3754226.691768177,-3913491.227000043,-4072755.762232028,-4232020.297463924,-4391284.832695901,-4550549.367927797,-4710539.886227801,-4869817.4567633495,-5029095.027298801,-5188372.597834282,-5347650.168369718,-5506927.738905162,-5666205.30944062,-5825482.879976146,-5984760.45051159,-6144038.02104713,-6304281.025801741,-6463572.978653401,-6622864.931505226,-6782156.88435711],[8994758.158794876,8835978.734926056,8677199.311057236,8518419.887188425,8359640.463319607,8200861.039450776,8042081.615581959,7883302.191713136,7724522.767844318,7565743.343975499,7406963.920106679,7248184.496237859,7089405.072369039,6930625.648500219,6771846.224631395,6613066.800762575,6454287.3768937625,6295507.953024936,6136728.529156121,5977949.105287289,5819169.68141848,5660390.257549659,5501619.258114367,5342877.706662928,5184161.54141889,5025467.135549948,4866791.254813769,4708131.019122044,4549483.867672505,4390851.708626615,4232229.575205522,4073614.908172244,3915009.832385,3756412.6020541023,3597819.1748506073,3439235.6395701226,3280653.0314762816,3122079.591670722,2963506.1518651787,2804940.5174609125,2646375.5859597754,2487814.9167712014,2329257.6058188956,2170700.294866586,2012149.1786330417,1853598.6701868437,1695048.1617406905,1536503.1654071063,1377958.7059127465,1219414.2464183234,1060869.7869240083,902295.6195097789,743745.1110635623,585194.6026174352,426569.8743910529,268012.56343877316,109455.25248642266,-49228.249559316784,-207793.18106045946,-366358.1125615351,-524923.0440626703,-683682.9636621438,-842256.4034676962,-1000829.8432732001,-1159403.2830787152,-1317976.7228842601,-1476834.6083149947,-1635417.5195371434,-1794000.4307592139,-1952583.34198137,-2111166.253203448,-2269749.1644255184,-2428731.05888224,-2587324.4860856757,-2745917.9132892005,-2904511.3404927067,-3063104.767696131,-3221698.1948996224,-3380291.6221031547,-3539428.7296163626,-3698033.805403646,-3856638.881190829,-4015243.9569781646,-4173849.0327654183,-4332454.108552657,-4491059.184339911,-4649664.260127142,-4808993.695844635,-4967611.647968225,-5126229.600091994,-5284847.552215666,-5443465.504339308,-5602083.4564629495,-5760701.408586688,-5919319.360710412,-6077937.312834077,-6236555.264957815,-6395173.2170813605,-6554767.265539393,-6713399.42458529],[8998482.197355632,8840417.48795807,8682352.778560502,8524288.069162946,8366223.359765367,8208158.650367805,8050093.940970252,7892029.231572681,7733964.522175116,7575899.812777544,7417835.103379987,7259770.393982416,7101705.684584856,6943640.975187292,6785576.265789726,6627511.556392157,6469446.846994601,6311382.137597032,6153317.428199466,5995252.718801901,5837188.009404335,5679123.300006775,5521058.590609205,5363022.049281323,5205011.763513773,5047023.415932817,4889053.7873669965,4731102.071283867,4573165.291702053,4415240.26923525,4257324.856482552,4099419.6253518052,3941524.640397802,3783634.477049308,3625753.468061948,3467877.130588347,3310006.5077554677,3152140.7237454336,2994279.4553979896,2836421.3485430162,2678568.4834592044,2520715.6183754243,2362870.1945465803,2205024.8559307624,2047181.5792570524,1889342.9591657817,1731504.3390745409,1573667.3320964202,1415834.6862785742,1258002.0404606573,1100169.3946428783,942336.7488249466,784472.6185270101,626633.9984357804,468795.37834447995,310880.7925407775,153035.45392486826,-4809.884690947831,-162783.08804899082,-320635.9531327598,-478488.8182165548,-636341.683300402,-794390.9849398099,-952252.2532871813,-1110113.52163456,-1267974.7899819836,-1425836.0583293885,-1583982.8000645787,-1741853.4228974544,-1899724.0457304046,-2057594.6685633063,-2215465.291396197,-2373335.9142291397,-2531605.8824453093,-2689486.8914327435,-2847367.9004201144,-3005248.909407418,-3163129.91839483,-3321010.9273821935,-3478891.9363695644,-3637316.0201522447,-3795208.5339136273,-3953101.0476751477,-4110993.5614366233,-4268886.07519801,-4426778.588959545,-4584671.102720991,-4742563.616482429,-4901178.549101837,-5059083.780232623,-5216989.011363357,-5374894.242494121,-5532799.473624811,-5690704.704755664,-5848609.93588648,-6006515.167017236,-6164420.398148015,-6322325.629278615,-6480230.860409468,-6639108.11688517],[9001883.538905213,8844471.612229284,8687059.685553364,8529647.758877438,8372235.832201504,8214823.90552557,8057411.978849664,7900000.052173724,7742588.125497791,7585176.198821866,7427764.272145942,7270352.345470009,7112940.418794086,6955528.492118157,6798116.565442232,6640704.638766302,6483292.712090375,6325880.785414447,6168468.858738521,6011056.932062589,5853645.005386665,5696233.078710737,5538821.152034807,5381425.899912226,5224058.1053233305,5066713.923584385,4909389.924461062,4752083.051641044,4594790.585967137,4437510.112093784,4280241.814072331,4122985.0463724155,3965734.9354116675,3808493.0519264024,3651259.384775093,3494028.8622542284,3336808.0471654665,3179587.78097214,3022376.655434139,2865165.626795249,2707961.4139627498,2550758.6835466586,2393558.952779142,2236363.6547501683,2079168.3567211851,1921977.6224007532,1764788.95891482,1607600.2954288647,1450415.2456515692,1293232.4817605652,1136049.7178696431,978866.9539786018,821656.9779990986,664468.3145132065,507279.65102719143,350020.07840255275,192824.78037353233,35629.48234459013,-121565.8156843707,-278890.4639426507,-436093.1943587437,-593295.9247748293,-750498.6551909558,-907899.0605382845,-1065110.0891771838,-1222321.1178160682,-1379532.1464549042,-1536743.175093878,-1694240.471020911,-1851460.7372142859,-2008681.0034075938,-2165901.269600909,-2323121.535794299,-2480341.8019875325,-2637961.51605919,-2795192.038580045,-2952422.5611009337,-3109653.083621789,-3266883.6061426774,-3424114.1286634766,-3581344.6511843614,-3739117.3553666174,-3896359.2388519794,-4053601.1223372295,-4210843.005822539,-4368084.889307812,-4525326.772793017,-4682568.656278402,-4839810.53976357,-4997772.5831004605,-5155027.024987929,-5312281.466875382,-5469535.908762768,-5626790.350650288,-5784044.792537801,-5941299.234425254,-6098553.6763127,-6255808.118200079,-6413062.560087591,-6570317.0019752085],[8999281.896832198,8841370.665111963,8683459.433391709,8525548.201671477,8367636.96995123,8209725.738230988,8051814.506510751,7893903.274790503,7735992.04307026,7578080.811350023,7420169.579629777,7262258.347909531,7104347.116189294,6946435.884469049,6788524.652748806,6630613.421028557,6472702.189308323,6314790.9575880775,6156879.72586784,5998968.494147588,5841057.262427351,5683146.030707105,5525234.798986861,5367330.77273169,5209454.370780811,5051601.768543267,4893769.552218703,4735954.6786949225,4578154.43916294,4420369.948952515,4262597.004967101,4104832.763025932,3947077.022615822,3789332.09731742,3631591.3338232543,3473859.2100029606,3316131.4433786105,3158409.446058912,3000691.5367415044,2842978.660045186,2685267.850205606,2527563.166691277,2369858.48317698,2212159.6960110497,2054462.350802958,1896765.3525452688,1739074.557898663,1581383.7632520646,1423692.968605511,1266007.638906192,1108322.6691762134,950637.6994460896,792952.7297160476,635238.9953725673,477548.2007259205,319857.40607940406,162094.2083062753,4396.863098222762,-153300.48210982606,-310997.8273178674,-468825.82008062303,-626530.5035949871,-784235.1871092878,-941939.8706235923,-1099843.257363297,-1257556.134059634,-1415269.0107559487,-1572981.8874523006,-1730694.764148634,-1888694.4681714773,-2046416.4654911608,-2204138.462810863,-2361860.460130606,-2519582.4574502446,-2677304.4547699727,-2835425.74280905,-2993157.8666293286,-3150889.990449596,-3308622.114269927,-3466354.2380902804,-3624086.3619105592,-3781818.48573076,-3940091.610459514,-4097834.9514345005,-4255578.2924095765,-4413321.633384623,-4571064.974359661,-4728808.315334782,-4886551.656309754,-5044294.997284777,-5202038.3382598385,-5360511.661580868,-5518267.40199098,-5676023.142401114,-5833778.882811271,-5991534.6232213825,-6149290.363631457,-6307046.1040414795,-6464801.844451673,-6622557.58486186],[8996680.254759187,8838269.717994628,8679859.181230074,8521448.644465527,8363038.107700958,8204627.570936402,8046217.034171849,7887806.497407285,7729395.960642728,7570985.4238781715,7412574.887113618,7254164.350349055,7095753.813584497,6937343.27681994,6778932.740055382,6620522.203290816,6462111.666526267,6303701.129761705,6145290.592997154,5986880.056232581,5828469.5194680365,5670058.982703475,5511648.445938914,5353237.90917435,5194850.954451058,5036489.613502142,4878150.154222804,4719829.430668938,4561524.525049387,4403232.872212857,4244952.195861867,4086681.6515833214,3928423.4407466836,3770171.1427084524,3611926.3442436494,3453689.557751674,3295455.832631951,3137231.1111456677,2979007.382699566,2820791.6932951193,2662576.968541315,2504367.649835894,2346161.013223352,2187955.737271903,2029756.3448847458,1871556.9524975866,1713360.1568825468,1555167.2310752906,1396974.3052680716,1238782.796051789,1080595.620482754,922408.4449135885,764221.2693444826,606034.0937754288,447816.75042461604,289623.82461744547,131430.89881018177,-26835.756148118526,-185035.14853525907,-343234.54092237353,-501433.93330954015,-659765.0824151002,-817971.7190276757,-976178.3556402512,-1134384.992252767,-1292791.1503032036,-1451005.8750569597,-1609220.5998107865,-1767435.3245645165,-1925650.0493183509,-2084151.9275747351,-2242375.656020779,-2400599.3844669126,-2558823.1129128933,-2717046.841359023,-2875270.5698051155,-3033893.1721577346,-3192126.897277493,-3350360.6223972663,-3508594.34751698,-3666828.0726367533,-3825061.7977563962,-3983295.5228762254,-4142068.7805318683,-4300313.578996681,-4458558.377461463,-4616803.175926261,-4775047.974391155,-4933292.772855878,-5091537.57132069,-5249782.369785532,-5408027.16825027,-5566998.8952191025,-5725255.934151903,-5883512.97308474,-6041770.012017503,-6200027.050950244,-6358284.089882925,-6516541.128815763,-6674798.167748556],[8994078.612686174,8835168.770877307,8676258.92906844,8517349.087259572,8358439.245450693,8199529.403641813,8040619.561832949,7881709.720024066,7722799.878215195,7563890.036406326,7404980.194597453,7246070.352788574,7087160.510979705,6928250.669170829,6769340.827361957,6610430.985553077,6451521.1437442135,6292611.301935333,6133701.460126465,5974791.618317588,5815881.776508722,5656971.934699846,5498062.092890971,5339152.251082082,5180256.186568229,5021385.89936471,4862537.771053137,4703708.574324372,4544895.434642509,4386095.7954732105,4227309.445391847,4068535.7193495333,3909769.858877534,3751011.52142212,3592263.9321449082,3433519.9055004064,3274784.5790812224,3116052.77623244,2957327.316659972,2798604.726545036,2639888.153733801,2481172.132980481,2322463.54326972,2163754.953558944,2005050.3389665224,1846348.8994003013,1687647.459834084,1528950.6988984868,1370255.6419306323,1211560.584962763,1052868.5717892982,894179.190381065,735489.8089729063,576800.427564811,418111.04615657777,259390.2431554906,100695.1861875616,-57999.87078027055,-216769.81496068835,-375471.2545269169,-534172.6940931678,-692874.1336594075,-851708.2509460934,-1010416.8406568319,-1169125.4303675964,-1327834.0200783946,-1486742.7393579632,-1645459.3121692427,-1804175.8849804439,-1962892.4577917047,-2121609.0306028984,-2280612.8492307477,-2439338.308803227,-2598063.7683756314,-2756789.2279480807,-2915514.6875205114,-3074240.1470929794,-3233363.804105375,-3392099.1305245757,-3550834.456943769,-3709569.7833629698,-3868305.1097820923,-4027040.436201304,-4185775.7626205087,-4345048.865583688,-4503795.121538326,-4662541.377492838,-4821287.63344752,-4980033.889402032,-5138780.145356648,-5297526.401311241,-5456272.657265753,-5615018.913220264,-5774488.725902721,-5933247.063358173,-6092005.400813632,-6250763.738268927,-6409522.075724319,-6568280.413179785,-6727038.750635251],[8991476.970613169,8832067.82375998,8672658.676906787,8513249.530053612,8353840.383200421,8194431.236347219,8035022.089494041,7875612.942640848,7716203.795787664,7556794.648934474,7397385.502081286,7237976.355228096,7078567.208374909,6919158.061521718,6759748.914668536,6600339.767815338,6440930.62096216,6281521.474108968,6122112.32725578,5962703.18040258,5803294.033549406,5643884.886696211,5484475.739843025,5325066.59298982,5165661.418685399,5006282.217153172,4846925.387883475,4687587.717979816,4528267.000468008,4368963.128614733,4209671.43871673,4050389.787115764,3891116.9965801258,3731856.641030349,3572601.520046165,3413353.8066018242,3254113.3255305067,3094876.3978118747,2935647.250620337,2776420.059921544,2617199.338926263,2457980.9180575926,2298766.0733160935,2139555.530507069,1980344.987698041,1821140.84630296,1661937.3595576584,1502734.166721698,1343536.978593219,1184339.7904646583,1025142.6023362279,865949.9358485751,706758.3486013487,547566.7613541745,388375.17410692945,229156.66169356927,69959.47356497869,-89237.71456350759,-248434.9026920125,-407707.9681314193,-566911.4548767544,-726114.9416221157,-885444.782864429,-1044655.3256734721,-1203865.868482452,-1363076.4112915024,-1522286.954100471,-1681698.0245276876,-1840916.445396319,-2000134.8662650213,-2159353.287133701,-2318571.7080023475,-2478077.233139541,-2637304.423838295,-2796531.6145371534,-2955758.805235982,-3114985.9959347844,-3274213.1866335757,-3433837.6386519074,-3593074.5663705133,-3752311.49408916,-3911548.421807684,-4070785.349526383,-4230022.277245,-4389259.204963647,-4549031.865615159,-4708279.579059489,-4867527.292503916,-5026775.005948149,-5186022.719392516,-5345270.432836853,-5504518.146281138,-5663765.859725453,-5823013.573169857,-5982981.15363162,-6142240.789609753,-6301500.425587729,-6460760.061565772,-6620019.697543815,-6779279.333521992],[8988875.328540156,8828966.876642656,8669058.424745146,8509149.972847652,8349241.520950137,8189333.069052638,8029424.617155139,7869516.165257633,7709607.713360127,7549699.261462623,7389790.809565122,7229882.357667619,7069973.905770116,6910065.453872612,6750157.001975107,6590248.550077598,6430340.098180104,6270431.646282594,6110523.194385095,5950614.742487581,5790706.290590091,5630797.838692584,5470889.386795075,5310980.934897561,5151072.483000077,4991183.037595572,4831317.637272506,4671472.861048859,4511645.660021257,4351833.320747117,4192033.4320416003,4032245.222089315,3872469.9412177894,3712701.7606385387,3552940.826137837,3393189.9370133057,3233442.071979806,3073703.542961728,2913967.184580734,2754238.2627555486,2594510.524118755,2434790.2551926374,2275069.986266546,2115356.1074552033,1955643.6115479246,1795932.7932056636,1636227.2592812814,1476521.7253568731,1316818.3152557798,1157118.995966576,997419.6766774766,837720.6813160479,678026.8882297687,518333.09514350817,358639.30205720663,198945.50897097215,39223.76094241068,-120475.55834675208,-280174.8776359111,-439874.1969250776,-599650.2156603672,-759355.7495848201,-919061.2835092433,-1078893.8106900565,-1238606.3065973148,-1398318.802504573,-1558031.2984118126,-1717743.7943191305,-1877657.0058122277,-2037377.2747383304,-2197097.543664485,-2356817.812590588,-2516538.081516724,-2676545.0793010034,-2836274.0011262335,-2996002.922951404,-3155731.8447765745,-3315460.7666017376,-3475189.688426994,-3635314.675797265,-3795053.2048153207,-3954791.7338333055,-4114530.262851499,-4274268.791869491,-4434007.320887573,-4593745.849905707,-4753484.3789237365,-4913766.951560244,-5073516.122494318,-5233265.293428436,-5393014.464362495,-5552763.635296561,-5712512.80623062,-5872261.97716482,-6032011.148098983,-6191760.319033019,-6352237.112906463,-6511998.047407098,-6671758.981907941,-6831519.916408688],[8986273.686467145,8825865.929525334,8665458.172583502,8505050.415641697,8344642.658699871,8184234.901758049,8023827.144816241,7863419.387874417,7703011.630932593,7542603.873990779,7382196.117048954,7221788.360107138,7061380.603165324,6900972.846223504,6740565.089281684,6580157.332339859,6419749.57539805,6259341.818456221,6098934.061514411,5938526.304572582,5778118.547630774,5617710.790688949,5457303.033747123,5296895.276805298,5136487.519863493,4976088.629010445,4815714.007391944,4655360.249750655,4495024.319574498,4334703.512879495,4174398.8623370957,4014106.285478739,3853822.885855468,3693547.8490627073,3533284.647223396,3373026.067424806,3212775.4390009455,3052530.688111566,2892290.557794025,2732056.4655895717,2571825.812637992,2411599.592327688,2251377.4753441587,2091156.6844033375,1930942.2353978083,1770727.7863923311,1610517.1590048783,1450309.5779014006,1290101.9967979155,1129898.2014685161,969696.7510187365,809495.3005688787,649295.4278582074,489099.428932894,328903.43000751734,168707.43108221516,8511.432156860828,-151713.40212996677,-311914.85257981345,-472116.30302957445,-632388.9764439762,-792596.557547491,-952804.1386510171,-1113011.7197544798,-1273346.7447121553,-1433561.1937176622,-1593775.642723132,-1753990.0917287096,-1914204.5407341607,-2074619.6832117103,-2234841.800195273,-2395063.917178791,-2555286.0341623984,-2715508.15114592,-2876016.387715284,-3036247.0406667963,-3196477.693618316,-3356708.3465699516,-3516938.9995215535,-3677169.652473081,-3837794.915541541,-3998035.045858979,-4158275.1761765704,-4318515.306494027,-4478755.436811581,-4638995.56712909,-4799235.697446555,-4959475.8277641535,-5120257.239040457,-5280507.867464319,-5440758.4958881885,-5601009.124312043,-5761259.752735868,-5921510.381159812,-6081761.009583697,-6242011.638007589,-6402262.2664314285,-6563236.033248544,-6723498.266271956,-6883760.499295406],[8983672.044394132,8822764.982408004,8661857.920421869,8500950.858435743,8340043.796449604,8179136.734463456,8018229.672477333,7857322.610491205,7696415.548505063,7535508.486518932,7374601.424532799,7213694.362546656,7052787.300560527,6891880.238574389,6730973.176588259,6570066.114602119,6409159.052615995,6248251.990629849,6087344.928643724,5926437.866657581,5765530.804671454,5604623.742685322,5443716.680699176,5282809.618713034,5121902.556726911,4960995.494740783,4800111.325813741,4639249.5845042085,4478407.038530434,4317580.809309309,4156768.337008735,3995967.3488681596,3835177.2790449318,3674399.6705718413,3513628.4683089536,3352864.936762931,3192110.1119226683,3031358.0260090437,2870616.10164441,2709874.6684235744,2549142.284345638,2388409.9002676886,2227684.964421734,2066960.9993807375,1906240.859247703,1745524.457143981,1584808.0550402626,1424097.4304458909,1263387.8021633364,1102678.1738807335,941973.8253600299,781270.2437495068,620566.662139032,459865.76272224635,299167.55795781314,138469.35319345072,-22228.851570989937,-182927.0563353449,-343654.8275236748,-504358.40913408995,-665061.990744561,-825837.3655101843,-986546.9937927909,-1147256.6220753677,-1307966.2503579184,-1468803.5849307477,-1629519.9870344698,-1790236.3891382404,-1950952.7912419587,-2111669.1933457106,-2272586.056726061,-2433310.0217670314,-2594033.9868080504,-2754757.95184898,-2915481.9168900438,-3076491.158382289,-3237223.5424601473,-3397955.926538121,-3558688.310616091,-3719420.6946939714,-3880153.0787719674,-4041278.357884608,-4202020.089501664,-4362761.821118556,-4523503.55273556,-4684245.284352489,-4844987.015969425,-5005728.747586466,-5166470.4792033285,-5327750.441500224,-5488502.52741383,-5649254.613327481,-5810006.6992410645,-5970758.785154745,-6131510.871068478,-6292262.956982054,-6453015.042895667,-6613767.128809251,-6774519.214722931,-6936001.082182124],[8981070.402321117,8819664.035290675,8658257.668260226,8496851.301229795,8335444.934199326,8174038.5671688765,8012632.20013843,7851225.83310798,7689819.46607753,7528413.099047084,7367006.732016632,7205600.36498618,7044193.99795573,6882787.630925283,6721381.263894832,6559974.896864379,6398568.529833936,6237162.162803484,6075755.795773033,5914349.42874258,5752943.061712142,5591536.694681689,5430130.327651233,5268723.960620776,5107317.5935903415,4945911.226559892,4784517.276526312,4623146.006954615,4461794.199103687,4300458.984760368,4139137.811680367,3977831.7262165025,3816537.412498884,3655251.4920809604,3493974.20903112,3332707.459325826,3171444.784844404,3010191.241441002,2848941.6454948094,2687698.3125784285,2526458.7560532615,2365224.6408489626,2203992.4534993414,2042766.6404008903,1881540.8273024634,1720321.1278956123,1559102.7726936564,1397885.2829904184,1236673.607528787,1075461.9320670925,914250.899701342,753045.1869301386,591839.4741590135,430633.76138797775,269431.685908135,108231.27530469,-52969.1352988109,-214169.54590224847,-375369.9565057531,-536600.5152386166,-697806.2280097343,-859011.9407808334,-1020289.848934561,-1181501.5243962333,-1342713.1998578608,-1503924.8753195666,-1665264.3313457966,-1826482.6865478046,-1987701.0417497419,-2148919.39695175,-2310137.7521537244,-2471556.1263552494,-2632781.939453732,-2794007.752552148,-2955233.565650586,-3116459.3787490204,-3277969.391301941,-3439203.506506283,-3600437.621710621,-3761671.7369149365,-3922905.852119293,-4084139.967323512,-4245374.082527861,-4407008.335743099,-4568251.668659538,-4729495.001575917,-4890738.334492266,-5051981.667408779,-5213225.000325106,-5374468.333241507,-5536246.558939524,-5697500.102342866,-5858753.6457463205,-6020007.189149685,-6181260.732553154,-6342514.275956579,-6503767.8193599805,-6665021.362763278,-6826274.906166762,-6987528.449570209],[8978468.760248117,8816563.088173343,8654657.416098583,8492751.744023832,8330846.071949058,8168940.399874293,8007034.727799523,7845129.055724764,7683223.3836499965,7521317.711575238,7359412.039500469,7197506.367425701,7035600.695350934,6873695.023276176,6711789.351201408,6549883.679126639,6387978.007051881,6226072.334977112,6064166.66290235,5902260.990827581,5740355.318752824,5578449.646678051,5416543.974603291,5254638.3025285145,5092732.630453758,4930826.958379006,4768923.227238898,4607042.429405038,4445181.741059821,4283340.303497832,4121514.022188587,3959700.483427383,3797897.5459528174,3636105.6162586715,3474324.907411713,3312549.981888732,3150783.8532974347,2989024.4568729457,2827269.4559797645,2665522.255129393,2503777.3209135197,2342039.38143022,2180303.5350995474,2018572.2814210542,1856844.620265182,1695117.798647277,1533397.4903470725,1371677.1820468567,1209959.412894193,1048245.6902534403,886531.9676127434,724820.1301107779,563112.2861790322,401404.4422473535,239696.59831553698,77993.19741597399,-83709.41902661696,-245412.03546912596,-407114.65191164985,-568817.2683541775,-730550.4652748741,-892258.3092066534,-1053966.1531384252,-1215746.426717095,-1377460.1493578255,-1539173.8719985709,-1700887.5946392827,-1862728.9839573652,-2024449.292257581,-2186169.600557793,-2347889.9088580385,-2509610.217158217,-2671529.892099399,-2833257.553255219,-2994985.2144111358,-3156712.8755670115,-3318440.536722876,-3480451.086474482,-3642186.932805218,-3803922.779135827,-3965658.625466533,-4127394.471797183,-4289130.318127908,-4450866.164458573,-4612999.784583494,-4774744.71879939,-4936489.653015144,-5098234.587231085,-5259979.521446839,-5421724.45566272,-5583469.3898785785,-5745214.32409437,-5907500.59225145,-6069255.593144737,-6231010.594037928,-6392765.594931118,-6554520.595824137,-6716275.596717276,-6878030.597610556,-7039785.5985038355],[8975867.118175099,8813462.141056024,8651057.16393695,8488652.186817866,8326247.209698786,8163842.232579695,8001437.255460624,7839032.278341542,7676627.301222466,7514222.324103382,7351817.346984303,7189412.369865223,7027007.39274614,6864602.415627064,6702197.438507986,6539792.461388901,6377387.48426983,6214982.507150741,6052577.530031664,5890172.552912577,5727767.5757935075,5565362.59867443,5402957.621555346,5240552.644436255,5078147.667317181,4915742.690198116,4753337.713079032,4590947.361232995,4428578.215841219,4266227.235942556,4103891.7135374285,3941569.24063826,3779261.720252905,3616964.8197771516,3454675.6057922877,3292396.076485336,3130125.0773376822,2967857.672304902,2805601.213921951,2643346.1976803467,2481099.662165029,2318854.122011481,2156616.5445544366,1994378.9670973904,1832148.4132278673,1669918.9040145613,1507692.208000511,1345469.9466020465,1183247.6852036119,1021029.4484397806,858813.6786199808,696597.9088000841,534385.0981990211,372175.123106692,209965.1480142437,47755.17292190343,-114449.70275441185,-276654.52503602207,-438859.3473175578,-601064.1695991866,-763268.9918807596,-925504.6776324473,-1087714.652724892,-1249924.6278172769,-1412207.0988578089,-1574422.8686776683,-1736638.6384974346,-1898854.4083173573,-2061197.5427653454,-2223419.8041638546,-2385642.065562289,-2547864.3269607313,-2710086.5883592106,-2872507.353958342,-3034736.863171663,-3196966.372385025,-3359195.8815982603,-3521425.3908116333,-3683654.9000249654,-3846173.821356807,-4008411.3988138475,-4170648.9762708545,-4332886.553727984,-4495124.131184973,-4657361.70864208,-4819994.436022736,-4982240.971538007,-5144487.507053338,-5306734.042568579,-5468980.5780839175,-5631227.113599159,-5793473.649114445,-5955720.184629634,-6118503.99713964,-6280760.455522619,-6443016.913905568,-6605273.372288436,-6767529.830671363,-6929786.289054312,-7092042.74743738],[8973265.47610209,8810361.193938699,8647456.911775304,8484552.629611906,8321648.347448509,8158744.065285116,7995839.7831217265,7832935.500958323,7670031.2187949335,7507126.936631538,7344222.65446814,7181318.372304747,7018414.090141343,6855509.807977954,6692605.525814564,6529701.243651159,6366796.961487774,6203892.679324369,6040988.397160981,5878084.114997582,5715179.8328341935,5552275.550670797,5389371.2685074,5226466.986343988,5063562.7041806085,4900658.422017223,4737754.139853827,4574853.671243279,4411974.6906226035,4249114.1683872845,4086272.6559510864,3923445.2986590564,3760629.7600030433,3597824.023295639,3435030.2337305564,3272245.6517373472,3109466.3013779484,2946697.5757413283,2783932.9718641452,2621175.055991428,2458422.0034165625,2295673.866601823,2132929.5540093426,1970190.2454258949,1807452.2061905973,1644720.8489198275,1481989.4916490726,1319262.7111572325,1156538.4966605566,993814.2821638472,831095.3896272816,668377.5726282634,505659.7556293011,342945.8039660379,180233.69771290943,17521.591459978372,-145189.98648224026,-307897.0146028623,-470604.0427235663,-633311.0708441511,-796018.0989648402,-958725.1270854995,-1121463.1523113288,-1284175.2585643418,-1446887.3648173362,-1609671.8653566688,-1772389.682355605,-1935107.4993545897,-2097825.316353444,-2260670.007769868,-2423394.2222665735,-2586118.436763268,-2748842.6512599885,-2911566.865756601,-3074488.511932265,-3237219.8692030013,-3399951.2264737524,-3562682.5837445073,-3725413.9410152584,-3888145.298286047,-4051164.1721611843,-4213903.480744526,-4376642.789328039,-4539382.09791144,-4702121.406494893,-4864860.715078339,-5027992.290060863,-5190740.426875621,-5353488.563690342,-5516236.700505078,-5678984.837319799,-5841732.974134505,-6004481.110949203,-6167229.247763932,-6329977.38457872,-6493268.232880108,-6656026.1487527415,-6818784.06462539,-6981541.980498165,-7144299.896370947],[8970663.834029084,8807260.24682137,8643856.659613663,8480453.072405955,8317049.485198231,8153645.897990522,7990242.310782819,7826838.723575113,7663435.136367395,7500031.549159691,7336627.9619519785,7173224.37474427,7009820.787536548,6846417.200328846,6683013.613121137,6519610.025913423,6356206.438705717,6192802.851498004,6029399.264290294,5865995.677082578,5702592.089874877,5539188.502667168,5375784.915459446,5212381.32825173,5048977.7410440305,4885574.153836331,4722170.56662862,4558766.979420896,4395378.883130519,4232010.634051701,4068659.3602672666,3905322.5059057046,3741997.915576294,3578688.844446335,3415388.5073135644,3252095.226989383,3088813.4514253363,2925537.501258943,2762267.477099644,2599005.3564438857,2435745.9830813874,2272494.6065538805,2109244.8684397265,1946001.5237544067,1782760.4840446003,1619522.7938250974,1456289.5884969085,1293056.3831686787,1129829.3081175126,966603.1405225582,803377.1006345227,640157.2364564389,476937.37227841094,313717.5081004426,150502.24741164967,-12711.990001965314,-175926.2274156995,-339139.50416974723,-502348.7381294668,-665557.97208919,-828767.2060489058,-991976.4400086403,-1155185.6739683785,-1318425.8893114552,-1481640.1267250776,-1644854.3641387522,-1808140.7262137122,-1971360.5903918147,-2134580.454569746,-2297800.3187477738,-2461146.3789708577,-2624372.54656573,-2787598.7141607255,-2950824.881755594,-3114051.049350526,-3277473.3660210148,-3440706.5713492036,-3603939.7766774036,-3767172.982005596,-3930406.187333811,-4093639.3926620074,-4257157.985218212,-4420399.024928063,-4583640.064637825,-4746881.104347669,-4910122.144057527,-5073363.183767259,-5236604.223477125,-5400243.08481206,-5563492.822926275,-5726742.561040424,-5889992.299154542,-6053242.0372686535,-6216491.775382876,-6379741.513497122,-6542991.251611285,-6706778.92521704,-6870038.298579425,-7033297.6719419435,-7196557.045304537],[8968062.191956073,8804159.299704049,8640256.407452019,8476353.5152,8312450.622947967,8148547.730695938,7984644.8384439135,7820741.946191898,7656839.053939862,7492936.161687845,7329033.269435814,7165130.377183789,7001227.484931763,6837324.592679737,6673421.700427713,6509518.808175677,6345615.915923662,6181713.023671632,6017810.13141961,5853907.239167573,5690004.346915561,5526101.454663536,5362198.562411499,5198295.670159464,5034392.777907451,4870489.885655442,4706586.993403406,4542684.101151371,4378785.552438498,4214907.16116407,4051046.682142766,3887204.095890183,3723374.638720736,3559556.166735729,3395746.7808965724,3231951.1314605493,3068161.903548822,2904379.0048562214,2740606.104094606,2576835.6568962988,2413074.826044021,2249315.3465059437,2085564.0070923604,1921812.8020829372,1758070.0312467553,1594327.2604105659,1430589.6853447333,1266854.631959077,1103120.1195744611,939391.9988812767,775663.878188178,611936.9002845995,448214.98892746493,284493.0775704421,120771.1662133038,-42945.57146391645,-206661.94003825262,-370378.30861257017,-534093.433535397,-697804.8733341619,-861516.3131329603,-1025227.7529317401,-1188939.1927305795,-1352650.6325293034,-1516392.8886328302,-1680109.2572071366,-1843825.6257814132,-2007613.6814290136,-2171335.59278607,-2335057.5041431487,-2498779.415500298,-2662501.3268573843,-2826354.777061492,-2990082.8977545835,-3153811.018447768,-3317539.1391409263,-3481461.916224595,-3645196.969610255,-3808932.0229959264,-3972667.0763815604,-4136402.129767243,-4300137.1831527725,-4464155.260528147,-4627898.03136424,-4791640.802200504,-4955383.5730366185,-5119126.34387286,-5282869.114709057,-5446611.885545172,-5610748.945347466,-5774500.284761049,-5938251.624174647,-6102002.9635882005,-6265754.303001858,-6429505.642415561,-6593256.981829122,-6757008.321242698,-6920759.660656251,-7085053.363385767,-7248814.194238104],[8965460.549883055,8801058.352586716,8636656.155290382,8472253.957994051,8307851.760697691,8143449.563401351,7979047.366105012,7814645.168808673,7650242.971512331,7485840.774215988,7321438.576919651,7157036.3796233125,6992634.182326963,6828231.985030633,6663829.787734287,6499427.590437941,6335025.393141607,6170623.195845265,6006220.998548922,5841818.801252574,5677416.603956248,5513014.406659901,5348612.209363558,5184210.012067217,5019807.814770876,4855405.617474541,4691003.420178203,4526601.222881847,4362199.025585521,4197811.842218839,4033443.2615860887,3869090.5720778294,3704751.3618651778,3540425.803331379,3376113.0987967886,3211808.4751080833,3047511.8957366142,2883226.3338511437,2718944.7310895603,2554672.109885011,2390403.669006696,2226141.3806646895,2061883.1457449794,1897630.2050319277,1733379.5784488954,1569135.0764863528,1404890.5745238103,1240652.880749464,1076415.979306396,912180.8572399691,747950.7834486514,583720.7096571773,419492.6055765711,255268.64704041556,91044.68850420415,-73179.15292583033,-237397.65266086534,-401616.1523958035,-565834.6521307714,-730051.774579186,-894265.4202170484,-1058479.0658548698,-1222692.711492747,-1386906.357130576,-1551120.0027683452,-1715364.1502755173,-1879582.6500104442,-2043801.1497454494,-2208090.731002353,-2372314.6895385794,-2536538.6480747648,-2700762.606610935,-2864986.5651471913,-3029340.913753595,-3193570.9875450097,-3357801.0613363907,-3522031.1351277344,-3686261.208919201,-3850691.063986279,-4014927.965429317,-4179164.86687243,-4343401.76831542,-4507638.669758618,-4671875.571201675,-4836400.500053301,-5000645.002015837,-5164889.503978349,-5329134.005941018,-5493378.507903479,-5657623.009866066,-5821867.511828586,-5986510.949194685,-6150763.8899077475,-6315016.830620781,-6479269.771333918,-6643522.712046981,-6807775.652760014,-6972028.593472943,-7136281.53418605,-7301071.343171693],[8962858.907810047,8797957.405469397,8633055.903128734,8468154.400788084,8303252.898447422,8138351.396106757,7973449.893766115,7808548.391425456,7643646.889084801,7478745.386744145,7313843.884403485,7148942.382062832,6984040.879722173,6819139.3773815185,6654237.875040863,6489336.3727002,6324434.870359552,6159533.3680188935,5994631.865678235,5829730.363337575,5664828.86099693,5499927.35865627,5335025.856315608,5170124.353974947,5005222.8516343,4840321.349293657,4675419.846952992,4510518.344612321,4345616.842271693,4180718.8708245326,4015839.841029443,3850979.466670096,3686134.6233151294,3521302.04237362,3356479.7036254257,3191668.595936317,3026868.309448678,2862073.662846083,2697288.517896371,2532509.438357305,2367735.518630082,2202968.7661376037,2038205.0203058254,1873447.7423851155,1708693.2003725916,1543942.8925621174,1379196.6594731882,1214451.1295398362,1049712.3800393566,884973.6305387914,720237.6887091212,555505.6618193872,390773.63492975757,226044.21651040018,61318.210795119405,-103407.79492012411,-268133.36528344825,-432853.9961790219,-597574.6270746291,-762295.25797024,-927014.5273010805,-1091730.378777992,-1256446.2302549332,-1421162.0817317888,-1585877.9332086667,-1750593.7846855335,-1915339.6742394865,-2080060.3051351123,-2244780.936030753,-2409501.566926345,-2574297.8806492426,-2739023.8863645084,-2903749.892079845,-3068475.8977950327,-3233330.9566422217,-3398062.9835318625,-3562795.010421451,-3727527.0373111404,-3892259.064200744,-4057188.854477089,-4221927.603977628,-4386666.353478074,-4551405.102978684,-4716143.852479219,-4880882.601979822,-5045906.43099501,-5210652.664083928,-5375398.897172861,-5540145.130261786,-5704891.363350704,-5869637.596439689,-6034383.829528533,-6199524.816227235,-6364279.358239807,-6529033.900252342,-6693788.44226478,-6858542.984277308,-7023297.526289731,-7188052.068302214,-7352806.610314816],[8960257.265737038,8794856.458352067,8629455.65096709,8464054.84358213,8298654.036197159,8133253.228812186,7967852.421427208,7802451.614042237,7637050.806657266,7471649.999272294,7306249.191887316,7140848.384502355,6975447.577117377,6810046.769732413,6644645.962347435,6479245.154962463,6313844.347577502,6148443.540192524,5983042.732807548,5817641.925422574,5652241.118037612,5486840.310652636,5321439.50326766,5156038.695882683,4990637.888497723,4825237.081112753,4659836.273727792,4494435.466342805,4329034.6589578595,4163633.8515728507,3998246.238497965,3832876.098444408,3667520.8713742476,3502178.281415863,3336851.754677659,3171534.483304221,3006224.7231607437,2840926.283918353,2675634.723762758,2510348.4556845296,2345071.54857973,2179796.330329869,2014530.9482889585,1849265.744967427,1684009.1364263147,1518752.993114408,1353502.7444225997,1188254.7802072763,1023008.7807722911,857768.1832143031,692527.5856563933,527290.6139816344,362056.6339937411,196822.65400590748,31591.73308603093,-133636.31980829313,-298864.3727027029,-464091.83996224403,-629314.6020185128,-794537.3640747704,-959760.1261310279,-1124981.6917011365,-1290199.7490171269,-1455417.8063330576,-1620635.8636489883,-1785853.9209649414,-1951071.9782808796,-2116319.460524816,-2281542.222581014,-2446764.9846373126,-2611987.746693559,-2777285.1661180593,-2942513.21901248,-3107741.271906782,-3272969.3248011805,-3438324.905727349,-3603558.885715142,-3768792.86570308,-3934026.8456909433,-4099260.8256787993,-4264690.341082893,-4429930.938640751,-4595171.536198787,-4760412.133756764,-4925652.7313147485,-5090893.328872748,-5256415.824189372,-5421663.78840483,-5586911.752620049,-5752159.716835372,-5917407.681050681,-6082655.645265959,-6247903.6094811335,-6413151.573696539,-6578798.029170707,-6744054.172482647,-6909310.315794542,-7074566.459106408,-7239822.602418385,-7405078.74573043],[8957655.623664025,8791755.511234738,8625855.398805449,8459955.28637617,8294055.173946873,8128155.061517597,7962254.949088306,7796354.836659023,7630454.724229734,7464554.611800447,7298654.499371158,7132754.386941878,6966854.274512587,6800954.162083298,6635054.04965401,6469153.937224722,6303253.824795447,6137353.712366151,5971453.599936862,5805553.4875075715,5639653.375078295,5473753.262649007,5307853.1502197115,5141953.037790417,4976052.925361146,4810152.8129318645,4644252.700502573,4478352.588073275,4312452.475644009,4146552.363214705,3980653.626401361,3814774.3247668445,3648912.460521335,3483064.2382259294,3317227.5096176974,3151400.498777272,2985587.828061562,2819781.7676949613,2653982.398517048,2488193.53842235,2322407.5785293914,2156631.0617568623,1990856.8762720842,1825090.2154607773,1659325.0724800862,1493567.3278686926,1327809.5832573213,1162059.1340303011,996309.4386886433,830562.7358898185,664820.2902744375,499077.84465895593,333339.6330577433,167603.69997169077,1867.766885522753,-163864.8446964696,-329594.9447699599,-495325.04484336823,-661054.576962404,-826779.470179271,-992504.3633961938,-1158229.256613128,-1323953.2677793056,-1489673.5309342518,-1655393.7940892726,-1821114.0572443232,-1986834.3203993253,-2152554.5835544094,-2318303.5091313235,-2484028.4023482874,-2649753.295565188,-2815478.188782055,-2981276.5459451005,-3147006.646018546,-3312736.7460919507,-3478466.846165456,-3644322.761008851,-3810058.694094997,-3975794.62718112,-4141530.5602672063,-4307266.4933534,-4473002.426439412,-4638937.969418898,-4804680.415034279,-4970422.86064972,-5136165.306265116,-5301907.751880512,-5467650.197496027,-5633678.374978341,-5799428.070320033,-5965177.765661716,-6130927.461003318,-6296677.156344995,-6462426.851686716,-6628176.547028422,-6794319.902700506,-6960077.6473118365,-7125835.3919230625,-7291593.136534624,-7457350.881146051],[8955053.981591012,8788654.564117411,8622255.146643804,8455855.729170222,8289456.311696598,8123056.894223006,7956657.476749406,7790258.05927581,7623858.6418021945,7457459.224328595,7291059.806854989,7124660.389381394,6958260.971907792,6791861.55443419,6625462.136960591,6459062.719486984,6292663.30201339,6126263.884539781,5959864.467066174,5793465.049592572,5627065.632118979,5460666.214645371,5294266.797171767,5127867.379698155,4961467.962224571,4795068.544750976,4628669.127277367,4462269.709803751,4295870.292330174,4129470.8748565484,3963071.457382936,3796682.071967941,3630309.14462664,3463950.258156495,3297607.148820553,3131275.7696151026,2964952.997968713,2798639.7378973924,2632336.9523581956,2466038.6211601943,2299750.1068248358,2133465.7931838688,1967187.9778887238,1800914.6859541386,1634646.5676530525,1468381.6626229845,1302122.3167121783,1135863.4878533408,969612.0613853037,803360.634917222,637112.9948924854,470868.70121955127,304624.4075467028,138384.745937448,-27853.140246946365,-194091.02643123642,-360325.51683723554,-526557.6640897207,-692789.8113422506,-859021.5762837715,-1025248.6006613448,-1191475.6250389293,-1357702.649416469,-1523929.2555355206,-1690151.7245296016,-1856374.1935236566,-2022596.662517745,-2188819.131511867,-2355041.6005059145,-2521291.8200592324,-2687518.84443675,-2853745.868814286,-3019972.893191863,-3186272.0201302804,-3352504.167382803,-3518736.314635366,-3684968.461887818,-3851200.6091404073,-4017562.4086713046,-4183800.2948556356,-4350038.181040004,-4516276.067224294,-4682513.953408703,-4848948.696311846,-5015192.989984691,-5181437.283657528,-5347681.577330396,-5513925.871003285,-5680170.16467607,-5846696.4238047,-6012947.850272723,-6179199.27674073,-6345450.703208752,-6511702.129676841,-6677953.556144938,-6844204.98261299,-7010456.409081072,-7177104.324739873,-7343363.670650773,-7509623.016561583],[8952452.339518007,8785553.61700008,8618654.894482164,8451756.171964262,8284857.449446335,8117958.726928424,7951060.004410496,7784161.281892577,7617262.559374666,7450363.836856751,7283465.114338826,7116566.391820913,6949667.669302991,6782768.946785082,6615870.224267164,6448971.501749237,6282072.77923134,6115174.05671341,5948275.334195483,5781376.611677573,5614477.889159658,5447579.166641742,5280680.44412382,5113781.721605901,4946882.999087986,4779984.276570092,4613085.554052161,4446186.831534236,4279288.10901634,4112389.386498399,3945490.663980473,3778591.941462567,3611711.2361299284,3444845.663696496,3277992.9406225756,3111151.040452959,2944323.0614081547,2777504.190343784,2610691.5061993357,2443890.277150519,2277094.066434117,2110304.4289702103,1943521.4107930604,1776742.2969752476,1609969.5806566589,1443200.0048658215,1276435.050167039,1109674.1029567495,942914.6840819456,776161.5264875107,609408.3688931614,442659.5577801727,275913.41604986787,109167.27431963012,-57574.04737940803,-224313.88666198403,-391053.72594462335,-557790.283336062,-724524.4777676985,-891258.6721993051,-1057992.8379264958,-1224721.9934646934,-1391451.1490028948,-1558180.304541085,-1724909.4600792415,-1891634.3298030011,-2058359.0046361573,-2225083.679469373,-2391808.3543024436,-2558533.0291356333,-2725257.7039687783,-2892013.5488465317,-3058742.704384733,-3225471.8599228747,-3392201.0154610984,-3559005.7831053063,-3725739.977536883,-3892474.171968501,-4059208.366400175,-4226070.029444072,-4392809.868726652,-4559549.708009176,-4726289.547291808,-4893029.386574417,-5059769.225857012,-5226709.261050001,-5393455.402780227,-5560201.544510677,-5726947.686240882,-5893693.827971213,-6060439.969701476,-6227471.092478171,-6394224.250072554,-6560977.407666989,-6727730.565261498,-6894483.72285594,-7061236.880450316,-7227990.038044602,-7394743.195639096,-7561895.1519772485],[8949850.697444985,8782452.669882763,8615054.642320523,8447656.614758302,8280258.587196058,8112860.559633824,7945462.532071592,7778064.504509369,7610666.476947132,7443268.4493849,7275870.421822662,7108472.394260438,6941074.366698201,6773676.339135976,6606278.311573743,6438880.2840115,6271482.256449278,6104084.228887042,5936686.201324801,5769288.173762574,5601890.146200348,5434492.118638111,5267094.091075878,5099696.063513637,4932298.035951414,4764900.0083891805,4597501.980826945,4430103.9532647105,4262705.925702507,4095307.8981402423,3927909.8705780134,3760511.843015777,3593119.3426287323,3425742.4001327697,3258382.287428323,3091035.1668315604,2923697.848143777,2756369.384996757,2589053.9477973543,2421743.901056178,2254441.7420718893,2087147.2654509842,1919856.5048585385,1752574.4313569628,1585294.0190165043,1418021.5603796355,1250750.5270990357,1083485.2351121455,916222.6866024658,748962.4180577844,581707.5293370411,414452.64061618596,247202.42455301434,79954.43476536497,-87293.55502241477,-254536.74689266458,-421778.5392735377,-589020.3316543363,-756259.1441931725,-923495.3858038373,-1090731.6274145357,-1257967.8690253012,-1425199.6485893615,-1592430.935288187,-1759662.2219869867,-1926893.5086858459,-2094121.3467545547,-2261348.227426842,-2428575.1080990396,-2595801.9887712523,-2763028.8694434687,-2930255.7501156814,-3097512.515577633,-3264743.8022764213,-3431975.0889752693,-3599206.3756741285,-3766511.4931858405,-3933747.7347965874,-4100983.9764072895,-4268220.218017969,-4435456.459628742,-4602823.348794028,-4770065.141174987,-4937306.933555767,-5104548.725936592,-5271790.518317476,-5439229.228230096,-5606477.218017869,-5773725.2078055665,-5940973.197593413,-6108221.187381081,-6275469.1771687865,-6442717.166956469,-6610252.685657218,-6777507.5743780285,-6944762.4630988315,-7112017.351819575,-7279272.24054037,-7446527.129261099,-7613782.017981999],[8947249.055371985,8779351.722765433,8611454.390158886,8443557.057552343,8275659.7249457855,8107762.392339241,7939865.059732697,7771967.727126152,7604070.394519601,7436173.061913056,7268275.729306497,7100378.396699956,6932481.064093407,6764583.731486864,6596686.398880318,6428789.06627376,6260891.733667221,6092994.401060672,5925097.068454117,5757199.735847575,5589302.403241031,5421405.070634488,5253507.738027929,5085610.405421371,4917713.072814831,4749815.740208283,4581918.407601757,4414021.074995187,4246123.742388662,4078226.409782095,3910329.077175526,3742431.744568987,3574534.411962459,3406649.5846838243,3238779.0761955567,3070920.7305050474,2903076.936783489,2735243.687630195,2567417.682783019,2399602.2703294605,2231794.4360420983,2063991.3471223451,1896198.2182107177,1728406.5657386743,1560624.8548755422,1392843.1440124176,1225070.6251230985,1057298.1343527213,889532.2174584456,721768.0676492676,554006.6897809319,386250.06993370876,218493.45008655638,50741.59521108493,-117008.2426341325,-284758.0804793015,-452503.35260250047,-620247.0980815478,-787990.8435606323,-955732.0994083434,-1123470.3881981634,-1291208.676987987,-1458946.965777792,-1626681.5660352483,-1794414.9838946909,-1962148.4017542154,-2129881.8196136393,-2297612.775384344,-2465341.8618955836,-2633070.9484068826,-2800800.0349181183,-2968529.1214294024,-3136258.2079407014,-3304015.7446299307,-3471749.1624894403,-3639482.580348924,-3807215.998208359,-3975021.2976246886,-4142759.5864144787,-4310497.8752042465,-4478236.163994081,-4645974.452783823,-4813840.735058121,-4981584.480537161,-5149328.226016223,-5317071.9714953005,-5484815.7169743255,-5652559.46245347,-5820502.729370311,-5988252.567215562,-6156002.405060731,-6323752.242905863,-6491502.080751032,-6659251.91859629,-6827284.583494581,-6995041.203341752,-7162797.823188819,-7330554.443035975,-7498311.062883131,-7666067.682730459],[8944647.413298966,8776250.775648106,8607854.137997242,8439457.500346389,8271060.86269551,8102664.225044656,7934267.587393796,7765870.949742936,7597474.312092067,7429077.6744412,7260681.036790336,7092284.399139477,6923887.761488614,6755491.123837754,6587094.48618689,6418697.848536018,6250301.210885168,6081904.573234303,5913507.93558343,5745111.297932572,5576714.66028171,5408318.022630855,5239921.384979988,5071524.747329116,4903128.109678259,4734731.4720274,4566334.834376542,4397938.196725666,4229541.559074832,4061144.921423942,3892748.2837730553,3724351.646122204,3555955.008471364,3387558.3708204944,3219180.5453343336,3050815.8569203,2882462.2723284513,2714118.2504783105,2545788.5589122195,2377465.2878543735,2209149.1579400543,2040843.1111570876,1872539.9315629061,1704246.9247133322,1535955.6907345913,1367672.6813488193,1199390.7231471688,1031116.7748870105,862842.8266269043,694575.9972057678,526310.2460972592,358047.499251239,189789.14827771857,21530.797304190695,-146722.9302458614,-314974.6161484271,-483226.3020510897,-651473.8645087183,-819719.5630860329,-987965.2616633363,-1156209.1489817724,-1324449.4849506654,-1492689.8209195621,-1660930.1568884477,-1829167.745802477,-1997403.2948225662,-2165638.8438426703,-2333874.3928628415,-2502108.6156921275,-2670339.9080424495,-2838571.200392779,-3006802.4927431084,-3175033.785093505,-3343265.077443775,-3511523.236003604,-3679758.785023771,-3847994.3340438195,-4016229.8830640055,-4184465.4320840985,-4352775.532390486,-4521015.868359402,-4689256.204328217,-4857496.540297203,-5025736.87626607,-5194107.726095803,-5362353.424673125,-5530599.123250365,-5698844.821827747,-5867090.520405047,-6035531.936837725,-6203783.622740351,-6372035.308642954,-6540286.994545482,-6708538.680448182,-6876790.36635083,-7045042.052253529,-7213578.294558138,-7381836.64553158,-7550094.996505171,-7718353.347478807],[8942045.771225957,8773149.828530785,8604253.8858356,8435357.943140434,8266462.000445245,8097566.057750069,7928670.115054897,7759774.172359711,7590878.229664535,7421982.286969356,7253086.344274177,7084190.401579006,6915294.458883818,6746398.516188641,6577502.573493464,6408606.630798283,6239710.688103115,6070814.745407936,5901918.80271274,5733022.8600175725,5564126.917322395,5395230.974627227,5226335.031932038,5057439.08923685,4888543.146541682,4719647.203846511,4550751.261151328,4381855.318456147,4212959.375760984,4044063.433065789,3875167.4903705996,3706271.547675416,3537375.60498026,3368479.6622850876,3199589.3704285603,3030712.6980185267,2861852.742306987,2693003.357894739,2524162.3774434626,2355332.46198985,2186511.4580042195,2017694.8751918226,1848889.926401144,1680087.5149369761,1511293.3849588968,1342502.2186852358,1173717.9107768126,1004935.4154212996,836160.0096714236,667384.6039214879,498616.5743543394,329849.2219462171,161084.84646884724,-7675.235631011426,-176435.31773099676,-345191.1518175639,-513944.685777653,-682698.2197377197,-851448.282611467,-1020195.9342869967,-1188943.5859625265,-1357690.2929133736,-1526432.6760613807,-1695175.059209358,-1863917.4423572645,-2032658.1878909692,-2201395.8680717275,-2370133.5482524894,-2538871.2284332067,-2707608.867678091,-2876342.3658674695,-3045075.864056837,-3213809.3622463048,-3382542.8604356386,-3551276.358625006,-3720009.8568144664,-3888772.669879306,-4057510.3500600867,-4226248.030240886,-4394985.710421618,-4563795.572724778,-4732537.955872662,-4901280.339020707,-5070022.722168632,-5238765.105316654,-5407634.877850927,-5576382.529526435,-5745130.181202024,-5913877.832877517,-6082625.484553151,-6251373.136228673,-6420318.37438,-6589071.908340037,-6757825.442300148,-6926578.97626029,-7095332.510220371,-7264086.0441804305,-7432839.578140371,-7601878.930127218,-7770639.012227178],[8939444.129152944,8770048.881413449,8600653.633673968,8431258.385934474,8261863.1381949745,8092467.890455475,7923072.6427159915,7753677.394976495,7584282.147237005,7414886.8994975,7245491.65175801,7076096.404018525,6906701.156279022,6737305.908539534,6567910.66080004,6398515.413060542,6229120.165321053,6059724.917581561,5890329.669842058,5720934.422102573,5551539.174363077,5382143.926623577,5212748.678884092,5043353.43114459,4873958.183405106,4704562.935665618,4535167.687926122,4365772.4401866235,4196377.192447148,4026981.9447076395,3857586.696968131,3688191.449228635,3518796.201489156,3349400.9537496604,3180005.7060101796,3010621.4089721795,2841249.6357361954,2671889.732355641,2502543.987203939,2333205.7964188885,2163875.3968630545,1994556.193232596,1825240.7605004348,1655934.8817742579,1486632.8520540074,1317337.5989473648,1148046.1495905742,978760.5431595109,809477.1927159503,640200.3294762522,470923.46623662487,301653.94890386984,132384.99519636482,-36881.268566221,-206143.08179257438,-375404.895018816,-544663.0695042424,-713918.4515217394,-883173.8335392773,-1052426.6069106944,-1221676.2116844282,-1390925.816458229,-1560175.4212320596,-1729419.9615302011,-1898664.391857218,-2067908.8221842833,-2237152.8923007324,-2406392.7036421783,-2575632.51498352,-2744872.326324966,-2914112.1376663744,-3083349.2353706323,-3252584.9393990748,-3421820.6434275396,-3591056.3474559523,-3760292.051484458,-3929527.7555128336,-4098790.81705625,-4268030.628397606,-4437270.439739015,-4606510.2510804385,-4775750.062421761,-4945064.13774427,-5114308.568071261,-5283552.998398282,-5452797.428725384,-5622041.859052427,-5791415.540576421,-5960665.145350099,-6129914.750123881,-6299164.354897633,-6468413.959671371,-6637663.564445138,-6807112.204152122,-6976367.586169705,-7145622.968187168,-7314878.350204632,-7484133.732222162,-7653389.11423967,-7822644.496257238],[8936842.487079937,8766947.934296126,8597053.381512322,8427158.828728523,8257264.275944704,8087369.723160895,7917475.170377087,7747580.617593278,7577686.064809469,7407791.512025653,7237896.959241845,7068002.406458044,6898107.853674233,6728213.300890424,6558318.748106615,6388424.195322802,6218529.642538996,6048635.08975519,5878740.536971375,5708845.98418757,5538951.431403761,5369056.878619952,5199162.325836138,5029267.773052323,4859373.220268529,4689478.667484719,4519584.114700915,4349689.5619171,4179795.009133311,4009900.456349479,3840005.9035656694,3670111.3507818654,3500216.7979980633,3330322.2452142406,3160427.692430457,2990533.1396466345,2820654.166111199,2650786.678115815,2480928.866491802,2311084.160256084,2141248.2471899,1971417.7758360095,1801599.4935394526,1631784.7859380543,1461977.9772764556,1292175.9429138862,1122379.5666787252,952587.4834505916,782800.5784968957,613016.0550310314,443237.73430164903,273459.4135721475,103688.12085457519,-66082.43415229023,-235850.84585415944,-405614.39020676166,-575377.9345594831,-745138.6833057776,-914895.9133807383,-1084653.143455714,-1254408.8374063298,-1424160.3952783681,-1593911.9531504102,-1763663.5110224,-1933411.3413571678,-2103157.818863321,-2272904.296369437,-2442650.7738756314,-2612393.801533848,-2782135.7440359406,-2951877.6865380034,-3121619.6290400364,-3291360.5165519044,-3461098.4264193512,-3630836.3362868465,-3800574.24615442,-3970312.156021841,-4140050.065889459,-4309787.975756973,-4479555.1690563895,-4649297.111558475,-4819039.054060467,-4988780.9965626,-5158594.41397389,-5328340.891480014,-5498087.368986204,-5667833.846492335,-5837580.323998518,-6007452.457822628,-6177204.0156947,-6346955.573566705,-6516707.131438628,-6686458.689310677,-6856210.247182682,-7025961.805054672,-7195913.426154025,-7365670.656228915,-7535427.8863038495,-7705185.116378792,-7874942.346453905],[8934240.845006924,8763846.9871788,8593453.12935067,8423059.271522567,8252665.413694428,8082271.555866295,7911877.698038185,7741483.840210065,7571089.9823819315,7400696.124553806,7230302.266725682,7059908.408897569,6889514.551069438,6719120.693241311,6548726.8354131915,6378332.977585064,6207939.119756941,6037545.261928818,5867151.4041006835,5696757.5462725675,5526363.688444444,5355969.830616323,5185575.972788191,5015182.114960063,4844788.25713195,4674394.399303828,4504000.541475703,4333606.683647575,4163212.8258194774,3992818.967991328,3822425.110163184,3652031.252335077,3481637.3945069797,3311243.5366788227,3140849.6788507253,2970455.821022587,2800063.7098008282,2629687.6418456323,2459323.825157387,2288968.8114153966,2118623.8770511076,1948289.7297565248,1777960.358682163,1607641.3589248285,1437326.9515047781,1267019.212907713,1096716.7875167131,926419.2881529704,756126.2202653363,585838.0167889558,415552.0023666285,245272.22414734587,74992.445928175,-95280.90979355946,-265553.066099979,-435823.8853947297,-606089.1608737968,-776354.4363528639,-946617.9932222553,-1116877.0713546164,-1287136.1494870037,-1457394.9740984738,-1627648.4850687683,-1797901.9960390143,-1968155.5070092306,-2138406.8155423477,-2308655.340227578,-2478903.864912834,-2649152.389598038,-2819399.1617468856,-2989643.235409599,-3159887.309072297,-3330131.382734999,-3500375.4563976564,-3670616.325117726,-3840856.4408243895,-4011096.556530919,-4181336.6722375117,-4351576.787944123,-4521816.903650645,-4692083.972036522,-4862328.045699194,-5032572.119361907,-5202816.193024635,-5373060.266687334,-5543377.30924695,-5713625.833932169,-5883874.358617432,-6054122.883302629,-6224371.40798793,-6394746.792235747,-6565000.303205878,-6735253.81417615,-6905507.325146452,-7075760.836116761,-7246014.347086981,-7416462.9622532725,-7586722.040385574,-7756981.1185180545,-7927240.196650475],[8931639.202933913,8760746.040061476,8589852.877189038,8418959.714316608,8248066.551444156,8077173.388571719,7906280.225699281,7735387.06282684,7564493.899954399,7393600.73708196,7222707.574209522,7051814.411337089,6880921.248464644,6710028.08559221,6539134.922719764,6368241.759847324,6197348.596974885,6026455.434102453,5855562.271229998,5684669.10835757,5513775.9454851365,5342882.782612693,5171989.619740252,5001096.456867801,4830203.293995373,4659310.131122936,4488416.968250503,4317523.805378055,4146630.642505629,3975737.479633171,3804844.3167607132,3633951.1538882963,3463057.9910158496,3292164.828143414,3121271.6652710047,2950378.5023985393,2779485.3395261094,2608597.544832971,2437721.8361754734,2266861.0768609066,2096008.300081905,1925163.1375891194,1754330.2441187557,1583501.506606765,1412681.789388705,1241867.25720058,1071058.5886680335,900255.3858624324,729456.7633701488,558662.3600347638,387872.8580357954,217085.03472257033,46303.79901359603,-124477.4366953075,-295253.14304065704,-466026.9006463997,-636800.3871881478,-807567.3937935196,-978334.4003989436,-1149100.9992535524,-1319861.9254433885,-1490622.8516332842,-1661383.777823113,-1832140.4810556285,-2002895.945124086,-2173651.409192592,-2344406.3840857,-2515156.955950059,-2685907.5278143436,-2856658.0996787213,-3027408.671543006,-3198154.9891045652,-3368901.193927873,-3539647.3987511955,-3710393.603574503,-3881138.6354943737,-4051880.9570399635,-4222623.278585609,-4393365.600131258,-4564107.921676915,-4734850.24322255,-4905592.564768091,-5076363.242161274,-5247109.446984582,-5417855.651807956,-5588601.856631279,-5759348.0614545345,-5930168.393236436,-6100918.965100668,-6271669.536965005,-6442420.108829312,-6613170.680693597,-6784048.939041674,-6954804.403110132,-7125559.867178731,-7296315.331247181,-7467070.795315713,-7637826.259384088,-7808777.120657176,-7979538.04684712],[8929037.560860908,8757645.092944149,8586252.625027401,8414860.157110645,8243467.689193882,8072075.221277128,7900682.753360375,7729290.285443636,7557897.817526864,7386505.349610111,7215112.881693357,7043720.413776606,6872327.945859851,6700935.477943103,6529543.010026341,6358150.542109577,6186758.074192837,6015365.606276082,5843973.138359314,5672580.670442569,5501188.202525818,5329795.734609067,5158403.266692304,4987010.7987755425,4815618.3308588,4644225.862942047,4472833.395025295,4301440.927108545,4130048.4591917954,3958655.9912750293,3787263.5233582444,3615871.055441497,3444478.587524757,3273086.119608024,3101693.651691241,2930301.183774486,2758908.715857735,2587516.247941008,2416132.0984605085,2244758.127283063,2073398.433226386,1902047.332491368,1730702.1262411997,1559369.7902765572,1388041.219609931,1216720.784931127,1045405.7030254342,874096.1045574099,702791.737951044,531491.9923301712,360195.9027589485,188905.1022372283,17615.15209905058,-153667.54109963402,-324950.2342984006,-496228.5788865313,-667503.9377918206,-838779.2966970503,-1010049.0889660306,-1181317.8266978078,-1352586.5644296259,-1523850.475647021,-1695113.2498943545,-1866376.0241416097,-2037636.3832389638,-2208893.80040567,-2380151.217572376,-2551408.634739157,-2722662.666030638,-2893915.285074096,-3065167.904117495,-3236420.523160871,-3407671.0051207654,-3578919.3411047645,-3750167.6770886816,-3921416.013072729,-4092664.3490566313,-4263909.884933721,-4435154.412318423,-4606398.939703096,-4777643.467087846,-4948887.994472504,-5120132.521857254,-5291377.049241893,-5462651.03692852,-5633899.372912519,-5805147.708896503,-5976396.044880465,-6147715.046898715,-6318967.665942125,-6490220.28498558,-6661472.904028907,-6832725.523072273,-7003978.1421157345,-7175358.898240656,-7346616.315407462,-7517873.732574113,-7689131.149740711,-7860388.56690751,-8031645.98407425],[8926435.918787891,8754544.145826822,8582652.372865748,8410760.599904694,8238868.82694361,8066977.053982539,7895085.281021476,7723193.5080604125,7551301.735099334,7379409.9621382635,7207518.189177195,7035626.416216126,6863734.643255054,6691842.870293991,6519951.097332921,6348059.32437185,6176167.551410774,6004275.77844971,5832384.005488629,5660492.23252757,5488600.459566502,5316708.686605433,5144816.913644359,4972925.140683277,4801033.367722217,4629141.594761144,4457249.8218000885,4285358.048839016,4113466.2758779507,3941574.502916867,3769682.7299557813,3597790.956994703,3425899.1840336677,3254007.4110726006,3082115.638111543,2910223.865150474,2738332.09218939,2566440.3192283306,2394548.5462672357,2222667.370683465,2050795.5359794945,1878935.894253824,1707085.9086437393,1535241.1882968023,1363408.3682299964,1191579.497691553,1019758.3455520272,847942.2889793478,676131.7605758794,504325.8437825218,332524.9750332199,160726.84843773022,-11065.250606607646,-182857.349650912,-354641.79619250074,-526425.9468809366,-698207.2173314244,-869984.1775360778,-1041761.1377407946,-1213534.2463912629,-1385304.7152493969,-1557075.184107583,-1728842.7219656259,-1900607.3442702964,-2072371.966574993,-2244136.1916187704,-2415895.5618837103,-2587654.932148706,-2759414.3024136275,-2931172.470469512,-3102927.136691954,-3274681.8029144444,-3446436.4691369273,-3618191.135359425,-3789941.75060286,-3961692.217747532,-4133442.6848920994,-4305193.152036738,-4476943.224505603,-4648689.957729366,-4820436.690953162,-4992183.4241767675,-5163930.15740063,-5335676.890624374,-5507423.623848148,-5679170.3570719585,-5850947.356338404,-6022697.823483087,-6194448.290627614,-6366198.757772334,-6538020.461141825,-6709775.127364226,-6881529.793586589,-7053284.459809221,-7225039.126031749,-7396793.792254277,-7568676.66983258,-7740436.040097401,-7912195.410362408,-8083954.780627444],[8923834.276714884,8751443.198709495,8579052.120704113,8406661.04269874,8234269.964693334,8061878.886687956,7889487.8086825665,7717096.730677189,7544705.6526718065,7372314.57466642,7199923.496661028,7027532.418655656,6855141.3406502595,6682750.262644879,6510359.184639495,6337968.1066341065,6165577.028628718,5993185.950623338,5820794.8726179395,5648403.794612565,5476012.716607191,5303621.638601795,5131230.560596413,4958839.4825910125,4786448.404585641,4614057.326580263,4441666.248574877,4269275.170569504,4096884.0925641134,3924493.0145587195,3752101.936553316,3579710.858547937,3407319.7805425785,3234928.7025371753,3062537.624531798,2890146.5465264097,2717755.46852104,2545364.390515635,2372973.312510226,2200582.23450486,2028203.3615018502,1855833.0493379068,1683473.4599432163,1511124.0285390206,1338779.2821480334,1166445.977979008,994116.3408517465,821794.4712514393,649477.0150623508,477165.55672338605,304857.70335699245,132555.71147898212,-39744.80292874947,-212038.2004956901,-384331.5980627127,-556618.966264762,-728904.5744430982,-901189.058375109,-1073467.6198792458,-1245746.1813833527,-1418022.866069235,-1590295.0660538338,-1762567.2660383545,-1934838.6643990315,-2107105.1347611174,-2279371.60512333,-2451638.075485468,-2623901.2295582667,-2796162.552921418,-2968423.876284644,-3140685.1996478364,-3312943.08266804,-3485199.796069566,-3657456.5094712116,-3829713.2228727154,-4001968.4224223644,-4174221.020727545,-4346473.619032841,-4518726.217338145,-4690978.8156434335,-4863229.91481838,-5035478.85388121,-5207727.792944051,-5379976.73200684,-5552225.671069689,-5724474.610132523,-5896723.549195312,-6068999.60208565,-6241252.200390853,-6413504.798696108,-6585757.39700143,-6758009.995306641,-6930334.064100951,-7102590.777502753,-7274847.490904339,-7447104.20430588,-7619360.917707421,-7791617.631108955,-7964002.253817327,-8136263.577180639],[8921232.634641873,8748342.25159217,8575451.868542474,8402561.485492783,8229671.1024430655,8056780.719393371,7883890.33634367,7710999.953293969,7538109.570244268,7365219.187194566,7192328.804144867,7019438.421095168,6846548.038045464,6673657.654995765,6500767.271946069,6327876.888896373,6154986.505846664,5982096.122796972,5809205.739747249,5636315.35669757,5463424.97364787,5290534.590598168,5117644.207548465,4944753.824498754,4771863.441449074,4598973.058399373,4426082.6753496695,4253192.292299973,4080301.9092502724,3907411.5262005776,3734521.1431508493,3561630.7601011507,3388740.377051471,3215849.9940017723,3042959.6109520867,2870069.2279023677,2697178.8448526785,2524288.46180298,2351398.078753272,2178507.6957035717,2005617.3126538582,1832740.070915619,1659870.6673582904,1487011.130294539,1314161.6921772435,1141316.4077948593,968482.6195236407,795651.7490903959,622829.1620293893,450009.8812743798,277197.4930000156,104387.31667424738,-68415.79833224043,-241218.91333867237,-414013.74743014574,-586808.4435197413,-759599.051316712,-932386.1169847436,-1105173.1826527677,-1277954.264821265,-1450734.4276248403,-1623514.5904284716,-1796288.8791109137,-1969062.8102217913,-2141836.7413327172,-2314606.621366862,-2487374.9397864565,-2660143.258206088,-2832910.803429216,-3005674.079890717,-3178437.3563521057,-3351200.6328135766,-3523963.1230022795,-3696721.8835829087,-3869480.644163545,-4042239.404744275,-4214998.165324841,-4387754.086029004,-4560508.815494917,-4733263.544960786,-4906018.274426721,-5078773.003892608,-5251525.428487487,-5424276.573389374,-5597027.7182912305,-5769778.863193132,-5942530.008094944,-6115281.152996942,-6288032.297898732,-6460810.839620002,-6633565.569085918,-6806320.298551723,-6979075.028017633,-7151829.757483602,-7324655.8557768315,-7497414.61635749,-7670173.3769381195,-7842932.137518749,-8015690.898099504,-8188449.658680253],[8918630.992568852,8745241.304474846,8571851.61638083,8398461.92828682,8225072.240192793,8051682.552098782,7878292.864004767,7704903.175910762,7531513.487816732,7358123.79972272,7184734.1116287,7011344.423534691,6837954.7354406705,6664565.047346659,6491175.359252645,6317785.671158626,6144395.983064607,5971006.294970594,5797616.606876569,5624226.918782565,5450837.230688558,5277447.542594537,5104057.854500518,4930668.166406495,4757278.478312495,4583888.79021849,4410499.102124464,4237109.41403044,4063719.725936439,3890330.0378424134,3716940.349748371,3543550.6616543587,3370160.973560361,3196771.285466347,3023381.5973723475,2849991.9092783295,2676602.2211842984,2503212.5330903046,2329822.8449962437,2156433.1569022574,1983043.4688082207,1809653.7807142157,1636277.4989248142,1462908.3900405802,1289548.905307863,1116198.8995583765,942852.5964228734,769518.2928638496,596185.722407598,422862.4178857915,249540.88761556894,76227.56940560788,-97085.31626554951,-270389.55440053716,-443693.79253560305,-616991.8914097995,-790287.8860221282,-963582.0513480753,-1136870.5745059028,-1310159.0976637006,-1483444.11236218,-1656725.8764652684,-1830007.6405682787,-2003286.1544207856,-2176561.8166580275,-2349837.4788953066,-2523111.804087486,-2696381.9705645703,-2869652.137041617,-3042922.3035186864,-3216189.5130564123,-3389454.742616046,-3562719.972175736,-3735985.2017354853,-3909248.06545439,-4082508.8732141852,-4255769.680973843,-4429030.4887336455,-4602291.296493381,-4775548.27427819,-4948805.134904835,-5122061.995531328,-5295318.856157884,-5468575.716784425,-5641829.765512824,-5815083.116253786,-5988336.466994606,-6161589.817735635,-6334843.168476507,-6508096.519217491,-6681349.869958423,-6854630.601796858,-7027887.462423392,-7201144.3230500445,-7374401.183676697,-7547658.044303209,-7720985.8361689225,-7894246.643928602,-8067507.451688416,-8240768.2594482],[8916029.350495847,8742140.357357517,8568251.364219202,8394362.371080866,8220473.377942521,8046584.384804187,7872695.391665867,7698806.398527538,7524917.405389203,7351028.412250869,7177139.419112542,7003250.425974211,6829361.432835879,6655472.439697545,6481583.446559218,6307694.453420891,6133805.4602825595,5959916.467144235,5786027.474005885,5612138.480867564,5438249.48772924,5264360.494590903,5090471.501452559,4916582.508314233,4742693.515175916,4568804.522037588,4394915.528899258,4221026.535760926,4047137.542622596,3873248.5494842567,3699359.5563459042,3525470.5632075835,3351581.5700692646,3177692.5769309253,3003803.5837926157,2829914.59065428,2656025.597515965,2482136.604377616,2308247.611239273,2134358.618100954,1960469.6249626148,1786580.6318242867,1612691.6386859454,1438815.645529382,1264946.217384871,1091086.7849831134,917235.6506824456,743389.1194183789,569552.9979996905,395718.26080325246,221894.23882077634,48070.21683819592,-125744.2140596956,-299558.46220497787,-473365.5567260049,-647170.9179895483,-820972.6324348785,-994769.9255697876,-1168567.2187046893,-1342357.9470065907,-1516147.9276541546,-1689937.1625020467,-1863720.5279045328,-2037503.893306978,-2211286.8919833526,-2385064.2853469923,-2558841.6787106246,-2732619.0720743574,-2906392.697457496,-3080164.7119919993,-3253936.726526454,-3427708.741060976,-3601476.0350765474,-3775243.2177344374,-3949010.40039232,-4122777.583050236,-4296541.196622875,-4470304.051561721,-4644066.906500623,-4817829.761439409,-4991591.995382838,-5165350.987169959,-5339109.978957251,-5512868.970744424,-5686627.9625316635,-5860386.954318844,-6034142.92589432,-6207898.482474364,-6381654.039054297,-6555409.595634259,-6729165.152214356,-6902920.7087943405,-7076676.265374273,-7250458.888616495,-7424217.880403623,-7597976.87219087,-7771735.863978073,-7945494.855765164,-8119253.8475524485,-8293086.860216171],[8913427.708422836,8739039.410240196,8564651.11205755,8390262.813874902,8215874.515692258,8041486.2175095985,7867097.919326967,7692709.621144323,7518321.322961662,7343933.024779028,7169544.726596376,6995156.428413736,6820768.130231084,6646379.832048442,6471991.533865789,6297603.235683151,6123214.937500502,5948826.639317861,5774438.341135194,5600050.042952566,5425661.744769928,5251273.446587274,5076885.1484046215,4902496.850221969,4728108.552039331,4553720.253856695,4379331.955674052,4204943.657491406,4030555.359308766,3856167.0611261055,3681778.762943445,3507390.4647607952,3333002.166578157,3158613.8683955185,2984225.570212869,2809837.272030223,2635448.973847607,2461060.6756649483,2286672.377482284,2112284.079299625,1937895.781116983,1763507.4829343222,1589119.1847517006,1414730.8865690492,1240354.5107293949,1065984.1493911222,891624.7693203166,717271.94554938,542925.1861568429,368586.734931089,194251.49435332045,19924.624834172428,-154400.1146091111,-328717.8573958501,-503033.035476733,-677343.8053084835,-851650.2897006497,-1025955.9705051668,-1200254.562162757,-1374553.1538202986,-1548848.2344868146,-1723139.6726242155,-1897431.1107616052,-2071718.3819426224,-2246003.34864451,-2420288.3153464273,-2594570.2162886895,-2768849.3407787606,-2943128.4652687237,-3117407.1204653457,-3291680.9830572605,-3465954.845649209,-3640228.708241161,-3814501.233733427,-3988770.3694895543,-4163039.505245745,-4337308.641001809,-4511577.614389811,-4685842.516507745,-4860107.418625686,-5034372.320743617,-5208637.222861454,-5382901.101756573,-5557162.224704415,-5731423.347652249,-5905684.470600128,-6079945.593547933,-6254206.716495819,-6428464.909632064,-6602722.672051169,-6776980.434470229,-6951238.1968893185,-7125495.959308378,-7299753.721727386,-7474011.484146483,-7648269.246565618,-7822556.823026381,-7996817.945974171,-8171079.06892211,-8345340.191869959],[8910826.066349825,8735938.463122867,8561050.859895898,8386163.256668949,8211275.653441982,8036388.050215015,7861500.446988066,7686612.843761107,7511725.240534142,7336837.637307176,7161950.034080221,6987062.430853257,6812174.827626293,6637287.224399335,6462399.621172366,6287512.017945413,6112624.414718446,5937736.81149149,5762849.208264515,5587961.6050375635,5413074.001810596,5238186.398583649,5063298.7953566825,4888411.192129711,4713523.5889027575,4538635.9856758,4363748.382448841,4188860.779221885,4013973.1759949215,3839085.5727679506,3664197.9695409667,3489310.36631402,3314422.7630870733,3139535.1598601155,2964647.5566331595,2789759.9534062,2614872.350179216,2439984.746952269,2265097.1437253095,2090209.5404983405,1915321.937271366,1740434.3340444155,1565546.7308174558,1390659.1275904812,1215771.5243635438,1040894.0945248455,866022.1860592887,691162.8583194613,516307.7841593437,341460.79663820565,166619.5036581643,-8216.221021935344,-183046.42407387868,-357871.8809778653,-532693.3606030568,-707509.4686193131,-882324.3001481593,-1057131.9076687992,-1231939.5151894875,-1406741.795801025,-1581541.6859811991,-1756341.4369466342,-1931134.3325737529,-2105927.2282008342,-2280719.4385796413,-2455506.006581001,-2630292.574582331,-2805079.142583724,-2979860.4650995918,-3154641.3207159787,-3329422.1763323843,-3504200.950237427,-3678976.6608867757,-3853752.3715362065,-4028528.0821855403,-4203301.42744121,-4378072.516295504,-4552843.605149906,-4727614.694004312,-4902385.075811919,-5077152.025108948,-5251918.974405862,-5426685.923703007,-5601452.872999959,-5776218.732772879,-5950981.986881405,-6125745.240989819,-6300508.495098405,-6475271.749206789,-6650035.003315359,-6824795.716726147,-6999555.684984252,-7174315.653242312,-7349075.621500492,-7523835.589758709,-7698595.558016837,-7873355.526274867,-8048115.494532995,-8222904.290291727,-8397667.544400379],[8908224.424276812,8732837.51600554,8557450.60773426,8382063.699462991,8206676.791191708,8031289.8829204235,7855902.9746491555,7680516.066377886,7505129.1581066,7329742.249835324,7154355.341564052,6978968.43329278,6803581.525021497,6628194.6167502245,6452807.708478941,6277420.80020767,6102033.891936383,5926646.983665122,5751260.07539383,5575873.167122562,5400486.25885129,5225099.350580016,5049712.44230873,4874325.534037441,4698938.625766181,4523551.717494908,4348164.809223637,4172777.9009523466,3997390.9926810935,3822004.0844097957,3646617.1761385053,3471230.2678672485,3295843.359595962,3120456.451324707,2945069.5430534203,2769682.6347821373,2594295.7265108936,2418908.8182395846,2243521.9099683054,2068135.0016970336,1892748.0934257228,1717361.1851544864,1541974.2768831886,1366587.368611928,1191200.4603406116,1015813.5520693734,840434.3969156742,665061.9219375327,489701.05198058486,314343.1665121205,138995.95086253062,-36348.69581927359,-211684.90460154787,-387018.90790339187,-562345.0822681673,-737670.7236810997,-912987.7616328858,-1088304.7995846495,-1263615.771894075,-1438924.5025432408,-1614231.6264845952,-1789532.8151874803,-1964834.003890399,-2140131.907502789,-2315426.260619674,-2490720.6137365364,-2666011.867116295,-2841300.0364171676,-3016588.2057179175,-3191875.0516732037,-3367157.6384159736,-3542440.22515874,-3717722.8119014762,-3893002.172239322,-4068279.730946105,-4243557.289652929,-4418834.848359704,-4594109.433541868,-4769382.475494511,-4944655.517447118,-5119928.559399746,-5295200.725950375,-5470469.722426519,-5645738.718902588,-5821007.715378664,-5996276.711854786,-6171544.888431743,-6346810.273700923,-6522075.658970028,-6697341.044239186,-6872606.429508328,-7047871.814777456,-7223135.347176351,-7398397.521273635,-7573659.695370831,-7748921.869468026,-7924184.043565139,-8099446.21766232,-8274708.391759545,-8449970.56585683],[8905622.782203803,8729736.568888208,8553850.355572628,8377964.142257038,8202077.928941434,8026191.71562584,7850305.502310256,7674419.288994659,7498533.075679068,7322646.862363474,7146760.649047893,6970874.435732303,6794988.222416701,6619102.009101114,6443215.795785511,6267329.582469936,6091443.369154337,5915557.155838743,5739670.942523142,5563784.729207558,5387898.51589197,5212012.30257638,5036126.08926078,4860239.875945183,4684353.662629603,4508467.449314013,4332581.2359984275,4156695.022682842,3980808.8093672562,3804922.5960516483,3629036.3827360366,3453150.169420449,3277263.9561048653,3101377.7427892685,2925491.529473679,2749605.3161581103,2573719.1028425246,2397832.889526926,2221946.6762113106,2046060.462895723,1870174.2495801374,1694288.0362645257,1518401.8229489326,1342515.6096333563,1166629.3963177502,990743.1830021478,814856.9696865976,638975.4179018848,463103.3022214845,287239.3503036499,111380.12722582743,-64469.35117031261,-240316.79494854435,-416154.55638546124,-591991.2492699474,-767819.7184798941,-943646.6103054434,-1119467.9145173617,-1295285.8824045658,-1471101.8823764324,-1646911.7361542135,-1822721.5899320506,-1998526.541439008,-2174329.0286645256,-2350131.515890043,-2525928.2080180384,-2701724.0186246745,-2877519.829231333,-3053310.7008508295,-3229100.471451126,-3404890.2420512997,-3580677.4183686487,-3756461.736237798,-3932246.0541069694,-4108030.371976096,-4283810.786470905,-4459590.193235125,-4635369.599999454,-4811149.006763734,-4986925.252035517,-5162700.247086436,-5338475.242137216,-5514250.23718816,-5690024.564805172,-5865795.608460374,-6041566.652115606,-6217337.695770763,-6393108.739425972,-6568879.56873326,-6744647.085163079,-6920414.601592779,-7096182.118022524,-7271949.6344522685,-7447717.150882125,-7623483.8009830415,-7799248.180919215,-7975012.560855441,-8150776.940791562,-8326541.320727915,-8502305.70066426],[8903021.1401308,8726635.621770896,8550250.103410978,8373864.585051084,8197479.066691162,8021093.548331253,7844708.029971353,7668322.511611452,7491936.993251538,7315551.4748916235,7139165.9565317165,6962780.438171817,6786394.91981191,6610009.401452005,6433623.883092089,6257238.364732196,6080852.846372281,5904467.328012378,5728081.809652463,5551696.291292562,5375310.772932662,5198925.254572749,5022539.736212839,4846154.217852923,4669768.699493031,4493383.181133106,4316997.662773224,4140612.144413309,3964226.6260534134,3787841.107693497,3611455.589333568,3435070.070973657,3258684.552613765,3082299.034253873,2905913.515893955,2729527.9975340534,2553142.479174165,2376756.9608142264,2200371.4424543176,2023985.924094418,1847600.4057344925,1671214.8873745948,1494829.3690146878,1318443.8506547809,1142058.3322948813,965672.8139349744,789287.2955750749,612901.7772151344,436517.15748359635,260145.40110079572,83777.75328871608,-92581.41745822504,-268935.1094602272,-445282.7813669145,-621625.1763738655,-797962.3533604592,-974295.7896131985,-1150623.3988994285,-1326949.9272727892,-1503268.8250954598,-1679587.722918149,-1855901.216022294,-2032212.1929285862,-2208522.8647357896,-2384826.6504839696,-2561130.4362321757,-2737433.0703959577,-2913730.338492371,-3090027.6065887325,-3266324.5678833947,-3442615.939783089,-3618907.3116827644,-3795198.6835824586,-3971486.7095696367,-4147772.7585651167,-4324058.80756072,-4500344.856556173,-4676626.792932283,-4852908.047754027,-5029189.302575681,-5205470.557397433,-5381748.88292212,-5558025.831071325,-5734302.779220313,-5910579.727369472,-6086856.592376381,-6263129.6832106635,-6439402.774044916,-6615675.864879154,-6791948.955713414,-6968222.046547726,-7144492.421267673,-7320762.068858005,-7497031.716448523,-7673301.364039041,-7849571.01162938,-8025840.659219854,-8202107.663920939,-8378374.249696307,-8554640.83547175],[8900419.49805778,8723534.674653564,8546649.851249335,8369765.027845125,8192880.204440891,8015995.381036674,7839110.557632452,7662225.734228232,7485340.910823999,7308456.087419785,7131571.264015559,6954686.440611344,6777801.617207112,6600916.793802901,6424031.970398666,6247147.146994454,6070262.323590228,5893377.500186006,5716492.676781776,5539607.853377559,5362723.029973338,5185838.206569117,5008953.383164887,4832068.559760662,4655183.736356451,4478298.912952224,4301414.089548014,4124529.2661437932,3947644.44273958,3770759.6193353403,3593874.7959311027,3416989.9725268763,3240105.1491226684,3063220.3257184476,2886335.5023142565,2709450.6789100077,2532565.855505768,2355681.0321015622,2178796.208697336,2001911.385293128,1825026.561888883,1648141.7384846471,1471256.9150804207,1294372.0916762017,1117487.2682720087,940602.44486776,763717.6214635707,586832.7980593331,409947.97465511784,233063.15125088394,56188.21857564896,-120683.17850945517,-297542.85748027265,-474401.3240071237,-651249.2240424119,-828096.7645666003,-1004934.4256554805,-1181772.0867442712,-1358601.6224149764,-1535429.9491619281,-1712253.6276572682,-1889073.4554154538,-2065892.9421474077,-2242705.0421822295,-2419517.1422171406,-2596326.869348742,-2773131.9536195435,-2949937.037890453,-3126739.5733395256,-3303538.29892575,-3480337.024511926,-3657134.61071397,-3833927.583913114,-4010720.557112243,-4187513.530311335,-4364302.925276116,-4541090.705397982,-4717878.485519875,-4894666.265641779,-5071450.191623475,-5248233.294502646,-5425016.397381678,-5601799.500260957,-5778580.326201737,-5955359.227449104,-6132138.128696457,-6308917.029943749,-6485695.931191064,-6662471.946677208,-6839247.0846905485,-7016022.222703941,-7192797.360717252,-7369572.49873057,-7546346.282014981,-7723118.060766056,-7899889.839517191,-8076661.618268207,-8253433.39701917,-8430205.175770283,-8606975.970279142],[8897817.855984764,8720433.727536235,8543049.599087697,8365665.470639169,8188281.342190627,8010897.213742079,7833513.0852935575,7656128.956845009,7478744.828396463,7301360.699947932,7123976.571499392,6946592.4430508595,6769208.314602319,6591824.186153798,6414440.057705244,6237055.929256719,6059671.8008081755,5882287.672359637,5704903.54391109,5527519.415462558,5350135.287014024,5172751.15856549,4995367.030116941,4817982.901668403,4640598.773219871,4463214.644771332,4285830.516322805,4108446.387874268,3931062.259425748,3753678.1309771854,3576294.002528649,3398909.8740800917,3221525.7456315756,3044141.6171830427,2866757.488734493,2689373.3602859545,2511989.2318374403,2334605.103388883,2157220.974940341,1979836.84649181,1802452.7180432733,1625068.5895947106,1447684.461146187,1270300.3326976337,1092916.204249151,915532.075800579,738147.9473520517,560763.8189035021,383379.69045500085,205995.5620064251,28611.43355794996,-148768.24535429478,-326139.2831415683,-503504.1928403564,-680863.2589253187,-858216.1229749694,-1035564.2511387952,-1212907.4661548547,-1390245.6113458686,-1567581.280851934,-1744910.3250596821,-1922239.369267486,-2099561.0477836914,-2276881.8054773547,-2454200.137693029,-2631513.360856425,-2808826.5840198435,-2986136.068052355,-3163442.4508457594,-3340748.8336392604,-3518051.906242229,-3695352.08931816,-3872652.2723940797,-4049951.0587423705,-4227245.633240845,-4404540.207739465,-4581834.782237958,-4759126.065488074,-4936415.576736361,-5113705.087984595,-5290994.599232875,-5468280.982544348,-5645565.933481045,-5822850.884417623,-6000135.835354179,-6177419.581874266,-6354700.436219789,-6531981.290565379,-6709262.144910976,-6886542.999256529,-7063822.398860127,-7241099.584052563,-7418376.769244939,-7595653.954437427,-7772931.139629945,-7950208.324822411,-8127482.5773165375,-8304756.4872282,-8482030.397139952,-8659304.307051674],[8895216.213911744,8717332.780418904,8539449.346926048,8361565.913433208,8183682.479940341,8005799.046447492,7827915.612954653,7650032.1794618,7472148.745968937,7294265.312476079,7116381.878983229,6938498.445490381,6760615.0119975265,6582731.578504683,6404848.145011819,6226964.711518975,6049081.278026115,5871197.844533271,5693314.411040409,5515430.977547559,5337547.544054705,5159664.110561857,4981780.677069002,4803897.243576134,4626013.810083296,4448130.37659044,4270246.943097597,4092363.5096047465,3914480.0761119034,3736596.642619027,3558713.209126165,3380829.775633309,3202946.342140468,3025062.9086476173,2847179.4751547966,2669296.041661922,2491412.6081690732,2313529.174676208,2135645.74118335,1957762.307690505,1779878.8741976488,1601995.4407047872,1424112.0072119571,1246228.573719073,1068345.1402262524,890461.7067333944,712578.2732405216,534694.8397476599,356811.4062548354,178927.97276197374,1044.5392691642046,-176838.8942237571,-354722.32771660015,-532594.6691782624,-710465.347667899,-888324.4372924417,-1066183.4510464333,-1244031.8344569504,-1421880.1907493286,-1599720.1041518413,-1777558.733445108,-1955392.1358789802,-2133221.8975474685,-2311050.500022821,-2488872.1876518987,-2666693.8752810284,-2844511.8257528096,-3022326.1720448397,-3200140.5183367617,-3377950.460846495,-3555758.1421625763,-3733565.8234787136,-3911370.0691039823,-4089171.709669728,-4266973.350235399,-4444773.9119684845,-4622570.087766368,-4800366.263564404,-4978162.439362489,-5155956.130205516,-5333747.372580133,-5511538.614954777,-5689329.85732948,-5867119.165695094,-6044905.964689195,-6222692.763683207,-6400479.562677197,-6578266.361671329,-6756049.457383499,-6933832.264827304,-7111615.0722711235,-7289397.879714906,-7467180.687158704,-7644960.272130959,-7822739.504502542,-8000518.736874096,-8178297.969245613,-8356077.201616995,-8533855.61850956,-8711631.659582011],[8892614.571838746,8714231.833301589,8535849.094764406,8357466.356227249,8179083.617690069,8000700.879152903,7822318.140615742,7643935.402078582,7465552.663541403,7287169.925004238,7108787.186467064,6930404.447929902,6752021.709392734,6573638.970855579,6395256.232318392,6216873.493781236,6038490.755244055,5860108.016706899,5681725.278169717,5503342.539632552,5324959.801095396,5146577.062558224,4968194.324021055,4789811.58548388,4611428.846946722,4433046.108409543,4254663.369872395,4076280.631335225,3897897.8927980606,3719515.1542608794,3541132.4157237057,3362749.6771865226,3184366.9386493787,3005984.200112205,2827601.461575033,2649218.723037882,2470835.9845007192,2292453.2459635567,2114070.5074263513,1935687.7688892148,1757305.0303520076,1578922.2918148357,1400539.5532776788,1222156.81474052,1043774.0762033463,865391.3376661986,687008.5991290174,508625.8605918363,330243.1220546961,151860.38351750374,-26522.355019588023,-204905.09355683252,-383287.83209400997,-561670.5706311278,-740049.3366195709,-918419.6558114514,-1096785.5109976605,-1275144.472420629,-1453499.874032218,-1631848.4584530815,-1810195.5651622117,-1988534.6785576828,-2166873.7919531167,-2345205.8607488684,-2523536.339878112,-2701864.4296974577,-2880187.047261972,-3058509.6648266315,-3236828.0063268356,-3415143.4757473804,-3593458.945167914,-3771770.0477312207,-3950079.0275699757,-4128388.0074087717,-4306694.061924759,-4484997.159980282,-4663300.258035678,-4841603.170392308,-5019900.947489731,-5198198.724587243,-5376496.501684688,-5554793.119428456,-5733086.09292952,-5911379.066430494,-6089672.039931528,-6267964.741075635,-6446253.388127066,-6624542.035178646,-6802830.682230048,-6981119.329281509,-7159406.2909401655,-7337691.051482156,-7515975.812024191,-7694260.572566226,-7872545.333108388,-8050829.148925729,-8229110.428476289,-8407391.708026849,-8585672.987577528,-8763954.267128162],[8890012.929765737,8711130.88618426,8532248.842602775,8353366.799021297,8174484.755439801,7995602.711858322,7816720.6682768455,7637838.624695368,7458956.581113863,7280074.537532389,7101192.493950911,6922310.450369421,6743428.406787938,6564546.363206468,6385664.319624964,6206782.276043494,6027900.232462004,5849018.188880526,5670136.145299029,5491254.101717551,5312372.058136072,5133490.014554597,4954607.970973105,4775725.927391608,4596843.883810146,4417961.840228649,4239079.79664718,4060197.753065698,3881315.709484229,3702433.665902734,3523551.622321222,3344669.5787397474,3165787.535158271,2986905.4915767964,2808023.4479953237,2629141.404413834,2450259.360832354,2271377.3172508646,2092495.2736693546,1913613.2300878894,1734731.186506413,1555849.1429249197,1376967.0993434042,1198085.055761937,1019203.0121804588,840320.9685990103,661438.925017491,482556.8814360127,303674.837854553,124792.79427306727,-54089.24930835515,-232971.29288994148,-411853.33647142723,-590735.380052831,-769617.4236343876,-948499.4672158398,-1127373.2453597002,-1306243.2052538544,-1485105.389132902,-1663964.2982248664,-1842817.1824140027,-2021665.9949634895,-2200511.5918746963,-2379351.189372264,-2558190.786869876,-2737022.455461826,-2915853.652051799,-3094682.079114005,-3273505.626613997,-3452329.174114026,-3631148.679415133,-3809965.27196417,-3988781.864513252,-4167594.828706596,-4346405.107067969,-4525215.38542942,-4704023.884704623,-4882828.440249894,-5061632.995795131,-5240437.551340383,-5419238.212410886,-5598037.590807714,-5776836.969204679,-5955636.3476015255,-6134431.737784348,-6313226.442411736,-6492021.1470390335,-6670815.851666547,-6849608.203794748,-7028398.6989036575,-7207189.194012657,-7385979.689121455,-7564770.184230335,-7743557.650530003,-7922344.364170305,-8101131.077810623,-8279917.7914508805,-8458704.505091041,-8637489.541166387,-8816272.8678962],[8887411.287692726,8708029.93906693,8528648.59044113,8349267.24181534,8169885.893189532,7990504.544563737,7811123.195937941,7631741.847312141,7452360.498686337,7272979.150060545,7093597.80143474,6914216.452808948,6734835.104183144,6555453.755557359,6376072.406931543,6196691.058305754,6017309.709679945,5837928.361054159,5658547.012428348,5479165.663802563,5299784.315176763,5120402.966550965,4941021.617925157,4761640.269299348,4582258.920673577,4402877.572047764,4223496.22342197,4044114.874796182,3864733.526170388,3685352.1775445677,3505970.828918757,3326589.4802929647,3147208.131667167,2967826.7830413785,2788445.434415577,2609064.085789798,2429682.7371639907,2250301.388538204,2070920.0399123859,1891538.6912866011,1712157.3426607698,1532775.9940349814,1353394.6454091892,1174013.296783369,994631.9481575824,815250.5995318145,635869.2509059906,456487.9022801854,277106.55365442485,97725.20502859354,-81656.14359715581,-261037.49222302064,-440418.84084880725,-619800.1894745976,-799181.538100414,-978562.886726167,-1157944.235352017,-1337325.5839777514,-1516695.7169089578,-1696065.3175054565,-1875425.0579442494,-2054783.9147053175,-2234135.4033099636,-2413484.4439879246,-2592829.5549958795,-2772169.636595696,-2951509.7181954645,-3130841.920017779,-3310173.8340684883,-3489503.448272366,-3668827.9257078245,-3848152.403143339,-4027473.8450175114,-4206791.560695145,-4386109.27637272,-4565424.803772509,-4744736.380656544,-4924047.957540665,-5103359.534424722,-5282665.550478581,-5461971.563513637,-5641277.576548576,-5820581.882529728,-5999882.862226054,-6179183.841922462,-6358484.821618766,-6537784.307144627,-6717080.742898464,-6896377.178652123,-7075673.6144059375,-7254970.050159685,-7434262.754858553,-7613555.098024845,-7792847.441191278,-7972139.784357667,-8151432.061970852,-8330720.728709377,-8510009.395447731,-8689298.06218639,-8868586.72892493],[8884809.645619718,8704928.991949601,8525048.338279486,8345167.684609396,8165287.030939256,7985406.377269148,7805525.723599037,7625645.069928925,7445764.4162588045,7265883.762588685,7086003.108918576,6906122.455248466,6726241.801578357,6546361.1479082415,6366480.494238113,6186599.840568014,6006719.186897896,5826838.533227786,5646957.879557659,5467077.22588756,5287196.572217449,5107315.91854733,4927435.26487721,4747554.6112070875,4567673.957536995,4387793.303866874,4207912.650196765,4028031.9965266623,3848151.3428565506,3668270.689186411,3488390.035516288,3308509.3818461765,3128628.7281760797,2948748.0745059643,2768867.4208358657,2588986.767165728,2409106.113495631,2229225.4598255157,2049344.8061553761,1869464.1524852794,1689583.4988151547,1509702.845145028,1329822.1914749108,1149941.5378048085,970060.8841346912,790180.2304645889,610299.5767944492,430418.9231243804,250538.2694542706,70657.61578411981,-109223.03788591921,-289103.6915560961,-468984.34522622824,-648864.9988963194,-828745.6525664479,-1008626.3062364906,-1188506.9599066824,-1368387.6135767214,-1548268.2672468647,-1728147.5099686272,-1908016.7512673922,-2087885.713001676,-2267744.517431751,-2447603.3218617737,-2627454.5367200077,-2807303.80552651,-2987149.4545258395,-3166990.0202279054,-3346830.5859299116,-3526664.254416641,-3706496.885928154,-3886328.5371726006,-4066153.944543589,-4245979.351914566,-4425803.503134105,-4605622.341940291,-4785441.180746395,-4965259.972929101,-5145072.848335773,-5324885.723742459,-5504698.599149242,-5684508.49066633,-5864315.96119114,-6044123.431715943,-6223930.902240686,-6403734.538842164,-6583537.11983794,-6763339.700833753,-6943142.281829581,-7122941.967890598,-7302740.134770699,-7482538.301650785,-7662336.468530916,-7842134.20304326,-8021928.394267097,-8201722.585490845,-8381516.776714608,-8561310.967938326,-8741104.905641325,-8920895.52547811],[8882208.003546707,8701828.044832274,8521448.086117845,8341068.127403431,8160688.168688984,7980308.209974557,7799928.251260145,7619548.292545708,7439168.333831277,7258788.375116844,7078408.416402418,6898028.457687991,6717648.498973563,6537268.540259136,6356888.581544693,6176508.622830271,5996128.66411583,5815748.705401416,5635368.7466869755,5454988.787972554,5274608.829258123,5094228.870543703,4913848.91182926,4733468.9531148225,4553088.994400417,4372709.035685977,4192329.0769715607,4011949.118257126,3831569.1595427077,3651189.200828254,3470809.242113821,3290429.2833993845,3110049.3246849813,2929669.3659705278,2749289.407256115,2568909.4485416785,2388529.489827268,2208149.5311128385,2027769.572398398,1847389.6136839818,1667009.6549695376,1486629.6962551046,1306249.7375406772,1125869.7788262405,945489.8201118149,765109.861397393,584729.90268296,404349.9439685233,223969.9852541238,43590.026539657265,-136789.93217470497,-317169.89088920504,-497549.8496036045,-677929.8083179817,-858309.7670324966,-1038689.7257468291,-1219069.6844613515,-1399449.6431757435,-1579829.6018901542,-1760209.5606045797,-1940589.5193190388,-2120967.466433972,-2301336.3484350927,-2481705.015496202,-2662063.7675952464,-2842422.519694321,-3022774.582644172,-3203124.079579305,-3383471.290464673,-3563812.340268966,-3744153.390073132,-3924489.4586585388,-4104822.807630744,-4285156.156603005,-4465483.6831211485,-4645810.020427585,-4826136.357733998,-5006457.615699645,-5186777.577634402,-5367097.539569076,-5547414.51010555,-5727728.684034936,-5908042.857964344,-6088357.031893723,-6268666.188827768,-6448975.116842322,-6629284.044856876,-6809592.620656073,-6989896.802951299,-7170200.985246502,-7350505.167541705,-7530809.349836864,-7711110.01539468,-7891409.913401216,-8071709.81140776,-8252009.709414318,-8432309.082738936,-8612605.122020148,-8792901.161301382,-8973197.20058269],[8879606.361473698,8698727.097714948,8517847.8339562,8336968.570197468,8156089.306438707,7975210.042679977,7794330.7789212335,7613451.515162491,7432572.251403737,7251692.987645001,7070813.723886248,6889934.460127514,6709055.196368764,6528175.932610029,6347296.668851265,6166417.405092541,5985538.141333783,5804658.877575046,5623779.613816291,5442900.350057547,5262021.086298812,5081141.822540065,4900262.558781316,4719383.29502257,4538504.031263838,4357624.767505093,4176745.503746344,3995866.2399876025,3814986.9762288667,3634107.712470118,3453228.4487113636,3272349.1849526186,3091469.921193879,2910590.6574351173,2729711.393676389,2548832.1299176514,2367952.8661589157,2187073.60240018,2006194.338641405,1825315.0748826656,1644435.8111239355,1463556.5473651662,1282677.2836064063,1101798.019847691,920918.756088946,740039.4923302047,559160.2285714522,378280.96481269225,197401.7010539919,16522.437295228243,-164356.826463487,-345236.0902222842,-526115.3539810218,-706994.6177397147,-887873.881498523,-1068753.1452572234,-1249632.4090160057,-1430511.6727747247,-1611390.936533492,-1792270.2002921999,-1973149.4640509374,-2154028.727809757,-2334907.9915684983,-2515785.9857085235,-2696654.508411858,-2877523.031115271,-3058382.8084348217,-3239241.508202981,-3420095.5410826355,-3600945.2661462277,-3781794.99120982,-3962636.596718721,-4143478.1306251697,-4324317.532743443,-4505151.599176396,-4685985.665609412,-4866817.141440559,-5047644.408682492,-5228471.675924368,-5409297.381973196,-5590118.467036493,-5770939.552099653,-5951760.637163021,-6132576.838417985,-6313392.310870081,-6494207.783322111,-6675022.246423468,-6855832.631927796,-7036643.017432094,-7217453.402936317,-7398262.891262345,-7579068.674856938,-7759874.458451614,-7940680.242046326,-8121486.025641039,-8302288.449657232,-8483090.078790054,-8663891.707922928,-8844693.33705584,-9025494.966188788],[8877004.719400678,8695626.150597623,8514247.58179456,8332869.012991511,8151490.44418844,7970111.875385387,7788733.3065823335,7607354.737779275,7425976.168976207,7244597.600173151,7063219.031370086,6881840.462567031,6700461.893763977,6519083.324960926,6337704.756157849,6156326.1873547975,5974947.618551726,5793569.049748679,5612190.480945606,5430811.912142553,5249433.343339492,5068054.77453644,4886676.205733371,4705297.636930324,4523919.06812726,4342540.499324193,4161161.9305211287,3979783.361718079,3798404.792915037,3617026.2241119724,3435647.6553088836,3254269.086505834,3072890.517702762,2891511.948899718,2710133.3800966702,2528754.8112935983,2347376.242490573,2165997.6736874953,1984619.1048844215,1803240.5360813625,1621861.967278283,1440483.3984752353,1259104.829672154,1077726.2608691156,896347.6920660511,714969.1232629865,533590.554459963,352211.9856568873,170833.41685383767,-10545.151949245483,-191923.72075223923,-373302.2895553373,-554680.8583584502,-736059.4271614663,-917437.9959645793,-1098816.5647675768,-1280195.1335706897,-1461573.702373702,-1642952.2711767629,-1824330.8399797939,-2005709.4087828808,-2187087.97758596,-2368466.546389058,-2549845.1151920855,-2731223.683995124,-2912602.252798155,-3093971.23119789,-3275339.3946036175,-3456701.6399505995,-3638060.2873877436,-3819417.412035171,-4000767.3652273193,-4182117.3184194528,-4363462.789577361,-4544804.807585936,-4726146.8255946785,-4907483.260564957,-5088818.044458706,-5270152.828352459,-5451482.516679291,-5632810.7138566375,-5814138.911033966,-5995463.848952815,-6176786.0571446195,-6358108.265336372,-6539430.186891645,-6720746.957866311,-6902063.728841007,-7083380.499815628,-7264695.976972289,-7446007.819966339,-7627319.662960395,-7808631.505954377,-7989942.769665249,-8171250.154559419,-8352557.539453551,-8533864.924347602,-8715172.309241682,-8896477.270677827,-9077780.630937167],[8874403.077327663,8692525.2034803,8510647.32963292,8328769.455785554,8146891.581938172,7965013.708090793,7783135.834243426,7601257.960396055,7419380.086548674,7237502.212701296,7055624.338853916,6873746.465006549,6691868.591159177,6509990.7173118135,6328112.843464419,6146234.969617054,5964357.095769677,5782479.221922305,5600601.348074919,5418723.47422755,5236845.600380173,5054967.726532809,4873089.852685415,4691211.978838054,4509334.104990685,4327456.231143303,4145578.3572959267,3963700.483448563,3781822.6096011978,3599944.73575381,3418066.861906413,3236188.988059044,3054311.1142116897,2872433.2403642964,2690555.366516942,2508677.49266956,2326799.6188222095,2144921.7449748255,1963043.871127436,1781165.9972800687,1599288.1234326698,1417410.2495852709,1235532.3757378943,1053654.5018905587,871776.6280431822,689898.7541958019,508020.88034844026,326143.0065010376,144265.13265372068,-37612.74119368568,-219490.61504101753,-401368.48888847604,-583246.3627358489,-765124.236583177,-947002.1104305945,-1128879.9842779525,-1310757.8581253514,-1492635.7319726422,-1674513.605820056,-1856391.479667399,-2038269.3535147905,-2220147.227362208,-2402025.101209581,-2583902.9750569686,-2765780.848904267,-2947658.7227516696,-3129536.596599046,-3311414.4704464152,-3493286.516793091,-3675154.320901081,-3857020.262142364,-4038878.857248485,-4220737.452354681,-4402590.37682258,-4584440.558143195,-4766290.739463933,-4948133.420955528,-5129975.923066404,-5311817.79179658,-5493653.293151047,-5675488.794505507,-5857324.295859911,-6039153.471530743,-6220982.598643564,-6402811.725756414,-6584637.054703683,-6766460.386023968,-6948283.717344403,-7130106.798953243,-7311924.868450604,-7493742.937947944,-7675561.007445224,-7857378.452459596,-8039191.752943575,-8221005.053427435,-8402818.353911199,-8584631.654394992,-8766441.242058404,-8948250.228251942,-9130059.214445531],[8871801.435254656,8689424.25636297,8507047.077471275,8324669.898579597,8142292.719687898,7959915.540796212,7777538.361904524,7595161.18301284,7412784.004121132,7230406.825229455,7048029.646337756,6865652.467446064,6683275.288554387,6500898.109662696,6318520.930770997,6136143.75187932,5953766.572987617,5771389.394095935,5589012.215204234,5406635.036312546,5224257.857420867,5041880.678529171,4859503.499637482,4677126.320745792,4494749.141854122,4312371.962962402,4129994.7840707283,3947617.6051790453,3765240.426287368,3582863.2473956533,3400486.0685039405,3218108.889612267,3035731.7107205726,2853354.5318288933,2670977.352937216,2488600.174045518,2306222.9951538406,2123845.816262141,1941468.6373704355,1759091.4584787507,1576714.2795870472,1394337.100695368,1211959.9218036532,1029582.7429119758,847205.5640203133,664828.3851286061,482451.2062369175,300074.02734520286,117696.84845354408,-64680.33043814078,-247057.50932979956,-429434.6882215217,-611811.867113255,-794189.04600491,-976566.2248966433,-1158943.403788291,-1341320.582679972,-1523697.7615716457,-1706074.9404633716,-1888452.119355023,-2070829.2982467003,-2253206.4771384783,-2435583.6560301594,-2617960.8349218145,-2800338.0138134994,-2982715.192705203,-3165092.3715968393,-3347469.5504885726,-3529846.7293802537,-3712223.9082719646,-3894600.3651974276,-4076967.8100076616,-4259335.254817996,-4441697.217785489,-4624055.760560591,-4806414.30093196,-4988764.710381132,-5171115.119830344,-5353463.97073397,-5535806.956946988,-5718149.9431600645,-5900491.411686294,-6082827.630501576,-6265163.849316731,-6447499.948946759,-6629830.00599505,-6812160.063043319,-6994490.120091639,-7176816.999225803,-7359141.453674726,-7541465.908123694,-7723790.362572528,-7906110.570170648,-8088429.938190699,-8270749.306210734,-8453068.674230672,-8635384.430859253,-8817699.188832812,-9000013.946806468,-9182328.704780161],[8869199.793181652,8686323.309245646,8503446.825309638,8320570.341373643,8137693.8574376255,7954817.373501621,7771940.889565622,7589064.405629624,7406187.921693606,7223311.437757604,7040434.953821598,6857558.46988559,6674681.985949599,6491805.502013586,6308929.018077573,6126052.534141577,5943176.050205561,5760299.5662695635,5577423.082333545,5394546.598397556,5211670.114461547,5028793.630525546,4845917.14658953,4663040.662653521,4480164.178717541,4297287.694781514,4114411.2108455133,3931534.726909522,3748658.242973525,3565781.7590375077,3382905.2751014736,3200028.7911654804,3017152.307229478,2834275.8232934773,2651399.339357469,2468522.8554214686,2285646.371485468,2102769.8875494655,1919893.4036134444,1737016.9196774513,1554140.4357414376,1371263.9518054314,1188387.4678694047,1005510.9839334227,822634.499997396,639758.0160614029,456881.5321254097,274005.0481894128,91128.56425343081,-91747.91968258843,-274624.40361857414,-457500.8875546269,-640377.3714906648,-823253.8554265946,-1006130.339362625,-1189006.8232986107,-1371883.3072346523,-1554759.7911706232,-1737636.2751066461,-1920512.759042628,-2103389.24297864,-2286265.7269146815,-2469142.210850723,-2652018.6947867125,-2834895.178722672,-3017771.662658721,-3200648.146594655,-3383524.6305307522,-3566401.114466667,-3749277.598402694,-3932154.082338754,-4115030.5662747025,-4297907.050210733,-4480779.861923564,-4663646.94743612,-4846514.032948695,-5029373.859442648,-5212232.349886868,-5395089.77513326,-5577940.412710983,-5760791.0502887815,-5943639.927236348,-6126483.397551708,-6309326.867866889,-6492169.336340636,-6675006.27261661,-6857843.208892517,-7040680.145168543,-7223512.120072082,-7406343.107055895,-7589174.094039634,-7772003.682519168,-7954829.260096662,-8137654.837674156,-8320480.415251702,-8503304.0630268,-8686124.729569353,-8868945.396111943,-9051766.06265457,-9234585.853713699],[8866598.151108634,8683222.36212832,8499846.573148005,8316470.784167686,8133094.995187355,7949719.206207033,7766343.417226719,7582967.628246403,7399591.839266073,7216216.050285755,7032840.26130543,6849464.472325116,6666088.683344797,6482712.894364482,6299337.105384147,6115961.316403836,5932585.527423515,5749209.738443195,5565833.949462864,5382458.160482549,5199082.371502225,5015706.582521915,4832330.79354158,4648955.00456127,4465579.215580963,4282203.426600624,4098827.6376203056,3915451.8486400004,3732076.0596596804,3548700.2706793416,3365324.4816990215,3181948.692718683,2998572.903738387,2815197.1147580706,2631821.3257777654,2448445.536797436,2265069.747817114,2081693.9588368107,1898318.1698564515,1714942.3808761388,1531566.5918958057,1348190.8029154912,1164815.0139351487,981439.2249548733,798063.4359745234,614687.6469942257,431311.85801391676,247936.0690335706,64560.28005326167,-118815.50892708823,-302191.29790732265,-485567.0868877135,-668942.875868056,-852318.6648483127,-1035694.4538287073,-1219070.2428089567,-1402446.031789299,-1585821.8207696415,-1769197.6097499505,-1952573.398730263,-2135949.1877105907,-2319324.9766909517,-2502700.765671253,-2686076.5546515696,-2869452.3436318897,-3052828.132612169,-3236203.9215924665,-3419579.7105728425,-3602955.4995531365,-3786331.2885334603,-3969707.0775138102,-4153082.866494082,-4336458.655474443,-4519834.444454808,-4703210.233435061,-4886586.022415459,-5069957.202863421,-5253323.929078292,-5436690.655293152,-5620050.187114019,-5803408.625227235,-5986766.618105687,-6170117.483812049,-6353468.349518299,-6536818.78835208,-6720162.742769562,-6903506.697186947,-7086850.651604421,-7270189.219496019,-7453526.87323276,-7636864.526969455,-7820199.813761957,-8003531.730681166,-8186863.64760045,-8370195.564519733,-8553523.80529008,-8736850.505996004,-8920177.206701972,-9103503.907408006,-9286827.312083997],[8863996.509035628,8680121.415010989,8496246.320986353,8312371.226961732,8128496.132937079,7944621.038912442,7760745.944887815,7576870.850863183,7392995.756838536,7209120.662813896,7025245.56878927,6841370.474764631,6657495.380740001,6473620.2867153715,6289745.192690722,6105870.098666094,5921995.0046414565,5738119.910616823,5554244.816592175,5370369.722567546,5186494.62854292,5002619.534518274,4818744.440493643,4634869.346469009,4450994.252444388,4267119.158419734,4083244.0643950887,3899368.9703704696,3715493.876345845,3531618.7823211886,3347743.6882965434,3163868.594271904,2979993.5002472717,2796118.406222636,2612243.3121980205,2428368.218173366,2244493.1241487563,2060618.030124126,1876742.9360994622,1692867.8420748264,1508992.7480501719,1325117.654025562,1141242.560000889,957367.4659762904,773492.3719516471,589617.2779270113,405742.1839023791,221867.08987772092,37991.99585315585,-145883.09817151725,-329758.19219608605,-513633.28622083366,-697508.3802454546,-881383.4742700569,-1065258.5682947077,-1249133.6623193324,-1433008.7563439794,-1616883.850368578,-1800758.9443931803,-1984634.0384178832,-2168509.1324425302,-2352384.226467155,-2536259.3204918504,-2720134.4145164043,-2904009.5085410513,-3087884.6025657095,-3271759.6965903006,-3455634.790614981,-3639509.884639576,-3823384.978664279,-4007260.0726889186,-4191135.1667134687,-4375010.260738146,-4558885.354762826,-4742760.448787395,-4926635.542812109,-5110510.636836667,-5294385.730861362,-5478260.824885927,-5662132.388017062,-5845998.754934251,-6029865.121851325,-6213726.2007994875,-6397584.586581722,-6581442.972363964,-6765295.923684388,-6949147.017519213,-7132998.111354046,-7316844.992600538,-7500689.431120224,-7684533.869639844,-7868376.47113993,-8052214.84233737,-8236053.213534899,-8419891.584732339,-8603725.933919452,-8787558.780774072,-8971391.627628647,-9155224.474483468,-9339052.913089141],[8861394.866962615,8677020.467893675,8492646.068824716,8308271.669755776,8123897.270686815,7939522.871617854,7755148.472548908,7570774.073479971,7386399.674411011,7202025.275342062,7017650.876273101,6833276.477204144,6648902.078135212,6464527.679066272,6280153.279997295,6095778.880928353,5911404.481859399,5727030.082790454,5542655.683721492,5358281.284652549,5173906.885583605,4989532.48651465,4805158.087445695,4620783.688376749,4436409.289307818,4252034.890238842,4067660.4911698904,3883286.092100952,3698911.6930320077,3514537.293963028,3330162.8948940802,3145788.4958251156,2961414.0967561863,2777039.6976872366,2592665.2986182887,2408290.899549339,2223916.500480389,2039542.1014114656,1855167.7023424525,1670793.3032735437,1486418.9042045642,1302044.5051356256,1117670.1060666367,933295.7069977336,748921.3079287633,564546.9088598341,380172.50979088247,195798.11072192714,11423.711652979255,-172950.68741598353,-357325.0864849016,-541699.4855539165,-726073.8846228756,-910448.2836917639,-1094822.6827607602,-1279197.0818296634,-1463571.4808986783,-1647945.8799676076,-1832320.2790365443,-2016694.6781055033,-2201069.0771744214,-2385443.476243425,-2569817.8753123954,-2754192.274381317,-2938566.673450239,-3122941.072519213,-3307315.4715880603,-3491689.8706571497,-3676064.2697260343,-3860438.668795027,-4044813.0678639784,-4229187.466932848,-4413561.866001878,-4597936.265070822,-4782310.664139729,-4966685.063208722,-5151059.462277602,-5335433.861346614,-5519808.260415547,-5704182.659484532,-5888557.05855348,-6072931.457622319,-6257305.417384513,-6441671.4250039235,-6626037.432623461,-6810401.900499135,-6994760.233950362,-7179118.567401685,-7363475.7323279455,-7547827.054291241,-7732178.376254655,-7916529.698217958,-8100875.069666609,-8285219.992288426,-8469564.914910324,-8653907.116206653,-8838246.204864748,-9022585.293522887,-9206924.3821811,-9391259.493560523],[8858793.224889599,8673919.520776333,8489045.816663075,8304172.112549813,8119298.408436531,7934424.704323275,7749551.000210011,7564677.296096746,7379803.591983477,7194929.887870208,7010056.183756948,6825182.479643672,6640308.7755304165,6455435.071417164,6270561.3673038725,6085687.6631906135,5900813.959077334,5715940.254964081,5531066.550850806,5346192.84673755,5161319.142624282,4976445.438511018,4791571.734397735,4606698.030284487,4421824.326171234,4236950.622057952,4052076.9179446828,3867203.2138314303,3682329.5097181704,3497455.805604879,3312582.1014916003,3127708.397378342,2942834.6932650898,2757960.989151811,2573087.285038557,2388213.580925286,2203339.876812041,2018466.172698751,1833592.4685854744,1648718.7644722369,1463845.060358949,1278971.3562456965,1094097.6521324068,909223.9480191469,724350.2439058647,539476.5397926159,354602.83567934856,169729.13156607747,-15144.572547189891,-200018.27666042373,-384891.980773665,-569765.6848869696,-754639.3890002742,-939513.0931134708,-1124386.797226794,-1309260.501339998,-1494134.2054533362,-1679007.9095665663,-1863881.613679815,-2048755.317793075,-2233629.0219063647,-2418502.7260196805,-2603376.4301329292,-2788250.134246234,-2973123.8383594155,-3157997.5424726903,-3342871.2465859093,-3527744.950699281,-3712618.6548124664,-3897492.358925782,-4082366.063039057,-4267239.767152265,-4452113.471265558,-4636987.175378893,-4821860.879492033,-5006734.583605394,-5191608.287718575,-5376481.991831917,-5561355.695945144,-5746229.400058407,-5931103.104171678,-6115976.808284894,-6300850.512398288,-6485724.216511518,-6670597.920624815,-6855471.6247380525,-7040341.939287432,-7225207.587609269,-7410073.235930905,-7594935.567333184,-7779793.848453462,-7964652.12957371,-8149508.459834546,-8334360.009926483,-8519211.560018413,-8704063.11011029,-8888909.019550294,-9073754.426274315,-9258599.832998417,-9443443.5009593],[8856191.582816593,8670818.573659012,8485445.564501433,8300072.555343857,8114699.546186265,7929326.537028678,7743953.527871111,7558580.518713529,7373207.509555945,7187834.500398367,7002461.491240782,6817088.482083197,6631715.472925624,6446342.463768035,6260969.454610455,6075596.44545288,5890223.436295289,5704850.427137713,5519477.417980119,5334104.408822541,5148731.399664965,4963358.390507381,4777985.381349789,4592612.3721922245,4407239.363034653,4221866.353877049,4036493.344719466,3851120.33556192,3665747.326404333,3480374.317246722,3295001.308089137,3109628.298931552,2924255.2897739764,2738882.2806163896,2553509.2714588325,2368136.2623012457,2182763.253143672,1997390.2439860944,1812017.2348284684,1626644.225670917,1441271.2165133469,1255898.2073557563,1070525.1981981434,885152.1890405901,699779.1798830181,514406.1707254313,329033.1615678556,143660.15241023898,-41712.85674728453,-227085.86590490863,-412458.87506244704,-597831.884220086,-783204.8933776729,-968577.9025352076,-1153950.9116928242,-1339323.920850344,-1524696.9300079942,-1710069.939165555,-1895442.9483230896,-2080815.9574806765,-2266188.9666383155,-2451561.975795895,-2636934.9849534594,-2822307.9941110946,-3007681.0032686032,-3193054.0124262013,-3378427.021583736,-3563800.0307413936,-3749173.0398989655,-3934546.049056519,-4119919.058214102,-4305292.067371629,-4490665.076529246,-4676038.085686896,-4861411.0948444195,-5046784.104002021,-5232157.113159586,-5417530.122317184,-5602903.1314747855,-5788276.140632298,-5973649.149789929,-6159022.158947445,-6344395.1681051105,-6529768.1772626415,-6715141.186420254,-6900514.19557786,-7085887.204735354,-7271260.213893071,-7456633.2230505645,-7642006.232208185,-7827375.586808801,-8012740.875832759,-8198106.164856762,-8383468.815519415,-8568827.044308744,-8754185.273097984,-8939543.012369521,-9124894.790589944,-9310246.568810396,-9495598.34703087],[8853589.940743584,8667717.626541682,8481845.312339792,8295972.998137906,8110100.683935992,7924228.369734096,7738356.055532208,7552483.741330314,7366611.427128406,7180739.112926509,6994866.79872462,6808994.484522712,6623122.170320826,6437249.856118934,6251377.541917027,6065505.22771514,5879632.913513235,5693760.599311348,5507888.285109434,5322015.970907544,5136143.656705647,4950271.342503756,4764399.028301852,4578526.714099966,4392654.399898082,4206782.085696163,4020909.7714942712,3835037.4572923854,3649165.1430904847,3463292.828888584,3277420.514686661,3091548.2004847713,2905675.886282889,2719803.5720809847,2533931.2578791063,2348058.943677198,2162186.6294753198,1976314.315273406,1790442.0010715127,1604569.6868696064,1418697.3726677224,1232825.0584658384,1046952.7442638911,861080.4300620034,675208.1158601269,489335.8016582206,303463.487456318,117591.17325442284,-68281.14094745368,-254153.45514936745,-440025.76935119554,-625898.0835531726,-811770.3977550529,-997642.7119569592,-1183515.0261588506,-1369387.3403607123,-1555259.6545626558,-1741131.9687644914,-1927004.2829664052,-2112876.597168304,-2298748.9113702103,-2484621.2255721204,-2670493.539774079,-2856365.8539759256,-3042238.1681777947,-3228110.4823797196,-3413982.7965815887,-3599855.1107834987,-3785727.4249853976,-3971599.739187304,-4157472.053389225,-4343344.367591031,-4529216.681793001,-4715088.995994922,-4900961.310196746,-5086833.624398731,-5272705.938600536,-5458578.252802443,-5644450.56700433,-5830322.881206248,-6016195.195408173,-6202067.509609953,-6387939.823812,-6573812.1380137205,-6759684.452215761,-6945556.76641766,-7131429.080619492,-7317301.394821458,-7503173.709023371,-7689046.02322524,-7874918.337427191,-8060790.651628979,-8246662.96583087,-8432535.280032761,-8618406.359948434,-8804271.289674811,-8990136.219401106,-9176001.149127416,-9361859.821516104,-9547717.997974537],[8850988.298670568,8664616.679424357,8478245.060178146,8291873.440931948,8105501.82168572,7919130.202439509,7732758.583193308,7546386.963947099,7360015.344700869,7173643.725454667,6987272.106208453,6800900.486962247,6614528.867716029,6428157.248469828,6241785.6292236,6055414.0099774,5869042.390731179,5682670.771484974,5496299.152238746,5309927.532992543,5123555.913746334,4937184.294500122,4750812.675253898,4564441.056007706,4378069.436761498,4191697.8175152782,4005326.198269075,3818954.5790228546,3632582.9597766493,3446211.340530427,3259839.7212841865,3073468.102037998,2887096.482791785,2700724.8635455705,2514353.244299382,2327981.62505316,2141610.0058069695,1955238.3865607325,1768866.7673145123,1582495.1480682958,1396123.5288221128,1209751.909575887,1023380.29032965,837008.6710834466,650637.0518372506,464265.43259104714,277893.8133448139,91522.19409858435,-94849.42514758185,-281221.0443938263,-467592.6636399627,-653964.2828862406,-840335.9021325298,-1026707.5213786438,-1213079.1406249218,-1399450.7598711029,-1585822.3791173324,-1772193.9983635135,-1958565.6176096946,-2144937.2368558794,-2331308.856102124,-2517680.4753483273,-2704052.09459465,-2890423.71384079,-3076795.333087001,-3263166.952333223,-3449538.5715793706,-3635910.1908256523,-3822281.810071837,-4008653.4293180816,-4195025.048564281,-4381396.667810425,-4567768.287056711,-4754139.906302925,-4940511.52554908,-5126883.144795388,-5313254.764041502,-5499626.383287739,-5685998.002533965,-5872369.621780176,-6058741.241026394,-6245112.860272571,-6431484.479518786,-6617856.098765008,-6804227.718011275,-6990599.337257475,-7176970.956503645,-7363342.5757499635,-7549714.194996066,-7736085.814242341,-7922457.433488585,-8108829.05273471,-8295200.671980865,-8481572.291227117,-8667943.910473384,-8854315.529719584,-9040687.148965769,-9227058.768211916,-9413430.387458183,-9599798.829135135],[8848386.656597558,8661515.732307032,8474644.80801651,8287773.883725988,8100902.959435452,7914032.035144918,7727161.11085441,7540290.1865638755,7353419.26227334,7166548.337982816,6979677.413692302,6792806.489401753,6605935.565111233,6419064.640820714,6232193.716530167,6045322.792239661,5858451.867949128,5671580.9436586,5484710.019368065,5297839.0950775435,5110968.170787021,4924097.246496484,4737226.32220595,4550355.397915436,4363484.473624935,4176613.549334377,3989742.6250438597,3802871.7007533405,3616000.776462825,3429129.8521722686,3242258.927881727,3055388.003591204,2868517.0793006793,2681646.155010164,2494775.2307196353,2307904.306429101,2121033.382138597,1934162.457848059,1747291.533557523,1560420.6092670094,1373549.6849764772,1186678.7606859505,999807.8363953903,812936.9121048711,626065.9878143631,439195.06352385506,252324.13923330233,65453.214942768216,-121417.70934771746,-308288.6336382851,-495159.5579287447,-682030.4822193459,-868901.4065098614,-1055772.330800347,-1242643.2550909482,-1429514.1793813854,-1616385.1036720015,-1803256.0279624537,-1990126.9522530138,-2176997.876543507,-2363868.800834071,-2550739.725124564,-2737610.649415169,-2924481.573705651,-3111352.4979961663,-3298223.422286749,-3485094.3465772346,-3671965.270867802,-3858836.1951582804,-4045707.119448852,-4232578.0437393375,-4419448.968029827,-4606319.892320443,-4793190.816610981,-4980061.740901437,-5166932.665192045,-5353803.58948249,-5540674.513773035,-5727545.4380636215,-5914416.362354059,-6101287.286644608,-6288158.210935093,-6475029.135225721,-6661900.059516162,-6848770.983806737,-7035641.90809726,-7222512.832387745,-7409383.75667835,-7596254.680968836,-7783125.605259396,-7969996.529549934,-8156867.453840353,-8343738.378130943,-8530609.302421473,-8717480.226712093,-8904351.151002616,-9091222.075293146,-9278092.999583505,-9464963.923874073,-9651834.848164655],[8845785.014524546,8658414.785189703,8471044.55585487,8283674.326520031,8096304.09718518,7908933.867850332,7721563.638515503,7534193.409180652,7346823.1798458025,7159452.950510973,6972082.72117613,6784712.491841279,6597342.262506446,6409972.033171599,6222601.803836759,6035231.57450192,5847861.34516707,5660491.115832227,5473120.886497376,5285750.657162537,5098380.4278277,4911010.198492847,4723639.969158009,4536269.739823181,4348899.510488365,4161529.281153474,3974159.051818639,3786788.8224838115,3599418.593148984,3412048.3638141155,3224678.1344792712,3037307.9051444214,2849937.675809581,2662567.446474755,2475197.217139913,2287826.987805065,2100456.7584702373,1913086.529135378,1725716.2998005338,1538346.0704656914,1350975.8411308415,1163605.6117960103,976235.3824611418,788865.1531263217,601494.9237914793,414124.694456622,226754.46512177587,39384.23578694835,-147985.99354787543,-335356.22288273275,-522726.4522175491,-710096.6815524399,-897466.9108872935,-1084837.140222054,-1272207.3695569783,-1459577.5988917649,-1646947.8282266632,-1834318.0575614758,-2021688.2868963443,-2209058.516231142,-2396428.745565988,-2583798.974900864,-2771169.2042357437,-2958539.43357056,-3145909.662905369,-3333279.8922402076,-3520650.121575013,-3708020.3509099185,-3895390.580244761,-4082760.809579596,-4270131.038914423,-4457501.268249221,-4644871.497584101,-4832241.726918984,-5019611.956253771,-5206982.185588695,-5394352.414923456,-5581722.644258294,-5769092.873593211,-5956463.102928016,-6143833.332262911,-6331203.56159763,-6518573.790932573,-6705944.02026736,-6893314.2496022135,-7080684.478937022,-7268054.708271883,-7455424.937606856,-7642795.16694165,-7830165.396276496,-8017535.6256113425,-8204905.854946062,-8392276.0842809,-8579646.313615799,-8767016.542950675,-8954386.772285521,-9141757.001620308,-9329127.230955139,-9516497.460289948,-9703867.689624965],[8843183.372451538,8655313.838072374,8467444.303693216,8279574.769314077,8091705.234934902,7903835.700555748,7715966.166176604,7528096.631797442,7340227.097418278,7152357.563039115,6964488.0286599705,6776618.494280806,6588748.9599016495,6400879.425522495,6213009.891143321,6025140.356764184,5837270.822385013,5649401.288005853,5461531.753626691,5273662.219247541,5085792.684868384,4897923.1504892185,4710053.6161100585,4522184.081730919,4334314.547351778,4146445.012972584,3958575.4785934407,3770705.9442142956,3582836.4098351374,3394966.8754559644,3207097.3410767857,3019227.806697637,2831358.2723184954,2643488.7379393466,2455619.203560181,2267749.669181023,2079880.134801861,1892010.6004227195,1704141.0660435464,1516271.5316643864,1328401.9972852357,1140532.4629060924,952662.9285268858,764793.3941477463,576923.8597685769,389054.3253894411,201184.7910103053,13315.256631098688,-174554.2777480036,-362423.81212723255,-550293.3465063125,-738162.8808854856,-926032.4152647033,-1113901.9496437907,-1301771.4840230085,-1489641.0184021145,-1677510.5527813025,-1865380.0871604644,-2053249.621539615,-2241119.1559187733,-2428988.690297913,-2616858.224677082,-2804727.7590562627,-2992597.2934354134,-3180466.8278145716,-3368336.3621937037,-3556205.8965728693,-3744075.430952046,-3931944.965331156,-4119814.4997103587,-4307684.034089517,-4495553.56846866,-4683423.102847796,-4871292.637227025,-5059162.171606097,-5247031.705985345,-5434901.240364391,-5622770.7747435905,-5810640.3091227785,-5998509.843501929,-6186379.37788108,-6374248.912260205,-6562118.446639381,-6749987.981018573,-6937857.515397683,-7125727.049776897,-7313596.584155999,-7501466.11853525,-7689335.652914345,-7877205.187293522,-8065074.721672639,-8252944.256051749,-8440813.790430993,-8628683.324810073,-8816552.859189346,-9004422.393568486,-9192291.927947626,-9380161.46232672,-9568030.996705905,-9755900.531085148],[8840581.730378523,8652212.89095506,8463844.05153158,8275475.212108113,8087106.372684633,7898737.533261158,7710368.693837701,7521999.8544142265,7333631.014990734,7145262.175567267,6956893.336143804,6768524.496720321,6580155.657296857,6391786.817873385,6203417.978449902,6015049.139026444,5826680.299602955,5638311.460179494,5449942.62075601,5261573.781332542,5073204.941909058,4884836.10248558,4696467.263062116,4508098.423638662,4319729.584215187,4131360.7447916977,3942991.9053682387,3754623.065944761,3566254.226521304,3377885.3870978113,3189516.547674328,3001147.7082508616,2812778.8688273914,2624410.0294039287,2436041.189980453,2247672.3505569603,2059303.5111335125,1870934.671710046,1682565.8322865441,1494196.992863074,1305828.1534396037,1117459.314016156,929090.4745926484,740721.6351691931,552352.7957456969,363983.95632225275,175615.11689876765,-12753.722524702549,-201122.56194815412,-389491.4013716541,-577860.2407950759,-766229.0802185833,-954597.9196421057,-1142966.7590655237,-1331335.598489035,-1519704.437912453,-1708073.2773359865,-1896442.1167594083,-2084810.9561829418,-2273179.79560636,-2461548.6350298375,-2649917.474453356,-2838286.31387683,-3026655.153300263,-3215023.9927237593,-3403392.8321472295,-3591761.6715706736,-3780130.510994166,-3968499.3504175767,-4156868.189841103,-4345237.029264588,-4533605.868688047,-4721974.708111528,-4910343.547535043,-5098712.3869584985,-5287081.226381995,-5475450.065805379,-5663818.905228872,-5852187.744652368,-6040556.584075812,-6228925.423499301,-6417294.262922712,-6605663.102346294,-6794031.941769682,-6982400.781193227,-7170769.620616704,-7359138.4600401,-7547507.299463667,-7735876.138887078,-7924244.978310563,-8112613.81773407,-8300982.657157585,-8489351.496580943,-8677720.33600448,-8866089.175428003,-9054458.014851473,-9242826.854274973,-9431195.693698294,-9619564.533121802,-9807933.37254534],[8837980.088305514,8649111.943837726,8460243.799369937,8271375.654902155,8082507.510434361,7893639.365966573,7704771.221498787,7515903.077031013,7327034.932563206,7138166.788095424,6949298.643627634,6760430.499159837,6571562.354692068,6382694.210224274,6193826.065756479,6004957.921288697,5816089.776820904,5627221.632353122,5438353.487885324,5249485.343417541,5060617.198949741,4871749.0544819515,4682880.910014171,4494012.7655463945,4305144.621078629,4116276.476610806,3927408.3321430255,3738540.1876752377,3549672.0432074554,3360803.89873966,3171935.754271852,2983067.609804049,2794199.465336291,2605331.3208685014,2416463.1764007304,2227595.031932935,2038726.8874651566,1849858.7429973632,1660990.598529555,1472122.4540617727,1283254.3095939904,1094386.1651262157,905518.0206583887,716649.8761906214,527781.7317228317,338913.5872550458,150045.44278724864,-38822.70168054104,-227690.846148327,-416558.99061608315,-605427.1350838356,-794295.2795516849,-983163.4240195267,-1172031.5684872344,-1360899.7129550874,-1549767.85742281,-1738636.0018906407,-1927504.146358434,-2116372.290826205,-2305240.435293954,-2494108.579761781,-2682976.7242296003,-2871844.86869739,-3060713.013165161,-3249581.1576329283,-3438449.3021007255,-3627317.446568474,-3816185.591036331,-4005053.7355040535,-4193921.879971914,-4382790.024439689,-4571658.168907404,-4760526.313375246,-4949394.457843062,-5138262.602310773,-5327130.74677863,-5515998.891246315,-5704867.035714153,-5893735.180181965,-6082603.324649725,-6271471.469117552,-6460339.613585256,-6649207.758053169,-6838075.902520895,-7026944.0469886735,-7215812.191456459,-7404680.33592432,-7593548.480392106,-7782416.624859877,-7971284.769327648,-8160152.913795449,-8349021.058263168,-8537889.202730909,-8726757.347198777,-8915625.49166657,-9104493.636134394,-9293361.78060218,-9482229.92506989,-9671098.069537751,-9859966.214005545],[8835378.446232498,8646010.996720403,8456643.547208298,8267276.097696208,8077908.648184093,7888541.198671995,7699173.749159893,7509806.299647786,7320438.850135682,7131071.400623573,6941703.951111474,6752336.501599361,6562969.052087267,6373601.602575173,6184234.153063055,5994866.703550961,5805499.254038845,5616131.804526752,5426764.355014632,5237396.905502544,5048029.455990417,4858662.006478321,4669294.556966219,4479927.107454136,4290559.657942042,4101192.2084299196,3911824.758917814,3722457.3094057236,3533089.85989362,3343722.410381507,3154354.960869381,2964987.5113572795,2775620.061845189,2586252.6123330984,2396885.1628209986,2207517.713308893,2018150.2637968007,1828782.8142846897,1639415.3647725675,1450047.9152604751,1260680.465748351,1071313.0162362643,881945.5667241514,692578.1172120497,503210.6676999405,313843.21818786114,124475.76867572218,-64891.68083636463,-254259.13034845516,-443626.5798605494,-632994.0293726288,-822361.478884805,-1011728.9283969104,-1201096.3779089712,-1390463.8274211176,-1579831.2769331597,-1769198.7264453284,-1958566.1759573892,-2147933.625469491,-2337301.0749815516,-2526668.524493698,-2716035.974005822,-2905403.423517976,-3094770.8730300367,-3284138.32254212,-3473505.772054188,-3662873.2215663046,-3852240.6710784435,-4041608.1205904894,-4230975.570102621,-4420343.019614767,-4609710.469126768,-4799077.918638941,-4988445.36815108,-5177812.817663137,-5367180.267175324,-5556547.716687303,-5745915.166199427,-5935282.615711503,-6124650.065223645,-6314017.514735773,-6503384.9642478,-6692752.413759999,-6882119.863272034,-7071487.312784098,-7260854.762296326,-7450222.211808346,-7639589.661320485,-7828957.110832609,-8018324.5603447035,-8207692.009856857,-8397059.459368907,-8586426.908880994,-8775794.358393148,-8965161.807905287,-9154529.257417366,-9343896.706929423,-9533264.156441405,-9722631.605953619,-9911999.05546578],[8832776.80415949,8642910.049603077,8453043.295046655,8263176.540490245,8073309.785933813,7883443.031377392,7693576.27682099,7503709.522264568,7313842.767708149,7123976.0131517155,6934109.258595315,6744242.50403889,6554375.749482477,6364508.994926056,6174642.240369627,5984775.4858132135,5794908.731256794,5605041.9767003795,5415175.222143949,5225308.467587535,5035441.713031098,4845574.958474688,4655708.20391828,4465841.449361881,4275974.694805454,4086107.940249013,3896241.1856926046,3706374.4311361946,3516507.6765797753,3326640.9220233522,3136774.1674669236,2946907.41291051,2757040.6583541017,2567173.9037976842,2377307.1492412686,2187440.3946848344,1997573.6401284318,1807706.8855720125,1617840.1310155876,1427973.376459159,1238106.6219027564,1048239.8673463576,858373.112789914,668506.3582334965,478639.60367706046,288772.84912063926,98906.09456424043,-90960.65999218076,-280827.41454858705,-470694.16910502315,-660560.923661422,-850427.6782178879,-1040294.4327743053,-1230161.1873306818,-1420027.9418871664,-1609894.6964435205,-1799761.4509999938,-1989628.2055563666,-2179494.960112784,-2369361.7146691903,-2559228.469225604,-2749095.2237820625,-2938961.97833851,-3128828.73289489,-3318695.487451289,-3508562.2420077175,-3698428.996564124,-3888295.7511205673,-4078162.5056769475,-4268029.26023341,-4457896.014789868,-4647762.769346215,-4837629.523902651,-5027496.278459135,-5217363.033015486,-5407229.787571959,-5597096.542128291,-5786963.296684753,-5976830.051241152,-6166696.8057975955,-6356563.560354017,-6546430.314910397,-6736297.069466874,-6926163.824023284,-7116030.578579709,-7305897.333136149,-7495764.087692529,-7685630.842249028,-7875497.596805431,-8065364.3513618335,-8255231.105918214,-8445097.860474579,-8634964.61503096,-8824831.36958751,-9014698.124143869,-9204564.87870036,-9394431.633256674,-9584298.387813076,-9774165.142369479,-9964031.896926016],[8830175.162086478,8639809.102485752,8449443.042885013,8259076.983284297,8068710.9236835465,7878344.864082811,7687978.804482082,7497612.7448813515,7307246.685280614,7116880.625679872,6926514.5660791565,6736148.5064784065,6545782.446877677,6355416.387276958,6165050.327676195,5974684.268075477,5784318.208474741,5593952.148874011,5403586.08927327,5213220.029672536,5022853.970071787,4832487.910471056,4642121.850870324,4451755.791269613,4261389.731668886,4071023.67206813,3880657.6124674026,3690291.5528666805,3499925.493265949,3309559.4336651955,3119193.374064453,2928827.3144637216,2738461.2548629865,2548095.1952622514,2357729.135661559,2167363.0760608036,1976997.0164600704,1786630.9568593241,1596264.8972586058,1405898.837657854,1215532.7780571058,1025166.7184564136,834800.6588556394,644434.5992549099,454068.53965417296,263702.48005345464,73336.42045270652,-117029.63914805278,-307395.69874871895,-497761.7583495006,-688127.8179501705,-878493.8775509931,-1068859.9371517524,-1259225.9967524149,-1449592.0563531816,-1639958.1159538664,-1830324.1755546592,-2020690.2351553477,-2211056.2947560847,-2401422.354356829,-2591788.41395754,-2782154.473558318,-2972520.533159055,-3162886.592759762,-3353252.6523604877,-3543618.711961217,-3733984.7715619244,-3924350.8311627246,-4114716.8907634243,-4305082.950364176,-4495449.009964943,-4685815.069565602,-4876181.129166383,-5066547.188767109,-5256913.24836782,-5447279.307968661,-5637645.367569264,-5828011.427170057,-6018377.486770742,-6208743.546371508,-6399109.605972253,-6589475.665572889,-6779841.725173734,-6970207.78477443,-7160573.84437523,-7350939.903975926,-7541305.963576652,-7731672.023177475,-7922038.082778133,-8112404.142378844,-8302770.2019796,-8493136.261580266,-8683502.321181,-8873868.380781725,-9064234.44038257,-9254600.499983326,-9444966.559583977,-9635332.619184636,-9825698.678785421,-10016064.738386221],[8827573.520013472,8636708.155368425,8445842.790723372,8254977.426078341,8064112.061433271,7873246.696788223,7682381.332143174,7491515.967498133,7300650.602853073,7109785.238208031,6918919.873562985,6728054.508917938,6537189.144272886,6346323.779627839,6155458.414982779,5964593.050337739,5773727.685692688,5582862.32104763,5391996.9564025765,5201131.591757536,5010266.227112465,4819400.862467429,4628535.497822376,4437670.133177342,4246804.768532317,4055939.403887231,3865074.0392422006,3674208.6745971423,3483343.3099521007,3292477.9453070536,3101612.5806619693,2910747.216016926,2719881.8513719067,2529016.486726852,2338151.122081807,2147285.757436745,1956420.392791722,1765555.028146673,1574689.6635015942,1383824.2988565415,1192958.9342114963,1002093.5695664696,811228.2049213722,620362.8402763344,429497.47563130036,238632.11098627746,47766.74634119496,-143098.6183038652,-333963.9829488769,-524829.3475939594,-715694.712238919,-906560.0768840648,-1097425.4415291026,-1288290.8061741255,-1479156.170819208,-1670021.5354642235,-1860886.9001093172,-2051752.264754381,-2242617.6293993853,-2433482.9940444343,-2624348.358689487,-2815213.7233345807,-3006079.0879796036,-3196944.4526246525,-3387809.8172696754,-3578675.1819147356,-3769540.5465597473,-3960405.9112048447,-4151271.2758498937,-4342136.640494965,-4533002.00514001,-4723867.369784974,-4914732.734430086,-5105598.09907515,-5296463.463720147,-5487328.828365296,-5678194.193010177,-5869059.557655316,-6059924.922300369,-6250790.286945391,-6441655.651590489,-6632521.016235486,-6823386.380880587,-7014251.745525576,-7205117.110170655,-7395982.474815734,-7586847.83946076,-7777713.204105839,-7968578.568750873,-8159443.933395959,-8350309.298040964,-8541174.662685975,-8732040.02733098,-8922905.39197614,-9113770.756621204,-9304636.121266216,-9495501.485911287,-9686366.850556254,-9877232.215201415,-10068097.579846457],[8824971.877940463,8633607.208251094,8442242.538561732,8250877.8688723715,8059513.199183004,7868148.529493638,7676783.859804282,7485419.190114919,7294054.520425547,7102689.850736174,6911325.181046822,6719960.511357453,6528595.841668088,6337231.171978738,6145866.50228936,5954501.832599998,5763137.162910628,5571772.493221267,5380407.823531892,5189043.153842557,4997678.4841531515,4806313.814463794,4614949.144774435,4423584.47508508,4232219.805395726,4040855.135706339,3849490.4660170004,3658125.7963276207,3466761.1266382765,3275396.456948893,3084031.7872595116,2892667.117570149,2701302.4478808027,2509937.778191434,2318573.1085020863,2127208.4388126917,1935843.7691233363,1744479.0994339976,1553114.4297446124,1361749.760055244,1170385.0903658755,979020.420676522,787655.7509871274,596291.0812977962,404926.4116084315,213561.74191906303,22197.07222969085,-169167.59745969623,-360532.2671490237,-551896.9368383959,-743261.6065276898,-934626.2762171738,-1125990.9459064975,-1317355.6155958436,-1508720.285285268,-1700084.9549745917,-1891449.624663964,-2082814.2943533324,-2274178.9640426785,-2465543.633732047,-2656908.303421378,-2848272.973110795,-3039637.642800171,-3231002.3124895096,-3422366.9821788967,-3613731.6518682055,-3805096.3215575702,-3996460.9912469946,-4187825.660936337,-4379190.3306256905,-4570555.000315085,-4761919.67000439,-4953284.339693803,-5144649.009383183,-5336013.679072503,-5527378.348761931,-5718743.018451188,-5910107.688140605,-6101472.357829958,-6292837.027519289,-6484201.697208695,-6675566.366897993,-6866931.036587425,-7058295.706276782,-7249660.375966184,-7441025.045655556,-7632389.715344839,-7823754.3850342855,-8015119.054723576,-8206483.72441303,-8397848.394102342,-8589213.06379167,-8780577.733480982,-8971942.403170466,-9163307.072859846,-9354671.742549188,-9546036.412238546,-9737401.081927828,-9928765.751617253,-10120130.421306692],[8822370.235867437,8630506.261133779,8438642.28640009,8246778.311666427,8054914.336932727,7863050.362199055,7671186.38746538,7479322.412731707,7287458.437998025,7095594.463264324,6903730.488530654,6711866.513796976,6520002.539063299,6328138.5643296195,6136274.589595926,5944410.614862248,5752546.640128572,5560682.665394895,5368818.690661205,5176954.715927554,4985090.741193844,4793226.766460162,4601362.791726489,4409498.816992825,4217634.842259152,4025770.867525449,3833906.8927917797,3642042.9180581085,3450178.9433244206,3258314.9685907383,3066450.993857043,2874587.019123355,2682723.0443896893,2490859.0696560126,2298995.0949223433,2107131.120188674,1915267.1454550046,1723403.1707213074,1531539.1959876157,1339675.221253939,1147811.246520266,955947.2717866004,764083.2970528901,572219.3223192096,380355.3475855291,188491.3728518486,-3372.6018818467855,-195236.57661553472,-387100.5513491705,-578964.5260828659,-770828.5008164793,-962692.4755502343,-1154556.4502839483,-1346420.4250175692,-1538284.3997512683,-1730148.3744849227,-1922012.349218607,-2113876.3239522874,-2305740.2986859716,-2497604.2734196372,-2689468.248153303,-2881332.222887069,-3073196.197620731,-3265060.1723543853,-3456924.1470880434,-3648788.12182172,-3840652.0965554155,-4032516.071289081,-4224380.046022758,-4416244.020756498,-4608107.995490145,-4799971.97022377,-4991835.944957469,-5183699.9196911715,-5375563.894424807,-5567427.869158581,-5759291.843892153,-5951155.818625923,-6143019.793359563,-6334883.768093225,-6526747.742826916,-6718611.717560604,-6910475.692294329,-7102339.667027928,-7294203.641761631,-7486067.616495319,-7677931.591228977,-7869795.565962814,-8061659.540696412,-8253523.515430026,-8445387.490163691,-8637251.464897364,-8829115.439631045,-9020979.414364763,-9212843.389098488,-9404707.36383225,-9596571.338565879,-9788435.313299417,-9980299.288033076,-10172163.26276698],[8819768.59379443,8627405.314016445,8435042.034238447,8242678.75446046,8050315.474682458,7857952.194904461,7665588.915126479,7473225.635348481,7280862.355570486,7088499.075792482,6896135.796014492,6703772.516236495,6511409.236458514,6319045.956680512,6126682.676902505,5934319.397124508,5741956.117346518,5549592.837568525,5357229.557790522,5164866.278012557,4972502.998234516,4780139.718456529,4587776.438678533,4395413.15890057,4203049.8791225795,4010686.599344559,3818323.3195665795,3625960.0397885814,3433596.760010591,3241233.480232591,3048870.200454572,2856506.9206765853,2664143.640898589,2471780.361120604,2279417.0813426226,2087053.8015646134,1894690.5217866339,1702327.242008619,1509963.962230619,1317600.6824526042,1125237.402674634,932874.1228966676,740510.8431186266,548147.5633406565,355784.28356266394,163421.0037846826,-28942.275993343443,-221305.55577135086,-413668.8355493285,-606032.1153272912,-798395.3951052614,-990758.6748833023,-1183121.9546613283,-1375485.23443928,-1567848.5142173246,-1760211.793995235,-1952575.0737732723,-2144938.353551287,-2337301.633329265,-2529664.913107265,-2722028.192885235,-2914391.4726632684,-3106754.752441298,-3299118.0322192684,-3491481.3119972423,-3683844.5917752534,-3876207.871553216,-4068571.151331242,-4260934.431109201,-4453297.7108872235,-4645660.990665253,-4838024.270443171,-5030387.550221227,-5222750.82999922,-5415114.109777164,-5607477.389555234,-5799840.669333138,-5992203.949111126,-6184567.228889186,-6376930.50866713,-6569293.788445212,-6761657.068223126,-6954020.348001212,-7146383.627779134,-7338746.907557137,-7531110.187335156,-7723473.467113167,-7915836.746891148,-8108200.026669145,-8300563.306447141,-8492926.586225048,-8685289.866003036,-8877653.145781055,-9070016.425559044,-9262379.7053371,-9454742.985115156,-9647106.264893115,-9839469.544671059,-10031832.8244491,-10224196.104227103],[8817166.951721419,8624304.366899118,8431441.782076806,8238579.197254507,8045716.612432186,7852854.027609874,7659991.442787577,7467128.857965272,7274266.273142954,7081403.688320631,6888541.103498329,6695678.51867601,6502815.93385371,6309953.349031408,6117090.764209084,5924228.179386779,5731365.59456446,5538503.009742159,5345640.424919842,5152777.84009756,4959915.255275197,4767052.670452915,4574190.085630596,4381327.500808299,4188464.915985996,3995602.33116368,3802739.7463413645,3609877.161519058,3417014.576696761,3224151.991874438,3031289.4070521053,2838426.8222298045,2645564.2374074925,2452701.6525851935,2259839.0677628927,2066976.482940575,1874113.8981182761,1681251.3132959493,1488388.728473641,1295526.1436512917,1102663.558829032,909800.9740067311,716938.389184393,524075.8043620959,331213.219539769,138350.63471748307,-54511.95010487363,-247374.53492720425,-440237.1197494082,-633099.7045717984,-825962.2893940397,-1018824.8742164262,-1211687.4590387456,-1404550.043860983,-1597412.6286833473,-1790275.2135055996,-1983137.7983279563,-2176000.3831502832,-2368862.9679725617,-2561725.552794859,-2754588.1376171783,-2947450.7224395126,-3140313.3072618432,-3333175.8920841403,-3526038.4769064337,-3718901.0617287345,-3911763.646551024,-4104626.2313733585,-4297488.816195618,-4490351.401017979,-4683213.985840302,-4876076.570662592,-5068939.155484937,-5261801.740307279,-5454664.325129513,-5647526.909951892,-5840389.494774096,-6033252.079596456,-6226114.664418776,-6418977.24924108,-6611839.834063407,-6804702.418885671,-6997565.003708057,-7190427.588530369,-7383290.173352607,-7576152.758174956,-7769015.342997201,-7961877.927819647,-8154740.512641937,-8347603.097464211,-8540465.682286568,-8733328.26710876,-8926190.851931058,-9119053.436753415,-9311916.021575786,-9504778.606398068,-9697641.191220403,-9890503.776042543,-10083366.360864982,-10276228.945687227],[8814565.30964841,8621203.41978179,8427841.529915169,8234479.640048552,8041117.750181912,7847755.860315286,7654393.970448677,7461032.080582052,7267670.190715416,7074308.300848787,6880946.410982165,6687584.52111553,6494222.631248918,6300860.741382295,6107498.851515651,5914136.9616490435,5720775.071782408,5527413.181915791,5334051.2920491565,5140689.402182547,4947327.512315879,4753965.622449268,4560603.732582651,4367241.8427160345,4173879.952849416,3980518.062982781,3787156.1731161717,3593794.283249546,3400432.393382918,3207070.5035162866,3013708.6136496477,2820346.7237830274,2626984.8339164034,2433622.944049768,2240261.0541831627,2046899.16431652,1853537.2744499035,1660175.3845832646,1466813.494716648,1273451.6048499905,1080089.7149834149,886727.8251167946,693365.9352501258,500004.0453835316,306642.15551690385,113280.2656502612,-80081.62421636656,-273443.5140830055,-466805.4039495997,-660167.2938162312,-853529.1836827956,-1046891.0735494941,-1240252.9634161368,-1433614.85328269,-1626976.743149422,-1820338.6330159642,-2013700.5228826627,-2207062.4127492122,-2400424.302615892,-2593786.192482449,-2787148.0823491067,-2980509.972215783,-3173871.8620823883,-3367233.7519490346,-3560595.6418156326,-3753957.5316822603,-3947319.421548832,-4140681.311415501,-4334043.201282062,-4527405.091148742,-4720766.981015388,-4914128.870881982,-5107490.760748636,-5300852.650615253,-5494214.540481839,-5687576.43034859,-5880938.320215087,-6074300.210081719,-6267662.099948347,-6461023.989814967,-6654385.879681662,-6847747.769548163,-7041109.65941491,-7234471.549281485,-7427833.439148098,-7621195.329014815,-7814557.218881302,-8007919.108748078,-8201280.998614624,-8394642.888481312,-8588004.778347902,-8781366.668214396,-8974728.558081105,-9168090.447947688,-9361452.33781445,-9554814.227681048,-9748176.117547624,-9941538.007414214,-10134899.89728085,-10328261.787147574],[8811963.667575398,8618102.472664462,8424241.277753528,8230380.082842592,8036518.887931635,7842657.693020707,7648796.498109774,7454935.303198831,7261074.108287894,7067212.913376938,6873351.718466002,6679490.523555051,6485629.328644118,6291768.133733185,6097906.938822233,5904045.743911292,5710184.549000347,5516323.354089417,5322462.159178473,5128600.964267559,4934739.769356564,4740878.574445637,4547017.379534699,4353156.184623774,4159294.9897128325,3965433.7948018815,3771572.599890951,3577711.404980026,3383850.2100690864,3189989.01515813,2996127.820247177,2802266.625336241,2608405.4304253235,2414544.2355143707,2220683.0406034254,2026821.8456924874,1832960.650781557,1639099.4558705948,1445238.2609596588,1251377.0660486966,1057515.8711377755,863654.676226858,669793.4813158661,475932.2864049487,282071.0914939754,88209.89658306912,-105651.29832787067,-299512.4932388328,-493373.68814972416,-687234.8830607049,-881096.0779715702,-1074957.2728825584,-1268818.4677935317,-1462679.662704423,-1656540.8576154374,-1850402.0525263138,-2044263.2474372946,-2238124.442348212,-2431985.6372591443,-2625846.832170069,-2819708.0270810165,-3013569.2219920196,-3207430.416902967,-3401291.611813847,-3595152.806724813,-3789014.001635749,-3982875.196546674,-4176736.391457632,-4370597.586368535,-4564458.781279508,-4758319.9761904515,-4952181.171101313,-5146042.366012353,-5339903.560923301,-5533764.755834188,-5727625.950745165,-5921487.145656053,-6115348.340567041,-6309209.535477988,-6503070.730388876,-6696931.925299842,-6890793.120210715,-7084654.315121777,-7278515.510032706,-7472376.704943657,-7666237.899854563,-7860099.094765492,-8053960.289676495,-8247821.484587364,-8441682.679498352,-8635543.874409348,-8829405.069320135,-9023266.264231086,-9217127.459142134,-9410988.654053122,-9604849.84896411,-9798711.043874934,-9992572.238785818,-10186433.43369671,-10380294.628607802],[8809362.025502391,8615001.525547141,8420641.025591888,8226280.525636638,8031920.025681381,7837559.525726109,7643199.025770862,7448838.525815614,7254478.025860366,7060117.525905082,6865757.025949837,6671396.525994576,6477036.026039331,6282675.52608408,6088315.026128808,5893954.526173561,5699594.026218297,5505233.526263053,5310873.026307803,5116512.52635254,4922152.026397252,4727791.526442008,4533431.026486762,4339070.526531516,4144710.0265762657,3950349.5266209897,3755989.0266657397,3561628.526710501,3367268.026755249,3172907.526799975,2978547.0268446915,2784186.526889447,2589826.026934212,2395465.52697894,2201105.027023701,2006744.5270684343,1812384.027113188,1618023.5271579064,1423663.0272026733,1229302.5272473954,1034942.0272921771,840581.5273369178,646221.0273816213,451860.52742637694,257500.0274711214,63139.52751588449,-131220.97243938595,-325581.4723946415,-519941.9723499045,-714302.4723051861,-908662.9722603597,-1103023.472215671,-1297383.972170934,-1491744.4721261635,-1686104.9720814787,-1880465.472036656,-2074825.9719919674,-2269186.4719472006,-2463546.971902434,-2657907.471857667,-2852267.971812967,-3046628.4717682675,-3240988.9717235044,-3435349.4716787525,-3629709.9716340005,-3824070.4715892486,-4018430.971544467,-4212791.471499793,-4407151.971454989,-4601512.471410323,-4795872.971365519,-4990233.4713207595,-5184593.971276082,-5378954.471231341,-5573314.971186534,-5767675.47114186,-5962035.971096989,-6156396.471052345,-6350756.971007593,-6545117.470962789,-6739477.970918119,-6933838.470873311,-7128198.97082866,-7322559.470783882,-7516919.970739134,-7711280.4706943855,-7905640.970649585,-8100001.470604919,-8294361.970560208,-8488722.470515467,-8683082.970470607,-8877443.47042586,-9071803.970381051,-9266164.470336422,-9460524.970291711,-9654885.470246889,-9849245.970202133,-10043606.470157407,-10237966.970112659,-10432327.470068052],[8806760.38342937,8611900.57842981,8417040.773430236,8222180.968430676,8027321.163431104,7832461.358431522,7637601.553431964,7442741.74843239,7247881.943432828,7053022.138433246,6858162.333433672,6663302.528434096,6468442.723434533,6273582.918434964,6078723.113435374,5883863.308435819,5689003.503436243,5494143.698436679,5299283.893437114,5104424.08843755,4909564.283437936,4714704.478438383,4519844.673438808,4324984.868439252,4130125.063439695,3935265.258440096,3740405.453440534,3545545.6484409627,3350685.843441412,3155826.0384418275,2960966.2334422227,2766106.4284426626,2571246.6234431006,2376386.818443544,2181527.013443969,1986667.208444396,1791807.4034448303,1596947.5984452441,1402087.7934456766,1207227.9884461053,1012368.1834465452,817508.3784469813,622648.5734473914,427788.76844784245,232928.963448219,38069.15844865888,-156790.64655089006,-351650.4515504725,-546510.2565500103,-741370.06154963,-936229.866549138,-1131089.67154878,-1325949.4765483662,-1520809.2815478705,-1715669.0865474828,-1910528.891547028,-2105388.696546629,-2300248.5015461743,-2495108.306545738,-2689968.111545302,-2884827.9165448844,-3079687.721544493,-3274547.5265440457,-3469407.3315435946,-3664267.136543203,-3859126.9415427074,-4053986.7465423085,-4248846.551541962,-4443706.35654141,-4638566.161541078,-4833425.966540616,-5028285.771540221,-5223145.576539744,-5418005.381539341,-5612865.186538845,-5807724.991538517,-6002584.796537995,-6197444.6015376,-6392304.406537216,-6587164.211536739,-6782024.016536303,-6976883.821535803,-7171743.626535475,-7366603.431535013,-7561463.236534581,-7756323.041534185,-7951182.84653376,-8146042.65153341,-8340902.456532866,-8535762.261532456,-8730622.06653206,-8925481.871531487,-9120341.676531084,-9315201.481530748,-9510061.286530346,-9704921.091529936,-9899780.896529436,-10094640.701529019,-10289500.506528527,-10484360.311528273],[8804158.741356371,8608799.631312486,8413440.521268606,8218081.411224721,8022722.301180828,7827363.191136938,7632004.081093063,7436644.971049169,7241285.861005293,7045926.750961388,6850567.640917516,6655208.5308736265,6459849.42082974,6264490.310785862,6069131.200741951,5873772.090698089,5678412.980654182,5483053.8706103,5287694.760566432,5092335.650522547,4896976.540478626,4701617.4304347485,4506258.320390863,4310899.2103469875,4115540.100303108,3920180.990259191,3724821.88021533,3529462.770171445,3334103.6601275634,3138744.550083667,2943385.440039765,2748026.329995893,2552667.2199519984,2357308.109908141,2161948.9998642523,1966589.8898203634,1771230.7797764689,1575871.6697325557,1380512.5596886761,1185153.4496447854,989794.3396009244,794435.2295570485,599076.1195131205,403717.009469226,208357.89942535013,12998.789381485432,-182360.32066242024,-377719.43070631847,-573078.5407501757,-768437.6507940777,-963796.7608379386,-1159155.8708818667,-1354514.980925765,-1549874.0909695737,-1745233.2010135278,-1940592.3110573888,-2135951.421101276,-2331310.5311451443,-2526669.6411890276,-2722028.751232911,-2917387.861276768,-3112746.9713207297,-3308106.081364639,-3503465.191408474,-3698824.3014523573,-3894183.411496252,-4089542.5215401277,-4284901.631584045,-4480260.741627894,-4675619.851671852,-4870978.961715724,-5066338.071759533,-5261697.181803428,-5457056.291847359,-5652415.401891176,-5847774.511935174,-6043133.62197895,-6238492.732022844,-6433851.842066824,-6629210.952110648,-6824570.062154561,-7019929.172198385,-7215288.282242358,-7410647.392286137,-7606006.5023301095,-7801365.612373978,-7996724.722417846,-8192083.832461789,-8387442.942505635,-8582802.052549548,-8778161.162593454,-8973520.272637323,-9168879.382681109,-9364238.49272509,-9559597.602768987,-9754956.712812893,-9950315.822856724,-10145674.932900615,-10341034.04294449,-10536393.152988404],[8801557.099283362,8605698.684195163,8409840.269106966,8213981.8540187655,8018123.438930553,7822265.0238423515,7626406.608754158,7430548.193665956,7234689.778577767,7038831.3634895375,6842972.948401353,6647114.5333131505,6451256.118224946,6255397.70313674,6059539.288048525,5863680.872960337,5667822.4578721365,5471964.04278394,5276105.627695747,5080247.2126075495,4884388.797519313,4688530.382431107,4492671.967342917,4296813.55225472,4100955.1371665373,3905096.722078312,3709238.3069901243,3513379.8919019178,3317521.476813728,3121663.0617255103,2925804.6466372926,2729946.2315490954,2534087.816460911,2338229.401372699,2142370.986284515,1946512.571196301,1750654.1561081111,1554795.7410198785,1358937.3259316944,1163078.9108434953,967220.4957553148,771362.0806671232,575503.6655788608,379645.2504906915,183786.83540248498,-12071.579685728997,-207929.99477389082,-403788.40986215323,-599646.8249503374,-795505.2400385514,-991363.6551266536,-1187222.070214957,-1383080.4853031635,-1578938.900391344,-1774797.315479558,-1970655.7305677012,-2166514.1456559487,-2362372.5607441477,-2558230.975832369,-2754089.390920516,-2949947.80600873,-3145806.2210969776,-3341664.63618521,-3537523.051273383,-3733381.466361545,-3929239.8814497553,-4125098.2965379283,-4320956.711626172,-4516815.126714379,-4712673.5418026,-4908531.956890747,-5104390.371978927,-5300248.787067145,-5496107.20215543,-5691965.617243532,-5887824.032331806,-6083682.447419945,-6279540.86250817,-6475399.277596343,-6671257.692684572,-6867116.107772782,-7062974.522860907,-7258832.937949166,-7454691.353037365,-7650549.768125616,-7846408.183213793,-8042266.598302007,-8238125.01339028,-8433983.42847839,-8629841.843566597,-8825700.25865484,-9021558.673742972,-9217417.088831194,-9413275.503919438,-9609133.919007652,-9804992.334095888,-10000850.749184042,-10196709.164272115,-10392567.579360396,-10588425.994448632],[8798955.457210349,8602597.73707784,8406240.016945317,8209882.296812803,8013524.576680284,7817166.856547765,7620809.136415259,7424451.416282734,7228093.696150222,7031735.9760177,6835378.255885189,6639020.535752663,6442662.815620151,6246305.095487638,6049947.375355103,5853589.655222602,5657231.935090075,5460874.2149575595,5264516.494825061,5068158.774692543,4871801.054559983,4675443.334427489,4479085.614294961,4282727.894162459,4086370.174029963,3890012.453897409,3693654.7337649018,3497297.0136324074,3300939.293499889,3104581.573367346,2908223.853234824,2711866.13310232,2515508.412969811,2319150.6928373054,2122792.972704785,1926435.252572257,1730077.532439746,1533719.812307205,1337362.0921747163,1141004.372042179,944646.6519096904,748288.9317771643,551931.211644616,355573.4915121347,159215.77137959003,-37141.948752928525,-233499.66888544708,-429857.38901795447,-626215.1091504693,-822572.8292829916,-1018930.5494154505,-1215288.2695480697,-1411645.9896805733,-1608003.7098130435,-1804361.4299455807,-2000719.1500780657,-2197076.8702106364,-2393434.5903431326,-2589792.3104756363,-2786150.0306081474,-2982507.7507406995,-3178865.470873203,-3375223.1910057478,-3571580.9111382402,-3767938.6312707476,-3964296.351403266,-4160654.0715357587,-4357011.791668292,-4553369.511800788,-4749727.231933374,-4946084.952065833,-5142442.672198329,-5338800.392330844,-5535158.112463459,-5731515.832595907,-5927873.552728467,-6124231.272860903,-6320588.99299347,-6516946.713125981,-6713304.433258448,-6909662.153391011,-7106019.873523489,-7302377.5936560705,-7498735.313788563,-7695093.033921137,-7891450.7540536225,-8087808.47418613,-8284166.194318712,-8480523.914451182,-8676881.63458372,-8873239.354716219,-9069597.074848615,-9265954.794981115,-9462312.515113756,-9658670.235246323,-9855027.955378808,-10051385.675511248,-10247743.395643733,-10444101.11577636,-10640458.83590889],[8796353.81513734,8599496.789960505,8402639.764783677,8205782.739606859,8008925.71443001,7812068.689253181,7615211.664076354,7418354.638899515,7221497.613722694,7024640.588545845,6827783.563369027,6630926.538192178,6434069.513015354,6237212.487838529,6040355.462661686,5843498.4374848595,5646641.412308024,5449784.387131201,5252927.361954359,5056070.336777547,4859213.311600691,4662356.286423856,4465499.261247011,4268642.236070201,4071785.2108933944,3874928.1857165378,3678071.1605397016,3481214.135362871,3284357.110186048,3087500.085009208,2890643.059832372,2693786.03465553,2496929.0094787087,2300071.9843018726,2103214.9591250494,1906357.93394823,1709500.9087713659,1512643.8835945353,1315786.858417727,1118929.8332408778,922072.8080640435,725215.7828872465,528358.7577103525,331501.7325335555,134644.7073567137,-62212.31782016158,-259069.34299694374,-455926.36817380413,-652783.3933505975,-849640.4185274653,-1046497.4437042437,-1243354.468881119,-1440211.4940579645,-1637068.5192347392,-1833925.5444116332,-2030782.569588393,-2227639.5947653204,-2424496.6199421026,-2621353.6451189406,-2818210.6702957377,-3015067.695472572,-3211924.72064944,-3408781.7458262667,-3605638.771003116,-3802495.796179913,-3999352.8213567734,-4196209.846533552,-4393066.871710431,-4589923.896887265,-4786780.922064133,-4983637.947240904,-5180494.972417712,-5377351.997594573,-5574209.022771452,-5771066.047948211,-5967923.073125128,-6164780.09830185,-6361637.123478711,-6558494.148655564,-6755351.173832405,-6952208.199009232,-7149065.224186026,-7345922.249362931,-7542779.274539739,-7739636.2997165695,-7936493.32489346,-8133350.350070246,-8330207.375247121,-8527064.400424004,-8723921.425600708,-8920778.450777575,-9117635.475954317,-9314492.5011312,-9511349.52630803,-9708206.551484942,-9905063.576661713,-10101920.601838514,-10298777.627015285,-10495634.65219222,-10692491.677369125]],"type":"heatmap","xaxis":"x","yaxis":"y","hovertemplate":"Home Price: %{x}\u003cbr\u003eDownpayment: %{y}\u003cbr\u003eOpportunity: %{z}\u003cextra\u003e\u003c\u002fextra\u003e"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Home Price"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"autorange":"reversed","title":{"text":"Downpayment"}},"coloraxis":{"colorbar":{"title":{"text":"Opportunity"}},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]},"title":{"text":"Opportunity Surface"}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### `home_price_growth_rate` and `investment_return_rate`

In [14]:
_x, _y = np.linspace(0.02, 0.1, 100), np.linspace(0.02, 0.1, 100)
x, y = np.meshgrid(_x, _y, indexing="ij")
x = x.flatten()
y = y.flatten()
surface_kwargs = kwargs.copy()
surface_kwargs.update({"home_price_growth_rate": x, "investment_return_rate": y})
render_plotly_html(
    px.imshow(
        RentVsBuy().calculate(**surface_kwargs).value.reshape((100, 100)).T,
        aspect="auto",
        labels=dict(x="Home Price Growth", y="Investment Growth", color="Opportunity"),
        x=_x,
        y=_y,
        title="Opportunity Surface",
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="613fec3c-a22a-44da-a71f-d1c090a6dfcf" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("613fec3c-a22a-44da-a71f-d1c090a6dfcf")) {                    Plotly.newPlot(                        "613fec3c-a22a-44da-a71f-d1c090a6dfcf",                        [{"coloraxis":"coloraxis","name":"0","x":[0.02,0.02080808080808081,0.021616161616161617,0.022424242424242423,0.023232323232323233,0.024040404040404043,0.02484848484848485,0.025656565656565655,0.026464646464646465,0.027272727272727275,0.02808080808080808,0.028888888888888888,0.029696969696969697,0.030505050505050507,0.031313131313131314,0.03212121212121212,0.03292929292929293,0.03373737373737374,0.034545454545454546,0.03535353535353535,0.03616161616161616,0.03696969696969697,0.03777777777777778,0.038585858585858585,0.0393939393939394,0.0402020202020202,0.04101010101010101,0.041818181818181824,0.04262626262626262,0.043434343434343436,0.04424242424242424,0.04505050505050505,0.04585858585858586,0.04666666666666666,0.047474747474747475,0.04828282828282829,0.04909090909090909,0.0498989898989899,0.05070707070707071,0.051515151515151514,0.05232323232323233,0.053131313131313126,0.05393939393939394,0.05474747474747475,0.05555555555555555,0.056363636363636366,0.05717171717171718,0.05797979797979798,0.05878787878787879,0.05959595959595959,0.060404040404040404,0.06121212121212122,0.06202020202020202,0.06282828282828283,0.06363636363636364,0.06444444444444444,0.06525252525252526,0.06606060606060606,0.06686868686868687,0.06767676767676768,0.06848484848484848,0.0692929292929293,0.07010101010101011,0.07090909090909091,0.07171717171717172,0.07252525252525252,0.07333333333333333,0.07414141414141415,0.07494949494949495,0.07575757575757576,0.07656565656565657,0.07737373737373737,0.07818181818181819,0.07898989898989899,0.0797979797979798,0.08060606060606061,0.08141414141414141,0.08222222222222222,0.08303030303030304,0.08383838383838384,0.08464646464646465,0.08545454545454546,0.08626262626262626,0.08707070707070708,0.08787878787878788,0.08868686868686869,0.0894949494949495,0.0903030303030303,0.09111111111111111,0.09191919191919193,0.09272727272727273,0.09353535353535354,0.09434343434343435,0.09515151515151515,0.09595959595959597,0.09676767676767677,0.09757575757575758,0.09838383838383839,0.09919191919191919,0.1],"y":[0.02,0.02080808080808081,0.021616161616161617,0.022424242424242423,0.023232323232323233,0.024040404040404043,0.02484848484848485,0.025656565656565655,0.026464646464646465,0.027272727272727275,0.02808080808080808,0.028888888888888888,0.029696969696969697,0.030505050505050507,0.031313131313131314,0.03212121212121212,0.03292929292929293,0.03373737373737374,0.034545454545454546,0.03535353535353535,0.03616161616161616,0.03696969696969697,0.03777777777777778,0.038585858585858585,0.0393939393939394,0.0402020202020202,0.04101010101010101,0.041818181818181824,0.04262626262626262,0.043434343434343436,0.04424242424242424,0.04505050505050505,0.04585858585858586,0.04666666666666666,0.047474747474747475,0.04828282828282829,0.04909090909090909,0.0498989898989899,0.05070707070707071,0.051515151515151514,0.05232323232323233,0.053131313131313126,0.05393939393939394,0.05474747474747475,0.05555555555555555,0.056363636363636366,0.05717171717171718,0.05797979797979798,0.05878787878787879,0.05959595959595959,0.060404040404040404,0.06121212121212122,0.06202020202020202,0.06282828282828283,0.06363636363636364,0.06444444444444444,0.06525252525252526,0.06606060606060606,0.06686868686868687,0.06767676767676768,0.06848484848484848,0.0692929292929293,0.07010101010101011,0.07090909090909091,0.07171717171717172,0.07252525252525252,0.07333333333333333,0.07414141414141415,0.07494949494949495,0.07575757575757576,0.07656565656565657,0.07737373737373737,0.07818181818181819,0.07898989898989899,0.0797979797979798,0.08060606060606061,0.08141414141414141,0.08222222222222222,0.08303030303030304,0.08383838383838384,0.08464646464646465,0.08545454545454546,0.08626262626262626,0.08707070707070708,0.08787878787878788,0.08868686868686869,0.0894949494949495,0.0903030303030303,0.09111111111111111,0.09191919191919193,0.09272727272727273,0.09353535353535354,0.09434343434343435,0.09515151515151515,0.09595959595959597,0.09676767676767677,0.09757575757575758,0.09838383838383839,0.09919191919191919,0.1],"z":[[293634.8620260644,340005.20875476394,388048.48977105645,437819.12548396224,489373.2125720808,542768.5735457679,598064.8077225396,655323.3436559464,714607.4930561257,775982.506244801,839515.6291859513,905276.1621345195,973335.5199508206,1043767.2941217455,1116647.316540463,1192053.72508768,1270067.0310675944,1350770.188548834,1434248.6656593657,1520590.5178923258,1609886.4634758723,1702229.9608631283,1797717.2883988637,1896447.6262270291,1998523.1404924835,2104049.0699061076,2213133.8147312435,2325889.02826459,2442429.710868923,2562874.3066391796,2687344.8027593708,2815966.831635717,2948869.7758712983,3086186.8761642496,3228055.342205738,3374616.466661609,3526015.7423140067,3682402.9824548806,3843932.4446181147,4010762.9577307836,4183058.052790448,4360986.09714735,4544720.432497662,4734439.516686788,4930327.069417806,5132572.221973659,5341369.671064763,5556919.836901648,5779429.025611503,6009109.596116372,6246180.131584377,6490865.615580803,6743397.613042166,7004014.456202107,7272961.435596546,7550490.996287895,7836862.939445867,8132344.629413215,8437211.206429122,8751745.8051274,9076239.778982662,9410992.93086111,9756313.749816965,10112519.654326184,10479937.242106162,10858902.54669506,11249761.300992621,11652869.207902033,12068592.218301825,12497306.81650942,12939400.313450612,13395271.147724729,13865329.194778724,14349996.084388899,14849705.526696429,15364903.646975873,15896049.329412028,16443614.570065461,17008084.839324538,17589959.454045333,18189751.95963283,18807990.522366717,19445218.332172748,20101994.016158044,20778892.063160233,21476503.25962032,22195435.137048222,22936312.431386426,23699777.554588325,24486491.078716293,25297132.232878085,26132399.413351145,26993010.7071997,27879704.429764997,28793239.676370684,29734396.88859796,30703978.435518768,31702809.210263602,32731737.242316633,33791634.325935334],[248212.70756100002,294398.30374760786,342254.32315202407,391835.1471106226,443196.83258443256,496397.1617104106,551495.6927657761,608553.8125855913,667634.7904719068,728803.8336370676,792128.1442225534,857676.9779356229,925521.7043514838,995735.8689220375,1068395.2567429012,1143577.9581217687,1221364.4360012976,1301837.595286699,1385082.8541270872,1471188.2172075603,1560244.351105012,1652344.6617638348,1747585.3741481379,1846065.6141344868,1947887.492698729,2053156.1924657887,2161980.056680681,2274470.680673604,2390743.0058766804,2510915.416473709,2635109.8387405407,2763451.843161367,2896070.7493862337,3033099.7341118203,3174675.941961692,3320940.5994499857,3472039.132104655,3628121.284842394,3789341.245681831,3955857.7728755176,4127834.325567649,4305439.198056375,4488845.657766771,4678232.087033594,4873782.128788773,5075684.836262325,5284134.826808344,5499332.439955609,5721483.899801796,5950801.48186891,6187503.68453137,6431815.405143602,6683968.120990181,6944200.075187428,7212756.467663832,7489889.651359118,7775859.333779564,8070932.784037673,8375385.045549199,8689499.154504478,9013566.364287278,9347886.375997845,9692767.575221062,10048527.275231495,10415491.966783915,10793997.574662922,11184389.721193617,11587023.996852798,12002266.238209657,12430492.813357651,12872090.915051699,13327458.861740638,13797006.406707924,14281155.055519547,14780338.392025076,15295002.41309116,15825605.872342797,16372620.63309267,16936532.030756377,17517839.244954944,18117055.681558,18734709.364970896,19371343.340866756,20027516.089681856,20703801.95112411,21400791.56000464,22119092.293661345,22859328.731279884,23622143.12542857,24408195.886112418,25218166.077664614,26052751.928825293,26912671.35631229,27798662.50226421,28711484.285898004,29651916.969735354,30620762.740786422,31618846.307067327,32647015.50984411,33706141.952000424],[201689.89293767978,247686.16959880572,295350.28626366006,344736.5838274772,395901.0781374797,448901.50953270635,503797.393796667,560650.0745630851,619522.7772129602,680480.6643056208,743590.8925850708,808922.6716039572,876547.3240128243,946538.3475557202,1018971.4788238704,1093924.758810467,1171478.600319731,1251715.8572804388,1334721.8960130098,1420584.6685070326,1509394.7877622978,1601245.6052494645,1696233.2905469965,1794456.9132183418,1896018.5269829337,2001023.2562499135,2109579.38507277,2221798.4485978,2337795.3270639093,2457688.342435183,2581599.3577237185,2709653.879088123,2841981.160772929,2978714.312970948,3119990.412684805,3265950.6176715484,3416740.2835465465,3572509.0841387026,3733411.135183615,3899605.121435162,4071254.427302555,4248527.271091497,4431596.842955779,4620641.446658202,4815844.645235918,5017395.410678792,5225488.2777324375,5440323.501925564,5662107.2219404,5891051.6264440715,6127375.125492135,6371302.526631202,6623065.215823776,6882901.343323996,7151056.014631871,7427781.486665651,7713337.369289983,8007990.832328072,8312016.818230581,8625698.260518458,8949326.308172788,9283200.556128314,9627629.282011587,9982929.689315546,10349428.157158945,10727460.496804498,11117372.215137407,11519518.78524398,11934265.924319126,12361989.87906449,12803077.718791358,13257927.636418227,13726949.257575983,14210563.958019864,14709205.189593684,15223318.814926136,15753363.451134082,16299810.822713373,16863146.123914797,17443868.390806794,18042490.883278027,18659541.47728329,19295563.067533657,19951113.980949212,20626768.4011242,21323116.804114535,22040766.405816536,22780341.62124237,23542484.536008596,24327855.390343137,25137133.075928792,25971015.6459334,26830220.838530988,27715486.61429455,28627571.7078024,29567256.193812553,30535342.06839346,31532653.845387813,32560039.168601677,33618369.44011663],[154033.002793191,199835.47463606065,247303.1336664944,296490.2790282918,347452.88411807036,400248.64611306135,454937.0369115658,511579.35552623915,570238.7819685349,630980.4326668782,693871.4174599317,758980.8982072598,826380.1490650373,896142.6184678813,968343.9928685157,1043062.2622783068,1120377.7876618248,1200373.3702356266,1283134.3227203432,1368748.5426029298,1457306.5874621985,1548901.7524136724,1643630.1497304598,1741590.7907041172,1842885.6697989944,1947619.851169074,2055901.5575954108,2167842.26191708,2283556.781013205,2403163.3724174113,2526783.833622255,2654543.6041589943,2786571.8705179263,2923001.673991405,3063970.0215156097,3209617.9995951666,3360090.891386612,3515538.297032861,3676114.257335267,3841977.3808437726,4013290.974472083,4190223.1777166463,4372947.100585679,4561640.9653370585,4756488.252120349,4957677.84863135,5165404.203891008,5379867.486248102,5601273.745724717,5829835.080822142,6065769.809898526,6309302.6472452255,6560664.883984805,6820094.573919597,7087836.724458219,7364143.4927598,7649274.387233455,7943496.47452132,8247084.592137819,8560321.566882337,8883498.439198405,9216914.693636052,9560878.495558228,9915706.934283141,10281726.272810832,10659272.20430801,11048690.115552604,11450335.357477868,11864573.523044784,12291780.73260447,12732343.92696482,13186661.16835106,13655141.949473467,14138207.51090103,14636291.16698692,15149838.640525213,15679308.406414025,16225172.044505455,16787914.601939984,17368034.965166904,17966046.24190391,18582476.1533393,19217867.436777648,19872778.25904734,20547782.640919708,21243470.892849788,21960450.062307548,22699344.393004883,23460795.796335045,24245464.335329555,25054028.721450895,25887186.824570872,26745656.196439255,27630174.608022988,28541500.601058193,29480414.05416919,30447716.763943005,31444233.041335966,32470810.323804654,33528319.80355888],[105208.78264742857,150813.03759670816,198079.76000163052,247063.2060154355,297819.30532017536,350405.7106420095,404881.8486788999,461308.97248096904,519750.2153217355,580270.6461028736,642937.3263338506,707819.3687286847,774987.9974675518,844516.6101642144,916480.8415911002,990958.6292049494,1068030.2805262357,1147778.5424225638,1230288.672345092,1315648.511574842,1403948.5605320279,1495282.0562044075,1589745.051751405,1687436.4983478514,1788458.329321031,1892915.5466497904,2000916.309884049,2112572.027557437,2227997.4511507344,2347310.7716873833,2470633.7190187005,2598091.6638840027,2729813.7228110917,2865932.8659389336,3006586.027838856,3151914.221418161,3302062.6549822595,3457180.8525474146,3617422.77749075,3782946.959617896,3953916.625755304,4130499.833946021,4312869.611355003,4501204.0959829725,4695686.68228397,4896506.1707949955,5103856.9218895715,5317939.013754673,5538958.404710017,5767127.099987214,6002663.323080392,6245791.6917948555,6496743.3991171,6755756.399034751,7023075.597434083,7298953.048214637,7583648.154758705,7877427.876883747,8180566.943450558,8493348.07074431,8816062.186801575,9149008.661839891,9492495.544930924,9846839.807108914,10212367.591062797,10589414.467586016,10978325.69898536,11379456.509588882,11793172.363581335,12219849.250329144,12659873.977408806,13113644.471528668,13581570.087557133,14064071.925856188,14561583.158165984,15074549.36222003,15603428.865366038,16148693.097372914,16710826.952721378,17290329.162579942,17887712.676719263,18503505.055668317,19138248.87331328,19792502.130257446,20466838.678191956,21161848.655587338,21878138.934974536,22616333.582121126,23377074.327418774,24161021.04978755,24968852.273414977,25801265.67767987,26658978.620565567,27542728.67594269,28453274.185063843,29391394.822624277,30357892.17777708,31353590.350479394,32379336.56356207,33436001.79092014],[55183.14838684397,100584.84027770953,147646.21539373742,196421.48572052084,246966.53602510225,299338.9733582712,353598.17796901334,409805.3556714528,468023.5917023709,528317.9061120255,590755.3107296415,655404.8677457413,722337.7499590255,791627.3027289836,863349.107685661,937581.0482399096,1014403.376947083,1093898.7847744091,1176152.47232114,1261252.2230483145,1349288.4785711863,1440354.4160704734,1534546.0278789927,1631962.2033076752,1732704.8127645254,1836878.7942353622,1944592.242184633,2055956.4989489997,2171086.248681464,2290099.613927132,2413118.254888415,2540267.471464782,2671676.308132441,2807477.661745985,2947808.3923380617,3092809.437001149,3242625.926927482,3397407.3076991616,3557307.4629151374,3722484.8412354714,3893102.586949838,4069328.674149021,4251336.044605619,4439302.7494627945,4633412.094826315,4833852.791368227,5040819.108054021,5254511.030092641,5475134.421228461,5702901.190492662,5938029.463525476,6180743.758596111,6431275.167443443,6689861.541066179,6956747.680590041,7232185.5333515685,7516434.3943361575,7809761.113098667,8112440.306338966,8424754.576250076,8746994.734811503,9079460.034184622,9422458.403351054,9776306.691185731,10141330.916113026,10517866.5225199,10906258.64412751,11306862.374461241,11720043.044647548,12146176.508699697,12585649.436506301,13038859.614712477,13506216.255706748,13988140.314912599,14485064.81663024,14997435.188608453,15525709.605621096,16070359.342229186,16631869.13502569,17210737.554564998,17807477.387229938,18422616.02733972,19056695.879699837,19710274.772912156,20383926.38369494,21078240.672522772,21793824.330855183,22531301.24025941,23291312.943743676,24074519.129606187,24881598.128118053,25713247.421389997,26570184.16672744,27453145.733854163,28362890.25634688,29300197.197634734,30265867.93195222,31260726.340622205,32285619.42406116,33341417.929904185],[3921.869318964891,49116.64677465055,95968.2588283103,144530.87211277755,194860.32527106162,247014.17844510498,301051.76417656336,357034.2397597092,415024.6410847567,475087.9380142321,537291.0913336803,601703.1113190623,668395.1179683809,737440.4029387115,808914.4932402521,882895.2167304689,959462.7694614385,1038699.7849305798,1120691.4052838814,1205525.354528401,1293292.0138072008,1384084.498792722,1477998.7392552597,1575133.5608705394,1675590.7693198142,1779475.2367515224,1886894.9906626092,1997961.3052723105,2112788.795446098,2231495.513250968,2354203.0471997284,2481036.6242695376,2612125.21475994,2747601.6400725665,2887602.6834883685,3032269.204026669,3181746.2534618736,3336183.1965900385,3495733.8348318357,3660556.533252411,3830814.3511050288,4006675.1759773227,4188311.861646236,4375902.369740641,4569629.915306715,4769683.11638455,4976256.147707684,5189548.898625143,5409767.135364823,5637122.667755779,5871833.520521002,6114124.109267195,6364225.421294933,6622375.20135762,6888818.14249706,7163806.082094977,7447598.203278304,7740461.241806358,8042669.698612513,8354506.058117736,8676261.012488784,9008233.69199783,9350731.901624471,9704072.364091737,10068580.969484556,10444593.031624608,10832453.551402908,11232517.487210156,11645150.03269324,12070726.901999893,12509634.62272545,12962270.836751446,13429044.609189356,13910376.745628033,14406700.117930792,14918459.998761635,15446114.405115444,15990134.451032897,16551004.709797313,17129223.585815266,17725303.69643396,18339772.263998732,18973171.518351432,19626059.110088315,20299008.534826815,20992609.568791386,21707468.715987045,22444209.667266123,23203473.771604545,23985920.51989287,24792228.04156024,25623093.614381038,26479234.18776896,27361386.91993863,28270309.72927686,29206781.86027802,30171604.46443151,31165601.196438268,32189618.826148324,33244527.866617024],[-48610.79284690367,-3627.2089894833043,43010.299167258665,91355.85179095622,141465.2402659338,193395.97666807286,247207.344651334,302960.4517862494,360718.2833885085,420545.75788039155,482509.78372620046,546679.3179841931,613125.4265223844,681921.3459395631,753142.5472429609,826866.801325785,903174.246297624,982147.4567179568,1063871.514781843,1148434.0835145782,1235925.482028469,1326438.7628977378,1420069.7917081737,1516917.3288455717,1617083.113576367,1720671.9504894395,1827791.7983572893,1938553.8614892666,2053072.6836346374,2171466.244516603,2293856.059054949,2420367.2793625346,2551128.799580976,2686273.3636374814,2825937.675998903,2970262.5155071453,3119392.852371873,3273477.9684125925,3432671.5806367113,3597131.9682340845,3767022.103094887,3942509.7839295836,4123767.7740972186,4310973.943240833,4504311.412825215,4703968.705685345,4910139.899697328,5123024.78567127,5342829.029585003,5569764.339276256,5804048.635704638,6045906.228910258,6295567.9987920225,6553271.580834258,6819261.556909386,7093789.651295882,7377114.932049498,7669504.017855757,7971231.290536293,8282579.1133264955,8603838.055097105,8935307.12067667,9277293.987415679,9630115.24818409,9994096.660950556,10369573.405117419,10756890.344812682,11156402.299279114,11568474.320588682,11993481.978844468,12431811.65508391,12883860.842073161,13350038.453205701,13830765.13970404,14326473.616370084,14837608.996063937,15364629.133185782,15908004.976341693,16468220.930490434,17045775.22877326,17641180.314279526,18254963.232051414,18887666.031528898,19539846.17975298,20212076.985577054,20904948.035196237,21619065.63926357,22355053.29189823,23113552.14190242,23895221.476491895,24700739.21785833,25530802.4329134,26386127.856519125,27267452.42858461,28175533.84537138,29111151.125361666,30075105.190077513,31068219.460227937,32091340.46757568,33145338.482921645],[-102448.87126111379,-57680.79811319616,-11261.772626793943,36862.2777956198,86747.09615291608,138450.14530572854,192030.6588483518,247549.6934322915,305070.182579685,364656.9920292129,426376.9766558814,490299.0390068549,556494.1895010234,625035.608333386,695998.7091358537,769461.2044375748,845503.1729778629,924207.1289218995,1005658.0930282809,1089943.6658252603,1177154.102848724,1267382.391997981,1360724.3330660122,1457278.619508055,1557146.9225021359,1660433.9773703502,1767247.6724191662,1877699.1402714318,1991902.8517478057,2109976.712378766,2232042.161604902,2358224.2747505778,2488651.8678364167,2623457.605312556,2762778.1107886476,2906754.0808448247,3055530.4019994894,3209256.2709261077,3368085.318005464,3532175.734293963,3701690.402014791,3876797.028650732,4057668.2847447963,4244481.945507443,4437421.036325728,4636673.9822825715,4842434.761797991,5054903.0644918345,5274284.453386692,5500790.531568836,5734639.113418369,5976054.400535404,6225267.162485402,6482514.922492284,6748042.148206863,7022100.44769014,7304948.770749069,7596853.615753121,7898089.242103957,8208937.8884757785,8529689.996999003,8860644.443544086,9202108.774246406,9554399.44846373,9917842.088314785,10292771.73497285,10679533.111915547,11078480.895271063,11489979.991488941,11914405.822497595,12352144.618562307,12803593.719033614,13269161.881199107,13749269.59743747,14244349.420920435,14754846.300042149,15281217.921850977,15823935.06466411,16383481.960162498,16960356.66516762,17555071.443353377,18168153.157195993,18800143.67036321,19451600.26086088,20123096.045186512,20815220.41379995,21528579.478179738,22263796.52977059,23021512.51113833,23802386.499637473,24607096.203909516,25436338.473561794,26290829.822331484,27171306.965114865,28078527.369203944,29013269.820084773,29976335.002185546,30968546.0949513,31990749.384637065,33043814.892217338],[-157629.47530090762,-113081.16915916,-66884.94173170859,-18986.769024546258,30669.042522213887,82139.90547990752,135485.00228816597,190765.33756309748,248043.79189508362,307385.1771788071,368856.2935168212,432525.9877389157,498465.2135849083,566747.0935919536,637446.9827379575,710642.5338842389,786413.7650704286,864843.128711849,946015.5827484056,1030018.663801841,1116942.562394382,1206880.200284849,1299927.3099788916,1396182.5164771858,1495747.4213151922,1598726.6889633564,1705228.1356459036,1815362.8206509356,1929245.140189637,2046992.9238855918,2168727.533951977,2294573.967141809,2424660.9595364477,2559121.094254563,2698090.9121573684,2841711.0256343544,2990126.235545454,3143485.6514116973,3301942.8149408908,3465655.8269688645,3634787.4779231073,3809505.3818875575,3989982.1143746586,4176395.3539036224,4368928.027479941,4567768.460084648,4773110.528284901,4985153.81806554,5204103.787000324,5430171.930880554,5663575.954912379,5904539.9496095525,6153294.57150472,6410077.22880789,6675132.272139616,6948711.190478425,7231072.812460003,7522483.513156701,7823217.426509248,8133556.663528697,8453791.53644088,8784220.78893032,9125151.832624562,9476900.990010353,9839793.743930098,10214164.993832694,10600359.318979807,10998731.248747736,11409645.540253062,11833477.463464355,12270613.09401334,12721449.613895804,13186395.620274976,13665871.442586355,14160309.468189456,14670154.476746295,15195863.983601125,15737908.592342379,16296772.356843704,16872953.152986083,17466963.060314048,18079328.75392886,18710591.906820104,19361309.602953482,20032054.761364806,20723416.571569726,21436000.940558333,22170430.951679632,22927347.335732486,23707408.954567935,24511293.297521215,25339696.991023067,26193336.321695056,27072947.773308918,27979288.577952035,28913137.281753384,29875294.325557932,30866582.640926376,31887848.26185207,32939960.952592745],[-214189.17213718686,-169864.90152979596,-123895.7991515752,-76227.89062431175,-26805.532859023195,24428.635374454316,77533.74428880634,132570.74537164858,189602.46516728494,248693.66059040232,309911.07581298705,373323.50076681515,439001.8313089474,507019.13109153695,577450.695187388,650374.1155144153,725869.3481120681,804018.7823198447,884907.3119070367,968622.4082104485,1055254.1953331772,1144895.527460514,1237642.068349556,1333592.3730564872,1432847.971954971,1535513.4571146555,1641696.5710978433,1751508.2982471115,1865062.9585215966,1982478.3039629827,2103875.6178489923,2229379.8166194344,2359119.5546401553,2493227.3318869486,2631839.6046253024,2775096.899170199,2923143.928801908,3076129.7139297123,3234207.7055902556,3397535.9123609583,3566277.0307952915,3740598.5794586753,3920673.0366712627,4106677.9820561907,4298796.2419887725,4497216.039054706,4702131.145629143,4913741.041676187,5132251.076887447,5357872.637277392,5590823.316346839,5831327.090941256,6079614.501926971,6335922.839813959,6600496.335452706,6873586.355944663,7155451.605903773,7446358.334197685,7746580.546340473,8056400.222654889,8376107.542376417,8706001.113856107,9046388.211003006,9397585.016157743,9759916.86954568,10133718.52548359,10519334.415540991,10917118.918796338,11327436.63941615,11750662.691719294,12187182.99294005,12637394.56387987,13101705.837660879,13580536.976779856,14074320.19870831,14583500.110218238,15108534.05070838,15649892.444711529,16208059.163880147,16783531.898651987,17376822.53984876,17988457.5705109,18618978.468169503,19268942.117873643,19938921.236222513,20629504.806712583,21341298.526668143,22074925.26606083,22831025.538534403,23610257.98493971,24413299.869698133,25240847.590343192,26093617.200544845,26972344.946996585,27877787.82050741,28810724.121652972,29771954.04137373,30762300.256897297,31782608.54337644,32833748.401640557],[-272165.563295912,-228069.63025723817,-182332.0136604947,-134898.78976490628,-85714.36699401727,-34721.43652093876,18139.07855709968,72928.07549495297,129708.32566483831,188544.52985325037,249503.37513076141,312653.5933371354,378066.0212291125,445813.66233218927,515971.7505477951,588617.8155590901,663831.7500883639,741695.8790561645,822295.0306913326,905716.6096486798,992050.6721872739,1081390.0034655803,1173830.1970099062,1269469.7364200703,1368410.0793659063,1470755.7439433802,1576614.3974485332,1686096.9476419361,1799317.6365614235,1916394.1369640552,2037447.6514551793,2162603.0143896337,2291988.7966103526,2425737.4131065197,2563985.2336670575,2706872.69661375,2854544.425689792,3007149.350195886,3164840.82846037,3327776.774724001,3496119.7895459323,3670037.293810018,3849701.666437193,4035290.3859030437,4226986.17565552,4424977.153541321,4629456.985352433,4840625.042592512,5058686.56458181,5283852.825018219,5516341.30310587,5756375.859377907,6004186.916336552,6260011.644039067,6524094.150757154,6796685.678849246,7078044.805983237,7368437.651838139,7668138.09045659,7977427.968366153,8296597.328641665,8625944.641065631,8965777.038527476,9316410.559853166,9678170.399213545,10051391.162285533,10436417.129367009,10833602.52558583,11243311.798430908,11665919.902767584,12101812.59355087,12551386.726426575,13015050.566433184,13493224.105003383,13986339.3855106,14494840.837540459,15019185.620161558,15559843.974376556,16117299.58505027,16692049.95251698,17284606.77411967,17895496.33598423,18525259.915229887,19174454.19293379,19843651.678099416,20533441.142938696,21244428.069736574,21977235.109603215,22732502.55343029,23510888.8153563,24313070.92905908,25139745.05722516,25991627.014500685,26869452.80430361,27773979.169839695,28705984.159676004,29666267.7082604,30655652.231763467,31674983.23963498,32725129.962272137],[-331597.09012587974,-287733.8069397644,-242232.0463186144,-195037.93615368987,-146095.93739502085,-95348.7946526315,-42737.485387457535,11798.832348278724,68322.87365943659,126899.28208763106,187594.68646352924,250477.75937409932,315619.2772922339,383092.1824100339,452971.6462271181,525335.1349373027,600262.4766664216,677835.9306116058,758140.258131098,841262.7958412501,927293.5307739619,1016325.1776503255,1108453.2583274292,1203776.1834818423,1302395.3365835948,1404415.160229342,1509943.2448929674,1619090.4201662266,1731970.848547284,1848702.1218580073,1969405.3603479862,2094205.3145700814,2223230.4700931343,2356613.155133547,2494489.6511818566,2637000.306708387,2784289.654023846,2936506.529387008,3093804.196445836,3256340.47309279,3424277.861840786,3597783.683798923,3777030.2163537764,3962194.8346552406,4153460.1570020597,4351014.194235306,4555050.503251547,4765768.344735164,4983372.845228672,5208075.163658445,5440092.662427468,5679649.083201539,5926974.72751206,6182306.642304254,6445888.810557838,6717972.347120178,6998815.699889013,7288684.856473377,7587853.556504786,7896603.509716224,8215224.619961731,8544015.215332974,8883282.284514073,9233341.719565894,9594518.565288322,9967147.275334492,10351571.975277916,10748146.732772889,11157235.835035995,11579214.073811121,12014467.038031392,12463391.414368046,12926395.295879178,13403898.498957098,13896332.888819816,14404142.713726398,14927784.948190656,15467729.645374095,16024460.298954878,16598474.214674901,17190282.891817734,17800412.41492042,18429403.855920374,19077813.687055346,19746214.204765975,20435193.96491094,21145358.229563285,21877329.425693244,22631747.616053887,23409270.98257444,24210576.32257964,25036359.558184586,25887336.25916971,26764242.17971585,27667833.80934163,28598888.938396927,29558207.238500938,30546610.858301297,31564945.034946054,32614078.721666157],[-392523.15635895263,-348896.8426006753,-303635.3144882228,-256684.7525065234,-207989.67111210572,-157492.86935077142,-105135.38006513426,-50856.41765206121,5406.675668393262,63718.48511507688,124145.5801800657,186756.5730776419,251622.17885530088,318815.2772080982,388410.9760477375,460486.67686950136,535122.1419701544,612399.5635668575,692403.6348662334,775221.6231402988,860943.444862341,949661.7429588004,1041471.9662336903,1136472.4510294623,1234764.505177875,1336452.4943096642,1441643.9305812032,1550449.563890745,1662983.4756421954,1779363.1751370993,1899709.698652965,2024147.7112926086,2152805.611670191,2285815.639515651,2423313.98627367,2565440.9087811024,2712340.846099025,2864162.5395911485,3021059.1563352873,3183188.4159483984,3350712.7209317777,3523799.290615421,3702620.2988073905,3887353.015247181,4078179.9509581514,4275289.007607333,4478873.630984274,4689132.968698481,4906272.032214142,5130501.863339715,5362039.705283782,5601109.178403762,5847940.460770495,6102770.473677603,6365843.072222494,6637409.241099338,6917727.2957406435,7207063.088936439,7505690.22310294,7813890.268318244,8131952.986297783,8460176.56046607,8798867.83226577,9148342.543895531,9508925.587624876,9880951.261860233,10264763.534162931,10660716.311359696,11069173.716973215,11490510.376135431,11925111.708196722,12373374.227221055,12835705.850580128,13312526.215845123,13804267.006221483,14311372.284706669,14834298.837245114,15373516.525061447,15929508.646468645,16502772.308353104,17093818.80758962,17703174.022688985,18331378.81587955,18978989.44594066,19646577.99203754,20334732.788867507,21044058.87338615,21775178.443418708,22528731.32847298,23305375.473058753,24105787.432831638,24930662.88391136,25780717.14567861,26656685.71743061,27559324.829237632,28489412.007354096,29447746.654572807,30435150.645898677,31452468.93993395,32500570.206372112],[-454983.1586537468,-411598.19132538047,-366581.3300356292,-319878.80882513756,-271435.19663734967,-221193.34795292933,-169094.3520144322,-115077.48060106346,-59080.13431535009,-1037.7873390978202,59116.0693825949,121449.98657163652,186034.61670889938,252942.77582920343,322249.5070706201,394032.1460213894,468370.38791731326,545346.3567398377,625044.6762639112,707552.5431123432,792959.8018696555,881359.0223115687,972845.5788066657,1067517.7319539743,1165476.7125102594,1266826.8076755786,1371675.4497954375,1480133.3075521016,1592314.379702947,1708336.0914465426,1828319.3934746692,1952388.863794825,2080672.8123889468,2213303.3887900305,2350416.6926527875,2492152.887402234,2638656.3170363824,2790075.6261747656,2946563.8834394757,3108278.7082492006,3275382.4011328784,3448042.0776419165,3626429.8059668546,3810722.748357445,4001103.306441125,4197759.27054833,4400883.973156179,4610676.446550098,4827341.584822141,5051090.310323397,5282139.744682105,5520713.384513954,5767041.2819473585,6021360.230093059,6283913.953584502,6554953.304329381,6834736.462609279,7123529.14365583,7421604.80987588,7729244.888842668,8046738.997226107,8374385.170818408,8712490.100796388,9061369.376411408,9421347.73425575,9792759.314279165,10175947.922756456,10571267.30234667,10979081.409471516,11399764.699175391,11833702.417680448,12281290.902826704,12742937.89261001,13219062.84201666,13710097.248400059,14216484.985579249,14738682.646933563,15277159.897674525,15832399.836591408,16404899.367472807,16995169.580456823,17603736.143612683,18231139.7049552,18877936.305209912,19544697.801578347,20232012.302813392,20940484.61587344,21670736.704460274,22423408.159757316,23199156.683672894,23998658.584906746,24822609.288189307,25671723.85699842,26546737.53013309,27448406.272486906,28377507.340374336,29334839.861798853,30321225.432038814,31337508.72494338,32384558.120335512],[-519018.7522308687,-475879.5075401268,-431111.7453775839,-384661.75426158775,-336474.1585617657,-286491.8691505189,-234656.03264515707,-180905.97920038505,-125179.16881304653,-67411.1360965157,-7535.433483396191,54516.42718565464,118815.03473446332,185433.14042655937,254445.72149428772,325930.04646524694,399965.74233878404,476634.86366279656,556021.9635599433,638214.1667599087,723301.2446908373,811375.692685945,902532.8093618564,996870.7782325125,1094490.7516122898,1195496.9368769755,1299996.6851409045,1408100.5824227347,1519922.54335785,1635579.907538054,1755193.5385366045,1878887.9257033137,2006791.2887954363,2139035.685525766,2275757.122104432,2417095.6668579793,2563195.567002063,2714205.3686594274,2870278.0402098303,3031571.0990524087,3198246.741887054,3370471.9785938226,3548418.76981613,3732264.1683467734,3922190.464411746,4118385.3349602465,4321041.997072405,4530359.365584269,4746542.215048934,4969801.346150926,5200353.756685673,5438422.81723049,5684238.451629821,5938037.322423925,6200063.021347853,6470566.265040647,6749805.096101844,7038045.0896239355,7335559.565372732,7642629.80573322,7959545.279593447,8286603.8723230995,8624112.12198776,8972385.461990058,9331748.47028622,9702535.125352085,10085089.069098951,10479763.876880419,10886923.334817266,11306941.724603035,11740204.11600373,12187106.667241458,12648056.933474932,13123474.183575645,13613789.725444932,14119447.240051847,14640903.124466123,15178626.844067162,15733101.294225696,16304823.171660038,16894303.355720077,17502067.299901262,18128655.433790363,18774623.575760573,19440543.356665425,20127002.65484171,20834606.04268966,21563975.245135605,22315749.61029371,23090586.592631176,23889162.24895545,24712171.747572623,25560329.890921935,26434371.65206571,27335052.725377522,28263150.09178166,29219462.598932922,30204811.556712378,31220041.348431826,32266020.058143444],[-584672.0009822776,-541782.8869801429,-497268.68771866104,-451075.74709728453,-403148.74583193334,-353430.6521239239,-301862.6709198635,-248384.19172223518,-192932.73491260177,-135443.89654466184,-75851.29156616936,-14086.495427139103,49921.01597286435,116243.92803975288,184957.15099349385,256137.8851736593,329865.6881945445,406222.54400035366,485292.9338692571,567163.9094232111,651925.1676964508,739669.1283187508,830491.0128700812,924488.9264702634,1021763.9416574738,1122420.1846241318,1226564.9238685025,1334308.661334442,1445765.226097351,1561051.870676835,1680289.3700343245,1803602.1233402025,1931118.2585761566,2062969.740054367,2199292.478929681,2340226.4467886607,2485915.792391659,2636508.961659605,2792158.820992172,2953022.783997796,3119262.9417421813,3291046.19659408,3468544.3997744694,3651934.49270771,3841398.652269993,4037124.4400433246,4239304.955686483,4448138.994522732,4663831.209462762,4886592.277380401,5116639.070052638,5354194.829790344,5599489.349882653,5852759.159983978,6114247.716570543,6384205.598606392,6662890.70855592,6950568.478871498,7247512.084128292,7554002.658923591,7870329.521713518,8196790.40474333,8533691.690212773,8881348.652867194,9240085.709163386,9610236.673183687,9992145.019499285,10386164.153123293,10792657.686781019,11211999.72565994,11644575.1598527,12090779.964683142,12551021.509127986,13025718.872533325,13515303.169870704,14020217.885713052,14540919.217204515,15077876.426205164,15631572.200907204,16202503.02712467,16791179.569509503,17398127.06299652,18023885.71467882,18669011.116431244,19334074.66853152,20019664.014588825,20726383.488048427,21454854.57057747,22205716.36264825,22979626.066623755,23777259.482663594,24599311.517799783,25446496.708487004,26319549.75700687,27219226.082068846,28146302.38396082,29101577.2246385,30085871.623129092,31100029.666641936,32144919.13778277],[-651985.610999682,-609351.0932152933,-565094.9784279913,-519163.6668219627,-471501.8963796869,-422052.69356868276,-370757.3226194605,-317555.2333545815,-262384.0075305933,-205179.30365025718,-145874.80020388868,-84402.13729741517,-20690.856619847473,45331.66029117303,113740.25553704146,184612.06014236296,258026.56118843053,334065.6708462215,412813.79735798296,494357.9180241022,578787.6542482409,666195.3486968405,756676.144629498,850328.0674640425,947252.1086298237,1047552.3117781235,1151335.8614076292,1258713.1739776977,1369797.9915672843,1484707.4781601038,1603562.3186142202,1726486.8204006702,1853609.0181767605,1985060.7812756398,2120977.9241884155,2261500.3201224655,2406772.017712303,2556941.3609745298,2712161.1125936415,2872588.5806190437,3038385.748679909,3209719.409796861,3386761.303896182,3569688.259125605,3758682.3370665954,3953930.9819515115,4155627.173997202,4363969.586954411,4579162.749992056,4801417.214033245,5030949.722655101,5267983.387678418,5512747.869570282,5765479.5627885815,6026421.78619518,6295824.978677884,6573946.900117968,6861052.837832205,7157415.818660955,7463316.826820213,7779045.027689953,8104897.997695286,8441181.960421711,8788212.029155325,9146312.45599664,9515816.887721922,9897068.628592603,10290420.91025357,10696237.168947598,11114891.33020853,11546768.101246381,11992263.271214424,12451784.01957093,12925749.23273449,13414589.829277951,13918749.09384096,14438683.020035308,14974860.662523992,15527764.498570522,16097890.799260508,16685750.01064846,17291867.145132117,17916782.183255993,18561050.486261643,19225243.21963429,19909947.787955508,20615768.28133061,21343325.933695737,22093259.593321115,22866226.20581495,23662901.309946403,24483979.546636753,25330175.181423686,26202222.640777793,27100877.062614318,28026914.861352913,28981134.30791442,29964356.125030473,30977424.098258175,32021205.703096725],[-721003.7861596555,-678628.3874577684,-634634.936579749,-588969.8909098767,-541578.0466296268,-492402.48942255974,-441384.5437707044,-388463.72080242913,-333577.66465348005,-276662.09729875857,-217650.76181345154,-156475.36402134178,-93065.51248286804,-27348.656781607773,40749.97594200028,111307.44625311624,184403.16953379475,260118.9849934578,338539.22662751656,419750.7961808089,503843.23816894833,590908.8170135538,681042.5963479406,774342.5205569812,870909.4986048327,970847.4902192149,1074263.5944902934,1181268.1409569392,1291974.7832380268,1406500.5952895442,1524966.1703456203,1647495.7226280035,1774217.1918897321,1905262.3508745027,2040766.9157680222,2180870.6597250476,2325717.5295483023,2475455.765611017,2630238.0251095877,2790221.508726987,2955568.090813256,3126444.4531622734,3303022.222490427,3485478.111716139,3673994.0651352387,3868757.4076006087,4069960.99781748,4277803.385853848,4492488.974985165,4714228.187989831,4943237.638007972,5179740.304089094,5423965.711552118,5676150.117286392,5936536.700120565,6205375.756399545,6482924.900906138,6769449.273256276,7065221.749939566,7370523.162122862,7685642.519389212,8010877.239568806,8346533.384803036,8692925.904032538,9050378.882057982,9419225.795347363,9799809.774790326,10192483.87554045,10597611.354172606,11015565.953317983,11446732.19399005,11891505.675791346,12350293.385213852,12823514.012231851,13311598.27543217,13814989.255861998,14334142.73986815,14869527.571109042,15421626.012035467,15990934.115042603,16577962.103545826,17183234.763282772,17807291.84404337,18450688.47214517,19113995.5739037,19797800.310407512,20502706.523866415,21229335.19583811,21978324.917649437,22750332.373316698,23546032.83528376,24366120.673326477,25211309.876928955,26082334.59151029,26979949.668844856,27904931.232029095,28858077.25538346,29840208.15966558,30852167.422986776,31894822.20782852],[-791772.2573207947,-749660.5196252028,-705934.3303477578,-660540.2049577613,-613422.9987368309,-564525.8575022304,-513790.1669220524,-461155.50038219104,-406559.5653668968,-349938.14831034467,-291225.05787802907,-230352.0666356082,-167248.85105793318,-101842.9298368874,-34059.60043684067,36178.12614565436,108949.59247065429,184336.56364229554,262423.2982439995,343296.6212781733,427045.9991632295,513763.616844133,603544.4570729341,696486.3819229803,792690.2165905992,892259.8355527096,995302.2511387328,1101927.704589182,1212249.7596589625,1326385.3988458444,1444455.1223023892,1566583.0495157507,1692897.0238211094,1823528.7198303575,1958613.7538520605,2098291.797386572,2242706.6937725022,2392006.578076151,2546344.000310435,2705876.0520639727,2870764.496646639,3041175.902830775,3217281.782293466,3399258.7308592787,3587288.5736379307,3781558.5141656874,3982261.2876616614,4189595.3184984596,4403764.882006432,4624980.270728007,4853457.965234426,5089420.809630782,5333098.1918724235,5584726.229021695,5844547.957571741,6112813.528977382,6389780.410530011,6675713.5917051835,6970885.796154528,7275577.6994599635,7590078.152822254,7914684.412840729,8249702.377525011,8595446.828730065,8952241.681162665,9320420.238133622,9700325.454255953,10092310.205229932,10496737.56494224,10913981.090041663,11344425.112204667,11788465.038280623,12246507.658529656,12718971.463151596,13206286.967351345,13708897.045120489,14227257.272009255,14761836.27706983,15313116.104267348,15881592.583560683,16467775.711905915,17072190.04448454,17695375.09635843,18337885.754868686,19000292.70302802,19683182.854216605,20387159.79844942,21112844.260520667,21860874.570341304,22631907.145774078,23426616.988284633,24245698.19175776,25089864.464783356,25959849.666791607,26856408.358379878,27780316.366184346,28732371.362685233,29713393.461321328,30724225.827306047,31765735.30454185],[-864336.646619719,-822493.1956500271,-779038.9503673147,-733920.4852246796,-687082.7155191633,-638468.8481338262,-588020.3308720193,-535676.8003438544,-481376.02836658433,-425053.86683637556,-366644.19103021733,-306078.84129579365,-243287.56308180885,-178197.94526753016,-110735.35674042255,-40822.88117836835,31618.750016185455,106671.22668026155,184418.7308018338,264948.00943518477,348348.4496732452,434712.1557330149,524134.02821044344,616711.8455685391,712546.3479124727,811741.3231201335,914403.6953864451,1020643.6162538747,1130574.5581871723,1244313.410772617,1361980.5796003924,1483700.0879140794,1609599.681093378,1739810.9340513847,1874469.3616226898,2013714.5320259463,2157690.183477235,2306544.34404576,2460429.4548384678,2619502.496594063,2783925.1197930416,2953863.7783625647,3129489.867081711,3310979.8627866916,3498515.469470151,3692283.767383443,3892477.366253294,4099294.5627118247,4312939.502059662,4533622.344478218,4761559.43580364,4996973.482988275,5240093.734372756,5491156.164897489,5750403.666380425,6018086.243001155,6294461.212127942,6579793.410616772,6874355.406753724,7178427.71795879,7492299.034423032,7816266.448835972,8150635.692344177,8495721.376931906,8851847.244372435,9219346.421923991,9598561.684970569,9989845.726748716,10393561.435387166,10810082.178422092,11239792.095000949,11683086.39596501,12140371.67202326,12612066.210216183,13098600.318914939,13600416.661535364,14117970.599241205,14651730.542817388,15202178.314009737,15769809.516533053,16355133.91700074,16958675.8360776,17580974.55005815,18222584.703187287,18884076.73097329,19566037.294802416,20269069.728124022,20993794.494510762,21740849.65791072,22510891.36539521,23304594.342721313,24122652.40305759,24965778.96917826,25834707.60950455,26730192.588336214,27653009.43062612,28603955.50168638,29583850.602202266,30593537.578945667,31633882.951584972],[-938744.3033590545,-897173.8230212876,-853996.2624917915,-809158.2561006448,-762604.780069707,-714279.10327374,-664122.736594189,-612075.3808254232,-558074.8730952679,-502057.13175732596,-443956.09971385915,-383703.68612688966,-321229.7064702725,-256461.8208813644,-189325.47076114034,-119743.81357934745,-47637.65583204664,27074.61589868134,104477.10583194578,184656.48187873373,267702.05058914097,353705.83421213366,442762.6499244035,534970.191291892,630429.1120178327,729243.1120457831,831519.0260758647,937366.9145666729,1046900.1572808963,1160235.5494549787,1277493.4006512072,1398797.636376494,1524275.902533759,1654059.672787271,1788284.358918325,1927089.424254598,2070618.5002498738,2219019.506305364,2372444.772919411,2531051.168245787,2695000.228167502,2864458.289964758,3039596.6296824897,3220591.603297279,3407624.7917776247,3600883.150146479,3800559.160657298,4006850.990182978,4219962.651936818,4440104.171642062,4667491.75826228,4902347.979418451,5144901.941615788,5395389.475409216,5654053.32563431,5921143.346843499,6196916.704084563,6481638.079150111,6775579.8824694855,7079022.4707611855,7392254.37061769,7715572.508179574,8049282.445039769,8393698.620568905,8749144.600810364,9115953.334118832,9494467.413742717,9885039.347491443,10288031.834714495,10703818.050754944,11132781.939090515,11575318.511352042,12031834.15543219,12502746.951882955,12988486.998847082,13489496.745703354,14006231.335699558,14539158.957754508,15088761.207724847,15655533.459339231,16239985.245052397,16842640.647121184,17464038.699104592,18104733.798104797,18765296.127998963,19446312.09397129,20148384.768613793,20872134.34990102,21618198.63135475,22387233.484702963,23179913.35535178,23996931.77101914,24839001.86383492,25706856.906286806,26601250.861354426,27522958.947184704,28472778.216697134,29451528.152494803,30460051.277473137,31499213.781523224],[-1015044.3415890755,-973751.5480753025,-930855.4447838515,-886302.7267895509,-840038.4321222943,-792005.8925476125,-742146.6829401297,-690400.5692099589,-636705.454743735,-580997.3253177712,-523210.19244220015,-463276.0350937429,-401124.7397898147,-336684.038962645,-269879.4475823152,-200634.197985142,-128869.17285499861,-54502.83630720293,22548.83697411744,102372.43460159004,185057.18140632752,270695.0164716197,359380.6723353327,451211.7564244452,546288.8347755559,644715.5181099512,746598.550321315,852047.8994485848,961176.8511919864,1074102.1050525522,1190943.873153503,1311825.9818278095,1436875.976037657,1566225.2267074296,1700009.0410462534,1838366.7759438232,1981441.9545157366,2129382.3858899474,2282340.288320965,2440472.415711851,2603940.1876511946,2772909.8230434516,2947552.4774382813,3128044.3841584204,3314566.9993202835,3507307.150856043,3706457.1916485196,3912215.1568781603,4124784.925701282,4344376.387376086,4571205.611948675,4805495.025625108,5047473.590952132,5297376.991935898,5555447.824225124,5821935.790498813,6097097.901195203,6381198.680710853,6674510.379241288,6977313.190380994,7289895.474655167,7612553.989139476,7945594.123309135,8289330.141307981,8644085.430786226,9010192.758480648,9387994.532737596,9777843.073119706,10180100.887323342,10595140.955569265,11023347.022679813,11465113.898032181,11920847.763600804,12390966.490287302,12875899.962783068,13376090.413144514,13891992.763354588,14424074.977052186,14972818.420724867,15538718.234567616,16122283.713260151,16724038.69696474,17344521.97274655,17984287.686733596,18643905.767265934,19323962.3593435,20025060.27064124,20747819.429396592,21492877.354485214,22260889.63798952,23052530.44057836,23868493.000046887,24709490.153320964,25576254.87230578,26469540.813920707,27390122.88467378,28338797.82016397,29316384.77988762,30323725.957740434,31361687.20861213],[-1093286.5449462924,-1052276.2188710622,-1009666.4100141749,-965403.8750598882,-919433.7147261482,-871699.3245648141,-822142.3443551958,-770702.6060508466,-717318.0802413933,-661924.8210868835,-604456.9096835204,-544846.3958183848,-483023.2380659585,-418915.24218503153,-352447.9977649455,-283544.81307770684,-212126.6480833385,-138112.04553831415,-61417.060157995205,18044.81422355259,100362.71954926988,185628.51054644585,273936.8339021653,365385.2096704198,460074.11496473337,558107.0700047514,659590.7265750165,764634.9589682678,873352.9574714829,985861.3244747291,1102280.1732614022,1222733.2295639822,1347347.9359512264,1476255.559128088,1609591.3002247876,1747494.4081584252,1890108.296143474,2037580.6614427827,2190063.608445663,2347713.775153042,2510692.463176992,2679165.771332917,2853304.732929901,3033285.4568591192,3219289.2725739805,3411502.879071152,3610118.497983643,3815334.0308850603,4027353.220924489,4246385.8189082835,4472647.753941039,4706361.308751692,4947755.299827609,5197065.262485667,5454533.641006921,5720409.983974942,5994951.14495447,6278421.488639295,6571093.102640633,6873246.01503411,7185168.417837281,7507156.896574235,7839516.666068619,8182561.812655405,8536615.542960417,8902010.439421263,9279088.72274974,9668202.521477185,10069714.148809033,10483996.3869518,10911432.779125098,11352417.92944875,11807357.810917655,12276670.081663042,12760784.409745086,13260142.8066568,13775199.969813073,14306423.634205997,14854294.93352241,15419308.770925837,16001974.199755762,16602814.814445809,17222369.151863143,17861191.103385817,18519850.337967902,19198932.73650181,19899040.837746114,20620794.29612434,21364830.351710156,22131804.312703878,22922390.050718315,23737280.509223115,24577188.225451812,25442845.86615145,26335006.777516417,27254445.549660206,28201958.596013024,29178364.748021528,30184505.865542352,31221247.46332618],[-1173521.9962002793,-1132798.9840719756,-1090480.3730388358,-1046512.9822562644,-1000841.9760070792,-953410.8145216936,-904161.2033920274,-853033.041539005,-799964.3676954806,-744891.3053621156,-687748.0061949743,-628466.5917825704,-566977.0937651065,-503207.3922545472,-437083.15250440175,-368527.75978585053,-297462.25241757743,-223805.2528991145,-147472.89709863625,-68378.76143864589,13566.211973432917,98453.79232575186,186378.5392501764,277437.8842015704,371732.21412210166,469364.95745891705,570442.6725935321,675075.1387552032,783375.4494764134,895460.1086706882,1011449.129391091,1131466.135353879,1255638.4652927201,1384097.2802252006,1516977.673707853,1654418.7851629946,1796563.9163538963,1943560.6510997918,2095560.9783172468,2252721.4184679445,2415203.1535201864,2583172.1605022587,2756799.3487533815,2936260.7009717682,3121737.418153909,3313416.0685338415,3511488.7406336553,3716153.2005245453,3927613.053417545,4146077.9097003806,4371763.555532732,4604892.1281256825,4845692.295828378,5094399.443150695,5351255.8608487435,5616510.941212955,5890421.378695694,6173251.37600706,6465272.855850292,6766765.6784147015,7078017.86479823,7399325.826516042,7730994.60123646,8073338.094934645,8426679.330612928,8791350.703761363,9167694.244758917,9556061.888356037,9956815.75046568,10370328.412425295,10796983.212942792,11237174.54791635,11691308.17834065,12159801.54649837,12643084.100681627,13141597.62862344,13655796.599912927,14186148.517575562,14733134.279114056,15297248.547212437,15879000.13035588,16478912.37366796,17097523.560167726,17735387.322763167,18393073.06723081,19071166.40649105,19770269.606447384,20491002.04369483,21234000.67541325,21999920.521750078,22789435.16101095,23603237.238006778,24442038.985861886,25306572.761662938,26197591.59629004,27115869.758784052,28062203.3356377,29037410.825386964,30042333.748894118,31077837.275719292],[-1255803.1109809554,-1215372.3267104337,-1173349.884601612,-1129682.6671398003,-1084315.9030463914,-1037193.1181193176,-988256.0846675308,-937444.7694995776,-884697.2804279067,-829949.8112465437,-773136.5851409151,-714189.7964875093,-653039.5509961736,-589613.8041536254,-523838.29791718395,-455636.4956152565,-384929.5150019531,-311636.0594157092,-235672.3469928368,-156952.03787942324,-75386.15938862925,9116.970952570438,96651.82452387456,187315.74296264676,281209.0217995066,378434.9964466756,479100.13059757743,583314.107109787,691189.9214295419,802843.9776381077,918396.1871780483,1037970.0703441892,1161692.8606045851,1289695.6118329912,1422113.3085293174,1559084.9791113017,1700753.8123537377,1847267.2770669898,1998777.245101058,2155440.1177555183,2317416.9557024017,2484873.6125000957,2657980.871804403,2836914.5883755954,3021855.832976238,3212991.0412680367,3410512.1668192623,3614616.838321765,3825508.5211370224,4043396.683287358,4268496.966004774,4501031.35896307,4741228.380316338,4989323.261672503,5245558.138128906,5510182.2435093755,5783452.110940006,6065631.778892143,6356993.002863913,6657815.472818364,6968387.036550233,7289003.929137548,7619971.008619729,7961601.998092215,8314219.734366656,8678156.423370354,9053753.902484953,9441363.909965592,9841348.361667171,10254079.635240518,10679940.862011116,11119326.226730637,11572641.275413513,12040303.231457517,12522741.320292905,13020397.102740366,13533724.817351228,14063191.731911369,14609278.504404383,15172479.553636435,15753303.439775424,16352273.255106095,16969927.025203437,17606818.12084085,18263515.680883057,18940605.046472877,19638688.206780527,20358384.256620206,21100329.866250113,21865179.76365988,22653607.22966443,23466304.606152274,24303983.817793146,25167376.907584473,26057236.586578295,26974336.798142303,27919473.297142766,28893464.244425785,29897150.816988066,30931397.834234297],[-1340183.6723751468,-1300050.0988223203,-1258328.8660067436,-1214966.9205995742,-1169909.5566314664,-1123100.3663535505,-1074481.189692879,-1023992.0622622701,-971571.1618863014,-917154.7536011278,-860677.1330868085,-802070.5684898854,-741265.240589058,-678189.1812625178,-612768.2102058879,-544925.8698574016,-474583.35847769724,-401659.46133405203,-326070.4799400354,-247730.159293985,-166549.61306340434,-82437.2466592649,4701.321856349241,94963.34309263993,188449.01993647777,285261.5935111139,385507.43154547643,489296.1192263132,596740.5525913406,707957.0345435375,823065.3735449668,942188.9850744959,1065454.9959149053,1192994.3513510572,1324941.9253551727,1461436.6338428333,1602621.5510758003,1748644.0293034734,1899655.821729172,2055813.2088816706,2217277.1284987964,2384213.3090015687,2556792.406664421,2735190.146581034,2919587.4675198346,3110170.6707780017,3307131.5731449574,3510667.664074976,3720982.267187724,3938284.706213247,4162790.4754936183,4394721.415167209,4634305.891158137,4881778.980100127,5137382.6593212765,5401366.002029496,5673985.3778356835,5955504.658743066,6246195.43077409,6546337.211353005,6856217.672615925,7176132.870804717,7506387.48188634,7847295.043587565,8199178.203994209,8562368.976888407,8937209.004024005,9324049.82448132,9723253.151327714,10135191.155746832,10560246.758849313,10998813.931354836,11451298.001358064,11918115.970377248,12399696.837929986,12896481.934816565,13408925.265384022,13937493.858952608,14482668.130700018,15044942.252205824,15624824.531908844,16222837.80577875,16839519.83840456,17475423.734816276,18131118.36328971,18807188.78944342,19504236.721896637,20222880.969792668,20963757.912504174,21727521.981823854,22514846.15696004,23326422.472684644,24162962.54093884,25025198.086275507,25913881.495480232,26829786.381724548,27773708.16363902,28746464.659682512,29748896.698199183,30781868.74355964],[-1426718.8664202401,-1386887.5569792376,-1345472.6446901988,-1302421.1412632596,-1257678.4069060716,-1211188.1012055646,-1162892.1326046116,-1112730.6064334153,-1060641.7714575375,-1006561.9649000783,-950425.5558968568,-892164.8873421862,-831710.2160782809,-768989.6513866424,-703929.0917306212,-636452.1597055974,-566480.1351442565,-493931.8863268178,-418723.79924709443,-340769.7048779642,-259980.8043831885,-176265.59221967123,-89529.77707355842,323.79943305440247,93395.24631909002,189787.70917300694,289607.45849143807,392963.98050012905,499970.070515221,610741.9289252087,725399.2598516606,844065.3725733384,966867.2857790897,1093935.8347310442,1225405.7814144203,1361415.9277572064,1502109.2319962075,1647632.9282808434,1798138.6496012015,1953782.5541205676,2114725.455019299,2281132.9539285135,2453175.578059111,2631028.9211255843,2814873.788158805,3004896.344316411,3201288.2678020326,3404246.906992562,3613975.4418927524,3830683.0500332527,4054585.0769245317,4285903.211192436,4524865.664518067,4771707.356511219,5026670.104643582,5290002.819381944,5561961.704657834,5842810.463802643,6132820.511119031,6432271.189207194,6741449.992217348,7060652.795185255,7390184.089591758,7730357.225336952,8081494.65927742,8443928.210500568,8817999.322535843,9204059.332644057,9602469.748411443,10013602.531811005,10437840.390944157,10875577.0796524,11327217.70521161,11793179.044307496,12273889.86753717,12769791.27261655,13281337.026567359,13808993.91706501,14353242.113242643,14914575.536154147,15493502.239148395,16090544.798456235,16706240.714192841,17341142.82209155,17995819.7162193,18670856.18298243,19366853.64669196,20084430.626992434,20824223.20847091,21586885.522749826,22373090.243382495,23183529.093899585,24018913.36931111,24879974.471443426,25767464.45845274,26682156.608868923,27624846.000557154,28596350.10497368,29597509.39710703,30629187.981501594],[-1515465.3378014248,-1475941.4218684947,-1434838.017854127,-1392102.203364335,-1347679.4056552658,-1301513.3525360716,-1253546.0218669977,-1203717.5896126782,-1151966.3764123968,-1098228.792625037,-1042439.2818073053,-984530.262583103,-924432.0688568987,-862072.8883295436,-797378.6992657664,-730273.205469647,-660677.7694157944,-588511.3434858276,-513690.3992612758,-436128.85481631523,-355737.9999574525,-272426.41935404716,-186099.91350333765,-96661.41746653989,-4010.917321770452,91954.63573390059,191341.41369204968,294258.80252217036,400819.4955006959,511139.58916011266,625338.681916249,743539.9754582662,865870.378966839,992460.6162419859,1123445.3358168332,1258963.2241406348,1399157.1219073487,1544174.1436214373,1694165.8004870405,1849288.126701055,2009701.8092566878,2175572.3213361576,2347070.0593979573,2524370.484058099,2707654.2648595152,2897107.4290383346,3092921.5143981036,3295293.7263912354,3504427.099526895,3720530.6632215763,3943819.6122044865,4174515.4816038413,4412846.326836456,4659046.908429944,4913358.88190385,5176030.9928496955,5447319.277346452,5727487.267840627,6016806.204661442,6315555.253289999,6624021.727553388,6942501.3189009065,7271298.331903207,7610725.926164733,7961106.364798387,8322771.269635889,8696061.883373853,9081329.338796947,9478934.935304262,9889250.422901984,10312658.293874886,10749552.082326647,11200336.671801433,11665428.611185309,12145256.4391323,12640261.01719508,13150895.871933527,13677627.546183001,14220935.959777128,14781314.779928101,15359271.801516606,15955329.337593056,16570024.62029249,17203910.21247927,17857554.430372003,18531541.77745673,19226473.389958285,19942967.494173072,20681659.87598066,21443204.36283701,22228273.31856873,23037558.15131641,23871769.83493144,24731639.444206014,25617918.704277594,26531380.55456169,27472819.72760035,28443053.343202658,29442921.51826875,30473287.9926937],[-1606481.2194900475,-1567269.9064347195,-1526483.2790436628,-1484068.4816833446,-1439971.0095377755,-1394134.6595327468,-1346501.479855639,-1297011.718030938,-1245603.767513014,-1192214.1127540944,-1136777.272705901,-1079225.7427129005,-1019489.9347499306,-957498.1159628946,-893176.3454613369,-826448.4093197384,-757235.75373476,-685457.4162884848,-611029.9552684026,-533867.3769878796,-453881.0610539834,-370979.6835267474,-285069.13791344315,-196052.45393444598,-103829.7140065534,-8297.96737593133,90648.85815798957,193120.04699962866,299228.1890537478,409089.2756410865,522822.79809182975,640551.8491010852,762403.2269117329,888507.5424065283,1018999.3291852577,1154017.1567105819,1293703.7465987122,1438206.0921465848,1587675.5811817078,1742268.1223151507,1902144.274704313,2067469.3814040218,2238413.7064114446,2415152.5755041772,2597866.52096569,2786741.4303068686,2981968.699094518,3183745.3879864747,3392274.3840921195,3607764.566774642,3830430.978007445,4060494.9974103114,4298184.522088085,4543734.151400908,4797385.376792503,5059386.776816266,5329994.217495953,5609471.058149804,5898088.362848772,6196125.117627643,6503868.453620149,6821613.876274872,7149665.50079312,7488336.293978918,7837948.32264998,8198833.008783289,8571331.391595174,8955794.396697097,9352583.112553623,9762069.07440526,10184634.555868827,10620672.86840539,11070588.668868026,11534798.275328167,12013729.991425019,12507824.439418338,13017534.902217569,13543327.674569154,14085682.42369698,14645092.559598679,15222065.615250057,15817123.63701922,16430803.58549277,17063657.7470301,17716254.15629625,18389177.030081358,19083027.212676436,19798422.63310896,20535998.774555236,21296409.15623276,22080325.828091636,22888439.87865311,23721461.95629979,24580122.804396957,25465173.810586307,26377387.570605986,27317558.467024237,28286503.263263073,29285061.713302888,30314097.187465157],[-1699825.8568685036,-1660932.4454820598,-1620467.9532283703,-1578379.5921064662,-1534612.926115211,-1489111.8221991472,-1441818.3997903755,-1392672.9789064922,-1341614.0267662448,-1288578.1028808095,-1233499.8025791696,-1176311.6989255175,-1116944.2829815536,-1055325.9023721889,-991382.698103752,-925038.5395913403,-856214.9578424762,-784831.0767474482,-710803.5424268371,-634046.4505799473,-554471.2717812671,-471986.7746688123,-386498.9469679473,-297910.91428763606,-206122.85663433,-111031.92257634923,-12532.140999687836,89485.66961661354,195133.99446377065,304528.7182852449,417789.2239357531,535038.4937016955,656403.2134484714,782013.8796760626,912004.9095593365,1046514.7540560309,1185686.0141590955,1329665.5603847029,1478604.6555822063,1632659.0811465606,1791989.2667396525,1956760.4235992925,2127142.6815411616,2303311.2297532,2485446.461476446,2673734.1226811763,2868365.464849227,3069537.4019619552,3277452.6718127467,3492320.0017604353,3714354.2790359156,3943776.7257275013,4180815.0785680283,4425703.773652381,4678684.136212158,4940004.575587155,5209920.785530352,5488695.949975388,5776600.95443705,6073914.603163456,6380923.842211404,6697923.988601072,7025218.96569179,7363121.5449686255,7711953.594388852,8072046.3334618155,8443740.595261924,8827387.095516337,9223346.70899339,9631990.753354637,10053701.28068313,10488871.376877826,10937905.46912658,11401219.64165628,11879241.96000456,12372412.803993471,12881185.2096781,13406025.220451748,13947412.247602781,14505839.440525744,15081814.066839103,15675857.902710905,16288507.63359496,16920315.265693493,17571848.548396554,18243691.408006277,18936444.39301584,19650725.131246157,20387168.799157873,21146428.60364107,21929176.27660252,22736102.582697794,23567917.840513274,24425352.457577135,25309157.47954078,26220105.15388444,27158989.50853426,28126626.94576715,29123856.851794608,30151542.22242213],[-1795559.7742427816,-1756989.675017789,-1716852.7894511055,-1675096.3980699005,-1631666.1345988126,-1586505.9369277698,-1539557.9966761055,-1490762.707313335,-1440058.6107979654,-1387382.3426923822,-1332668.5757122724,-1275849.9616685212,-1216857.0717545073,-1155618.335137227,-1092059.975801393,-1026105.9476031242,-957677.8674806226,-886694.9467717675,-813073.9205896147,-736728.9751992039,-657571.6733429991,-575510.8774587451,-490452.67073332146,-402300.2759295469,-310953.9719313588,-216311.0079398835,-118265.51526163239,-16708.416616817936,88472.6670903312,197393.51261933986,310173.3929177001,426935.1784056984,547805.4410876632,672914.561570609,802396.8390670568,936390.60446472,1075038.3365399707,1218486.7814060347,1366887.0752826883,1520394.8706673048,1679170.4660143554,1843378.9390016142,2013190.2834885344,2188779.5502662947,2370326.9916934315,2558018.2103259796,2752044.3116528513,2952602.061036003,3159894.044974326,3374128.836807443,3595521.16697177,3824292.097934515,4060669.20392825,4304886.755615089,4557185.909806941,4817814.904381572,5087029.258531134,5365091.978472113,5652273.768787188,5948853.249517824,6255117.179178728,6571360.683850624,6897887.492492943,7235010.178666101,7583050.408812487,7942339.197269629,8313217.168215204,8696034.824685559,9091152.824893503,9498942.266008593,9919784.975612182,10354073.811017334,10802212.966665795,11264618.289800769,11741717.60465989,12233951.045368506,12741771.397806603,13265644.4506306,13806049.355745368,14363478.998428863,14938440.37736182,15531454.994863696,16143059.257537464,16773804.887639092,17424259.34542228,18095006.26276597,18786645.888354905,19499795.54471621,20235090.09742911,20993182.436810747,21774743.97239739,22580465.140568625,23411055.92561946,24267246.394659273,25149787.246679157,26059450.376141347,26997029.451477896,27963340.50887493,28959222.561733723,29985538.226204894],[-1893746.4458607757,-1855505.1086418172,-1815701.339923068,-1774282.4896219028,-1731194.2620651992,-1686380.6669792477,-1639783.9690765976,-1591344.636199776,-1541001.2859835876,-1488690.6309938421,-1434347.4223012174,-1377904.3914478892,-1319292.1907600993,-1258439.331965012,-1195272.1230610441,-1129714.6033982849,-1061688.4769163146,-991113.0434895959,-917905.1283311453,-841979.0093981549,-763246.3427467174,-681616.0857795491,-596994.4183302103,-509284.6615209654,-418387.194339497,-324199.36786709353,-226615.41709965374,-125526.37028937787,-20819.955749004148,87619.49396119174,199911.14052252006,316177.7411065521,436545.752449885,561145.4378354829,690110.9770573685,823580.579451764,961696.6000709971,1104605.6590915872,1252458.7645429214,1405411.4384367093,1563623.8464039396,1727260.9309177967,1896492.548208015,2071493.6089659361,2252444.2229344323,2439529.8474914106,2632941.4403377185,2832875.6163889347,3039534.8089899616,3253127.4355686335,3473868.067840647,3701977.6066913763,3937683.461857468,4181219.736536866,4432827.417054036,4692754.567719858,4961256.53102299,5238596.13328158,5525043.895925841,5820878.252530115,6126385.771765785,6441861.386431354,6767608.628701181,7103939.871782689,7451176.578131041,7809649.554394716,8179699.213291704,8561675.842557896,8955939.881193519,9362862.203170557,9782824.40881379,10216219.124045072,10663450.307703461,11124933.56713966,11601096.482329033,12092378.9386837,12599233.468836566,13122125.603578703,13661534.232245412,14217951.972753309,14791885.551540816,15383856.193713117,15994400.023594486,16624068.47600343,17273428.718501285,17943064.08492197,18633574.52045278,19345577.038569413,20079706.19014225,20836614.54501681,21616973.18638751,22421472.21831266,23250821.286675014,24105750.113967367,24987009.048244167,25895369.626593336,26831625.15351493,27796591.29458322,28791106.685783222,29816033.558918297],[-1994450.2707172246,-1956543.225436369,-1917078.1641820567,-1876002.507120524,-1833262.0300536458,-1788800.8154313108,-1742561.2019610968,-1694483.73277531,-1644507.102116989,-1592568.1005028402,-1538601.5583216911,-1482540.28782622,-1424315.0234710816,-1363854.3605558076,-1301084.6921216417,-1235930.144058988,-1168312.5083727594,-1098151.1745557785,-1025363.0590210683,-949862.5325365616,-871561.3456094088,-790368.5517640044,-706190.4286568994,-618930.396966055,-528488.936999429,-434763.50295578316,-337648.4347787313,-237034.8675322719,-132810.6382393865,-24860.19010403473,86935.52594213001,202699.15245227888,322557.02950194106,446639.30163267814,575080.0277783582,708017.2942570793,845593.3309051218,987954.6304445192,1135252.071170207,1287641.0430373,1445281.5772551298,1608338.479466266,1776981.4666163502,1951385.3076136727,2131729.9678728087,2318200.7578509543,2510988.48568773,2710289.6140481,2916306.421287073,3129247.167052445,3349326.2624380374,3576764.4448128045,3811788.957448612,4054633.7340756534,4305539.588491822,4564754.409365936,4832533.360371088,5109139.08577757,5394841.921675265,5689920.112944659,5994660.03614736,6309356.428492686,6634312.623021804,6969840.790198951,7316262.186059033,7673907.407084868,8043116.652013637,8424239.990714367,8817637.640362054,9223680.249071464,9642749.187203057,10075236.846530959,10521546.947485158,10982094.854666637,11457307.900879543,11947625.719861105,12453500.587981587,12975397.775096495,13513795.904845374,14069187.324600449,14642078.485316638,15232990.331584614,15842458.702089403,16471034.740789935,17119285.319070607,17787793.46917174,18477158.829169545,19187998.099808343,19920945.513502054,20676653.315808,21455792.259692036,22259052.11293263,23087142.178968858,23940791.831570923,24820751.06367493,25727791.050735243,26662704.728981882,27626307.388958797,28619437.28473445,29642956.259180933],[-2097737.049444271,-2060169.923134009,-2021049.2578690932,-1980322.5449316674,-1937935.6324798283,-1893832.6765828049,-1847956.0908537111,-1800246.4946401007,-1750642.6597340927,-1699081.4555596486,-1645497.7927959599,-1589824.565394593,-1531992.5909434156,-1471930.5493358686,-1409564.9196945908,-1344819.9155062456,-1277617.4179146457,-1207876.9071226255,-1135515.3918531137,-1060447.336813318,-982584.5881090821,-901836.296553371,-818108.8388123773,-731305.736326444,-641327.5719509637,-548071.9042501356,-451433.17938446347,-351302.6405204879,-247568.23470416572,-140114.51711833756,-28822.552665524185,86430.18520779721,205769.91851121373,329326.67161590885,457234.38114206214,589631.008911035,726658.6580390949,868463.6922637401,1015196.8585889917,1167013.4133300427,1324073.2516636625,1486541.040763083,1654586.356622612,1828383.8246712154,2008113.2642693147,2193959.8371974565,2386114.2002474675,2584772.6620158115,2790137.344017865,3002416.3462392734,3221823.9172367947,3448580.6289140144,3682913.5560948625,3925056.461023626,4175249.982917874,4433741.8327142205,4700786.993143179,4976647.92426241,5261594.77461836,5555905.598155372,5859866.577043116,6173772.250578961,6497925.750306694,6832639.041541037,7178233.171447258,7535038.52384933,7903395.080965843,8283652.692215624,8676171.350318737,9081321.4748558,9499484.203497885,9931051.691097207,10376427.416850474,10836026.499733798,11310276.022453014,11799615.364090359,12304496.541719453,12825384.561171168,13362757.777244281,13917108.263564318,14488942.192342341,15078780.224334663,15687157.909206722,16314626.096616017,16961751.358265128,17629116.421231944,18317320.612847432,19026980.317423828,19758729.44515004,20513219.91345752,21291122.141175263,22093125.555821955,22919939.114339694,23772291.837648176,24650933.359361045,25556634.489017535,26490187.790216938,27452408.174031533,28444133.50808974,29466225.24172511],[-2203673.254641033,-2166451.8307851045,-2127681.408925299,-2087309.5523793558,-2045282.1825968823,-2001543.5302174464,-1956036.0847267932,-1908700.5426712595,-1859475.7543921839,-1808298.6692379792,-1755104.2792127188,-1699825.5610189876,-1642393.4164480115,-1582736.6110755475,-1520781.7112127556,-1456453.0190686383,-1389672.5060714222,-1320359.7442991016,-1248431.835969829,-1173803.3409358975,-1096386.2021284774,-1016089.6688970537,-932820.2181870425,-846481.4734928487,-756974.1215314753,-664195.8265696336,-568041.1423451928,-468401.42151150666,-365164.722546041,-258215.71404377185,-147435.57633652538,-32701.900354560465,86111.41633548029,209134.27639747225,336500.4925737083,468347.9005970126,604818.4752435517,746058.4496174455,892218.4377536466,1043453.5606193105,1199923.5756201819,1361793.009690526,1529231.2960720593,1702412.9148809295,1881517.5375570348,2066730.1753043616,2258241.331632818,2456247.159101477,2660949.6203816663,2872556.653756235,3091282.343167333,3317347.0929380897,3550977.8072910123,3792408.074791844,4041878.3578454377,4299636.187383151,4565936.362878427,4841041.157819624,5125220.530809927,5418752.342413885,5721922.57792087,6035025.576182275,6358364.2646639105,6692250.40090299,7037004.820518874,7392957.691951005,7760448.778123431,8139827.705177635,8531454.238499315,8935698.566202171,9352941.590280842,9783575.225623049,10228002.707093287,10686638.90488617,11159910.648393854,11648257.058768319,12152129.890450176,12671993.881847018,13208327.11545462,13761621.387624925,14332382.588231996,14921131.090537475,15528402.151458113,16154746.322550781,16800729.871965624,17466935.21767462,18153961.372245733,18862424.399465494,19592957.88312699,20346213.408286072,21122861.05530487,21923589.90703017,22749108.56941139,23600145.705936838,24477450.586229738,25381793.649157576,26313967.08084175,27274785.40794363,28265086.106617957,29285730.22753033],[-2312329.8592853057,-2275459.93023834,-2237045.6063128393,-2197034.52371961,-2155372.6780709364,-2112004.3754618824,-2066872.1821501902,-2019916.872794793,-1971077.3772147633,-1920290.7256264635,-1867491.9923175941,-1812614.237715982,-1755588.448806237,-1696343.4778525173,-1634805.9793769345,-1570900.3453498902,-1504548.6385400114,-1435670.5239736848,-1364183.198455071,-1290001.3180901646,-1213036.9237622647,-1133199.3645026633,-1050395.218700109,-964528.2130863843,-875499.1394429076,-783205.7689615609,-687542.7642004266,-588401.5885631545,-485670.41324311495,-379234.02155316994,-268973.71058188286,-154767.19009287003,-36488.478601031005,85992.20345520508,212808.54352197517,344098.2498015361,480003.1672636941,620669.3968883995,766247.4182256283,916892.2153530773,1072763.4063380454,1234025.376282012,1400847.414053306,1573403.8528070068,1751874.214386371,1936443.3577142144,2127301.6312849578,2324645.0298572676,2528675.355465333,2739600.382865549,2957634.029530715,3182996.530316945,3415914.616926505,3656621.7022950016,3905358.070029307,4162371.069036263,4427915.3134782305,4702252.888184795,4985653.559690534,5278394.993018036,5580762.974376831,5893051.639934784,6215563.710803435,6548610.734426617,6892513.332521714,7247601.455746689,7614214.645292284,7992702.301541401,8383423.960020868,8786749.574808821,9203059.809609935,9632746.33668826,10076212.143870091,10533871.849815216,11006152.027800485,11493491.53819653,11996341.869909946,12515167.49097273,13050446.208573373,13602669.538732806,14172343.085876774,14759986.932605723,15366136.03986522,15991340.657831842,16636166.747765455,17301196.41513496,17987028.3542877,18694278.30496543,19423579.52098357,20175583.251376674,20950959.234329198,21750396.204238974,22574602.412218094,23424306.1604099,24300256.350463666,25203223.04652001,26133998.053094618,27093395.50823575,28082252.49234699,29101429.65307124],[-2423777.9282070654,-2387265.3913139054,-2349213.1257498735,-2309568.8414866882,-2268278.609180977,-2225286.811279075,-2180536.0917208306,-2133967.3042025114,-2085519.458960548,-2035129.668033952,-1982733.0889641414,-1928262.8668899895,-1871650.0749911605,-1812823.653238195,-1751710.3453985583,-1688234.6342553217,-1622318.674985846,-1553882.2266506972,-1482842.581743475,-1409114.493745408,-1332610.1026317636,-1253238.8582741818,-1170907.44168232,-1085519.6840222161,-996976.483356325,-905175.7190383505,-810012.1637037145,-711377.3927841745,-609159.6914880769,-503243.95916666184,-393511.6110077547,-279840.476973095,-162104.69791336823,-40174.61877996754,86083.32114289608,216806.70106870588,352137.23467836715,492220.88932017516,637208.0085263746,787253.4379267478,942516.6546654049,1103161.9003995312,1269358.3179854508,1441280.091950884,1619106.5928478474,1803022.525594755,1993218.0819181548,2189889.096993979,2393237.2104067486,2603470.03154302,2820801.3095313106,3045451.107853922,3277645.983753456,3517619.172562616,3765610.7770839315,4021867.9621588113,4286645.154562588,4560204.248354434,4842814.81585229,5134754.324351736,5436308.358759721,5747770.85029937,6069444.31142762,6401640.077154825,6744678.552915625,7098889.469164369,7464612.142894389,7842195.746222876,8231999.582267029,8634393.368474212,9049757.527618522,9478483.486653611,9920973.983633867,10377643.38290256,10848917.998790743,11335236.428007688,11837049.890995072,12354822.582426991,12889032.03114978,13440169.46976497,14008740.214107059,14595264.052916843,15200275.647913666,15824324.94458113,16467977.59391741,17131815.385457125,17816436.69183486,18522456.925193325,19250509.005752742,20001243.84284445,20775330.828727625,21573458.345536634,22396334.28566359,23244686.58595481,24119263.77606275,25020835.54130625,25950193.300426953,26908150.79861689,27895544.716208972,28913235.293426387],[-2538088.9498258233,-2501939.8880093037,-2464255.8299466614,-2424984.560309268,-2384072.2257524747,-2341463.286044179,-2297100.4637928028,-2250924.6927348697,-2202875.064543887,-2152888.774118357,-2100901.063307665,-2046845.1630337061,-1990652.233761237,-1932251.30427561,-1871569.2087168396,-1808530.5218269406,-1743057.4923577192,-1675069.9745894158,-1604485.357910823,-1531218.4944046005,-1455181.6243852107,-1376284.299833138,-1294433.3056691838,-1209532.5788060613,-1121483.124922134,-1030182.9328908529,-935526.8868062841,-837406.6755334688,-735710.6997251017,-630323.9762250613,-521128.03979982994,-408000.84211437777,-290816.64788649324,-169445.9281384684,-43755.250470018014,86392.83373072371,221139.90521468222,360631.7960286811,505018.7119881222,654455.358545864,809101.0701633617,969119.9432631684,1134680.97286757,1305958.1930226032,1483130.8211020837,1666383.4060995104,1855905.9810189288,2051894.219464235,2254549.5965455873,2464079.5542188473,2680697.6711707264,2904623.8373746127,3136084.4334401526,3375312.514884996,3622548.001455403,3878037.8716350356,4142036.362478587,4414805.174899253,4696613.684579901,4987739.158627021,5288466.978138328,5599090.86684029,5919913.125936987,6251244.875359926,6593406.301567586,6946726.912068332,7311545.796865674,7688211.896967879,8077084.280187342,8478532.424392551,8892936.508425081,9320687.710871158,9762188.51690023,10217853.033368845,10688107.312433777,11173389.68385505,11674151.096261011,12190855.467557693,12723980.044776099,13274015.773561072,13841467.67755308,14426855.247963805,15030712.843548823,15653590.101291917,16296052.358051967,16958681.083479606,17642074.32447341,18346847.16147883,19073632.176946267,19823079.936251245,20595859.481395595,21392658.837837074,22214185.534752022,23061167.139109824,23934351.80390005,24834508.830866292,25762429.248133175,26718926.40310243,27704836.571009457,28721019.579536006],[-2655339.9411043813,-2619560.42688934,-2582250.712642281,-2543358.658591682,-2502830.488262324,-2460610.739627924,-2416642.214863523,-2370865.928657933,-2323221.0550480266,-2273644.872732547,-2222072.708824334,-2168437.880998768,-2112671.6379914638,-2054703.0984037742,-1994459.1877652528,-1931864.5738099026,-1866841.599913333,-1799310.2166414065,-1729187.9113608808,-1656389.6358559155,-1580827.731897587,-1502411.8547104788,-1421048.8942799252,-1336642.8944368726,-1249094.9696661644,-1158303.2195704188,-1064162.6409311378,-966565.0372951822,-865398.9260282144,-760549.4427556656,-651898.2431324096,-539323.4018574851,-422699.30886803847,-301896.56263148785,-176781.86045940872,-47217.885760577396,86936.80784391984,225827.9156339625,369605.5042188149,518424.1373608187,672443.0053006159,831826.0576627227,996742.1400464326,1167365.1344012981,1343874.1032815045,1526453.4380873293,1715293.0114045832,1910588.3335415907,2112540.713382096,2321357.4236704335,2537251.8708412014,2760443.769518706,2991159.321809016,3229631.4015130745,3476099.74338773,3730811.1375935934,3994019.629466842,4265986.724743465,4546981.600406204,4837281.321272995,5137171.062497806,5446944.338140128,5766903.235944709,6097358.658520609,6438630.571068998,6791048.255832732,7154950.573467042,7530686.231473079,7918614.059919864,8319103.294617273,8732533.867952764,9159296.707580883,9599794.043178517,10054439.721463742,10523659.529722271,11007891.528022375,11507586.390389916,12023207.755126247,12555232.584562201,13104151.534451827,13670469.33325744,14254705.171626568,14857393.102264008,15479082.450513897,16120338.235902227,16781741.60494728,17463890.275507506,18167398.992970243,18892899.99859736,19641043.510331012,20412498.216378264,21207951.781921767,22028111.3692616,22873704.17176625,23745477.961974327,24644201.65420024,25570665.882031027,26525683.5910897,27510090.647456184,28524746.462142535],[-2775606.450479674,-2740202.7355121924,-2703273.68338131,-2664767.230793052,-2624629.679076997,-2582805.6453630957,-2539238.012358774,-2493867.8766866233,-2446634.495744259,-2397475.2330442467,-2346325.5019930312,-2293118.708066362,-2237786.1893347246,-2180257.155296882,-2120458.623971154,-2058315.3572006212,-1993749.7941203094,-1926681.9827359426,-1857029.5095655965,-1784707.4272875492,-1709628.1803419208,-1631701.5284299427,-1550834.4678543163,-1466931.1506381836,-1379892.801367687,-1289617.631691265,-1196000.7524164012,-1098934.083132715,-998306.2593025565,-894002.53673987,-785904.6934184749,-673890.9285261016,-557835.7586984504,-437609.9113520691,-313080.2150399666,-184109.4867468942,-50556.416047739796,87724.55396190751,230883.34904925246,379074.38962237164,532456.7200090196,691194.1413146919,855455.3479653606,1025414.0680337269,1201249.2074435037,1383144.9981599106,1571291.1504770014,1765883.00950169,1967121.7159527428,2175214.3713909183,2390374.207992601,2612820.7629922144,2842780.0579160815,3080484.782736337,3326174.485071511,3580095.764573086,3842502.4726343974,4113655.917551282,4393825.075303819,4683286.806078651,4982326.076702275,5291236.189141763,5610319.015214428,5939885.237695614,6280254.597973594,6631756.150425255,6994728.52371121,7369520.189132426,7756489.736273963,8156006.156098023,8568449.131699448,8994209.336912341,9433688.742980981,9887300.933492638,10355471.427816473,10838638.013229158,11337251.085999092,11851774.001611605,12382683.434428599,12930469.746986136,13495637.369181648,14078705.187650867,14680206.945538582,15300691.652976817,15940724.008522015,16600884.831857955,17281771.508034132,17983998.443542924,18708197.53455154,19455018.64759201,20225130.113027915,21019219.23164484,21837992.794669747,22682177.61759734,23552521.088164613,24449791.72882721,25374779.774124265,26328297.763307445,27311181.14862511,28324288.919658024],[-2898967.827727541,-2863946.246848171,-2827404.2584469398,-2789289.876615593,-2749549.4814110417,-2708127.7700566645,-2664967.7067443514,-2620010.4709976506,-2573195.4045575205,-2524459.9567486704,-2473739.628285335,-2420967.913474158,-2366076.2407674384,-2308993.9116252074,-2249648.0376353343,-2187963.4758483833,-2123862.7622747566,-2057266.0434941077,-1988091.0063281893,-1916252.8055204633,-1841663.9893701584,-1764234.4232645049,-1683871.2110526785,-1600478.6141990768,-1513957.968660703,-1424207.5994219594,-1331122.732627728,-1234595.4052432198,-1134514.3721821886,-1030765.010824115,-923229.2228614166,-811785.3333932767,-696307.9872001857,-576668.0421179701,-452732.45943557005,-324364.1912329681,-191422.06458333693,-53760.66252794303,88769.79826205224,236323.59321759734,389059.61910346523,547141.5268285424,710737.8579398161,880022.1848988589,1055173.2552355519,1236375.1396869682,1423817.3844322339,1617695.1675229492,1818209.459627686,2025567.1892065667,2239981.4122283002,2461671.4865549216,2690863.251116853,2927789.210007062,3172688.7216206016,3425808.1929789567,3687401.2793758577,3957729.0894731935,4237060.396017301,4525671.852294105,4823848.214494418,5131882.57014505,5450076.572747787,5778740.682815013,6118194.415451115,6468766.594653403,6830795.614530895,7204629.707583275,7590627.220265463,7989156.896000097,8400598.165850788,8825341.447045065,9263788.449560076,9716352.490968427,10183458.819788687,10665544.947520753,11163060.989638232,11676470.015720107,12206248.409014888,12752886.235641249,13316887.623676084,13898771.152430842,14499070.252119265,15118333.614231212,15757125.612863116,16416026.73731218,17095634.036204316,17796561.573458422,18519440.89640353,19264921.51635191,20033671.401946366,20826377.48562946,21643746.183539107,22486503.92920892,23355397.721414294,24251195.68651809,25174687.65570212,26126685.757460423,27108025.025745187,28119564.024161547],[-3025505.551655342,-2990872.5213405946,-2954724.0799467447,-2917008.3198444108,-2877671.700724813,-2836659.0008257525,-2793913.2667589714,-2749375.761898878,-2702985.9132944425,-2654681.257062273,-2604397.382219467,-2552067.872914152,-2497624.2490069633,-2440995.904961707,-2382110.046994842,-2320891.6284400085,-2257263.2832755693,-2191145.25776497,-2122455.3401609645,-2051108.788417466,-1977018.2558561815,-1900093.7147321724,-1820242.3776417477,-1737368.6167102885,-1651373.8805048233,-1562156.608604759,-1469612.1437714268,-1373632.6416452182,-1274106.9779117052,-1170920.6528575001,-1063955.6932567786,-953090.5515053049,-838200.0019357223,-719155.0342333652,-595822.7438762598,-468066.21951648407,-335744.42722610943,-198712.09151715413,-56819.573048939,90087.25705729518,242167.14640271198,399583.5940728141,562504.9870822085,731104.7406010143,905561.4420573628,1086058.9992243536,1272786.7924017627,1465939.8307925668,1665718.9131922824,1872330.7931074752,2085988.348415739,2306910.755692132,2535323.669325034,2771459.405549757,3015557.1315266555,3267863.059602703,3528630.6468933364,3798120.8003135007,4076602.0872273277,4364350.951835986,4661651.937473934,4968797.914970044,5286090.317215096,5613839.380124478,5952364.390145738,6301993.938483624,6663066.182242097,7035929.1126247905,7420940.830419753,7818469.828930415,8228895.284566082,8652607.355280288,9090007.487070382,9541508.728735873,10007536.05513938,10488526.69915111,10984930.492548509,11497210.216053383,12025841.958799917,12571315.487437183,13134134.625117358,13714817.640670212,14313897.648167312,14931923.017190035,15569457.794052612,16227082.134286534,16905392.746656924,17605003.349012867,18326545.136288956,19070667.260959834,19838037.326267667,20629341.892568424,21445286.997102864,22286598.687569425,23154023.569840975,24048329.370178264,24970305.512326963,25920763.709873952,26900538.574253645,27910488.238800444],[-3155301.3739520223,-3121063.4952542298,-3085315.271601389,-3048004.8745987737,-3009078.8445430025,-2968482.041673096,-2926157.596023241,-2882046.8558374937,-2836089.3345081923,-2788222.6559958532,-2738382.4986895686,-2686502.5376654216,-2632514.385296518,-2576347.530172623,-2517929.2742792075,-2457184.6683921544,-2394036.445635878,-2328404.9531549774,-2260208.081850395,-2189361.1941235447,-2115777.049576144,-2039365.7286093556,-1960034.55386604,-1877688.0094534429,-1792227.6578914858,-1703552.0547196642,-1611556.660703607,-1516133.7515698522,-1417172.3252102844,-1314558.006277102,-1208172.9481091704,-1097895.7319065155,-983601.2630870193,-865160.6647443203,-742441.168130625,-615306.0000818362,-483614.2673080098,-347220.8374585528,-205976.21687594894,-59726.42495729402,91687.13498196844,248427.8084212253,410663.8260295708,578568.4438415654,752320.0873080548,932102.4993305327,1118104.8923891336,1310522.1048645796,1509554.7616718719,1715409.4393220618,1928298.8355244165,2148441.94345407,2376064.2308078688,2611397.823776869,2854681.696062195,3106161.8630732903,3366091.5814450383,3634731.5540029835,3912350.1403460503,4199223.573165938,4495636.180473886,4801880.61389078,5118258.083142417,5445078.596948657,5782661.21045587,6131334.279385649,6491435.721098935,6863313.282717185,7247324.81652637,7643838.56282549,8053233.440433316,8475899.345041273,8912237.455625948,9362660.549118727,9827593.323576499,10307472.730033968,10802748.313309561,11313882.561947031,11841351.267586095,12385643.893965743,12947263.955811521,13526729.407907035,14124573.044553213,14741342.909729589,15377602.718208436,16033932.287928356,16710927.983897561,17409203.173929237,18129388.696525738,18872133.34121369,19638104.341649823,20427987.88184316,21242489.6157998,22082335.20096707,22948270.845819116,23841063.871936917,24761503.290969156,25710400.396849938,26688589.373663872,27696927.919554707],[-3288442.4274602784,-3254606.3125616484,-3219264.9864606983,-3182366.7008806523,-3143858.077620088,-3103684.059829288,-3061787.8618886443,-3018110.917849602,-2972592.828399694,-2925171.306309656,-2875782.1203214526,-2824359.037434969,-2770833.763546697,-2715135.882398665,-2657192.792787328,-2596929.6439885977,-2534269.2693469273,-2469132.1179785626,-2401436.1845396296,-2331096.9370032167,-2258027.242392346,-2182137.290413165,-2103334.514931797,-2021523.5132322162,-1936605.9630004885,-1848480.5369682377,-1757042.815156499,-1662185.1946483515,-1563796.7968322076,-1461763.372036011,-1355967.2014938537,-1246286.9965613047,-1132597.7951137992,-1014770.8550469633,-892673.5448027235,-766169.2308384301,-635117.1619621962,-499372.3504439378,-358785.44981554244,-213202.629279932,-62465.444623166695,93589.29444943462,255129.66065810248,422328.749273113,595364.8221136034,774421.4555287678,959687.6924719028,1151358.1987671563,1349633.423687094,1554719.7649574159,1766829.7383007398,1986182.1516449451,2213002.2841185406,2447522.069961642,2689980.2874789424,2940622.7531741206,3199702.521201741,3467480.0882662423,3744223.6041368674,4030209.087898299,4325720.650107227,4631050.721010897,4946500.284969591,5272379.121271543,5609006.05148978,5956709.1935538165,6315826.222735265,6686704.639689127,7069702.045776231,7465186.425828872,7873536.438572902,8295141.714894546,8730403.164165152,9179733.28882145,9643556.507445013,10122309.486522043,10616441.481154468,11126414.684905414,11652704.58907167,12195800.351587007,12756205.175807578,13334436.699479807,13931027.394093744,14546524.974936867,15181492.822098251,15836510.412730677,16512173.764839733,17209095.89290347,17927907.275638096,18669256.336212896,19433809.93523334,20222253.87683823,21035293.428217202,21873653.852925256,22738080.95833605,23629341.65758739,24548224.54640498,25495540.49518033,26472123.256693564,27478830.089877002],[-3425013.889154153,-3391586.3923799708,-3356658.890188774,-3320179.715408758,-3282095.572343314,-3242351.488072983,-3200890.762359866,-3157654.9161149794,-3112583.638390295,-3065614.7318533864,-3016684.0567033538,-2965725.472986064,-2912670.7812619205,-2857449.6615843736,-2799989.6107389443,-2740215.8776989994,-2678051.397246079,-2613416.721705093,-2546229.9507448548,-2476406.659188232,-2403859.8227788373,-2328499.7418486117,-2250233.9628295247,-2168967.1975472756,-2084601.2402417008,-1997034.8822473027,-1906163.8242747113,-1811880.586221858,-1714074.4144561877,-1612631.1864889115,-1507433.3129820349,-1398359.6370051801,-1285285.3304760838,-1168081.787703761,-1046616.5159584992,-920753.0229854286,-790350.7013853323,-655264.7097718548,-515345.85061885044,-370440.44471746404,-220390.2021370912,-65032.08961132169,95801.80575603992,262284.4165637903,434593.839059487,612913.4810669189,797432.2140054656,988344.5291011184,1185850.6979073584,1390156.9372519115,1601475.5787216937,1820025.2428111285,2046031.017856325,2279724.6438837238,2521344.7014996633,2771136.80596005,3029353.806556402,3296255.9914475903,3572111.298106633,3857195.52950136,4151792.5761801973,4456194.64441824,4770702.490566205,5095625.661790125,5431282.743351791,5778001.612602495,6136119.699889272,6505984.256515285,6887952.629979862,7282392.546660207,7689682.402147975,8110211.5594288595,8544380.655118661,8992601.913953068,9455299.471774952,9932909.707200319,10425881.58223363,10934676.992016094,11459771.12399855,12001652.826743841,12560824.988609161,13137804.926608812,13733124.785660826,14347331.94853163,14980989.456729371,15634676.442652712,16308988.573264852,17004538.50559585,17721956.354388572,18461890.172191992,19225006.442219876,20011990.584321737,20823547.474371403,21660401.97745026,22523299.49516694,23413006.52746643,24330311.249315284,25276024.10263782,26250978.403895132,27256030.967701897],[-3565107.966753583,-3532095.927208,-3497589.1571692973,-3461536.0714614093,-3423883.4577966854,-3384576.4281033995,-3343558.3684563753,-3300770.8875711057,-3256153.7638229737,-3209644.8907494796,-3161180.2209946243,-3110693.7086526835,-3058117.24996536,-3003380.6223301645,-2946411.421569744,-2887134.997418734,-2825474.3871755456,-2761350.2474697162,-2694680.784095261,-2625381.6798540475,-2553366.0203566,-2478544.2177242083,-2400823.9321359275,-2320109.991158231,-2236304.306801988,-2149305.7902402636,-2059010.2641278254,-1965310.3724509971,-1868095.4878492784,-1767251.6163297594,-1662661.2993151257,-1554203.5129421512,-1441753.5645445976,-1325182.986239775,-1204359.425542499,-1079146.5329236686,-949403.8462367691,-814986.6719216872,-675745.9628996486,-531528.1930785263,-382175.22836324945,-227524.1940918304,-67407.33879265655,98348.10583632253,269920.0688205045,447491.7877821475,631251.9609045051,821394.903086816,1018120.7064078376,1221635.4050143883,1432151.1445469428,1649886.3562274016,1875065.9357316494,2107921.426975468,2348691.2109400295,2597620.6996765137,2854962.5356256217,3120976.7963818144,3395931.2050710674,3680101.346461408,3973770.8889770694,4277231.8127719145,4590784.644003775,4914738.695498841,5249412.313954862,5595133.133856427,5952238.3383011855,6321074.926878819,6701999.990827944,7095380.995633366,7501596.071276335,7921034.310326468,8354096.07408829,8801193.30699992,9262749.859527558,9739201.81973678,10230997.853811637,10738599.555704523,11262481.806209382,11803133.141661901,12361056.132518211,12936767.772111844,13530799.875792569,14143699.490761342,14776029.316851772,15428368.138565032,16101311.268627763,16795471.00337617,17511477.09028145,18249977.20792044,19011637.458709255,19797142.874746807,20607197.937073633,21442527.10872293,22303875.381905545,23192008.839681916,24107715.2325073,25051804.570025936,26025109.728504542,27028487.074301593],[-3708814.3348356285,-3676224.8552031787,-3642145.99417969,-3606526.2496001837,-3569312.493617587,-3530449.924056293,-3489882.0143693252,-3447550.4621594017,-3403395.1362256464,-3357354.022093972,-3309363.1659898385,-3259356.617211251,-3207266.368855499,-3153022.2968576634,-3096552.097290812,-3037781.2218840546,-2976632.8117063455,-2913027.628966292,-2846883.9868786163,-2778117.6775413617,-2706641.8977709534,-2632367.172839424,-2555201.2780570835,-2475049.1581384446,-2391812.844296271,-2305391.368997205,-2215680.678319554,-2122573.541842391,-2025959.460007039,-1925724.568872149,-1821751.5422030818,-1713919.4908125736,-1602103.8590866774,-1486176.318615037,-1366004.6588494722,-1241452.6747079948,-1112380.051047667,-978642.2439154489,-840090.3584913546,-696571.0236426061,-547926.2629839266,-393993.36236447,-234604.73367676325,-69587.77488875482,101235.2737954529,278047.4777543023,461037.3598572146,650399.0565626277,846332.4783237716,1049043.4744181614,1258744.0023130774,1475652.3016921673,1699993.073265601,1931997.6624923702,2171904.2483412214,2419958.037229089,2676411.462273583,2941524.3879886065,3215564.320592478,3498806.6240473855,3791534.742001094,4094040.4257865213,4406623.968621137,4729594.446194671,5063269.963794136,5407977.910139967,5764055.218131008,6131848.6326410845,6511714.985591829,6904021.478464352,7309145.972462241,7727477.286514515,8159415.503331773,8605372.2837125,9065771.189343607,9541048.01427593,10031651.125345781,10538041.811725605,11060694.643895518,11600097.842240669,12156753.655524777,12731178.749539858,13323904.606136112,13935477.932945672,14566461.084050804,15217432.491903458,15888987.110765645,16581736.871973788,17296311.15134217,18033357.249009237,18793540.882044617,19577546.690163698,20386078.754854966,21219861.1322971,22079638.40040771,22966176.220376085,23880261.913066972,24822705.050670438,25794338.063988484,26796016.865754474],[-3856229.678676647,-3824069.8580202633,-3790426.07647455,-3755246.9158041975,-3718479.333534954,-3680068.614333374,-3639958.319990756,-3598090.237971452,-3554404.3284872635,-3508838.6700561545,-3461329.4035037523,-3411810.6743656746,-3360214.573644197,-3306471.0768771376,-3250507.981469096,-3192250.842241069,-3131622.9051464847,-3068545.039103744,-3002935.665896104,-2934710.688082902,-2863783.4148693113,-2790064.485878822,-2713461.7927718842,-2633880.3986485647,-2551222.455179884,-2465387.117401449,-2376270.456110009,-2283765.3677920783,-2187761.482025589,-2088145.0662759729,-1984798.928027263,-1877602.3141653035,-1766430.8075470151,-1651156.2206747932,-1531646.4864000976,-1407765.5455732625,-1279373.2315629506,-1146325.1515547158,-1008472.5645424575,-865662.2559319949,-717736.4086516369,-564532.470690324,-405883.01895873994,-241615.61937456112,-71552.68307738751,104488.6813348718,286696.819658082,475265.6881227307,670395.0137413237,872290.4590784218,1081163.791555548,1297233.0574160088,1520722.7604723219,1751864.045764355,1990894.8882549927,2238060.286702184,2493612.4628435653,2757811.0660232063,3030923.3834292833,3313224.5560619608,3604997.8006022424,3906534.637336964,4218135.124282666,4540108.097695949,4872771.41912007,5216452.229140809,5571487.208049886,5938222.843558498,6317015.705785286,6708232.729681866,7112251.505107677,7529460.57474369,7960259.740056986,8405060.375514297,8864285.751287667,9338371.36463361,9827765.280216206,10332928.479557628,10854335.219907675,11392473.402737172,11947844.952105494,12520966.203202467,13112368.301268267,13722597.611205226,14352216.138131782,15001801.959185967,15671949.666847179,16363270.824079907,17076394.431614332,17811967.407667227,18570655.080421597,19353141.69361096,20160130.925514545,20992346.421739355,21850532.342131346,22735453.922168262,23647898.0492208,24588673.854056887,25558613.31798047,26558571.89599991],[-4007449.160324129,-3975726.328477308,-3942525.031600105,-3907793.936398122,-3871480.0867971047,-3833528.8553492734,-3793883.8932441813,-3752487.078884655,-3709278.4649891956,-3664196.224179206,-3617176.5930096433,-3568153.81440099,-3517060.0784260808,-3463825.4614098133,-3408377.8632916687,-3350642.9432073184,-3290544.0532370917,-3228002.1702717766,-3162935.8259461317,-3095261.0345846065,-3024891.219106105,-2951737.134832105,-2875706.7911417475,-2796705.3709114613,-2714635.1476841057,-2629395.4005010044,-2540882.3263377026,-2448988.9500724636,-2353605.031928587,-2254616.9723119484,-2151907.7139842203,-2045356.6414891481,-1934839.477765549,-1820228.1778663797,-1701390.819707701,-1578191.491764891,-1450490.1776392655,-1318142.6374046337,-1181000.285647815,-1038910.0661219908,-891714.3229082432,-739250.6680053929,-581351.8452438498,-417845.5904247081,-248554.48758928291,-73295.8213114664,108118.57509761117,295882.4755726028,490195.42158217914,691262.8868390583,899296.4465436637,1114513.9512865543,1337139.70573226,1567404.6522127483,1805546.5593569893,2051810.2158959117,2306447.6297784336,2569718.2327285353,2841889.090411786,3123235.1183308307,3414039.3036204614,3714592.9328974467,4025195.826307794,4346156.577959059,4677792.802887529,5020431.3907330185,5374408.766319886,5740071.157286264,6117774.868986756,6507886.566830289,6910783.566266429,7326854.130608253,7756497.776904782,8200125.590060471,8658160.545445133,9131037.8401754,9619205.23333857,10123123.39534164,10643266.26667788,11180121.426314853,11734190.469954986,12305989.398468327,12896049.016701235,13504915.342974816,14133150.029523697,14781330.794181738,15450051.863584146,16139924.428189047,16851577.10943392,17585656.43932966,18342827.352811243,19123773.693190627,19929198.731017835,20759825.696726657,21616398.327406768,22499681.428055003,23410461.447692,24349547.07071984,25317769.823910598,26315984.69942262],[-4162573.4359196294,-4131294.9835677054,-4098543.6363790412,-4064268.14696349,-4028415.6466246466,-3990931.5967929214,-3951759.7390637505,-3910842.0438011596,-3868118.657268592,-3823527.847244925,-3777005.947084436,-3728487.2981787426,-3677904.1907740887,-3625186.8031020374,-3570263.1387737175,-3513058.962393559,-3453497.7333406145,-3391500.5376677895,-3326986.0180694554,-3259870.301861761,-3190066.926922868,-3117486.765536997,-3042037.946086243,-2963625.7725275774,-2882152.6416000295,-2797517.957695554,-2709618.045334285,-2618346.059173284,-2523591.8914899398,-2425242.077061152,-2323179.6953792516,-2217284.270121622,-2107431.66580782,-1993493.9815636883,-1875339.4419162562,-1752832.2845365237,-1625832.644853779,-1494196.4374506548,-1357775.2341529373,-1216416.1387334187,-1069961.6581245381,-918249.5700605614,-761112.7870446332,-598379.216542081,-429871.6173052285,-255407.45172194391,-74798.73407775536,112148.1253689006,305632.4806152284,505859.6145635843,713040.9082033709,927394.014450483,1149143.036767058,1378518.7126895245,1615758.6023918372,1861107.282422386,2114816.544751063,2377145.60125597,2648361.293818256,2928738.310144482,3218559.405487258,3518115.6304193735,3827706.5648037083,4147640.558146864,4478234.976486122,4819816.455982694,5172721.163419254,5537295.063744664,5913894.194890045,6302884.950018778,6704644.367423005,7119560.428255308,7548032.362307889,7990470.962037394,8447298.905078089,8918951.085425073,9405874.953557562,9908530.865686124,10427392.44241478,10962946.937023312,11515695.613619603,12086154.135462375,12674852.96365774,13282337.766543385,13909169.840010986,14555926.539073199,15223201.720945226,15911606.199942984,16621768.214514343,17354333.906705447,18109967.814380992,18889353.37654485,19693193.452065777,20522210.85218563,21377148.887151208,22258771.92732264,23167865.979144633,24105239.27635558,25071722.886825383,26068171.335417457],[-4321704.2046499625,-4290877.676095148,-4258583.898839535,-4224771.712386349,-4189388.3364211805,-4152379.3222721387,-4113688.5029754112,-4073257.941907105,-4031027.8799424423,-3986936.6811008747,-3940920.7766358643,-3892914.6075268,-3842850.5653271778,-3790658.9313265774,-3736267.8139768485,-3679603.084538311,-3620588.310894318,-3559144.689484107,-3495190.9753048117,-3428643.4099269384,-3359415.6474700198,-3287418.6784833004,-3212560.7516743736,-3134747.2934239563,-3053880.825031561,-2969860.8776254766,-2882583.904677883,-2791943.192054254,-2697828.765538167,-2600127.2957526557,-2498722.0004191026,-2393492.543870572,-2284314.9337536357,-2171061.4148378214,-2053600.3598567685,-1931796.1572983488,-1805509.0960668148,-1674595.2469268655,-1538906.3406433798,-1398289.6427359702,-1252587.82474346,-1101638.8319187192,-945275.7472494487,-783326.6517062029,-615614.4806230292,-441956.8761026971,-262166.03533669934,-76048.55473957583,116594.73021968827,315968.9096060172,522285.16813763976,735760.9589621127,956620.1822114345,1185093.3684640508,1421417.867240969,1665838.0406742226,1918605.4624842554,2179979.122395573,2450225.636159269,2729619.4613015037,3018443.1187692992,3316987.420627944,3625551.7039529923,3944444.0711044203,4273981.6365328245,4614490.780290037,4966307.408442738,5329777.220531663,5705255.984300328,6093109.817856429,6493715.4794776905,6907460.665251303,7334744.314759359,7775976.925007796,8231580.872842172,8701990.7460315,9187653.683290377,9689029.723422999,10206592.1638804,10740827.92893554,11292237.94772682,11861337.54246951,12448656.82703929,13054741.116241096,13680151.346014068,14325464.504878806,14991274.076897003,15678190.496445585,16386841.615121113,17117873.181077324,17871949.331114113,18649753.09586414,19451986.918382335,20279373.186515167,21132654.779391,22012595.628384247,22919981.292939737,23855619.55163227,24820341.008851815,25814999.717510413],[-4484945.567892211,-4454578.678730503,-4422750.265006948,-4389409.254145512,-4354502.955257077,-4317977.010626765,-4279775.345807523,-4239840.118279984,-4198111.664640794,-4154528.446276949,-4109026.993485542,-4061541.8479962335,-4012005.503850403,-3960348.3465948803,-3906498.5907402965,-3850382.2154402095,-3791922.8983389693,-3731041.9475388634,-3667658.2316367673,-3601688.1077748872,-3533045.3476526528,-3461641.0614439924,-3387383.6195633523,-3310178.572218721,-3229928.5666959453,-3146533.26230836,-3059889.242952195,-2969889.927197003,-2876425.4758522157,-2779382.6969312523,-2678644.947953661,-2574092.0355027886,-2465600.1119724773,-2353041.569422649,-2236284.9304670226,-2115194.7361108875,-1989631.430461864,-1859451.2422233261,-1724506.0628845133,-1584643.32152648,-1439705.856138629,-1289531.7813670961,-1133954.3525895867,-972801.8262187047,-805897.3161390088,-633058.6461694837,-454098.1984420791,-268822.757595595,-77033.35066768155,121474.91743163764,326913.03297640383,539498.2471366134,759454.254467925,987011.3763070535,1222406.7492001317,1465884.518501943,1717696.03728294,1978100.0706731398,2247363.0058114603,2525759.067519957,2813570.539873451,3111087.9938198216,3418610.5209931526,3736445.973907819,4064911.212682642,4404332.358468492,4755045.053776884,5117394.729852831,5491736.88131544,5878437.3482295275,6277872.605820179,6690430.062018925,7116508.363054326,7556517.707284242,8010880.167513063,8480030.021975268,8964414.094255185,9464492.102327056,9980737.017005907,10513635.43001462,11063687.93191733,11631409.500218568,12217329.89783254,12821994.082235273,13445962.625551224,14089812.145879295,14754135.750129458,15439543.488671318,16146662.82211068,16876139.10049683,17628636.0552788,18404836.304356676,19205441.870533388,20031174.713743743,20882777.277401626,21761013.04921855,22666667.13687964,23600546.85895191,24563482.351415507,25556327.190213572],[-4652406.770809585,-4622507.270228364,-4591152.045237748,-4558290.112133654,-4523868.870417058,-4487834.054308839,-4450129.682871323,-4410698.008696805,-4369479.465124758,-4326412.611946002,-4281434.079552444,-4234478.511490326,-4185478.505370759,-4134364.5520952754,-4081064.9733466404,-4025505.8573011286,-3967610.992510115,-3907301.7999015017,-3844497.262851406,-3779113.855270512,-3711065.467652336,-3640263.3310273816,-3566615.938766987,-3490028.966174593,-3410405.1878092783,-3327644.3924751626,-3241643.29581739,-3152295.4504540116,-3059491.153584552,-2963117.3519970616,-2863057.5444140127,-2759191.681094419,-2651396.060626057,-2539543.223827093,-2423501.844681031,-2303136.61822241,-2178308.145296406,-2048872.8141020797,-1914682.6784332804,-1775585.3325361358,-1631423.7824783372,-1482036.3139511207,-1327256.3563987855,-1166912.3433776787,-1000827.5690498613,-828820.0407035891,-650702.3271905053,-466281.40317961015,-275358.48911030404,-77728.88672823645,126818.18990781344,338499.78807318024,557539.3948011603,784167.1202113703,1018619.8858698383,1261141.6183185112,1511983.4479115214,1771403.9130867198,2039669.1702414434,2317053.209331749,2603838.075365722,2900314.0959457885,3206780.11500277,3523543.7329089474,3850921.5531201456,4189239.435519295,4538832.756659737,4900046.677050894,5273236.415710395,5658767.532145191,6057016.2159741055,6468369.584380297,6893225.987606244,7331995.322688796,7785099.355677122,8252972.052515341,8736059.918859335,9234822.349011855,9749731.98426683,10281275.080867317,10829951.887828175,11396277.034922231,11980779.931034401,12584005.17319672,13206512.96655526,13848879.555574749,14511697.666750995,15195576.963133143,15901144.510971678,16629045.258794472,17379942.529229432,18154518.523919944,18953474.84183783,19777533.011371206,20627435.0365277,21503943.95760654,22407844.426725052,23339943.298574876,24301070.236798123,25292078.336379237],[-4824194.984485118,-4794770.941438505,-4763897.056389549,-4731522.435612506,-4697594.570122482,-4662059.287218202,-4624860.700631779,-4585941.159246173,-4545241.194341813,-4502699.465330662,-4458252.703936504,-4411835.656779356,-4363381.026317512,-4312819.410105482,-4260079.2383176405,-4205086.709493978,-4147765.7244557696,-4088037.818341768,-4025822.090715306,-3961035.133686689,-3893590.957998179,-3823400.917015621,-3750373.6285703303,-3674414.8945892947,-3595427.61845819,-3513311.720051185,-3427964.0483678794,-3339278.291707091,-3247144.885318026,-3151450.916450762,-3052080.0267463336,-2948912.3118839553,-2841824.2184191383,-2730688.4377320744,-2615373.797010283,-2495745.1471827403,-2371663.247729078,-2242984.648273101,-2109561.566875087,-1971241.764941494,-1827868.4186476236,-1679279.986793832,-1525310.074990293,-1365787.29607235,-1200535.1266517118,-1029371.7596950568,-852109.9530212991,-668556.8736157604,-478513.9376453534,-281776.6460572798,-78134.41565023363,132629.59450764023,350738.6614357885,576422.6815019678,809918.3587128948,1051469.3981736433,1301326.7048517987,1559748.5877761375,1827000.9698377606,2103357.6033134963,2389100.291281827,2684519.115086479,2989912.667990105,3305588.2952057365,3631862.3404554054,3969060.399228936,4317517.578940729,4677578.76612732,5049598.900909653,5433943.258883024,5830987.740646133,6241119.16915887,6664735.595140375,7102246.610705752,7554073.6714835,8020650.427395919,8502423.06237176,8999850.643175103,9513405.477641484,10043573.482525794,10590854.561212532,11155762.991587633,11738827.824276166,12340593.291558472,12961619.22721624,13602481.497613851,14263772.444285175,14946101.338328188,15650094.84692225,16376397.512271665,17125672.243293043,17898600.82039278,18695884.413639754,19518244.114709962,20366421.482944313,21241179.105871867,22143301.174585357,23073594.07434306,24032886.990788102,25022032.53218032],[-5000426.236735072,-4971485.694735069,-4941101.271570008,-4909222.164470296,-4875795.956957019,-4840768.5704127755,-4804084.214259934,-4765685.334707256,-4725512.562026089,-4683504.65631464,-4639598.451709092,-4593728.798999183,-4545828.506602322,-4495828.279853931,-4443656.658564271,-4389239.952797884,-4332502.17682365,-4273364.981186014,-4211747.582847706,-4147566.693348472,-4080736.4449270866,-4011168.3145507076,-3938771.045795137,-3863450.5685141124,-3785109.916242269,-3703649.14126553,-3618965.2272993727,-3530951.999704795,-3439500.033182405,-3344496.5568666644,-3245825.3567605056,-3143366.6754279835,-3036997.108878562,-2926589.5005627405,-2812012.832402547,-2693132.112774713,-2569808.261369534,-2441897.9908352904,-2309253.685122112,-2171723.274444434,-2029150.1067573475,-1881372.8156673657,-1728225.184672961,-1569536.0076367194,-1405128.9453941584,-1234822.3783914354,-1058429.2552424017,-875756.9371040594,-686607.0377539601,-490775.2592523694,-288051.22307796404,-78218.29661188275,138946.58515247516,363673.1028015986,596197.7407231182,836763.980494028,1085622.4995727763,1343031.3754241727,1609256.2952455655,1884570.771413805,2169256.362823341,2463602.902270414,2767908.7300260626,3082480.933785012,3407635.5951406714,3743698.0427582897,4091003.112444669,4449895.414257171,4820729.606875492,5203870.679399503,5599694.240784625,6008586.817103911,6430946.156848878,6867181.544466911,7317714.122377727,7782977.221650727,8263416.701612769,8759491.298570298,9271672.983936407,9800447.33196791,10346313.897362653,10909786.603016041,11491394.138141517,12091680.3670672,12711204.748960264,13350542.76878406,14010286.379758766,14691044.457627019,15393443.267040187,16118126.940368263,16865757.969251037,17637017.7092368,18432606.89781393,19253246.186211403,20099676.68530982,20972660.526015446,21872981.434483297,22801445.322564024,23758880.893865146,24746140.265821826],[-5181215.402295798,-5152766.620649197,-5122880.000206517,-5091504.830198241,-5058588.787710974,-5024077.889286518,-4987916.441129015,-4950046.987880887,-4910410.259928949,-4868945.119199192,-4825588.503398845,-4780275.368663649,-4732938.63056414,-4683509.103428766,-4631915.437934225,-4578084.056918821,-4521939.089367356,-4463402.302517528,-4402393.0320387315,-4338828.110227449,-4272621.792166613,-4203685.679792928,-4131928.6438159607,-4057256.7434269404,-3979573.1437419206,-3898778.0309130354,-3814768.524848709,-3727438.5894720657,-3636678.9404581618,-3542376.950372707,-3444416.5511513464,-3342678.1338382764,-3237038.445517091,-3127370.4833540954,-3013543.3856770443,-2895422.3200077787,-2772868.367971327,-2645738.4069914576,-2513884.9886868224,-2377156.213886517,-2235395.6041604383,-2088441.9697854938,-1936129.2740420233,-1778286.4937434392,-1614737.4759032056,-1445300.790432062,-1269789.578755565,-1088011.3982512038,-899768.0623887181,-704855.4764560964,-503063.46876066364,-294175.61717947386,-77969.07093680277,145785.63251905888,377324.75566672906,616891.554228507,864736.475789493,1121117.363848161,1386299.6674669515,1660556.656642018,1944169.6435629167,2237428.209916616,2540630.440379061,2854083.1624810137,3178102.192998288,3513012.5910388976,3859148.9180247393,4216855.504711194,4586486.7254677825,4968407.279982951,5362992.482604969,5770628.559507595,6191712.953892538,6626654.639426956,7075874.442157844,7539805.37108559,8018892.957665887,8513595.604424078,9024384.942972336,9551746.201634873,10096178.582931144,10658195.651216438,11238325.730683737,11837112.314040061,12455114.482107706,13092907.33465662,13751082.432737196,14430248.252816219,15131030.653030785,15854073.351863354,16600038.419555465,17369606.782606795,18163478.74166388,18982374.50317549,19827034.725155752,20698221.077407356,21596716.816591155,22523327.376516398,23478880.974042647,24464229.230988316],[-5366680.897273914,-5338732.323804538,-5309352.037685487,-5278489.421232787,-5246092.2462005615,-5212106.625408672,-5176476.962979071,-5139145.903141178,-5100054.277568433,-5059141.051203651,-5016343.266532535,-4971595.986262856,-4924832.234363277,-4875982.935419552,-4824976.852258647,-4771740.521796495,-4716198.189057802,-4658271.739318347,-4597880.628319858,-4534941.810502676,-4469369.665202576,-4401075.920756767,-4329969.576462039,-4255956.822323207,-4178940.9565373687,-4098822.3006460173,-4015498.112297857,-3928862.4955498567,-3838806.308648374,-3745217.0692123603,-3647978.856758166,-3546972.212484314,-3442074.036249509,-3333157.4806639254,-3220091.842216895,-3102742.4493593574,-2980970.5474636983,-2854633.180571027,-2723583.069840042,-2587668.4886161415,-2446733.134016617,-2300615.994952406,-2149151.216481393,-1992167.960395812,-1829490.2619480863,-1660936.8826081362,-1486321.158741951,-1305450.8461110797,-1118127.960076347,-924148.6113885213,-723302.8374552336,-515374.4289584439,-300140.7517003622,-77372.56354977004,153166.1736378912,391718.487265015,638534.5925372783,893872.0964393038,1157996.2073086929,1431179.9501280468,1713704.387705166,2005858.8478963636,2307941.1570155974,2620257.8796165045,2943124.5647971556,3276865.999200005,3621816.4669050593,3978320.016358914,4346730.734563515,4727413.028686922,5120741.915308874,5527103.317488806,5946894.369869407,6380523.732012752,6828411.910211986,7290991.587959824,7768707.965343611,8262019.107550779,8771396.302775105,9297324.429728854,9840302.335011063,10400843.220630309,10979475.041887632,11576740.915930726,12193199.5412313,12829425.628291013,13486010.341845432,14163561.754868912,14862705.314694835,15584084.321554316,16328360.419851644,17096214.10252135,17888345.228773482,18705473.555602096,19548339.283398867,20417703.616024032,21314349.335720994,22239081.393248357,23192727.513620894,24176138.817854065],[-5556946.630752801,-5529506.765622749,-5500641.395999748,-5470299.998299109,-5438430.439970609,-5404978.931154963,-5369889.974949403,-5333106.316243061,-5294568.889083879,-5254216.762535017,-5211987.084979914,-5167815.026833694,-5121633.721614635,-5073374.205333911,-5022965.3541536145,-4970333.820269199,-4915403.965964586,-4858097.795790298,-4798334.886815092,-4736032.316895895,-4671104.590912603,-4603463.564912796,-4533018.368109218,-4459675.32266861,-4383337.861236726,-4303906.442132202,-4221278.462151766,-4135348.1669146465,-4046006.5586876757,-3953141.301613558,-3856636.624281181,-3756373.21955713,-3652228.1416112036,-3544074.7000558954,-3431782.3511234773,-3315216.5857986324,-3194238.814829314,-3068706.2505262606,-2938471.7852647435,-2803383.8666078346,-2663286.3689467562,-2518018.4615785377,-2367414.473116614,-2211303.752136334,-2049510.5239602868,-1881853.7434758414,-1708146.9438754544,-1528198.081218822,-1341809.3747003675,-1148777.1425051577,-948891.6331421714,-741936.8521290999,-527690.3839072995,-305923.20885822177,-76399.51529485174,161123.49370999448,406895.7987948824,661174.7682100479,924225.3673142605,1196320.3737886678,1477740.5987369306,1768775.1138272397,2069721.484618606,2380886.010258395,2702583.969700886,3035139.874619866,3378887.729212122,3734171.297035886,4101344.3751064874,4480771.075413002,4872826.114067493,5277895.10827494,5696374.881336976,6128673.775886377,6575211.975594947,7036421.835536452,7512748.221474187,8004648.858256748,8512594.68761263,9037070.235548621,9578573.989602102,10137618.786245998,10714732.208651036,11310456.995117629,11925351.45842833,12559989.91642646,13214963.134090578,13890878.777407337,14588361.87935705,15308055.318315612,16050620.309189962,16816736.907633323,17607104.527645245,18422442.472932674,19263490.482372977,20131009.28993178,21025781.19942135,21948610.67447382,22900324.94411977,23881774.62436794],[-5752136.106616636,-5725213.713835622,-5696872.111022139,-5667060.869802961,-5635727.954452573,-5602819.6735787215,-5568280.630417634,-5532053.671699616,-5494079.835046614,-5454298.294859988,-5412646.306657509,-5369059.14981712,-5323470.068681689,-5275810.211982271,-5226008.570530662,-5173991.913136633,-5119684.720698895,-5063009.118419278,-5003884.806091573,-4942228.9864090495,-4877956.291237561,-4810978.705799483,-4741205.490710917,-4668543.101811146,-4592895.107728974,-4514162.105118667,-4432241.631508037,-4347028.075686608,-4258412.585575189,-4166282.9734994527,-4070523.6188064627,-3971015.3677432863,-3867635.430530358,-3760257.275549881,-3648750.520572587,-3532980.8209410813,-3412809.7546323594,-3288094.704109881,-3158688.7348788865,-3024440.4706642823,-2885193.965106628,-2740788.569896373,-2591058.7992422655,-2435834.1905755615,-2274939.161395302,-2108192.862146916,-1935409.0250246227,-1756395.808596952,-1570955.6381387468,-1378885.0415529478,-1179974.4807704985,-974008.1785038505,-760763.9402309787,-540012.9712828211,-311519.68890694156,-75041.52916915528,169671.25144266523,422875.7798427362,684836.7757663745,955826.7669047061,1236126.309924204,1526024.217526514,1825817.7916908488,2135813.0632864498,2456325.038204141,2787677.9501804207,3130205.5205107573,3484251.2247960623,3850168.566944884,4228321.360594615,4619084.018163418,5022841.847721398,5439991.357893342,5870940.570990505,6316109.344613733,6775929.701909961,7250846.170750795,7741316.1320179645,8247810.17728476,8770812.476099921,9310821.153122544,9868348.675408002,10443922.250048531,11038084.232481541,11651392.545715787,12284421.11078157,12937760.288674349,13612017.33409422,14307816.861295631,15025801.322350854,15766631.498144511,16530987.00244511,17319566.799358666,18133089.73454075,18972295.08050762,19837943.096399162,20730815.602579672,21651716.570450403,22601472.727864683,23580934.180540558],[-5952378.63660004,-5925982.599060172,-5898173.73227535,-5868901.704173779,-5838114.576960761,-5805758.758833523,-5771778.954306319,-5736118.113106532,-5698717.3776031695,-5659516.028726349,-5618451.4303363385,-5575458.972000204,-5530472.010129893,-5483421.807439605,-5434237.470672839,-5382845.886555073,-5329171.655920569,-5273137.025963333,-5214661.820563072,-5153663.368630866,-5090056.43042093,-5023753.121753944,-4954662.836094606,-4882692.164421869,-4807744.812836883,-4729721.517841653,-4648519.959230274,-4564034.670521085,-4476156.946871294,-4384774.750396198,-4289772.61283228,-4191031.5354634793,-4088428.8862426262,-3981838.294029329,-3871129.5398668693,-3756168.445216596,-3636816.7570725903,-3512932.0298667606,-3384367.504078293,-3250971.981466503,-3112589.6968229683,-2969060.186162845,-2820218.1512516253,-2665893.32036855,-2505910.3052123804,-2340088.453841472,-2168241.699538935,-1990178.4055021051,-1805701.2052394915,-1614606.838558996,-1416685.983035326,-1211723.0808322076,-999496.1607567873,-779776.6554182228,-552329.2133641448,-316911.5060566254,-73274.02955123968,178840.09924987704,439695.3519389294,709564.0036571212,988726.354032848,1277470.95415866,1576094.8397493921,1884903.7706682198,2204212.476970913,2534344.911640458,2875634.510209454,3228424.4574139155,3593067.961100908,3969928.533553809,4359380.28044601,4761808.197612427,5177608.475850284,5607188.813947586,6050968.740180409,6509379.94246196,6982866.607411526,7471885.768528279,7976907.663759161,8498416.102666508,9036908.843445355,9592897.980088742,10166910.339905972,10759487.891705956,11371188.164896257,12002584.679803818,12654267.389486425,13326843.133337695,14020936.10279992,14737188.319487866,15476260.126041066,16238830.690050509,17025598.521364663,17837282.00315114,18674619.93705466,19538372.10280374,20429319.832652822,21348266.60103233,22296038.62979926,23273485.509481266],[-6157807.187969322,-6131946.492583128,-6104679.434588016,-6075955.779266992,-6045723.687824342,-6013929.669130048,-5980518.530075052,-5945433.32449796,-5908615.300644879,-5870003.84712068,-5829536.437290498,-5787148.572089409,-5742773.721194146,-5696343.262514749,-5647786.419956572,-5597030.199408416,-5543999.322905753,-5488616.1609183345,-5430800.662713989,-5370470.2847423665,-5307539.916985937,-5241921.807222858,-5173525.483145081,-5102257.672269832,-5028022.219589599,-4950720.002893441,-4870248.845701933,-4786503.4277437115,-4699375.192915197,-4608752.25464613,-4514519.298609398,-4416557.482695132,-4314744.334181083,-4208953.644019768,-4099055.358166239,-3984915.4658637643,-3866395.8848110884,-3743354.3431208134,-3615644.2579834852,-3483114.610955716,-3345609.819768925,-3202969.6065782616,-3055028.862547811,-2901617.5086739054,-2742560.352751706,-2577676.9423772264,-2406781.413875686,-2229682.3370553367,-2046182.555669887,-1856079.0234735515,-1659162.6357564926,-1455218.056236297,-1244023.5391829293,-1025350.7466493659,-798964.5606812946,-564622.8903675862,-322076.4735956583,-71068.6733817216,188664.73139091022,457395.76094631106,735404.4554721862,1022979.1020131558,1320416.467544157,1628022.0384097118,1946110.2662800364,2275004.8207957074,2615038.8490984756,2966555.242391741,3329906.9097532723,3705457.059363492,4093579.4873605613,4494658.874511331,4909091.090909924,5337283.5089016035,5779655.32447405,6236637.887297943,6708675.039685773,7196223.46465317,7699753.04337235,8219747.22222296,8756703.389690885,9311133.263412155,9883563.287568022,10474535.040942632,11084605.65589419,11714348.24854502,12364352.360460423,13035224.41211814,13727588.168483414,14442085.21699227,15179375.458260588,15940137.609865218,16725069.723501183,17534889.71589219,18370335.913794417,19232167.61344597,20121165.654849213,21038133.011257656,21983895.394260477,22959301.874857448],[-6368555.266290484,-6343239.175467281,-6316523.278937366,-6288357.440492544,-6258689.921507568,-6227467.332714349,-6194634.584587622,-6160134.836302758,-6123909.443227034,-6085897.902903132,-6046037.799483354,-6004264.74657264,-5960512.328434348,-5914712.0395164145,-5866793.222248694,-5816683.00306702,-5764306.226612875,-5709585.388058484,-5652440.563508367,-5592789.338421997,-5530546.7340038605,-5465625.131506819,-5397934.19439078,-5327380.788275974,-5253868.898635359,-5177299.546159173,-5097570.699733727,-5014577.186962982,-4928210.602173854,-4838359.2118282,-4744907.857280327,-4647737.854799065,-4546726.89278743,-4441748.926119996,-4332674.067521547,-4219368.475904938,-4101694.241591135,-3979509.2683217116,-3852667.151977822,-3721017.0559244314,-3584403.582875966,-3442666.6432035053,-3295641.319579128,-3143157.7278595464,-2985040.874114141,-2821110.5076896064,-2651180.9702021163,-2475061.0403559487,-2292553.774472438,-2103456.342611989,-1907559.8601786736,-1704649.21488134,-1494502.88893014,-1276892.7763395887,-1051583.995212255,-818334.6948649101,-576895.8576602116,-327011.09541559406,-68416.44022107124,199159.87045395374,475997.61453418806,762384.8121477123,1058617.9586131014,1365002.2637916245,1681851.897953935,2009490.2443341464,2348250.1585681085,2698474.235159626,3060515.0811972097,3434735.5974845216,3821509.2672959063,4221220.452945802,4634264.700383818,5061049.052013511,5501992.3679759875,5957525.656081429,6428092.410656335,6914148.960491419,7416164.8261794895,7934623.087048383,8470020.757939842,9022869.176131135,9593694.398605634,10183037.609983487,10791455.541363597,11419520.900381941,12067822.812756225,12736967.275618669,13427577.622951798,14140295.003429875,14875778.870983385,15634707.488432765,16417778.444495939,17225709.184545565,18059237.55545728,18919122.364900704,19806143.955459725,20721104.79395519,21664830.076361313,22638168.348709945],[-6584762.653236792,-6560000.547894882,-6533845.28380927,-6506246.824404796,-6477153.532367225,-6446512.121447653,-6414267.606878756,-6380363.254364018,-6344740.527600734,-6307339.034295934,-6268096.470633746,-6226948.564151967,-6183829.014982114,-6138669.435410588,-6091399.287711539,-6041945.820207083,-5990234.001504093,-5936186.452856578,-5879723.378605809,-5820762.494641793,-5759218.954833191,-5695005.275371054,-5628031.256968885,-5558203.904857699,-5485427.346521362,-5409602.747104715,-5330628.222436676,-5248398.749597191,-5162806.074968394,-5073738.61969381,-4981081.382483393,-4884715.839684424,-4784519.842550846,-4680367.511631068,-4572129.128198237,-4459671.022640249,-4342855.459733576,-4221540.520710036,-4095579.982031146,-3964823.190788931,-3829114.936629099,-3688295.320116774,-3542199.617440678,-3390658.141357557,-3233496.098282039,-3070533.441414725,-2901584.7197983153,-2726458.923202079,-2544959.322717201,-2356883.3069472387,-2162022.2136815228,-1960161.156927241,-1751078.8491775207,-1534547.4187879656,-1310332.222334953,-1078191.6518175341,-837876.9365669657,-589131.9397341255,-331692.9491879251,-65288.46270384267,210361.03272611275,495543.2876175288,790554.5228944737,1095699.6691531409,1411292.6124389302,1737656.4467089176,2075123.733176779,2424036.766684143,2784747.849320492,3157619.571455002,3543025.100391576,3941348.476836005,4352984.919386717,4778341.137247296,5217835.651402006,5671899.124436874,6140974.699274821,6625518.347009001,7125999.2241240665,7642900.039310569,8176717.430122554,8727962.349776099,9297160.464294292,9884852.560309894,10491594.963777073,11117959.969896853,11764536.284526523,12431929.47737468,13120762.447296463,13831675.899991993,14565328.838425308,15322399.066309296,16103583.704962213,16909599.723910846,17741184.485581625,18599096.30443176,19484115.02090667,20397042.590596624,21338703.68898391,22309946.332175065],[-6806572.739393411,-6782374.122399608,-6756789.083504624,-6729767.686860915,-6701258.397575227,-6671208.033543364,-6639561.715897449,-6606262.818026479,-6571252.913131573,-6534471.720274556,-6495857.048878552,-6455344.741638507,-6412868.61579583,-6368360.402734636,-6321749.685850557,-6272963.836647417,-6221927.949011132,-6168564.771610172,-6112794.638374014,-6054535.39699411,-5993702.335393811,-5930208.106112767,-5863962.648548626,-5794873.108994486,-5722843.758417097,-5647775.907909098,-5569567.821756855,-5488114.628052731,-5403308.2267930005,-5315037.195384074,-5223186.691495726,-5127638.353181214,-5028270.196196351,-4924956.5084385015,-4817567.741428272,-4705970.398752933,-4590026.921393588,-4469595.569846941,-4344530.302955566,-4214680.65336541,-4079891.599507087,-3940003.4340206357,-3794851.628519723,-3644266.6945974296,-3488074.0409786087,-3326093.8267113212,-3158140.8102879114,-2984024.1945954245,-2803547.467578184,-2616508.2384966724,-2422698.0696706623,-2221902.3035824876,-2013899.885217473,-1798463.1795141455,-1575357.7837977745,-1344342.3350586444,-1105168.311939858,-857579.8313038312,-601313.4392118882,-336097.89619517326,-61653.956648528576,222305.85780800506,516077.49114295654,819965.5920882132,1134283.759825645,1459354.7963574901,1795510.9657573625,2143094.2604458127,2502456.67471309,2873960.4856513496,3257978.5417088587,3654894.5590540245,4065103.425961118,4489011.515415942,4927037.006182365,5379610.212512556,5847173.922768883,6330183.747142114,6829108.474755509,7344430.440359443,7876645.9008673765,8426265.4220302,8993814.275454974,9579832.846278701,10184877.051748853,10809518.771015294,11454346.28640341,12119964.736470547,12806996.581159849,13516082.079354845,14247879.779151563,15003067.021194164,15782340.455379207,16586416.571303762,17416032.242798556,18271945.286898013,19154935.03763396,20065802.93502494,21005373.129653506,21974493.10322483],[-7034130.415462214,-7010505.043161141,-6985500.079233531,-6959065.689729113,-6931150.443362506,-6901701.263378558,-6870663.378030918,-6837980.269634295,-6803593.622152198,-6767443.267278351,-6729467.128971083,-6689601.166397908,-6647779.315245116,-6603933.427349506,-6557993.208603299,-6509886.155087866,-6459537.487384809,-6406870.083014989,-6351804.406955729,-6294258.440181466,-6234147.606174061,-6171384.695348537,-6105879.787336337,-6037540.171065776,-5966270.262583743,-5891971.520551975,-5814542.359360263,-5733878.059784693,-5649870.6771323085,-5562408.946794944,-5471378.187151322,-5376660.199736466,-5278133.166611247,-5175671.544852732,-5069145.958088304,-4958423.08499225,-4843365.544667311,-4723831.778821722,-4599675.930655891,-4470747.720377213,-4336892.317240025,-4197950.208029631,-4053757.0618874896,-3904143.591378578,-3748935.409706982,-3587952.8839711305,-3421010.9843506273,-3247919.129122913,-3068481.0253938027,-2882494.5054255053,-2689751.358450068,-2490037.157844335,-2283131.0835433993,-2068805.7395653594,-1846826.9665204678,-1616953.6489669103,-1378937.5174769051,-1132522.945283711,-877446.7393425312,-613437.9256849438,-340217.528897278,-57498.3455683738,235015.28843587078,537627.7370581347,850652.3096447941,1174411.5132233817,1509237.3116587587,1855471.3918325324,2213465.437066961,2583581.4079575427,2966191.8308248855,3361680.093974851,3770440.7519783713,4192879.8381694015,4629415.185601298,5080476.75664535,5546506.981498262,6027961.105784675,6525307.5475423895,7039028.263796531,7569619.126972845,8117590.311446678,8683466.690434214,9267788.243536342,9871110.475187,10494004.844310101,11137059.205455132,11800878.261714209,12486084.029732648,13193316.317119002,13923233.212568633,14676511.589049049,15453847.620350268,16255957.311376743,17083577.042520925,17937464.128471155,18818397.39183968,19727177.751983713,20664628.829410892,21631597.566162977],[-7267585.57001801,-7244543.385961851,-7220128.53553945,-7194291.287885866,-7166980.316529067,-7138142.6512862,-7107723.628773794,-7075666.841491935,-7041914.085444796,-7006405.306255389,-6969078.543733746,-6929869.874856142,-6888713.35510984,-6845540.9581606295,-6800282.513794361,-6752865.644087567,-6703215.6977568455,-6651255.682636103,-6596906.196233267,-6540085.354310796,-6480708.717437017,-6418689.215453006,-6353937.069798532,-6286359.713635489,-6215861.7097133035,-6142344.665910195,-6065707.148391759,-5985844.592315669,-5902649.210023535,-5816009.896643078,-5725812.133038919,-5631937.886032305,-5534265.505821319,-5432669.620523164,-5327021.027761056,-5217186.5832146,-5103029.086056175,-4984407.161183784,-4861175.13816447,-4733182.926807332,-4600275.889262172,-4462294.708563869,-4319075.253518704,-4170448.439834117,-4016240.0873977654,-3856270.7735977154,-3690355.6825749334,-3518304.4503073096,-3339921.005408693,-3155003.405526286,-2963343.669225609,-2764727.6032374073,-2558934.6249449253,-2345737.5799838603,-2124902.554827921,-1896188.684222404,-1659347.9533299468,-1414124.9944581185,-1160256.8782030046,-897472.8988872543,-625494.3541241027,-344034.3183524329,-52797.41019966267,248520.44651319087,560232.268274704,882660.2640665323,1216136.0944241863,1561001.1375189368,1917606.7624826469,2286314.610139182,2667496.88135341,3061536.633186441,3468828.0830686595,3889776.9211887233,4324800.63133911,4774328.820401683,5238803.556740135,5718679.717685109,6214425.346400095,6726522.018333999,7255465.217510242,7801764.722949516,8365945.0054325685,8948545.634912744,9550121.698830806,10171244.231635977,10812500.655782893,11474495.234507333,12157849.536693119,12863202.91413553,13591212.991515547,14342556.169433214,15117928.14080302,15918044.420987833,16743640.892011844,17595474.361204572,18474323.13466192,19380987.605897322,20316290.86007377,21281079.294211265],[-7507093.4404894225,-7484644.494496899,-7460829.901511863,-7435600.0348923,-7408903.674132997,-7380687.95679311,-7350898.329037944,-7319478.494755608,-7286370.363210464,-7251513.995191353,-7214847.547613984,-7176307.216535101,-7135827.17853245,-7093339.530408639,-7048774.227169268,-7002059.018231191,-6953119.3818100225,-6901878.457436005,-6848256.976550819,-6792173.191127954,-6733542.8002652135,-6672278.874693766,-6608291.779146341,-6541489.092524139,-6471775.5258066645,-6399052.837637811,-6323219.74753046,-6244171.846617901,-6161801.505893208,-6075997.78186002,-5986646.319532786,-5893629.252706575,-5796825.101428825,-5696108.666593596,-5591350.921581957,-5482418.900866192,-5369175.585501634,-5251479.785415571,-5129186.018408187,-5002144.385783621,-4870200.444508211,-4733195.075815431,-4590964.350153679,-4443339.388379249,-4290146.219098998,-4131205.632055903,-3966333.0274479818,-3795338.261080006,-3618025.4852311965,-3434192.985123286,-3243633.0108764675,-3046131.6048296187,-2841468.424101509,-2629416.5582665354,-2409742.3420170974,-2182205.1626759153,-1946557.2624214385,-1702543.5350971464,-1449901.317437742,-1188360.1745919809,-917641.679772798,-637459.1878797673,-347517.60295165144,-47513.13926230557,262866.9240896981,583944.4952702541,916050.9286740683,1259527.2909087576,1614724.6340195723,1982004.2761193737,2361738.0896337554,2754308.797350755,3160110.276486391,3579547.870964043,4013038.712148484,4461012.0482174605,4923909.582438551,5402185.820536183,5896308.427437283,6406758.593601245,6934031.411184542,7478636.260336153,8041097.205830667,8621953.404349472,9221759.522660471,9841086.167002752,10480520.32394366,11140665.813012812,11822143.751423847,12525593.031190537,13251670.808950912,14001053.008847248,14774434.838765785,15572531.320311736,16396077.832860366,17245830.67203606,18122567.623005584,19027088.54895797,19960215.995162386,20922795.808997907],[-7752811.745258192,-7730966.285129655,-7707762.293198839,-7683150.248205905,-7657079.036787996,-7629495.905438246,-7600346.4110794645,-7569574.370213574,-7537121.806608065,-7502928.897478156,-7466933.918123695,-7429073.18497812,-7389280.997024532,-7347489.575535642,-7303629.0020892,-7257627.154813927,-7209409.642815316,-7158899.738731116,-7106018.309367416,-7050683.7443602625,-6992811.882809231,-6932315.937828613,-6869106.418958805,-6803091.052376742,-6734174.698850331,-6662259.269370136,-6587243.638399852,-6509023.554675127,-6427491.549490789,-6342536.842400467,-6254045.2442667205,-6161899.057581346,-6065976.973989077,-5966153.968934156,-5862301.193354403,-5754285.862340126,-5641971.140681513,-5525216.025213964,-5403875.223877052,-5277799.031404176,-5146833.201540657,-5010818.815709827,-4869592.148022771,-4722984.52653462,-4570822.190651782,-4412926.144583046,-4249112.006725557,-4079189.854884412,-3902964.06721011,-3720233.15873708,-3530789.6134117898,-3334419.711486237,-3130903.3521541506,-2920013.871302504,-2701517.854251845,-2475174.943347417,-2240737.640265219,-1997951.1029029712,-1746552.9366904628,-1486272.9801969063,-1216833.0848679915,-937946.8887367975,-649319.5839660093,-350647.6780357659,-41618.74842650071,278088.8093751278,608806.0407435894,950873.6977999713,1304642.5125360806,1670473.4773420133,2048738.1331469119,2439818.86536232,2844109.2078394573,3262014.155038338,3693950.4826492276,4140347.076849913,4601645.272465806,5078299.200218119,5570776.14334856,6079556.903825879,6605136.178384833,7148022.944693845,7708740.857857369,8287828.6575644985,8885840.58613344,9503346.817758046,10140933.899224743,10799205.202403214,11478781.38882275,12180300.886639621,12904420.380309999,13651815.313315913,14423180.404247977,15219230.176620554,16040699.502759676,16888344.162116546,17762941.414391402,18665290.58784131,19596213.683162518,20556555.99334161],[-8004900.18523797,-7983668.783486256,-7961086.0662302915,-7937102.6188738365,-7911667.436497206,-7884727.875847986,-7856229.605946978,-7826116.557270777,-7794330.869471489,-7760812.837593381,-7725500.8567442885,-7688331.365180348,-7649238.785757812,-7608155.465710087,-7565011.614700463,-7519735.241106192,-7472252.086483261,-7422485.558161294,-7370356.659920197,-7315783.9206925295,-7258683.321239197,-7198968.218743116,-7136549.269263892,-7071334.347992742,-7003228.467251685,-6932133.692171247,-6857949.053987587,-6780570.460888962,-6699890.6063514,-6615798.874887673,-6528181.245147701,-6436920.190290314,-6341894.575558756,-6242979.552980799,-6140046.45311672,-6032962.673773471,-5921591.565608161,-5805792.314530894,-5685419.820821656,-5560324.574879614,-5430352.529501739,-5295344.968610241,-5155138.372325489,-5009564.278286,-4858449.139120756,-4701614.175967004,-4538875.2279233895,-4370042.597338889,-4194920.890819969,-4013308.855840808,-3824999.2128442917,-3629778.4827095065,-3427426.809463408,-3217717.7781096287,-3000418.227446828,-2775288.0577398594,-2542080.0331071634,-2300539.578494709,-2050404.5710705668,-1791405.1259184238,-1523263.3758621886,-1245693.2452662159,-958400.2176683936,-661081.0970606636,-353423.7626661733,-35106.917041441426,294200.17269327305,634837.9406393915,987156.7951428294,1351517.3992176857,1728290.9585818462,2117859.5174926817,2520616.2625949383,2936965.834978359,3367324.650685422,3812121.2298531383,4271796.534754932,4746804.31692896,5237611.473680422,5744698.414163291,6268559.435292486,6809703.107782032,7368652.67251581,7945946.447561685,8542138.246079363,9157797.80542767,9793511.227739185,10449881.432266694,11127528.619812112,11827090.74954364,12549224.028515898,13294603.41424023,14063923.130608525,14857897.197546937,15677259.974739045,16522766.719771557,17395194.16108733,18295341.08611898,19224028.944994055,20182102.470205005],[-8263527.330527389,-8242920.616825327,-8220969.902477967,-8197625.880735264,-8172837.65631954,-8146552.697448365,-8118716.78647312,-8089273.969094526,-8058166.502116598,-8025334.799697247,-7990717.378055172,-7954250.798590004,-7915869.609370712,-7875506.284949717,-7833091.164453123,-7788552.387903519,-7741815.830723727,-7692805.036371737,-7641441.147057785,-7587642.832488317,-7531326.216583664,-7472404.802114714,-7410789.393201335,-7346388.015611712,-7279105.834807078,-7208845.071665478,-7135504.915826108,-7058981.4365835795,-6979167.491272438,-6895952.631066032,-6809223.004127456,-6718861.256032932,-6624746.427399848,-6526753.848640606,-6424755.031764888,-6318617.559149636,-6208204.969198903,-6093376.638804546,-5973987.662521666,-5849888.728377911,-5720925.990212825,-5586940.936467619,-5447770.25532103,-5303245.6960738795,-5153193.926687125,-4997436.387366125,-4835789.140082309,-4668062.713931063,-4494061.94620979,-4313585.819100067,-4126427.2918418236,-3932373.1282755714,-3731203.719630301,-3522692.9024295546,-3306607.771388756,-3082708.4871670753,-2850748.07883624,-2610472.240938464,-2361619.1249656137,-2103919.1251396127,-1837094.658325009,-1560859.9379193261,-1274920.7415775117,-978974.1725857314,-672708.4147337489,-355802.4805134721,-27925.952448312193,311261.2825920228,662109.3063119836,1024978.4493096378,1400239.5790060218,1788274.3953728117,2189475.734669909,2604247.881390851,3033006.8886561505,3476180.90723864,3934210.523486715,4407549.106331743,4896663.163667049,5402032.708304416,5924151.633758284,6463528.100153819,7020684.930464897,7596160.017393198,8190506.7411383055,8804294.398364808,9438108.642634153,10092551.93660536,10768244.01631584,11465822.36784741,12185942.716692742,12929279.530168548,13696526.533180151,14488397.237712122,15305625.486386176,16148966.010437481,17019195.002496183,17917110.70454553,18843534.01144858,19799309.090436567],[-8528863.656622732,-8508892.461542232,-8487584.680767084,-8464891.116591154,-8440760.984593317,-8415141.865692377,-8387979.656818574,-8359218.520162631,-8328800.830963641,-8296667.123794507,-8262756.0373041015,-8227004.257373424,-8189346.458640752,-8149715.244352885,-8108041.084493684,-8064252.252145473,-8018274.7580321655,-7970032.283194641,-7919446.109748408,-7866435.0496695805,-7810915.3715548385,-7752800.725301392,-7692002.064649454,-7628427.567526525,-7561982.554137743,-7492569.402736214,-7420087.463014713,-7344432.967048387,-7265498.937728459,-7183175.094611043,-7097347.757119313,-7007899.745018636,-6914710.276098043,-6817654.86097735,-6716605.194964896,-6611429.046883393,-6501990.144786635,-6388148.058478244,-6269758.078746537,-6146671.0932338005,-6018733.458837188,-5885786.870560843,-5747668.226715621,-5604209.490368462,-5455237.546946423,-5300574.057888361,-5140035.310235165,-4973432.062057419,-4800569.383604707,-4621246.494060153,-4435256.593788452,-4242386.691953324,-4042417.4293820225,-3835122.896549333,-3620270.4465547036,-3397620.502954604,-3166926.362314364,-2927933.9913496245,-2680381.8184915166,-2424000.5197539255,-2158512.79873525,-1883633.1605988517,-1599067.6798899174,-1304513.7620031647,-999659.898150634,-684185.4136580043,-357760.2093930561,-20044.496181456372,329311.4780104775,690667.7082904708,1064394.7206625976,1450873.8676461838,1850497.6319024153,2263669.9380666316,2690806.473026853,3132335.0148316585,3588695.770493783,4060341.7228761464,4547738.986946799,5051367.17560897,5571719.775356773,6109304.532052141,6664643.847029246,7238275.183837716,7830751.485874131,8442641.605207846,9074530.742869262,9727020.900903944,10400731.346504096,11096299.088522855,11814379.366685621,12555646.153845675,13320792.671587791,14110531.919555504,14925597.218841892,15766742.769796178,16634744.224632073,17530399.275209818,18454528.256382804,19407974.76530333],[-8801083.185496677,-8781758.59531555,-8761104.937511547,-8739073.124677392,-8715612.484521251,-8690670.711954074,-8664193.819794558,-8636126.088051498,-8606410.01174643,-8574986.247233903,-8541793.556979317,-8506768.752751362,-8469846.63718427,-8430959.943666518,-8390039.274508027,-8347013.037340323,-8301807.3796997825,-8254346.121743142,-8204550.687046934,-8152340.0314351,-8097630.569781967,-8040336.100735908,-7980367.729306099,-7917633.787251992,-7852039.75122013,-7783488.158560966,-7711878.520768777,-7637107.234472769,-7559067.489920644,-7477649.176877739,-7392738.78788087,-7304219.3187657865,-7211970.16640199,-7115867.023554394,-7015781.770796479,-6911582.365392314,-6803132.727071101,-6690292.620604375,-6572917.535100361,-6450858.559934148,-6323962.257210542,-6192070.53067908,-6055020.490998028,-5912644.317249189,-5764769.114608606,-5611216.768066032,-5451803.792084269,-5286341.1760974135,-5114634.225731773,-4936482.399633525,-4751679.141791103,-4560011.709228719,-4361260.994947797,-4155201.3459898774,-3941600.376493804,-3720218.7756097075,-3490810.1101341825,-3253120.621736221,-3006889.018608773,-2751846.2614241056,-2487715.3434248343,-2214211.0644956995,-1931039.7990736011,-1637899.2577095646,-1334478.2421332411,-1020456.3936473597,-695503.9346562997,-359281.4031838011,-11439.380159571767,348381.7906896658,720552.2905545644,1105453.121052429,1503476.4077419862,1915025.7118417174,2340516.3503923602,2780375.725046385,3235043.6597515643,3704972.747513946,4190628.7065279707,4692490.745879229,5211051.941070482,5746819.619666327,6300315.757263329,6872077.384095747,7462657.002527706,8072623.015736546,8702560.16785613,9353069.995883532,10024771.293659814,10718300.588231176,11434312.628904365,12173480.889342982,12936498.08300943,13724076.692326818,14536949.511901744,15375870.206159998,16241613.881779853,17134977.67529671,18056781.35626878,19007867.94639823],[-9080364.643640544,-9061697.994468812,-9041709.901635364,-9020351.38933582,-8997571.898730591,-8973319.240065321,-8947539.543408932,-8920177.207970455,-8891174.849955954,-8860473.248924535,-8828011.292601839,-8793725.920109343,-8757552.063563934,-8719422.58800459,-8679268.229598202,-8637017.532079477,-8592596.781373981,-8545929.938354924,-8496938.569684003,-8445541.776681593,-8391656.122173034,-8335195.555256065,-8276071.333932654,-8214191.945544196,-8149463.024954562,-8081787.270414681,-8011064.357050436,-7937190.847903199,-7860060.102463109,-7779562.182619676,-7695583.755967138,-7608007.996385227,-7516714.481827637,-7421579.089238714,-7322473.886522417,-7219267.021481475,-7111822.607649857,-7000000.6069290675,-6883656.708942782,-6762642.207028282,-6636803.870761753,-6505983.81493708,-6370019.364894489,-6228742.918101442,-6081981.8018906135,-5929558.127247816,-5771288.638541164,-5606984.559090449,-5436451.432460288,-5259488.959361685,-5075890.830049632,-4885444.552092796,-4687931.273393409,-4483125.600329164,-4270795.410891505,-4050701.662682034,-3822598.195631435,-3586231.529311111,-3341340.654671848,-3087656.8200877104,-2824903.311537642,-2552795.2267692722,-2271039.243302617,-1979333.380087791,-1677366.7526666913,-1364819.3216666728,-1041361.6344300304,-706654.5596346185,-360349.0146853812,-2085.6857119835913,368505.26003665477,751804.4695910364,1148203.7076627798,1558106.168248091,1981926.7946732156,2420092.6082644984,2873043.04591034,3341230.3067005724,3825119.707930267,4325190.050674107,4841933.995181441,5375858.446387712,5927484.949748576,6497350.097707555,7086005.947046936,7694020.447427675,8321977.881386466,8970479.316093277,9640143.067180455,10331605.174949192,11045519.893267278,11782560.191505142,12543418.269814067,13328806.088121746,14139455.909185346,14976120.85605425,15839575.484327585,16730616.36957905,17650062.710340247,18598756.947035547],[-9366893.687823333,-9348896.437128127,-9329585.471574225,-9308911.92820543,-9286825.362896938,-9263273.702508828,-9238203.19565786,-9211558.362068655,-9183281.940465562,-9153314.834964273,-9121596.05992163,-9088062.68320191,-9052649.767813828,-9015290.311875738,-8975915.18686051,-8934453.074074848,-8890830.399323322,-8844971.265705924,-8796797.384501165,-8746228.004079003,-8693179.83679049,-8637566.983779982,-8579300.857661724,-8518290.103001377,-8454440.514545735,-8387654.953135266,-8317833.259240469,-8244872.164051971,-8168665.198064394,-8089102.597078143,-8006071.205557264,-7919454.37726348,-7829131.873098865,-7734979.756078061,-7636870.28335333,-7534671.795211293,-7428248.600963706,-7317460.8616433665,-7202164.469419461,-7082210.923650571,-6957447.203473094,-6827715.63684399,-6692853.765934786,-6552694.208779008,-6407064.517077673,-6255787.030056432,-6098678.724265004,-5935551.059217898,-5766209.818760846,-5590454.948046286,-5408080.386006827,-5218873.893202027,-5022616.874916596,-4819084.199382506,-4608044.010998523,-4389257.53840971,-4162478.89731087,-3927454.887844285,-3683924.7864259593,-3431620.131879069,-3170264.505706299,-2899573.306346746,-2619253.517273605,-2329003.4687483925,-2028512.5930804946,-1717461.1732206196,-1395520.0844921898,-1062350.5293151774,-717603.7647035532,-360920.8223699406,8067.778769839555,389742.32788430527,784494.2255142815,1192726.2950605229,1614853.1027114429,2051301.285992831,2502509.891207803,2968930.719951287,3451028.684986826,3949282.1756917834,4464183.433320675,4996238.9363821335,5545969.796336606,6113912.1639245115,6700617.646375388,7306653.735803239,7932604.249056369,8579069.779324822,9246668.159816924,9936034.939809918,10647823.87338933,11382707.421223212,12141377.2656762,12924544.839637194,13732941.869401779,14567320.931960922,15428456.027081475,16317143.164550468,17234200.966974437,18180471.288527112],[-9660859.167646144,-9643542.990112647,-9624920.933000602,-9604944.247463541,-9583562.605370296,-9560724.051491836,-9536374.95430709,-9510459.955388729,-9482921.917330213,-9453701.870173445,-9422738.956294972,-9389970.37370956,-9355331.317745127,-9318754.9210466,-9280172.191859977,-9239511.950551892,-9196700.764314253,-9151662.880003441,-9104320.155065762,-9054591.986493338,-9002395.23775782,-8947644.163667152,-8890250.33308772,-8830122.5494723,-8767166.769137084,-8701286.017222013,-8632380.301276527,-8560346.522399317,-8485078.383873008,-8406466.297218021,-8324397.285602642,-8238754.884531292,-8149419.039741537,-8056266.002232198,-7959168.220345155,-7857994.228819793,-7752608.534742711,-7642871.500303665,-7528639.22227183,-7409763.408111225,-7286091.248632331,-7157465.287099584,-7023723.284690885,-6884698.082212149,-6740217.45797107,-6590103.98170371,-6434174.864444541,-6272241.804239523,-6104110.827585734,-5929582.12648196,-5748449.89097796,-5560502.137099208,-5365520.530023919,-5163280.202385835,-4953549.567575669,-4736090.127904281,-4510656.277491109,-4276995.099748699,-4034846.1592972483,-3783941.2881879285,-3524004.3662670404,-3254751.0955259763,-2975888.7682942357,-2687116.0290904585,-2388122.6299805045,-2078589.1792715453,-1758186.8833458293,-1426577.2814886048,-1083411.9734920524,-728332.3398680724,-360969.25446286425,19057.210718013346,412138.08868344873,818675.8288278431,1239084.6167400554,1673790.702632416,2123232.7386572286,2587862.1252960376,3068143.3671085685,3564554.4380477555,4077587.1565903425,4607747.570978876,5155556.35478178,5721549.213081248,6306277.299539685,6910307.644649502,7534223.59543483,8178625.26690786,8844130.005591895,9531372.865415059,10241007.096290234,10973704.645726677,11730156.6737785,12511074.08170386,13317188.05467512,14149250.61889308,15008035.213488456,15894337.277585346,16808974.852914833,17752789.20237446],[-9962454.004626561,-9945830.844075028,-9927909.748744844,-9908642.085216112,-9887977.642686717,-9865864.585192867,-9842249.402449204,-9817076.859269295,-9790289.943528663,-9761829.812628135,-9731635.73841729,-9699645.05053544,-9665793.078124704,-9630013.089873116,-9592236.232338028,-9552391.46650623,-9510405.502539696,-9466202.732656768,-9419705.162100201,-9370832.338136444,-9319501.277033806,-9265626.388964146,-9209119.40077106,-9149889.276544496,-9087842.13594544,-9022881.170214865,-8954906.555808714,-8883815.365588069,-8809501.477505239,-8731855.48070941,-8650764.57901048,-8566112.49162094,-8477779.351108382,-8385641.59847963,-8289571.875319828,-8189438.912905229,-8085107.4182122275,-7976437.956733646,-7863286.83201685,-7745505.961841629,-7622942.750935978,-7495439.960148473,-7362835.571974222,-7224962.65233676,-7081649.208530892,-6932718.043218916,-6777986.604372434,-6617266.831058014,-6450364.994950626,-6277081.537459876,-6097210.9023561645,-5910541.3637733385,-5716854.849465853,-5515926.759192279,-5307525.77809988,-5091413.684971763,-4867345.155201236,-4635067.55836354,-4394320.750219734,-4144836.8590304945,-3886340.06601258,-3618546.3797832094,-3341163.4046485927,-3053890.102552755,-2756416.54853509,-2448423.679525677,-2129583.036282288,-1799556.4983234815,-1457996.0116395503,-1104543.30901438,-738829.6227512471,-360475.3896108791,30910.05224681273,435728.7745166719,854394.5784371905,1287333.3230982013,1734983.26263256,2197795.3924795315,2676233.8050047643,3170776.0546837635,3681913.5330992825,4210151.854047008,4756011.248956736,5320026.972939055,5902749.721708108,6504746.059684403,7126598.859547369,7768907.753539596,8432289.596834626,9117378.94327318,9824828.53378186,10555309.797821343,11309513.368167691,12088149.609401409,12891949.160445005,13721663.491499979,14578065.475769505,15461949.976337906,16374134.448598534,17315459.558623005],[-10271876.242285576,-10255958.309878767,-10238750.499904796,-10220204.295737507,-10200269.605291063,-10178894.713273996,-10156026.232064001,-10131609.051163934,-10105586.285200283,-10077899.220423456,-10048487.259668246,-10017287.865732746,-9984236.503130142,-9949266.578170847,-9912309.377326299,-9873294.00382967,-9832147.312463056,-9788793.842481196,-9743155.7486223,-9695152.73015151,-9644701.957883514,-9591717.999129813,-9536112.740513336,-9477795.308590122,-9416671.988222145,-9352646.13863527,-9285618.10710406,-9215485.140192762,-9142141.292493023,-9065477.332782486,-8985380.647541946,-8901735.141752165,-8814421.136902018,-8723315.266129183,-8628290.366417129,-8529215.367766643,-8425955.179264836,-8318370.571962433,-8206318.058474096,-8089649.769219578,-7968213.325203906,-7841851.707255646,-7710403.12161953,-7573700.86180677,-7431573.166607002,-7283843.074155515,-7130328.271946898,-6970840.942693977,-6805187.605916303,-6633168.955142135,-6454579.690612303,-6269208.347361866,-6076837.118557479,-5877241.673963238,-5670190.973408245,-5455447.075119035,-5232764.938780539,-5001892.223195866,-4762569.078380326,-4514527.931966202,-4257493.269753087,-3991181.410246756,-3715300.2730446514,-3429549.1408830266,-3133618.415195156,-2827189.365008626,-2509933.868986683,-2181514.150467273,-1841582.5052819662,-1489781.0221882015,-1125741.2957068384,-749084.1311747953,-359419.241802752,43655.06245993823,460552.1944936216,891697.6181929559,1337529.1855334044,1798497.482706979,2275066.1856121607,2767712.4249050133,3276927.160861764,3803215.5683468245,4347097.43209444,4909107.552612349,5489796.1629594825,6089729.356700916,6709489.527309515,7349675.819316838,8010904.591524452,8693809.892580755,9399043.949237507,10127277.667632718,10879201.147903357,11655524.212503262,12456976.948565431,13284310.264661398,14138296.462342344,15019729.822834108,15929427.209276512,16868228.68490112],[-10589331.311575267,-10574130.959828435,-10557648.899277348,-10539836.731401611,-10520644.484161096,-10500020.564284543,-10477911.708179103,-10454262.931421826,-10429017.476794377,-10402116.76082003,-10373500.318761788,-10343105.748039449,-10310868.650020227,-10276722.57014058,-10240598.936310332,-10202426.99555445,-10162133.748842264,-10119643.884053387,-10074879.707032397,-10027761.070676325,-9978205.302002156,-9926127.127140215,-9871438.594195183,-9814048.993915625,-9753864.77811509,-9690789.47577944,-9624723.60680179,-9555564.593274914,-9483206.668280905,-9407540.782102823,-9328454.505796077,-9245831.93204014,-9159553.573202882,-9069496.256538352,-8975533.016441911,-8877532.983681127,-8775361.271525057,-8668878.858682934,-8557942.46896725,-8442404.447598625,-8322112.63405097,-8196910.231355828,-8066635.6717624925,-7931122.4786569,-7790199.124643575,-7643688.88568387,-7491409.691182384,-7333173.969919637,-7168788.491715822,-6998054.204709498,-6820766.0681394,-6636712.8805058375,-6445677.102988781,-6247434.677996494,-6041754.842716867,-5828399.937535735,-5607125.209184751,-5377678.608490251,-5139800.582557423,-4893223.861267569,-4637673.237922169,-4372865.343877219,-4098508.417026339,-3814302.0639470685,-3519937.0155599527,-3215094.876128968,-2899447.8654077034,-2572658.5537849814,-2234379.5902129635,-1884253.4227497429,-1521912.011510279,-1146976.5338335186,-759057.0814576373,-357752.3495040573,57350.682969447225,486677.081029281,930664.2908068635,1389762.485505361,1864434.920759298,2355158.299552016,2862423.146942675,3386734.1948957928,3928610.777422089,4488587.236338779,5067213.337900847,5665054.700607158,6282693.234450571,6920727.591914006,7579773.631024666,8260464.890770189,8963453.079192292,9689408.574502762,10439020.939527132,11212999.449849512,12012073.635999463,12836993.840031944,13688531.786885884,14567481.170893483,15474658.257829998,16410902.502898809],[-10915028.388691902,-10900558.205007661,-10884814.594580863,-10867749.278325686,-10849312.405594854,-10829452.50650266,-10808116.442869855,-10785249.357750727,-10760794.623504305,-10734693.788368687,-10706886.521496555,-10677310.556411156,-10645901.632835887,-10612593.436856039,-10577317.53936383,-10540003.332741242,-10500577.965731515,-10458966.27644834,-10415090.72347376,-10368871.314990543,-10320225.535894984,-10269068.272836402,-10215311.737125318,-10158865.38545085,-10099635.838350598,-10037526.796367679,-9972438.953836428,-9904269.91022614,-9832914.07898333,-9758262.593796939,-9680203.21222406,-9598620.216597274,-9513394.312145304,-9424402.522248473,-9331518.080752425,-9234610.32125864,-9133544.5633146,-9028181.995414447,-8918379.554724894,-8803989.803454656,-8684860.801764794,-8560835.9771398,-8431753.990115564,-8297448.596267556,-8157748.504362814,-8012477.230570219,-7861452.948619984,-7704488.335810514,-7541390.414748091,-7371960.390702728,-7195993.484468719,-7013278.760605723,-6823598.950939065,-6626730.27319077,-6422442.244616078,-6210497.49050769,-5990651.547431447,-5762652.661064699,-5526241.578471569,-5281151.3346930165,-5027107.033485234,-4763825.622050662,-4491015.659618596,-4208377.079691952,-3915600.9458076917,-3612369.2006410547,-3298354.4082571846,-2973219.489364531,-2636617.4493520856,-2288191.0989439413,-1927572.76726361,-1554384.0071171522,-1168235.2922859266,-768725.7066310123,-355442.62476895005,72038.61586497352,514155.05182980374,971356.4395568483,1444105.6105772816,1932878.836305678,2438166.202630032,2960471.9946024306,3500315.091437768,4058229.3721298464,4634764.131935235,5230484.51002986,5845971.928605925,6481824.543712981,7138657.708153203,7817104.446736831,8517815.944210675,9241462.046207093,9988731.77351711,10760333.850061357,11556997.244899772,12379471.728631783,13228528.444570668,14104960.495066099,15009583.543363236,15943236.431393757],[-11249182.059950333,-11235454.871260017,-11220462.65296399,-11204157.246757345,-11186488.924750283,-11167406.341826044,-11146856.48662095,-11124784.631087877,-11101134.278604016,-11075847.110582625,-11048862.931546766,-11020119.612623777,-10989553.03341484,-10957097.022196513,-10922683.294406893,-10886241.38937036,-10847698.60521131,-10806979.931906743,-10764007.982428249,-10718702.92191916,-10670982.394853264,-10620761.450120598,-10567952.463983338,-10512465.060841084,-10454206.031750321,-10393079.250631269,-10328985.588104617,-10261822.822887462,-10191485.550688067,-10117865.09052524,-10040849.388408849,-9960322.918302976,-9876166.58030362,-9788257.595952008,-9696469.400607497,-9600671.532798022,-9500729.52047169,-9396504.764059674,-9287854.416265815,-9174631.258500613,-9056683.573857516,-8933855.016551204,-8805984.477713853,-8672905.947452791,-8534448.373073786,-8390435.513363397,-8240685.7888216935,-8085012.127744669,-7923221.808039652,-7755116.294659313,-7580491.072541369,-7399135.474930007,-7210832.506958662,-7015358.664364602,-6812483.747210646,-6601970.668476181,-6383575.257381499,-6157046.057315782,-5922124.118204551,-5678542.7831928935,-5426027.469479162,-5164295.443142798,-4893055.587823814,-4612008.167069932,-4320844.580199834,-4019247.1115112714,-3706888.67263929,-3383432.5379177146,-3048532.0725269765,-2701830.4532605894,-2342960.38170404,-1971543.789634399,-1587191.5364322066,-1189503.0983062051,-778066.2490924112,-352456.73244325817,87762.07485821843,543039.5086458959,1013837.9724654779,1500633.3022229932,2003915.1406376958,2524187.3217918947,3061968.265987303,3617791.3852157593,4192205.4994949065,4785775.264373831,5399081.609876368,6032722.191185698,6687311.851380225,7363483.096526269,8061886.583441354,8783191.620474722,9528086.68160864,10297279.934254453,11091499.781084396,11911495.416249864,12758037.39637151,13631918.226672973,14533952.96264873,15464979.827659238],[-11592010.226891465,-11579039.23423663,-11564811.729756568,-11549279.677397268,-11532393.47351775,-11514101.899283243,-11494352.071681032,-11473089.393120315,-11450257.499577051,-11425798.20724352,-11399651.457640696,-11371755.261151621,-11342045.638931073,-11310456.563147804,-11276919.895511877,-11241365.324041713,-11203720.298020761,-11163909.961093605,-11121857.082452789,-11077481.986061022,-11030702.477856189,-10981433.770884218,-10929588.408302583,-10875076.184194842,-10817804.062139284,-10757676.091466658,-10694593.321148332,-10628453.711244617,-10559152.041853102,-10486579.819482505,-10410625.180788282,-10331172.793592684,-10248103.755119594,-10161295.487366512,-10070621.629537169,-9975951.927452639,-9877152.119864784,-9774083.821582135,-9666604.403323827,-9554566.868218672,-9437819.72484833,-9316206.856753096,-9189567.388297385,-9057735.546797846,-8920540.520817969,-8777806.314523516,-8629351.59798972,-8474989.553359305,-8314527.716734955,-8147767.815691713,-7974505.602296965,-7794530.681513194,-7607626.334863508,-7413569.339231163,-7212129.78066708,-7003070.8630686235,-6786148.711593118,-6561112.17067714,-6327702.596496219,-6085653.643742735,-5834691.046555601,-5574532.393445972,-5304886.896076707,-5025455.151710821,-4735928.899178326,-4435990.768189702,-4125314.021801645,-3803562.2918874957,-3470389.307397075,-3125438.6152360626,-2768343.293560777,-2398725.6572954804,-2016196.9556638822,-1620357.061536461,-1210794.1523537152,-787084.3824418075,-348791.5464565791,104533.26623341814,573352.0265072621,1058140.1300039478,1559386.7714360617,2077595.3289985359,2613283.759080343,3166985.0015884675,3739247.3961342014,4330635.109385937,4941728.573857706,5573124.938435111,6225438.530950747,6899301.333112113,7595363.468098365,8314293.7011703625,9056779.953598201,9823529.830281321,10615271.161399655,11432752.558449056,12276743.985044409,13148037.342863444,14047447.073120527,14975810.773964569],[-11943740.928634137,-11931539.45334177,-11918090.102752388,-11903344.964445665,-11887254.560431015,-11869767.799576256,-11850831.92865959,-11830392.482005918,-11808393.22966952,-11784776.124121292,-11759481.245400216,-11732446.744685953,-11703608.786249004,-11672901.487733696,-11640256.858727375,-11605604.737569816,-11568872.726353195,-11529986.124062322,-11488867.857806163,-11445438.41208588,-11399615.75604632,-11351315.268656172,-11300449.661759946,-11246928.900941348,-11190660.124142215,-11131547.557971202,-11069492.431643656,-11004392.888482973,-10936143.89492274,-10864637.146935333,-10789760.973823572,-10711400.239297237,-10629436.239766348,-10543746.599771842,-10454205.164478146,-10360681.889145875,-10263042.725507138,-10161149.504955223,-10054859.818463085,-9944026.893148046,-9828499.465382501,-9708121.650367402,-9582732.808067812,-9452167.405411478,-9316254.874655876,-9174819.467816958,-9027680.107051305,-8874650.230890261,-8715537.636210198,-8550144.315823976,-8378266.291581253,-8199693.442853548,-8014209.330283383,-7821591.014668588,-7621608.870856816,-7414026.396512469,-7198600.015620112,-6975078.87659565,-6743204.644838912,-6502711.289606934,-6253324.86504025,-5994763.285187345,-5726736.092884382,-5448944.222306026,-5161079.755036488,-4862825.669489015,-4553855.583479932,-4233833.489809178,-3902413.484631829,-3559239.4884523824,-3203944.9595354013,-2836152.5995415673,-2455474.051179994,-2061509.5876780786,-1653847.7938304693,-1232065.2384423204,-795726.1379025467,-344382.01070031524,122428.67740135267,605180.8777400889,1104363.332928151,1620478.9611185193,2154045.250571493,2705594.6648309864,3275675.058759816,3864850.1057394855,4473699.736302316,5102820.588498272,5752826.470307849,6424348.834405605,7118037.265588,7834559.98121234,8574604.344949815,9338877.39422772,10128106.38169939,10943039.331095602,11784445.607839163,12653116.5047977,13549865.843562305,14475530.59164701],[-12304605.84009486,-12293187.456568826,-12280529.954861611,-12266585.547586605,-12251304.88382289,-12234637.001579266,-12216529.278882645,-12196927.383451523,-12175775.220916165,-12153014.881544381,-12128586.585431904,-12102428.62611516,-12074477.312561449,-12044666.909493603,-12012929.576001076,-11979195.30239218,-11943391.845237598,-11905444.660555063,-11865276.835086077,-11822809.01560996,-11777959.336242246,-11730643.343662625,-11680773.920215406,-11628261.204822572,-11573012.511653032,-11514932.24648267,-11453921.820686536,-11389879.562793441,-11322700.627542498,-11252276.90236667,-11178496.91124084,-11101245.715815805,-11020404.813769445,-10935852.034297327,-10847461.430665445,-10755103.169744754,-10658643.418449061,-10557944.226988964,-10452863.40885512,-10343254.417449377,-10228966.219263136,-10109843.163519576,-9985724.848178951,-9856445.98220912,-9721836.244024787,-9581720.13599041,-9435916.834877785,-9284240.038176855,-9126497.806144463,-8962492.399474874,-8792020.112482054,-8614871.101666536,-8430829.20954915,-8239671.783641029,-8041169.4904253315,-7835086.1242130045,-7621178.410737149,-7399195.805355813,-7168880.285699269,-6929966.138638297,-6682179.741408419,-6425239.336733345,-6158854.801806165,-5882727.410942735,-5596549.591757882,-5300004.674691953,-4992766.635693071,-4674499.8319092505,-4344858.730172332,-4003487.628107302,-3650020.3676605932,-3284080.0408547036,-2905278.6875626296,-2513216.9851013795,-2107483.9294067547,-1687656.5076054744,-1253299.361718841,-803964.443312861,-339190.6588073559,141496.49476035312,638585.303768348,1152578.2221329398,1683992.2657291964,2233359.4174181707,2801227.042930983,3388158.317913901,3994732.6664031036,4621546.211031586,5269212.235278811,5938361.658067815,6629643.521023862,7343725.488740932,8081294.362359162,8843056.606827836,9629738.892193537,10442088.649265204,11280874.640040431,12146887.54326481,13040940.555515043,13963870.00819799],[-12674842.603258802,-12664221.144947775,-12652369.448057165,-12639239.851653013,-12624783.133321732,-12608948.461670402,-12591683.34745171,-12572933.593274651,-12552643.241862888,-12530754.522819042,-12507207.797854645,-12481941.504442822,-12454892.0978495,-12425993.991499519,-12395179.495630274,-12362378.754186898,-12327519.679909788,-12290527.887564039,-12251326.625261623,-12209836.703822054,-12165976.424117679,-12119661.502349975,-12070804.99319902,-12019317.210786229,-11965105.64739462,-11908074.88988052,-11848126.533718498,-11785159.094609536,-11719067.917592224,-11649745.083582066,-11577079.313276144,-11500955.868344795,-11421256.449841822,-11337859.093754472,-11250638.063617602,-11159463.74011021,-11064202.507556414,-10964716.637243848,-10860864.167472618,-10752498.780253138,-10639469.67455259,-10521621.436006308,-10398793.902993986,-10270822.02898157,-10137535.741034139,-9998759.794393176,-9854313.623010043,-9704011.185934296,-9547660.809440887,-9385065.024781324,-9216020.401447287,-9040317.375820875,-8857740.075093227,-8668066.13632042,-8471066.520493517,-8266505.32148386,-8054139.569728266,-7833719.030525096,-7604985.996775998,-7367675.076050589,-7121512.971808668,-6866218.25862404,-6601501.151266934,-6327063.267461712,-6042597.384168286,-5747787.187216286,-5442307.014097342,-5125821.589768015,-4797985.755248193,-4458444.188845839,-4106831.1198030487,-3742770.0341714695,-3365873.37270822,-2975742.220594626,-2571965.9887378737,-2154122.086472232,-1721775.585394781,-1274478.8741488978,-811771.3038710319,-333178.82409287244,161786.39114950597,673626.3272943273,1202857.5235272609,1750011.477659192,2315635.0618572086,2900290.949534811,3504558.05366813,4129031.9768414274,4774325.473332815,5441068.923543621,6129910.821087185,6841518.2728812955,7576577.512549054,8335794.427501727,9119895.100043435,9929626.362849396,10765756.369201817,11629075.178355537,12520395.356423527,13440552.593174685],[-13054694.48318931,-13044884.079401718,-13033852.44199639,-13021552.037956575,-13007933.774860978,-12992946.953420239,-12976539.218638867,-12958656.509563582,-12939243.00757991,-12918241.083215714,-12895591.241410524,-12871232.065208986,-12845100.157833168,-12817130.083090585,-12787254.304070545,-12755403.120082937,-12721504.601789959,-12685484.524480663,-12647266.299439225,-12606770.903352015,-12563916.805701043,-12518619.894088274,-12470793.397434823,-12420347.806993872,-12367190.795122117,-12311227.131743804,-12252358.59844853,-12190483.900153812,-12125498.574271243,-12057294.897301871,-11985761.788797947,-11910784.712612994,-11832245.575371252,-11750022.622077761,-11663990.328794312,-11574019.29229846,-11479976.116647994,-11381723.296564814,-11279119.097549489,-11172017.432646807,-11060267.735760406,-10943714.83143454,-10822198.801000921,-10695554.844993386,-10563613.141734496,-10426198.701987682,-10283131.219567794,-10134224.917806914,-9979288.391761126,-9818124.446042571,-9650529.92816518,-9476295.557279192,-9295205.748174632,-9107038.430425059,-8911564.862545239,-8708549.441027472,-8497749.50411829,-8278915.130208302,-8051788.930669175,-7816105.837015675,-7571592.882226534,-7317968.976069171,-7054944.674284717,-6782221.941450136,-6499493.907366306,-6206444.616800316,-5902748.77238846,-5588071.470551562,-5262067.930207729,-4924383.214114368,-4574651.942632221,-4212497.999722611,-3837534.2309664376,-3449362.133408252,-3047571.536986202,-2631740.2773631066,-2201433.8598955236,-1756205.114551846,-1295593.8414969333,-819126.4471347667,-326315.57035985216,183340.30127527937,710357.225685548,1255266.2109797,1818613.631038174,2400961.652245976,3002888.671650015,3624989.766841583,4267877.157876942,4932180.681538228,5618548.278250091,6327646.49199754,7060160.983548716,7816797.057356179,8598280.20247671,9405356.647861514,10238793.932400096,11099381.490090974,11987931.250727538,12905278.256494187],[-13444410.4571273,-13435425.574096046,-13425228.591743134,-13413772.106511075,-13401007.157534856,-13386883.17921497,-13371347.952416765,-13354347.554256815,-13335826.306438187,-13315726.722093195,-13293989.451093126,-13270553.223782424,-13245354.793092515,-13218328.874992331,-13189408.087227687,-13158522.886303969,-13125601.502662804,-13090569.874001935,-13053351.576689785,-13013867.755220162,-12972037.049653398,-12927775.520989785,-12880996.574418765,-12831610.880382562,-12779526.293399077,-12724647.768579125,-12666877.275777511,-12606113.71131033,-12542252.807176657,-12475187.03771085,-12404805.523601675,-12330993.933201343,-12253634.381054686,-12172605.32357026,-12087781.451758303,-11999033.58095296,-11906228.53744123,-11809229.041912079,-11707893.58963828,-11602076.32730931,-11491626.926415987,-11376390.453102041,-11256207.234383194,-11130912.720634911,-11000337.344253473,-10864306.374383977,-10722639.76760811,-10575152.014488945,-10421651.981857369,-10261942.750726312,-10095821.449719297,-9923079.083889723,-9743500.35881081,-9556863.499806575,-9362940.066199834,-9161494.760438832,-8952285.231967464,-8735061.875709072,-8509567.625000166,-8275537.738850085,-8032699.58336214,-7780772.407159787,-7519467.110675519,-7248486.009118179,-6967522.58896878,-6676261.25783233,-6374377.08745203,-6061535.549738452,-5737392.2455979735,-5401592.626392234,-5053771.707822841,-4693553.776050188,-4320552.085837286,-3934368.5505208,-3534593.423569724,-3120804.9715484567,-2692569.1382193863,-2249439.199598115,-1790955.409676589,-1316644.6366074085,-826019.9890997969,-318580.43273371086,206189.604015816,748820.6343529299,1309858.528583113,1889864.9406883828,2489417.7463230416,3109111.4925328195,3749557.8595076203,4411386.134672519,5095243.69943108,5801796.528906614,6531729.704984792,7285747.943031993,8064576.1326284595,8868959.89266811,9699666.14120926,10557483.680447403,11443223.797200982,12357720.879302181],[-13844248.649678718,-13836103.92924817,-13826756.37216897,-13816158.705818526,-13804262.10238361,-13791016.131470202,-13776368.711340554,-13760266.058737777,-13742652.63726005,-13723471.104243403,-13702662.25611148,-13680164.97215109,-13655916.156667972,-13629850.679479975,-13601901.314699855,-13571998.677762484,-13540071.160645975,-13506044.865238331,-13469843.53479805,-13431388.483456634,-13390598.523708468,-13347389.89183311,-13301676.171195127,-13253368.21335875,-13202374.056963,-13148598.84429184,-13091944.735479366,-13032310.820282336,-12969593.027357507,-12903684.030971378,-12834473.155077389,-12761846.274683882,-12685685.714443984,-12605870.144387735,-12522274.472722847,-12434769.735620344,-12343222.983908562,-12247497.166588236,-12147451.011081593,-12042938.900133625,-11933810.74526671,-11819911.856703054,-11701082.809656171,-11577159.306892335,-11447972.037466515,-11313346.531527227,-11173103.01108167,-11027056.236619981,-10875015.349482635,-10716783.709856369,-10552158.730286973,-10380931.704583641,-10202887.631995687,-10017805.036532298,-9825455.781300263,-9625604.87772262,-9418010.289502371,-9202422.731201198,-8978585.461270494,-8746234.069409499,-8505096.258087158,-8254891.618070614,-7995331.397818448,-7726118.266554473,-7446946.0708715655,-7157499.584693529,-6857454.252402099,-6546475.924980059,-6224220.5889562145,-5890334.087983575,-5544451.836844783,-5186198.527694032,-4815187.828326397,-4431022.072276175,-4033291.940505985,-3621576.1345018893,-3195441.0405115485,-2754440.3847359195,-2298114.879193064,-1825991.8580448255,-1337584.90413782,-832393.4654655196,-309902.4613433145,230418.12201235816,789113.6467499398,1366745.2497988753,1963890.280702956,2581142.7511745803,3219113.7966792323,3878432.1503549814,4559744.629581768,5263716.63554417,5991032.666093681,6742396.842282295,7518533.448907904,8320187.489422701,9148125.255589098,10003134.912254132,10886027.097632766,11797635.539493129],[-14254472.198079336,-14247182.562270427,-14238699.48289178,-14228975.819751006,-14217962.879601847,-14205610.368791452,-14191866.344535064,-14176677.164779523,-14159987.436616678,-14141739.963205978,-14121875.689165367,-14100333.644388046,-14077050.886240551,-14051962.440099137,-14025001.23817608,-13996098.056591768,-13965181.450641152,-13932177.68820656,-13897010.681265563,-13859601.915441304,-13819870.377541354,-13777732.481029706,-13733101.989376973,-13685889.93722724,-13636004.549325079,-13583351.157139353,-13527832.113123197,-13469346.702541512,-13407791.052805215,-13343058.040238079,-13275037.194212807,-13203614.59857868,-13128672.790311726,-13050090.655309299,-12967743.321253255,-12881502.04746026,-12791234.11164078,-12696802.693480756,-12598066.754958386,-12494880.917314742,-12387095.334578093,-12274555.56355841,-12157102.430211416,-12034571.892274465,-11906794.89807776,-11773597.241426585,-11634799.412444655,-11490216.44427791,-11339657.755543677,-11182926.988409307,-11019821.842189187,-10850133.90233503,-10673648.464699956,-10490144.3549476,-10299393.7429805,-10101161.952250985,-9895207.263819136,-9681280.7150275,-9459125.892629346,-9228478.720246753,-8989067.239993416,-8740611.388106406,-8482822.764444327,-8215404.395668304,-7938050.491954554,-7650446.197067615,-7352267.33159983,-7043180.129230261,-6722840.965786245,-6390896.080941621,-6046981.292343464,-5690721.701978046,-5321731.394566093,-4939613.127790146,-4543958.014114015,-4134345.1940118037,-3710341.5003412403,-3271501.113674354,-2817365.208301656,-2347461.5887025706,-1861304.316232495,-1358393.3257339224,-838214.0318627991,-300236.9248224683,256082.844745446,831305.8920162506,1426009.0356529877,2040785.7471941262,2676246.612468876,3333019.805344183,4011751.57411832,4713106.740904927,5437769.21431303,6186442.515794739,6959850.320001952,7758737.009502206,8583868.244238134,9436031.546102475,10316036.899017766,11224717.364913974],[-14675351.02365956,-14668931.680220738,-14661328.418760166,-14652494.233030673,-14642380.565888353,-14630937.261977227,-14618112.519042395,-14603852.837832334,-14588102.970551714,-14570805.86782476,-14551902.62412689,-14531332.421643432,-14509032.472510265,-14484937.959393585,-14458981.974360337,-14431095.455995213,-14401207.124713028,-14369243.416218035,-14335128.413059294,-14298783.774229296,-14260128.662752038,-14219079.67120528,-14175550.745121732,-14129453.10420739,-14080695.161322232,-14029182.43915765,-13974817.484550849,-13917499.78036879,-13857125.654899053,-13793588.18867486,-13726777.11866981,-13656578.739785835,-13582875.803564562,-13505547.41404417,-13424468.92068595,-13339511.808289403,-13250543.583816832,-13157427.660042156,-13060023.235936066,-12958185.173705816,-12851763.872390639,-12740605.13792814,-12624550.049591841,-12503434.822701793,-12377090.667512082,-12245343.644170137,-12108014.51363961,-11964918.58448427,-11815865.555399181,-11660659.353372592,-11499097.967368053,-11330973.277401019,-11156070.878891462,-10974169.902162176,-10785042.826958833,-10588455.291854007,-10384165.898399644,-10171926.009898633,-9951479.544631245,-9722562.763413118,-9484904.051320504,-9238223.693425551,-8982233.644400356,-8716637.291805668,-8441129.212913312,-8155394.924890861,-7859110.628155546,-7551942.9427480735,-7233548.637513544,-6903574.351918776,-6561656.310302217,-6207420.028364476,-5840480.011690568,-5460439.44610646,-5066889.879631005,-4659410.895838443,-4237569.778368894,-3800921.166398082,-3349006.7007835023,-2881354.6606792994,-2397479.590370193,-1896881.9160328284,-1379047.552214533,-843447.4977227561,-289537.4206744358,283242.76759819686,875469.3500540182,1487735.2531632185,2120650.5081525333,2774842.724578604,3450957.5765445083,4149659.301904142,4871631.214758366,5617576.231616523,6388217.411562294,7184298.510776043,8006584.551796909,8855862.407896642,9732941.402954526,10638653.927226651],[-15107161.922479827,-15101628.375601308,-15094920.570966437,-15086991.63781272,-15077793.15665869,-15067275.112026393,-15055385.843792945,-15042071.997132089,-15027278.471007258,-15010948.365175068,-14993022.925658768,-14973441.48864898,-14952141.422787238,-14929058.069789575,-14904124.683361346,-14877272.36635954,-14848430.006151408,-14817524.208120609,-14784479.22727058,-14749216.897871807,-14711656.561099391,-14671714.990605893,-14629306.31597405,-14584341.943987569,-14536730.477664564,-14486377.632990047,-14433186.153285183,-14377055.721147608,-14317882.867899682,-14255560.880471792,-14189979.705656959,-14121025.85165846,-14048582.286863988,-13972528.335764332,-13892739.571944337,-13809087.708063023,-13721440.482744498,-13629661.54429435,-13533610.331153313,-13433141.949007344,-13328107.044454414,-13218351.675143093,-13103717.17628458,-12984040.023438092,-12859151.691475596,-12728878.509618996,-12593041.512442205,-12451456.286736751,-12303932.814124854,-12150275.309305627,-11990282.053822743,-11823745.225228157,-11650450.721523201,-11470177.980747756,-11282699.795591874,-11087782.122893961,-10885183.887888405,-10674656.78307452,-10455945.061541932,-10228785.324629629,-9992906.303753443,-9748028.636246558,-9493864.635069551,-9230118.052208027,-8956483.83560507,-8672647.879459132,-8378286.767691651,-8073067.510438688,-7756647.2733497955,-7428673.099526923,-7088781.623897739,-6736598.779831309,-6371739.497788735,-5993807.395809852,-5602394.461597774,-5197080.726016674,-4777433.927739795,-4343009.168859728,-3893348.5611768253,-3427980.8629600517,-2946421.105928734,-2448170.212164931,-1932714.6007457376,-1399525.7837894857,-848059.9516640529,-277757.5470556058,311957.1723728329,921676.5830330439,1552010.156485241,2203584.9328367747,2877046.0067806877,3573057.0266187675,4292300.706573218,5035479.352759875,5803315.403162956,6596551.981961869,7415953.468594898,8262306.081930682,9136418.479936797,10039122.37523944],[-15550187.051721865,-15545555.217363052,-15539758.925777845,-15532751.443347245,-15524484.489934584,-15514908.191646332,-15503971.032222386,-15491619.803017585,-15477799.551534552,-15462453.528468864,-15445523.1332241,-15426947.857854873,-15406665.229394333,-15384610.750521211,-15360717.838520199,-15334917.762489809,-15307139.578747343,-15277310.06438306,-15245353.64891122,-15211192.343967069,-15174745.670994362,-15135930.586869119,-15094661.40740437,-15050849.72867374,-15004404.346099071,-14955231.17123732,-14903233.146206029,-14848310.155680377,-14790358.936399348,-14729272.984108016,-14664942.457871893,-14597254.081686717,-14526091.04331355,-14451332.890261715,-14372855.422844429,-14290530.584224092,-14204226.347370552,-14113806.598845642,-14019131.019326454,-13920054.96078585,-13816429.32023133,-13708100.409917295,-13594909.82393132,-13476694.30105612,-13353285.58381119,-13224510.273569325,-13090189.681638982,-12950139.676211396,-12804170.525057325,-12652086.733858213,-12493686.880059732,-12328763.442124084,-12157102.624060648,-11978484.175106075,-11792681.204429476,-11599459.990724573,-11398579.786554527,-11189792.617318619,-10972843.07467766,-10747468.10431523,-10513396.787868645,-10270350.11887515,-10018040.772589752,-9756172.869492223,-9484441.73233154,-9202533.63653696,-8910125.553801792,-8606884.888693038,-8292469.208070025,-7966525.963145979,-7628692.203985672,-7278594.286248047,-6915847.569966566,-6540056.110166591,-6150812.339085072,-5747696.739803854,-5330277.511037733,-4898110.222885482,-4450737.463263448,-3987688.474812273,-3508478.7820294052,-3012609.808333017,-2499568.48285025,-1968826.8366207965,-1419841.5879671574,-852053.7167266123,-264888.0270773098,342247.3013433553,969961.1743389592,1618880.056376323,2289648.4564441293,2982929.4268930033,3699405.075560581,4439777.0915546715,5204767.28503437,5995118.141340174,6811593.389856614,7654978.587979101,8526081.72057471,9425733.815329198],[-16004719.670546424,-16001005.648146998,-15996137.107922334,-15990067.454973377,-15982748.55010534,-15974130.662627703,-15964162.421783295,-15952790.76676963,-15939960.895313088,-15925616.210754775,-15909698.267607976,-15892146.71554435,-15872899.241764642,-15851891.511710797,-15829057.1080712,-15804327.468034718,-15777631.818743017,-15748897.110891972,-15718047.950432029,-15685006.528314237,-15649692.54822858,-15612023.152279092,-15571912.84454086,-15529273.412436986,-15484013.845880486,-15436040.254116438,-15385255.78020367,-15331560.51306929,-15274851.397072941,-15215022.139008898,-15151963.112480564,-15085561.259572055,-15015699.989745853,-14942259.0758901,-14865114.547438562,-14784138.580482962,-14699199.384798205,-14610161.087695725,-14516883.614616776,-14419222.5663842,-14317029.093013909,-14210149.764000922,-14098426.43498086,-13981696.110668667,-13859790.80397835,-13732537.39121865,-13599757.463256598,-13461267.172547223,-13316877.075913778,-13166391.972964406,-13009610.740033338,-12846326.159521323,-12676324.74451646,-12499386.55856675,-12315285.030478371,-12123786.76400364,-11924651.342282645,-11717631.126909051,-11502471.05145611,-11278908.40934048,-11046672.635857765,-10805485.084235154,-10555058.795557404,-10295098.262384495,-10025299.185908083,-9745348.226477496,-9454922.74730003,-9153690.551168926,-8841309.610003237,-8517427.787032764,-8181682.551420666,-7833700.685134456,-7473097.981855679,-7099478.937730785,-6712436.433724698,-6311551.409392696,-5896392.527807664,-5466515.831454046,-5021464.388806559,-4560767.931385331,-4083942.4810380004,-3590489.967158191,-3079897.8336295336,-2551638.6351882406,-2005169.622956004,-1439932.318836525,-855352.0785103776,-250837.64272394404,374219.32343561575,1020444.7035853937,1688482.4157242402,2378994.910888493,3092663.6850893684,3830189.804904796,4592294.44706592,5379719.452388667,6193227.894434407,7033604.663271103,7901657.0647243485,8798215.435510986],[-16471058.603649195,-16468278.801717784,-16464354.56243915,-16459239.43123123,-16452885.411459878,-16445242.917277267,-16436260.72509099,-16425885.923625333,-16414063.862536531,-16400738.099540453,-16385850.346012391,-16369340.411016626,-16351146.143721264,-16331203.374154875,-16309445.852257863,-16285805.185182584,-16260210.772793224,-16232589.741315408,-16202866.875085423,-16170964.5463467,-16136802.643038405,-16100298.494522719,-16061366.795194708,-16019919.5259127,-15975865.873195082,-15929112.146118011,-15879561.690853894,-15827114.802783687,-15771668.636119857,-15713117.110968385,-15651350.817764306,-15586256.91900507,-15517719.048212124,-15445617.206041632,-15369827.653470673,-15290222.801976087,-15206671.100627707,-15119036.920010995,-15027180.432890562,-14930957.491534092,-14830219.501596518,-14724813.292480845,-14614580.984075272,-14499359.84976913,-14378982.175651185,-14253275.115785321,-14122060.543455884,-13985154.89827994,-13842369.029072603,-13693508.032349452,-13538371.086355278,-13376751.280493997,-13208435.44004062,-13033203.946005978,-12850830.550029492,-12661082.184163405,-12463718.76541181,-12258492.994896237,-12045150.151483303,-11823427.879751554,-11593055.972132389,-11353756.145069644,-11105241.809054947,-10847217.832356425,-10579380.29828842,-10301416.25585246,-10013003.463555211,-9713810.126255974,-9403494.62482854,-9081705.238469504,-8748079.859447852,-8402245.700104099,-8043818.991891541,-7672404.676259831,-7287596.087145045,-6888974.62487971,-6476109.421260912,-6048556.995588068,-5605860.901387583,-5147551.363617074,-4673144.90609929,-4182143.968894504,-3674036.515402306,-3148295.6288846023,-2604379.0981610306,-2041728.9921723045,-1459771.2231440172,-857915.0980481617,-235552.85805371776,407940.79433951527,1073209.1807966642,1760914.1456173211,2471736.5674843937,3206376.8848633952,3965555.635392878,4750014.009616494,5560514.419439983,6397841.081685603,7262800.617131941,8156222.665432803],[-16949510.36559132,-16947681.50783686,-16944718.43653263,-16940574.839032244,-16935202.862902,-16928553.068798643,-16920574.381978102,-16911214.042397726,-16900417.553372495,-16888128.62874487,-16874289.13852738,-16858839.052976117,-16841716.385049477,-16822857.131210245,-16802195.210522998,-16779662.402001154,-16755188.28015432,-16728700.148686677,-16700122.97229566,-16669379.30651851,-16636389.225572797,-16601070.24813511,-16563337.261004642,-16523102.440587388,-16480275.172148313,-16434761.96676505,-16386466.375923414,-16335288.903688017,-16281126.91638456,-16223874.549721982,-16163422.613289444,-16099658.492352474,-16032466.046877755,-15961725.507709334,-15887313.369820394,-15809102.282559482,-15726960.936812317,-15640753.948993843,-15550341.741783248,-15455580.42151982,-15356321.65216105,-15252412.525717944,-15143695.429069055,-15030007.907053739,-14911182.521750066,-14787046.707830725,-14657422.623890825,-14522126.999643525,-14380970.978870416,-14233759.958011072,-14080293.420279866,-13920364.765185583,-13753761.133335406,-13580263.226392597,-13399645.12206445,-13211674.08398288,-13016110.366342582,-12812707.013166256,-12601209.652034791,-12381356.282158105,-12152877.056622032,-11915494.058655702,-11668921.071776846,-11412863.343632413,-11147017.343382258,-10871070.512456186,-10584701.00849015,-10287577.442294024,-9979358.60763609,-9659693.20367647,-9328219.549843494,-8984565.292962693,-8628347.106429577,-8259170.381227948,-7876628.9085565135,-7480304.553878356,-7069766.922130961,-6644573.013908278,-6204266.8723330945,-5748379.220410619,-5276427.088615295,-4787913.43241892,-4282326.739550427,-3759140.6266803183,-3217813.425280463,-2657787.7563547716,-2078490.0937730782,-1479330.315906763,-859701.2452554218,-218978.17576122656,443481.6125035882,1128339.3516196087,1836275.2973786741,2567989.254489865,3324201.115772415,4105651.415686488,4913101.898586191,5747336.102065057,6609159.955783643,7499402.396172352],[-17440389.35569731,-17439528.487283707,-17437543.77471777,-17434389.048928846,-17430016.60334548,-17424377.14681281,-17417419.755141363,-17409091.821250163,-17399339.003864557,-17388105.17472858,-17375332.36429092,-17360960.70582184,-17344928.37791698,-17327171.545345113,-17307624.298191685,-17286218.589253284,-17262884.16963293,-17237548.522487443,-17210136.79487615,-17180571.727657914,-17148773.58338334,-17114660.07212662,-17078146.27520164,-17039144.566701964,-16997564.53280752,-16953312.88879538,-16906293.39369279,-16856406.762506295,-16803550.575963665,-16747619.187697064,-16688503.628801731,-16626091.509695426,-16560266.9192069,-16490910.32081777,-16417898.445981063,-16341104.184434624,-16260396.471432405,-16175640.171806518,-16086695.960773839,-15993420.201404609,-15895664.818655033,-15793277.16987789,-15686099.911713334,-15573970.863260955,-15456722.865436755,-15334183.636411047,-15206175.623018313,-15072515.848037379,-14933015.753227746,-14787481.03800591,-14635711.493651623,-14477500.832918148,-14312636.514927754,-14140899.565224245,-13962064.390856067,-13775898.590354266,-13582162.75847013,-13380610.285541065,-13170987.151324263,-12953031.713171592,-12726474.48838358,-12491037.930585478,-12246436.199983163,-11992374.927316198,-11728550.971356831,-11454652.169783216,-11170357.083235385,-10875334.732403766,-10569244.327937108,-10251734.99300173,-9922445.478286318,-9581003.869260553,-9227027.285481244,-8860121.571745902,-8479880.980856717,-8085887.8478106,-7677712.25515217,-7254911.689301018,-6817030.68757239,-6363600.475681823,-5894138.595485423,-5408148.522664756,-4905119.274145905,-4384525.004946366,-3845824.594200138,-3288461.220056668,-2711861.92318685,-2115437.1585925967,-1498580.335411802,-860667.3444139212,-201056.07287200168,480914.0935322754,1185922.7820757888,1914669.1580046415,2667872.4635285586,3446272.5711498037,4250630.551709704,5081729.257524811,5940373.921000287,6827392.769115761],[-17944017.443613216,-17944141.975511275,-17943153.18170455,-17941005.038392868,-17937649.986584764,-17933038.885054577,-17927120.96193165,-17919843.764884975,-17911153.109862804,-17900993.02834798,-17889305.71308725,-17876031.46225249,-17861108.621990014,-17844473.52731378,-17826060.44129611,-17805801.49250954,-17783626.610670734,-17759463.460437223,-17733237.373306192,-17704871.27756319,-17674285.626225766,-17641398.322928805,-17606124.645694777,-17568377.16852832,-17528065.680779167,-17485097.104209952,-17439375.407707304,-17390801.51957003,-17339273.237311237,-17284685.134902865,-17226928.467396796,-17165891.072847687,-17101457.271466307,-17033507.76192741,-16961919.51475484,-16886565.662703946,-16807315.38806194,-16724033.806781013,-16636581.849357044,-16544816.13837172,-16448588.862599421,-16347747.647594701,-16242135.42266023,-16131590.284097936,-16015945.354646377,-15895028.639000392,-15768662.875304494,-15636665.382518144,-15498847.903538002,-15355016.443962917,-15204971.106389282,-15048505.920113083,-14885408.66611815,-14715460.697223376,-14538436.753262242,-14354104.77115908,-14162225.689766545,-13962553.24933359,-13754833.785442488,-13538806.017289625,-13314200.830146976,-13080741.05184754,-12838141.223152904,-12586107.36182003,-12324336.720215585,-12052517.536307435,-11770328.777840124,-11477439.879545808,-11173510.473176703,-10858190.11019088,-10531117.97688606,-10191922.601789974,-9840221.55510021,-9475621.139973715,-9097716.075430386,-8706089.170683783,-8300310.9906382635,-7879939.512363471,-7444519.772263315,-6993583.5037337355,-6526648.765057154,-6043219.557245735,-5542785.431620318,-5024821.086821843,-4488785.955002509,-3934123.7768946253,-3360262.1654893756,-2766612.1580226086,-2152567.755958982,-1517505.4526702128,-860783.7484930009,-181742.6528235525,520296.82705679536,1246033.210795544,1996185.0837634578,2771491.650179893,3572713.3009514585,4400632.196596868,5256052.865645029,6139802.818901718],[-18460724.713221267,-18461852.433352973,-18461877.498904817,-18460754.03310096,-18458434.626307897,-18454870.28903137,-18450010.403547112,-18443802.674126495,-18436193.075818907,-18427125.80175031,-18416543.20889616,-18404385.762287654,-18390591.97760676,-18375098.362125307,-18357839.353943326,-18338747.25947868,-18317752.18916042,-18294781.991274826,-18269762.1839152,-18242615.884981945,-18213263.740179464,-18181623.848954406,-18147611.68832054,-18111140.034509137,-18072118.88238818,-18030455.362587854,-17986053.656270105,-17938814.90747662,-17888637.132991705,-17835415.129648566,-17779040.379013486,-17719400.94937265,-17656381.394951195,-17589862.65228697,-17519721.933683988,-17445832.61766316,-17368064.136333406,-17286281.85959588,-17200346.976095706,-17110116.370838005,-17015442.49937028,-16916173.258447144,-16812151.853075966,-16703216.659847934,-16589201.086456683,-16469933.427300489,-16345236.71505978,-16214928.56814809,-16078821.03392147,-15936720.427532725,-15788427.16631695,-15633735.599585503,-15472433.833708657,-15304303.552357905,-15129119.831782853,-14946650.950986853,-14756658.196665116,-14558895.662775472,-14353110.044580257,-14139040.427033408,-13916418.067350376,-13684966.171604782,-13444399.665207596,-13194424.95708924,-12934739.697430179,-12665032.52877178,-12384982.830312366,-12094260.455241736,-11792525.460899137,-11479427.831586849,-11154607.19383416,-10817692.523920558,-10468301.847450364,-10106041.930780213,-9730507.964062002,-9341283.235717122,-8937938.798078287,-8520033.12401206,-8087111.754238892,-7638706.9351439625,-7174337.246828143,-6693507.221110277,-6195706.949268665,-5680411.679216851,-5147081.401863702,-4595160.426353063,-4024076.943917092,-3433242.5800395906,-2822051.9346214384,-2189882.109842401,-1536092.2254074737,-860022.9208319932,-160995.84446210414,561686.871140942,1308743.1477931812,2080911.5146078914,2878951.6756248027,3703645.0925214365,4555795.582794018,5436229.933802858],[-18990851.73888034,-18993000.69563657,-18994057.82161465,-18993977.388769858,-18992712.13855595,-18990213.23496223,-18986430.21618568,-18981310.944898747,-18974801.557074133,-18966846.409327205,-18957388.02473305,-18946367.037077848,-18933722.133499693,-18919389.995474752,-18903305.238102928,-18885400.347646184,-18865605.617270634,-18843849.080943797,-18820056.445434704,-18794151.020366292,-18766053.64626547,-18735682.620555118,-18702953.6214341,-18667779.629583262,-18630070.847642154,-18589734.617391918,-18546675.33458437,-18500794.361349713,-18451989.936120965,-18400157.081002362,-18345187.50651727,-18286969.5136591,-18225387.89317666,-18160323.822014883,-18091654.75683606,-18019254.3245406,-17942992.209708408,-17862734.038875397,-17778341.26155846,-17689671.027946018,-17596576.063157115,-17498904.537982386,-17396499.93600969,-17289200.917034656,-17176841.17666155,-17059249.301988497,-16936248.623269744,-16807657.061452918,-16673286.971476378,-16532944.981213022,-16386431.82594682,-16233542.178259667,-16074064.473208515,-15907780.72866382,-15734466.360684684,-15553889.993794123,-15365813.266018923,-15169990.628563505,-14966169.139957761,-14754088.25455113,-14533479.605191898,-14304066.779934976,-14065565.092634918,-13817681.34724282,-13560113.595655032,-13292550.888943005,-13014673.021772167,-12726150.269859731,-12426643.120259646,-12115801.99430481,-11793266.963002086,-11458667.45468888,-11111621.954744197,-10751737.697153974,-10378610.34769594,-9991823.678556364,-9590949.234119378,-9175545.987737462,-8745159.989203788,-8299324.002717335,-7837557.135091271,-7359364.453915391,-6864236.595461018,-6351649.36202291,-5821063.308448382,-5271923.3175497465,-4703658.1641327925,-4115680.067339599,-3507384.230995126,-2878148.371655684,-2227332.234042801,-1554277.093521744,-858305.2453177348,-138719.48009936512,605197.454410173,1374183.4061464071,2168997.386121407,2990420.150921203,3839254.8006675616,4716327.392840706],[-19534746.506654438,-19537935.09618154,-19540042.833847273,-19541024.142067917,-19540831.915136658,-19539417.472301893,-19536730.509480666,-19532719.049568694,-19527329.391308684,-19520506.05667602,-19512191.736740988,-19502327.235965338,-19490851.414889436,-19477701.131165624,-19462811.17889154,-19446114.226197958,-19427540.751040045,-19407018.97514484,-19384474.7960643,-19359831.71727869,-19333010.776300818,-19303930.470722392,-19272506.682149097,-19238652.597963393,-19202278.63085822,-19163292.33607868,-19121598.326310255,-19077098.18414767,-19029690.372080304,-18979270.139923993,-18925729.429632388,-18868956.777413584,-18808837.213081196,-18745252.156562805,-18678079.311490074,-18607192.55578933,-18532461.829193994,-18453753.017594237,-18370927.834135264,-18283843.696983293,-18192353.60366121,-18096306.00186778,-17995544.65668214,-17889908.514055997,-17779231.560496066,-17663342.678833522,-17542065.499971658,-17415218.250509955,-17282613.59613035,-17144058.480630383,-16999353.960492186,-16848295.034862857,-16690670.470826356,-16526262.623839095,-16354847.25320387,-16176193.332445497,-15990062.854452938,-15796210.631257955,-15594384.088288074,-15384323.052969726,-15165759.537517253,-14938417.515752487,-14702012.693812246,-14456252.274561355,-14200834.715559762,-13935449.480413143,-13659776.783313753,-13373487.326624334,-13076242.03128957,-12767691.75990811,-12447477.03225974,-12115227.73309647,-11770562.811990336,-11413089.975038134,-11042405.368188385,-10658093.252002742,-10259725.667592626,-9846862.093539357,-9419049.093519583,-8975819.954425842,-8516694.314733297,-8041177.782823697,-7548761.545054164,-7038921.96326701,-6511120.161488529,-5964801.601515882,-5399395.647121478,-4814315.116576411,-4208955.823179707,-3582696.1034942605,-2934896.3329701573,-2264898.428617306,-1572025.338418521,-855580.5171140134,-114847.38801699132,650911.2094911188,1442453.5871666372,2260559.790184796,3106032.1948813647,3979696.1223517433]],"type":"heatmap","xaxis":"x","yaxis":"y","hovertemplate":"Home Price Growth: %{x}\u003cbr\u003eInvestment Growth: %{y}\u003cbr\u003eOpportunity: %{z}\u003cextra\u003e\u003c\u002fextra\u003e"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Home Price Growth"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"autorange":"reversed","title":{"text":"Investment Growth"}},"coloraxis":{"colorbar":{"title":{"text":"Opportunity"}},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]},"title":{"text":"Opportunity Surface"}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

## Simulate
We can also simulate market conditions.

In [15]:
n = 10_000

In [16]:
simulated_investment_rate = np.random.beta(10, 5, n) * (0.1 - 0.03) + 0.03
render_plotly_html(px.histogram(simulated_investment_rate, title="Investment Rate"))

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="8f9cf0ba-fc3f-4d8f-aabd-f3cb111660c7" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("8f9cf0ba-fc3f-4d8f-aabd-f3cb111660c7")) {                    Plotly.newPlot(                        "8f9cf0ba-fc3f-4d8f-aabd-f3cb111660c7",                        [{"alignmentgroup":"True","bingroup":"x","hovertemplate":"variable=0\u003cbr\u003evalue=%{x}\u003cbr\u003ecount=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"0","marker":{"color":"#636efa","pattern":{"shape":""}},"name":"0","offsetgroup":"0","orientation":"v","showlegend":true,"x":[0.06325458050401336,0.08300393123471894,0.08150155398544813,0.09514267587764112,0.08406191386668893,0.06542924708415321,0.06504825596752967,0.08381872419591484,0.08444249660933681,0.07719584969196988,0.07504348237858277,0.09009086449912351,0.08327313729098618,0.07481106091068494,0.05886207622022542,0.08585863544795536,0.07216081911541827,0.08473338810122757,0.07815195724643398,0.09463393882588936,0.0847980466972025,0.07517553570250096,0.08215629964596144,0.06460395934280122,0.07772485269429905,0.074623068755189,0.08825890468116719,0.08597196105995415,0.07925521876906694,0.09116701046561598,0.08932107047597238,0.07365300164442412,0.08041375266001091,0.08304496065027868,0.07292242351336314,0.06426964542463993,0.09029040127110731,0.08163173870967282,0.08433110203309267,0.06476603393867303,0.06200028955188079,0.07031320842889563,0.07684631660478504,0.07485458805929981,0.0790635972622626,0.07859066928960515,0.06470052724672362,0.0886789040377125,0.0681854319919786,0.07953744974570817,0.07561143025442402,0.0761097831745816,0.07932853331058941,0.08256619762970376,0.08514993272043789,0.08021661484835865,0.07331419620631113,0.0783464063372374,0.0837219540118643,0.09168378917894654,0.06372207299154733,0.08147554797547893,0.06730887226311427,0.07392990744045452,0.06782548078174946,0.0802702966276847,0.08001628783090808,0.07550333262294445,0.08088660189759386,0.06550296449619322,0.07104785655243877,0.07964507437773247,0.0640732454089073,0.09324038362856617,0.09060464539287394,0.07249435873154814,0.07811957115404185,0.09205358576620942,0.06559747798379717,0.08422653432391833,0.07316102070563119,0.07390647231655365,0.07105489403333823,0.07606493329327219,0.06538083242054067,0.07327629175073222,0.0917109398929753,0.08681476717643899,0.08057039248836656,0.06991135514327869,0.0727983953177423,0.08036487081506474,0.07037547192303495,0.07863351216101486,0.09061098705270132,0.0759344485698663,0.07375150274043009,0.08612501006706955,0.06205233055192584,0.07697245787411179,0.0862454570610552,0.07545244585541641,0.06866257282108537,0.07778203602888095,0.08471010971161932,0.05809918342756325,0.07912129521453365,0.07643699013934839,0.0826994824014729,0.07390999457536426,0.08014611660739746,0.07196636037813622,0.07681194234002658,0.07626251600135232,0.08617483503076845,0.07104659342220146,0.07252891326890237,0.07437780498290347,0.06928629099885178,0.06674211598636501,0.0731788587037343,0.07115099029883981,0.07733038414112928,0.08441374103195437,0.0779408390176618,0.08760060185075216,0.06920696080266706,0.07697534995419056,0.07129644554415746,0.08526626716030006,0.0797640481090633,0.07731560514384142,0.07557719714589523,0.07807268284411956,0.08253754271389259,0.08568034382947806,0.0805743324039799,0.07562760548955347,0.07088615347031413,0.08217855891238182,0.07399140525592399,0.07414905088540202,0.07265013871703446,0.07456334652253452,0.08069404754681538,0.07955334534313363,0.08423815928801726,0.08732151971366192,0.06900493511078803,0.07458345209916693,0.08420071479995236,0.0777947822214381,0.0849576239300235,0.06724218934276688,0.07873061713479379,0.0793278988246881,0.06866574235883623,0.057697160599532935,0.07150451554340542,0.07965567821422453,0.082290731419604,0.06849927039989356,0.07293556972162202,0.0713957059680437,0.06689164469088635,0.07862133900491405,0.06555465913091046,0.08493860207087009,0.07009647852364148,0.07473015384775807,0.055550037537565156,0.08363543452022512,0.09766176035825189,0.08364718381116457,0.06647496864750255,0.07572602073547079,0.0863172278778136,0.07998884475466375,0.07965607437532786,0.07989987213633817,0.07343512183789327,0.07000204177466271,0.07359543313650346,0.07317210538306872,0.07997928612962785,0.07997566754589469,0.08677407977958743,0.07753811140448441,0.05636126454469625,0.08618126668685717,0.0889441035246447,0.0799340831180737,0.07587186077227179,0.078511519712052,0.06607230378682968,0.08358241795964462,0.06554471533802891,0.08364555073315794,0.08747667380487745,0.08791990875797884,0.08330828960784015,0.07368539365575716,0.0761657378617069,0.08080663570588537,0.08003094780813745,0.06505521304361972,0.05491658884804276,0.07395381905311704,0.09593907462537407,0.08575829757180935,0.06291588830221378,0.07983470561662637,0.07621037847630699,0.07987601619066204,0.07597505702849822,0.0661956290179944,0.07102128626336796,0.08909344038910605,0.07956695804090276,0.07930114609560693,0.08791482479505944,0.062094358552594826,0.06309737739351823,0.07009003207317051,0.07476716807253724,0.07308710395079529,0.078346077788996,0.060886207902416604,0.07455865665713976,0.0884455537664624,0.07900203404161954,0.07612405906384165,0.07909632842209713,0.08227164761033647,0.08301114556958011,0.07314989706964761,0.0744836774894925,0.07451105712875758,0.08294445302386932,0.07911751538075774,0.07627687277699507,0.07748310046166498,0.06833176810681335,0.0808242778976011,0.05943364129279234,0.08126972350769324,0.06467964915494308,0.07993957500325939,0.06482504136851347,0.09220144552252685,0.06474017883109179,0.07566208939492972,0.0782058790424677,0.07908864218223517,0.08045003989615873,0.0836062197120797,0.05937737620816466,0.0668911178810837,0.08998339162434929,0.07093422975840033,0.08213628309141294,0.06402833093330779,0.08325285655734133,0.07347742693353897,0.056869171619688116,0.07374213264083268,0.08034391981649783,0.0759597232180689,0.07289165016542425,0.07703698414443047,0.0730623518281347,0.07779059358931362,0.08188852062557495,0.06778189617835925,0.08021966938942748,0.07383599582707517,0.0732686646873005,0.08154867918886925,0.0830194575919683,0.06655809202528157,0.08634995270596893,0.06709117144036528,0.08650685860658813,0.07040040868415218,0.07036464308644115,0.06790642600345986,0.08190206759161943,0.07541967417892645,0.0876606069643596,0.07906883379394297,0.08172023752326693,0.08390884361362744,0.08619041460012447,0.07322619418753815,0.06785530823751709,0.09138200887988064,0.0842237594967862,0.07658490613415732,0.06436493078262415,0.07798422112703401,0.07161337400218543,0.07700270647469917,0.07828603958859201,0.08628926478619503,0.08402935706072687,0.06394868326216543,0.08989036921380936,0.07488090130426034,0.07041433715961097,0.0669123470211688,0.08445383528975405,0.08061292781313362,0.07989102719551239,0.08214591376560156,0.08479233044932354,0.08442378365954961,0.07498305863980267,0.08516897461816825,0.06599935127763842,0.07971395626816914,0.07452547144770638,0.08950496528800088,0.06350227228955721,0.07067458986349277,0.09157458286257396,0.08325677806862196,0.06587825883369838,0.07543542491745577,0.08518129769880481,0.08126462949511587,0.0735705063097093,0.08425444919625595,0.07286588267450392,0.07765791234782321,0.07683370428930833,0.07737256297097529,0.08034009231287674,0.08979059939526235,0.08401791379096371,0.08366879422686316,0.07403288710157685,0.07362408594084077,0.08154132428448707,0.08920118773279395,0.08517151965318459,0.08203038274352323,0.07436835096499117,0.07631259997423309,0.06333688499589477,0.0871365933305322,0.06835436758090911,0.07696675017778179,0.0764523341833857,0.06963520161958352,0.08111579684185957,0.07637551334167698,0.07156344827377875,0.07421237543682513,0.08378597744102947,0.09025775825826832,0.08765322806604889,0.08538420455235493,0.08521125423372036,0.07325147389558187,0.0732942295512507,0.06716127463078957,0.07389080250659666,0.08842108426294404,0.07433634750801103,0.07704213944012234,0.0843634262879141,0.0766553954751895,0.08605534512021357,0.08401801357838143,0.07111625352927387,0.06961403989057376,0.07686276566992059,0.08081595527258659,0.06967317594420731,0.08177746500432553,0.07391893516594332,0.08117869684557553,0.06837992325334435,0.067296684358657,0.07007619561672966,0.07428060803368841,0.08019762951808164,0.06033617690208128,0.06746236628790449,0.07820843503598514,0.05845801086695074,0.08096082626733386,0.06652057061463632,0.07789906422643286,0.0829997630901486,0.07606118100597904,0.07488480156614477,0.07197552998139053,0.07023615437084325,0.08192246903769335,0.08112442772691639,0.07764986409977384,0.08294142438864857,0.0647788413225581,0.0750388870296604,0.07365139688994424,0.0744565098581383,0.08628309347838614,0.09062697631310862,0.06429645535472467,0.07248309506023931,0.07769062185718648,0.08891308746242121,0.06602826728177225,0.07782279130279857,0.08382745233931208,0.07047100848689947,0.07016594862499138,0.07655212654223657,0.06933449194698144,0.06132219706436854,0.08089721202575273,0.0763468337075619,0.05952252502674611,0.08007542934813888,0.08053885402588574,0.0894259129128195,0.08221433396180883,0.06607216900884486,0.07490888778692294,0.07539526846650368,0.08819344116489328,0.0645524051671059,0.08694941638561718,0.08224433463061981,0.07752039754758476,0.052177213291970725,0.08804151290626486,0.07857689384657102,0.08506011255200188,0.07376003834567105,0.08404671668517966,0.06719380936598648,0.0732696683618585,0.08556868930021694,0.08049041945624708,0.08206226530868066,0.07473401182492256,0.08085137968048567,0.07716776174488613,0.0621726991222039,0.060773448506631134,0.08235262079030993,0.08532096101597136,0.07708699569582161,0.07911527439702098,0.07253923581333477,0.08184719838620938,0.08715959920619928,0.07353666750301094,0.08071670850167625,0.07502240303041438,0.08806428475299305,0.0859593593660472,0.08550147601716548,0.06351155482422383,0.0866500926353791,0.09023014033509441,0.06458161506561699,0.06925832491058337,0.07242807406904489,0.08722474372108566,0.08183102456679182,0.06894999427875073,0.061733561557651195,0.07301583705746244,0.06751444209182496,0.07431598985278143,0.05311761867945006,0.07903344972614981,0.08723615837340648,0.07672651012365536,0.08651529095009682,0.063412015109234,0.05983298959772852,0.08246734603247416,0.07121666156145873,0.06805622093115615,0.09489109524387915,0.06445728318892145,0.07259208483074342,0.08270560594389985,0.06860189721468095,0.07521475059272953,0.0671325373949879,0.08308176993260127,0.07935024558299009,0.06997267744437025,0.07653227895723136,0.08929634768104325,0.07328818651997879,0.07055258026538291,0.0695817461179258,0.07053109013856876,0.07970117153193096,0.07580904838431318,0.08720153780181325,0.08535297858837226,0.06873618585378244,0.07989442747473915,0.06588683048965036,0.05777832991696619,0.0854211163957036,0.07027935272448539,0.0830610368999658,0.07244866207707798,0.06703264168115242,0.09088297892700267,0.057190651123858745,0.07035616096892724,0.08373580366690644,0.06774154882495169,0.08315478830661778,0.07492234381106672,0.07502618678331346,0.09014305151744145,0.07892884427434727,0.08348878863821652,0.08285750160962653,0.08643862641852251,0.06972180893710916,0.06900747053498646,0.09371792710871787,0.07384919333778849,0.08254349743036835,0.06559034743812539,0.07435999710717395,0.0855116528195502,0.07262496157794376,0.0824160027036739,0.0806477013838748,0.0794546267592619,0.07918104441056487,0.08221355694202079,0.07161063992944325,0.062174210163058755,0.07490517126437163,0.07878154540735693,0.08442691355983833,0.07674387750229429,0.08729191869483607,0.061127487799540955,0.0814812542672035,0.07787475360891456,0.07554942213902427,0.082534977636842,0.08058786224390183,0.09391295256248539,0.08840510697073353,0.08347598723679077,0.07522979029991331,0.06282996012564283,0.06286916133006706,0.09061063137447595,0.08824719167878499,0.08131204820737836,0.08135196144595283,0.08266023783417387,0.07491958457193912,0.06846652566901024,0.07052844804682937,0.07989527458804935,0.06447403417767525,0.08499978581036427,0.07806604248329194,0.07202666554910458,0.06431586326750541,0.06927985912692865,0.08557379505870577,0.07863522470055737,0.07673847373964232,0.07814303655804136,0.07722961681648347,0.07156492087727506,0.07935037698891459,0.06364915423130235,0.08297392394242017,0.07589539014730423,0.06709379731201148,0.07061593300260943,0.08423119865474094,0.08534733410280912,0.05503300349240782,0.07258617531495917,0.08545904027835641,0.07872748822825434,0.06783854651018749,0.06689055534700031,0.07984962798506112,0.07658864112975172,0.06492237879180168,0.08140837093854392,0.06611116102817423,0.07917773561218372,0.08037377476877339,0.07255820908694002,0.08915945537481332,0.07838313246529091,0.08553007641628564,0.06658255455238907,0.0754309604079123,0.06726956377571972,0.07480209807605281,0.08406139419596692,0.06515805546562983,0.08565344775158999,0.09120738727097294,0.07788690379646604,0.07489265347246671,0.06876472988173508,0.08457705793066074,0.07839366248700883,0.07646844744949449,0.059865540327024014,0.07873494263998211,0.0807187148005419,0.07751837294529482,0.0883230349548538,0.06861289761588601,0.08807800349876917,0.0857713211121488,0.07108703682414752,0.07113707341387546,0.07434643341296499,0.08139965610914807,0.0839800876648355,0.06894306924550561,0.06825821365235407,0.08652247197122612,0.06327258632703872,0.07010837146427726,0.07551148428702338,0.06516668813055458,0.07597351878286862,0.08108808717686969,0.07424240972333838,0.0741666298690882,0.07624094484744257,0.08587686084566032,0.07284385190184159,0.07285730925490949,0.07966097188932511,0.08143008020118875,0.07950656485528461,0.07902371149931275,0.07140682299580046,0.07374446310752389,0.07453999618773835,0.08482155999817137,0.07503533321154615,0.08669719446214805,0.06643964686250488,0.08324706019438152,0.08014513454033356,0.08452452231346794,0.0878049151436669,0.08680674165232266,0.06674687923447928,0.06798090101365996,0.07115309718798787,0.07020576177294559,0.06670705117162416,0.08996827197035999,0.06572659043329501,0.0749082902634288,0.0696473618197179,0.06179067984178712,0.0824202294444358,0.07653512317175447,0.08053137961160857,0.08449423395612352,0.06309846697900809,0.07776929569618818,0.0661856404547067,0.07783413459949581,0.0667568295350005,0.060543458533756936,0.07488731143832406,0.08259683941233324,0.07883938885216354,0.05568720749286277,0.07447127640178584,0.08055135034571534,0.07937856468555474,0.0864437857207426,0.06884716500732693,0.07903193354646215,0.07903003748200291,0.06145996710827663,0.09193593803303657,0.08406374696371313,0.07006830072411979,0.0646565483109755,0.08137849470237746,0.08330388030153402,0.06868412849072435,0.09047490411177572,0.07397368308673874,0.06910262661458785,0.07542600839226446,0.0829352453936291,0.0692340729548773,0.0764994490837799,0.08397347481355164,0.07997093632306566,0.092949279869423,0.08926908254886043,0.08350119392275632,0.07438788269999141,0.08299221084456493,0.09121184552495017,0.07279214592449826,0.0701458751479509,0.08748127365145109,0.08429177469950437,0.08353228809717511,0.0780595052362386,0.0665062728850157,0.08193590759189062,0.07477547377538186,0.06224472445691762,0.08340736503936368,0.0784186175449165,0.08039918171951083,0.07176390282004112,0.08527888174224091,0.07566110878818313,0.08189648247282089,0.0757325673178195,0.07195047329177931,0.08234021942258721,0.07025556983388623,0.07313405379408804,0.08491662461905065,0.07403984731082108,0.08521448775089496,0.06574891843989125,0.08406375136798908,0.07787375951995465,0.08030248236756998,0.08060102758038618,0.07324233263709884,0.08194005412352966,0.07763047851496638,0.05921090001645932,0.08095026497280111,0.07813911766893311,0.08139191439988189,0.07766966219241292,0.07133661977062467,0.07957063504377115,0.07984140822788463,0.0797584792851801,0.0858008698688722,0.08916084744927796,0.08746234728613424,0.08791728938310953,0.07666108576944386,0.06601901696500673,0.07218698275320379,0.058649522279322415,0.07793895112362244,0.08036734465497897,0.08330599813682857,0.07566054097847692,0.07397315219149589,0.06844427111188134,0.08447197733269306,0.0858466768596011,0.07241654493107941,0.06754709429375721,0.06819419015638314,0.08100207605558475,0.08901540362299723,0.07174713469617323,0.08369513887639052,0.07582580918887583,0.07658271534651873,0.07030518478014924,0.0798739373442673,0.07470689783020781,0.07942730954708986,0.058931769448056115,0.08430303980607647,0.05658418693478941,0.0853601338002665,0.06885345333400231,0.08111616221498297,0.054832613177690614,0.08459595795224151,0.06613882964622067,0.08079157486279782,0.07855218529997568,0.0893679075112378,0.08015675908426456,0.08142596837253052,0.08439937070568229,0.07612257345086895,0.08545408990289352,0.07149137835667843,0.07873253868025651,0.06696772433228965,0.0780633427796639,0.08873480522810206,0.0824394023148011,0.07989870647460802,0.07033141907928768,0.08185410404215011,0.08163429537907088,0.061098589285309576,0.06362785745165878,0.08166074632361751,0.0872750327764795,0.07841418542634437,0.06420132656736506,0.07828496485407908,0.08714422059436061,0.06132220869933001,0.07554021241042977,0.06735637618358031,0.06429604174148794,0.0736217679897362,0.07911647159790552,0.08004718041356547,0.08372222014137207,0.08323494229509157,0.08429815948974609,0.06968666061129089,0.0700050504984642,0.08272835817129533,0.0552329219445724,0.07457728864647667,0.0656041573595949,0.0772904579395115,0.0742506024323972,0.07380395686720395,0.07997598550111458,0.06871322867758337,0.07111237804691438,0.07844705321592273,0.07372089869831458,0.07285565835615378,0.09500226953898988,0.07804782535991814,0.07386465787400065,0.065467308224258,0.0707461148105955,0.08306622017090251,0.07013739838873465,0.08780708716799548,0.06657815058866931,0.08218639060936644,0.07992557502035773,0.06065750003150923,0.08363788456340296,0.07834044641572077,0.08497096047543307,0.0807489724832695,0.0751036921771899,0.08828064706354066,0.07655937660244694,0.08583484824265113,0.08673925381237824,0.07267266907842737,0.08667762562609327,0.07536894275849246,0.07221682651717153,0.07070537077415291,0.08690834911578132,0.08561088997247523,0.06702260967886267,0.06769716514184043,0.07969715855803272,0.06125271358221642,0.06330376175727748,0.07568416076000528,0.08132500103241724,0.062195662547251666,0.056505795622188285,0.08036452751161305,0.058990172800504184,0.08929844607020748,0.07759990027430837,0.06974654214602986,0.07742301114041736,0.07614570308720664,0.07258342102324458,0.08942873373539527,0.0775469044151558,0.08912277328396626,0.068051525915762,0.07307594636041206,0.075751058483732,0.07641997735965925,0.07630593754990501,0.08227627101367965,0.08047197712340315,0.07881083725985362,0.08036287425482933,0.07332570482132178,0.08567954813835643,0.07317958968466962,0.08716250648712001,0.08256562298632614,0.07179172730059101,0.06381075780011794,0.0829411319077902,0.07476813184652706,0.07115464963151379,0.05965563743029362,0.0553985418459714,0.07501539021816303,0.07443489094440367,0.068286398675186,0.07224042077171763,0.05925181959035497,0.06364173378416703,0.08308532453449761,0.07177221731674091,0.07796368674602026,0.0726282841713695,0.07226602196381192,0.05554683305573055,0.07314774180731523,0.07572499452047274,0.08492945334872747,0.07828633454277972,0.06985887491379038,0.08241257626177571,0.0773809826852946,0.07904019306069987,0.078166807842771,0.07444885681096591,0.072230277628833,0.07092146656983994,0.09278539028359079,0.07708438880191454,0.08106304122208477,0.07052213039346761,0.07299884844212565,0.053607297830635864,0.06951067174473607,0.06869086064188855,0.08378950182456546,0.08880648540392656,0.07580542092324996,0.058471039817248695,0.07593322444151349,0.07677612575461876,0.077775807610522,0.06651189506267739,0.08218063003064138,0.08781420051299355,0.08046608344090937,0.08511658961241159,0.07375724631402802,0.08628571984578995,0.0720348923870405,0.062140171972658725,0.08382106884556102,0.07162364232816819,0.08415140849455291,0.07478439586492233,0.06867323185641988,0.07210827923461338,0.0730162784093478,0.0829888746988619,0.09100292661088517,0.06291823073740627,0.07376478547988241,0.09263103355985909,0.07428942586261783,0.06443981741483018,0.07205563480978781,0.07838793761246282,0.0915496548842087,0.07084030481166416,0.08135242662306313,0.08403588021389508,0.08479324900137339,0.06673784110814723,0.07649963002354167,0.07748947666633164,0.06869012066604652,0.0773463852349094,0.06960868816084917,0.08333484210694793,0.07457158427693594,0.0794394847012778,0.07938363966682907,0.08082669275643507,0.07144558429974718,0.06043271945753353,0.07389857367972184,0.07054345908763418,0.07407061854076388,0.07451113819220567,0.08074737807939604,0.06727639483641576,0.059421880633694815,0.07332192763153289,0.0769513676208605,0.06642144674506462,0.07384017345702731,0.0764743462096029,0.07812808736761129,0.08663518879367021,0.09051329555737941,0.06885209545389737,0.07271367722620653,0.07625420257014845,0.08690602164428096,0.09314272470658304,0.07498496583406447,0.07021123713329966,0.07152696200153705,0.08219244803068851,0.07349247244866633,0.07735231163372924,0.08845745211728083,0.07801388550057166,0.08123950678272643,0.07777134005566477,0.07511068808710121,0.07874323376985469,0.0639409908933868,0.07917684040245215,0.08149233776111321,0.08978619008927893,0.06380112538215295,0.06699799485720082,0.06942710180573597,0.0807201296560816,0.08036020606993288,0.07970198932375266,0.07068446467424971,0.08554404000559397,0.08589383747678461,0.06973661754398018,0.08950652282232645,0.07546397792406971,0.08616148516199014,0.09105009057626078,0.08190775061795144,0.08266626922068217,0.06662780703057934,0.08744376391921427,0.0818089836059685,0.07332016816677747,0.06328776413537285,0.06917455972415948,0.08517822441266278,0.06685874554869375,0.0840602445504022,0.07460869405546164,0.07365303415497257,0.07706441449899865,0.08452029444326434,0.07289592348788365,0.07152621939877335,0.05833464698813812,0.07615489514368382,0.0714171646447549,0.06956559105071361,0.06931635946399584,0.0665322812763676,0.07628091512863185,0.0698694862218891,0.08308293357984076,0.08113583274846803,0.0771870251666591,0.06766899948517972,0.06739579922125426,0.07013941351390793,0.06812913412582627,0.06471487199425308,0.08438434028342362,0.08890807648516728,0.0743126263089906,0.08453494728588967,0.0841078813084822,0.08056568658199702,0.0891170371261232,0.07510293729990479,0.06201543932975032,0.08584887641172437,0.09041325730445579,0.08127512913780646,0.09033331357745833,0.07956814828668021,0.06787974153205657,0.08836242986048756,0.07030318065058486,0.05840184038727091,0.076586749007603,0.07385547251491441,0.07389999410033543,0.07182639132991936,0.06056714262456926,0.059264761684173446,0.07208907877719992,0.07718134441213968,0.07953404107025755,0.08245843647237905,0.08188651889219459,0.07193855069596355,0.0836578253754361,0.06765675242927362,0.06936293989927542,0.08768801033518035,0.06452290123717791,0.08250590990320289,0.06415213077284233,0.08311920207883035,0.0774642532390084,0.06532119016320384,0.0809005569205496,0.0837141175887833,0.08690726929446144,0.07486525382670262,0.07075623950824753,0.0730786536465042,0.07733763163080654,0.06326785099264076,0.06737549004344562,0.07090834885401887,0.06630693467952467,0.0688335619034002,0.07274271574060676,0.07049518634462942,0.07600657158198983,0.0842305458269283,0.08364802357701887,0.07950797512512517,0.0740520562287757,0.08552621063785927,0.07540871142377595,0.08459559886654536,0.06750990736748747,0.07493370431380902,0.07862693937267741,0.06981103095220474,0.0550565371609829,0.0823394602853241,0.0726570376083724,0.07893393731381035,0.07264485381227057,0.07996659076106374,0.07834297693078451,0.06145773251159769,0.07310459233841596,0.07689174505763324,0.08219251964201565,0.07437089004873218,0.08748098303939772,0.0810990595941057,0.07003458617219022,0.06442672426669299,0.07821294161348996,0.06844151120195188,0.08493101608855066,0.0689377591928691,0.08018363150467485,0.07493100656025076,0.0703565289869772,0.09317024908993025,0.08627559868298998,0.07243505203910011,0.07953442785014625,0.07448576751529692,0.07249631292717842,0.0739966398676296,0.08141203152071269,0.0822916532818912,0.07957795393733388,0.06958476502843185,0.07964089083916617,0.08014947708216946,0.07983515107283821,0.07695904132550399,0.05328461861323308,0.07127355924068679,0.07103998451508296,0.09431664594147146,0.08772869561286638,0.08059457793092112,0.0718653511871645,0.08019439536315046,0.08064959964086267,0.08953305672272341,0.07633066447179257,0.07600448766381537,0.06686077595114602,0.08851447181781982,0.08653917497555236,0.06827409992654004,0.07361328764253791,0.07474649478421794,0.07934041593102661,0.06792886820302432,0.0820599449813525,0.08894640545133486,0.0631289879145216,0.0887297703967451,0.07168380032418604,0.0825637609666632,0.07748220540273304,0.0760620476894813,0.08855169670707745,0.08534155034611579,0.08221114739027012,0.08347391962684367,0.0842383430636756,0.07301317005524915,0.07069294548859012,0.09113512229766188,0.09158126491550334,0.06590670310407157,0.07209378825635925,0.08918136237607116,0.07158549033166839,0.05941519733955113,0.07903598350267706,0.07290843713719879,0.0877094450138072,0.07927528635827849,0.08648269858820094,0.07362292661157263,0.07767545554979291,0.07593089610099236,0.08753950337711183,0.07382804381148786,0.0725808845266154,0.06622935466172129,0.08788200255854575,0.0837877942546382,0.08689328736293345,0.07938316661570577,0.06773397280878571,0.07622450034082884,0.07777251105668159,0.08030654906275532,0.08745943041773686,0.08743267793349746,0.06657119568212949,0.07169915569728932,0.07152814857997072,0.06017179875383223,0.09397497357820431,0.07977704019766849,0.08497368042877668,0.0856993416002308,0.06361466365353363,0.07219495811132638,0.08241511216280753,0.07966821552028205,0.07650589653981985,0.08015915393876794,0.07556272302472931,0.06688360668325774,0.08648509110068425,0.0856197669443994,0.07034308406003911,0.08593025987033706,0.08654607143016854,0.08584095591969063,0.06907361271243169,0.07164817579849436,0.07044187801117861,0.07610401626677166,0.07439875390560874,0.065770714138733,0.07343762101273721,0.08866862937041511,0.07265452375652601,0.06037713877388394,0.08657672689886425,0.08043170912595143,0.07653722899553225,0.08352902795482375,0.08352395307062996,0.06998519156526035,0.07887428056429778,0.07756262863020749,0.08508260870214637,0.07181356269233821,0.07264964912292057,0.08993546658042262,0.093937557398728,0.08471172890530627,0.07520663766367681,0.08231137396710395,0.07562168010098658,0.0736577466464091,0.07008507613529505,0.067465231912663,0.07477634484683882,0.08713315589916543,0.07754975497875968,0.08354506231346102,0.09344399560223189,0.055255989063209776,0.07374971209896142,0.07566283050200417,0.07144125993776573,0.08694002984402138,0.07610085161050882,0.07169200704571493,0.07937940290807205,0.07238079190048594,0.06833293469920676,0.060130879223206535,0.07725117545715943,0.07177078942819456,0.06957039287372116,0.0873063178553174,0.08208069806829058,0.0897815738295889,0.0911214740020548,0.0716903066674435,0.07603295962519854,0.07968221512360157,0.08054501368492192,0.08300340186062652,0.08396465267967285,0.0802274603299321,0.09074362660765278,0.05971782328231216,0.07376841337725103,0.0808127455556305,0.07500494115699985,0.060228994781073936,0.08599807566658081,0.05898158448202655,0.07631383559994982,0.08511003661300717,0.07553370906626813,0.06168618534076322,0.07051603600071726,0.07847794248024176,0.08092295407293777,0.08230085767498835,0.0860895963927355,0.09513585415175305,0.0753271646169086,0.0755915736791985,0.07559211431318398,0.0635378110571398,0.0623861428918703,0.06364214648140759,0.08594844796341652,0.0758254242715366,0.06839327082057248,0.07011528701783008,0.0820162526213189,0.06374197885808944,0.08810555293553105,0.0748376598834237,0.06948087306616635,0.09365121621060539,0.06000649899082289,0.07846573998306085,0.07959456505867145,0.09058902966570476,0.07677637009285898,0.07459001262110126,0.0851158050303612,0.07624871517546003,0.06503466591726342,0.06757173767660944,0.08396901549022678,0.06298301818452913,0.07784070260376388,0.08455026398591968,0.07823204735686981,0.08264767953607752,0.06676900108466205,0.0798599053397493,0.07720196242274929,0.09469932197805907,0.07794679316993106,0.0727474500497928,0.0718773796785258,0.07240233036405899,0.0805882613825089,0.06441209220527032,0.07002650690557397,0.07983357524980585,0.067367532976022,0.06436074906049057,0.06763128598877782,0.07228804457104343,0.07921692074595665,0.08089773771945469,0.07901131753007104,0.07359466119386039,0.0776910290900444,0.0568735346516667,0.09035280713850137,0.07704537955377434,0.05784187491211948,0.07334125107125461,0.08329612011410921,0.08764598068616342,0.06469451382122604,0.06896154183198312,0.08936717291584231,0.077220599349518,0.08004039017223051,0.0798533520139082,0.08586333818973732,0.08433483220952098,0.07632690429349882,0.08902108102838993,0.0672767972154347,0.07902294317539596,0.08525383327475206,0.08732216258018949,0.08493422631217779,0.07689295256274781,0.08180740834592698,0.08335992304931861,0.06364054642241007,0.06957500360998056,0.08215861484531498,0.08478017225783632,0.0565558228693154,0.07892362843210954,0.07431653768867795,0.0709431577497625,0.08383801307104592,0.07637989516733806,0.07414934284844166,0.07490762904461781,0.08287889318323013,0.07331410454198263,0.06500767809329841,0.08195765620921486,0.06930601999761056,0.07443873495831688,0.06390701767105167,0.08819292520744859,0.06395531969459635,0.08219625109475828,0.08121877049915924,0.07327686037943748,0.06570910667273158,0.07780028656446736,0.07287226635545593,0.09021787759278344,0.07216864621823259,0.08014372957833696,0.08415430954559855,0.07148799962300403,0.07698138686430346,0.06375673168198479,0.08038535215865386,0.08561924843357752,0.08294410747229365,0.07595431971312018,0.08021981183703339,0.07075759903498977,0.07394797936890171,0.07331225951788581,0.06657707808680646,0.05770299355609522,0.07963062816667964,0.0690510272145536,0.06832628467953734,0.07610596843532833,0.06456873820084608,0.06948670951831831,0.08276819489435386,0.08701180629090377,0.07946236471680483,0.07471786685125092,0.08284924630861254,0.07612273516164708,0.07385042337610856,0.06851432697716929,0.0787949339019309,0.08144958391141502,0.066639737827085,0.09271243795650838,0.07358884287693145,0.08231392897762296,0.06923522887954138,0.06807736455153857,0.08771675095187441,0.08164633677711967,0.08331465636661675,0.06666319794460962,0.06976925659732405,0.0738197755955857,0.07894229341652452,0.07884617863986981,0.08087254576383154,0.0834488239661727,0.08288748266031629,0.0831769621188054,0.07552579171541973,0.08451406853949558,0.09119174829838647,0.08402548568451876,0.07436963351864774,0.0603527590012604,0.07675480282130068,0.08506978354794231,0.08211998358842956,0.0769676275221286,0.07378714988507126,0.09078670137948155,0.07860414027091495,0.078977921380838,0.08405760281511926,0.08084712944306684,0.08995667212938835,0.08414877728360225,0.08837300215867513,0.07042160852162543,0.08884602527895588,0.07830549841513298,0.08597951232378487,0.08436045473968748,0.06327533857400272,0.08836006155323962,0.08087266826866199,0.08991188108757259,0.08194888928011691,0.07236348306557794,0.08486153255699018,0.08501546885502263,0.061399322193157094,0.0817873863832143,0.07929666194963117,0.07822827704689606,0.07439867430729447,0.07875743312846126,0.07309534852370345,0.06784388743334142,0.09279391766193577,0.08448800280556373,0.07545738701462187,0.0786743356247179,0.0503422308131197,0.07483125229923095,0.08687604707038693,0.08208146223823899,0.07772158728596193,0.09563681702781954,0.07559008724595945,0.06776828110509456,0.0810221832128082,0.06922356849616053,0.06561625128412803,0.07929059837946598,0.0693007684524233,0.06471360206469104,0.08285552374363929,0.07458015230299253,0.07889401281469025,0.07239896702257759,0.08685501484520716,0.0733971540974869,0.08170344768158203,0.07175008100385986,0.0886862373721655,0.08203896290660873,0.07887630143917368,0.08551978306736706,0.07377075313377396,0.06662432900588675,0.0802792069957957,0.07708318167098877,0.07754497808556571,0.06958135502602349,0.081801719770885,0.07323883397546505,0.08690506145604204,0.06270000819630743,0.07900852694946789,0.0867412233853678,0.07858601063443975,0.06771624603328373,0.0725802702348774,0.07504024405962273,0.07868676098649825,0.08270551510338864,0.0840052088447524,0.08434798041674049,0.06425760562065491,0.07990542855563174,0.07060425962332346,0.08439344990094957,0.07232468997118029,0.0753806148956326,0.07183574892249657,0.08026112806161478,0.07721741348140893,0.06522032239935496,0.08713047673923204,0.0833620444272479,0.0709721207656441,0.06434844810757653,0.08005271660872748,0.085322703205157,0.07225979786377104,0.08318962621311873,0.07019028030088505,0.056781624095008615,0.07553696733948692,0.05782912826559981,0.07846568263391673,0.08645186735565222,0.08581391834200652,0.08474330754431544,0.08183124105551118,0.04920509292833549,0.0659140263346483,0.08898245810456643,0.07584683080425839,0.07342708041501479,0.0813725547699729,0.0546308975769336,0.08441412855899869,0.0810069918225941,0.06276325740818284,0.0820002509273503,0.08169441420728993,0.08149383668292681,0.08944081666979492,0.078685823614138,0.07064579942407895,0.07658477268040492,0.07230770222296129,0.06598246158462785,0.0787593817267419,0.06752278164416459,0.08167448247737472,0.0781273908256184,0.07551104378148224,0.08176740024530935,0.08489151584099366,0.08371578894343745,0.05859299931740837,0.0860529714105675,0.07307461803309012,0.06049297671343697,0.09287209205987315,0.08519637060556119,0.08716952683521159,0.07419146430141024,0.06845354477140558,0.0788777655346663,0.08067858347603017,0.07263814257664844,0.08860428783918682,0.08123250331675277,0.07526402648017311,0.08846660804395629,0.07501691379428128,0.07532640460419582,0.08699580690925279,0.080950964448946,0.0723702974243069,0.07550574890816847,0.08509042970892991,0.08006396386338702,0.07072123232302063,0.088729561023617,0.07240740452514413,0.08802504713396139,0.07417359820098637,0.08016764926821084,0.06536109784772054,0.07756080862852485,0.08268787416889903,0.08673210341757687,0.08337400091156963,0.07658578097013505,0.08433022347918667,0.0815535248876587,0.07706659204409963,0.06616663953843668,0.07998725487431343,0.07678184508254832,0.07174210804769915,0.05712368778064778,0.06035745695115936,0.07330434294813877,0.08354419877154862,0.07868762954486602,0.07399160624092962,0.06828952247369782,0.08765427196832044,0.09037060303420286,0.07631075467209616,0.07311351796395196,0.0655249425639833,0.08113399003462804,0.09412415415150606,0.09008961393332557,0.061719482718923864,0.07221608871840614,0.08214640316019094,0.0865522141923176,0.0859199062627576,0.08554252199858883,0.06745489268495528,0.07206040119084588,0.07273432095961548,0.07298261732821347,0.06915236958450377,0.08417026239836216,0.07711075938114177,0.07260259740270988,0.08244290446226825,0.07713183636824705,0.07776034436279933,0.09359739149415323,0.08754885812253757,0.07389221954519933,0.07358814039203337,0.0653783023398963,0.07870718970443145,0.06729688941091211,0.08910847758837957,0.0683577329435176,0.09046889674963468,0.08031901502698918,0.07877849576192303,0.08293875629239054,0.06808314468076279,0.08186114192895516,0.0759610146098747,0.07501826769358863,0.07805436210457639,0.08814105969837738,0.07289344359537756,0.07387736254227137,0.07200691763861117,0.08337215194294556,0.07599908456953305,0.06907287960909347,0.07919479616568642,0.072276601658773,0.07372273830135351,0.0871621514345731,0.08133228353184077,0.07828467378332998,0.07418030802628053,0.08308138925833052,0.08838334210468665,0.06530794441296775,0.07790345242211909,0.07823390606626668,0.07192993324377547,0.08577671512434848,0.057889833487387285,0.07629938460790482,0.07943536931593434,0.06599596893313434,0.0806196173660432,0.0836364903970744,0.07182795533817324,0.08032415484880961,0.07409719179177682,0.06768022820824646,0.08504716482864111,0.07416931649461858,0.08032953928626346,0.08728517092010639,0.08926928377592647,0.08598998044497708,0.07933322866856299,0.08171545619421303,0.07405040580021821,0.0694846908472081,0.0638473510972192,0.08701194298000609,0.07930791192838245,0.08116476575675691,0.07370277612226919,0.07739258457733057,0.07415628141607702,0.0841148206862928,0.0895427323367334,0.08977653957772372,0.08598375011176654,0.08219087698522687,0.08267005575980838,0.07991529028394366,0.07186265728327551,0.06605946735920548,0.07385117194103648,0.06960986490190425,0.09038297551375823,0.08083838738826385,0.08914568621759689,0.06315311462574172,0.08677586143539864,0.06592718958088618,0.0827244782578529,0.06849558706958576,0.09251623466422935,0.08124203478908758,0.07438350103502885,0.056619028319333686,0.08978776976364147,0.07233940601796257,0.07488776700825334,0.0763132931110578,0.08046282393213547,0.06793513935276209,0.07689540506291073,0.08218972583130409,0.05084095722269397,0.0900950263037825,0.07331297705629546,0.07398903931049935,0.077793886539118,0.08323844828461043,0.06905648780777253,0.086018823898551,0.05685187727468621,0.06550505989223318,0.0866082350584044,0.07819482608180423,0.07804579784377288,0.07061602195126584,0.06499018716680004,0.06323365446859412,0.07776431419006474,0.06729060325828959,0.05780410406202582,0.07909967554264886,0.07396607360683427,0.07585041430208123,0.08508900349330034,0.07298746565658652,0.06963149650336484,0.07646656087452972,0.0662844701327507,0.06243003456081113,0.0805999565189411,0.06406549677510957,0.07291074444787335,0.0905081980846944,0.06489705497131903,0.08152789555116684,0.08756615119846155,0.07798598451350151,0.08438567469445987,0.08267379820127208,0.08473572383427108,0.064557347853268,0.08599218817924176,0.07812061281211777,0.07841442465254868,0.09222597370806232,0.09216922838509323,0.07937820787161985,0.07017843464935036,0.08358932189659238,0.08454067188200448,0.09018334706176295,0.07212215326371084,0.07506013556404159,0.06921505391590467,0.06880312990053566,0.07545296904336926,0.08025538537206248,0.07575089870949114,0.07745036847128081,0.08729387748639295,0.08656511689917062,0.07592032994664737,0.08259270014319496,0.06205252412070255,0.08470613844035987,0.08280620427647814,0.06639584601917842,0.07524658212963786,0.06406213186656046,0.07852745270045544,0.07542880772676841,0.075909735069036,0.07427629444550574,0.07141967196286649,0.05571452264821215,0.07440596805828707,0.0815751686285912,0.05259271587015364,0.07996882558627445,0.08405257338722646,0.060680800252845085,0.056119431127160266,0.08211441682638129,0.07557667544389718,0.08382812557106212,0.08882013012479575,0.08527370002730217,0.08202514497136212,0.06483401964382146,0.08362814168330909,0.0841564558730668,0.07921658447833888,0.07740316088788976,0.09383522732754468,0.06917167181477116,0.07800522180849423,0.08150023920949692,0.06335081004754975,0.07019972510896051,0.0798566835528432,0.06463999181131413,0.08285776850739039,0.08214909210879118,0.07699883186468577,0.07452799549303787,0.07326776784920133,0.05537871665346872,0.08477263881961335,0.08385430122646861,0.07777948890410928,0.08717460652297997,0.08279383652749814,0.05571750777026477,0.07135047846331068,0.08046091876314332,0.07926459044492828,0.07703684969866412,0.07234430358100254,0.060947734581470694,0.07207369339452363,0.07685561687490558,0.08450335010930976,0.08466233765140782,0.06105535974320653,0.08117278543957833,0.07939790988438805,0.07410198888569637,0.08169221936279562,0.07471861008553075,0.08069205247877848,0.08216888234241532,0.07826309068509421,0.09283333226028877,0.07690537765130662,0.08737308451365247,0.07116355432073995,0.05938130873815049,0.05860857088196154,0.08148196548806966,0.08233179649958913,0.0874251382922537,0.07873862793208845,0.08011211479747635,0.09224301607304633,0.08551261595558621,0.07962712339031822,0.0626380725485603,0.08069335599685273,0.07278415829438015,0.07378851762147798,0.0704390564343336,0.08509331659081631,0.08509370609693677,0.09353180778552367,0.0878779063937953,0.07968944762379578,0.07548218615031352,0.08266606031671406,0.09336071469910896,0.08549526300871549,0.08429359558357985,0.06819906335611978,0.07579393904025042,0.0742745695689548,0.07578894731536753,0.05780792147840475,0.06985156364220896,0.08385049638418961,0.08703969128348987,0.07369192547452338,0.05719607159954804,0.07446202722662555,0.09109574592445638,0.08435518950617821,0.08012847687514377,0.08718613219170224,0.08443571855570331,0.08692736787947858,0.08163194573528393,0.061711193736192244,0.06053145920572608,0.06672926831019349,0.07985754435961687,0.06001728558275046,0.07951845051672038,0.06207349975697777,0.07171665100463687,0.07421409088229397,0.08137729003576369,0.0653109802964369,0.0731915728885274,0.0821457836218992,0.07975309602248323,0.08723943781475155,0.07462155431512124,0.059829789502739025,0.07628712910920069,0.08551785758636052,0.09133286931739582,0.07251157824168586,0.07645397838087856,0.07082812458956322,0.0766148844989247,0.0681009527636671,0.0815168475715339,0.07938068996663894,0.07618662815796429,0.07479929225363129,0.05769413526705351,0.07391683619651744,0.07028208988958723,0.08450549689640904,0.08290447422176522,0.06606962516565763,0.0729434346925187,0.07677853471863172,0.0762908650868235,0.08736525468621197,0.08436882016581905,0.07970222973292163,0.06949013567742766,0.07698183402012765,0.06306845439953797,0.07741529155746524,0.06313017028349481,0.06580390416942906,0.07705924962417329,0.07800896599391832,0.08369378451061514,0.06913844020546772,0.083485511123869,0.09196527376123309,0.07982575973489817,0.07275809482526616,0.08338724259314945,0.08060586470584762,0.09290859189325483,0.06922188411624378,0.07787206012441295,0.07301006984808966,0.08691496466893076,0.0913552687918754,0.09177100876550984,0.0742209471574844,0.09156003196971511,0.07935611101158342,0.07463958754724452,0.06880925071104411,0.07094511810648305,0.09270082030294503,0.07257162810912601,0.06783211239029552,0.07942606235442405,0.082520565309263,0.07112982374500885,0.06250042297617475,0.0835224063425355,0.0712149810261,0.08061771712941604,0.07408071383797037,0.08020594748548618,0.07724656016087945,0.07738676283738455,0.08707778663955118,0.08932091828106814,0.08064559325042521,0.07542597024090443,0.07905671803181355,0.059820184490080336,0.07284495107818867,0.07634886426004014,0.08338440208733501,0.06766538865551577,0.06619368075725368,0.0789118271422073,0.06983245065430208,0.08150442828792624,0.08387718367920682,0.07835545327782001,0.07079176627781195,0.07257525238875477,0.07211281444682091,0.08115166693523097,0.08209581788415092,0.07433844957282632,0.08818667205472946,0.08187560211558695,0.08206441865866837,0.07786832714457839,0.07300685804091173,0.08006674527078259,0.06313954010279357,0.08431684208731055,0.06785616890062726,0.0776358296589943,0.07369693405847988,0.06784213541924111,0.07185229896611531,0.07804807468120677,0.08249541887245507,0.06874368408538886,0.06728939190293995,0.0739204452748974,0.07575775765419727,0.08079039899494171,0.0884719207888984,0.07735765539741538,0.09039260853453618,0.07190788557934016,0.0739622678939322,0.07963110190524525,0.07677003367073659,0.0783715114480855,0.06276858713331662,0.09289480299326806,0.07176226651856686,0.08428185399773373,0.06277793590846516,0.07819191031129336,0.08575898951410171,0.07057904835359004,0.08987223655184323,0.08076561145942032,0.07710821017962761,0.07301187696920415,0.07363462942233931,0.06905227090510492,0.07498249847706542,0.06425380624791302,0.06867021776909046,0.056175058595946306,0.08358399161498795,0.06947385723191066,0.07744242202596544,0.08976571602285258,0.0656306707019352,0.0744586845320524,0.07850249469254841,0.07894014498828737,0.07425570112957382,0.0713352061746293,0.06713606197107155,0.07457184117608287,0.08502292170558855,0.07397084030177586,0.06909219486264076,0.07755253802711022,0.07182334795756569,0.07955469677178661,0.07789204064123037,0.06570261950618217,0.07939681455391906,0.07638438835844347,0.06793463977623213,0.06563110950944469,0.08759080526530724,0.091373157934524,0.06834722349991156,0.06704720255812441,0.07471795386066597,0.08588163686210339,0.07678192282827967,0.07481915129152372,0.06328748965033684,0.0808667076001411,0.06898970980926455,0.06867255641330941,0.07832420838537196,0.07355651668231156,0.07282331728095247,0.06732330392857855,0.07076736275855648,0.07778571008037827,0.08882760775255885,0.08326560219525359,0.07215588040064383,0.08147560326581224,0.08961808771537624,0.08032811571546403,0.0856460445930771,0.08981658556733856,0.07465897226921894,0.06836310075690274,0.08048202972381371,0.08048046389593791,0.07378616590415402,0.07130703398794593,0.07786993717263156,0.07856218091614847,0.07586542066553338,0.07407079873601125,0.08587818983070511,0.0677911519273133,0.0584752784157286,0.07841757613684071,0.08838011199729476,0.07536037502787005,0.08753566349751274,0.08009673945976739,0.0594566879654255,0.08259532170051642,0.07853273935544103,0.08561464791489554,0.08024276113101667,0.08541832722167239,0.06505229854966861,0.07568476795983539,0.06456305131832069,0.08723847032066123,0.08548399563590284,0.07956351774208129,0.07903750500306296,0.08594372253103749,0.08646930704488553,0.0850046272790412,0.06511616386645092,0.07010067917864585,0.08355191827087552,0.07986540326767079,0.07221448039731118,0.08161480468838289,0.09219013463408582,0.07033166117146185,0.08871102879823947,0.07701531704316864,0.08429719051671716,0.08435091604596923,0.08537079015681695,0.08896980022532229,0.07537276235075764,0.08133954459758028,0.08108276013541028,0.06522490111450624,0.07006901242022931,0.06407804847187895,0.07694583718817288,0.08623460206992506,0.07510335209050009,0.06906236761815131,0.07499595208128039,0.06943707582363558,0.08786063165654028,0.0891999516322018,0.07661361503702627,0.07862289940469841,0.08052576433705216,0.08344789082823457,0.06779188972411787,0.061930593408284676,0.06871682369253329,0.09132874101731114,0.07746148101078017,0.06998411982659056,0.07798584522678992,0.06980931144727506,0.062458298755212416,0.06604390566211234,0.08168425226124121,0.07225846731205601,0.0817668871097809,0.05944573829431695,0.07552096778135717,0.07007335221659397,0.06996606436378261,0.07890509672247932,0.0715989961339395,0.07299845804000732,0.07049638561386622,0.06409873545304054,0.0596639301855658,0.07961863641796363,0.07773666914023268,0.07791388005050143,0.07204492762440898,0.0747513427863363,0.06546152599105787,0.08166496739741223,0.06412884612110081,0.08240771607776418,0.08089513015586168,0.06310322732196424,0.07572580620412592,0.08181452080542617,0.0690262325697999,0.08112862022821389,0.085013434267618,0.05111959301513718,0.07785231967651338,0.0783434528149971,0.07887733690715212,0.08160386184649013,0.08263533731657194,0.07815247827091591,0.0844837128470691,0.0878491009031083,0.06825218869068214,0.05918753480012362,0.08040548775404843,0.07415360839619423,0.07335290802708958,0.08016696875896262,0.07417832944659691,0.07423268621745996,0.07314620082039308,0.08258730592765035,0.07557656756576385,0.08093926531867507,0.08348397364129165,0.06786469734115738,0.0970760843873296,0.08273425087023491,0.06361894929391843,0.07631524983676019,0.0697173293600823,0.07572144187223205,0.07232188805412126,0.08791748934431119,0.07671276961337896,0.0820871865670236,0.0782354572699488,0.07949119207928386,0.06728962786855575,0.07239760723720148,0.08985872130791034,0.08121120314254865,0.08226636365814101,0.07216162853478034,0.059292269354541585,0.07473719161596581,0.06593711151738316,0.08650738790728438,0.08008823013869015,0.07443768037829882,0.06056611511804032,0.07486477869143518,0.07588852608581127,0.07081115342968226,0.07118156663684795,0.0818966868543561,0.07896119503905893,0.0740769292318488,0.08220291153345226,0.07131489103975032,0.07433513796344762,0.07808687692386732,0.08194872410526924,0.0648247959031404,0.0678001794005196,0.07081174070179645,0.06968353717707354,0.07338580230751952,0.0844049031737716,0.07571259128083128,0.07551942672158042,0.0781409773415648,0.06690503227968776,0.06775754336034219,0.08722820526497133,0.07394819873424227,0.08156750363907361,0.06682684801646774,0.07814556880196119,0.06661268699451217,0.089500032989244,0.06389134425929993,0.07518886552810833,0.06912018225140028,0.06277078660457129,0.07964917383170933,0.06771777452637603,0.08046344037216535,0.08085471516933432,0.08028644906143804,0.07317837188610972,0.0828325590744759,0.07481106480861903,0.08268764728059465,0.06121443612903933,0.07327242962483713,0.061432737865698236,0.07223231976442827,0.06477520875332066,0.07408789347070774,0.06866929969593683,0.07621337149982482,0.0813249679788561,0.07549846595537563,0.0801229297507825,0.08731912988712812,0.07819885148378562,0.07147977239878137,0.06652589531428171,0.08086789813627732,0.06892129611984771,0.08462603113357384,0.07779530935200449,0.07631533914790031,0.08174625133222296,0.08350575240305633,0.07842743319042517,0.07794979228039958,0.0758647432103621,0.09211325737571122,0.09010359118895617,0.08337108812533225,0.07721926070444371,0.0873408626504571,0.0745826345505468,0.07265847997358896,0.06632969193401902,0.08137114413713036,0.05063495367026286,0.07397420508507573,0.07438352508449514,0.07237969054341953,0.08664115703770973,0.0726549492253189,0.06587820653344409,0.08135121958575545,0.06807781437301555,0.07038060919372957,0.08290249575965261,0.08159329060478077,0.07442924277137541,0.08206855170485158,0.07794940257499705,0.06362927346044579,0.0782016981876849,0.06576345864186994,0.08858542047874103,0.056850377975255814,0.06555994412241513,0.08247889524335367,0.08404830628411925,0.0794491031437847,0.07328729053024471,0.07037165063005943,0.07452910266998654,0.08845143865769961,0.0793425376991927,0.0867132073901036,0.08650045915013635,0.0724663676387782,0.07994155181920448,0.07331662328810519,0.06053911903737878,0.07482518634253185,0.07818751598288323,0.08718707210609009,0.0798731023675188,0.07778463242877268,0.06078645874603289,0.07791490205967183,0.0880109017501622,0.05511008828076097,0.06879863693172009,0.07261568035014482,0.07355079336482143,0.08675738222204099,0.08361920922185714,0.08680970044261616,0.07756021020936946,0.07523976494547724,0.06622844649102358,0.07246172462983216,0.07749583526653725,0.07703780644191593,0.07901210308403891,0.07588647300994164,0.08062982496782362,0.07337340997034612,0.06527330360430544,0.0776031893191105,0.06994237487567029,0.08267290076417326,0.08243735697079557,0.07474015359375899,0.07366164155610286,0.07983372036818823,0.06392407901778292,0.07138661202218047,0.0706565405115542,0.08255680268777721,0.06960054385739234,0.07740545319965328,0.06480324701906082,0.09075886029937474,0.09510974175210021,0.07936753394522209,0.06602906360596444,0.08375769659649596,0.08653376107100802,0.07696714563480742,0.06669507771059907,0.07888672704399362,0.07515043236181174,0.07832348094531336,0.08397461363598244,0.07258297701497361,0.07215071457846182,0.07396952913741632,0.08785942890820199,0.06813675984962722,0.05657596808992015,0.07941678191367027,0.08828538296238209,0.08659443083724108,0.08968922196591297,0.07282615056711159,0.07280638249444393,0.08345563231209335,0.0771600914466416,0.0816250208967122,0.07016157884695831,0.06566736936606672,0.08305154576918464,0.07714631132925898,0.08176337215503444,0.08213280109727036,0.08641704414325342,0.08010645947660991,0.07327115313266344,0.0841416676394525,0.08213962189578375,0.08080665250651095,0.07003523500192665,0.08096857578960048,0.0637726269728093,0.08730432507613797,0.0621465479842103,0.07305009228680272,0.07964733695815213,0.08977707861672718,0.06957683216826882,0.07488254873100261,0.07456570953654587,0.057729220451527404,0.06897411327994235,0.07370591945832208,0.06425300380634191,0.08176036457500999,0.07564759147638574,0.09015351402826297,0.094112855959779,0.0818294450832641,0.06860448513424036,0.07194891645614906,0.08505165635007543,0.06885400263883953,0.07943919025460416,0.09183267887200683,0.08176633841995684,0.06614866918937524,0.07526275335600005,0.07402051309000376,0.08294021132007433,0.07756098713916598,0.07010021675885517,0.07586444664834172,0.07237189301269337,0.07915522919779513,0.0663415464596655,0.07302973949508068,0.07424226794179387,0.06652550251969652,0.09657273438995143,0.09308859987443384,0.07093055717345638,0.07795342062546119,0.07709392156591358,0.07047821715592886,0.06231574329260612,0.07497453887917403,0.07298484511269657,0.08859136918888026,0.0720362121887628,0.07225400764753803,0.08128872859443254,0.0663346979070559,0.08673213302052449,0.08276464406397102,0.06855126845979749,0.07477199014865182,0.06434466849528894,0.08607922338119493,0.07840796279841417,0.07616068979150488,0.08312944554818812,0.05472925156839613,0.05956084416009756,0.0647202072820032,0.0818481168746455,0.07411511967511161,0.08473202204172565,0.08003523236434763,0.0795657393250499,0.06347314266987492,0.06242983346641822,0.06923004404366079,0.06574807456576848,0.0691102431785999,0.07094419178820295,0.07691224882785572,0.07497265640579265,0.06870716634179849,0.07837644637013597,0.07471463183298142,0.08546198465242436,0.07785595421359238,0.08115639696666982,0.08820322913236045,0.07583121605101079,0.06658899842972137,0.08987930360740186,0.08812829485836078,0.0826102455981449,0.0718675845135281,0.08089449815714586,0.07602362524949362,0.09257888052976737,0.06850018495140195,0.08985505651684539,0.07708998535078908,0.08124906636539295,0.0794041441052516,0.08842110041004946,0.08627112732957312,0.07008931572310545,0.07763572906596049,0.08050278905426765,0.08454842566584463,0.06714609624353479,0.08383781677608852,0.08736522717820153,0.08350622437675129,0.09005548668292576,0.07263599905963714,0.07948375853754536,0.07473812909270411,0.07657059707789034,0.07783034031968152,0.08118903278590799,0.08231039641895652,0.06505382388112646,0.07631180841455779,0.07954825525763767,0.06608048940972544,0.07960829034542852,0.08043001009482664,0.08178323590179622,0.07371329085739656,0.07661681946009888,0.08316050104518255,0.07524431042990679,0.07595888619385188,0.07247015087959718,0.07310172572160431,0.06998859458029819,0.08903424200678345,0.07013767897124093,0.07951117982139642,0.0893639598730217,0.08781722706320383,0.08297065055389861,0.0807232997052185,0.06409257876077293,0.07901133770781946,0.07120673806043525,0.0834876610658796,0.07712525930030818,0.08665964016113827,0.06471924416869032,0.08528915576444117,0.07841271263259394,0.08014092393152614,0.07615235649866937,0.08173136756876816,0.09053440770971388,0.08846194914546504,0.07122747187212176,0.08249655561570789,0.07545861848187274,0.08100196680506463,0.07993251201197094,0.07896898393652182,0.07286877003187557,0.06738749462965989,0.08129087021464951,0.08322131874081291,0.07880312097141359,0.08024214685138442,0.07180757944330488,0.08751357598596757,0.08150272882947703,0.07542156497549479,0.05930529710331103,0.08159284827353576,0.08485400738921975,0.07608700720402792,0.07632814709814095,0.08106709575562122,0.07426051801700204,0.07533351126689733,0.07606083913203011,0.08264999521422017,0.08488069732835679,0.05926277175764053,0.0793458460385793,0.06956794426383102,0.08465555317764864,0.07953742585507714,0.07657523609107272,0.06754406967004174,0.07203173160650575,0.08489972560144893,0.07395237787788898,0.08303913605334315,0.0757419106892335,0.08435794871099705,0.07115806316785339,0.07732244492387441,0.07323993893613759,0.07293860958229564,0.07214950482416375,0.07320014931275631,0.060976082177089413,0.06665883857894284,0.06139196719931903,0.09078556899900489,0.0798831989879593,0.09078678990240494,0.07247176055699592,0.09103091625089264,0.07813352486534542,0.07015951053971474,0.08959772575617464,0.06999143678250447,0.06986733017673286,0.07897200754579473,0.0747829554860344,0.06322523593252236,0.09493130702728762,0.07526100796935448,0.0782282588629187,0.07864334221660024,0.07436138543877108,0.07941371322284024,0.0856259514961408,0.06946712780297576,0.07324934716300265,0.08678200677934145,0.07999673195161575,0.07614771795390049,0.06591512110705386,0.06588925435897727,0.07229623011424953,0.07990524066142951,0.08366305872091179,0.056481356941114225,0.07525525390062315,0.08337200152696445,0.07642448212840489,0.08212034068796237,0.07999091888945611,0.07373027923586997,0.07392210944226466,0.08786843675086921,0.09013885979040447,0.08188265496835465,0.06967149763508806,0.08318513827485821,0.07227178860903442,0.0874026138100665,0.07442295167292745,0.0768576916807173,0.05932161000718532,0.07647772494982027,0.08729779218207978,0.05933183901918528,0.06804603974073502,0.06832806915655112,0.08218254949917911,0.07343038027699746,0.0794612104548287,0.07631489512143094,0.0719517023230069,0.06904760343837235,0.07873070491771669,0.07388889795288155,0.08562774164707357,0.06848999460310524,0.06734051328557236,0.08017829329358506,0.08888639039616816,0.07362798962830726,0.08302234269155301,0.07090467193963326,0.08167944713937772,0.08189428648463355,0.08453338378927332,0.07933139355634933,0.08080911288974127,0.07266982276635352,0.08900217121942061,0.07171010057611303,0.08293458023261621,0.07424061209878849,0.07535075387073548,0.06014641698181101,0.07585951137061318,0.07449579798186812,0.08140331407572739,0.08506354663698035,0.08283908300998682,0.09202321714066997,0.07873379506694683,0.07224389929956285,0.08383467992706534,0.08354826742482802,0.0858901168560065,0.07208770014799562,0.07717931783333147,0.06995833445411528,0.07625061402805328,0.06697397720157654,0.07670715952802454,0.05106315497354347,0.07788310983396224,0.07354838745867287,0.08140978307896227,0.08100921902617433,0.07875747741647468,0.058866370164598995,0.06764418602387143,0.08020497435743795,0.08385713356568415,0.0797902676488354,0.0719397979568974,0.08440491544188708,0.08410305054013464,0.07326112567568002,0.07329283638466733,0.08487657818165542,0.06881157284737786,0.06745555786514525,0.07849490913662485,0.06071866359511369,0.0862336888577669,0.06860253672335848,0.08900876332591023,0.08433265251286824,0.08859398976701983,0.07720012929049039,0.0857623823282606,0.08783349043457978,0.07975607913482903,0.07215239872124453,0.08205771601663175,0.07533413406313064,0.0803729496359194,0.0811632997544428,0.0653896168095118,0.06862646197358517,0.08429651754587558,0.07213324975738794,0.07382264302121791,0.07518933201340855,0.06417724054041592,0.06012841634339675,0.07623557297567543,0.0728309692226859,0.08949077279569408,0.08899255157354741,0.06927297793528009,0.0668728039837263,0.08321958561766674,0.07732279600567513,0.08060034773155483,0.05545844765014753,0.07988117824626018,0.07455779691926198,0.08839524693514912,0.08496390577714547,0.06768313345447276,0.07516686968269883,0.08579722641499202,0.06857051882478066,0.06742457819376735,0.08386930142680527,0.0791188831530668,0.08611050648559912,0.0632829954595267,0.08256259482400505,0.07065858489972132,0.06714509330321239,0.08957862889959187,0.08047500802344618,0.08996085484310043,0.08430943219361578,0.08812082646213923,0.0679393117979481,0.08568273031616835,0.08626573100105152,0.06490570224797779,0.0841864481678489,0.08772062232707262,0.07199595951464902,0.06647150183367212,0.07484329310308643,0.06434998630534322,0.07073601582835809,0.07516520475785518,0.061938783497609236,0.06949539452654813,0.07974479569026874,0.08014799324988144,0.08149951254155412,0.06487297231615131,0.07821087799724044,0.07642577142688863,0.06972463835425734,0.07525216796936687,0.09017697708552852,0.0721457000339768,0.081240762949138,0.08236902413587122,0.07612888544800227,0.06175153684759079,0.07499934961801993,0.08117580491254568,0.08662028574334171,0.07935343813097682,0.09354299649839563,0.08492360569656957,0.08179615235523191,0.07536205973871402,0.08675871738346505,0.0705914760921961,0.07851371686233316,0.09167172461703588,0.06952874329811398,0.07238247864427966,0.07522490701518339,0.0859606654126765,0.06288287473280484,0.0664936364039827,0.06290568489251996,0.08303914757831918,0.08447514952446791,0.06472735867139504,0.061418007752163044,0.0793349897208661,0.086809511452062,0.07362606340184924,0.06311152304243117,0.08960542223049525,0.07416047935729851,0.08706430826513774,0.07053403449019569,0.06792862531959726,0.06290457014107625,0.08258130888710563,0.06444967529762476,0.07729121471793726,0.06842862909587802,0.06688986300902927,0.07235395292707147,0.06868805064883138,0.08154010615165833,0.08247131703635538,0.06484751213891066,0.08810762970017835,0.06867411625560793,0.07678476022433114,0.07761549345294741,0.07971857076586158,0.07958632458653389,0.08187549353153395,0.08474155516714339,0.08245243789398261,0.06118696159511977,0.08523995462588593,0.08390433661336243,0.07927545621874976,0.078242171068207,0.07632273990211252,0.08286304587139601,0.056298112118968006,0.06536814185957965,0.06928375937592249,0.0840905634120905,0.07906203721202226,0.06989802108456167,0.07817425639934794,0.06001215626458558,0.09047308316422528,0.07986740132318726,0.07625162502630314,0.07675156461153854,0.08631209943991204,0.07232013406645013,0.07372940057858088,0.07907938674522702,0.07391128715677964,0.07079804132565029,0.06982063515943432,0.08523224888834391,0.06713346485767381,0.07839633042177219,0.09131076264317098,0.0875935060734094,0.08132455955908136,0.07669801142210986,0.07692591043380959,0.06888937746983788,0.07564832461356014,0.07314085401037328,0.06793575626687146,0.0653941838630155,0.06379076171037298,0.08560183828442368,0.09026564634505978,0.0759183536152772,0.07930717441653276,0.06808770773584674,0.07538223032087513,0.08304042610699147,0.0765923431766809,0.08008609815005846,0.08412085525821417,0.07193698150059444,0.07560440440174106,0.0954936384155402,0.07301829431948934,0.07903175901386254,0.07523597069105095,0.07428845356342664,0.07582670197199096,0.09149909201734312,0.07674590502410861,0.08331607083661673,0.0631455768286828,0.09339114902301215,0.08579277216659295,0.060590061467045694,0.07370089957720578,0.08365368972246354,0.0800061833996508,0.08177438681551653,0.06044391347854233,0.08336115566632034,0.0661582223516021,0.07473980506882266,0.07503229569569474,0.07157100330607083,0.0807267763166243,0.08822760223416443,0.07533821996699075,0.09085563237654523,0.07397288426034647,0.08050865337016683,0.07899738304559902,0.06964264715694922,0.08901431717267907,0.08343679487641076,0.09011163295203885,0.07897518167996362,0.06392977059302513,0.08560237606120619,0.08439154812682839,0.07580630250963291,0.07676239956081193,0.07335816780810757,0.0853096175549439,0.07564881834497135,0.0694359158009007,0.08079990368746205,0.0856029408971832,0.06455883035480697,0.07632559107464125,0.08311618528860487,0.07563965849075238,0.07066579310748367,0.0659959542644469,0.08640629981337708,0.08804364452999158,0.08473336390709402,0.08566293797381994,0.08560146639067884,0.07594632936938914,0.09286430250372602,0.08692422561923743,0.08499394969155372,0.08573549205523695,0.061991894053810004,0.07976465338499476,0.07390226432831215,0.08388488334410106,0.07066921743322141,0.07717118694783023,0.07580524191587457,0.06429695094388813,0.07571247139723902,0.0733768009644861,0.06875829278933689,0.07804367657264434,0.08292400555216334,0.0814643436819781,0.07784768920102975,0.07823588080942764,0.07904563391959349,0.07585306875641132,0.07801330473517301,0.08290729895113785,0.05605597798317789,0.07209756630579661,0.08502318511262105,0.0661576552411014,0.07799585002797811,0.07770004051052636,0.05514889582222626,0.07322802348474225,0.07706523557392718,0.08218125943238574,0.07088056076259766,0.08670212077095829,0.08180286194271043,0.07506032246115918,0.0763834436918998,0.07700340414467108,0.06752689388212943,0.06437380344263299,0.06889186665019195,0.0713162534265816,0.08746303757014903,0.06217187470226574,0.08401058695638934,0.07510056550587342,0.09014789595845069,0.07744714431813135,0.08388112733300172,0.08409233904573665,0.08352410607289018,0.07462182814733034,0.07934570239411876,0.06726241066381163,0.08957089361169461,0.08508729608331891,0.082130693801832,0.07456155440601184,0.09226379559107595,0.0667630234806893,0.08945611910467524,0.07231936931753351,0.08060037574164805,0.07912230311607957,0.08007028911156094,0.07916966976695625,0.08161624971555365,0.08887367559573424,0.07652247808389485,0.09050798478967192,0.061414017941185485,0.09012957938726118,0.06627646950253342,0.07305354433098352,0.06043898796914306,0.07283225688937855,0.06632688487347155,0.06362774184371206,0.0737808555317862,0.0846297266881288,0.08877524527020933,0.06596234683400795,0.08236839743997337,0.06176221931538944,0.0877296485533203,0.07380515405977008,0.07911454406285698,0.06821711569615352,0.08652606433641587,0.06886198842779502,0.07986018743044551,0.08555531179955808,0.08154756694512347,0.07996373181237176,0.06743229570476972,0.07550629605169713,0.06215817105624836,0.07456701521556153,0.08117036917984942,0.0919750847652267,0.07905910840616863,0.0803163004763329,0.07056024600621433,0.07435748334591108,0.07634951146001484,0.0851905043532521,0.08368155920172564,0.07105050319290604,0.08597932695992833,0.07249910018999087,0.07648303510169413,0.07125445982766748,0.07549818586114533,0.08033227283426103,0.0871780402399545,0.08323689923931701,0.08028252383371265,0.07765070709407901,0.06675260620883727,0.07189444537106665,0.0838210670407242,0.08085759618319027,0.06765954394343693,0.0763434966115252,0.08187657279414028,0.08276271595481227,0.08116059038917804,0.07959402163502671,0.07479638387175302,0.07327556414302339,0.07717775506589956,0.06867366542849285,0.07663594054164083,0.09055749767294624,0.06873974148168402,0.08698593087301681,0.08425437352187187,0.08220679028819566,0.07442020066991473,0.07931059865581466,0.08724017276423116,0.06714064997062394,0.08524566161666629,0.06325976235099015,0.0783094687054577,0.07723260634699303,0.08415355181890671,0.08746126998648253,0.07904394445411048,0.09095834769724682,0.07107479311114312,0.07334206657010649,0.08105247062010273,0.08702851260517036,0.08066748406068905,0.08625418044158717,0.0846810441193053,0.07234576772993473,0.08801758057740061,0.0803051424752303,0.09091775037700317,0.08023141267794086,0.09091311263652269,0.08431651336038097,0.07881369217863086,0.07395601382180464,0.09055920825606445,0.06842870712838192,0.09096813669655071,0.06719497672267186,0.08349051675734431,0.07193049173881554,0.07391267057504555,0.06660793123359218,0.07277241932685843,0.08635528602326958,0.09542784044814237,0.08684454949886883,0.08384964737541711,0.08847743594944138,0.0708100274861396,0.060844107616968804,0.07020627650622548,0.08479421028656414,0.07945068844562857,0.07039501778351398,0.07649424215693802,0.08423005393729378,0.06675685159201368,0.0636046489723456,0.07986070307506334,0.07870196210696265,0.06990215425932347,0.0808301399792061,0.07842330691872065,0.06535489985115908,0.07568957503375727,0.07655186141572703,0.06499200736174773,0.07902442150793142,0.07663803334917033,0.06896102953392562,0.07145260452588785,0.0726350405990467,0.06906631990700504,0.07346415490663763,0.07318514815027424,0.07676862695245132,0.07349861757320258,0.08196843688014528,0.07716048651144472,0.055895319022796,0.09221533876037599,0.07815020801326097,0.07645309606162413,0.08268869544827587,0.07999244191559604,0.07128554977340802,0.0844309443336605,0.07657691426064565,0.07702972955814952,0.07480651695426509,0.08963786317261505,0.06651760789040963,0.07476191602462984,0.08368994459394118,0.07461825526527333,0.07537533196809981,0.06853897954897677,0.08093816256155681,0.07495011233250545,0.08611316628528455,0.08026067548601315,0.070333425551575,0.08456195594956913,0.07126743962439187,0.06879914565871237,0.07580188044512698,0.07338188684399685,0.07232605998115496,0.07605244664034944,0.0912366195767122,0.08508692359911206,0.08961848469243874,0.07917881100580543,0.0780758146826993,0.08417274331053864,0.07190258329839766,0.08228964327607145,0.0886957163397945,0.08632396874140812,0.07368041153012167,0.08228368609721098,0.07535188970294279,0.08058439780297233,0.07837425386769456,0.061328425425842714,0.08141255094598174,0.07621607722182258,0.07885752960423173,0.06616562825376729,0.08351040152396466,0.06461848954640897,0.05805019797235093,0.0806050725114921,0.08072833291543142,0.06750779712462443,0.08980787202284621,0.059118358815068064,0.07157822523306102,0.06550020553439018,0.08054229324380166,0.08613792668099933,0.08870976608278922,0.06796655474807017,0.07520840843622165,0.06284348000812492,0.06773675114307881,0.08095931052399317,0.07205135986794355,0.08189682126765183,0.08161176819965935,0.0826427234612016,0.05966272918829184,0.07499849152747098,0.07271306574063932,0.0729746173280448,0.08435748399558723,0.05449607618659754,0.07250909289433452,0.08533739270233742,0.08248418896865986,0.092104295339657,0.08048621461753636,0.08065014050381389,0.08288049782816709,0.07470900381755582,0.08156815401745145,0.06744204557854627,0.08118098185664546,0.07578984267386304,0.07294898593370067,0.07176540958644453,0.08486549426284248,0.0882476560577039,0.0874677697863506,0.06443289770594576,0.06884250035024875,0.07911587878408521,0.07089607322659443,0.05751908934565195,0.07572960696485587,0.08362334367174515,0.06956218320566522,0.0818285633719126,0.07189868331115559,0.07360908670548841,0.07608106573979419,0.06548945158705241,0.08017454708284943,0.09435907580982268,0.09008285266950614,0.0883027244592321,0.08429156675395312,0.08409441534080261,0.08685900448356541,0.08097045657907143,0.07403573778000559,0.06958973664451015,0.09040994884543224,0.06252162022907726,0.06215542931231859,0.08604477513942116,0.06717339120084105,0.07358373181317494,0.05529268451694101,0.07031994004907025,0.08206395788477826,0.07914374021176118,0.0788289724197692,0.07344038567761965,0.07695556792654347,0.06886118789987025,0.07535048457403509,0.0775227011466746,0.08507367839756733,0.07442853408608516,0.09058944563512047,0.08349845975388083,0.06633700029336445,0.083813468021845,0.08260071888015665,0.07132672973552191,0.06868998845340538,0.08271593912699693,0.07168593084228789,0.09107633937022536,0.07835047834338706,0.0763095685539491,0.08558519932145603,0.08026936375168375,0.06335575805458799,0.07540678737840562,0.07372042195173906,0.0789009251360728,0.07755279531814646,0.08414939242253502,0.07142004932498883,0.09061412808986267,0.07058910826700715,0.07747692682412416,0.08151916887109328,0.07883127664793188,0.08312686361796946,0.08460684052970982,0.0791980548584294,0.08311087436366506,0.0732313377671237,0.07453819222249444,0.05726031687369774,0.05541222459663957,0.08580513726705247,0.08373169640416207,0.07852865229360384,0.0700271733988035,0.07522614569452907,0.08867220024221946,0.07035008070390339,0.07032891827940126,0.0717384175442653,0.0712089049845413,0.08552136170591525,0.0664771348262937,0.0788005058863392,0.06912171144393511,0.05923062464158302,0.09093805631445584,0.07950601883115305,0.07532960520667421,0.07258594802793124,0.08810151358451487,0.08647300454043688,0.09002278662812954,0.08088874394741022,0.07011868424947743,0.08343242823665106,0.08416324469655428,0.08146093629984784,0.08484633216009604,0.06979035168734463,0.06695968750257408,0.08098775079310666,0.08174882681830177,0.07923149226763135,0.08247096932410483,0.08961724741120256,0.08246839319317725,0.08263799177564302,0.0676817923184071,0.07891852083006341,0.073488610479097,0.06971494707856644,0.06975052395623021,0.08439457957257337,0.07776349637929166,0.06358251195861558,0.05738308524453602,0.07562339868512497,0.09302703294591386,0.06709796956133679,0.07434484293503028,0.0770102804952166,0.08008837593500188,0.08200692222528741,0.07652709488093454,0.09053518193695281,0.08336779712327905,0.07861715231039205,0.07001331697651109,0.07967288005056694,0.07550881499224199,0.08147227166778924,0.07712758609347875,0.07161935471908612,0.07679965538497607,0.06588697828879117,0.05794709225254627,0.081324272450253,0.07806447541545379,0.07142472205586542,0.0751062183681536,0.08740269401212614,0.06736383861171455,0.07036051047381556,0.08165599047071584,0.07948330094235109,0.079044201084787,0.08230745070888432,0.08035909737678226,0.07258398556283163,0.08536299192089823,0.07279915813834703,0.06888382635574039,0.07508239339880152,0.07325678332000839,0.0886161720189082,0.0795752932047458,0.06966421617761795,0.06710370611757033,0.08201201471842691,0.06997935049438936,0.0738852902036962,0.07885332293139807,0.08023506929253899,0.07309683639819747,0.0735536527139615,0.07519944698649125,0.08209603951350625,0.0680347501976907,0.07431813552435286,0.06921827619827642,0.09207794556003648,0.054797725174068654,0.06193727185094542,0.07121969290353335,0.07581744937104083,0.06875456864535262,0.08211619847806587,0.09135039629332135,0.0890504206036214,0.08643915561197069,0.095294082454101,0.0757730933701498,0.08441541991692803,0.07341620841263263,0.08047534039751605,0.07867638218772488,0.07810852737257243,0.0768005524290023,0.08733516752197007,0.0905803212701319,0.0795009680059344,0.07222773766003801,0.08011195915605387,0.0710757672226775,0.07864642620673631,0.08991308186990543,0.082997927886368,0.08125056167768453,0.08398087795811812,0.07176783502808781,0.07487826589549054,0.07471011898189844,0.07356016286896155,0.07867884115383088,0.07107976937813165,0.07455593081610531,0.07020240914185721,0.08635532198252885,0.055018268140002206,0.0684157928591739,0.07393851818320663,0.07561357820964555,0.07788310924207337,0.07651143153863252,0.0919634020554938,0.07261566251782386,0.07681620911302495,0.07813472960706083,0.06718822632298004,0.07445389881810019,0.0777226926620544,0.0848509010495977,0.07946877090524919,0.07830251920807162,0.08605296106672934,0.07856349880092145,0.0915185114379018,0.07776645787184047,0.07169319412986999,0.06900000569628087,0.06913537145042731,0.07571099834863573,0.07270257389103943,0.08065199870029431,0.06779837323822885,0.0875601805566527,0.07453635665012359,0.08605853844073627,0.07381228456245417,0.08027534993940325,0.07590963102199946,0.08805831262022992,0.06965798366337447,0.07936992103817186,0.08523638721429258,0.08478800032835543,0.08420541115806904,0.08790399426554808,0.07858168243575046,0.06961487225191687,0.061926146261862595,0.0895387228841398,0.07525594219425517,0.07858687425404381,0.07255292413270019,0.07469721601106706,0.0789091119994399,0.0784127494669845,0.08095588325302713,0.06797185737309966,0.08308553778505741,0.08038406251007768,0.06345327788154387,0.0856852288003423,0.06469474253177437,0.06318688013842834,0.07532008975670876,0.08061069542704469,0.08117303246773647,0.07693440639290784,0.07703937824204618,0.08191211708321744,0.06582681283362746,0.07260504471488094,0.07736007774056904,0.07118967422827119,0.05842488291894106,0.08603295410256759,0.0890486593194818,0.06576121939929219,0.07076092911131006,0.07651166164637718,0.08271462908289923,0.08303553669353222,0.06380363050718615,0.0845170342481403,0.09260298874746851,0.0801124013728117,0.0718758402094419,0.07880732437442386,0.0697302177185178,0.06804436577173405,0.06865640814351975,0.07411823133612458,0.07927812718599081,0.0705699114381235,0.08826950582665463,0.0777604788818343,0.07486591526307607,0.08643815309622026,0.0708654498010578,0.06298512097562514,0.08073744150748496,0.06920559375134713,0.07658144930819073,0.09160596932888226,0.07208941633692989,0.07273582534261974,0.07820591276840264,0.062121426413470016,0.07262876878855182,0.06932326868489325,0.06382752423157882,0.08846549686062619,0.07249747831698952,0.07654008339939619,0.08108471253700167,0.0718023768380803,0.0863121978762817,0.06423723458688452,0.06154262585679742,0.0857544771606072,0.07126243131320381,0.08972309077971166,0.07487244354507293,0.0718560967695077,0.07310936110743417,0.06633995670625911,0.060152990221787786,0.07129632177827029,0.07860376780003134,0.07452987910263023,0.07682228772674077,0.07156526332372001,0.08550342234810374,0.07281485912433011,0.0847116001221912,0.07845015684190088,0.07023153469405885,0.08176009689517072,0.07122931398900854,0.07875300800077467,0.07191268428001779,0.07947633701356369,0.07839027561845743,0.07292640452219241,0.08445694295152598,0.07670528856565487,0.07591709280093722,0.07577986712343247,0.07381068092317204,0.07154130610476632,0.07463981788703795,0.08522168043898568,0.08162843458698622,0.08139304781865582,0.08380911849052783,0.07758273991973319,0.0607849298654517,0.0755499080788,0.07235094293180797,0.08545445109412891,0.07339062120034198,0.06923886353672414,0.07348237461139098,0.07735172784518093,0.0802098262340692,0.07854753305395473,0.08778921544721165,0.06384232648423979,0.062782939581523,0.06725072773778758,0.07712536532755435,0.07312507216128211,0.07644554791337191,0.07338357676906052,0.08782756332696139,0.08251605649460127,0.08165882769847785,0.08592528629979634,0.06833035040275007,0.08106911290295332,0.06128983614797845,0.08781134578775485,0.07574298993369519,0.07732305497386036,0.066332418332483,0.08326583229745929,0.07213207064685959,0.0808743155113257,0.08381218504227163,0.060710057118969436,0.07389939056442113,0.06452270573434685,0.07621897118610196,0.08702888827924367,0.08092140141579231,0.08102298456632578,0.07878230568962304,0.0817236864249235,0.07006468656693532,0.08571735000758868,0.0821310015040971,0.08437272803388957,0.07232329926988729,0.07757862780246123,0.060710339291139455,0.08820569938700026,0.08296290000279904,0.05979398672017435,0.0935617200610131,0.0821492223100872,0.07832501111648946,0.07756228420557013,0.08319902818404637,0.07906755842942284,0.08296807890761032,0.08907312576626192,0.07057271921997647,0.06830188443783847,0.08211180357484361,0.08103865075109723,0.07750625776579168,0.08639649142807015,0.08613192650186022,0.08058938776042221,0.053068173144716776,0.07822498362107619,0.07819533073807644,0.06174828999647763,0.07612969407701439,0.08228577808891793,0.07269087944881092,0.07654246331745343,0.07758054653907753,0.08271076947355176,0.08338352802163031,0.08204264353021276,0.06800757863585442,0.0710735786204255,0.07495325618019574,0.08008823182890498,0.07470560995916337,0.0697229183951798,0.0788935154294628,0.09030476222720667,0.07491548508711365,0.061982988399786464,0.08481577595323075,0.08425237222479837,0.08467972166331358,0.08760648217761179,0.09260928192511371,0.0768268667143186,0.08317603949468444,0.06583935480920877,0.07185649174332251,0.07447271821439769,0.07302061288827541,0.08205634806323353,0.0614065249758134,0.07148948561385846,0.04512989973316061,0.07445348037138788,0.06669119188153427,0.0750394034122061,0.08376126605709208,0.09121889223078157,0.07167059228128744,0.07189720600288219,0.07014137110625374,0.07512812094712136,0.0627823544376698,0.07971016848116237,0.07351200829211937,0.08049089221997732,0.0625043368732523,0.07949817816815791,0.0936830384625511,0.08047493832517033,0.07568317458476213,0.07826467647890815,0.06811053831756539,0.09653507801370875,0.08231409946834414,0.07889658644590139,0.07892023738909432,0.06296710117336342,0.08166894813129091,0.066797382317963,0.06995936884324544,0.07044934470485881,0.09353628071693036,0.08773735094631269,0.06930819757581827,0.07789006821375061,0.07848982295223275,0.07058740816771245,0.0890317759653996,0.069102280708977,0.07873753230456773,0.08687618209874658,0.07567265944146387,0.07486737625300628,0.08500734478825248,0.07950622753013523,0.08661106929237536,0.07482010203091469,0.0797448883037902,0.07615988518850075,0.07282105728710841,0.08017576836054627,0.0815040852312818,0.07574845029634661,0.07774152683271615,0.08916084258695477,0.08873205083889302,0.05799223993905487,0.07796377389968837,0.08021730040906622,0.08719025025990265,0.08194313247723536,0.06857376371505207,0.07475191294233138,0.08627160903360434,0.07572568109899958,0.05835068618528058,0.08095868860434494,0.08534299255924277,0.06318121068384062,0.06219757984317845,0.0775173601779166,0.08411933324794829,0.06924180568287629,0.07638817536632533,0.07572989302916476,0.08885222590325817,0.08029409016339444,0.07845963168060097,0.08645646628712042,0.08079725748527085,0.06411064797009564,0.06545401290001523,0.08865404980701577,0.07151167547524384,0.07238302403292349,0.0739243387875303,0.07424542398569373,0.0732116614719453,0.0749080579082875,0.08289099578191278,0.062419405344488914,0.07814779461068172,0.07578548845695995,0.08393866094588383,0.0891026253903267,0.08779223104918493,0.07136425898986856,0.07958756009699458,0.07078068255002137,0.08502400960138179,0.08709116001683151,0.08876749036841608,0.06556921143547241,0.08686887551129249,0.07620366132908057,0.0798302507603873,0.09020451911154317,0.06833175028219554,0.07365946151925329,0.08425209770344114,0.07656079775500182,0.06378098658656252,0.07635779014529666,0.07609240056957974,0.08267954294811446,0.070476653191752,0.08091857855966222,0.07261393284227965,0.08396966560052735,0.06636816952537569,0.07626756740156572,0.06097111525369019,0.07689566626476173,0.06881237330487462,0.07794174308486411,0.0840781032193563,0.0692994911762968,0.08733253993129445,0.07875707304812815,0.07785441208231779,0.08262636159186314,0.05683054952075206,0.09081221739469061,0.08650516512592714,0.06333920119170403,0.08079695114734542,0.08093061146207775,0.08728265107880195,0.05445256664661342,0.07798397118270073,0.07386424829837909,0.08068250439324556,0.058483686294544895,0.05260889518681375,0.08824307937625071,0.08142536838212595,0.09419132116762419,0.07701850173245167,0.06542985009523233,0.08023998437061114,0.07601449913363624,0.08522205735712601,0.07449729974367797,0.06384305986964271,0.08013387162848952,0.06529061159213612,0.07911329578996779,0.07255413744456174,0.090914176668396,0.06277104540242501,0.07539101624079081,0.0657551641871438,0.07282928561085175,0.07711446801521457,0.0774078020465015,0.06819565546666964,0.07699956277464118,0.08539243985109413,0.07368159218157869,0.08692184329812222,0.06919747966412629,0.07926362530314371,0.09157922130447507,0.08514098289446045,0.08545131641065956,0.06862286960379775,0.07834238301106183,0.08548402823418945,0.07522044099889219,0.0698687170074398,0.07218296632903679,0.0832831354080045,0.07821479916871153,0.07929773848811336,0.0834606109809841,0.08736950257387538,0.07281545200888764,0.08571848160199888,0.0750008010364229,0.08005043540613441,0.08117144378597846,0.06787807073415857,0.05975562662579799,0.07520082619187257,0.09269224550593556,0.07248123367785105,0.06157360833964359,0.06818555242868601,0.07681932163163578,0.07514835841879855,0.07938250843067049,0.06849237523522556,0.07788839665823034,0.08049489666262762,0.06970796319968428,0.07867339758656569,0.07474862495048279,0.08428332349041842,0.08416468391746859,0.08328054541742293,0.07803453502861393,0.06989269606273904,0.07744386117323475,0.07334266269949147,0.08058036926282544,0.07721741296762472,0.06451954641010574,0.06596042366745893,0.08847807852696313,0.07811830418438803,0.08141732818175822,0.07271006565930097,0.08012375437644642,0.08170049510822394,0.07109586728457132,0.08244633002101401,0.07781888757235719,0.08811571438538546,0.08421370466354824,0.0740776691938048,0.08380706349492908,0.06682002022637361,0.08316195673462595,0.07073292409275843,0.08365428372078115,0.06510531481015808,0.08594937120253023,0.08090979349377558,0.07214204692942325,0.07761986153106865,0.08034563507860007,0.07769239998449817,0.07197899014484538,0.06981833128135936,0.07903466191413089,0.0851829390975553,0.07451520695525632,0.0786722849932785,0.055870452347469526,0.07945933187059842,0.07737535826375719,0.09060870788510463,0.04955887973986227,0.055492579539042027,0.08475475374350279,0.08097463370990157,0.0822195312980849,0.06958788275500577,0.055898800281695346,0.07193549988276306,0.08046442087710182,0.07546496428403568,0.06793576310563372,0.07010533572581154,0.08934663586902492,0.08211852802614023,0.09049925149214968,0.07110197211063657,0.07123970154638717,0.07057769189094149,0.07710452666433826,0.07409467585171253,0.0783849434280099,0.08077280595773495,0.08037664243361592,0.06980423565786194,0.0659345187425745,0.08990328718229496,0.08296334172349845,0.07803755402006193,0.08042283465284514,0.08662598093746543,0.06946281903334893,0.07910383536385504,0.0848279008102378,0.07881395967946542,0.07552087398631471,0.08591615861292701,0.07523672332409204,0.09216367458452811,0.06514252219942543,0.08041615156250648,0.07732652738399623,0.07237529597593968,0.08901582895333293,0.057155390774051344,0.08535719601338397,0.05900281420761594,0.09302394935077399,0.07692424626697389,0.08718481256673075,0.07958886031009191,0.07156618147169279,0.06302013722498737,0.07957833295612832,0.08355927327102605,0.08547034089811209,0.06844686653405387,0.058363760292876286,0.07287295274511119,0.07666670167125514,0.08597156415329012,0.07144155880000547,0.05284926837129866,0.08004156710973488,0.08977968409911108,0.062046169029835306,0.0684423205861655,0.0762949284648398,0.08784376579272818,0.07384430351858012,0.0917984515464566,0.0751913198132062,0.08395669731297228,0.07653171729824629,0.07330935711953468,0.06203538701818363,0.06414586348142641,0.06259215495537378,0.06421235367870451,0.06134003754045502,0.08007102320960666,0.08273315245218757,0.07883893499537398,0.0795192979551491,0.09435694331257403,0.06262595443204441,0.060438870996124466,0.06317605133881508,0.08305299587916928,0.07619660432682981,0.0841760836777531,0.07920931916034193,0.08570783955534036,0.07653403234657308,0.07329915150887303,0.08306989308060694,0.07742405231774648,0.07814804489372212,0.0792695557553633,0.07868697679310557,0.07783129737760522,0.08408035840002887,0.08682922354095848,0.06737017869660905,0.07322533651336169,0.07578141331292162,0.0767360894911451,0.08666351599945582,0.08262830284972743,0.07263172247373706,0.0812205452925709,0.07512261985775096,0.05578006782006786,0.07073794409778472,0.07966597317760425,0.08404538426988409,0.07954476857932297,0.07947834068939268,0.09214398904595233,0.0796205915532115,0.07709136198054949,0.07452548687205553,0.07590084983618925,0.07889186701079692,0.0633518631989333,0.08365036714073674,0.08033587815248702,0.0708407915242962,0.06398938272576551,0.07696740136049207,0.06515052422523623,0.08480033558499614,0.09185725414735527,0.08228040738846017,0.07454549633851174,0.06244271576015864,0.08169691175245027,0.07490923905496188,0.07343210700823008,0.07093198680681484,0.07121945292552614,0.07732323615105416,0.08713159928308127,0.07792889117374059,0.0727482684827132,0.08501636688557776,0.07662540564905741,0.06794402157932966,0.07673837447745883,0.07012078661382415,0.07964360315626559,0.06412589972363128,0.06567222461721653,0.08653447054013166,0.06849069116113096,0.08297424489210409,0.08601289767511527,0.07453158277006519,0.0606231894193833,0.07691369164748445,0.08888829441039911,0.0842862480705317,0.08213337000100564,0.08210997277175493,0.07486517737171326,0.07669761806897116,0.08231879940731716,0.07667620297643964,0.06376069273821633,0.06787783690965618,0.08318949697468109,0.08153221730533684,0.08584553559110097,0.07041370451070172,0.07026825686945297,0.06065649183982249,0.08491972660251082,0.07625312432956421,0.08075157901979435,0.08968440710298944,0.08226115444167248,0.06300067868049065,0.07105222511741474,0.05008941951205972,0.07994457195646455,0.08333127673969812,0.09187966237928727,0.08983228563710276,0.07360343026870633,0.06827422501036794,0.07293823515606275,0.06794654560798022,0.07636556320888946,0.07112554923452946,0.05743627392228316,0.07803931835079767,0.08157065522311886,0.06231723612727834,0.07395531026414731,0.06665481543079466,0.0884419276758584,0.07390174624981444,0.07558884069170918,0.07073946482570956,0.0781945643067254,0.08581717823400031,0.06739482260896991,0.07239603101463858,0.07435135489227963,0.08736166801544541,0.07599891607589668,0.06667606626486226,0.06589227186303452,0.08747528412739164,0.08508913587704052,0.0761311513703786,0.09301769815162085,0.08671274527574771,0.0792897980977857,0.07371221993758666,0.06985981025239588,0.06799601486602566,0.08297725102475917,0.0679157340123708,0.0819626178838575,0.06236479561010371,0.08373002198609375,0.08173512340522948,0.08459410426582478,0.09059043170942091,0.07068237618378673,0.08229832475128188,0.07453110369723934,0.08557605149653347,0.0772344940333824,0.07656064950914898,0.07606231139547395,0.07163556305197476,0.08174424658869363,0.08468213194168384,0.08219554906192535,0.07627024066825172,0.08383415369919181,0.07053327121047523,0.07151304905534789,0.08161012340465129,0.0768535544780837,0.08110334430244234,0.07199155308170291,0.07360546188181259,0.08368767790728936,0.0732275664936102,0.08103150448280988,0.0743184663967325,0.07717497431562,0.07174578443099278,0.0730891366556773,0.08669960301959079,0.0801202453153482,0.07079362828889156,0.08658008603449042,0.0822935321896164,0.07416654728220579,0.0781775761780204,0.08670911704123076,0.08697487728671406,0.09796787268809744,0.08793412177348904,0.08192969007734441,0.07175162882474376,0.08030859897281034,0.09473513349795284,0.07464732404121904,0.08585834520537557,0.063120443791972,0.08068386024704566,0.05918466190105706,0.08231270243918476,0.09004779758647968,0.07502103126314547,0.09240184907508925,0.06945886818088864,0.06678780831617967,0.0699943644757308,0.08105938737775513,0.07887866713959604,0.0667658686887877,0.08186807846479993,0.07269871949694143,0.08249120431790039,0.08322049065415246,0.06624018068464724,0.06457474038075378,0.08657313200140271,0.08622986823633005,0.0684671486245172,0.07761931291099197,0.07440896817248142,0.07441581817757209,0.07051169725517743,0.07978099828279173,0.0695920179775761,0.07490511119992621,0.0701695602904826,0.08522449467409192,0.08670719840796182,0.06596431550381016,0.08446812472470061,0.07685800122397395,0.08217704153535643,0.07788777144860942,0.08637666530428945,0.07989555659093833,0.08507294057465523,0.08263571892503553,0.07740140909162041,0.07312913242915994,0.07923206116774817,0.06715773118146154,0.0772180456812831,0.0667591707160656,0.08095224510985047,0.08780209101682411,0.08507898079883428,0.07232187960705977,0.08013035202722443,0.07853752159435222,0.07011371015474631,0.08084544511603431,0.07502304584926592,0.08068650527669183,0.07892373218296773,0.0655329987670341,0.05622184298491144,0.07761754262976919,0.07308976959721653,0.07139576765279901,0.07552130156373395,0.07258982114071288,0.08099871280996394,0.06750460276551981,0.08598987641463973,0.07428723162699824,0.07421811653175614,0.0706110610567495,0.08446444339408442,0.06253924852634121,0.07577409072115894,0.0858680773623281,0.07629677136430302,0.08649371407365117,0.08266779871236093,0.06073832795709338,0.08364364547623997,0.05735509210174271,0.0752613602438972,0.08027529820955222,0.08460184511827179,0.07372132165838542,0.07462803613007434,0.07683247449834305,0.06411335455464978,0.07538320346837687,0.07644775375578812,0.0743277418127297,0.07828948048388978,0.06928922292312864,0.07079335335037396,0.07432270707651106,0.07646095593685193,0.07366478776622795,0.07749508142765152,0.09306596400051653,0.05947917118184562,0.07685979884779352,0.06419971003999356,0.08130683782109895,0.08629626285585579,0.069852297329028,0.07999453497896314,0.0836151993523209,0.08588304406239933,0.06687306025146102,0.06834691389451927,0.07220137636769383,0.08945450730610256,0.07125700697702891,0.08108883246350737,0.06785848346779999,0.08467552794885158,0.06918361758081959,0.07742786229851961,0.08023298278713341,0.0741772761512942,0.07672644794379778,0.06647054467846263,0.08334919975120145,0.08185042355929462,0.07008397229792282,0.08035455691546886,0.07802849976898703,0.0820095449507644,0.0584312638286387,0.08638588556627114,0.08167566843451285,0.07005325719497027,0.07383048315251328,0.08059342440694867,0.0708242820000066,0.0870870383200985,0.07228785035706783,0.07355359925261884,0.08862751711894594,0.07721265634517904,0.07445771211931795,0.0721797172018947,0.07287085645140129,0.07809358493667001,0.08802712589805928,0.08512388675883417,0.06280589326999467,0.07198033707066791,0.08017750945965223,0.06647748776624174,0.07738676733634578,0.0674908841747166,0.07759255560338381,0.0816882512693688,0.0734572836451016,0.08198342258400337,0.08733131225062701,0.08330397686578889,0.07995143469198956,0.060080596281469584,0.07784229464479908,0.08902304309407184,0.06436625079347934,0.07577415911739721,0.0745110389751577,0.08157360308602535,0.08251678234674145,0.06513229715423871,0.09202422254656988,0.09695504987263578,0.08096174720750816,0.08180464676845264,0.07602703586973006,0.07756984020770313,0.08346501843441897,0.09253697915707716,0.05468618325311986,0.07097268409907612,0.0715862783279865,0.0854213599811975,0.09050369686609858,0.07657766525554807,0.08571783783553098,0.08982081788784199,0.08316525539405675,0.08455284321943518,0.06975380816943774,0.07503885052578929,0.06213802869558834,0.07994326287104239,0.08167610637662974,0.0689941518420939,0.05908047255395262,0.06973992160425382,0.06852570378324889,0.08232323324972265,0.07082866393762863,0.07538025568327936,0.06652569150691322,0.07751762470500434,0.07654733919749095,0.08969773414702488,0.07983233487966712,0.07759696761547327,0.07913545388581679,0.07527919280086426,0.05784809367576844,0.07340285653373745,0.06504606569862348,0.07024002763087883,0.0872739939774069,0.07220983909895229,0.07449755676982267,0.08215872189823956,0.08061474061468128,0.0658206734843576,0.06885658175065322,0.06834257225397371,0.0714845856173181,0.07277946246553132,0.08262639841499941,0.07634591818829176,0.06914881627121076,0.06890580710363339,0.07149326196338239,0.08605618359809698,0.07902704599452472,0.09066871416934086,0.08448060600115598,0.08101230110488085,0.07827574013974001,0.05998532788455904,0.08897627634452732,0.0854957317045005,0.09260216719701149,0.05844122187866396,0.08311751051168957,0.07442879638758715,0.05404877516672039,0.0822882114825161,0.07685432745758597,0.0595848697412229,0.07040981643208188,0.08245937479790097,0.08793832374326385,0.0566835678843927,0.08864285146082568,0.08204212837578728,0.06616858431895077,0.05906351374113114,0.0813463476149037,0.08090581264541319,0.07405968175074643,0.06073384694063344,0.0592738867451689,0.07495584969550931,0.06641169303640442,0.08292853308617786,0.07162890330966122,0.07823689246015522,0.08327659536638231,0.06033439588473677,0.07707072422490066,0.08197591035761394,0.07347251396491061,0.07132291553234871,0.059266527606185906,0.0759248516177616,0.08370608866059753,0.08406709880541305,0.08743483186208201,0.07858556020077823,0.08179807305151651,0.0650810576729284,0.0817152919147843,0.09090508684923371,0.07094201908049891,0.07706327739101895,0.06988150668094949,0.0780461479469851,0.07541613451316968,0.08085359846203857,0.07008338725717118,0.08730381461225992,0.0879202662469821,0.0903174536841244,0.06018694576342498,0.0746751091681104,0.06495062478376812,0.06951172878050157,0.07446387492728732,0.06659609251611165,0.07732714996608506,0.07860132838234377,0.05734340834091846,0.07914093333216429,0.08961361373975679,0.0735492261984266,0.09083656519854154,0.07664188678980909,0.07708574121566211,0.08034758996338251,0.06235520425421512,0.0796830991742186,0.056914716403997936,0.0688286643411484,0.06890927414586948,0.08771815982454287,0.06711769046736768,0.0837419082260388,0.07297319958148712,0.07479750265512078,0.08371449477117285,0.07685005641852646,0.06726876673764308,0.08777418466725415,0.07496432790757032,0.08109882440036695,0.08757167930270965,0.08195432804358446,0.08852834039208513,0.0887913023877468,0.0781892802112081,0.05866275947970408,0.08225928254377421,0.07853171415698104,0.07689457362196998,0.057260546943189425,0.0748963865377903,0.07380220372376098,0.07666424455820622,0.06958956189076215,0.0828519712454347,0.06739530662042831,0.07601675413735864,0.08537858994563663,0.06775934525322752,0.07683841644608236,0.055145138636956983,0.08432971611827644,0.07954803033319191,0.07364182693665813,0.07817706698894633,0.06913871518592393,0.08049506772217807,0.08310868798691286,0.08294649656244452,0.0784423502930647,0.08522323031060003,0.08146853321752184,0.07018416415045509,0.07218361237852536,0.07933237932190801,0.07693782489524068,0.08478282187445985,0.07732791489813964,0.07263613836958499,0.08269954609017408,0.08317874373710615,0.07526365522668915,0.07892730634219608,0.06170771720380028,0.08284690990121267,0.06884910344989917,0.07219891899665234,0.06626924394319467,0.07396152129588793,0.08211537043708109,0.0611758915493206,0.07409011580078961,0.08009090428399498,0.0692054338476864,0.07615971669789386,0.06925917022265003,0.08088867055066526,0.086681956675801,0.07663056375776645,0.07167564183161229,0.07223992109704518,0.08782298322405566,0.07224241942406563,0.06017377690467467,0.06720235940618949,0.08622403901014813,0.07171805235152415,0.06912208495142341,0.0828276599404794,0.07060136710973283,0.07890825382772712,0.07624157150419983,0.06042765273524604,0.06332191200715084,0.07545177516238168,0.07437690275205401,0.09034740073008835,0.07276120579414384,0.0826876151704805,0.06805153699804578,0.06961682087476027,0.07120107382320004,0.08115604820754568,0.07618374750339363,0.08662137617923787,0.07395567399775332,0.07822646404522333,0.06841158405215422,0.07630439369839298,0.07221540126468552,0.0636969452819243,0.07020747414971093,0.07407420798774236,0.07242043203973345,0.07314912421640361,0.06796538037936167,0.07123023150963155,0.08292593256498823,0.06680918552296711,0.07219331354241153,0.08007828372208164,0.08302194467485059,0.07645714139083523,0.0803119063853215,0.09566648069552922,0.06513398117002636,0.07123663912943909,0.07187929702551854,0.07584787848153997,0.08731141109853036,0.08167913303723613,0.07571598212520927,0.08507001227255351,0.06579088830112935,0.07884653193037437,0.07645970794890389,0.08917531719535937,0.07020210703250834,0.06916868417180277,0.07579244489608869,0.0602505370562172,0.08485430555026693,0.06855027034648672,0.0770733403839646,0.08025958540490605,0.08313692058408624,0.06663488381829483,0.07864550131342993,0.08162591707058273,0.08450686188914938,0.08976579119121808,0.06588203624266986,0.0759243402640114,0.07865847025730158,0.07560593394594278,0.07167417503096679,0.09219790233827396,0.079996912628291,0.06767792291572094,0.07481122539976934,0.084655241721329,0.07233791152413603,0.08136454437249982,0.06527532091303746,0.08554900678022696,0.08423559923829924,0.07877729682623995,0.0712596449304709,0.0744586511027259,0.07687550125751316,0.06353520664397898,0.07296374254218127,0.09436128422195662,0.086111271597858,0.0833440189592005,0.08388292166597242,0.06268964668534018,0.09007711415409188,0.08256255781056097,0.08406064816509587,0.07983831607379245,0.059221252239663805,0.08854496007254078,0.07660183116292665,0.0817349991134188,0.07151254750106513,0.07266656349647378,0.08707264143776206,0.07514886692781336,0.0812401846572962,0.05963644847463781,0.07832670576702555,0.07300580776312707,0.06688703663605232,0.08420518112813197,0.06072723417716315,0.06651222392515159,0.0754296366035197,0.08529174877788628,0.06929765564146786,0.07917561668965878,0.09028068558717392,0.07197958218870698,0.06626498140949202,0.06662488257807858,0.0698322812880425,0.07711601307108841,0.07160489141326443,0.06972518697689692,0.07897527017255407,0.07446575236988671,0.07947399304953556,0.07723964873158157,0.06809630124350657,0.07274252077946125,0.06907448726733167,0.09133717881273076,0.0682546797698999,0.08090214165037965,0.06544596393590016,0.0726868618638431,0.06118575546617884,0.07241101991978047,0.068504000209589,0.06273142074349346,0.07137428142349728,0.07681177728843433,0.07342764596772319,0.06794895696919435,0.07134056866109043,0.08606465323014906,0.06749647280611298,0.0726101605324673,0.09067603111804656,0.08653106841302527,0.08307872684202142,0.07789722177867654,0.08962470558642224,0.06459153163657935,0.07104289458217877,0.09282808545026676,0.08694210328141122,0.07102213561196231,0.0891663099878962,0.0853259982705748,0.07518836894067062,0.08751332945052645,0.08869647008982127,0.08474987903421899,0.06484921796915899,0.0715653281662631,0.05526517276146226,0.06053221839749737,0.08044635482219942,0.07773076804872134,0.06867504290671742,0.08029614772462462,0.07080791802412395,0.06264554574766805,0.0820880416207054,0.06760946055661965,0.06814682723924087,0.06572184366666617,0.07784080921589598,0.07457811776664991,0.07926846154381124,0.0752859866080451,0.073684787896841,0.08055731278995235,0.08096968918238043,0.06073797490073064,0.08917275447291728,0.0563960524518407,0.06688133609929572,0.0894158154563551,0.0889923825691091,0.071608792759881,0.07900753036579848,0.0800636764190833,0.08219490067496882,0.07746898403460281,0.07695368524299766,0.07213364820318384,0.08601740344848464,0.0745261852826749,0.06832048426659652,0.06117385608939947,0.07379429846294594,0.08073065755752695,0.08581526198868379,0.07065948757785137,0.07059819749878998,0.07403935634127246,0.079529044829906,0.08543502387742537,0.05921274500058052,0.08768076052792945,0.08888547573328726,0.06972098485812564,0.07989685817630521,0.06420157728417694,0.07240452475499148,0.07159818319532148,0.07747855099740159,0.08089614460168956,0.058606980634576186,0.059410105744183234,0.07005129899549892,0.0870008572310304,0.07217990225389131,0.0816284428381708,0.05711757446419717,0.07209162757280867,0.07680307357151574,0.08250825517769778,0.07133916232600515,0.06746857916124877,0.08120909367530968,0.0795944287166751,0.08257310847819554,0.06754298393582153,0.08769931599336334,0.08272999015427171,0.08279021573873854,0.06311927891042873,0.08115205361067525,0.07245199484536935,0.08030652818003087,0.05886650583120932,0.06786656083206652,0.08195937291248198,0.08111525833807762,0.06730455998235527,0.07097935892665251,0.07383474914286857,0.07489739870312234,0.08209909306577817,0.06734785648545624,0.0757306614020885,0.08087886097560752,0.08025663258903643,0.06292999310629374,0.06640491099031032,0.06413861724791727,0.091714623630746,0.08959615373845554,0.08577529689062499,0.08041424328695995,0.06679875224614577,0.09007928153856898,0.08233403080254312,0.07397065097397457,0.07411753233723319,0.08229591718530806,0.08106292592891158,0.06634844480406571,0.06510959953243316,0.08406645733097984,0.05898053465117781,0.0791127603361674,0.08973267710555045,0.084966091138612,0.0775800426740594,0.07783653232732327,0.0804199107680921,0.07216774058556233,0.08355774755576857,0.0779394104736857,0.06769790339875115,0.07137093505023796,0.07614619418136526,0.08904739694268168,0.06761582728826301,0.07453700832903584,0.08102492747577951,0.07991360380631732,0.0791782432989454,0.0793349199863678,0.07940815851426344,0.07807937956877345,0.0887761265527923,0.08844668142896754,0.07129316119867507,0.07882753086400526,0.06854425887362789,0.06907676322442305,0.07959366607925064,0.07671374895913882,0.07066859081229473,0.07869450240038367,0.06575652429660897,0.07758072059939819,0.07430011811304979,0.07914014654319779,0.0640285971559019,0.07493477079191417,0.05921395618390416,0.0722704934870771,0.08293427464264581,0.07795998068991439,0.07410543880635909,0.07574840262559335,0.08277222324049971,0.07277869929159889,0.07937094686498014,0.08348956436577028,0.06421868568242203,0.07303146037250402,0.0766313018515751,0.0745721875660614,0.05989721584570215,0.07048527018682377,0.0719480957042922,0.06510283716491211,0.07666804622142584,0.08273303837557591,0.0817165731350197,0.07655859133049392,0.07768857301618926,0.07594278946224409,0.06975917490044348,0.06415451791516785,0.06865024037919314,0.08361748948401142,0.08141799564789032,0.059858694188470334,0.08378633241737501,0.09580323640184116,0.0688944539370843,0.07590252579156237,0.0827272536539263,0.08316910506673562,0.08412680940216842,0.07123965612509581,0.09060488558745375,0.07665868211928008,0.06617882112823485,0.06824957918548931,0.0826172060053518,0.08407525027190135,0.06688048813072503,0.08213764862469028,0.06359176050339929,0.08249895381429649,0.08145820598153039,0.07396792952350281,0.07556100162725438,0.08962204720189401,0.07046868637118754,0.06820834875609916,0.06833107591460238,0.06769608956789744,0.07981759469366412,0.06511971459710071,0.06575068511215795,0.07411751393266079,0.07749470636981207,0.058915330800250784,0.07706254821730096,0.07462307171330385,0.08386833211774697,0.07677295444138915,0.07914189082475796,0.086255530363131,0.06768556950764121,0.0865259915877632,0.07591600990606362,0.08388635255493901,0.07570164180683006,0.0802156334313642,0.07839004471764152,0.07493577727801864,0.08860073886613437,0.09451369745241908,0.08481669491233793,0.08256881237572763,0.08901646661903675,0.07605915120130763,0.07718206345629998,0.0644779459318785,0.08165938512773344,0.0674880630236793,0.0549189782651074,0.07634409443974335,0.07741324420220067,0.06013290154793392,0.08636384955132119,0.07568432513809356,0.08386738202213935,0.07730253465710463,0.0745821767228646,0.06496755596775905,0.07023727739085303,0.07208302984457851,0.0804527076056321,0.08725951313513719,0.0802273484720554,0.0720283804166976,0.081705259942592,0.08568334131067393,0.06943641661276007,0.07422068238316068,0.08153914028921583,0.08490349475912727,0.052357410645577085,0.0562430244588761,0.08678239245879049,0.07282411917318712,0.07387679753456558,0.07438555654430667,0.08738267783130482,0.0657865254359675,0.07516772917333733,0.07932687523133314,0.06679857357786964,0.06874670177442396,0.07341230930628637,0.0758645496808066,0.08882253424011731,0.07196758304683165,0.07411899055590568,0.08253824783560787,0.06682091572108723,0.056188094173875144,0.08612673628162615,0.08134007483855087,0.07502678680871339,0.06709106916911436,0.06649806594550373,0.06165630397374831,0.06461556114367656,0.08304183199933901,0.07495623093413875,0.08593509287562037,0.0759096165487827,0.08224011558046848,0.07250107960856492,0.08610399901455726,0.07702436537362806,0.06409418997031283,0.07435692222124943,0.08752257472341878,0.06813183301753771,0.06515075851781554,0.08017535362804257,0.07710781609715342,0.08127824188688126,0.0691605925991361,0.08337354477717054,0.06662541271676793,0.061144786736851364,0.08228968591745872,0.07803757678588585,0.08593507577262943,0.07647825672784016,0.08576664877626423,0.08413268993772487,0.08104978834295654,0.08440563375372412,0.0825469293657737,0.08100840774875581,0.07865990082173929,0.09468295133018577,0.08571969693123102,0.06061515976707535,0.07569487127007143,0.08797813934499146,0.0632471551062476,0.08209021195219551,0.06229771658572929,0.07726308604053961,0.08706242771297022,0.08388039187290772,0.08037793329923265,0.08517665725818999,0.07369049390872072,0.06389627491968908,0.08988853672653074,0.06579428089964917,0.07147361756897519,0.062174459381593536,0.07551113958848923,0.07520511567035063,0.07470394774922497,0.06878232294172687,0.08007688028799215,0.08370420390577246,0.08764885507772933,0.0693263788925487,0.060743165000542276,0.08330735259449992,0.0796756181048586,0.08048651891151085,0.08803748721066734,0.06697596425568952,0.07140676880801511,0.06917480047066413,0.07095180202872775,0.09590196246720703,0.06893370781732627,0.07979892516542797,0.07374971777852404,0.06499929843373349,0.08600850382508668,0.07455475657969748,0.09197852920128888,0.08349764563891593,0.07052669413891541,0.08005290297703059,0.07615471119300946,0.07700957238824588,0.07513263265256623,0.09303278592221237,0.08250058247799323,0.07552624468006658,0.081704696655192,0.0817437981787897,0.06769621289559981,0.08119085580439057,0.05590587861347432,0.06897491336069231,0.08686412070027992,0.05619556053423197,0.08263147647374355,0.07646423813588199,0.06723814496834402,0.07302017900827379,0.08112565848055683,0.06997441294422155,0.08187671833865486,0.08315935318443565,0.07137124202685802,0.07870771044973933,0.07361965335285017,0.0681142914250622,0.09554907124356257,0.08346820871534934,0.07350341329529483,0.08252039332865868,0.08480528975929316,0.06772734117367707,0.07649720048313438,0.08154943259310157,0.061479353251405064,0.08568332370145695,0.08626642712521035,0.07901718430426946,0.08074312893231435,0.07646207621344843,0.08125374303590113,0.0863397357207314,0.05952258392452699,0.07866249700406017,0.059289712319924026,0.07708155609513762,0.05572039879951325,0.0905463981553036,0.08392184526314665,0.07971943956987221,0.08701641728544002,0.07423480203861701,0.06236525820668528,0.06947230601775269,0.0691517636366617,0.07630725088714968,0.07572386157274108,0.0813585677429576,0.07760944818531514,0.07624972026554278,0.07759937632608754,0.06402236680024899,0.08519117484398799,0.06722908270165755,0.08478401639147423,0.06491308876202068,0.07675727269648795,0.06517363411269997,0.09306849597893603,0.08851647401895596,0.06617459474979054,0.07540031799420727,0.06755167580549881,0.07426499375918558,0.08210170389162622,0.08682443195638978,0.07418847516628527,0.07507795966528291,0.08196516812061311,0.06765677555046104,0.07495859535569335,0.08501701477414782,0.08431957314142888,0.09161022882272915,0.07782319825532083,0.05656330704478545,0.08036172472963363,0.08414649703808044,0.09224950897982147,0.06725675091833941,0.07462295640876537,0.0865381321673876,0.06291182195735737,0.0619401313191573,0.06807001588025532,0.08984638291377511,0.0751023220950823,0.0779831690103317,0.0719754959047013,0.07008082180827332,0.04652024159003688,0.0637911201388989,0.08941940177355459,0.08263817665979697,0.07422004875159513,0.08528474965055835,0.0868678743572325,0.09436133738849908,0.07057976541903874,0.07533382439774902,0.0907479267252071,0.06862489440530337,0.07576527296610377,0.07436351270073634,0.08971879312406515,0.08215866046763455,0.06725526363688797,0.05968671995290329,0.0844725911774073,0.08874357121822225,0.0799423361400746,0.08106917863804966,0.08163351669064207,0.06224536658675502,0.07373759962073215,0.08802044639619458,0.05900977415617313,0.08057712874576337,0.0739614762356712,0.051464283854986026,0.06792550592687469,0.08703015660644811,0.0779659402907667,0.07902326569208569,0.08493716775375443,0.06649368634362612,0.0758877694130021,0.06885934894240175,0.08539086230787274,0.08250783326418537,0.08673901838075958,0.08228152334938012,0.07478898525218558,0.0714769585126055,0.06705990545784529,0.09401866133020857,0.08163425952616288,0.06743691681288974,0.0727700330823797,0.0823314985899691,0.06676792257403927,0.0753953194476206,0.08064602921853326,0.07016712011513382,0.08168365563127361,0.07383352228580753,0.0842094999349277,0.07763506127100975,0.06994140367077836,0.07395137753042089,0.07451177309486216,0.06150171514636884,0.08483292565586692,0.07643566299512519,0.066498509750581,0.09185677656992416,0.08918091029625247,0.09036836462897335,0.07937082260888265,0.0836589614503564,0.07865186013878406,0.0682177034885153,0.07680331560778811,0.07898218622535444,0.06669581105969988,0.07863759800512785,0.08134714656537387,0.09197070733438772,0.08150566944603174,0.08200044778678942,0.07072036478971239,0.07282409074856558,0.09158746682900851,0.08813567747005757,0.0722979398361811,0.07719395244065745,0.07084945438681811,0.08374645699289704,0.08797165505786111,0.08171386831387137,0.07357895301079648,0.05486774872441441,0.0835207133563845,0.07801264176732445,0.07247165088237237,0.0806024031432976,0.06952296577737323,0.08567353389546357,0.08033386629391674,0.061386613815325425,0.07807778071070057,0.06062422194203039,0.09132215535628829,0.061173560970296165,0.09101811708802462,0.06601417562501465,0.0906249395121346,0.07937469747578527,0.07587999745506624,0.08342561216888894,0.07037859529025273,0.06720951271324582,0.08816630644919005,0.07138587153252784,0.06728054742382997,0.0684585479074198,0.0684743699882651,0.07580953647388973,0.06974002508523354,0.09245088101726284,0.07635938802523978,0.0688895078587611,0.08998915781928304,0.07073672601394128,0.07386408207265432,0.07168736365884144,0.07816780764686453,0.08245503592332329,0.08073648967868854,0.08287111419467522,0.08573254999536492,0.07301279977615698,0.08391414393314564,0.08443725744658037,0.07072653720800483,0.09132910210171252,0.06876664917077592,0.07029517890000854,0.07641628139945478,0.07119819082307258,0.07676389206987605,0.08156682897254373,0.07913647886895406,0.07183098718315806,0.08043287945474052,0.07697446764986499,0.08150847657538116,0.05386914188565707,0.0886726812105307,0.06480282054178721,0.06768500571412193,0.05780419586439567,0.07480097321446733,0.07497167647022496,0.07935390745537427,0.08283934781124366,0.08757496035686096,0.08115584882523064,0.08264187893980658,0.08919413892812353,0.07545910217546704,0.08561832855050838,0.07286329370099348,0.0890411663888678,0.06799582977969207,0.08225976719300607,0.07128676159924667,0.07597056070205056,0.06743572560152665,0.06660883339224302,0.08142901858999832,0.0655982957904989,0.06453922078827956,0.06345659419242533,0.0664796920478165,0.08280537301247602,0.0767406573850159,0.07479108101510956,0.07719221493557482,0.06439073493718675,0.08586365823826846,0.06302573640759175,0.07317512452026363,0.08246286047606975,0.07330649057707588,0.07120130947063102,0.08465835583715961,0.07463941672431602,0.08642790277515078,0.07556773241527787,0.08701400042753786,0.07673320503065993,0.0848585554266281,0.0776334551492252,0.07710677137071123,0.058451611425604996,0.07461472349811776,0.08506656965414358,0.06755643421840496,0.08696886764149064,0.08122713326109748,0.0856884451140383,0.07888412963798269,0.08227480758039646,0.0867033002239336,0.0866508165142736,0.08642261936603603,0.07422363036678714,0.07216065252917406,0.08462152376354137,0.08418051342142804,0.07425352773143981,0.08632225065707973,0.0791908675720478,0.07737299359896724,0.06352528437492361,0.07714954453311698,0.0640246575978716,0.07755872772604344,0.08716630336150244,0.07272016867632257,0.08056851385024648,0.07022765867463623,0.08062496012797749,0.07984099561169522,0.0777874128756588,0.07298164619121375,0.07673297975706081,0.08281460538869091,0.0748420679644172,0.06322169742684512,0.0623514875142869,0.09412483673053645,0.07430389128865159,0.06202069995672833,0.0792580826368714,0.07456704783559195,0.06892293453616077,0.08299723236683355,0.08147898887043248,0.07742544144508287,0.07641920301419049,0.08113262434449652,0.07514664492471848,0.07125329685934438,0.07859396549044291,0.06509214124974907,0.0799647512253504,0.07006698663641396,0.08314643536030888,0.07756205309090607,0.06671331270481962,0.06739595900611448,0.08044647560621454,0.07751393154651201,0.0804935432411672,0.08582154339141973,0.08353955453545728,0.07008464051880729,0.08223120819306856,0.06849225541543427,0.08431361897997651,0.08234945715140074,0.0674178058341819,0.08909095541749376,0.07499959036505999,0.08091538996049896,0.06376923274002895,0.07952939899902665,0.069471384698743,0.07254334151600242,0.07080941875305485,0.08880016813317679,0.07469968107228007,0.0737858744385588,0.07774144338663877,0.06410534302403048,0.08070094039494023,0.07284993472031401,0.08215044211972305,0.06984157452653461,0.07114821459706488,0.06680401623870483,0.08430745637496909,0.08307086428027047,0.07396034143012908,0.07146678775714749,0.0844895907397253,0.08237528111567224,0.07818776035752334,0.07230856256773036,0.08535395573464491,0.08183695718630507,0.08282841903689897,0.07278812063664547,0.08499011024253855,0.06839384280595667,0.07536446901603197,0.07838248560807251,0.07351759832404692,0.08650966713914654,0.07359461094040881,0.07860963448934737,0.07196941060311421,0.0613825155077953,0.06981787287762858,0.06573275273626064,0.0746293736719088,0.06958993981732747,0.08080379374756924,0.07776722259770456,0.08103267207641994,0.07538493999483463,0.07230441342009172,0.06395634845450937,0.0664846843554115,0.07039549919582652,0.07842195561379553,0.0824152310039281,0.08080832677789987,0.08719954188092706,0.07807579028075842,0.07700456406699466,0.08988661860277092,0.08445517738348571,0.09063736435730316,0.07183033024919151,0.07441024313147293,0.08690268495391157,0.06511140813461122,0.07767642553996053,0.08764301816569905,0.08453405430569144,0.08275122185401947,0.06888037797531935,0.0901963989542156,0.0641862667794286,0.06632245540139517,0.07272738032168488,0.07977026434518075,0.08881401138045604,0.07796323487850153,0.0773450540767314,0.07856398810690202,0.08743330609760767,0.07254138268573602,0.09283511649333359,0.07804924078139774,0.07946286184218096,0.0743396108648805,0.07861367322488855,0.07140989081715854,0.07874910959723405,0.07849064494447819,0.08422463740730353,0.05727376984925675,0.06997373093596479,0.0710257425844367,0.07773818959021254,0.08481330355365829,0.09538655839476713,0.07797905711960258,0.07558811149367106,0.0796857531528509,0.06587694263584147,0.08611006435238039,0.08771295909589315,0.08869480899855495,0.08068097475586986,0.08163373209663902,0.082861129016094,0.06109348089255727,0.06254432642136876,0.07242975742997856,0.08201420895317452,0.08855922591978832,0.07058909558340903,0.07995853936891739,0.07618293898425246,0.06037051261946173,0.08183067420609258,0.08902229703673017,0.07476911439790193,0.08171310689402705,0.08620852022440609,0.0813186038138134,0.07711673344647105,0.07328050370189114,0.07400731313545073,0.07385937220381927,0.06496496591657502,0.06572952030825839,0.07311155691383152,0.06673767866893246,0.06391743893832513,0.07258343987679428,0.0794542642839999,0.07385163202754502,0.07147551823398958,0.07403095566332243,0.0877726697004041,0.07345981834252796,0.07830913892972988,0.082770971783641,0.06890754304716691,0.06609920600894684,0.07303254413917992,0.08384386137638636,0.08180403594538721,0.08986998587205691,0.08415629163690458,0.07128492670518155,0.089528905139268,0.06445684432732816,0.06759405482633402,0.07273246081228378,0.0825340311546319,0.07298193093410296,0.05680407538673161,0.0822123122126715,0.08379111969369213,0.08414844383480356,0.07664572340121778,0.06662766878741147,0.07392064336073018,0.09190029064620212,0.07736399377098616,0.07164014013840433,0.07532022223590223,0.08645567718281622,0.06441826830911969,0.07525013118640936,0.0791299649255057,0.06690384079714803,0.07835283098285145,0.06928499265436933,0.07926430713764906,0.07884675011667326,0.06464240320511999,0.07643369207972416,0.08300102023219742,0.08418374749940813,0.07183951923818424,0.07109488849663612,0.07979188079267535,0.0797744723476024,0.09126877726748163,0.07177281289600604,0.08792810044449473,0.08826802958982116,0.0746817969532026,0.06658148699631444,0.06325112089794399,0.08073231606380568,0.07674590439120788,0.05811158461867301,0.08280412430087786,0.07991620190588715,0.06878932536600643,0.08498084793298784,0.06070280158570862,0.08017656663246774,0.08545057301105795,0.08880230907269368,0.08130904362263659,0.07001906146235236,0.07987006940948391,0.06034341727873416,0.07709465776845344,0.0741730364970559,0.07662953134630408,0.0838822113856155,0.08247017571961995,0.06183805990790517,0.06024302885364344,0.09073020325975296,0.08554473031617814,0.07429233517235426,0.0820122740724413,0.08072219975140904,0.0777210632266318,0.07792387065887785,0.08187401359723676,0.07902768278676883,0.07786036282490279,0.0872047607834148,0.06572080904786695,0.06720123621351315,0.08727998951176724,0.08528921240216304,0.07886721135860106,0.0675364110288285,0.08278889167788131,0.08734618019774348,0.08179595003968781,0.09287722619354251,0.07168329336976953,0.0589786338541194,0.08396984521120404,0.0858343096353105,0.08005254860330256,0.06384545600448668,0.09316294872384583,0.07452834556133153,0.08986270968222765,0.07323458353415604,0.05864689845742135,0.08129542510515339,0.08137135867556225,0.08383752556377444,0.08859785195380918,0.08299149849414861,0.07352506899329686,0.08881597268095173,0.06005586321851339,0.07392999797042984,0.08058521071832195,0.07910332219012259,0.06818568541067266,0.08947997410483424,0.08080654585119701,0.07779454034606435,0.07997455823803924,0.08694236913162934,0.0849843651834961,0.07583189650987451,0.08312395643846249,0.07870530863930814,0.07358973633376584,0.060617923142043936,0.08091324333662162,0.09365738098236338,0.07688510576140947,0.06558717413996969,0.07260880504754115,0.07671828856318776,0.06917289610644348,0.07768384109723162,0.08659732053933045,0.07389218235538629,0.08712648223114144,0.07491879858754871,0.08429221445519683,0.09094657173589463,0.0714184958337785,0.09515140676281163,0.06885722673216962,0.06972042246676464,0.06989147915835052,0.08443084036047835,0.08560296871053544,0.0835971762701673,0.07602230942716445,0.07068211631416696,0.09222868846855524,0.08459002482496321,0.0692930386535244,0.08793820547073213,0.08350638039454497,0.06382502560481679,0.0673076330623453,0.08279112330089637,0.07314089397528672,0.07247228500252392,0.08230859117082914,0.06809892707198434,0.08326168253005768,0.0775922049881306,0.08374433140063552,0.07233359929188872,0.08522071495880426,0.0738897801442857,0.08262432096987833,0.07810487574536634,0.060447892330477065,0.07635043417028844,0.0698216670342916,0.06652322613359815,0.08213233144082946,0.07832204182373823,0.0767934882448516,0.06650429294180288,0.0644838104021824,0.06515846669865921,0.08111369384097561,0.07545667848543133,0.08374943948769142,0.07841670674263346,0.0807923852347204,0.08488428528287556,0.08277123522260366,0.08170288421634067,0.07780518372782796,0.07491215878597418,0.0888021820074406,0.06485545511149757,0.08007100463346867,0.07784290010607364,0.08166138147783446,0.07695230447330734,0.06752467043109327,0.07031002002388773,0.06301257327862722,0.08509209929272567,0.06642246135193011,0.08629858746149807,0.08007050102996699,0.07854571820178041,0.06407340519035434,0.08066009370874841,0.07127659076398359,0.07482286757011458,0.07238127427965889,0.05789917304396425,0.04774396356255986,0.07902034935441372,0.07253497714120632,0.08289843976323405,0.06277594688221629,0.07326820990162797,0.07648663373186261,0.06408509879690916,0.06057704697111442,0.0652404974168101,0.08608575682381192,0.06790330389024696,0.08521957560900026,0.07671776078429511,0.09079399976720211,0.0917902509729166,0.07478278316415601,0.06551177321012669,0.08337072436172199,0.06184619896052,0.07972892945786281,0.08545163343648207,0.08849046652534,0.07195576262417998,0.08455698505052583,0.07286508078861267,0.06883655649186482,0.07320845845263711,0.07602468518194566,0.05747232822874586,0.08010610570648674,0.06220481489221072,0.08604901395531234,0.07185572660191912,0.05854016158737857,0.07414850428073204,0.07549647256466435,0.08679510083934287,0.06542798095801236,0.08646360753281129,0.07648278024062016,0.07877661142352786,0.06948827026929252,0.06734065139514217,0.06584112411304653,0.08522588082759991,0.07263408850891705,0.0741594149370868,0.07998503081981033,0.08140992220423951,0.07959713329818172,0.06837825449194389,0.07018381198595162,0.07535074052265141,0.0749949463139941,0.07482654671237265,0.07897892028895057,0.07303860831300937,0.0607840768319208,0.06408449824124544,0.07803855730164436,0.0825326435263203,0.06741239647215262,0.07482744892553284,0.08152293647921614,0.0751220683060047,0.08514051680601423,0.07851101199253482,0.07312262225216487,0.08378593045247291,0.0786930505511101,0.06060460107106784,0.07119955743900275,0.07514387637472478,0.0760352120440219,0.06242857642911811,0.07226086049583522,0.07238362453716271,0.08230382008108551,0.07409486834207901,0.08447821645072867,0.0824628124838255,0.08248390704511699,0.07432222870463684,0.06765603247036212,0.08010897554431712,0.06330739031873002,0.08149601881698423,0.0767390613023492,0.08409137076653789,0.055517860617428194,0.0703684367529606,0.07871002750738505,0.05494172743386595,0.07264211704761826,0.08423057932861251,0.07704515052215641,0.08242598726103079,0.08391439004371357,0.06772362950126334,0.07116178334587889,0.0729455050340277,0.07929932337210299,0.07902518184154383,0.07835178994202358,0.07666681696226296,0.07182760886435566,0.06052469996047813,0.07231209735761578,0.06971041688712662,0.0795715758459706,0.06341180026811874,0.07385744651750495,0.08675613048242475,0.09084328547107468,0.0853651619321012,0.06417053078453257,0.08288375943729348,0.0706884335314459,0.07787199070096909,0.08344638547761717,0.08471266317192225,0.0760481531616446,0.08595256810057508,0.08989131184528273,0.08313065772724035,0.062303364086944125,0.07112131738605071,0.07077155960838416,0.07753844170076321,0.07574699814708966,0.07547115726931852,0.0644315495929434,0.07591967723073227,0.07231560699703597,0.06397123589043602,0.07470623636083645,0.08786671656508002,0.07554956312982371,0.08110046005255295,0.07108025799464629,0.06416173693174576,0.076965376842174,0.05891861006610287,0.08238763724930195,0.08831575239769139,0.057717882586062856,0.06687528573296042,0.068424933662178,0.06951830774324713,0.06476563331965822,0.06913810948874197,0.090715744658139,0.07406410532788105,0.07773132922119391,0.08694145315360616,0.08891194719341822,0.07772148334216265,0.08911301413466283,0.07463014529527985,0.08684292556235582,0.08662472583464995,0.06293816913332936,0.07212055145891698,0.07385724516595157,0.07465724704020646,0.08092452158696516,0.07819388256584811,0.07663580349246947,0.05759308033104963,0.08127041454911571,0.06267585717043905,0.07171164013569785,0.09295263424601138,0.08709964220179811,0.08228851985035332,0.06714050912184476,0.07726518855537341,0.08817895526447425,0.07293741469912346,0.0753811788768683,0.07836653126828924,0.08487693602081509,0.07473430894372426,0.08429279680553455,0.06800957640780775,0.06784555964267625,0.07648048565254195,0.06781277925487207,0.07558843430700504,0.06053231782479828,0.07567381718406278,0.08745637911426446,0.07670137524692397,0.08182918809958863,0.0819113379596072,0.09093846909099107,0.07735963020060567,0.08490637926676642,0.0756553461142383,0.07027454891332957,0.08184093894798544,0.07249730918117221,0.06930731676157056,0.08585558403697677,0.08221609167840482,0.08006783914774887,0.08894704415593147,0.07988830180052536,0.06438399464412702,0.06890544569923009,0.08494137357804948,0.08112830957799617,0.07946201316887738,0.08999559219013656,0.0702463976199954,0.07748381736372256,0.08395839318578086,0.08315826690134212,0.0852814895201227,0.0886230837235761,0.07963749902885738,0.08375613647357882,0.06915043455947514,0.07671562040024055,0.06454653076080576,0.08364666695511563,0.07702458560848759,0.07533799153432158,0.06222241561324199,0.08383479773981287,0.07823458601962191,0.06460266493985291,0.07270488427359603,0.09284814907837712,0.07523691810462102,0.06937548669388319,0.0832727120252448,0.07624231113982327,0.0745030712078523,0.0919091681720639,0.07889220025796072,0.08772588784325135,0.07506076192908495,0.0894795553533555,0.07959270080353274,0.08475192902451475,0.08550672101595028,0.0703261961290414,0.06186341764588279,0.0814200996639215,0.07496466546352099,0.07468732347979565,0.07668368433275038,0.08738942232683206,0.07746640725545863,0.07226210355455331,0.0712247250824869,0.06341350984893976,0.0743269228305192,0.07153070754104188,0.06043540544811146,0.07478955451477828,0.0808649607135107,0.07727474230403814,0.0800400882362271,0.07855835946103247,0.0731210491753074,0.07199491344054829,0.08245801242435773,0.07716976838206323,0.07717365208475996,0.07392274404835392,0.08902138790250749,0.0681315105181407,0.07563035087670106,0.07548177232721505,0.07656651737011246,0.0840949222181652,0.08177439696176017,0.08160163258507983,0.07319432860756925,0.060474110772938766,0.07062498237470813,0.07189308039188384,0.07533590453183313,0.06603141196784601,0.08788602020490502,0.062374950747444924,0.060537393473062195,0.06544436982342913,0.07750222267023328,0.07507316607555156,0.06092288785856881,0.06681589224088352,0.08131509284757697,0.08365762193815726,0.07119263328447528,0.08011345601997891,0.08178513408883854,0.08528537718323545,0.065021733919433,0.08117439049559175,0.08714896941979988,0.08003723893628727,0.08162070507280522,0.08029447297809854,0.05955796607730046,0.07178142728346637,0.07297574234589436,0.06814925886249965,0.0825380230404727,0.07953843361937041,0.07704052118852003,0.07073638036865759,0.07608218461887581,0.08482700340953445,0.07510959920651866,0.06716053785030576,0.06436507423971397,0.0772099045995118,0.09360903406921847,0.07115837023847518,0.08666631182797896,0.07825209849795071,0.08070006580555922,0.07527643468202169,0.07189950118037498,0.07391266838964705,0.06221201551586765,0.0738314035699637,0.06735717265149824,0.06418409635747063,0.05975720202803507,0.08738732998957163,0.08442832770301573,0.07865283831839381,0.07672316137118018,0.09034910642369542,0.08924941166693928,0.08577537531471116,0.07330863546448198,0.07694280152695304,0.07989664925979878,0.06704908531687631,0.07588859880342302,0.0839682274065065,0.08050371461675715,0.08860725973033554,0.0726071823695765,0.08933598828084874,0.07991764636091928,0.07263613731042531,0.06963186403776733,0.09392737690915089,0.0784729745014944,0.08302826422759832,0.06325551868550795,0.07943268218119046,0.07459584079919392,0.06931293711685488,0.07608693397529291,0.06882254824263509,0.08628634918255157,0.07058037523114466,0.07983829153266803,0.08484126773578007,0.06722697609525766,0.07822417934998932,0.08447532937669382,0.0839496354175719,0.09221259092336553,0.07935333945585085,0.08137282665850236,0.0737842773195928,0.09057721960028817,0.06858639197840855,0.07956313010048431,0.07049934624871124,0.08513926555845344,0.0738492573096814,0.07933701471457427,0.07034512747437893,0.08479035489793241,0.07321569195695482,0.07711856218960055,0.0892603542145234,0.07984597450217651,0.06674855192343151,0.08669794540263462,0.07089074082016195,0.07234024813003306,0.0854137268175092,0.06923185461460454,0.07847175882516777,0.07959459878264619,0.08361809580561194,0.07516819719700915,0.08204320258947485,0.07876167438804113,0.07799975274948673,0.09242384116973204,0.059235955077002675,0.09647257039614332,0.08377657311184028,0.08442220103761916,0.06887627178252352,0.08459995102779005,0.06048266590855496,0.08719622090538207,0.08228149679913094,0.08138937188473414,0.07798387587702921,0.06114882507142009,0.07663179317112731,0.08364760978103111,0.09284730671974394,0.05164323837148435,0.08227805050723962,0.06655353096145705,0.068237800704086,0.08132920361419783,0.08031057780559282,0.08473868604894175,0.0789077095397844,0.07919765181326525,0.08174743300771296,0.07111015698019951,0.08207932102675826,0.07501402551368817,0.07151158873347874,0.06897827318714389,0.0887525389754364,0.0712942710623024,0.06303528379519688,0.06693848490966668,0.07018452252181234,0.06898831316602938,0.07261306526050884,0.08486954315378918,0.06337376918634752,0.06953884119748877,0.0741439787764364,0.0782816200891451,0.0867943080435663,0.07690801167079342,0.07721092896377046,0.07951271092533571,0.07711305036608283,0.09000445101553795,0.07834936436049833,0.07326841190623994,0.08899766440724854,0.07253620859849297,0.06082831639027155,0.054541361645956354,0.08805503465294003,0.07146947012964468,0.0734235845660991,0.08583553636466777,0.06609489554149015,0.08514321359672272,0.07444202280272758,0.0904749778013943,0.07546109113073399,0.08852233124535856,0.07711362573254764,0.07700827691990272,0.07652715964758539,0.08401612774936329,0.06436716985065771,0.06632451940294093,0.0851833124166321,0.0873566630043428,0.06914711629743697,0.08833143091332271,0.07528146079726111,0.07957465926075465,0.09032163235343732,0.09250072564655203,0.08360097061395708,0.07678868748182904,0.06873946692142618,0.06757532580798534,0.08171644130811456,0.06884784153717657,0.0785177047547582,0.053830038857182835,0.08033388868545342,0.0756960945840773,0.0686348382445569,0.08383739522711606,0.09123713694780285,0.08052342533961168,0.07811981087455294,0.07355799673861896,0.07221772627311987,0.08232927301059643,0.0703470579516786,0.08886997223447082,0.07995391880285983,0.06516628224880229,0.08615085543502092,0.06972346674971644,0.07143203888384456,0.06865389443673682,0.08034115129921214,0.07820498934136805,0.09544602543172823,0.0725029584058745,0.06928660503374767,0.08405392290263836,0.07416266538960514,0.08790174819476405,0.07135461310842704,0.08612743778546497,0.07304407071386756,0.0845977606213953,0.07779193375184879,0.08310001079282966,0.0804317410829176,0.07217425754950704,0.06137240741019151,0.08111201533099428,0.06966516658171329,0.07877007798574513,0.06559038852802129,0.07884938716668757,0.07390605355595456,0.07429191304528927,0.08329935753939063,0.06655924032403585,0.08210084243353266,0.09360988627550798,0.07433978880170755,0.08349447791777745,0.07268471362648202,0.07111227414506664,0.07140823689081653,0.0771645331440486,0.0873470018159519,0.08165876575627373,0.08149489849722254,0.08474848754304977,0.06672196737015682,0.08622636393610425,0.08639068395316563,0.08898867609860033,0.08216482741754723,0.07584550758023936,0.0761365928280737,0.0768140434621078,0.08877583829132918,0.09033520120658198,0.0726943779378582,0.08241393311136022,0.06759801597580237,0.08171261292510171,0.06405494139421604,0.07872305280514527,0.062395631433828966,0.08245218448354977,0.0779306187935539,0.07866258803643156,0.07274515195007422,0.07682124163729681,0.06663118475785619,0.08308797008727381,0.0622560791177852,0.06656674411158792,0.0835666328696699,0.07266856214623926,0.07767543267743793,0.06062110473135986,0.06282030105704148,0.067238102548147,0.07043672037443326,0.07943718593318318,0.05086666156465762,0.07998790080867962,0.09160839884876831,0.0861272313584765,0.08747363569717781,0.0760513176546688,0.07046219526312222,0.06530787439287124,0.06974405287109883,0.07206922582653694,0.06796321759136431,0.07711691983727892,0.0726697804249957,0.06540532421244658,0.0711761145858005,0.08375260186189304,0.07870730442032645,0.08958216218027157,0.07578131201378124,0.06632639900374053,0.08356752213853039,0.08100336040536699,0.0680758563539427,0.08515531501784163,0.08443167116717847,0.08272268027643771,0.08014580606788929,0.05997834053817916,0.06965947719768893,0.08440989344843988,0.07008957365172266,0.08925939569466697,0.07716369669984939,0.08808500531058683,0.06660216870477223,0.08778879865735181,0.07127529005892608,0.07077857688500566,0.08585316029626582,0.06830696465546333,0.07834963986032073,0.07655507383480374,0.07085080359069673,0.07400591871347534,0.07413505416001086,0.07763582164007687,0.08540824975215983,0.0696242945382004,0.06870906391856618,0.08398665454195842,0.06762294199905722,0.08263687170007802,0.08137692196436849,0.08014020173129688,0.07431946892631093,0.06477431157525876,0.08377076672296502,0.0834274163161379,0.07905539054154245,0.07081404829786596,0.0888283126220456,0.07140852515889831,0.07243035612385472,0.06857112582881084,0.07295158911764218,0.08194794471312683,0.08048338671332111,0.06890478391767044,0.09086492909189792,0.07597384070784355,0.07348626216760368,0.06141634138639824,0.08745208368314714,0.07875103629787389,0.0655376144233632,0.08947964623367405,0.07901560697319812,0.08654932673327793,0.09677566334144991,0.08043350797886648,0.07875608196317946,0.07591873548709789,0.06959398367057028,0.08317816275755191,0.07714815535318484,0.06994848054852473,0.07145967420323068,0.06934396996012404,0.07240504074450652,0.06626602880045356,0.08242970636755931,0.08291621851461924,0.07628577402479789,0.06961234118519347,0.07705378612694284,0.051348907464778645,0.06344112174009789,0.07467151674460175,0.07971730266889997,0.08611507764159748,0.07065293238999276,0.07724188242075522,0.07323056798589442,0.06168670637273315,0.08042031018027587,0.06440128333444667,0.08398359971411648,0.06680118275065486,0.06897385315877327,0.06965960036636665,0.07803113976286599,0.07856200946940617,0.06383034514710492,0.07368669795432808,0.08100843523875223,0.07868675723831013,0.07600108672859518,0.08127393322918972,0.08375721738111327,0.08167225698429298,0.07517284694535067,0.06049976806950764,0.06561840648249297,0.07321176832174217,0.07496808927992496,0.08844959628823032,0.07981363595981047,0.08350371969585273,0.08076517719043194,0.08408944754005714,0.05989381455378527,0.06425020744835999,0.07819778680987519,0.06581436511790537,0.08400727281153914,0.07868372376140814,0.05895667605026736,0.0705594380330089,0.08264358438090535,0.08521170664351638,0.07610066147300185,0.0822896341586436,0.07344320276193148,0.07471116832527211,0.06316794438585602,0.078070487448525,0.08107319922134865,0.06593767723918362,0.06788658954884694,0.0677981474710324,0.072939216039235,0.07531712626547163,0.07297596127964281,0.07998596213411703,0.07822133482220518,0.08370790663846245,0.08310049615532597,0.08142701680720124,0.08066740073595212,0.0786412909361005,0.061281200628781195,0.08621918234403238,0.0788634058073213,0.08456948021418288,0.07796961092356477,0.08180622494582258,0.07380761559434718,0.07289345152423103,0.07160153314987723,0.06810219770582478,0.08588352556266565,0.08316632035023158,0.06947114586222325,0.07635270960461189,0.06883946114098283,0.07990453139991233,0.06060982257246186,0.07441772435058137,0.06921732712668603,0.07930049750161838,0.06859473259434677,0.07461115153675549,0.08050574976155403,0.08823997920415527,0.07632188870252146,0.075210531867937,0.08096432626504466,0.07632894793302744,0.0856949384643416,0.07439625969271155,0.0829908982544298,0.08540814879978414,0.0765879143399155,0.06895725864733693,0.07204354314449651,0.08062500309696174,0.07367422758709082,0.07700288940459526,0.07055890439709002,0.08296186400323141,0.07075418277466744,0.07354629534486051,0.07278840713214252,0.07397659673944265,0.07949872358947181,0.07124987735452132,0.06789139487039884,0.07782680832984391,0.08130622406838778,0.06609093093615709,0.08061729892170835,0.07892879789794831,0.07189091988538208,0.09304426068824898,0.0735206475863126,0.07540732364534587,0.09275270709940704,0.07339983526415028,0.0897686600328482,0.08309620001991846,0.07361704457747127,0.08673249965981611,0.06856448352458837,0.0647125922037558,0.08660199929972726,0.0720931852507356,0.07933603836444963,0.07284523257148903,0.08682622910160775,0.08574413433973252,0.08409961861771187,0.07734312256774804,0.0812463729576663,0.08211596372737462,0.07659683917017798,0.08130807835174611,0.07257261684333831,0.08266880936500122,0.068265682495602,0.0802093146934856,0.06955638041196074,0.0775036088904459,0.06229750383958209,0.07572822532314766,0.07696045714230583,0.07330598061231419,0.08967972532335378,0.07279612402402716,0.06791192800904385,0.07428744854969187,0.07041895739028,0.07819349233256602,0.07701098174393153,0.08266632461871479,0.07742825959689412,0.07772277764747487,0.08674420746678402,0.06852468490139113,0.06994806131593188,0.08564055310654724,0.07846631705985765,0.07651312371841704,0.07210141626097634,0.07997188123421806,0.06651294995886353,0.07087823663017997,0.05099114001799684,0.07134639417178483,0.07251674284598245,0.06050040859216738,0.05222239398076217,0.08097462530144633,0.0850975284671098,0.07088702932365204,0.08636894301756631,0.0911444752403938,0.0666546887519777,0.07600789579067607,0.08111800168819726,0.07511385374361984,0.07043331459641795,0.08016303875620817,0.07091595888919208,0.06398957396609967,0.0699452645878656,0.07085583176051338,0.07931858808042111,0.07123686652545794,0.08228485734037208,0.06994833631606005,0.06890135381535459,0.08493166696003014,0.07698663325443168,0.06479014343268204,0.07797436307871414,0.073627339731339,0.08241793657875002,0.06884419597585151,0.07215801963363108,0.08380610704115318,0.07563688953506398,0.083559356670726,0.08863879380995837,0.08161101797354134,0.08541234748890328,0.08237236405645298,0.076854194910746,0.07610740602945207,0.08122653068474117,0.08133438497628788,0.0751934437321366,0.07210414935250847,0.07500439400411182,0.07931590739381761,0.08434141607131483,0.09192333467762379,0.07125977234811225,0.07017454488766275,0.057122475335421474,0.072850968171221,0.069188614743774,0.08029738769528019,0.08364392771323206,0.07781551441708992,0.08073459937973163,0.07327133419411822,0.08357786157480931,0.0833333602708319,0.08500012543929197,0.08143090840884246,0.09105184578668259,0.07835474312223323,0.08684835890964329,0.07335543819345282,0.06533215672245656,0.06192535074333265,0.09653385721087467,0.08589792192850672,0.07380573758366883,0.06063400222201208,0.08486445577244903,0.08292728285145548,0.06358501375523873,0.06813883194682283,0.08371968616919293,0.09307070861651012,0.0655509190305381,0.07201776003732985,0.07203789167003602,0.07652127333590081,0.06943527701968844,0.07148536649088466,0.07938421946616811,0.07683341663587243,0.07867666213959948,0.08756518064657096,0.072947671976128,0.08827055157926739,0.07779479028021319,0.06804533624343777,0.07104759765785323,0.06842891266697543,0.08249277720559864,0.07885747411598916,0.07698150658568995,0.0763909608603888,0.08109196101834074,0.05923663401528145,0.0865635068314109,0.08133780034972457,0.0719268051915871,0.06407670389217132,0.0763969048587784,0.07613753901427567,0.08371686991311923,0.07705141524384898,0.07611967637552422,0.0849206706027241,0.07334220555358108,0.07075137818758037,0.08480955681423571,0.06180743397866305,0.08266143928042914,0.08278175889767826,0.06483179024663,0.059858010779089144,0.06846477486040341,0.0777010853719704,0.0867355963683967,0.08351635804016042,0.08277697983880242,0.07932726102056151,0.08020923265934662,0.06923250637889822,0.06769214630257082,0.07726715540184617,0.0755122123289205,0.0900785958794969,0.07951351272763163,0.06736920314185588,0.08778638797597135,0.08434410643406379,0.0775941956965504,0.06762525240137064,0.07905735979087869,0.0759306679430233,0.08006169721262121,0.07903606131058284,0.0736296487969326,0.08642354434908264,0.09032250268476989,0.08171208506271124,0.06316218477144206,0.06822437992006825,0.07624784054623884,0.08783676117714005,0.07494764485146171,0.08436149224470293,0.08501402902659516,0.08441483480676892,0.07047865898905023,0.08322659659290652,0.08213504490814862,0.07629140192436651,0.08323401962154237,0.07982838687254959,0.08749396195802936,0.07942961506027128,0.0845599638198479,0.08880228613782168,0.08523003615495414,0.05936524351427912,0.07761186213646881,0.07093559690615345,0.07625864370027838,0.06940212392340267,0.08909401102013248,0.08036385059430869,0.08289500702559865,0.08013699216237338,0.07339207090212148,0.077070349791033,0.07473418465236628,0.08292651655791446,0.07373529787416655,0.07349831902172083,0.07394583060643564,0.0783498433278077,0.06949088310205553,0.0663052842790321,0.07157494097391301,0.07836935675120775,0.06844337485124757,0.0562940916735397,0.08486330162473321,0.0841237308887266,0.060288080042800604,0.07658721335065587,0.06513776505742602,0.07845320888340707,0.08512761492349324,0.05858174326059536,0.07952380377128782,0.07946684011327232,0.06678769208836272,0.08527188851294369,0.07900198959802057,0.0818964064911196,0.07158630390603232,0.06929555825816297,0.09102597151064848,0.0700753330201519,0.07091628199588282,0.08095354555522173,0.08309174335582362,0.0858897432494306,0.07763442357101409,0.08654469092688083,0.08493856872251143,0.09152547039627497,0.08716790641138272,0.07123567360309976,0.09145834158522106,0.06942175527742639,0.06866977274024509,0.08016029417644772,0.08746899342426184,0.07915544815255893,0.07136165242123278,0.07775745705962726,0.08290196405534024,0.07464576474543956,0.07727353829359977,0.07560550833723477,0.08574762473756375,0.07195976540069123,0.06679702573937368,0.07289757138987277,0.08390190387963731,0.07540064787401828,0.08384474717408398,0.07183397516046455,0.07565743568478872,0.05721926980863141,0.0770335609916975,0.09165567436747704,0.08137592512484722,0.08825671903082516,0.0720949917084364,0.06796607973156601,0.08126584006617686,0.08146602792871854,0.08247883149368054,0.07234596974440746,0.08581147320507732,0.08161419909112441,0.08352775857430335,0.07531607203113022,0.08549598449054965,0.07046553537836889,0.08056317249061809,0.08565114801962163,0.07875685484971578,0.07795884030091263,0.08176811173728582,0.0800069406896905,0.07295891397750273,0.08425875514309261,0.06598119443024492,0.06660015819747264,0.08914460860205563,0.08664563774796141,0.08631664362446248,0.07611427940719498,0.07799897560990326,0.08916858431942462,0.07568527882851292,0.05096312536624319,0.0674194987199778,0.0745038069932692,0.09181801616630168,0.0750203796235449,0.0861980978765528,0.07508357732716966,0.0897536886199297,0.056947091107528325,0.07878090010813005,0.07959344430508553,0.05456275707603718,0.07822503339922554,0.09162891876280191,0.07604408790764026,0.08139221151281868,0.0821470980701556,0.06547287447001399,0.07433966653256248,0.07363316198314065,0.08428719992647349,0.07422251154195744,0.07761556987307103,0.07233082529492563,0.08706606837580237,0.08319575888556396,0.077291051152547,0.07895563398837423,0.07744049145857855,0.07079464129946356,0.06817807961880692,0.06822339111460547,0.07298477574187096,0.07156950916792887,0.08251185380662494,0.0626935802711932,0.07754888017095467,0.07950690751069125,0.0572773105375809,0.09300542060052741,0.07028572035016559,0.08438931459803908,0.07771562314712815,0.08609738824376631,0.077309255903093,0.08020032715128897,0.07209692141800962,0.07524471256664333,0.0790760220588374,0.077647957953903,0.07690887781915749,0.06504871343365073,0.07773217793736466,0.08070354607180827,0.06251153515955218,0.06991201826588819,0.08822003525406108,0.08265030892648959,0.07840279140235985,0.06338672272432272,0.07328570440095616,0.06546828425547324,0.07721340516266734,0.07076244531929039,0.060836691574413335,0.08947147307221961,0.08102440958475546,0.07677573208930591,0.052745346438420916,0.07382235730021192,0.07882689709112109,0.08974062631528548,0.07653017033737586,0.08980707878361766,0.0814409074768397,0.06078359635684305,0.05582830059241069,0.08371448572104213,0.07528859313721022,0.06367797109090832,0.0942054773961289,0.08043980432476226,0.07139164932228076,0.07858003955082478,0.08189056224078531,0.07271274152753693,0.07817011043211727,0.08038939697827308,0.08183411623606526,0.07319467336114209,0.08546119754430268,0.07958966402070809,0.07312459728629489,0.06983519608106506,0.06918473999790277,0.08325453045418263,0.09002429399037039,0.07354490920454387,0.0756008805188326,0.07926645675369598,0.06546547728319026,0.07468347751941866,0.06786744313070034,0.09245880304644419,0.08015558223707403,0.07771446626318484,0.07037805337760651,0.08152451767906968,0.07429023554613386,0.07865742507994744,0.0706171728040561,0.0784685627771948,0.09009657976687017,0.08462407623776773,0.0696368686695609,0.07293732416890426,0.05809790367489198,0.07851823485129247,0.07171198155378887,0.07038587686077952,0.08121522152345484,0.06741834709659861,0.07559383380846552,0.08753796898113209,0.07408616259550427,0.08033497280504133,0.07379233580075292,0.07993470722532128,0.08274007888419349,0.08238909157729743,0.07744131239291618,0.08825154680542191,0.07079502804473517,0.07967096912649549,0.0653795952193261,0.07473982540382315,0.07573443554783962,0.08607829156700522,0.0831211028814441,0.0610511867687947,0.07464948401331759,0.07136664552068493,0.06895221167205527,0.08023418273255815,0.07967742591884211,0.06779880571072833,0.07670226655048246,0.07493140285055516,0.07913893092075508,0.07662907514322814,0.08223134430695087,0.06400174110985925,0.07069433695444058,0.0769739535840547,0.08691036663711525,0.0867349100906252,0.07982693041037017,0.08049263083680266,0.08030686419453503,0.08453350154861954,0.07093384057552168,0.06388556421260494,0.07149525869926152,0.07676350295161874,0.06891111824146667,0.060113814687886916,0.061613861197611794,0.08556061569427242,0.0835571044630461,0.08104602904237967,0.07810729143330654,0.08561757690105531,0.07278047450673675,0.07477213808648026,0.07306825380701237,0.07202901844461873,0.0805589974067642,0.0820025395273778,0.08462959217263148,0.07092681516980878,0.07540522793187521,0.06670940287111125,0.07135433354066928,0.07387276440061444,0.08466742334314636,0.06627383022204653,0.08059727638392071,0.08088639702032903,0.08182634346464297,0.08239882518329422,0.0721335552175634,0.08578784776014123,0.07966905557774276,0.0794383538326659,0.08583099468611281,0.06654783306347636,0.06124370339364765,0.07053621819926006,0.07478627217341755,0.0781862904726869,0.08087729050862626,0.08022602612460303,0.07129365102087426,0.08349158230187337,0.08727175716787763,0.0720323573707648,0.07381085575193799,0.08150178447287366,0.06722424146696483,0.06936151040130047,0.07716608512577858,0.07642306731064015,0.08541401224352184,0.08337186100479915,0.07594479475878058,0.07623760043990589,0.07608649177795895,0.06243076083015202,0.07407605190430264,0.07799459497197872,0.08194844632579872,0.0803116810203637,0.0842192850700548,0.06110661731817252,0.0666424111719397,0.07624133215712439,0.07386403143557599,0.087315967721726,0.0583861151261726,0.08502651012527546,0.0672718124255264,0.06829305189008796,0.07769079314505384,0.0704943863230635,0.07576319893016753,0.07561467521739043,0.0742510238395887,0.06874681134046137,0.08776405083794359,0.06646032348286622,0.08150890477637185,0.06362587964646763,0.07916997244325996,0.08137080481634668,0.0766113032424911,0.08666818308886912,0.08800713110004171,0.08463591349843412,0.08375454840620915,0.0686133721547641,0.09052116692862339,0.07919033813331323,0.06673801622511262,0.07884313706596244,0.0921220159017577,0.07125577723026508,0.08867172481456599,0.07406405454654491,0.07860149977335225,0.08515783201416299,0.06540002705297548,0.08664461074670951,0.08036178984938791,0.06885747346782542,0.07718403171162654,0.09244653260218795,0.08230558978092138,0.08127435207795136,0.07737954922713827,0.07629199294551009,0.07256040405024924,0.0808976866453425,0.08312805570809144,0.08183460943066087,0.07564142677823998,0.08304559257659583,0.07325124181467704,0.06417402990179814,0.09578800300955917,0.06777634836014737,0.07343556183325964,0.08389336377086035,0.07742994679481592,0.06511405015492283,0.06850218338171288,0.07178167424651094,0.06198917816841748,0.07362647649864788,0.0901021497821076,0.07455641913856051,0.0761175221562019,0.06742191117888902,0.07943130560942696,0.08161260475557328,0.0644147032405736,0.061175529672564746,0.07450854429377052,0.07756746503808716,0.07174862395243932,0.08540498236153814,0.06103987542963364,0.0934047025879658,0.07298202745754222,0.06233761452146499,0.08015425184593197,0.08386737056945329,0.07825357122549276,0.08769581195102832,0.07389615065585481,0.08462210453346705,0.07843555322447,0.0798354187747131,0.07628386568965978,0.08336640566376276,0.07159407717221405,0.07540915921646145,0.06757665768375261,0.09039525067289306,0.08408943444292985,0.0770332013417803,0.07979380896116804,0.07852531803078094,0.06279016744225699,0.07394928890604402,0.08705417691438996,0.0823710905430074,0.07758143513653105,0.05600157509927244,0.06758930335986116,0.05845418294653795,0.06957412615778504,0.08390696499395911,0.07884464710766939,0.07075694640609193,0.08615389978888735,0.07814478200762873,0.07716594321039696,0.06420382887982412,0.08380241657018979,0.0742134451994772,0.07510030753832496,0.08563547415690963,0.09082245096616366,0.06392685499001223,0.08749914393291752,0.06696666396185902,0.07720499399724823,0.08023662552920471,0.09343041866641733,0.08234031231812372,0.06516929649912362,0.06930423442357533,0.08869133063375555,0.07089788183881623,0.07168400191155652,0.09216985986893535,0.08614021609144604,0.06550521247814303,0.07224274402154249,0.07303277606345202,0.06998740221934314,0.09381029021963265,0.06385524267325607,0.08278515331131613,0.08305366967157811,0.0900954003141417,0.08836867501214762,0.07809105368988778,0.08199841523979615,0.07487576197648127,0.0875931482178536,0.06857816570399804,0.07824928690270343,0.06302976313413572,0.085804135853186,0.07224522190449631,0.07095826106601273,0.06184647838283547,0.059884948775892025,0.07826789422427721,0.07085157506542442,0.08402663725432738,0.07977913541613338,0.08049971338309274,0.07654463875485545,0.08001529651800604,0.07676290211969958,0.07745164863996805,0.08778950892398918,0.07778347650589716,0.07715746799236883,0.08557656596186712,0.07676202592895076,0.083977898140351,0.05679739315817128,0.08538420351107405,0.06542665915777379,0.07885248444562201,0.07716196836630622,0.08082481638227464,0.08055836716336089,0.07577857281730309,0.07434327078099061,0.06260651666155895,0.07852297966119966,0.07820849342123293,0.08306383041683987,0.06881421853144046,0.07971220127914647,0.07905748193158735,0.05965493636829686,0.06537454542614729,0.06513276869399101,0.07245810419522061,0.07310901643731464,0.0838799900920465,0.08463012344707771,0.0648063070768092,0.08589167133075105,0.07893597677524161,0.08706553427169371,0.0907232609131284,0.08160553091778666,0.05142310650094514,0.07251067781198192,0.07796478455748869,0.07423851860873246,0.08231974805854697,0.07681755971390117,0.06888489571207199,0.06150684508704153,0.07208979325457349,0.07119662482997152,0.07379613037877296,0.07707953236775494,0.07361975199780149,0.06462768994301754,0.08454480768553305,0.06973942355799434,0.08612876895819618,0.0751680307139844,0.07052198927411923,0.07494980091674015,0.09047435203969617,0.08357228432747033,0.08027878742086819,0.08117361928427125,0.07134690330054888,0.0871331814956798,0.08198803608947774,0.07631913712771649,0.09208525160259091,0.08246952214019353,0.08695042503195691,0.07734306823530163,0.07915554576524328,0.07798193550583249,0.08258149934510114,0.07732628603782715,0.07295568075341072,0.07207811770911823,0.08000590800134681,0.07040347045269874,0.06632251365631718,0.06879112072337665,0.07406927847205447,0.06624400692427199,0.08715400388583261,0.08252060415303053,0.08035472033776042,0.06745481951182075,0.08818901706663804,0.08777866097076817,0.0721703408001477,0.0889864552592965,0.0927048838201469,0.08629111588346339,0.0667510651458427,0.0801116819337061,0.07503773712178233,0.09014216813478787,0.08703287399690118,0.0872178948577345,0.08368183296406978,0.07532565737267478,0.06936976871812359,0.08298008454404038,0.07768781792927387,0.07208677358793855,0.07248138313157466,0.06868672847865073,0.06939619016078324,0.08722485468291741,0.08195892229874804,0.08314453073642816,0.08709304604890969,0.0723171499752508,0.0703773936903971,0.0688369644001416,0.08529565941862224,0.06914059273370615,0.07881539968003405,0.08015746435770618,0.07396230950517294,0.07851891212986964,0.07850590310084402,0.06921810868926698,0.084629792712802,0.07599771079649312,0.08534150548740604,0.06637624653379545,0.08224645766381306,0.06506820944149097,0.06616951584832159,0.0658661958326236,0.08524556325934766,0.09240492756623825,0.06710082911922954,0.0848754119737313,0.0947110704103442,0.08145896420673351,0.08710873368517022,0.07348656442875742,0.08195910425009194,0.07234139772063153,0.08580262510874662,0.07905228318646226,0.06186323364031539,0.07133911211878685,0.07425515362034804,0.08983947679954946,0.08983511690141188,0.07892836662572014,0.06681794312061236,0.08471836619896565,0.07884427963142115,0.07216940194627178,0.07744437201032478,0.08349929920473434,0.07497728850311111,0.09150223289960907,0.06745789778580513,0.08453253526932711,0.0579098125568748,0.07544624673165115,0.07776405394655897,0.07851205962818372,0.08314701401576685,0.08461498576062246,0.07640238038538347,0.0730185029780419,0.06491630217854458,0.07187450629698126,0.07617372998095362,0.07790837537828199,0.053617947035904466,0.07701128835049954,0.07118603732452128,0.08179269218484231,0.0615687066134634,0.0736904505504157,0.07518989002258003,0.08007252784280967,0.08522259041024227,0.08490118748195867,0.06685447367294536,0.07568981555640024,0.07427341885052366,0.07589141830385915,0.08650392069836008,0.06348066335760959,0.07366036843209797,0.09109092945935988,0.07601046029180995,0.08534889325404535,0.07123982188606597,0.08384652457096459,0.06639700118112273,0.07395797257303904,0.08656162964289389,0.08222874770745349,0.07095315049689024,0.06767526086542776,0.08239955452273648,0.07314982517325924,0.0753744115794941,0.07995756453697683,0.07348906074681513,0.08890841467692592,0.07978062966132335,0.07141941437283653,0.07785112466467198,0.08552519542912432,0.07552099275219251,0.08650322420403303,0.09626294335795448,0.06240882906528976,0.07223297973982917,0.07453399316251577,0.07417220481916462,0.07904195989201743,0.07481620580654055,0.0735667992160842,0.09389859137670874,0.06879878616846992,0.07316565056190075,0.07916814246810265,0.08379482213040906,0.07731508837871387,0.088939764644232,0.06727151456376287,0.07770981411067249,0.07993211121011122,0.07659558088798901,0.07030039406506533,0.0863878866267182,0.06957208594163247,0.07247800224128877,0.06658990594660603,0.06354588257235091,0.0901747921690751,0.08895023290200871,0.07581906151280708,0.08436629980829258,0.06058466112520285,0.08524168161090998,0.07965338137226605,0.08597343794001744,0.07629912578031611,0.07963988328098502,0.08218005334682452,0.07530707871089694,0.08062283194517963,0.08077080798180186,0.056218831719852674,0.08073663931866096,0.05294929183126846,0.07355442819053867,0.08362783143901525,0.07187120350353018,0.08147780482766298,0.07102839901870053,0.08175679949657971,0.0685156688640593,0.06639072869463836,0.07902801260518995,0.07565165836566853,0.06607631376960021,0.07028046130019805,0.06569172971889667,0.07592288360132438,0.08044501650186073,0.08649684774955757,0.060922250211509626,0.08132908298075112,0.05982715683123412,0.0845327874931698,0.08023173654132461,0.08180635360905379,0.06983698205980066,0.0623579344823656,0.082380709842925,0.07691774717658695,0.07018590560137611,0.08129492743624615,0.08009080942052402,0.07659656837618556,0.07636910052506368,0.08766025440888922,0.08018834371965627,0.06504389612194769,0.07686217507675393,0.07674203419390443,0.07313779922596403,0.06360594667703809,0.08451046312618918,0.07043420566570788,0.08849508165360549,0.08016208246344736,0.07182608853811585,0.0745743395494183,0.08409718433456345,0.082837842655508,0.0819723687414698,0.07993992660253509,0.07873202779549793,0.07483088858838433,0.07409245902806218,0.08706958803478754,0.08501283460674272,0.08431200260869018,0.07587515598391449,0.08217365543523228,0.08333955694678341,0.07749051813375402,0.07726606198360358,0.08572987322804826,0.07881287556802236,0.0786678102779917,0.07906246628119186,0.06380103516220559,0.07317025264255037,0.08714943073094983,0.07233996499189,0.07212416889513645,0.08012254145808156,0.07810835863771491,0.0795148165791845,0.0738799640348579,0.08496064479443044,0.058420277543585496,0.0810363969563903,0.06274455508419899,0.07061670809720508,0.08511642743148679,0.06522682247518292,0.0816165625823628,0.0752801007180795,0.07735492745218084,0.08252200005554915,0.057159615925429494,0.06799002574574367,0.08462578088084448,0.09156964260852021,0.09209023599315394,0.07335275196315993,0.0770919767018676,0.07066055286504888,0.07418687594612408,0.07653908127097696,0.07301137122274776,0.08464328168591007,0.06904701956757994,0.07908823430907067,0.07317537946528062,0.08750792203642838,0.07874292302043481,0.09143578188274935,0.0682312464070409,0.07567824213915553,0.08159498735758311,0.08405218944519047,0.08529646875510592,0.07110769592891092,0.07403881183752789,0.08392711661567359,0.07134050180768342,0.08275172450347501,0.0779688179325525,0.06137532325455687,0.07972641758962615,0.0842849421223418,0.08606649709911521,0.07343520498581649,0.07307087288040898,0.07736635135699568,0.06131704801216592,0.07846698449644554,0.08504981913375761,0.07960179563047245,0.06943776343027075,0.05899828696385084,0.07620640638575475,0.07463098725290461,0.07033024633518606,0.07239429467475153,0.07479133744016549,0.09285747621113996,0.06733311833840185,0.06195982301506029,0.06979546412222218,0.07121208917145795,0.07176807219655874,0.07540425242032789,0.0801481451480315,0.08443916666387535,0.07525852975314623,0.07161469805632253,0.09120785469740797,0.07182994858011223,0.07584253053347459,0.07263343456855334,0.08404664767214909,0.08307849362754946,0.07664635517912108,0.05772673327744688,0.0864442957619034,0.07222807760667388,0.08653015637430624,0.07993016447570325,0.0866454833371131,0.06747562881370087,0.08324121994962427,0.06547031381735352,0.072859543086852,0.08191030495422949,0.0705063104871074,0.07345649094201998,0.07553114130292979,0.06210768553612434,0.08271632168295795,0.06966941004351526,0.07870687626060444,0.08399585800184292,0.05307867355561408,0.08326278422501529,0.07309904120070682,0.058175051539321176,0.06986532431546619,0.08456616206216766,0.07673241927080289,0.06813750458792017,0.08333959532922436,0.07704928027007728,0.07072817934962457,0.07827307015053858,0.08531560179082834,0.07535374449103087,0.07424999339918956,0.07270597266068819,0.09090841537669711,0.08095178328994979,0.09116916132476044,0.08639437535151534,0.0656743141525124,0.05506169552197065,0.05966195668192325,0.07791629981055079,0.08305165430356704,0.08875510227281921,0.07685806610179943,0.07405647708111146,0.08608407403813585,0.08639888231596868,0.07987314788241676,0.08325303339438642,0.08752169086106762,0.07688887831882926,0.05641979916389171,0.07487762219702894,0.07605472135706326,0.07288457534159389,0.0852733066078975,0.08154557656376682,0.06189796502653454,0.07584086170965414,0.06173568314241618,0.08627063787528513,0.06742692448831245,0.07845814349876185,0.08475598479863347,0.07721947642503305,0.0830336938166439,0.05810623602084529,0.08649747519457808,0.07059511572139385,0.05945075869431209,0.07093782345565372,0.07266596660170573,0.07084690111864037,0.07582798674580474,0.07134411680594663,0.07892164053492919,0.0867851472416247,0.07340600638211858,0.07501300126485391,0.05327367313388823,0.07895974616514893,0.06950269045737179,0.07783331212421593,0.07193948963046357,0.06290999542501671,0.07631467997481314,0.07382401272965841,0.08893082948141497,0.08020890881309602,0.07901168524352473,0.07984530259078446,0.06904486270179885,0.0821715008680251,0.07357863602018927,0.06701611434627894,0.09235607347374172,0.07813734428766357,0.06330591454292601,0.06830047943579715,0.08721387740885306,0.08885579447530512,0.08844682272720542,0.07273678318821482,0.06663588973129331,0.0675628103698715,0.07051739764681572,0.08520386464631985,0.06360555305140844,0.0788249511042448,0.05938015889812494,0.06598692761773761,0.07108571237822627,0.07104609255849083,0.07894552613499724,0.08558899066175667,0.0624977355313302,0.07293113819774534,0.07634351611894993,0.06512809283184079,0.07785427549229641,0.06554786652823458,0.07043959603113076,0.06980994805290108,0.08450914702253275,0.07012493029237853,0.08394748386013438,0.059753613924383275,0.06371226225153036,0.08094118775381554,0.05786419607751551,0.08181437856684035,0.08448626385643623,0.0892613327546224,0.07962770643723846,0.08433282453956759,0.0861085667085105,0.06429440224244179,0.08175501503192892,0.08312244114968889,0.07647320813487944,0.08033411881925223,0.07375859967922034,0.05737595573603527,0.0684541352016178,0.09300380750288005,0.07717622088260821,0.06546893050249482,0.08372115833842589,0.07530174336335976,0.08416655138712112,0.07077552925460437,0.08193964912511548,0.07939668933133245,0.08759199531223955,0.08361170753484089,0.07167027793155159,0.07688635594441812,0.07909997043892707,0.07539805330555929,0.07934340982151666,0.07930267128973617,0.08365989260925191,0.0802323528149341,0.08598808055110319,0.08233170099508044,0.06486376730740565,0.07368811102906261,0.08301289582380782,0.05453896211988484,0.07300002928567023,0.07982916306775961,0.07701591201956029,0.08945148284740095,0.07213859147335047,0.06889316361183839,0.08150231239255791,0.07817870719070891,0.0707592205707473,0.08691886580031645,0.07977954136124746,0.07311686164400724,0.07101564490842004,0.06374501117857229,0.06563738648721532,0.07676636932344026,0.0673240765975846,0.07095727792908765,0.08694474712202041,0.07746714932320797,0.07552861610701281,0.06307644066316917,0.07155776519648446,0.07035017193542595,0.07743119777998089,0.07229438773157859,0.08137056680979127,0.05227823769390727,0.07692999568057564,0.07559353681640048,0.07046994528052508,0.07946868834429849,0.08018129581777614,0.07820200342370048,0.07009813967437484,0.08479304123416842,0.06667787177082571,0.07867414301304029,0.08307215479680927,0.08395345699174259,0.08454470970421687,0.08945822234731116,0.07744006654889213,0.08543816815572347,0.07354317093562779,0.07268602311786879,0.06705416938871367,0.08439509830086564,0.0648281604081771,0.07346900407481566,0.08518760599653127,0.08375794291906136,0.07031823688519781,0.07830443419932963,0.07733043291958824,0.07402484260907202,0.08274595609896225,0.08438633365468845,0.08373455202677446,0.07818883323403028,0.08918274728018749,0.08298914815396286,0.06546183561773068,0.08935579354325646,0.08387609140379935,0.06775560089516186,0.06260693231901224,0.08653410117572624,0.07278520294302365,0.08065431198925664,0.07420666467728006,0.071752841462968,0.07896473467701172,0.07240969882058648,0.09279629884810642,0.07773869074754926,0.06817521537358284,0.0770235562490938,0.0882875563828108,0.07597557309325674,0.08972128675162155,0.07679984138121734,0.08264445196695556,0.06688483606408281,0.07125728705317488,0.08393538066854443,0.07163872366199475,0.09069424350404343,0.07539049820788772,0.08408685161266985,0.07133988575692421,0.07602721678323,0.08016029665598823,0.07860582126575663,0.08080389340381558,0.07024788550169551,0.08050571814999222,0.09306770474653896,0.06822161022525713,0.06955795354648704,0.07835603594012018,0.06728202719180827,0.07703389723031497,0.07849347568770695,0.07655134761708815,0.08395649505034614,0.06955657702974971,0.07893556915996805,0.08299960321403536,0.06605678926804043,0.07143281820831712,0.0800629007539507,0.07735358130142395,0.07263931664156069,0.08013138349769414,0.07400665182359734,0.0817527483425689,0.08500945984084751,0.07729535831033119,0.06870784025375529,0.0744199865311253,0.08283630889135296,0.06846797600931355,0.07427013288886039,0.07449601111526563,0.08843999545251788,0.06506250641001934,0.07195149804178486,0.08665692978101691,0.07367416538136498,0.07583438827346863,0.06077273436827431,0.08392624459892142,0.07513379489125743,0.0785387621581443,0.0876609199499781,0.08219815420998658,0.08876483046587412,0.075936379109133,0.0715666797935578,0.08694624596722567,0.06869285269675393,0.07617438561763087,0.08381560706009875,0.07730369699127206,0.08392326210754289,0.07129700356396666,0.0869810501814367,0.07702672409612135,0.07647898380974077,0.06993080626759841,0.0958123544143744,0.0807343911083648,0.0675448313030288,0.08465474325845726,0.06494697344668227,0.07591178066396966,0.0722480408647253,0.07782344782247308,0.08075473886565675,0.06863330055171045,0.07948348061180666,0.07250185923268002,0.08931209713899427,0.07918563455424343,0.0838450011006775,0.0765443219108067,0.07214714516026234,0.08265744313843418,0.08370790145448727,0.08569241124416294,0.0633962962063665,0.0766684917261912,0.08722675935597718,0.07718273284280402,0.06827071210146031,0.07603195412579791,0.07867344792248586,0.07423249709175077,0.08399718084666177,0.06564680133072884,0.0865659175505622,0.07984467545716079,0.0892768483772639,0.07263293114045205,0.06458618860520896,0.06858897369452148,0.09086198848892554,0.07711329299702786,0.07264691218490438,0.07370096008422135,0.08042998166232099,0.07209958352993437,0.07166408984272804,0.06847744431720469,0.08932120009133679,0.07224655180434222,0.07854369576614756,0.07597582941249306,0.08154140858504962,0.07748785380297588,0.06202476190087271,0.07982225212972173,0.07897667169018491,0.08676412769592164,0.0735438567137609,0.08024504464632998,0.06294953542442755,0.06570948269936654,0.05900085053379292,0.0813317147633894,0.07249306208501151,0.06873646933315429,0.07371580387744149,0.0738316887554607,0.08565966989803545,0.07922897811528892,0.08838361286924379,0.07108036723169607,0.08159740513965483,0.08281588948277986,0.06652961453735809,0.09190252274414756,0.09305554073508251,0.08298442705912565,0.07471268868461645,0.08590601243660362,0.0642146083513366,0.072534643381018,0.08865965752164229,0.0716123395287349,0.08302258779009859,0.07687789623758351,0.06977927945750287,0.0680463233029047,0.06717165563397601,0.07946943876101964,0.07572107816286056,0.08100082435573616,0.05729544311675991,0.06981363878116423,0.07203239057434022,0.07905589193112422,0.08771499639538671,0.08929264251466504,0.06902793512943031,0.08393456943712088,0.08442615087419728,0.0688384001422494,0.06904650365240625,0.07771994837767349,0.08726634243749981,0.06820113436739599,0.07693506549130687,0.08155742688843082,0.08321457227825402,0.08899966759414579,0.06238874131825868,0.07986743562452232,0.06845114411243708,0.08337011798705687,0.07227967780719186,0.07997669370511211,0.0709198330104756,0.07145352108724319,0.07611348751501168,0.07987040844333426,0.09382085850404391,0.08069511481232544,0.07839585006369837,0.07041129251413152,0.08219917504037519,0.07251567071689677,0.061397071376453755,0.07143568218313939,0.08237565669999093,0.06215576404686371,0.07049162120133398,0.09597448653803077,0.07502151824242967,0.08285398410385242,0.07942335424573013,0.06827342919541612,0.07209046221890045,0.06793930304664217,0.07129337650066822,0.0806519923918873,0.08743870665976085,0.07236276433995822,0.07906700578510391,0.07373449911614696,0.06968766032617082,0.07844519157086165,0.07601312857489007,0.08475079657668422,0.0833973879313385,0.07301568154362505,0.07854310275930783,0.0687223164910831,0.08666583959069434,0.0648650961784073,0.07486091095938253,0.06386526576272014,0.0766753218697985,0.0842954179668575,0.05816909550548442,0.0802297347802162,0.06513475250209885,0.06817833071170082,0.07802837477189012,0.08236102759511697,0.0776684471700002,0.08340345256804305,0.07508927811845896,0.08546088737966537,0.08070134651218971,0.078599671599406,0.08382855153176245,0.06938312694182404,0.06776191562157602,0.07077660144471129,0.08050569545301531,0.07021788404382698,0.06734185895846159,0.07547323320931551,0.06789195354281768,0.08682374176252948,0.08386746838183012,0.08267922103230799,0.07729908660263349,0.07582690943841203,0.06861752049389008,0.08099695191139719,0.07746926864672549,0.08339263593794964,0.0770404866754443,0.0710090069623061,0.09075328924680018,0.07328415312025191,0.085488201853216,0.0682324283752252,0.07497331150259867,0.07943937359148823,0.07818763243439786,0.0776974749760443,0.07938039105148142,0.07660443918466354,0.08889557176283555,0.07891311140604998,0.08430597571460341,0.06621684075367121,0.08513494963847822,0.07159833814830674,0.08605305386316069,0.08203290735217636,0.05234121275836735,0.07183024653021862,0.0671309938213922,0.07220439839733578,0.08110226957207597,0.07841833912080472,0.082172036793074,0.08699780725327047,0.061854867223457406,0.08551703202990302,0.06833277791542375,0.07385396030957572,0.07548850713513908,0.07944280523329664,0.08666590964217867,0.08650763863624736,0.069646846298709,0.07018736776154566,0.07367530587642286,0.0707352682400059,0.08225444859578727,0.08090902338849788,0.08816310034430684,0.09101514762331182,0.08787279599635778,0.08340091071406522,0.07258465183571289,0.07555071596535812,0.06906011502184306,0.06263066702538991,0.0813183362010867,0.0813606613491441,0.08497890900440089,0.07714978338721029,0.06956914644869922,0.0683678325600517,0.07168825133002417,0.07213221783647045,0.07447080217674737,0.08289549885991579,0.08023197255862083,0.08085756128762958,0.085148850675565,0.07991425906043254,0.09073884273000474,0.07379879859876848,0.08298513724104259,0.08286837152190779,0.08535185626863873,0.08906271458266887,0.061499816676135445,0.09277490379648422,0.06760231825657198,0.06455427814711645,0.06582822612275299,0.07214799746211037,0.06695835931255481,0.07938728331399053,0.07420933455742408,0.07049028373509302,0.08225199975709901,0.06676406249518932,0.07658750507318703,0.07627160812815534,0.092353433913196,0.06861715842741731,0.07688852504529124,0.0802182418125077,0.07688877238679437,0.08303995568415443,0.061782911198716546,0.07378638555605185,0.07103244566810415,0.08614381430054485,0.06566398005621896,0.06297996414284351,0.06625841325521978,0.06282826983040958,0.08932385588228826,0.0709147089804624,0.07180128525505944,0.0748345459919676,0.07539041861263526,0.08190852114377399,0.07717383411295682,0.08303589926355429,0.06739251752521838,0.060644836514583154,0.07774561333402748,0.08892212033070829,0.0736326075661709,0.08112019646241286,0.06882010520541687,0.07974270616175178,0.07869840594458949,0.08349268574231578,0.08130372322977225,0.07285407561882531,0.06698362798801867,0.06363560331145074,0.08506193525605568,0.0837607952721401,0.0646301555079754,0.08165667450892652,0.07065484135715373,0.06375872627715456,0.07411970289499159,0.08277398948872286,0.07130895063910761,0.0807201616370432,0.07935365277765762,0.06684967124584887,0.06751516530012225,0.07649171781681204,0.07936207222359323,0.08105800185427703,0.07507573694911512,0.08150649155908103,0.088819821836925,0.08157344277322892,0.0696637306277023,0.0721867697975046,0.08022246564437964,0.08812290667728129,0.06788398485661007,0.07553419823306859,0.08511059242555508,0.07883996428907931,0.06818408358055864,0.06791410736493271,0.0705796432301338,0.0702323293074194,0.0827174825128984,0.0822471400688835,0.058902225918697204,0.07608508827485755,0.08105741664168183,0.08258237316316425,0.0782512315955651,0.07888931672414128,0.06388723027788493,0.06847774021508252,0.0823107151630171,0.08657003570604702,0.07429883880234064,0.0895492277421503,0.08297895526721395,0.07364714824289986,0.08230191437355157,0.06415728787653882,0.08104261210361258,0.09348960966906016,0.07046159046739515,0.09497856108248087,0.07451959297395352,0.062234602521065475,0.07672943970514953,0.07652572807103034,0.08102308695506973,0.05967613773810373,0.07163730639877133,0.08534668114272748,0.07544464507786572,0.08740473305874304,0.09109053074913841,0.07750523179137132,0.08094232363291562,0.072751437839911,0.07888641064425173,0.0686270785269239,0.08872897430918347,0.06740862241277934,0.08012529695945214,0.08112725730990525,0.08225748470902024,0.08644782124903636,0.07505047684721447,0.08342615101292614,0.06914913923644159,0.08458047424001508,0.08503540679727215,0.0709679911948217,0.07538614944701545,0.07877718699935152,0.08645988921751518,0.07963278255329201,0.06897776761168023,0.08249516187490036,0.07645870038116453,0.08737425233772486,0.07671353826129688,0.07716928704757586,0.06847071211457223,0.07100469990995048,0.07087395973882796,0.07389547737332741,0.07405149169726857,0.07509388612737568,0.0657444142360106,0.07489993962894254,0.09423007938998804,0.06197572103308174,0.06393049032184948,0.07281086938845632,0.06552006341307512,0.07462876579182845,0.07358088371768594,0.08177071927555687,0.07765030882015146,0.055965182785443074,0.08108255503169448,0.07133602279781741,0.07558060643360955,0.07195961826636289,0.07368884358330924,0.07991500577534022,0.08753568566749782,0.07942496908400257,0.06872715306714382,0.08478005901570582,0.06399619730805436,0.06712214801955077,0.06947743974950304,0.08354734187746994,0.07068988152641406,0.06842567829392912,0.07815073624012503,0.08402457626752502,0.0784933239140502,0.07505212895140065,0.07052085625788594,0.07585620160742329,0.08587868182378747,0.08092640692895636,0.07894902213700208,0.06329471113501889,0.08143226986870755,0.06876981199075938,0.07569296148701912,0.07348816596238265,0.08536875256958916,0.06445257301409912,0.08214973735151902,0.06579320699905444,0.08410475171593855,0.07043249044515236,0.08621787780971303,0.07838064090383437,0.07174622252226642,0.05605074978416979,0.08839419933280464,0.077991411455496,0.08384379326080327,0.06748491834189503,0.060082425556689585,0.08260489975604014,0.05008974703474815,0.08127407788980998,0.06300552817578009,0.08020515002243761,0.08145441777134602,0.07930457388650511,0.08350814549993532,0.07779910403500562,0.0855057184877949,0.08134615336045706,0.0725946565961583,0.06939303560187973,0.07599763254281595,0.07075860736142787,0.08258134807613732,0.09185869706544213,0.08712799844029982,0.09268298765723265,0.08844738849607721,0.08804276244672246,0.07173143726139017,0.0704297599255248,0.07572495001419767,0.0754867011239162,0.07856529584818306,0.0819220153022874,0.07658480983782118,0.0838014363550267,0.07524635189823206,0.0810171333596236,0.06329333323062658,0.07897005880860064,0.06622039915285005,0.07070287889351506,0.0751323830158468,0.06161039619436691,0.07712374340213046,0.07221076137071183,0.07135008586465263,0.06232031881387187,0.07658375537187304,0.08127034350509521,0.0862323386125958,0.08683308226147754,0.06957094971233825,0.09365680061457166,0.06111392692585392,0.08282935282104589,0.0742104027606216,0.07655637519515934,0.0827143606760473,0.07928495980098459,0.07730540562265292,0.0755801234579637,0.091300056191968,0.07828972006850068,0.07896973795246694,0.07706967279602164,0.07348124541764596,0.07802894418373926,0.0668020256965678,0.07970281947548591,0.0755154335159315,0.07860613973556682,0.07759061419768339,0.07211110111481897,0.08018431243604675,0.07826519753054838,0.09471655090867317,0.08293521258544057,0.06808658750703891,0.06000513956961973,0.08304036831232862,0.07376582573608345,0.07438501751784733,0.08822111186953117,0.07646650852756634,0.08668256232522316,0.08139190124535536,0.06892219125437905,0.07297494518993247,0.08919728244314123,0.07385807622201575,0.0688793473818684,0.08602942493845099,0.07798103615180699,0.08247686489693506,0.07711547910682068,0.07588870815552745,0.0697529469982993,0.07554460288662271,0.0779219915731576,0.08346655297706743,0.07925147056565349,0.07805641711207649,0.08373106269284747,0.06589467658804748,0.07953955554651865,0.08762667629060228,0.07886139777884141,0.07084973105225004,0.08470554369088759,0.08335117465492284,0.07762447700701988,0.08419948318741088,0.07186500377847337,0.07846537227541746,0.06352531587065002,0.06348112857135806,0.09011194367053582,0.07522560091029407,0.077364537530219,0.07612234295414694,0.07157211464613318,0.06366872044163671,0.07139273953420336,0.07398255742425325,0.06107160696610765,0.07236280791184152,0.07107966973991589,0.07074212446412764,0.07602767844684724,0.06947345827195228,0.08329341520511971,0.08017796075384856,0.0644018712774963,0.07692066470429615,0.07373990459545,0.07938378425289638,0.08714667946365642,0.07653146551454296,0.08209809436524768,0.06453276973553032,0.06867208205751937,0.0750563629719529,0.07213336532527428,0.0926611704962386,0.09177965734992369,0.08190415934518586,0.07813415309960985,0.07355683214225867,0.08022476513296412,0.06225737403868755,0.0767280962152321,0.08044766193472822,0.06833182922452485,0.09177185649951322,0.07345139483791079,0.07926798980880981,0.09364914140005767,0.06629861645653896,0.07679928725090548,0.06895813970083248,0.07002325732777415,0.08378049343522828,0.06950136406723097,0.0866232699342864,0.07588109873080018,0.0729351517962645,0.08708360997714229,0.08485694522946774,0.0842593080142672,0.08426586530977724,0.06746724991966788,0.08102142762941592,0.06959187904046006,0.08843085174057322,0.08414242240304368,0.08717334277172521,0.07783450208736484,0.08289693371148818,0.07110049631593311,0.06479921081178666,0.08073713302899069,0.07172682196999243,0.07417260412631013,0.07812918202339315,0.07054328630634327,0.08246423786409182,0.06640031422270534,0.06074789591622914,0.07986849178768061,0.056737427962797904,0.0737767209047074,0.0816039916444792,0.08306116912046652,0.08270533152913978,0.06565055057468816,0.07457904786376021,0.07064648751197337,0.09456585758559502,0.08933658926796648,0.07025569748856814,0.08105792890074098,0.08191850590326236,0.07484011345314909,0.08658963623092825,0.0720643409025882,0.07046752694222537,0.08768470141856724,0.08161634345863986,0.05657432199512191,0.07481917075273997,0.06996540507270882,0.08306269076511835,0.08035380858469411,0.08550215328669594,0.08957628043123912,0.06554584245959759,0.07662065528676701,0.0791424466582467,0.06857813139447488,0.06621066854547772,0.08202718852793256,0.08610983739668242,0.06996813223942785,0.07424258847649064,0.09025999588098323,0.07671589544840965,0.07268684176009965,0.07469302016765222,0.07468901005486503,0.08088818541048481,0.08352834487451638,0.09061893406086562,0.07977641663935764,0.07820787375697644,0.07005803393447299,0.07365932047533046,0.0741489698711213,0.06760987655746031,0.07447517104081014,0.06452607533355229,0.08132394198139042,0.08842681220175143,0.0815913006380457,0.0822252749970484,0.07555022637111611,0.06614004880048599,0.07877972412058962,0.07285113465618423,0.0860187259198329,0.08168503065195873,0.0761846978935346,0.07675305108540692,0.08647318128670536,0.08023942456424465,0.08015541434948163,0.07939512252580949,0.0723356156855213,0.0773125985193621,0.08661109797966712,0.085298543634307,0.07694663609721057,0.07763820847645864,0.08772363729871574,0.08293877117253709,0.08819761315119558,0.08169854855448905,0.07577331218618474,0.08741136483034395,0.07233749979190272,0.08260456923233045,0.06004690412673061,0.07376532168227451,0.08669961071066512,0.06854662886476345,0.0811159311190654,0.0877245544817129,0.07984537230205124,0.07558892060237002,0.06637734551836008,0.07822364914153897,0.08764901198473124,0.0625072711696313,0.07375821559733926,0.07986708906732568,0.08218897028549155,0.08962389782577532,0.07088066406924518,0.08525272000584225,0.07570560096188195,0.08609487155466686,0.07363264154806606,0.0635291397174694,0.053196398657834806,0.07858881833487648,0.06821102217633168,0.059683574656676605,0.08193427562188606,0.06904654011577374,0.07686332207849031,0.08161179366448887,0.057398872278755925,0.07073488344012267,0.08472575875738714,0.08192523538064703,0.09192052047974796,0.08585028799695936,0.08150359236812535,0.08381857031968115,0.07579663274116337,0.07367907285023859,0.07830422640686621,0.08620859117998644,0.09060453149003132,0.08615667403527691,0.07368340355638403,0.07091373499244846,0.06133576759571905,0.06939656992086798,0.08404556913505239,0.07720272296308575,0.08692528094164559,0.08017142479740785,0.07110322536161282,0.06560260636862249,0.08322171760092084,0.07663403080180617,0.07276224856784522,0.08407087798500974,0.06276449562481627,0.06588024310760446,0.08192267773225378,0.08770040711292779,0.07685244479248346,0.06613047612753298,0.07957232754958118,0.07026778634111391,0.08113711877552536,0.07951609804071577,0.08626838077119936,0.07925615324968693,0.07409120641019407,0.07516563607719226,0.08790208524447359,0.07173363805095925,0.06752481236000152,0.08709058452387097,0.07987294208931806,0.0820381684566868,0.0768151368259682,0.08108509446268444,0.06961113625431148,0.06309416858959542,0.07225861845904634,0.07321070206153668,0.08831795130931973,0.06405681934735075,0.06689472564031736,0.0852080272401427,0.07942021911187679,0.08779155404123615,0.06596401466729979,0.08105159521459185,0.07487000692481614,0.08538179277117477,0.05235969842221452,0.07874181605202663,0.06632643638244287,0.07419838968382886,0.08008874887689396,0.073328238375779,0.07774133962085711,0.07729599417815018,0.07053230387468332,0.0735695549020208,0.07941668514575626,0.07384631631346336,0.07598166849448057,0.08401941546165434,0.07979280595899446,0.08648425088639017,0.09004472319244003,0.0768070586593458,0.052841517898788146,0.07558995934602164,0.0880352572121472,0.08536640843243802,0.08873475646347086,0.07879946001496135,0.08352563805114807,0.0821534519327406,0.06509325209085332,0.07809508292117173,0.0671151373052831,0.07000743578248197,0.06920000493186793,0.07477239785752274,0.08202399895892107,0.058886962959558586,0.07713975593033062,0.09135463321662513,0.08526239044934031,0.05935331439731729,0.0685657799077444,0.072333544466012,0.082938734253938,0.08217148041293208,0.0710354179843743,0.06614744821935487,0.07280967061399088,0.07178832736270385,0.0756999243601721,0.07666280433747226,0.07457458025117242,0.07370625853149954,0.0785591466154921,0.07935550762933898,0.08008693712645149,0.08401434299963542,0.09032069001977161,0.08577716212123132,0.089204869197828,0.06971588599557932,0.08193200000132211,0.061824615782106206,0.0771381571009083,0.07636403012943863,0.08666727807551705,0.0676937078948977,0.08147294860742038,0.06707524640190378,0.07807198248057187,0.07267360849941619,0.07223245335197173,0.09191014449440778,0.06623282306675116,0.06992667527995028,0.07564879202277056,0.07685869401286483,0.09157068782753548,0.08494162349266406,0.08743037850710975,0.08548793497839913,0.08613022434916819,0.07149350100820534,0.07551247105936466,0.07156692223915205,0.08655341372932962,0.09003295124102516,0.07417805994252093,0.0701799514787794,0.08574505279171271,0.0671328013094168,0.07271790781506593,0.08047815076058512,0.06847014989070568,0.0643542610996026,0.08203132480591181,0.08050602828326664,0.06791463897447582,0.07096315825430712,0.08192466532794718,0.08552549012039809,0.08510246451868214,0.08702587553333849,0.08128354546148756,0.08454866936203996,0.08470099569512111,0.0814731528306883,0.06330468964866395,0.07999286560403215,0.08012495254907542,0.07615700504104916,0.07694985926533428,0.07322433883322085,0.07627028648053205,0.0683208364323076,0.059890482816160794,0.07748284453033671,0.07535860880893983,0.06391033660500677,0.08029175155874446,0.05902604114054186,0.07135582832578305,0.06731609901203481,0.06353647196772358,0.08057922751654428,0.07281264164887771,0.05973715351234804,0.07497187442171138,0.08185045890246004,0.07253252462055179,0.07321295055955857,0.0812497509425389,0.08009265575766597,0.07681852135182667,0.07206451967954627,0.08789601828471347,0.05981425811758788,0.07037326460640929,0.08055994618840145,0.07687936429028944,0.08464675707753373,0.07352355715562893,0.08373453610193846,0.0708950949648065,0.07289505379437275,0.0911774778820604,0.07860804393492922,0.08854214591567118,0.08742624728228354,0.07049936120973793,0.06167037322317823,0.06234215314042568,0.08547235236184476,0.08045792820383205,0.07624747040043706,0.08542878212333946,0.07506513065482603,0.07554289384668847,0.0795088481139245,0.09019966698547104,0.08331721494552108,0.08239731794376672,0.08864634894367349,0.07592150594916658,0.07979665019392837,0.08381478517903364,0.07105318514196154,0.07747326141158906,0.0870055073963614,0.07357168832703292,0.06962921593754066,0.07573900591909508,0.08428540644746635,0.07381988275969317,0.07944367721042372,0.06795609299466618,0.0715022139411492,0.07239866783614177,0.07483066656857118,0.07063364737686215,0.08194047792043263,0.06382563022959603,0.08112672336869181,0.07295835454704466,0.08294115383885914,0.0794913867224119,0.08428820562744338,0.06374464546954589,0.0625864474159369,0.07553773563010636,0.08708573856969323,0.06343417336936957,0.08024001125410736,0.06560103216897396,0.0714377959073195,0.07521205992830321,0.06499881448784778,0.07089772445934651,0.081136579123499,0.06522151941881012,0.0757828344589574,0.0727161638788692,0.08753389223872887,0.07439608219964583,0.0814342079355259,0.06910752337792503,0.06399807741962159,0.08760054659781746,0.0750672132134711,0.08020098147595123,0.08495467628106015,0.07426764494993199,0.06993212219932493,0.07914130761091398,0.07208053489419444,0.08024903331044389,0.06889705914069685,0.08623940078296773,0.08047205157615578,0.07922326878619294,0.06415685922728062,0.08755125937914308,0.07119901786161686,0.08721471309284884,0.08159732475152587,0.0853430458541258,0.08497473075917227,0.07772154513887503,0.0808733803430563,0.07821843251570897,0.07955281199076181,0.07163099009053953,0.07504171670398606,0.07510031826687319,0.07684338888488942,0.08526663423067754,0.08911054839283379,0.0804222952735896,0.0712901569382228,0.07898274081534634,0.07015273359961957,0.08882544722419804,0.08221688929343096,0.07702158031078786,0.07660417821489132,0.0847671335882075,0.08035365221227875,0.0778223676223791,0.06414637716432273,0.075352717699584,0.08390986532847683,0.08104628135920947,0.0757850437006503,0.07962897329753275,0.07957648103776704,0.07491681194219077,0.06581186312066611,0.06523337764838172,0.08152277737285199,0.08737182244551067,0.0685391854362597,0.08054776832368366,0.0819542457395313,0.08307563610423936,0.06464354202582884,0.0645853819890285,0.08076885317575336,0.08702592324789069,0.07846287353479894,0.058682412287537775,0.07311974521855497,0.06170622191824948,0.067839842136112,0.05916898332405267,0.08813812377176666,0.08057504652565184,0.06964712117111271,0.0804572355516661,0.07438620757634577,0.0737624964954658,0.0847533038785839,0.0800775317074848,0.06138224950886866,0.09150040542098592,0.06481248928752457,0.07169966415359796,0.07981421106803688,0.06315216441105109,0.07904318888646339,0.08349863128361362,0.08736894074767393,0.07846556848000322,0.07102533710354496,0.0698314214294267,0.06487901111121146,0.06122773765587507,0.08920473801110715,0.07820274776029269,0.08138020702538243,0.08552460841951595,0.08313293390866247,0.06401762200901875,0.08754251697058238,0.07169356320511232,0.08350514504159683,0.09503107616269964,0.046616091477369274,0.0870729635487536,0.07973413340634813,0.07864040582328383,0.07113917486555628,0.07302946635652244,0.08506110894652036,0.07092682128510897,0.06963354195676204,0.07854750884576944,0.09231150797148514,0.0848237811385784,0.0788488041993485,0.07960560609327327,0.07323295131169942,0.0687200897379551,0.07477470486946262,0.07499835423938833,0.07887687187576811,0.09460887566116959,0.08123789224139971,0.06314779143177839,0.08704535993806876,0.06992457295977526,0.07663429158285384,0.07409901893629231,0.07355181368335134,0.08356409268495377,0.06794994065030807,0.0837946185745231,0.08112153807298342,0.08079081553102546,0.0856016736773807,0.06993753123966273,0.07173480414881359,0.0853594403389043,0.07717713354972994,0.06865743325741054,0.0876673285524398,0.08400067784805121,0.07637378944278014,0.09000135613557743,0.08382397115854592,0.07852798500085696,0.07753759013769636,0.07912222698454352,0.08506283350779506,0.05307358172875884,0.07926558361843494,0.05966719103918702,0.05407029133399152,0.08964475974039876,0.09465124785457497,0.06272820795066361,0.06740462387251406,0.08258755345930313,0.08185039356023802,0.07943426572122689,0.08199703106587458,0.08310825005763264,0.06447489532556466,0.06416751583345595,0.07672138738664147,0.07204122649910052,0.055037737555500246,0.08207284974731915,0.07929354601093647,0.07336857753089693,0.0811467268603705,0.0810770729220524,0.08437270280597334,0.07786202747088894,0.06791240929634232,0.0923329586685834,0.06476094022230527,0.07488133498346947,0.08408704799907077,0.07000431607355061,0.08489871979785757,0.08276972417425005,0.07995675977854477,0.07438314179974104,0.07605527069765558,0.06706498021176305,0.08352856858482341,0.07384763366974412,0.07758838350327445,0.0747306996956043,0.08960205618005508,0.0798901228856072,0.061982620949302146,0.07700475712272536,0.07706712208393812,0.08768767703814834,0.08908749197371343,0.08561603738783939,0.08278702083131377,0.08594405196597989,0.07471062686105076,0.08188560549826156,0.07522459516406454,0.07130747158302447,0.07367394137437606,0.059448261735318614,0.07605771141097364,0.06612276201865379,0.0587597097939456,0.08309636690987865,0.08437143791350807,0.08797559464886728,0.0758941619180136,0.07937509003639415,0.07764539271499002,0.07334316521352294,0.08934305211597755,0.06952383559523316,0.08916390940519367,0.08801886728101355,0.08389752247509122,0.06633697149303366,0.0526027996736293,0.07974871441981436,0.08180342612266739,0.09001987423985633,0.08560267882618568,0.0681249547636114,0.06289889246589578,0.08046977353803195,0.08651976907365963,0.0659698477954726,0.05817738446288799,0.08856757514161034,0.08575104694103684,0.08356579344579185,0.08343900793252211,0.07008107073599756,0.07371967939147012,0.08803258987031187,0.07929767088222806,0.08528249473692945,0.07474048958521935,0.07763589956080608,0.07189211993560447,0.08260688457252809,0.07509078181069254,0.08539624209187682,0.08043148047915125,0.08724932297342491,0.07698949663684669,0.05651813563810548,0.07934133021021525,0.07463495479332814,0.06537104813116112,0.0679894356329426,0.07396104905841844,0.08250056910454862,0.09397234878421712,0.061425061214525484,0.08381783962645958,0.07554381520902571,0.06908068997904399,0.06890183539129986,0.06756070344198295,0.08514350305600926,0.09427018771504649,0.08028362572736561,0.07323394947088277,0.08793924521935298,0.07960846198938962,0.08769297768152035,0.0750387312135663,0.08005652138255873,0.07992197625588435,0.07441581130388661,0.0792209444123523,0.0937336010258983,0.07080442150395255,0.08095235563050548,0.06415844391595832,0.07802873012833914,0.07598757334088291,0.08293615531133731,0.07910539904562414,0.07589325843265715,0.07914595463540237,0.07207343769310157,0.07293013561328104,0.055237536946495525,0.07370155974593656,0.08836346066548922,0.06843557730535393,0.06852167594926581,0.07133185483499141,0.07240140630908617,0.07843250941732445,0.07755338159060228,0.06946688747095682,0.0841848300341812,0.08753651383546382,0.07255254990952585,0.07875985240304778,0.07852546927759851,0.07425330893610821,0.0862161648267292,0.0635015357517878,0.07814701852112724,0.07376499482072169,0.08646942819566833,0.08970000191700739,0.08035698107358577,0.0659858743627445,0.08336216290961798,0.07705818580446355,0.06264232135355999,0.08825916295883314,0.07927840139329978,0.07430633384939526,0.09011791230733453,0.07462821806823795,0.07421953005383806,0.08870499042613651,0.061658854279865234,0.08620608340146627,0.08058964996563484,0.07394833265011891,0.07264821667668846,0.06310528792004155,0.074509248027129,0.07689397819494309,0.07417856337210299,0.0732809167167568,0.07811172173213847,0.07203783838516538,0.08565469047746144,0.07918853392331521,0.0809152163837574,0.08091600274470981,0.07530733554276574,0.08291565627628256,0.07693580809951199,0.06694053587407685,0.07900691200693943,0.08720910745714709,0.08274224137756682,0.08171713591950565,0.07829018355028569,0.0794197876486511,0.07831861934907167,0.08128571218417885,0.07923534516014963,0.07248270071008549,0.07307700490410537,0.07708047716314158,0.07730645068466961,0.06761412839896035,0.07555194934517967,0.07204846711460733,0.06802465868510467,0.08552528545430599,0.07896196374100307,0.07948888528979793,0.07183966976319284,0.0841712924739571,0.06458995181363515,0.07981205330868683,0.06863128697769894,0.07932574018691252,0.058584510554355956,0.0749711209810042,0.06258499091508396,0.07551986758663204,0.0834462231351632,0.07692837034160002,0.05957923634225405,0.07302640687647052,0.082502859951132,0.06893702034245437,0.0754906324791485,0.07693809373790421,0.07290501850417765,0.0814366345348364,0.08388913977047166,0.07281619561175474,0.07221301328431737,0.07760004497310444,0.06044597219093292,0.07413086346381581,0.06754571081744465,0.07488191057013718,0.0874809253678025,0.07060684342309345,0.07040104728664749,0.06223581044664282,0.07550813378310643,0.08341567476676968,0.06611127746781775,0.07386257899751958,0.08492356676566293,0.076927383688085,0.08156999387279827,0.06586567974328225,0.08039655082431724,0.07196613320412229,0.08559813293423693,0.07553069035590614,0.07844666122918907,0.07997502555704622,0.08958217494719804,0.07573425906411618,0.08447856082924078,0.06584188448471898,0.07192318503353004,0.07636430702523903,0.08969380424718972,0.07517086959770153,0.08110532816114444,0.07165080824399597,0.06808575920996698,0.07668378952586516,0.08791988420618702,0.06590988625351007,0.06940975749518186,0.064317980319782,0.06509968792220319,0.08443171004941319,0.07343716980418064,0.07952385771951939,0.062416025114098744,0.07731370277616137,0.07197628355566929,0.07320865913596664,0.08737588624608661,0.069818830678641,0.08432481794692293,0.0754970033733664,0.07771095826522503,0.08304377580517464,0.07748800987964735,0.07153026393440282,0.08811716370359207,0.07502307650489598,0.08395662105306845,0.0717185813611914,0.08149001669387279,0.07286490994027178,0.08262461956381117,0.07727345118868957,0.08440918087206767,0.07839100737007328,0.07964453797543285,0.07478182220398526,0.08537526107286673,0.07013420494230656,0.08798782746525512,0.08739632889550478,0.07149694755935206,0.07765170577340916,0.08034927175125299,0.07391915022008357,0.08107954115467451,0.07672242032799897,0.06765110066984889,0.07481276831292978,0.07439885977635509,0.06352108209741973,0.07343636188528116,0.08719044483760327,0.09105322756162315,0.08168621922646739,0.08102781190112224,0.07593830509383963,0.0859892650547221,0.07165602524005814,0.08922516504853845,0.07127896049403734,0.06288574477630815,0.0790832142231325,0.07105292895682104,0.08435520925455209,0.08170672618419988,0.08639482797214934,0.07789560143383598,0.08965253997125455,0.08785153731765882,0.06898165373534411,0.07693104616873642,0.06168514273836508,0.08872480364344065,0.07582180758425903,0.06929331775237846,0.06875306767169984,0.08437664908433477,0.07947351783519163,0.0698874198314692,0.07334685108506318,0.06884632132781339,0.07801875546717595,0.06697389480020387,0.08843683859226642,0.08173482836552126,0.07217958119584014,0.08838209422016892,0.07602442637771263,0.0869752324961034,0.06666024757577539,0.06852148451422,0.08475626720565901,0.0795369400451887,0.07738220297486542,0.08098150314101632,0.06752223678326405,0.05618883585124474,0.0770802630739387,0.07031615329270176,0.08203393258688982,0.07602182052975279,0.07030665417972427,0.06841657357141873,0.08741968261126135,0.07885093329715352,0.07627750614981049,0.0855349500797273,0.07438353939483724,0.08280788364706279,0.07069980901269951,0.07993560192915511,0.08835426119746756,0.07256495202565766,0.07825271407839327,0.06770128215945423,0.09615827309954593,0.08021649109442845,0.0866021360330087,0.08237755311569789,0.08458830092871236,0.07691344918350564,0.07541107497172186,0.07630799043898888,0.07112228788718761,0.09030403028714168,0.08055328834071676,0.08770156800436477,0.07472014159738544,0.06398996060682904,0.07982684615790897,0.07791952242763142,0.08540589268005006,0.08938142976839755,0.0706778093246882,0.06436563444950268,0.08697826237383001,0.06932279430482549,0.07393415884587703,0.08229442803622213,0.08540906816137915,0.08135140750578217,0.07865460850812997,0.08377918864733638,0.07400325108621678,0.06517810979919705,0.07139632883049205,0.07987838113630238,0.07487579654849456,0.08248404776594784,0.08359900401598773,0.06725918242760001,0.08514785797013158,0.06834288757776602,0.0731234413345137,0.07751234564899309,0.08724557125858678,0.0702490817148467,0.06562062594227291,0.08255799411020759,0.08271580682982746,0.07640957366155789,0.07635477827836022,0.08925678085355508,0.07456927791956428,0.08773678982902625,0.06179700428903241,0.07424459512769335,0.07844643824816311,0.08412216621746316,0.08440988025021928,0.08085726802483156,0.06400703165506541,0.06828435102866971,0.08385608315911727,0.07769676650929189,0.07252142727545503,0.09191824300949916,0.08306582605698651,0.08281084319326598,0.07560696212692003,0.07582702289004703,0.06920856113458262,0.06997066019293205,0.07843435421205627,0.07670777913851175,0.07885465388194038,0.07390904085363117,0.08671976270289966,0.09137963367425703,0.08645995307936494,0.07568429987279979,0.07090759743577298,0.08863085435318893,0.0833608539476006,0.06561113585264575,0.07049226871996006,0.0679841410582869,0.0867605696526801,0.07420614972609463,0.07926811856919475,0.08384689101262598,0.07027069434864422,0.08027729761225293,0.08866149409942273,0.07043951144371503,0.0835578128112352,0.07474791201935038,0.08067781151331335,0.06616804840180701,0.07490936821780053,0.08015897918960294,0.07337668171236655,0.07629939021444121,0.07341991191233355,0.0569422249218982,0.07409582349260012,0.06015822950831376,0.08223641569717205,0.07691000134259898,0.07824284611639581,0.08628514558685643,0.06453158769167489,0.08053328650665662,0.07656210479778663,0.05744875038778223,0.08499164147898361,0.07427517854809808,0.059415346240591804,0.08195376964974568,0.07104322474055791,0.0818304136606,0.06970425478158632,0.07282642410631025,0.08701492543942409,0.07140641433315928,0.0840737379205594,0.084125619159831,0.06404649332869541,0.08994617917774143,0.08521998098878372,0.08339938238785383,0.0847821955837897,0.06554252095726343,0.0882939065455606,0.06665490217052972,0.0810448792220794,0.07157819293224471,0.07201319256701286,0.08858118864720471,0.08526515040767467,0.0646793933699707,0.07847122365742042,0.06724107291701323,0.0789520039411701,0.08433703893274289,0.07043084467507665,0.08692078417830941,0.06839383508401817,0.0747100165152112,0.06918955101418194,0.0835973383337777,0.07407434141109266,0.09011204734822441,0.08235790786587374,0.08057243992894364,0.06726622243564669,0.07172889454077916,0.06446811597685792,0.07332880403467368,0.08701092520711257,0.06788186016814479,0.08211500251271411,0.08230766141662438,0.07167802165829787,0.08361103114623722,0.0569228373973261,0.08086177009967183,0.06409035445191455,0.07420621679798176,0.06201347512797485,0.08755942012273339,0.07617244573032994,0.08087912949894291,0.0818832876992899,0.08248983778199831,0.076013418624861,0.0773298262715012,0.08348785093187772,0.07446630987741631,0.0859532448598386,0.08388857189631779,0.07555770195422326,0.0748846125558654,0.0839960113665236,0.07843803343592706,0.08452037778950057,0.08235598759913396,0.07943095275384933,0.06350381159213438,0.08226420036910112,0.070825630642989,0.09077181098310487,0.06857824180339117,0.08563610780566043,0.0715715511334207,0.08334070852450226,0.07430601465289302,0.07768429990741782],"xaxis":"x","yaxis":"y","type":"histogram"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"value"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"count"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"title":{"text":"Investment Rate"},"barmode":"relative"},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

In [17]:
simulated_home_growth_rates = np.random.beta(10, 5, n) * (0.08 - 0.03) + 0.03
render_plotly_html(px.histogram(simulated_home_growth_rates, title="Home Growth Rate"))

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="3200d717-5730-44a3-83bc-e02a23bf2a16" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("3200d717-5730-44a3-83bc-e02a23bf2a16")) {                    Plotly.newPlot(                        "3200d717-5730-44a3-83bc-e02a23bf2a16",                        [{"alignmentgroup":"True","bingroup":"x","hovertemplate":"variable=0\u003cbr\u003evalue=%{x}\u003cbr\u003ecount=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"0","marker":{"color":"#636efa","pattern":{"shape":""}},"name":"0","offsetgroup":"0","orientation":"v","showlegend":true,"x":[0.07014050631959876,0.06674667069439738,0.06981226013655298,0.06713893871996737,0.07154049512530708,0.07413094944010623,0.06822845844607434,0.06527666489459957,0.06944823959945534,0.0654655310928474,0.048853290162149836,0.07074883312429955,0.06332888858563057,0.0672728754361123,0.060518140381806265,0.06534798492412391,0.06876166138362821,0.06564181293959664,0.06918060866189668,0.06361748968159306,0.06487208464223485,0.0632924744264173,0.0572663761475353,0.059926715959950506,0.05227319359883694,0.06478369032008911,0.06858783995727548,0.07208994260905804,0.07229142481085385,0.060715719443064725,0.05392645349349676,0.06749385779155756,0.05329433107728945,0.0645620904573983,0.0646310409608537,0.05976874625749075,0.05358724654901112,0.0690011405341428,0.06567757168995939,0.06891120781078476,0.06816123997690207,0.05987139079056165,0.07298980318759016,0.061433567577236,0.06543213405609671,0.059901182752534296,0.06966860859931878,0.06481732349885314,0.05471875114485138,0.06615685743006448,0.06569757255007297,0.05964795331456281,0.0670807512649858,0.06584994499590838,0.07630012376536796,0.06433375743229663,0.06781304837160612,0.06407826877226143,0.06413866170286531,0.06020749323012599,0.06158446445843393,0.07261128688016699,0.06636076020673881,0.06533203079541111,0.06077058834487073,0.06697502051107442,0.0682456539984137,0.05887298199785585,0.06433601951322501,0.0664927734437733,0.0712994495706228,0.06460109505978791,0.056338168837651664,0.07245619711999071,0.06306598538241673,0.06733353760062151,0.07224579058725897,0.06434304677516797,0.06879952155540557,0.07132993335369736,0.06575996190688435,0.05223475461383041,0.06841449959267731,0.051219766519719656,0.07154634140781949,0.07296813991303044,0.062311057116608,0.06345756952954415,0.05971443247300895,0.06142957458854979,0.06439931921202155,0.06799999529913017,0.056277936856013885,0.06479469131590625,0.06402424453975113,0.06647222221179437,0.06833834436125073,0.056669827335374956,0.07370393015559826,0.06087859656755418,0.06839872443091227,0.05351404482693713,0.06679611843231639,0.06058085373466239,0.06458191779370132,0.06418693549509243,0.06638330904986943,0.05755379207601907,0.06708721598457425,0.060611937836258,0.06988771092390242,0.06643990050803594,0.06120046960817528,0.06619177935277552,0.06726794933336142,0.07155556147883199,0.06263565259613654,0.06927227949096518,0.06399927522845852,0.04671836270539842,0.057878564118987896,0.056040765260010006,0.06961975792793096,0.05726063673683846,0.06501449357842157,0.06296018806807423,0.058492331536499534,0.06424363790333412,0.0683548814347199,0.07274137505043826,0.05511006204435878,0.06600946901707935,0.06458303885700326,0.06303893369392033,0.0683983400050612,0.05156828916352331,0.05693880984904435,0.06923641680923392,0.0589595352459919,0.05679035330042396,0.06592642483612854,0.060299990729007054,0.0754273554955992,0.05553921477163912,0.06770359421029712,0.06147601257419752,0.062373874697862104,0.06279934270025214,0.06669930900070442,0.06680919762807697,0.05718717944357443,0.05524776037878683,0.05737067587384233,0.05014434146002114,0.0621337749068865,0.06362510362601953,0.061260811132250695,0.06834491875280013,0.0648681642581807,0.07428498580892946,0.05855589231223641,0.06285479182894285,0.055823169889414576,0.0725379167610157,0.06608132917079526,0.06384351428893145,0.06993614655454189,0.0632238275992137,0.06706687100175389,0.05710201090150878,0.05610376278275346,0.05574358335273549,0.06101083378401159,0.0715176727239072,0.06734027340051879,0.06471242478699629,0.06247246563450537,0.06102707352181514,0.07048596632934302,0.07176525384146384,0.0681357636866266,0.0699134971674851,0.05422800298604798,0.06530527863509661,0.06061802718678615,0.06853233501789216,0.0688913385083571,0.0668147563219762,0.06530727637824729,0.06873767845715942,0.06829443342639893,0.06356363109511366,0.06027905450228877,0.07104807450951528,0.06151651199396079,0.06681239017539453,0.056312401274720814,0.06535045890503544,0.05994974107782034,0.054555234587484944,0.06962079707254337,0.06651282410352753,0.06374177850958222,0.07105866782861009,0.060970121947737155,0.07230579757076872,0.04611860448079608,0.06673289517761252,0.06202071671707618,0.057021407533089485,0.05935205161193187,0.0673114455003414,0.0656368929182766,0.05520579626192623,0.06671843571661422,0.06719932518006488,0.06389773551459083,0.07300616819673522,0.07023298381384568,0.07414454911219888,0.05993016939068656,0.06408727028088118,0.06713048039171507,0.0642334543433623,0.06527157530415167,0.05539097203883936,0.07353780954513078,0.06610301626728236,0.06361128616123066,0.06149788754374308,0.06270482714821782,0.07334925111505836,0.06066898366663008,0.06262910565683405,0.06676855766007178,0.07171019306007409,0.05551580648546074,0.05509790324417214,0.0622630663223792,0.06682576568933318,0.06300588837316248,0.06956935548618953,0.060824453808235215,0.07419517811300118,0.06664797839727933,0.06294962189838071,0.07149931601499665,0.06109259061458673,0.0636708509154498,0.06966956876071069,0.06684780460617037,0.07685891454395813,0.05626565734678381,0.06335901417323436,0.06486002691734008,0.06187130053324873,0.05935940415983816,0.057225373620302,0.0630745833018746,0.060477425951687494,0.06143758236549234,0.06500573360460661,0.059351719054472896,0.06167773700918438,0.06913381975425356,0.06471430416712673,0.06838230559671599,0.058727320575581374,0.06269203404280368,0.06510786894445458,0.07059734088646893,0.06512473353216779,0.0668085271887486,0.06092576305027761,0.07055059578258696,0.05716786125903259,0.05533889917334646,0.06615563614338323,0.05143403199521987,0.0584772497761977,0.06439282198425278,0.0677141822402294,0.06436979462468172,0.05477265727916554,0.0631373023050619,0.06873161877563562,0.06116174894981266,0.0618468090820928,0.07016645820678881,0.06813898546110037,0.06413720271542953,0.058025170833100145,0.05881434750135059,0.05886091354227006,0.07719289263603007,0.056467981462987905,0.0631870046576457,0.05618164577360282,0.0682509071971246,0.06537378344806574,0.05578974303730151,0.07465274964723566,0.06625449341634082,0.06824093188256401,0.07043884127369868,0.06339047915532406,0.07287653084057077,0.05594996621534938,0.06470144135177397,0.07315597538533425,0.06909012996149982,0.0645759133155733,0.06243561550250155,0.06181402215632518,0.06677211167323427,0.062334300269838445,0.05254916865972442,0.06489962137443639,0.055103999564358304,0.05233531805622664,0.06419659055800675,0.06223642484053201,0.05891618093880392,0.05264660346169578,0.07075302603145611,0.06757510789852511,0.06204641358454672,0.07070958248087128,0.06865802756648276,0.05623849834392064,0.0609516925926155,0.06782110422834473,0.05291938404646363,0.06277187745316098,0.06332680810468905,0.06491607767350446,0.06436114655387136,0.06481305522233746,0.05960055960723082,0.07309405139567962,0.057266540935528215,0.06745970197927825,0.0642508221483648,0.06621326841237682,0.05896125262167781,0.06870060117673027,0.06880580565440814,0.067649447588955,0.05391188882613082,0.07564645699574594,0.06808077802835708,0.06724319509321441,0.06825533097840539,0.06074643531685859,0.07094777494248541,0.06981119439101205,0.060974033159339504,0.07000078384273373,0.06686329313005052,0.06295801038460139,0.060116582702062785,0.07124873101866963,0.06176456858538397,0.05969736968912962,0.0581719940313971,0.05343709160567048,0.06466161596939853,0.06537056986780007,0.0679054057805008,0.0676929925342172,0.0687312315594315,0.062394034474588723,0.06882793654953945,0.056376086005578474,0.05220193658984698,0.06266404605358156,0.0721389184682219,0.06876529001850412,0.0671968470383649,0.06317161955743159,0.06563942539651511,0.06967733037584227,0.05814506466189338,0.0747785225862726,0.06344228766542229,0.06910273041668875,0.06937730961571081,0.060375984419989294,0.055021941294354934,0.0628035597704664,0.06146219218558755,0.0634236892054747,0.06645571797394167,0.05557173255511873,0.06898722302853477,0.05182584019627244,0.05518489580208269,0.062454883427622995,0.06469921789694048,0.0684219893218225,0.06638758958065841,0.06471200116969081,0.06928103842214907,0.06333633573941072,0.06199056805639739,0.050356452291232666,0.0665295703640786,0.06787945593480033,0.05653742945319086,0.06481204715856717,0.05756142164763704,0.06414307230944824,0.06926971122952097,0.06331925627153695,0.058746609150961304,0.05894985879480789,0.0585234057685326,0.05929038022609161,0.06685964576365644,0.06026161359519436,0.05937139660589455,0.07051933114905921,0.07051264311555784,0.07178481191255928,0.05535705811900643,0.043424414460273994,0.059844174528279684,0.05318366438246426,0.05972078113618405,0.06305297598780925,0.06775851572597949,0.055855365020299405,0.06513829618471575,0.0688293771744596,0.06448776503611842,0.06197229824995896,0.05903874370902013,0.06041426063439393,0.06451531007880307,0.06445744829993486,0.06624452787387927,0.056779707276644886,0.05661253169016171,0.06450457663893767,0.055584222181519316,0.054468645311648356,0.07376901230112479,0.06889545109978742,0.06054291233300366,0.06510237664067488,0.06828917619377853,0.06115580934847005,0.06301462770057185,0.05630308977175219,0.053747897712262196,0.05886479921466617,0.05923301870359013,0.06371116772886146,0.0505276534186566,0.06011788203427647,0.06675160915856074,0.068468090412977,0.06081394517326118,0.05828944821827933,0.06048638874361177,0.05709588814558376,0.058642119089025474,0.06307633680235973,0.07164338534100839,0.060886071552814364,0.07164343424995354,0.0642454620948853,0.0723964487179623,0.0646559697143482,0.06871173829295033,0.06353415274117533,0.06288296512230193,0.05758889521759013,0.07376476666390122,0.06443070129776968,0.06698245591504215,0.06465923119015124,0.0532136962950503,0.06029163138723199,0.05484521132571006,0.06801353288464612,0.062379158221792456,0.06556795851919288,0.06930493473616539,0.0658333469981339,0.06107645565127583,0.061452661855922734,0.054315665637930846,0.05184854487079586,0.05623439004443599,0.06494862212171926,0.060227476164801966,0.06789103236905875,0.05265797390991314,0.07261927221268336,0.0653901522778795,0.06757993249067062,0.06488768664879199,0.0650194276254698,0.06404905736255176,0.046183115792129054,0.04976745211390355,0.05996793904668489,0.0647344141618158,0.05975083208460045,0.05657705931328722,0.05170967947567938,0.05088390439069001,0.0699903236242242,0.061216537532743004,0.06782829456550477,0.06104128124080174,0.07179400258793431,0.06951449684346785,0.06313003360668773,0.058982262484712646,0.06321927292194433,0.0654449575618695,0.06597296461201702,0.0659448906199906,0.05956795909875798,0.06837205483852524,0.05087835447234876,0.06575934721343482,0.06860236341312415,0.07013746125527534,0.07201390773517741,0.06099446730178877,0.06566054897683311,0.06591095386416711,0.07212379605715594,0.06635178242780765,0.06096796618762175,0.058190669628524296,0.0618591238978853,0.06584275758188145,0.061243821410728386,0.05703990866960128,0.06748530152664078,0.06658491680683698,0.06334654483637395,0.0725333471210391,0.06504470466298508,0.06066069730065059,0.07178007623694138,0.05569386535916776,0.06203903321076721,0.05856637713793407,0.06869461167867619,0.06219898485565386,0.06634598934343111,0.06027002858559184,0.06358228274155812,0.06304131716331467,0.0589801632639998,0.06872632596038841,0.06680039927974338,0.05693438139426805,0.06709312352854037,0.05754968090189809,0.06837623741421349,0.06810639642526249,0.06454604673210451,0.07276554594138686,0.06039028198890374,0.0584644533175991,0.06711080529759574,0.05994866958029409,0.06489789075251305,0.06982738394188084,0.06753780081331683,0.06390955170370057,0.061748344193043384,0.07373158727197039,0.06585413604774222,0.062429981642425074,0.05065850899901276,0.06130886287801002,0.06997754459051153,0.057969334690396015,0.060489882854832935,0.06692280290263036,0.057886147927465895,0.060059868578317,0.05986168514771675,0.056679252788279594,0.04911624583773138,0.07037770825349761,0.062145007994330585,0.06586738628157526,0.053732593337589715,0.06744364539749664,0.05690749322155998,0.06858685083572393,0.05841677823620256,0.06214575590298389,0.061439018569187126,0.06718029871314835,0.052003080771588606,0.061607598692422855,0.0699278513530906,0.06214597606846695,0.07057842364692025,0.07166625418504408,0.06592622864374195,0.059703136476088686,0.05558918281850492,0.06607478943623168,0.05926697318439683,0.061547576238593645,0.060136928028609316,0.06564385789807797,0.07145157745634295,0.05922620633674265,0.06970008644163092,0.060767963414389045,0.07393502368397331,0.05571651831818562,0.07084122500658002,0.06256330412133426,0.06861703516721088,0.06764174307087042,0.06945561471239764,0.06658390442039852,0.06700830964689317,0.06901985677577499,0.0733976271772106,0.07244550382275279,0.059715880356847426,0.06526733240970783,0.07157871563559322,0.0520859567742033,0.06864141526804216,0.07064753283715405,0.06126817592238772,0.05900982336585507,0.06922840878879805,0.0669486070594976,0.05893340521440592,0.0587170762020091,0.06976425998983644,0.06568104749614984,0.06079058506089639,0.05798628849664206,0.06740955873079274,0.04462037958839948,0.0575050336322111,0.05961661762250873,0.06467385488119504,0.07381041269780939,0.061384742431851394,0.06958700751194931,0.06578736694989379,0.06776984094070675,0.0571289065101632,0.07194606479164092,0.0582130440211405,0.06075738761189643,0.06058150491242473,0.07266375347825374,0.05137134336288858,0.06348046468504506,0.05581061535162435,0.06143951614770225,0.05806701813485927,0.06645890726294237,0.06064016151799535,0.06162508914805775,0.07530192595027266,0.06163827787352154,0.06877460378973421,0.06571892624172165,0.060635561012662445,0.062381174188937606,0.05981266419274066,0.06045470834447218,0.061103773654743565,0.06089468623100415,0.07023653090631926,0.061260637141179776,0.05928529773866689,0.06619004512567155,0.05306990734312417,0.061577506536401205,0.05380582439425857,0.0646044734083111,0.059280993859054,0.06938221894725322,0.053873542336601736,0.059874944508851946,0.06236398132570754,0.04907484677950433,0.05192402121423485,0.06920445801493075,0.06505277358632378,0.05657290990689935,0.06207091297022684,0.06913535339612581,0.061022474405281524,0.06486436843587717,0.06235492631378927,0.05974418150019848,0.07016990336210419,0.07373862220798581,0.06118161203554618,0.06699448910194335,0.06003278055068879,0.06110946700210604,0.07483577456259488,0.06498551866082823,0.056689914298593894,0.05869306277165846,0.05450537936441402,0.06271455455048763,0.059684612748168456,0.06634281469658448,0.06380067685042913,0.06934733634381901,0.05789865929615957,0.05817814332291377,0.06157465461954267,0.05813912835915628,0.052852317026667706,0.05273800344994936,0.06742414270632664,0.05904337941217781,0.056545597658551244,0.06327535806732223,0.0749949259869521,0.06652121243701808,0.059058609207912824,0.05911704960196866,0.05662945660360723,0.0566361383253339,0.056501115913834866,0.06692863970543853,0.05847811394127356,0.06157735731294466,0.056857116957833166,0.05872625115197472,0.0664925622233222,0.06275144183781572,0.06647591076585116,0.061408011007824795,0.05954003769215578,0.05753368879049983,0.052328708443346375,0.05134743288293338,0.05996980859035472,0.07166707545411716,0.06737273827805829,0.06459649600061086,0.06134099298311407,0.05697592606628858,0.06807223752665642,0.06455264312597228,0.06841955536461397,0.06399848523979884,0.06793106857744793,0.06574020553458068,0.0609131173592861,0.05134863401551207,0.05345584324537279,0.05316702492959044,0.05239253537375543,0.07015145373865722,0.06833589441987226,0.06765066095218683,0.05839377696137886,0.06881501657897099,0.06562627172538543,0.0615209991892329,0.06457855039358283,0.07021524251008215,0.06185059855430417,0.060146705940668385,0.07010041427307567,0.06715593146711887,0.07219879069816904,0.05683146971956701,0.06649165729805906,0.05871466334497871,0.053244757237957165,0.07054541187636308,0.05564362570602453,0.06830765137787113,0.05370758854773116,0.049019658394247365,0.06918884131956216,0.06641886425393814,0.05716846583664356,0.05630310963907392,0.06209106305611716,0.0716251115147536,0.06666395989469592,0.06262548136371407,0.06356910344660444,0.07356923011772011,0.06575160618399906,0.05802782540384464,0.05034731524777665,0.07041047947750538,0.06572438059155113,0.07013502458333085,0.06873232568179125,0.06248453028363962,0.0740051140184588,0.04855067813886063,0.054863288967182205,0.06922313785724202,0.061380416141206984,0.07028655497671635,0.06096081565274332,0.06829795753415038,0.059377851553171664,0.05128368369930276,0.0690897479748166,0.056334536266469455,0.07274244689721843,0.0705891798521242,0.05744053973390149,0.05487209995622615,0.06335676069732596,0.06720349746009532,0.06795023038348377,0.07161072441305158,0.07041705321642291,0.05691261635572507,0.06728145436216326,0.06372214619917507,0.05260464712633717,0.06525853280165037,0.05832491683820182,0.06786705336110514,0.0718213745814964,0.0641714545057964,0.05897092290654196,0.06570713478931378,0.04623468712034578,0.06497373454785621,0.06761810351328304,0.06577053112530479,0.061038718379933896,0.06428908121306769,0.06201190896666469,0.06610023300833406,0.06832712575883351,0.05861773756843526,0.0540615728094941,0.06867116181863775,0.07171389031086436,0.06428673363789636,0.06591386964645704,0.06527291334707833,0.0703799214558837,0.06338589865130816,0.0694523119724964,0.06684838540220817,0.06762567403122469,0.06625432294623926,0.06763600689086449,0.06918777643123866,0.05631351649682231,0.06099387985555392,0.07172285598976336,0.06693942406658987,0.06236699985366526,0.06300496517354093,0.06678150933485293,0.06425256168364632,0.060673819670250095,0.04755915337351366,0.06816303422904677,0.05263065264829203,0.065112124986202,0.06775610876946103,0.0721511747471204,0.06611816522096961,0.06250570547311896,0.060159437577935,0.058779227257041156,0.07405701241706733,0.06411580189139217,0.06833825130265966,0.06969094786973906,0.07268385603765161,0.06599720677128912,0.057787311507935646,0.060308291665486714,0.055726899542976,0.05194620766956223,0.05805605208852921,0.06908960764423441,0.058707437809380064,0.06419571584154138,0.07371434919010117,0.05935011034963403,0.05991265209896996,0.07382245919692584,0.060215423908107926,0.06168374512058443,0.06364521286892104,0.06856615483257003,0.06740274694729195,0.06595779822064224,0.06112639063064799,0.05936048415043521,0.07264721130638015,0.05778936364684159,0.05643361861329059,0.05808026108924218,0.05628592139973919,0.05626373280990813,0.06693148818889597,0.0731710012078049,0.06438283020399557,0.06562304958983065,0.06442351508093506,0.07498258666421193,0.06479894733613115,0.0574851102727809,0.057674388517725286,0.062769465273718,0.07092122092607522,0.05521745678317092,0.06414796360627328,0.059987323840344095,0.05560702090811387,0.05641466984791149,0.06090446448116768,0.06680501044712298,0.0647903228987918,0.06263509658446072,0.05995307791261574,0.06455832195107364,0.06515363343195815,0.0712043794187309,0.06700389052831329,0.06256865411417288,0.06589171384600896,0.05167970724653422,0.060012677444909186,0.06922966965972169,0.07484016382573766,0.055711879955507596,0.07044595029822576,0.07332471526003963,0.06208811479101612,0.06409688756675877,0.05312953253116724,0.05982032414303265,0.06773868161585733,0.06494645463665483,0.07095430945024706,0.06772703849501058,0.06418069910349271,0.06336480447429216,0.055151102576378785,0.06720484237183255,0.0687931133488823,0.06624168671957698,0.05578684960039841,0.05829674856390652,0.05890870398860771,0.06579823546405797,0.06324631066155081,0.05458126912045114,0.058583005703131824,0.06065408201360426,0.06355121066124414,0.06023192680323719,0.06518887254965794,0.06237370219576166,0.05417728475547284,0.06613434361761067,0.06890858769147959,0.06556161919216627,0.06664135345996326,0.07771962192203757,0.06263617389616899,0.07119268249072229,0.061942153974099995,0.06550508551670525,0.06979592375183225,0.05823632992984004,0.07229921277668402,0.06579568446506534,0.06556633977957996,0.06780554329259789,0.06837342886118435,0.05879715698531532,0.05647581294858923,0.06871775530914044,0.07230731668449134,0.047460499740527434,0.06303200729502942,0.05944282515694607,0.058166550085524335,0.06137542226202923,0.05987438394981426,0.06966548510518505,0.05857254646840922,0.06618071304632911,0.057475824177832854,0.06710287503044704,0.06385574372669081,0.06489008713160943,0.06177670220728276,0.06289304746270163,0.07612789064656544,0.0540159893911798,0.06423133860190601,0.05792011287252999,0.061178273133136576,0.059949975239827494,0.062133316219259514,0.05618934579967298,0.058304722719500685,0.06564189814898636,0.07061595776716537,0.07226085494211343,0.06379352403968899,0.07067410935025045,0.05591941811194254,0.063126752391027,0.05705912902985162,0.05236975421781538,0.06646588760014523,0.06341231816631533,0.05760711542925394,0.07156786097623927,0.06534555494118208,0.05342827548462896,0.07393099037691958,0.06351454219230177,0.06383673586348725,0.06633073651333274,0.06852899831887313,0.06237873933801184,0.06249178624013319,0.07008430654842943,0.07021861413179192,0.0570553761195847,0.05093422687194417,0.06925666363785865,0.05398941019404068,0.0658399902229993,0.06918929952417867,0.06857949919073514,0.06502779995515368,0.06595075357624572,0.06528819586584617,0.06080822892696093,0.06826649142220609,0.06851320114067196,0.058843767834709196,0.06703493701922278,0.0521261451687659,0.06604615478268454,0.06925226630295232,0.054588667372417424,0.06774289775108965,0.061562923013686335,0.06172595662599871,0.07009628676578987,0.05890012482297391,0.07274643336815625,0.05939132157256786,0.0587324404304071,0.06221367966159792,0.06513010599698313,0.06288521642391257,0.05707252830405969,0.0626060161426869,0.07085416746375348,0.06392791336422768,0.061034788983279444,0.05281111117392741,0.0637179051918787,0.06457700308789566,0.05829330754583624,0.0602232628758003,0.07310351967914963,0.06754478650305362,0.06047415953343083,0.06457841961549507,0.06542472010631867,0.06027023389678361,0.07225512772700118,0.06288014071982784,0.06770539881705055,0.05681743950026908,0.06448662395275795,0.061428465506474926,0.061377914192945325,0.06106124698909966,0.0655083620142487,0.061865834474521715,0.06933666465683506,0.0527141353027986,0.06562110109059553,0.06885107428872174,0.06599806900096067,0.06257454892480202,0.06681583723128351,0.05265738704973828,0.06777536409386625,0.05930767626179785,0.06303197706804695,0.07030094200527784,0.060571060426783147,0.06538641730361304,0.06850067074086288,0.05804046591968196,0.05698551078044523,0.06869565976140854,0.0644530745582377,0.06188734390127459,0.05632045736176876,0.06781528034379666,0.05673189822306654,0.05336733563986816,0.0539497732378673,0.06575765797559842,0.05855744869226116,0.06194099080787058,0.06479253177464397,0.05772481815028932,0.056065013933434577,0.06439940802633773,0.06593371183054553,0.05535519702452295,0.05978948327866879,0.0672483276377735,0.06784081731464443,0.06130942870165797,0.0593219249487284,0.05959153590043295,0.06474653467602265,0.06593659916585276,0.057674508901878405,0.07337258522747861,0.06578524469227937,0.05754752843739855,0.060783062396212104,0.062264433305834026,0.0687975309336592,0.06979923883980094,0.06393830948912821,0.05489154226833344,0.06809639367140449,0.06512870787675422,0.06755520207549581,0.0537663404648617,0.06649337515745118,0.06593250693693953,0.0629651834018796,0.06357153355817899,0.05859168478683263,0.06788298806977777,0.06863401518897358,0.0579797953473187,0.06479121772044244,0.06348324321574703,0.06268183622075008,0.06274942200377592,0.05541885219330944,0.06760146193637129,0.0751583234351296,0.07514031075410518,0.05935980914458775,0.0657132503667924,0.05810903036989962,0.06932198590073779,0.06079947160199473,0.057470336764289184,0.05664828439546529,0.06993229600477774,0.0649263230670506,0.058212773192705417,0.06071175254201553,0.06751334086886478,0.07128496783812427,0.05766633739919477,0.06779933296638067,0.06435497869436624,0.05618267676219511,0.06226243820790644,0.058420221334716894,0.05998683899775474,0.06863313817309369,0.06906865215261859,0.06209617069394695,0.06137592378208606,0.06270935436115062,0.06110961389801253,0.06840610764599889,0.06254798199545838,0.05584443847010054,0.06993299453235674,0.06539455524713905,0.07489799274329159,0.07570956326479755,0.06152843584362839,0.063092775740148,0.06169149350642326,0.06473697094074876,0.05909005925515373,0.06384919468513105,0.06136478044291402,0.06252194231399821,0.055674442391945494,0.06782836598997131,0.07026375699481141,0.05201663026501573,0.06484946311849546,0.06550246898224854,0.07650797480415641,0.06539591524963737,0.06323809450688872,0.05418164223897298,0.06369936478784137,0.058740170770991376,0.055634621656039836,0.06291861228680729,0.05719142994486534,0.06095725503393362,0.06302065236283227,0.07395674094745311,0.0570297386444307,0.07101849343166616,0.07326361195752239,0.06842409844787983,0.07075233693716862,0.06704747356385943,0.06099132697248992,0.05999559998510201,0.07597856975952402,0.05825368480229044,0.05405195329418558,0.07063193659671974,0.062044999443140875,0.061178274052634554,0.06093145326705971,0.06952060917705337,0.06161985770124958,0.06010318105223869,0.058750999118987235,0.05882434053303022,0.05613679838725566,0.0679476972317622,0.07152792112187553,0.06097116273746864,0.05411601432984454,0.0642745480448765,0.0645916255781504,0.06709219427887711,0.06276274831708906,0.06227892929463651,0.06586408720678182,0.06254813607108453,0.05953240061097118,0.0591062822112778,0.06344569523249331,0.0750341985831412,0.05971243462252963,0.06786370683231668,0.058972781733488355,0.06339911552181171,0.050579705753524606,0.05862666413233293,0.060986935135261135,0.07428496816556301,0.06320065010764053,0.06370097750064041,0.05630412187853843,0.057015093161871644,0.05477639619413413,0.06654256756016594,0.052762784670261714,0.0532081828155697,0.06803525835588167,0.06763973489247158,0.06210365787770034,0.04642724554207666,0.05718245955167667,0.06409106395935488,0.060490989662312075,0.07175948901920055,0.05391318406055079,0.06701431429434465,0.06280645788064507,0.06079944063872146,0.05412082857337512,0.0568255141058443,0.06376626332386065,0.06356301421259192,0.056944770140798046,0.07388456731351636,0.06275918918227219,0.05632823191868045,0.07468064853272426,0.052639341721060384,0.05957958839039291,0.06937987041406297,0.05754516840528795,0.05367341039805126,0.05554163230378853,0.07119638521909293,0.06921387574007921,0.0596959029770099,0.06875318650782392,0.06549154461646338,0.07234907466865204,0.06521576897587752,0.068264192577631,0.0625742783327218,0.06417792646435189,0.060706095991289274,0.0661194478527024,0.07464562440804678,0.05962397196223768,0.05823910490543384,0.05877774953019599,0.06092684395992232,0.06820386265356387,0.06934614368028057,0.07073846537967954,0.0559264970553844,0.05807136072866764,0.06755215933470057,0.056313049680768196,0.052500360599977106,0.06267691038347556,0.07111903795155108,0.05984720274952893,0.07001262610511308,0.07201707169285076,0.06184980505187503,0.05838763909143341,0.06243124736907212,0.06792663259816914,0.06423610239654047,0.06943715976008596,0.06810250329044944,0.06330537693661548,0.04370180945558384,0.06500582851380425,0.060448055687043936,0.06876569396097673,0.06853187154237311,0.06584454942545753,0.06030273988119275,0.060652685238575046,0.06812121924543213,0.07096441297767794,0.06675134226524088,0.06742701750897531,0.06191747048804868,0.06179807219072126,0.049502174296146416,0.066109631667604,0.05543529267495435,0.057245453636431416,0.057837835551494715,0.06576574732546728,0.0612609763225291,0.0608994644216766,0.05627730121685902,0.06507746622001165,0.057456505607341335,0.06281475634214577,0.060010534597598614,0.06132304605801439,0.06461624437166469,0.06076138409428254,0.05754542006218502,0.06375103961787729,0.052362735702001416,0.06900613933711974,0.06869238758103449,0.06078025887498578,0.06853519994031707,0.06464327805851072,0.0677699370783871,0.07109476473307928,0.06011503674984195,0.06242826580438097,0.07302419771318544,0.0712219653197044,0.06150611859108434,0.06682718694589268,0.06267747535184004,0.06816570938792993,0.06501943779859878,0.07010289008559274,0.06196610607624691,0.06661420826484453,0.07051756567967754,0.06889855202634204,0.06807744889688963,0.07216713720761807,0.06558616654516952,0.06267481072540941,0.054598622474600206,0.06731748553275627,0.06466828862383281,0.06122408133712488,0.07269607350960636,0.060105593481478405,0.06866862075676564,0.06170002385123246,0.06769234717728449,0.06590884205503907,0.07225285101913939,0.06656105887612179,0.0693485572134667,0.06807224899026539,0.06551340149479352,0.06248355501660696,0.0731763197417406,0.06522526277050986,0.06484338874877447,0.06190848851825987,0.07352969187561584,0.060500879681237886,0.06877927303765857,0.06962629637683299,0.06376719172865562,0.07629546148834446,0.06306665842671641,0.06422043278611875,0.05915720671302294,0.06744813310348077,0.06864380769224852,0.07315431030000724,0.05761363132862106,0.06624498149290108,0.06887429203911072,0.05752923532573893,0.06136264711521805,0.06115390020479453,0.06999105456034799,0.06983286001746686,0.06975961199395983,0.05991544679942905,0.06990654248129974,0.05962224262972036,0.04768797202630913,0.06361127326273466,0.07211932540823213,0.07184622159398793,0.047603857190076926,0.05954128531384034,0.0606247716199257,0.06268026975435669,0.05381548560041967,0.051159312584686784,0.0535127527849272,0.05988681382101907,0.06207987237559008,0.06421216591411932,0.05792253638160287,0.06898142309304955,0.05501153226137402,0.06972618756530621,0.0683517796454125,0.05077701882588589,0.06904702383596859,0.05589294528904163,0.06516671320433436,0.06893759478491418,0.07367703661080427,0.060431596740364774,0.061394579066495064,0.06672074027523439,0.06485315710484789,0.0676408936488471,0.06220025566955129,0.0652360539437466,0.06310496279113659,0.06425235211867322,0.061342939697182616,0.06023169011422708,0.07009801259535708,0.06596878640894037,0.0667807825940398,0.06840582015582816,0.06389817493608232,0.06470703758776455,0.058543954805910306,0.06478746283097847,0.06050656660822348,0.06362180410046508,0.060603648653662065,0.06004484894650104,0.06885495092044258,0.06503108964062526,0.06677619653567132,0.06983946327085702,0.06151968334133288,0.06836154173659134,0.05079750613074835,0.06279028272227355,0.05368335491805144,0.05979389770603766,0.058854275869884176,0.06865248573501201,0.060522740720744164,0.05945161468952527,0.05602392047557084,0.06944834844072527,0.0729830867560855,0.06293863552844126,0.06356546850355618,0.067809115345872,0.06891640419801548,0.06300461766618654,0.0517907174408551,0.051521201582239474,0.06460712711052093,0.06564556607090577,0.070774182079435,0.06672712783654552,0.06513797091361506,0.05720101866709022,0.06944838346203003,0.06612705103216118,0.05530996806797803,0.060346441382174015,0.05274549605091956,0.06555017369942948,0.05462566186695916,0.06441640629877418,0.0627203431413968,0.06651006116552227,0.0547135951442509,0.05976604324670041,0.06525159235012641,0.05457914133948025,0.0570132424292662,0.07063278726709077,0.06153896105779414,0.06133938744180391,0.06722823216648671,0.06377968238174145,0.06150404119616241,0.0598895140279241,0.0671339209514018,0.07054315556104764,0.07010639548237516,0.06841439409920569,0.056202253253000115,0.057646484596635655,0.06318310690788786,0.06929031354916435,0.06465713337706008,0.06390592844638474,0.05835435835673315,0.058465868672752976,0.0690447464142146,0.059068668162680665,0.059472125014089466,0.074134799270654,0.06449176167200538,0.059467077758101373,0.06297716716467541,0.05011104195585217,0.06796321150432916,0.05929440298989652,0.06735958712700872,0.06396025868522215,0.07383510844278925,0.05421798366929445,0.06791146266220202,0.05690932029592581,0.05599760100308118,0.07149561834841567,0.06902295439118203,0.05785974239002466,0.056691221622660826,0.05399449000048313,0.06733736434563312,0.06543831185029796,0.05264894584056363,0.061204438222784305,0.0648051453353238,0.06586154278564793,0.06838390582601177,0.05560401915104719,0.06547521911254928,0.06846784733347172,0.06369700093608544,0.0669155342156645,0.07292650447179652,0.06321975541102419,0.06861314057261353,0.06937399607723665,0.054012557861311386,0.05311648662608896,0.06978216541255065,0.07189455658527306,0.05934320013759749,0.061134062205974546,0.07246891142195991,0.06372028721142253,0.06319930175128233,0.06071670495348236,0.0653098759652135,0.059765273028717925,0.06780559460713484,0.06097269406312758,0.06034472993862905,0.06328837320627208,0.0684017410239735,0.06445221859538322,0.06696149667636672,0.06598094092681625,0.0642305270209754,0.05543180109494673,0.054104225589280824,0.0714951896785542,0.05535066801858053,0.06306942550138853,0.058030536123166485,0.06816101953154194,0.061537417141862415,0.06541348810842129,0.0617380109393482,0.07100480405036078,0.07257614354914672,0.06594951952897589,0.07424172314015089,0.06680185548033019,0.07100663282345107,0.06643135188216365,0.06336788871004205,0.05093852933247732,0.06335798119833597,0.06548146298456979,0.05992865856367498,0.06150311299419606,0.07063789511931867,0.05552098364893161,0.06632464771654922,0.07404046685390567,0.06428933949180082,0.05185409871909054,0.06629905839391276,0.0719362358110576,0.06551506019301888,0.06932140126586173,0.05694487057571425,0.06455254188147358,0.06584718455206082,0.07194012658247975,0.07715736180543156,0.06087151114327012,0.06063306489295886,0.058326838497732014,0.06793863531919188,0.06623466651218055,0.057163386391058515,0.06143510736726719,0.04820725691385663,0.06309658159226761,0.05935336966560048,0.0719420130112299,0.0560734749312989,0.0646509157048392,0.07744976735654682,0.06984953468521571,0.06869969005077874,0.05925658100440413,0.05574466463910288,0.06792909879130973,0.0695379718325331,0.05835426433116066,0.05981635373761025,0.07453698979031895,0.062086801363110025,0.06963932435364173,0.06004202478860954,0.06182008422560347,0.06701511874443952,0.06621826319079127,0.05701531639536987,0.06735957300297889,0.06150925379886842,0.06512371125664036,0.07077251402221865,0.07268162237907538,0.06091706511800303,0.05744065120673098,0.06790220506367323,0.06923433764831637,0.05299181678826897,0.053573662702544725,0.0669067012763845,0.06674297724460404,0.06563173524807514,0.06831127569421516,0.07511081087113647,0.06069117995138118,0.069633622282541,0.07040150272035986,0.07372378084513626,0.05811642736670121,0.07177646224173746,0.06475338946985351,0.05085127881139969,0.06132777096700527,0.06916127828607116,0.06892270774514014,0.06287848624209444,0.06362993969438205,0.07052375833432636,0.06746906270356617,0.07085313435918496,0.07076999064846315,0.06771415673729536,0.0545869032293351,0.06996323837910404,0.0701247114443959,0.06674536959967142,0.07205653296148512,0.06359907561178763,0.057729211324875296,0.056054087593374526,0.0697322718569495,0.052916327689938064,0.05533147936553644,0.0605306655829313,0.07055641338149843,0.07248847276913277,0.07352090806235421,0.05679183170601437,0.06397524691618071,0.0589163867581184,0.055681149760107976,0.06931813910004783,0.05930026029559586,0.0732201726320795,0.0725240769831292,0.06380040767140877,0.06650511493664805,0.05384233405647988,0.060740089004894554,0.06593944610780872,0.06706075264873927,0.05202668068259717,0.06230339115842626,0.06408798588136033,0.06069555733856438,0.0664023573423822,0.06391591496339694,0.06566168900622565,0.06354497170204858,0.06578261103003855,0.05039479040138463,0.05134049926143945,0.059954222554770044,0.05699553363136649,0.054780010785515254,0.05565690995328981,0.057600424443082415,0.06764278659326373,0.05817719928327526,0.06156241878634913,0.06355954206895399,0.06035731501782153,0.067428689437215,0.05666517939583727,0.0702872004799329,0.05741063229786743,0.06160778210296116,0.06350641341323915,0.06410865360722381,0.05706679127148802,0.06011967112965169,0.06766812637608101,0.05944565019758122,0.06480278444420079,0.060249326174954695,0.05772364515986586,0.057663101968115985,0.0695605985872714,0.07196355868764318,0.05624363038240527,0.06160779975629154,0.06352601540021155,0.06787368215274237,0.062278808987072214,0.06631267370601798,0.06437403423279521,0.05937932102960615,0.061772520949302745,0.06604810237707409,0.06858831365541719,0.07372446364588854,0.06508694821497707,0.04763310943897693,0.05357918894107608,0.0618199957308529,0.06682714772343945,0.059824064078034866,0.06985619974020033,0.06652113549409558,0.069290866631371,0.06276878133807251,0.06295304000048604,0.06250123813280228,0.06673870354588177,0.06520249600231291,0.06777358201374323,0.061856871956064145,0.05465164807274703,0.06707113759758093,0.0590288278433484,0.06459449630273295,0.06117319282695067,0.06003540403591209,0.06306354242400539,0.0587052897526614,0.04877949978627772,0.058543260799657146,0.06368777484695907,0.0629690943665116,0.06668771621968106,0.06672997638790405,0.05980287554943513,0.053893407566343324,0.06987739064686288,0.06829494748926282,0.06688673027650727,0.060962558560512164,0.06301734612455295,0.06739166394897539,0.06731194232348449,0.05832709094057556,0.05200884513291249,0.06724562880100521,0.06353020693948375,0.06919213729491733,0.06381501378613183,0.060688240162551366,0.06804087687594348,0.06557509655064733,0.05819803818540434,0.06817299693948653,0.06291770559920501,0.06952544872112026,0.06568276693632766,0.06367514596969645,0.058416349473232126,0.06481786687332294,0.07349658296536629,0.05348895800144138,0.06724526651516442,0.058533296307636276,0.05817706497528477,0.064579282707078,0.05841391189011963,0.06570371389202284,0.0654613075139403,0.06962534586367905,0.04959974573442785,0.0708538702040285,0.06506882789573268,0.0666369861126197,0.06351187223699736,0.06726421173868272,0.06558031944581338,0.07090235507322323,0.061737131632281,0.06283963184056718,0.06722726029718133,0.06276995645225045,0.056890005773312935,0.06462608894753705,0.07001236425923925,0.0684868153493529,0.06608291163104948,0.05054814916553656,0.05871299860719727,0.07253174755888499,0.06752600756765695,0.06135992053954389,0.058439834607307395,0.06346699538494673,0.06722337091213282,0.05713980616966474,0.06827580294461601,0.07028305791410017,0.06511298279284039,0.06406514896551793,0.06692457090972807,0.07058478942593341,0.06756967464033439,0.07023871381398725,0.06524236568367536,0.06881533747272131,0.05966972193801204,0.06758865076810827,0.05929472774882842,0.06092217387843306,0.05931874054405757,0.06061300126962613,0.04749372198925305,0.06763293693758729,0.05919879703559426,0.06761727493873257,0.07367946692283753,0.06785914434159626,0.05770414375199421,0.056510247020160734,0.06119580526708948,0.06675482159493915,0.05813775947772565,0.06378026135190826,0.05987766709553864,0.06105230327278727,0.06482440073025296,0.0763611750689043,0.06421694086419105,0.06731316632080012,0.053112789201144844,0.06411336705303433,0.06866301186303225,0.06092713143244974,0.06332023123294551,0.06791550101680834,0.0749964346141907,0.062153826207791835,0.06568242616589892,0.06889593168127842,0.059702985662109666,0.05504301764211916,0.06476200104025827,0.06124071929938282,0.0700369963587501,0.06490005188696013,0.06428185000389305,0.07444529165780897,0.06482775465927693,0.060403149273842396,0.06084468866672685,0.06596714994778041,0.06223300104464303,0.06292736084118027,0.06772709353640859,0.05434739003558843,0.057422259722457084,0.056956167607494196,0.07007688245494949,0.057669730085293254,0.06525047317307106,0.06967396787411365,0.06580565655313725,0.05785865418200987,0.06264042278437554,0.06417378591460882,0.06504571812056704,0.0563308157288355,0.053408670949904355,0.0696785867576981,0.07190365614328437,0.06184408214616622,0.059811095551017865,0.06712628112928827,0.06378721762808755,0.06295299817765071,0.06250140027318424,0.06905479892566241,0.06697371410140925,0.07240286730330026,0.060082319609591844,0.0724161918232315,0.07138840087034098,0.07203893087192782,0.06299107318093469,0.0534867205880861,0.05501654302568769,0.06245200949505585,0.05686751413745614,0.05888881578816804,0.06154439945146625,0.06789041254095055,0.0684034044012073,0.06797333743999157,0.07167562933463198,0.06524630660132866,0.06503093529175316,0.06958165817869265,0.06328519307925827,0.057174643021146096,0.05390266135562521,0.06277344537032203,0.06074403549735262,0.07613617653543546,0.06483926630203689,0.06681838325338146,0.06588508953111291,0.07190000843152891,0.058446979775322205,0.06360895189299975,0.061838565524645894,0.05774978617912702,0.06933704064072,0.06512816196677204,0.06019954505168211,0.06873243867993462,0.06982550329181494,0.07180979730586268,0.07023260700106701,0.07126133932643275,0.05459758815519226,0.05647685763539404,0.0660030341985278,0.06450005328221672,0.05855930928157069,0.07036337961248182,0.06348039767949398,0.05986606311134973,0.0691502311483609,0.07192614884624961,0.06767337574364758,0.06705516907218062,0.07004526728091276,0.05516378654306449,0.06319977528019312,0.062444223383013524,0.06440879144420258,0.06648912391612553,0.07236379122971126,0.06682372926706989,0.05980864546329406,0.06328073607298432,0.061440931052103034,0.06353454958768254,0.06988822299399627,0.061538404974977194,0.06522822557918592,0.0655423926008088,0.0673698693692546,0.07349883255841902,0.07271337873362042,0.05978542985784652,0.07200751328209722,0.06781239837609623,0.06277949129152477,0.054411010004382324,0.06802879130505339,0.06819371413512584,0.0559233346619322,0.06352035874283124,0.0686325880258869,0.062139129004941784,0.069288997448711,0.059522903028935564,0.06028765247039573,0.06754923623337923,0.06655737357135036,0.06615266666663877,0.06003161088130941,0.07104309201821599,0.06534473492350829,0.054403791327779574,0.06195977051127213,0.05647663963719804,0.06731426487378273,0.0587613036638434,0.06455675518781462,0.0646854839270257,0.06497003452630473,0.06507292213344623,0.06614904799058005,0.07228376075680462,0.059644150467983506,0.0610778569382768,0.05642410952057073,0.061518277168769446,0.058892114891276326,0.06215316522651819,0.05195130678770141,0.05522543095920583,0.06625725948411978,0.05409287662589813,0.07386927679712854,0.06491498740381665,0.05827622951269658,0.06204331019246194,0.05815133772926324,0.049319676934074866,0.055301464650963264,0.06579743602972041,0.05840316442282418,0.07286575005922771,0.07040950826262149,0.06353454547737052,0.0668194366169512,0.05020780032079584,0.07019468785795857,0.07336999851419333,0.06208267598392329,0.06613100837743746,0.06523597529490202,0.06392989260603366,0.05394347197891763,0.056766565259501134,0.05416613751270742,0.07098826916891673,0.06323448517098952,0.070834807270302,0.05456683106512622,0.07576239693567102,0.06958119081870186,0.06963276181933406,0.060134239739761036,0.05711728078653758,0.06626247554993883,0.056562005551332756,0.07113853816711752,0.06578615726710857,0.04835103058027748,0.058661743224792715,0.0653113376878092,0.06050025931938033,0.06427891960780718,0.0631682652318469,0.055258986831719384,0.0632185388448285,0.07027183988458802,0.060698388212742135,0.06714727111801827,0.06438855495156585,0.06421663170866684,0.06217352308970922,0.06384493929755505,0.05565401535442295,0.06650751446157574,0.0653880824920309,0.05705794705727125,0.062202751102535196,0.054418085277241096,0.06566084584298065,0.05889807351877638,0.060799227561616695,0.06125801886960535,0.06262675400749787,0.06280021658024032,0.07459651052425553,0.06530916912422643,0.06493952009823606,0.07058070861137186,0.0639844814814183,0.06721995312142653,0.05225146888595171,0.06500242128116082,0.05997776511069745,0.0624337311616968,0.06633324304030047,0.06409655717889534,0.058722502886663794,0.0649072983149545,0.0646304961639702,0.05423172553640057,0.06251512632993517,0.060085113211679364,0.06557632331349711,0.06128165051728036,0.06415005755989281,0.07480686634134764,0.0583078405730867,0.065031516491357,0.06260000102835545,0.05854301090430368,0.05390616408243118,0.05863934877563419,0.0661196687729305,0.06389758414180549,0.06794394393890131,0.06272490614404475,0.0712320923278309,0.07300319007019201,0.06366241163890851,0.06573236557646908,0.06923703632166443,0.05821509261019034,0.05434551776458881,0.0591157676005053,0.06875867580351955,0.05902761760471925,0.06262390875443812,0.0642072366559743,0.04781555737185818,0.07122215379779276,0.07089061195628535,0.06375606650509469,0.05632053924299488,0.06444507360191815,0.0634288579787144,0.05516014221313449,0.0590053636059505,0.06271952273858339,0.05905618983393604,0.0713274758229524,0.06413022354761394,0.057628858248353274,0.054287490062865096,0.06035242780099116,0.06872820670035899,0.07229619307024904,0.06670799813766554,0.0634317659336735,0.06342346912438834,0.06716958543102791,0.06331400499220421,0.06418352189887591,0.06394764608865255,0.05762652735575548,0.07047765317581739,0.06087163581457951,0.05025767412347272,0.06705801104000547,0.0666199416633726,0.06415424179268439,0.06637361457201667,0.06916774074104039,0.06530557938032565,0.05175513277733626,0.06677583890398003,0.07539104138605571,0.06952884222891322,0.06804508617056915,0.07305660973701666,0.07096168835033197,0.07213091652501391,0.06967174133874987,0.06210537161768688,0.06864086171320227,0.060654772683723664,0.06959065579357923,0.06936816702344653,0.06174798477622633,0.06929683316965374,0.05385525259379771,0.05974918722902481,0.06572506483056223,0.06465806680304256,0.053039025046587696,0.06671219058192762,0.05911001455464307,0.06739469452003578,0.06113323036080999,0.059401426936033064,0.06613869103164785,0.06580016343305163,0.054856931284965896,0.0567786288467549,0.05799279997501185,0.07129586022668752,0.06450959966501038,0.06850496904592976,0.070669563028287,0.06014914171979897,0.05844995038584516,0.07443001951933886,0.06247933619020135,0.06818097904571943,0.06683822652462656,0.05914297317603166,0.06312104127035191,0.055003474145219625,0.06503254548551686,0.06912644711289337,0.06610249703575156,0.05954960897832602,0.06443574446330454,0.06226370863624522,0.06050003200392744,0.06885893173931698,0.05897217221128839,0.06753728172957868,0.06140511320081974,0.06297518385474904,0.05277475637354534,0.06881195074022717,0.0630325072520824,0.0625743189063431,0.0683429231344875,0.05910368588180876,0.06587467757780666,0.06941495394230462,0.06325610062984446,0.06710219959463337,0.06584542490974704,0.053600097527185545,0.05876423161141318,0.05868686512847483,0.05911824176165924,0.06974592491334916,0.06664600143838642,0.0589055030107974,0.058048672411946264,0.06697423684516866,0.06058865454601817,0.05465455536835477,0.06158793908242543,0.06641811406812093,0.057742025254507987,0.06660741977385617,0.0676586919431197,0.05598308509678486,0.06252320268511638,0.06685674312372289,0.06553006092647887,0.05830642329915668,0.07246518871194725,0.07287218664436688,0.058129591063889055,0.07098746780880216,0.0671494219338804,0.07101563132190017,0.07290420338363408,0.06068982412374067,0.059604786434749375,0.06656288403747063,0.0661607016816077,0.06437010847860591,0.06121943817510665,0.06115761946192988,0.05458199334591894,0.07205470939358613,0.06511485390030264,0.06839878252125381,0.061790197904883215,0.05722164403520245,0.06332375052736583,0.05855689911330192,0.05411432547706982,0.0741564713759586,0.05579802064532306,0.06029240070649631,0.0609694244759434,0.0532728457891367,0.06626226757001347,0.06362483244148703,0.07116226473394921,0.06171541213167991,0.060440085387506864,0.06169549330647428,0.054907015172308074,0.06805244202052867,0.0656527460239885,0.06262901229166276,0.05789118052537283,0.059806211753620414,0.062250919486018234,0.07347367540246277,0.0525896056875752,0.06271808455113273,0.06402840128804613,0.0672752917239895,0.04889697587312108,0.05613776592943513,0.06726536412659045,0.05845251878318743,0.060811762707985634,0.04983245518158062,0.0695509943096451,0.05986733137502054,0.07415507456705678,0.061221870766289224,0.05153108205962349,0.05927444803521971,0.06478831292632889,0.06008209631820977,0.055783792399547685,0.071803681228089,0.06585138126517551,0.0668306064992881,0.06489931025461212,0.07067475653909137,0.06377611770856237,0.0482706250927224,0.05500663982232061,0.06708114351396197,0.06874915976591933,0.06436701434156836,0.055086561495896816,0.05403339930673506,0.05722580951283408,0.07242021566945012,0.0667073915287667,0.055726658720646,0.0697927778222496,0.06620525481982982,0.06538686027021247,0.07193241467882178,0.05546028435587394,0.07037851182572345,0.0699128699733452,0.062344366568954854,0.06429894516575817,0.06484739634965583,0.07128616735951068,0.06290526776941344,0.07313531808908788,0.06658793052853929,0.06187445295657195,0.06842287969540539,0.06121193885810415,0.07090557016410756,0.05872702236357624,0.060216694333028606,0.05897285200812587,0.059535656349963724,0.060505668855176865,0.07124359705196975,0.07778342969993293,0.07152754700242142,0.061490283339782716,0.05810277276816994,0.05759856423814595,0.06854324437693016,0.06000113062026111,0.07211672693887086,0.07186767652238865,0.05579516255772378,0.07001997002802653,0.06517093781121111,0.06316080356204071,0.059175919150559406,0.0708760720893619,0.06546349688509012,0.0672299674191904,0.05192738178657447,0.06789929264281977,0.0722164047895133,0.06096233711345975,0.05363980628445046,0.06703256768086496,0.06961416254014785,0.07664761245681728,0.058746140453516935,0.05756666323463369,0.06430077646555485,0.06462861764417539,0.07068042460095675,0.06940087914809309,0.06574464926795594,0.06843551352946853,0.05991005900769325,0.05042464026949428,0.06018602804667292,0.0668633511891081,0.05943820251176613,0.0552641119530091,0.06675550390912809,0.07147388025129198,0.06460585453484152,0.06218078454001263,0.06153558793578703,0.0729717827155702,0.06395915097756,0.05186172625954945,0.06403590594498432,0.0632993339512953,0.07435381285611604,0.05930695013968004,0.06467203286073842,0.06934872742539339,0.06253321596282913,0.06382147828054749,0.06946870852920292,0.0636969633571811,0.06348952740115263,0.0621043507232961,0.05994455495256594,0.061372299334427026,0.06936270894456212,0.0643416609649546,0.05991493255031011,0.06384553737240434,0.06771189658082882,0.05780604996915621,0.060355325677270255,0.05998955120130868,0.05983780425790567,0.06479757386296453,0.06889366047505964,0.06978578318842865,0.06671538689430906,0.06053781941516227,0.0671706641037644,0.06334387548013604,0.06173202510199374,0.06234457218968138,0.06067968010704751,0.06575511882187758,0.05972600815972762,0.06448630958131366,0.06073579332956093,0.062337331155738855,0.06021946593663907,0.07280529147675557,0.0643081279165886,0.055646721402129375,0.06749246006467158,0.07017752439030443,0.06495598320284221,0.05350156170069295,0.0637120875764089,0.055532388971266645,0.06883714239337932,0.06264450735347113,0.06976845472905466,0.051392161191379536,0.07137454740046945,0.06600293147420284,0.0646882011856931,0.06408897140630068,0.06741446158041631,0.06967925707112826,0.05453652805592067,0.060546912382315674,0.06525774352315228,0.06523775895721332,0.06540890524765165,0.060734402149090136,0.04653233608273917,0.05852271939079501,0.0642488090996011,0.06559879776004032,0.06556433507582365,0.064428664273326,0.07533132651619717,0.057522458497800036,0.07436050028733862,0.05695732265505772,0.06056998386206208,0.06601899033857228,0.06136060073881418,0.06426992027997526,0.05831866369952099,0.0608174702160133,0.07144598083518014,0.056183430772911165,0.07466657356641059,0.05713307744225812,0.06290016760202835,0.06675677139892396,0.07168282183997603,0.060278870545376506,0.05433947311459593,0.07097635249499558,0.06946893336324929,0.0694037043935733,0.056976717945308855,0.05475586268885695,0.06903968008636005,0.06167156244135052,0.054467971725226993,0.06181305391602396,0.06186552734355802,0.060199391759999034,0.05773637740853879,0.048330910984488946,0.06300386961487847,0.05579588412302973,0.06831214875011808,0.0705353874710042,0.060156977269659434,0.06159052158103106,0.06343685258781276,0.05790965742287538,0.05844619840262601,0.06175201195836279,0.0697364618287318,0.05258807394664731,0.07098636512810835,0.06969149411930986,0.07375510021681272,0.06694053867271152,0.06065494127929849,0.05469900263963229,0.07207630907058339,0.06775772767429514,0.0679502757249283,0.06873032241501292,0.06943514090894916,0.053653106645821916,0.06429343016512104,0.06964715414627182,0.06433099315152407,0.07094962261926363,0.058393428773567374,0.07030352076724776,0.07247383691443654,0.06459587616242957,0.07066692368081859,0.06888889558814099,0.06449848254364003,0.06272239440961011,0.06718452336995062,0.06891739619335424,0.0612275912122654,0.056215806810085346,0.053998090682018406,0.06165488139672058,0.05813857807901929,0.06517567049479672,0.06438736943154486,0.06918372444876045,0.0457273880544945,0.057191819621164236,0.06371927140850348,0.06782581733187071,0.06869095924833994,0.06801241204674224,0.054436532521438735,0.06326913731886999,0.06779922750366685,0.06322619894728834,0.0702991390398314,0.0547423448364331,0.05575072077087702,0.06392111026679767,0.07235838214422002,0.0646681075046141,0.0561750663643729,0.05830625211756563,0.06596495537654672,0.06073688662191977,0.05387172032979315,0.06284740450837137,0.05387343537716864,0.06426030820848319,0.059309583072173586,0.06880118962687755,0.06067468535298072,0.05185415921370587,0.05762381633853662,0.05956564907724982,0.057489079524466385,0.06709138665414034,0.058573080295622255,0.06052365273296738,0.07294339331840144,0.05423394941755877,0.06364434664558191,0.06315596901167095,0.05500401073509288,0.06183771170475041,0.06264493786269383,0.06533546366049642,0.0708509140894591,0.06586180190538446,0.06319422812800357,0.05659936992966991,0.0667507858135882,0.05286447513955422,0.049007563060664616,0.06954259431249504,0.05804468113796392,0.060247988948615674,0.05778644269809619,0.06343562857377227,0.06440483006564758,0.06377489365566086,0.06738418894034344,0.05966071679558586,0.07173418957153938,0.07187554420908329,0.06703808424677149,0.0665987344283884,0.06782823242242317,0.06472629317021966,0.06806659377095658,0.0603909412174828,0.06523167463678231,0.06713523843362036,0.062074040500736775,0.06338405038602093,0.05713310744920881,0.05192259563317447,0.061025745728609956,0.06618296228440862,0.058311751855412854,0.06631434517661411,0.07379569332921855,0.06270309762259335,0.0547955720061161,0.06616388969994527,0.05988975732574649,0.05977453103909397,0.07059899147417267,0.07010214170959365,0.07391881588115523,0.05994969579517233,0.06887409842861533,0.06905514016844384,0.05508614915241798,0.05745285032860581,0.06439562401715587,0.06412896244527583,0.07458269236904004,0.06610675733744917,0.06719489696903297,0.06474942225440392,0.06006360624454797,0.05944018826469116,0.06933200148127808,0.061753478741051214,0.059755646160269904,0.06410590717355455,0.06379496711474478,0.059440634096433145,0.06154304822893535,0.06624971404732977,0.05792109747468066,0.07340860253925921,0.07154459852659267,0.07038069664192037,0.06644415362446562,0.0653251265423194,0.07552136402433891,0.06834624167113588,0.06948365254920699,0.0624019619311596,0.06442555121827001,0.0550374129149445,0.06982474989895021,0.0596417401698609,0.06255754872552996,0.0678759076864362,0.06499312514873823,0.06691136996920183,0.060661710700239055,0.05572047990465755,0.06712959979499009,0.06125756475675797,0.07090221242022149,0.06435495395487767,0.05464437644638759,0.06031825767796335,0.06565035961136917,0.06791449667916705,0.0709694338438207,0.06818578083298882,0.052125398108872345,0.0638436433994465,0.0639040197464778,0.06671115441437324,0.057616633552528315,0.04690676411821716,0.0588372570200624,0.06353365824299116,0.06976567371755435,0.06794781039565634,0.057650581591603065,0.06039313629624511,0.06561482756080247,0.0637051179387711,0.06625484611412788,0.05735594159365276,0.07109418863112511,0.06359435872243097,0.07186179836100412,0.06176184209530868,0.06225968230513953,0.0709613003871761,0.05744083914493479,0.061627909209669325,0.0630931346219809,0.058157479373695295,0.05227952512463504,0.06279729727404582,0.05829130608247744,0.05046327057663125,0.06516263518317364,0.06966077714846608,0.06730831339879852,0.06428053452725963,0.0630033314052619,0.06569365874267638,0.06144590808791444,0.06249170255195911,0.060842371831022396,0.06700076370489108,0.059756608949179835,0.0574590869130254,0.06978282990768925,0.05228301915610249,0.06599897122644688,0.06027488665233542,0.055254259473641376,0.0671715654767264,0.055076782879657296,0.06255995003403864,0.06898337552572303,0.06015940121848812,0.0657389793149912,0.0663452056838798,0.06925142611721558,0.06573770031271804,0.06187520060035334,0.06588587220931505,0.0543478472799928,0.06758367654920133,0.06644545902251528,0.06668208646851151,0.06538370299257172,0.07086165989945242,0.06429613240350802,0.0465747548712242,0.06608560566126603,0.0619427552581663,0.05102620992202255,0.06743303267444493,0.07339474311190308,0.0647197767600618,0.07130910493449433,0.06754354813914466,0.0660717096177318,0.052355370447176974,0.06552634856453371,0.055462500420108155,0.06947873416380795,0.06083970658290122,0.06933693145638553,0.07205539078668236,0.06273906157686973,0.0657317689416036,0.06772936335118518,0.07341950693038904,0.06695917176370292,0.06222246288841395,0.05627072699341076,0.06411715357720876,0.06790398431450564,0.05828693265384362,0.06380306054318338,0.06406794697863433,0.06792489653502864,0.05561483432189518,0.06397156874093388,0.0644601733304685,0.07372916272735308,0.06766661217585127,0.06711681448966367,0.06880843789424074,0.06660828974574853,0.07481079216854564,0.06329126173093005,0.04932984075152322,0.07246435517531966,0.06480148598658446,0.0727616642746271,0.06210226706494125,0.06580169461715207,0.06114486171710212,0.059796804827383934,0.06234684803717177,0.06605502943134477,0.06402850359609823,0.0671818584377587,0.056529488723081146,0.06603024131799369,0.06104585642064174,0.06819802048092238,0.058190018449654254,0.05911928680505488,0.06324680118507398,0.06374720892870162,0.06691157872556876,0.06906350581620782,0.05715752235531718,0.06544952404862414,0.06033389587138735,0.06681805867916057,0.0672825083675663,0.06333025697866004,0.06466626605356857,0.0748869873813541,0.06912827359266235,0.07098174688733366,0.06322956144735535,0.06674781702648733,0.07071404399109027,0.06051152951882781,0.06520888614513347,0.06112802361284793,0.06487804265939381,0.06864661758967402,0.07099429710476056,0.0609390856343281,0.06775552866815854,0.0730352761124949,0.07108170820942056,0.05965034934480705,0.05978643882156127,0.057460256525814865,0.06707292559624374,0.057146854407683814,0.06936754672407636,0.06675804372349461,0.06382425678961434,0.06679482554290694,0.06440439729223707,0.06547416620552934,0.06768181146154806,0.056855485031236004,0.059319974741248505,0.06724099837469436,0.0675793082755607,0.06284994408757547,0.06530885691149496,0.06321457815732187,0.06502823857130785,0.06463655013154368,0.06610869471503952,0.058249734271058076,0.057794924381718224,0.0695980326547631,0.06271956622805516,0.056717165256915454,0.0640750551132388,0.06390828065205759,0.06439701803345896,0.06775987177418583,0.06545757103589275,0.07050095252173919,0.062070752853574565,0.06221981161917183,0.06617874229834744,0.06667002549145765,0.06823293710725525,0.06130197568722453,0.06611854804557304,0.06824668210040191,0.06775810911632542,0.06108490314541472,0.06297522406406172,0.06859891930072581,0.04561510742859434,0.06743365813850617,0.04955271061006709,0.06598726138060276,0.06830306578544718,0.06882878066109138,0.0644919082830282,0.06458935674809815,0.06696011571440583,0.0595607244485466,0.0687939245461304,0.05882182763310595,0.06363632173071129,0.051503340734530945,0.06981203825584864,0.05301747701626258,0.07405691622360504,0.06976830404408357,0.058001148899638685,0.062453320877093425,0.07285155618982012,0.06177044383284627,0.06226679805290846,0.061793589239979574,0.06648415632312554,0.06258436455806521,0.06489712078369916,0.060221717513472536,0.061711171412312985,0.058165372089155495,0.05023015801377753,0.05668522741585971,0.05060683900663081,0.0643799082878358,0.06806235670276492,0.0713254067001608,0.05858821347429871,0.059540284518687855,0.05768176383596817,0.06642511648534835,0.05618025256875143,0.06759482972917298,0.06024704098417423,0.0587595272556032,0.07331132365209254,0.06847741808295991,0.06820893236726797,0.06903598449207494,0.06227620752523745,0.055599420823446745,0.06457490030546267,0.05475920725665619,0.07253355529198521,0.057777405600693346,0.054307882604780144,0.06915998216044439,0.055050217337676524,0.06550163497385977,0.06616427314149445,0.06436724522690328,0.06653446317654807,0.06490535381206422,0.058939473224647,0.05296524444281421,0.06014401288056069,0.0711285711624883,0.06836421312461165,0.06971127097579802,0.06329334169479746,0.05354967428688567,0.06022065159760791,0.06366045426706957,0.06922841424088769,0.05735646298532465,0.06290350527542002,0.06444127844634898,0.0726123165476912,0.061515973807790544,0.060267426929994546,0.07230126899975284,0.06094358485110753,0.07054595871682048,0.06475106130107963,0.060428523937534506,0.06439752035976876,0.06222666269678289,0.06681761398399397,0.06366082566753214,0.06124075311041459,0.0676945942824653,0.06253395915863882,0.0700556830893186,0.05462989165316598,0.06447012801838832,0.06179535106036363,0.06274986439441574,0.0671103280472359,0.06313524378508686,0.07210358083754903,0.06095222686637096,0.06175237722674914,0.05910510093687092,0.06700651143409923,0.057751965506036826,0.06688120865441416,0.06542747065238111,0.06354464609818206,0.05999565525449603,0.058458693199194,0.06418169829096115,0.060635451132220075,0.04898324927314977,0.06901877691446065,0.05621609159978112,0.052579530859533305,0.05979928704831569,0.059377851665025946,0.06121518656804257,0.06464468368866869,0.06409187236709876,0.05648215350746072,0.06908553530174905,0.05982287142965529,0.059725662844011596,0.06183388110588255,0.053609868581310435,0.06156707655788867,0.06748181689746467,0.06121010018102624,0.06326198390011054,0.05951563406207878,0.07289135797352345,0.07809977035891219,0.06531945347432425,0.0626588492995479,0.05545707405373587,0.061969489491323776,0.06287367877279632,0.07077404500014836,0.060544800286742737,0.0646398965704412,0.06700258089466511,0.05466020804505166,0.06971336073426496,0.07042352287588442,0.058919704492280914,0.06127864671239981,0.0621030288609756,0.06450740302344996,0.06821895477659361,0.051175890309416626,0.06187468076051987,0.06466673162858316,0.05927489691804862,0.04888144683127085,0.0503640637584319,0.07202911349543875,0.0709921335776324,0.06298698158794516,0.06391778841950733,0.06689312879455575,0.06640512396709478,0.06219104432740448,0.06449890045805498,0.05200007726081032,0.059370740158397245,0.06554032805325223,0.07033485729542527,0.06453070727828908,0.06670548194897802,0.06478013211949371,0.07134202418246033,0.05714605520631675,0.06685099676784723,0.07157640528647574,0.061154829025345775,0.06287475640769624,0.060500629886609175,0.06781362847881581,0.05777851990299378,0.061067359359108106,0.057480241033502646,0.06705673535391146,0.047473308403388745,0.06852021612592632,0.06043697691715051,0.076686238581122,0.06147839416483228,0.06309462694715043,0.06711494988239235,0.0618148158038717,0.06427159986877815,0.06800892698505726,0.06250551030756923,0.057069047772224754,0.054601206476535866,0.06020207546411607,0.06354231882341141,0.05569399841408519,0.068454622759661,0.07346342702004068,0.060007165719837136,0.057982121074169266,0.06769945560306626,0.06120293835888256,0.058012457181247847,0.07055445082271199,0.05772625781016888,0.06519920160378812,0.06844271590398637,0.05830162869677877,0.06156492474287342,0.044255975380314824,0.05635279671867187,0.059814739706696554,0.0669354423961593,0.07616918535511324,0.05785355153735228,0.06911353319786395,0.06477952059071404,0.07429701366268565,0.05800845941786599,0.05054123791335296,0.06955347684101751,0.06280750494175989,0.05099067797072161,0.056700691231324804,0.06285871648661456,0.06020838294463455,0.06302517350674067,0.06692485976131193,0.0642999706819973,0.06557331742308303,0.044885547027567814,0.06813019999698246,0.05076695411418908,0.06283777185624836,0.06516388456559696,0.0704647431972622,0.06210148889674514,0.07085156772950216,0.059642831813655636,0.06712121553162342,0.07069365246000778,0.06257620032762327,0.0708796478970102,0.06549911721199521,0.06295593994260235,0.05830287350055502,0.06189527381371839,0.06528375844358952,0.0657372765968166,0.06472313775295198,0.055221753026489996,0.067700212024802,0.06584679948460717,0.05865383615154725,0.06997696922545688,0.06005927064138484,0.0638213842226453,0.06340734567180573,0.0577692296617454,0.06513331193415123,0.06422646028379278,0.0690463855124415,0.06629526706553616,0.06128071752203273,0.06123515173912628,0.06690993772335749,0.06964077577694944,0.057874956069177835,0.05455463948094662,0.0731629953236024,0.05257098145701794,0.06406184023049893,0.06415815210511816,0.06197569327710929,0.07123116117017135,0.06977510970880597,0.054443702259558066,0.06787481268612397,0.06692113055157672,0.06354206371149096,0.06756670202147452,0.06240213303961069,0.06227149355723387,0.06053776910622111,0.06568241092735586,0.06823579230573991,0.06055396043263389,0.0700855931256856,0.07224585408452039,0.05945120456448204,0.05195378722899274,0.06067742604221195,0.0517693397389331,0.07286960967165726,0.060880175723430435,0.05142803472124917,0.06764837862452963,0.06266629187090446,0.05838232308032394,0.05166154542874861,0.04665918458987934,0.06805408213897762,0.0651434049057969,0.0713512556908113,0.06413838207829152,0.0658415722273514,0.05350067324955851,0.06408125231525252,0.07050617690033657,0.07077448208859435,0.0695573771020057,0.0671242333294022,0.060511293749402346,0.07277220070091109,0.07345512984711597,0.049201205458453294,0.06886730337692348,0.06566689725905418,0.060946088459529174,0.06759606865066253,0.06570071753220788,0.057336301583919225,0.07309343621853512,0.0584725347516309,0.06987206462405732,0.06396559661233031,0.05547234946704403,0.0651025831733151,0.05750256114059664,0.06125294111419804,0.062057452492621325,0.057359384292320806,0.06824870174749567,0.05767383969006021,0.06626300452439979,0.07240201781483865,0.06565956028625314,0.06896129249168956,0.05585331418211949,0.06430465897906457,0.06099561238606649,0.05917105757051261,0.06667125855274025,0.06614352426753628,0.052739373028751715,0.06007048464909258,0.06862281285064256,0.06118764541122139,0.0700369237405555,0.05852017848278318,0.06604022960195724,0.06816490965413605,0.05252522807476616,0.07042654566704173,0.06370421011776317,0.06096811156953588,0.06621562925364441,0.07117041094941756,0.06414892539224452,0.06723276228423311,0.06752806967683925,0.07284624368947457,0.06439873022016906,0.06222442656126002,0.07357705548414663,0.06694919640900192,0.06246897511906261,0.06871270852208253,0.06561122356216068,0.07409029341732346,0.06154693832176008,0.0728224907925083,0.05340937481854056,0.062310949134878106,0.061789841998228,0.0714719893526936,0.05600353863655326,0.06760883928702194,0.0661542777823651,0.059263816599372784,0.0678487572411466,0.06601773102985078,0.049148507200866075,0.06701277263399816,0.06851283751978515,0.06660444712067104,0.06581717193072112,0.06279659079202138,0.06646989345335655,0.07106411605193665,0.06681997115327912,0.06475137149983257,0.059574116617572614,0.059632896177858136,0.06793378462353986,0.05809317744226784,0.06791232181729248,0.05868909653334725,0.0689693284972008,0.06037293917105084,0.07631328525460043,0.06959795447280775,0.05986065654138882,0.0738389280597807,0.0644594120949896,0.06706426100638821,0.060265071886836614,0.059412177951289204,0.06201528078786087,0.058823841396957774,0.05773949721166788,0.07315866936429866,0.07130201226168037,0.06937168220338567,0.07007198902849658,0.06641654311474271,0.06774529509964812,0.06536258466500702,0.054860721330126316,0.06295010333236209,0.06595247107856334,0.06163765729720556,0.059690477494612676,0.06265499816148576,0.0634286437581083,0.05676841114425369,0.07083734585275549,0.05352524858904002,0.06759840722421812,0.06038576261948246,0.05504544267364955,0.05625492032904543,0.07569139897579082,0.05899540162482894,0.05699078357223095,0.0706050466657985,0.05433087593019847,0.07014555830457385,0.07229010324974797,0.07248642777591233,0.06243585745989464,0.0632593025485099,0.05956849720441253,0.06905084613146323,0.05970511498454405,0.06410566682895008,0.06571749620843824,0.061074610261304216,0.06097368423574749,0.04977451106816175,0.05187881007365347,0.0695741730472268,0.06348720705634459,0.05950102428659921,0.06309863573893065,0.07647125753846142,0.07186506354662163,0.06194100886976799,0.05147668149148944,0.06360335787254001,0.06408237949171772,0.06510732421977514,0.06597646822220275,0.07024248283120282,0.07050225052370675,0.06651118912150794,0.059028124122007225,0.055372368021802856,0.0665041050090894,0.06894315694878028,0.068630028786446,0.0690622595324369,0.06485510458176234,0.05949009613024406,0.06995627938483667,0.06480174614211191,0.06324883229313488,0.0659114669851717,0.06938832030920666,0.06361923269506253,0.06519475463244315,0.06909730162493363,0.04799265041582973,0.0699112570580748,0.061536599312889316,0.060626938673615606,0.07324457854393124,0.05860983986163913,0.05198279774672364,0.06872867570117677,0.059437523621446595,0.06246875141775959,0.05621086621239732,0.05474194380942783,0.06069856700674134,0.05242264510141781,0.0701253243225699,0.06700882630835342,0.06354487260058644,0.053186800052607994,0.06828729396973955,0.07333050220188117,0.0596721310090262,0.06543568293743704,0.07075822746285479,0.06880129506815907,0.06021851940860426,0.050839602607107695,0.06828489358517503,0.06040282554809104,0.06841282469018159,0.06247032405376531,0.06348977906015733,0.06374209081089978,0.05964786853794868,0.060473028410843024,0.06886075100299531,0.0625702802120859,0.058249626304287025,0.06610436329189343,0.0629478400819361,0.05811985283243291,0.06263044917279945,0.06452556139695254,0.06336042910341194,0.07325794226236143,0.05720412683167365,0.05998411621852716,0.07412689854195692,0.061206633449485556,0.05689525488181972,0.07018549463092258,0.05901152145923022,0.06181582615351876,0.07053414977354264,0.055179124701250956,0.06315296587271621,0.05977702756384759,0.07388341577409388,0.06313891085936207,0.06306390605308523,0.06543549022611264,0.06510667602755893,0.06100894993355364,0.05217680785906463,0.05344860931656306,0.05589520753336547,0.07166510035551088,0.06457587006382035,0.05151260322904069,0.06197429600140273,0.07029880524075746,0.06622813539223732,0.06062548971534422,0.07123985304935701,0.057830503229754096,0.051401892810288376,0.05963872179207391,0.059519981821230944,0.05263254089560526,0.061789890280409786,0.06526402882724192,0.05945291821822571,0.061442500608050475,0.05724910141876183,0.07326885852433285,0.06437324508544748,0.059653112550507245,0.06410475538806171,0.04979534095238157,0.05121727349404645,0.06981894579075171,0.061416019244940966,0.06516210093233288,0.05536913842902283,0.05694296095365388,0.06343319923001689,0.0729365451518422,0.06711637705117096,0.05002151325144684,0.06629851414344445,0.07247147224474815,0.06260056387354998,0.07214315813850951,0.07338050796124779,0.05463206571182706,0.06511481258775281,0.07111970330433545,0.06791746841352661,0.06841016567354805,0.06146746923085963,0.06280563434403016,0.06213318487738075,0.05871236422700838,0.07312795027302633,0.0572111849786075,0.07058541585149142,0.06263805694652222,0.06317011161464073,0.06716037872909579,0.06875259079509638,0.0580276907777643,0.06531784399330293,0.06216442024058711,0.06247830805671023,0.06029705094180105,0.06662967278091929,0.06564346802760276,0.05837461432821939,0.05819179992933497,0.06576345619798663,0.06430969913981832,0.05313562752830299,0.0655796720512559,0.05996996849357887,0.0674417194675445,0.05900589788856886,0.0636037458534331,0.0710739288486775,0.07130556493795559,0.06129375941153609,0.05387461776109649,0.06529274697156728,0.07453444186365471,0.06729615495797363,0.0633580653761897,0.06441348537294896,0.05916342114038624,0.059965487773324805,0.06702698739164666,0.05731935695903309,0.06064069195752188,0.05928167320436277,0.07073002596638947,0.07190677166725673,0.06323719316139412,0.059384812400188225,0.0697358640057405,0.0479897857863089,0.06599391220160683,0.06651434385440273,0.07519853442261137,0.048892905827932716,0.06944244244770623,0.06029724435354962,0.05983283053029065,0.06256576625742524,0.05935340991279424,0.06368743310607752,0.06500480443515172,0.0645252785162913,0.06278766668860111,0.0503128234438992,0.056827247191055905,0.06451404746427607,0.05919219578885779,0.06992093801183147,0.057151438338388774,0.07177063152537891,0.07001602162977463,0.06592324663688166,0.061581787767643274,0.055854120003842225,0.06480273342834778,0.05599070011294949,0.051962147441130126,0.059811826712643686,0.06483670228331885,0.07629055720042352,0.06107621315717876,0.057125148233285836,0.05341458273295978,0.05768914603551838,0.06142561687909922,0.06741725257688033,0.06479706264780874,0.053614843256582057,0.06855482400676693,0.061031922347653686,0.061705709215412595,0.0687667766552214,0.06675077613816618,0.05310468563683407,0.05844769837779604,0.05882897072825102,0.057322477606193994,0.07085263919223878,0.05913927685448554,0.06256108874688357,0.056965024123865726,0.06487083812377903,0.06224139622837462,0.06378736454976261,0.06236907417335433,0.06038413450800806,0.060423013358290784,0.06313825751323085,0.06753875464877701,0.06926624712035892,0.0682969178333564,0.06285928613373831,0.06370621987384423,0.07201909415453354,0.07018734382972597,0.050871993360800186,0.0610394687624399,0.06256877438161129,0.06751499400606281,0.06774451651005314,0.06747404264971074,0.06189007308682441,0.06644421863185848,0.07029522800600518,0.05961601940367263,0.06804556782103692,0.07251947125270629,0.0657713421924783,0.056328803392163784,0.06931305374704733,0.05999384094626935,0.05955013646572714,0.06282523902419554,0.06745685768311138,0.060641817962267655,0.06387657825215404,0.06561410790022032,0.06479198918535381,0.06989415020581928,0.061227387651225984,0.06468796230310325,0.05600891756364955,0.06544247052065513,0.06845058195450868,0.06280215372388051,0.06160168713032151,0.055964943432606426,0.06913613699708863,0.0714591557942184,0.06720985052773096,0.06646419905306636,0.0657690563124467,0.06259624401652925,0.0684687830138403,0.06082057291263543,0.06712745671158227,0.07047692520436455,0.06858618366131594,0.07283731080000358,0.059825147236385115,0.06771068296313235,0.06236341248206756,0.06934899095179053,0.05669755644816994,0.0688133068435881,0.052042749841978625,0.05352067960901224,0.06877631054114304,0.05721618839915174,0.06968476311762464,0.05912443994101782,0.055788158934234555,0.06044917114982848,0.06259140262508225,0.06852703390506196,0.061276821752679796,0.062466204574418156,0.072658965663609,0.06319870235755187,0.071352227252965,0.05235000884024819,0.07232552990796033,0.07325385678381646,0.06182132634442983,0.06680475589578402,0.06945286416899406,0.06568667928251673,0.0503400227926354,0.07121708222374501,0.05706573313360766,0.060734379980656686,0.07245562818029327,0.05893052220885597,0.0649303136208476,0.06546002762473974,0.06974133221838973,0.07027885750354439,0.06925585602227707,0.06754451060522407,0.0722188398408022,0.06183189886107886,0.0678052014459965,0.06256739931328295,0.07169531011190382,0.06630585696928254,0.06884599698932622,0.04951095694224872,0.0655081598801951,0.060712022377875464,0.06813045421328065,0.06851162687438073,0.062216563333564866,0.05866658325927471,0.0716038147514457,0.06578584992966358,0.07025325944649763,0.06880992323946383,0.051087256471325476,0.04757634941690435,0.06181746489605663,0.06355488915326091,0.05055062955036725,0.05356372659830121,0.060883227509147125,0.072578323553895,0.05167986341736748,0.0657829512964371,0.04961997547256049,0.053163834743714375,0.059517865914031784,0.06985175344707767,0.06288288496049196,0.05816311627898149,0.06514150670356475,0.06061353084574067,0.06847514926056933,0.06216902053429917,0.06429650474341703,0.07512349946323976,0.0596069092731422,0.06623417175162236,0.06589039442568603,0.06592583638300403,0.06965199204274967,0.055482075544608024,0.06239063960080736,0.06678384148868313,0.0749297614810922,0.060537029037222204,0.06443305978348543,0.06376368224356346,0.06250337950005265,0.06557059538164173,0.05627911812981921,0.07083007183689069,0.06585836561175819,0.06491697354716006,0.06587606566780427,0.0673896865739286,0.06878389930252654,0.06878660422417725,0.054998700011392246,0.07356617592865501,0.06251967207791562,0.0713285858311322,0.06049217064786801,0.06673929788487487,0.06488584237209433,0.06076169066801025,0.05784840169961703,0.06896310822228874,0.06505770905851932,0.054400234014102926,0.05217255870414782,0.07116730252308437,0.06778756727139396,0.06582748220471225,0.07360896627823058,0.06864587533214281,0.05526602839699538,0.06404416744461927,0.05985733123410182,0.07178719191772226,0.06280670984637285,0.06035066895904197,0.05847527951857304,0.06015559732480756,0.06957284572195371,0.056216628537496205,0.06899599776918543,0.05478300127803816,0.06560127037402413,0.060607668945181395,0.05918384184259374,0.05389314155578945,0.05412469990606508,0.057660299476897166,0.06876069561343531,0.06402595322167276,0.066303957726974,0.06251178322365664,0.06784646257565662,0.06159284459416752,0.06186428290595102,0.07269671758605328,0.06723060791274459,0.057543228360270274,0.06058971962972809,0.0699431260641271,0.06423613849703044,0.05098769610569508,0.06830888020758144,0.07080439165695021,0.06626785156464401,0.06941481224553242,0.04768544308676437,0.06264999432684264,0.06711634251475829,0.06438529941844745,0.06339618251627392,0.06908815033925728,0.0712643108806061,0.05932873361057521,0.06884326888170986,0.06901222266048734,0.060891464183628854,0.06498283435643229,0.07071347084785337,0.05689954754188556,0.06591070584079109,0.0548022081686984,0.05896580814795259,0.0632136791937383,0.06193906881363439,0.06638175418342183,0.06922049879509867,0.07230358583564689,0.060827975378149796,0.05623092982287307,0.05291184281203889,0.06936071814731812,0.06136083510634508,0.05867471764282099,0.07162153653761277,0.05978659226929118,0.06736278550824062,0.05583424358633307,0.06621493347356688,0.054190111851749476,0.06016436183219738,0.06281331526555829,0.06718386069307855,0.061056510085173196,0.062036296534629484,0.061397116861333256,0.05935465581447619,0.06455140406938797,0.05105458472674733,0.0752543092732681,0.06591903010278444,0.05803763008240007,0.05713195818203745,0.05233592064535367,0.06394401503708494,0.06715196594916777,0.06962573280089018,0.060264065719770346,0.055078306085215245,0.06569660699031851,0.061119069859569816,0.05634965841680038,0.06244101419721242,0.06480178101177808,0.06773094863126539,0.07400039335197832,0.06398743616610664,0.060325626065146604,0.06457424367626861,0.06581008706414279,0.05507107018496356,0.06411373415485036,0.06772915067258997,0.05990072411065257,0.06915360696268336,0.052527556423009034,0.06098911756088804,0.05909672737464806,0.06584326670584807,0.06186961357354454,0.06567180759293795,0.057860953364387915,0.06665926712593709,0.06370667760398552,0.07023923921793437,0.060855544512038603,0.06561359045689477,0.06122917766707142,0.057344043564583,0.06764910423196749,0.06961924545061773,0.05783945104422947,0.06061744747150513,0.06909723165755333,0.06234066739828272,0.07001264431633286,0.059016671876865545,0.06590351178183922,0.06617227073219517,0.06379833465882671,0.054978564745502986,0.06476574782450813,0.07136839782149298,0.059571542553215465,0.052656339771613145,0.057628229169326174,0.05478375233020845,0.06111414631183176,0.0662158644091537,0.06683658663393396,0.06344549669913799,0.07070271318528497,0.0636375501141909,0.06872033955471438,0.06430836883910743,0.05539099563219306,0.049190522145011614,0.06551811859145096,0.06630763747573662,0.0654624275243105,0.05398164744637264,0.05284667417178454,0.07193763507577922,0.06032391531033897,0.05860025325111737,0.06118959666109762,0.059456167849067945,0.06796460229770013,0.07648596010937428,0.05663056088170883,0.07015390720810852,0.05774836356247133,0.06278081921346954,0.06281306068827754,0.06643853349017383,0.05666496863650654,0.0601023272284264,0.05893628587699176,0.06296976543636364,0.07066929172836578,0.05826894144794423,0.06875054932522501,0.06410263303472102,0.06598898093793391,0.07651026008167888,0.05609227366147246,0.056513994875244396,0.045670983518465366,0.05036493996043935,0.06987916256916071,0.058968921180990244,0.06483963383177968,0.06650687922888962,0.0713367908108305,0.05408305647156137,0.06996833722979702,0.06576282889180857,0.0635330439947073,0.07349576397286724,0.07111243117172883,0.05888357171287234,0.07012339220061002,0.06349321587418211,0.06629527035858866,0.07025974189414724,0.058900838585321844,0.05777048675445805,0.0688036785577221,0.05769179176670737,0.06049852675537272,0.06757819889805136,0.07013530483386779,0.059243536699145685,0.07291034135642332,0.06779664848929873,0.058222952232939724,0.0545372010860454,0.07409249015122545,0.06621659799033337,0.06474352950649424,0.06404198125026861,0.05857509851495667,0.05846074485883551,0.057484502283988964,0.06193901370332008,0.052109669547954623,0.04829676345751241,0.06388688591344982,0.06202195701367842,0.07159056808986047,0.06588859297887759,0.05820595237274526,0.058788009087672496,0.06490741794881022,0.059215698812693875,0.06781150613988979,0.06688437302895082,0.07183837672216778,0.06594370116897534,0.06807927854538398,0.06871005076717475,0.05657741023844765,0.06726800093417068,0.05596813851349995,0.06436859548943627,0.07463387286337411,0.0710969098519045,0.05617969617808508,0.06360747855498641,0.06155510978271315,0.06608684309598147,0.06443486523953218,0.06376763743352815,0.06781075833144073,0.061768872445865715,0.06424820773235441,0.06559549634310205,0.06816387712173425,0.0724245125227119,0.06470357282378494,0.06306889680103234,0.06868733952335676,0.05915148289473508,0.06161356294087208,0.05559101237140142,0.06512716910218452,0.07141270973182584,0.06406258709445883,0.07278887858689961,0.06332628866244075,0.06688847897713454,0.05785838774989889,0.06693961596680131,0.06787426735886201,0.061903079814680836,0.06888682249511396,0.06881336790257006,0.06155558568970397,0.06723078296493185,0.05838518212183101,0.05771750543354706,0.06891931164395929,0.06883755629070856,0.07148942136811584,0.057529778030438836,0.06261249283364703,0.0601368545569342,0.06907955253806405,0.06639420234852839,0.06135684796695725,0.0517291952706292,0.06694406074740578,0.0734753030936437,0.06153183856442269,0.05939724980516796,0.06145335296843059,0.06113627635612977,0.06187084691611371,0.05919769593593582,0.05671116878206578,0.06678420532102408,0.07206631492051974,0.06260785502359645,0.06726849516927977,0.0620683802659232,0.06884829151289507,0.0699027065182816,0.06100248151240911,0.07073215799439173,0.06118795468884159,0.06145900070167797,0.06271144343279525,0.06384897221846295,0.06738304791265931,0.05852069133857969,0.05489763210536318,0.0639044197801961,0.06200920528860541,0.06712779525933987,0.060631371003057016,0.07251536586535699,0.06630045449755156,0.06641968283236356,0.057298965103992225,0.06156935759473666,0.0589637587807738,0.07024311119739295,0.05810569580463232,0.07417781124480471,0.06171532010004437,0.07011399391374215,0.06294791514441037,0.05580884394212293,0.06470644629382316,0.052078138256150625,0.0699880732490382,0.06383860871986127,0.05397341846251691,0.05911878199601725,0.05983037884298889,0.060837980978912146,0.07024299803767428,0.06606645322114069,0.042607112790961285,0.0686663104410149,0.06576831879047634,0.06199300683329645,0.0679469244651574,0.059170660611073705,0.056393918687731345,0.06362625824647944,0.049204889340385846,0.0634149802383808,0.06396967952946903,0.050967891524865805,0.06320241527290194,0.06585439424251485,0.06375772280569006,0.06882331888796976,0.051242463718274074,0.06418521448937424,0.06771892712939424,0.06130771531007333,0.06886315552799807,0.07072275860995168,0.05112396084978936,0.07111486752104165,0.062151417381868766,0.06738197581695354,0.057412017361781545,0.061734125428319554,0.05274321232235257,0.06850738017209584,0.06917344526931887,0.0692862051464653,0.052608147798196433,0.07370054654401934,0.060663887620371754,0.07309579044039444,0.0635417139950229,0.0613249718690975,0.05842431553649646,0.0711472148481702,0.06320893300559494,0.06316206515328165,0.07137955953575964,0.048184201173718806,0.07285860250270021,0.07148538749633722,0.07119032084708782,0.07223416668096827,0.05565535043434046,0.04975549953008952,0.0727532471429651,0.06300974890346034,0.06629247729772014,0.062236223105518726,0.06894003045502095,0.061115989454655545,0.05962202869047929,0.06915493224690158,0.07283285654873635,0.06531306321652208,0.06545433698319567,0.06584429045171013,0.06735542568839603,0.06712486273503498,0.0546749545776159,0.05476510553708318,0.07081004298345642,0.05415062892753937,0.06993912201262101,0.0710384268939954,0.06435456820768995,0.054923042689706644,0.07219912655167535,0.06629356360931467,0.062100176278579465,0.06244200226441445,0.06647767913171612,0.05896265104048115,0.04938843740583433,0.06510943832593757,0.058631180704225495,0.0664916927073515,0.06917255831702493,0.06019903254488079,0.06975336162177298,0.06308904714935115,0.0731727456065444,0.06550725062415877,0.06550448799419764,0.06343149113265802,0.05001295853508826,0.05618049218192819,0.06129520343876926,0.06877188287800143,0.06942421654176076,0.05346301361307654,0.06858510217016207,0.061775568999287406,0.06541522909606287,0.05686053947053446,0.06763846169083541,0.057717902116074894,0.07019629292795514,0.06105233336674657,0.07365818490214032,0.06713845857603333,0.06947382396792513,0.05645026366917567,0.06412855536563783,0.060870210398373675,0.060342549363029394,0.05984077552304885,0.060031259150649524,0.06386708262421129,0.07116958956740155,0.06569472888604054,0.05578772338266614,0.0649573881183543,0.060443000933153884,0.06647635580507794,0.06777137923375984,0.06323438639891964,0.061845900094259366,0.058968207428295595,0.06781614206178868,0.06517057773718817,0.06515583789865417,0.061933752827701666,0.06844544749616881,0.06978657821624787,0.05862934097800475,0.059927282568942974,0.04901195971374325,0.056050003875988304,0.06825371591075532,0.06898756054551744,0.061953249913581976,0.06311175314051728,0.0680401814231989,0.06717567872730412,0.06827936876319528,0.06321117135231882,0.06735189573995737,0.05903856847399287,0.06765081210028448,0.06942624679926171,0.054898890325997936,0.04657288838316318,0.06481975906397225,0.046812730746308394,0.06656369249057334,0.06296225410805772,0.059496994029543865,0.059466958846609436,0.06803985937836604,0.062456749234940795,0.061778701747484836,0.06825153177240648,0.07266329671816249,0.07175080601959835,0.060589160173548706,0.06376754642124877,0.059285976442883886,0.058311325471337834,0.06675127785430954,0.06886174931229538,0.049089472846979815,0.056694670147036566,0.06392943929443806,0.059925323802199254,0.070009917061393,0.06829123184103611,0.06616855840551848,0.07067174214280042,0.05910054871363846,0.06767714193502221,0.06833135453251585,0.06415656640335551,0.06972850821907534,0.07117301843988455,0.07672723188952793,0.06127044673321689,0.06464962021375872,0.06848301666975329,0.06557756156689282,0.06718472401272013,0.06138904423690173,0.07251023075554199,0.057523087987821145,0.0691012193542472,0.06720098929991006,0.06938297442896747,0.06027180804956864,0.057972513755556176,0.05088908976882345,0.05826843961256621,0.06401317540588869,0.0750156225189532,0.06469614529395214,0.04654691953788476,0.07547375005552509,0.06804772942316634,0.05726554896958933,0.05819397680394004,0.05731839873114444,0.06519140826854286,0.07124331179085006,0.06732869702961214,0.053670222323381506,0.06743634100974832,0.05827851662535405,0.06376773908729196,0.06884525873809103,0.07685909889927597,0.060009695617720765,0.0602314735026106,0.05813074356777901,0.06358421487181878,0.05760816900818025,0.07278067607430425,0.06740919865581138,0.07493188186282743,0.054764222211149016,0.061346931538398604,0.0636109693850283,0.06326131868958976,0.06964256849219551,0.0673618662073619,0.06832287842499751,0.058262673857040495,0.055104383659190403,0.06706657774380442,0.05005533254858581,0.0622059847286166,0.0632534023897982,0.04902124929142219,0.059441449166293514,0.06967546618887645,0.059325976517923706,0.059648168249511496,0.055737832411132976,0.06905802462668323,0.07084941359737057,0.057565456841742385,0.06676107706126064,0.05191870363616403,0.07295558089026549,0.06367473174212993,0.059576516478054425,0.06319664911936762,0.07373748153826765,0.05686474330663545,0.06478891973524133,0.061435555446288745,0.06689471546060374,0.061760035629504245,0.06403726539494714,0.06082258571184282,0.054016999660258455,0.07557550913756211,0.05209551855725949,0.05858482029381322,0.06028477660892975,0.06290340497613486,0.06881968870103869,0.06417444476313455,0.06659061869135138,0.06685111865875437,0.06951491967739606,0.06525162884694027,0.07239300144367387,0.058302479095339435,0.0618844374469686,0.06531102970092507,0.07158347309122584,0.070296654239725,0.05502099064716588,0.06788127775096038,0.06018258666622299,0.07452437198584398,0.06053252126156926,0.06049865023294754,0.05633371021111834,0.06426898634661549,0.05173390456454133,0.06507591754414006,0.06663870060026597,0.05089437156512332,0.05877225660018964,0.07163384667210257,0.05614714981255842,0.06142866784375681,0.0695602750319157,0.07125486533678724,0.06444657878773222,0.07397502126865925,0.0758567131834135,0.07018000257898467,0.06879737833105498,0.05091111854500219,0.053654123223486005,0.06495411209130324,0.07287033118894323,0.05575063697464717,0.07125405026224474,0.059020381412189,0.06743618691857703,0.06876564668278014,0.07370251452959314,0.05925289362995813,0.05945795924666737,0.06217266061180294,0.06283796678251335,0.06281694627969191,0.05602875577156938,0.0638131936720118,0.07327931910533614,0.057875039989160715,0.05486140337398558,0.06189512964711713,0.052811865463225395,0.06772121290718175,0.07017262762234024,0.05967402875249701,0.05733688498808313,0.06512519311126724,0.06503592511771925,0.06416793955660755,0.07003990410324802,0.07076573376589917,0.07112743834148427,0.060624332734274866,0.06420504514708734,0.07314153538749912,0.06456770852639157,0.06614750451376256,0.06964061658802059,0.056865013436410684,0.05752917881923296,0.060459981367761345,0.05824054513159777,0.06934401310573711,0.06589650806420842,0.05634498470971061,0.06653330124249705,0.07449783911402001,0.06145043431980151,0.06922692315627056,0.06469457847320684,0.0596248980975886,0.05533384984611725,0.059648449102597895,0.06493789366288308,0.07499527846710968,0.062389726430439385,0.06028580235578242,0.060705130617847144,0.05928592163622629,0.0629003261072435,0.06562796171178108,0.0627661401727126,0.069635525942307,0.06582025034881667,0.06907563297777947,0.057682739079922,0.06309657578793154,0.06902435408838248,0.06957625570715378,0.0731506660531552,0.061612865580080854,0.05843825019334298,0.060585857290115064,0.0641124996452535,0.0629891132738366,0.06861745512221099,0.060397331352900305,0.054987671933887115,0.056227050435039956,0.07468568751821046,0.057028714299547735,0.07216335410818503,0.0670451041369883,0.0712392437428534,0.05778826577984569,0.0739936409537284,0.0607536611709648,0.0622366958596865,0.04922380975092723,0.06889049316105562,0.07329081859031146,0.06417789161831841,0.06827609439233397,0.0566267761666697,0.05973998275948614,0.04777258952761703,0.051168859736242966,0.05576450202247374,0.06571037292713883,0.05310720075998853,0.058782219075394423,0.06616556695546705,0.07429835861915843,0.0643307404863955,0.06599409684180206,0.06777962215843986,0.06096225681890841,0.06923226966891813,0.0651419560945132,0.06148100188417506,0.05128698227559206,0.06964807631702263,0.0556865552724192,0.058781642305403954,0.059201706860406114,0.06497429329507183,0.067820917679258,0.07235998496185081,0.05183614759126631,0.06783510629220216,0.05284177319000888,0.05940583864364557,0.056679589650599535,0.05979546628777635,0.06760765015965406,0.06606105557140413,0.07369658603804949,0.05298891235423341,0.05389833761532781,0.07328398513693266,0.05667168887649879,0.058937123741808,0.057423409482918075,0.0697217378059869,0.07562142653017853,0.06765365443828814,0.04950207529738866,0.0567949773044127,0.0636295677867571,0.06525369848809826,0.06522425224638065,0.06382746696184473,0.05528222042708761,0.07402593691143075,0.06967911086749781,0.0553619642418917,0.07024511732162346,0.06736775725276797,0.06308027564968159,0.06711192523786728,0.05543393243984466,0.061378351544262987,0.06599459931913515,0.06527135130036699,0.062199729964894354,0.06322953048945405,0.0686845248812262,0.07015090923187207,0.058857099693886555,0.07415693460325032,0.06334242564379093,0.07364532665663162,0.066524993750462,0.05616667680496341,0.0612539147183491,0.05867125363591946,0.06244122502903101,0.06465218475472786,0.07320138009960594,0.05700519378365135,0.05584980957894081,0.0676292147005517,0.06237322679126673,0.06288267976375639,0.0672583543961564,0.0630454261390789,0.059081110987289166,0.05622282232405621,0.06679185600362361,0.06339800720334571,0.06063678849427005,0.06430500023621674,0.058431257814953265,0.0614163569176487,0.05985145369862647,0.06835902733414295,0.053469658853623384,0.06194751791238936,0.06135729066529582,0.05406104350881988,0.06083490217851987,0.06733604889570592,0.06756215840974511,0.06683460072744393,0.06181332093722334,0.0741639046431313,0.06869415793728093,0.06843356141274543,0.07321453089659205,0.05269903839478196,0.057155801613685886,0.053562583692523515,0.061889130551571726,0.06543742141859252,0.06361672359961289,0.06721164400618189,0.06431787246700169,0.05770832664475737,0.05897476931527579,0.06586427288955293,0.060921628927724486,0.06875586903844591,0.06687018355793828,0.06170866433914114,0.05149211967744277,0.06770885734641145,0.048607046015083734,0.06941637110252716,0.06624259265086567,0.06662568894046181,0.05739564302123294,0.05671860350650457,0.07203297103946636,0.06292837218698405,0.07220700712975152,0.06285555013495603,0.05928915148988652,0.05824263148536446,0.06726785726716702,0.05368997652325434,0.06368523470153012,0.06901312800383289,0.058250420394082715,0.057309474058747686,0.06420185330451939,0.06983870968996876,0.07473595066120331,0.06095348147149926,0.055477179087411774,0.06070922220441009,0.059891543337505335,0.06349393002447554,0.06612692893331135,0.06645428982024038,0.06332454671139304,0.06421611838133467,0.06481213637108499,0.060965900787124355,0.06469581691737264,0.0645669316909687,0.06662884198227721,0.06401979471248802,0.052535398590143834,0.06251878263626967,0.05840132425643929,0.051072727728568246,0.06278032597926034,0.07467592598428663,0.05926607450571497,0.0641777154970583,0.06588968734196457,0.054709440679583554,0.05461358627151806,0.06792849962628153,0.060012565317410047,0.05721666440957744,0.06585539970695412,0.06735721972617326,0.06306761915712261,0.05108043696032975,0.06157993103119867,0.05680097198960876,0.07135272714687046,0.07053110456879198,0.060053867751344844,0.05306083159374716,0.07168889827051328,0.06833447140285856,0.06825971585914817,0.06898972126094666,0.058874726555231266,0.06998688672628753,0.061611259133562093,0.07067291454171179,0.06479629168697273,0.06230133879919996,0.05674728213044283,0.06890769131140123,0.0683017750314587,0.06287732749906319,0.05780427489130917,0.05775467710621522,0.06333200044520126,0.07022044366377184,0.05741148649349746,0.0694260203133308,0.06255681473724431,0.058634002327646165,0.060027848295912074,0.06508745071408464,0.059562792143908506,0.062428040085805915,0.07230710951400986,0.06877849844892339,0.059075526639238776,0.07103455800248273,0.0574311630802094,0.05604118086086762,0.052264833478020155,0.05813946762474319,0.05424722062783473,0.0626920803056788,0.06509709786711086,0.06254704346984077,0.06494237919202248,0.0711648503913804,0.06440491316945918,0.06390835845484008,0.07691325082996678,0.05801640491703104,0.06439175642855308,0.0680616236104274,0.059636936720887525,0.06561068655614785,0.06978787534223944,0.060217423812648704,0.06602418061267634,0.06850723810942838,0.05364120447596854,0.0620392897755237,0.07619460389823568,0.059554692137891246,0.06836646489918911,0.06413975134581791,0.056736577413480246,0.05848861081229638,0.053719434587442993,0.05514313134937891,0.06283969476779963,0.06538324718761923,0.06294306297596061,0.06222663126681383,0.0654415768774988,0.07086259145819493,0.06037298102352002,0.058660621937109754,0.061787748494229265,0.06962580911412578,0.07033704137434822,0.06714715240931912,0.0644813408173292,0.07255333701214894,0.062148104889237235,0.06491830199845916,0.05690576507120906,0.07335055591345609,0.07265062084599266,0.06223969143905298,0.06934441845881487,0.055097716386656184,0.06430698464605235,0.07484914320755419,0.06941988266165294,0.0707903439733687,0.05330357678740495,0.07204613482540603,0.06283055838997514,0.06472904384929457,0.05957899044898876,0.05587586852764734,0.062328115485857066,0.06973181772776245,0.0738095365711203,0.05867206832619512,0.06053927382142074,0.061616323814511494,0.0604764954098337,0.06149536251147616,0.07052856269335057,0.06516021976089495,0.05711113038203301,0.06567737586691622,0.059161856237897305,0.06107520105739737,0.05560960180555842,0.06214126069602171,0.06649011880180111,0.07259970930068649,0.06253183117984179,0.06989305454393882,0.049496229203327004,0.06953270909884736,0.05929906345747209,0.0532743151122488,0.060595639891351694,0.05624810120837359,0.06536160921508694,0.06057465280793413,0.06003509325841365,0.06656321676640162,0.06099338387404266,0.06315464991954524,0.06241544844858368,0.06634782971760304,0.062356742506400546,0.06896789964269659,0.06262267328537102,0.05981472397424856,0.0632499076227315,0.06369863377815563,0.06257754968978887,0.0606516732925376,0.05930820902812681,0.062410459098348324,0.06670440507411116,0.06864908674589307,0.06691402897509907,0.06401595760911727,0.05808480324121973,0.05381682951287303,0.07396846280418846,0.06735101920154,0.06349867962681985,0.0666811023681589,0.06830911726099029,0.06795024529283936,0.05524803920714094,0.058500030671474926,0.07035683727510311,0.06419124088393449,0.06881265315966598,0.0728101311181732,0.059520908904168654,0.07207883816702841,0.07035844909899838,0.06197552353116504,0.06442748114232194,0.05936013619912223,0.05529102814814732,0.06834144401518655,0.05615621288114661,0.05472311694702037,0.061664617481802395,0.0661501333949937,0.0608441710336763,0.06255325091561764,0.07323873653464244,0.06749201059885224,0.07361706173506646,0.05184717021053245,0.06370001913328688,0.06523495373344154,0.058520872963569684,0.06606211901197748,0.06793997047393191,0.06940735825348004,0.0683039927650875,0.06335190616907443,0.07184607097418082,0.06544088326677464,0.06287718074797885,0.0678125424191755,0.06344108934205461,0.057430179652623065,0.05974037686704747,0.06286244827019344,0.06597254913199864,0.06515040956911658,0.07429179429605723,0.0642130536498895,0.05632966490494873,0.06078606830530181,0.06000761461487259,0.06846141073289327,0.052227538548635426,0.051787039522909833,0.06557408654551869,0.07172941937906849,0.07119602311464632,0.07290766588358497,0.0657622397043521,0.065528432328497,0.06875578307613314,0.06947271156663479,0.06263478983733892,0.07278011426092836,0.07117369587620659,0.06214525251833294,0.06621646329980753,0.059870906765805615,0.05734184191261962,0.07126421788719452,0.07640246147351823,0.0640379186815308,0.05739653265467552,0.06253356040636712,0.07132470958803266,0.07179869525936441,0.05327641442241059,0.0714564855546436,0.04832647647096125,0.06911248871015313,0.05665312354450913,0.0582754864997958,0.060257306713860766,0.06607913999349228,0.06669013919946121,0.06498125788642532,0.06882070414960782,0.06654199000969784,0.05872885835860951,0.06379394593242863,0.06085307264971457,0.06176659912416738,0.06884174433456727,0.06185902671479264,0.06144874535626082,0.07392049197977343,0.06593341249571291,0.06598188154238549,0.05094024232943052,0.05866780753227113,0.059058012821907646,0.06676010602286467,0.05705502028564919,0.07264839776753765,0.06279396912100715,0.06349000117901715,0.06487748060602433,0.0649993173341745,0.06418326696966768,0.06222059253690589,0.06886512271822418,0.06301630195647492,0.07429234878203822,0.0661026780917466,0.05938632693596611,0.06678384493636819,0.07011995818971187,0.06303145602615988,0.07320592318745234,0.059283140785497186,0.0689497724541222,0.0670146047993267,0.06202785003572625,0.06738636362060013,0.059225694953217015,0.06926700461814235,0.05856487468796374,0.053500873707437,0.06967974727043043,0.07063378303788084,0.06916259933348423,0.05681963729601715,0.0706772423653783,0.07262395014964468,0.05845674543559285,0.05856307754163331,0.06323751634729502,0.06978146154962785,0.06672396063504356,0.06606901088887594,0.06998429956157182,0.0712563455031916,0.06757449335961752,0.05412971444680541,0.06931335356342921,0.0681309949983939,0.06703780351278177,0.06420422329034128,0.06629097926052682,0.07075091078147369,0.06431133461635578,0.07230142206116005,0.06705231841165873,0.06580003686992092,0.06444115367810764,0.07042200060714136,0.06523691377936326,0.06602522449088893,0.07698814138665389,0.06708138196621949,0.07081288109456435,0.058375188162743064,0.05671116695758831,0.06025849196859321,0.052526436870702256,0.0558586162018648,0.05646722387873402,0.06311802004072986,0.06756580636683572,0.058510867254571136,0.06898362637406821,0.0737047763692372,0.05916655661063336,0.06482780361863308,0.07017976013235883,0.05867317701973611,0.0686399129186776,0.06338158343936501,0.06310202427805356,0.059521591628828335,0.07052396576897055,0.06374677364578313,0.055665051656980644,0.0681624411996054,0.0755750360986381,0.05654351657021156,0.06800525923847697,0.05868104416936523,0.06521026845878594,0.06736544094899663,0.05735463807391246,0.07171041976835146,0.05751760708644075,0.06559410943666188,0.06365121625453497,0.06878158814535165,0.0628050596057434,0.057745784259659434,0.05907609695563719,0.06057255921889644,0.05318775925657473,0.055734155005386504,0.05760765119983946,0.060416123844173955,0.068311119499087,0.06726861020100836,0.054370194820957465,0.05481317643791596,0.05846461912920209,0.06126219485780199,0.061184371713161,0.06386162311796034,0.050583876300629466,0.06810664851936742,0.06527375581160187,0.0621123806013416,0.061836321533418774,0.055314253603221925,0.07218313358092274,0.060359727667759835,0.06652204175480611,0.060019031474727896,0.06333231769818701,0.06091867664656146,0.0666203979777372,0.06508571858962776,0.05845390726856432,0.0634977489368325,0.06309627442435572,0.0663926953424101,0.05390155631159338,0.05519581492318243,0.0687710715103173,0.06108732830712417,0.057072905666397486,0.06949228498874452,0.05995371705856603,0.0621398591416516,0.05224807908671504,0.0709743578574974,0.06122727980162722,0.06919260705599696,0.06724428179942252,0.06468503226145934,0.046470854288291424,0.06370412169307194,0.0555576977355445,0.07168597575908592,0.0531710115458829,0.06035391857404547,0.06890289757862167,0.06229663413534467,0.06047406739634141,0.058198287134789,0.0636164222113631,0.05234025238408686,0.05853158965329856,0.057403210824097245,0.0648172135508007,0.06504087828667282,0.060990661809630975,0.05922259567114711,0.058587953785902536,0.06901940460558664,0.06778081217972531,0.05550524765699348,0.06968888658898137,0.05006928146856854,0.06722836346469174,0.07214265294685626,0.05301820775781466,0.06967504536134628,0.06098357492785597,0.06872484533699517,0.07067762028974364,0.06356136470796978,0.05832145029253534,0.06936681194363445,0.072616286174527,0.048182904336443566,0.0638428148280196,0.06371141690211106,0.061896801159020236,0.059418767310004006,0.06328452490921116,0.06980505064985082,0.06789786080336202,0.06323256354718816,0.06030749934545469,0.06588793597027376,0.07493008599198686,0.05576816398525651,0.05775283372354295,0.0751208299844628,0.07136869386302688,0.05758192618266687,0.06901853478360523,0.06580528678536016,0.0605255793904389,0.06336624728637441,0.0650131173139647,0.07708542938440724,0.06729013559688538,0.06419023783193635,0.07027321907033741,0.05702087058038345,0.06466139642829599,0.0742433104883704,0.07187398492937501,0.05097396872039031,0.07151015361346211,0.05504440608585645,0.06113532354142016,0.07418916462405178,0.05188349880906959,0.061344638131061675,0.05295215635250481,0.06495981670523386,0.05647998398270021,0.0658089173354603,0.06890182991671909,0.0613274167309431,0.05652440280732779,0.06615535261224664,0.060619821974510935,0.0706641068612571,0.07093824150436731,0.05626747423838078,0.05846440977306144,0.07076208792803994,0.055466261699331096,0.07050511134354663,0.05738309116174979,0.0659968235265698,0.05823507806527224,0.06367428784579313,0.06734570552462528,0.06436941707450489,0.0634014194696021,0.05236605265270398,0.06775297143612033,0.07031677563575617,0.06678956362557087,0.06343946481797463,0.0661526323089076,0.0682117217498337,0.06831239335426478,0.05799093594553948,0.06488685805032322,0.07395995390984107,0.06751322566418996,0.05658799655596079,0.067020402262709,0.06210006026131767,0.05997533397437073,0.06696541339968996,0.059851709796030994,0.07108244291028537,0.05810047709323444,0.05696821712752818,0.06687375959778713,0.06557478529449798,0.06606003744798958,0.07129513386532318,0.069259764629511,0.05584731477848011,0.06629204866487395,0.054403201189674434,0.06404865976806715,0.057856743528752266,0.062346176741453076,0.06590357164037852,0.058725544746224496,0.06252925432291437,0.06829009274270831,0.061882911113401474,0.07002410062688326,0.058994704797377756,0.05416224187001957,0.06350454375583761,0.07077202956415632,0.06629489482131237,0.06453069142465023,0.057775851829864,0.06744267307425553,0.06450106546944574,0.06953255845843587,0.06785891358082488,0.07364603344403417,0.06934758745611644,0.05827184230877573,0.06081315995719826,0.05260830822807691,0.05989499656898874,0.05416753051817111,0.061961701122855734,0.06524556407568555,0.06670065354651869,0.05098787098324009,0.06496858490786106,0.06729622533599298,0.06183340217580092,0.06581035358534809,0.06317037537231304,0.06542299350305916,0.06200800251820622,0.06562123842411877,0.05490973676294546,0.06536075643829878,0.06432942380289913,0.07260909766666332,0.056825568913368416,0.06358248924051561,0.06552581761824869,0.0693603791472206,0.06312664971518966,0.057077930399598714,0.05500060765580521,0.06207738092133004,0.0715723516292214,0.061211653059216053,0.06960588472487995,0.060193939879188904,0.06836812264728247,0.05941838859050677,0.05535629499195439,0.06488340089441194,0.05348904640981478,0.052829547543756614,0.062362641179815353,0.0523846002313121,0.05944013389154644,0.07195863534053734,0.05549453253506684,0.05352947510149214,0.07273454320381059,0.05138090555293976,0.06644700126741088,0.06477612710320435,0.06956946670937547,0.06284115447914049,0.058040652329035256,0.06319549273074454,0.0692961711588777,0.06392685908342693,0.06391403809191243,0.0626244559137466,0.06752641436113224,0.07122662166934055,0.0521276189116851,0.06283187800772538,0.07369918920785308,0.06407842669153524,0.06290132494134154,0.06682159751464228,0.055057750996217375,0.06542312675979306,0.06431807854671288,0.0738407313959292,0.06593538854064247,0.05549288591981181,0.07087671381687176,0.061378043635075326,0.07125617444408697,0.06791254465356257,0.07130639116771473,0.07796043284472243,0.05497273696985627,0.07406998029611697,0.07134879123276411,0.06414169145809837,0.06500256245577593,0.07280251657093646,0.07016583763506992,0.06632445990865715,0.0673427216309864,0.0675869344424215,0.06488188211602497,0.05910771726528334,0.052970497714046494,0.06304194794073867,0.0539796569642612,0.06588290637085159,0.062111479687264326,0.07117975524702808,0.07075743165083165,0.061102102968601704,0.06990044575587004,0.07557177061317077,0.07206447274244493,0.06440637045990094,0.06764989118489517,0.06686477557349026,0.06982722502033392,0.06593023137433787,0.06141861671760669,0.06472489881517152,0.06850797102073494,0.06621499803865749,0.0522079215077235,0.06203777796180464,0.060687157613082916,0.06818028190885969,0.06296333469163565,0.07429998397547656,0.06327349914822067,0.06074142342873076,0.06204795358290465,0.06709770317868999,0.06256132553472862,0.06376974761450363,0.06558730159886744,0.07434892034128981,0.06603864393338221,0.054629507641639,0.06837643260595656,0.06349596834377554,0.0648901752866333,0.0634993355860522,0.06129697787592266,0.06627597679793268,0.0675263892644109,0.06038625699112027,0.06550310343863561,0.051708723081588,0.05674677443876874,0.07148372544140263,0.07009324221889765,0.0600422585612399,0.05477305276755587,0.06067546289065233,0.07357089015650677,0.06567833025572306,0.058691660321197436,0.06571904059365963,0.06534106139260806,0.062149725605988,0.07001739546030333,0.06785456258414874,0.05227385292094899,0.06701382708476597,0.04996244890256346,0.061122126769850744,0.06799345612685447,0.06161561339605829,0.0692588680750452,0.06104748931110858,0.058420435485495234,0.055547714519522284,0.06332773401983788,0.06289396930659225,0.06295331018591965,0.06038127329358668,0.06334582050825183,0.060903114804817894,0.06727054540540107,0.06229603798210855,0.05928079171628679,0.06662800758157447,0.054620909743388096,0.05641910103277736,0.07180585100822875,0.058122868550964615,0.05234664460694609,0.06685758509904932,0.059771820440552056,0.06323582293080593,0.06985488538289072,0.058919217579847125,0.06494255350146641,0.05612591886780226,0.0608972814279292,0.07111265316510904,0.05061784035505808,0.05932305867355548,0.06166723149439798,0.06992890390209655,0.07643294604925704,0.06376230587446571,0.06620112484590349,0.06043208281867478,0.07398827835194166,0.06586212645284066,0.07102604809109048,0.0604540322914191,0.06026063661650956,0.06929445830827152,0.05705766484192282,0.0671425649484664,0.06640303032039128,0.06838447626989763,0.05097120049378741,0.06685087754378397,0.05444610601525593,0.053679065437261086,0.06522724699302815,0.06291559240913308,0.06135845555328717,0.060955969013725526,0.06330152704285777,0.06342978192905599,0.06038828759215562,0.06615972735447054,0.0642461943279807,0.06844211823573758,0.05998062194483483,0.06896685073949754,0.06553460432593274,0.05806783755981726,0.06443829154853764,0.06302523846640365,0.06661693841411803,0.06377709771913201,0.07508145422818391,0.06382144949579177,0.06307659329298673,0.06977003046216423,0.04482674259332755,0.06563514945804327,0.0639388586793837,0.0520301765927899,0.06288656797539086,0.07086445474330849,0.07321450261962023,0.06675516285597988,0.06283693354313996,0.05961775238976777,0.06664527022085237,0.06235644862448335,0.07599212549085504,0.065110425299898,0.0548713266233413,0.05123760319560397,0.058387273336871326,0.062313240206834083,0.06273453808136706,0.06515909846959571,0.060253672748725215,0.0671102809441372,0.0632858661771589,0.07152338310063971,0.06308624050182482,0.05890951214196691,0.05907755872184465,0.060385843079381475,0.0680485577060484,0.05243535039498125,0.06340957739734827,0.05595223126883554,0.04596182981541258,0.05669933824962856,0.06077771247709997,0.05989028231662991,0.06900867027113816,0.06831510041153593,0.067405503929034,0.060259224501535674,0.06285888754378868,0.058661430364758904,0.06691495980537471,0.05364755863276037,0.05989325150408323,0.07388802160621558,0.06393514461546924,0.0636863753063734,0.06537787578871584,0.07294747981538274,0.06009427076689719,0.06443413281512121,0.0596040845726871,0.057246876564752,0.06090377500438769,0.07041418401183605,0.06989298224415919,0.05694459383265539,0.06203162994851716,0.05328671812660289,0.068311775893621,0.06052410013143437,0.0689749385596295,0.06436119955211217,0.05975259817189974,0.07294745935014613,0.05697932150671306,0.061300890662504846,0.06658739345430106,0.055983833420567355,0.057013733166478675,0.06043331088423043,0.056768333674857625,0.06774870464460622,0.05664388283611893,0.06133290849049229,0.06779186310376259,0.06348943837220533,0.06665592164966128,0.061280342542631365,0.056912980414903586,0.07650738050340752,0.058164811651319294,0.057246194826197264,0.06243532211782974,0.07147338524742741,0.06240525126576732,0.06777980014281973,0.0666334495739678,0.057117110531443055,0.07349078146291516,0.06533280240692782,0.06079777702948497,0.06272474014996429,0.0538601369905556,0.0652315503579889,0.05405631327074846,0.0724489200301528,0.06341972185790365,0.05693007905201439,0.06042937518110595,0.0690183881321211,0.06581688250768714,0.06374752339252598,0.055943979815567736,0.0589452388075416,0.07008838026521064,0.05741724841521274,0.054828571951363245,0.06353949239221898,0.059597176148334874,0.07222829330415066,0.06555579861452945,0.06704282347093522,0.06800258794874893,0.07533223824157158,0.062209405578303976,0.07263973360090817,0.06276099261753634,0.0671267024116936,0.06346822931140474,0.06444730953202774,0.05884534849141171,0.05854757003064033,0.0657827031057206,0.06360009972619154,0.05802018034049415,0.06593622206180086,0.06607502700948358,0.06443084616804817,0.0656964808970977,0.07191100372095614,0.05872239748760411,0.06616510748347526,0.06179601067563981,0.0628755924192928,0.07252883551084202,0.054721179395146695,0.06638077237809803,0.047719827471075726,0.07470687662393449,0.06279527301120615,0.06658395750369853,0.05394737079268775,0.0623880216891981,0.05902952779585319,0.07577153009185869,0.057874485179681835,0.0570832926977769,0.06989475418963076,0.05749954768293092,0.07112146169321362,0.06890382618255309,0.05244754685522682,0.06098498892615513,0.058290339871576655,0.06401108292930782,0.06408716573840728,0.06837966385259178,0.060902847753826414,0.06189387066989401,0.06763624575196846,0.06200535160332622,0.06407822537019482,0.06599878180577493,0.05838685384592147,0.05745281976610859,0.06431938484934557,0.05931928911759782,0.07295424044510776,0.056210813498874135,0.05959607306887993,0.066993554929003,0.05517558190712442,0.05813706682954835,0.06980647362737355,0.06516228459962209,0.061685765995764245,0.07094932297092156,0.059199932660798246,0.06470472211399156,0.061811446375110435,0.0586338564482579,0.05584437916146768,0.06802059423737811,0.05951373936434077,0.05983511670194262,0.06923207914205323,0.05544782433833319,0.059470542598086526,0.06754743035800037,0.055659633041308734,0.07012829925401537,0.05597943569179073,0.067620671269754,0.058233464360858675,0.051181660229950424,0.07227544317142576,0.06413284004587327,0.06828138056557617,0.04729297830785899,0.06850703975650635,0.06612821635355859,0.06626878297337993,0.06312753104802549,0.05187560988839296,0.06849925022199388,0.06765467113976331,0.062107901235700994,0.07061623902633238,0.056802562207989224,0.07013355621104003,0.05618976116709043,0.07267984912058953,0.053921429487618945,0.05670758985083654,0.05600661311668135,0.06756992718265933,0.06580143056130397,0.05958134726605631,0.06065567406259595,0.0685046106037556,0.06809676314033125,0.05533956677294463,0.06430067083612008,0.06015785135770825,0.07144386407309256,0.05019423734329646,0.06358522543089873,0.06831536660443985,0.059851911980378786,0.05500103306891185,0.05644937856524577,0.060850964842637675,0.06833698879592001,0.05716645228634279,0.0714716929038332,0.0689853759445474,0.06375948438449122,0.061379523865582326,0.06686303448632394,0.062090639057530705,0.06352010785965828,0.06629655738994586,0.059459723711283496,0.06975378972431052,0.06499541950724029,0.06102836356226235,0.06391762642673564,0.06925156466163643,0.051845445047442956,0.057145379051322276,0.049341839433166404,0.06485831550285134,0.05820140009142194,0.06565836846149503,0.06471603921700836,0.06705891532958694,0.07162203367098269,0.06080730160530205,0.07046778600453833,0.06831569660625618,0.06356131796518029,0.05820712553605749,0.058263187468861846,0.05929216809538107,0.0632210923938118,0.07207383416406685,0.06791165646025413,0.05815325142518531,0.06110086949724726,0.06745814358646648,0.0741397949379195,0.05846936673497203,0.06176833458011265,0.05034653462972835,0.07064134720609269,0.0668526694082823,0.05799479112582062,0.059465911472191275,0.061425623998003345,0.04933050133215179,0.06440787327740985,0.062285616528707344,0.0671252675884254,0.0686089778135748,0.06648064514547872,0.06894215079535329,0.07016308487455121,0.06989522938480283,0.06498418653557743,0.05980797386873701,0.07322250517336497,0.0625624158537727,0.07224785118972618,0.056259932918512195,0.06575070708834296,0.06796830605213663,0.07383715443018257,0.06702819047870426,0.06524057611535622,0.06724944862139148,0.061655672672138936,0.06006842998302321,0.07095598931905155,0.06440901063232027,0.05313216726615949,0.06834896273006398,0.05602635212622778,0.06611703573480536,0.06625270410582512,0.06439968577969468,0.0733038786321997,0.0640696867249105,0.06637127849105076,0.0612354379474485,0.06489750666146266,0.05332575996385494,0.06406512348407789,0.07440582849334859,0.06491044029373082,0.07629414142879247,0.05812274425192143,0.06211832255607435,0.06582503816564102,0.059223175466733675,0.06467103764521961,0.052006482135779256,0.06354796633941216,0.06689027566627648,0.06855013351022886,0.056877061214889155,0.06837215999017834,0.0580803783465621,0.062166273371348994,0.06296612436734336,0.054316141717283176,0.06801793637349295,0.06606633497708292,0.06469190350145543,0.06449895800603068,0.07223456931170605,0.068321706614345,0.06809435475825644,0.05293064864153106,0.06866924362986723,0.0717465318777804,0.06417047694954064,0.06452919482289396,0.0580350726703297,0.06607273515234924,0.07388273828995434,0.052312024802272025,0.06787684829784879,0.06794337386496586,0.057047188727243583,0.07209600835605201,0.06431637152357776,0.07129903791104054,0.05708334427240142,0.06549089804232432,0.06976361535698211,0.04893874699173005,0.0473151367554909,0.06835705606860397,0.04951209989807491,0.060055585108943904,0.06063395288985446,0.05990501467482108,0.07382636688206165,0.059787032606761915,0.06051716713139382,0.06250702315278212,0.06679101726734686,0.07246345479824098,0.06531425680360967,0.07076463810362284,0.06465019033999697,0.07502849487413027,0.06889708457322613,0.06850155336848046,0.06757659978411133,0.0627309300382548,0.0677656372124367,0.04523519130864864,0.06522977730250659,0.0678337508097471,0.0726315835869725,0.05886415114638126,0.06335829740208943,0.061426096245311305,0.07802800658642733,0.05256904250207786,0.06309848017976291,0.07329155327378123,0.06244306917580976,0.058920945905197836,0.060762585849996754,0.06011221843247641,0.07114139772395504,0.062480390125703104,0.06322269126758426,0.06435377141570917,0.07037912930247184,0.053526004592132956,0.0683325069666483,0.0613278824798847,0.06522884020480953,0.06121296310945729,0.07116635388608814,0.06497216061074515,0.06738871168200428,0.06786101498809985,0.06700280354155982,0.054593523920579284,0.058314992957003584,0.06163316572302239,0.0576588907001408,0.06849216030587733,0.06959512078502769,0.047066088536283246,0.055464330700654665,0.06508495118044985,0.05163544427361785,0.06454737637743832,0.05212212070325198,0.06834796563007114,0.056711156974411876,0.0684685057067477,0.06069800025316958,0.06252269166694885,0.0605553750007717,0.07332072022009417,0.06977884191626574,0.06751780983113348,0.06381871336411901,0.06161491433556509,0.0631994265585164,0.06663946839100734,0.07508874892924058,0.04948362225757069,0.06474345141745971,0.056658640594097344,0.06910709442992513,0.06819069013474924,0.06302676252177872,0.06183406615991577,0.0633449213919936,0.0526974298174837,0.0638640432343433,0.07115805463437574,0.0731126144956348,0.06147730273325929,0.06942513682813409,0.07032357626893168,0.05643293146182636,0.07475395511616988,0.061264509942384686,0.06537075812753576,0.05998452795747015,0.0714659009236357,0.06450578702656871,0.06964375112174329,0.06562948241946931,0.06010088565111872,0.06240276247489619,0.060468870587487435,0.06409341570031019,0.07262509644620291,0.07440175303026295,0.06757189618098239,0.06247896031339912,0.06076287698420538,0.07123230282755122,0.050367242747754115,0.05843837102265144,0.05199046827095239,0.06972998251744357,0.0712762055938686,0.0745756389434803,0.06309810927666973,0.0752892166824331,0.052632332224446146,0.05466921172605718,0.06161684486478723,0.06567886875481928,0.053827191089187654,0.05815013111363633,0.06274163414650469,0.06962850340633295,0.06912688405887613,0.06426797179459495,0.06753144851950348,0.058892629945911545,0.060328027095157055,0.06663882100415608,0.07032464621790788,0.05478590884575421,0.0675027551387645,0.07191863609430857,0.06396613183132077,0.056728465190552455,0.05584321983859789,0.057396646229304155,0.06751244042029358,0.05774581948114815,0.07435638937772487,0.05696489405334738,0.05781881718949777,0.07226128938529972,0.061983451014754914,0.060765436666186015,0.05382037022125204,0.06156225110697266,0.06849121822740407,0.07063010685255652,0.05324357203552936,0.06444651739095952,0.06203012051952652,0.0681133561016582,0.05523116267899324,0.0633585992638652,0.06420747061247602,0.0606053932517068,0.05970757731921511,0.05412885232034775,0.07311259385582913,0.06416780267330444,0.057957338575808814,0.07383352772902657,0.06126529743720923,0.058140342151315735,0.06048248430970776,0.061615293869124226,0.06269439785506328,0.06998606773630131,0.06479464741001978,0.05267137810404696,0.06697964589996644,0.06765642067676607,0.06339027409621634,0.06875271913812095,0.06058714518737479,0.066574883869924,0.05425258860108785,0.053918429016407296,0.06278817310245915,0.072263962633205,0.05816316614196758,0.058338274665415246,0.059239540055402565,0.05822081621973839,0.05320524358734938,0.05489352086670282,0.057769886118923315,0.06310572460712137,0.06396407528599989,0.055028424602548406,0.06293269330241595,0.06520515636954441,0.06620524133510461,0.06748013446723347,0.06585526884749326,0.07113942869452458,0.06271524457242747,0.06808732031926112,0.0645023493770995,0.06277897659031931,0.06422007222952361,0.05902422065708132,0.06587424719036139,0.07178495220756925,0.06411170810453065,0.053715988163332146,0.07039505308206807,0.06147498768064674,0.05945809800386655,0.06560964878175644,0.05692811593486697,0.06646296000978273,0.06345393043715669,0.06783221163260295,0.05895230105610316,0.05849858796193447,0.06479631063821945,0.06715827421801503,0.0652727229398007,0.06514299979411986,0.06534136986046096,0.06889333472957054,0.05756301620000581,0.04960253727761153,0.06351312270209479,0.06083743254584724,0.0678196743135952,0.06511861321496044,0.06697734281429837,0.05076718213809413,0.05820561049840787,0.07147718524088337,0.06680368808359606,0.06599230289540794,0.06900931031249496,0.059419273778776344,0.06904439171018374,0.0736165719531841,0.05949614207079945,0.06689254707769116,0.05430464507751602,0.0719396394093946,0.06227991162676302,0.07048941316688004,0.06164428875820135,0.05770782339014341,0.06531200648831093,0.0735302379084102,0.0641151790885893,0.04789452492905415,0.06528996878697127,0.06914438429449138,0.05976205267135731,0.06923835818004234,0.05502359830452308,0.06409869438434954,0.06937461896109531,0.07007419087148406,0.07616193454074957,0.05581525132231137,0.05789669999429259,0.048761344516100674,0.06205369176178304,0.06694647803544759,0.06350966132765022,0.06535009999204432,0.05378647859222832,0.06495942939397342,0.06642063382726557,0.06320375010750418,0.06871816438286724,0.0625858303870443,0.0654219482166733,0.06694186234712907,0.06395093974940652,0.05801578748425065,0.06976940502017297,0.06563731937545828,0.06184507032250485,0.05214723925676015,0.06672912495592949,0.07059191644826696,0.05995351562310179,0.07567927377402467,0.06367026963254838,0.06888719744331342,0.06592149817358957,0.054195044848240684,0.06504999616738978,0.06830495074950471,0.05904500313570521,0.06751057257906426,0.05852695824173498,0.06745897665166047,0.06991503899335116,0.07414022269278046,0.07055294575166621,0.07365011536751084,0.06637446558962493,0.06332593170980727,0.06651872112048313,0.05970204138178627,0.06167445137469429,0.06789117641671336,0.06580652549916688,0.07049641409368684,0.06230323826114553,0.05958731150348265,0.05732805734041442,0.055512287641260934,0.057178688403300496,0.05854249039841264,0.06234593788532745,0.07042982289569505,0.06972603780963504,0.06599196901401197,0.07201763892830987,0.06869800588611827,0.05818966537813593,0.06537153851979077,0.07041394818902241,0.06952705638093196,0.06824531767924927,0.06735303628126693,0.07362489630422092,0.055978675043313525,0.05405366123805655,0.06239367669172126,0.06641035213670132,0.07150983136527533,0.07071483429495952,0.05793957780155293,0.073775150328209,0.056757766279723634,0.055430617505598634,0.06157500895146871,0.06548655008156407,0.0712591478258805,0.06474463637760458,0.06330123718664629,0.06263134114608482,0.06669803712793564,0.07342814178799852,0.05841854331498246,0.0616954170520144,0.06485331221718882,0.04887736065296152,0.060979235958823375,0.0630160218502401,0.0651505520624644,0.05391517597343584,0.06547258043480898,0.0624514309486552,0.06122492972797326,0.060915634820042315,0.06947804681715491,0.07124716536029414,0.06529689323063634,0.06290854560511244,0.062288583717042115,0.07065561054620076,0.07229367201607356,0.06574237750463291,0.05162760504988331,0.05892191214836931,0.05948606805810626,0.0702794901721153,0.07035256410695831,0.07237442387842173,0.06953078949358227,0.05988175341843724,0.05469509062098506,0.0655793814715345,0.06586126720693583,0.07291399767530107,0.06461090885746781,0.06438539632966131,0.06549493312616222,0.0635573009311588,0.06917466872498315,0.07051611915863855,0.062175400693150105,0.061555243687843104,0.0639178351451484,0.05214103642418706,0.049471260448651744,0.05734009165682125,0.06156253430254386,0.04880350077174713,0.054311048923671834,0.06822144744343839,0.05954258622171805,0.06741763639022344,0.06927646795691611,0.07386566863881572,0.0617657498760046,0.05723132092554867,0.07022246161555931,0.045741674691271425,0.06322331021865314,0.04463733628547999,0.06844300380700806,0.057718042104914186,0.05000853934830428,0.06902875678854842,0.06096649908977195,0.05813303600544023,0.0627647734839465,0.05696012159529469,0.06302771019681735,0.0640100028848696,0.046710747153075716,0.06595531757494955,0.05907638304621261,0.06028449280701399,0.07231952875385703,0.06947519217802052,0.06042767239184463,0.06738979140510137,0.07019531474165105,0.07165849464632348,0.06550583129512794,0.06864850654526188,0.04983806728447013,0.07597648071616869,0.0638097841443273,0.07075964219794734,0.059504417806498816,0.059498049684235846,0.06920843029695917,0.06937805380492681,0.0648214893673266,0.054610302115650405,0.06860563551211438,0.06170832479505056,0.06376033457660021,0.07009870040365351,0.06375300290066052,0.05965134927833,0.07210116134447822,0.056164114332142134,0.07448718048040415,0.05884505027970176,0.07070574503002655,0.0733289933685326,0.0707700467290299,0.059455013705560396,0.058771265146856294,0.052571692407347215,0.06925525901047724,0.0490142199467574,0.0684207917335228,0.06030735008307561,0.06460396929491007,0.07069221860035507,0.0715226902342947,0.06604209972494553,0.06031619307094645,0.0593866390160436,0.06593408046575923,0.06655059335566475,0.0689377242388731,0.055860266432374364,0.05816020244518581,0.06934559883078864,0.06766985694741473,0.060196484506912395,0.06908106338239085,0.06964948099283327,0.06674217116260209,0.06656038359856697,0.053318285763072015,0.06106275586940106,0.0699680224156614,0.07070412345258831,0.06303193508319002,0.05870071018751635,0.07093428561156506,0.0629951668101742,0.061504371762881434,0.060207018044718216,0.06819527994136063,0.063280939416316,0.06922742372316526,0.06341065433930473,0.055229959614473634,0.06621862239202642,0.07140678297879317,0.0695217417485967,0.062271034596691324,0.06862442752278014,0.07164062228123932,0.06009366137982819,0.05680531087628494,0.062049562535675576,0.061631548807632205,0.07033013376962155,0.06284434598122593,0.06552986784640454,0.05492076240204448,0.07120982966327735,0.06802854632401258,0.05948850536134376,0.07320413386341432,0.05862826766855654,0.06519760693942706,0.07406046534850833,0.054396111245945705,0.07188212503198675,0.06004632507791727,0.07297517218683466,0.05956761474090891,0.07523537755258002,0.061241803541325086,0.06672332456763438,0.07314005498154069,0.0588297125401068,0.057998827264362826,0.06785127282384101,0.06490159885534191,0.06657888792797047,0.06162993681123027,0.060516232699386475,0.061355590163441935,0.07178066118188846,0.064186596701934,0.0572059632559591,0.06680434707671512,0.06820317198110482,0.06561658277778308,0.06872194645866732,0.06259088251236614,0.06153335868179814,0.06093757594615509,0.06725188691918,0.05207050664284463,0.0654743997959528,0.06816980081682028,0.07299160698390617,0.07255921125442678,0.060957805329033346,0.06732074792581244,0.06038504182695695,0.05611320641784594,0.06579670689354977,0.057039008688084314,0.07362334715810519,0.05745331913699993,0.061747766492272226,0.06676833795497364,0.06444754231374497,0.06431079186239616,0.06567836124755955,0.07414390751372171,0.0632478846926587,0.06550018590473806,0.05602478255513778,0.05807381889162434,0.06290287459564886,0.06501567594845695,0.05610174448062591,0.06261757832495132,0.05946396957217298,0.05146077618901687,0.06500859625109603,0.07132711628524491,0.06995058423097313,0.06615521451577122,0.06895959301193777,0.06460323802660392,0.06011811786073222,0.06344708266419705,0.051353305617306756,0.0640146623189236,0.0670174079319896,0.0568210929161302,0.07014367341355486,0.05384138802572423,0.061322340452813436,0.0612310893975966,0.06098480774633802,0.06496732888962474,0.057690793854238656,0.06921204270565179,0.06620149122760126,0.06373813800350482,0.07004268608692737,0.06906364068238909,0.06544017775733914,0.06153334804866969,0.07058381963676513,0.07109086825924099,0.06434309327030716,0.06394807399885036,0.06936064600343611,0.062195706498212176,0.06267461239137376,0.06566162228157998,0.06421773182875556,0.06067372462089793,0.049814107621165706,0.06407281501997653,0.06867111790081096,0.06631306104026682,0.0659926332115286,0.0639952077774539,0.057023382019551205,0.05895647257171445,0.04748808633450427,0.06439014658346523,0.0623634629551574,0.06400704406948245,0.07141206995149124,0.065192351404607,0.05557210975610678,0.061094769626326526,0.057054959148657214,0.05349062602183319,0.06521069930118625,0.05989754085372834,0.05458104974390801,0.05987585783653768,0.058627879667787486,0.05947011933715275,0.06424055403795659,0.053920675921482725,0.05965023049193905,0.06582853962904808,0.06888788312377728,0.05845013323658374,0.06939125529694243,0.06247619233125179,0.0675299351993645,0.07789638632935465,0.054439712982259206,0.058645724178720535,0.06705693755486361,0.0646090189154631,0.07028927630880336,0.06799119517500889,0.07067202136703238,0.06898826858464822,0.06805534705184207,0.06933696594705731,0.06880035953064462,0.06071433049294554,0.06933672952607411,0.06395888823997478,0.0693894361903328,0.06925539224731414,0.06396460367685482,0.06269109732369085,0.06939389910264956,0.05380449907133855,0.06388215421368777,0.06746177916385968,0.06795000167339989,0.05842009550586627,0.06229131844731225,0.06875052167523485,0.06168047191275088,0.07508962255039725,0.052983453334551935,0.05771311256780272,0.05069452291729466,0.060233855568211564,0.07124911818432864,0.06327960411401658,0.07189322788323241,0.06403693677325678,0.05800882456264401,0.051266294833113166,0.06972228007854211,0.06067693025278091,0.0584645287751443,0.06944939665061753,0.0617962596817676,0.06005694776540514,0.05900863217843594,0.06551042176002406,0.05429052523589378,0.05088049202905322,0.05740911103831499,0.05729129867911623,0.054800891717463204,0.0530776725039413,0.06117324845958044,0.0688990253496988,0.0475557683469843,0.06026529330134775,0.059025703950865566,0.06141966382053879,0.06413121193391624,0.05879455652622913,0.06442569344901836,0.06692565871202072,0.06169473369137327,0.07374641404792692,0.0617957731634002,0.07654600024353625,0.0653032290678789,0.07410780326385222,0.05722873888369795,0.060273549705445545,0.06663307867939804,0.056770879838400665,0.06425237836632794,0.06277895903998866,0.06457933068793138,0.07114909908045489,0.06716342452309619,0.0548029541028034,0.07439897990895053,0.056962715346172865,0.06699008662325825,0.06383698551085526,0.06720963723027934,0.06367721815919541,0.06256982600918645,0.0658374540949477,0.061574277504505885,0.06478933135221099,0.06823939373100815,0.055206130712613245,0.06079438763602031,0.06915550424593761,0.07497261189890769,0.06964547747772176,0.0750969852683287,0.06235971200340614,0.06398445114887716,0.06942154278206109,0.05337222973519058,0.06674828322710172,0.06855552035444099,0.07253933571732082,0.06377802776280447,0.069059908866082,0.0675350864327092,0.05836027250550216,0.06188139581626183,0.06426824496563902,0.062437123981032794,0.049145672012695193,0.053472389914137594,0.06306919159271651,0.0708368495961379,0.06434749192147197,0.06516868225818254,0.05362131197422084,0.06337142355240528,0.07433352809940709,0.06322560540815046,0.06004838408416477,0.06327217353057268,0.06458276565640446,0.06508243385008572,0.06500082700256407,0.06419051284150065,0.06931397280561605,0.0648356787379177,0.06424407261253882,0.06134962810495638,0.06748327624843384,0.06569917462834504,0.06133019273034901,0.06056414486333761,0.06093905898969017,0.07468419595028974,0.05703231801600643,0.06793810999078473,0.06544658445375606,0.06372902752971538,0.06718403261221692,0.06769937100891377,0.060529970158871474,0.07499066762763873,0.0571685476101124,0.05788156515038145,0.07481564932414714,0.05960820132368058,0.0588913196787303,0.06332504339009229,0.06216800313183655,0.07267628081906488,0.06753735245558454,0.062374498074975186,0.0626670993098744,0.06561680901707816,0.063024318625893,0.06579716926578626,0.056589813194658445,0.06477981406192837,0.06807451756315655,0.07233393701441188,0.057625391611723434,0.0661800816968085,0.07164348288250821,0.06474241949694817,0.05961428060919488,0.06977069309329727,0.06438621471370243,0.0651774214613752,0.06096088322488859,0.0696550398726283,0.06607154488247893,0.059438272542005136,0.05962501798532109,0.06669290568133954,0.06268315285421165,0.060958574268911256,0.06953152644808529,0.06137942867605327,0.06436418797086188,0.06557317894063687,0.062420032778624256,0.0630783386944431,0.05790320364305336,0.06352121004706901,0.0722400155833455,0.06675255177671274,0.05704278389479808,0.06242595326158467,0.06225526928042006,0.06811396644736828,0.06814646811065878,0.0541956292651664,0.06066323028559227,0.07399771140737407,0.0704880504441377,0.05347975143977732,0.05540655390659632,0.06899009583844345,0.06261299607996738,0.07504249636144854,0.07000135627767652,0.06723205565761833,0.06795858642749408,0.05873767504296665,0.06347858317720348,0.06490527981972521,0.06625180517495885,0.06896617600207924,0.06218999610452693,0.056991784000281695,0.05530821370772019,0.06570719570809452,0.07239429059185074,0.058859884766767095,0.06719617003952474,0.06100453639271336,0.06384581335444209,0.06705677447036901,0.06148026872922972,0.06044038620121649,0.07580357519038937,0.05224018524710508,0.06669032679235773,0.0560158808417512,0.06332062513200687,0.04988720244700362,0.06317516786687322,0.051022972485415236,0.06254961445335391,0.05729721426028421,0.06549200338781491,0.0662492546693263,0.06851552414673895,0.06680860861846233,0.054381082213630375,0.06737176919618472,0.06220377873627014,0.06228892746415842,0.07002238765733615,0.07033230391106302,0.06225759674326975,0.06114622352412538,0.06528404878355556,0.05955148153977387,0.06433196274750014,0.06732321502938318,0.05881679554003218,0.057864258596622864,0.05535250220947456,0.06449041348394041,0.06019445342103214,0.067428707424708,0.06283962561973161,0.06555451850454808,0.0689331786970374,0.06934518792277526,0.05940438901282347,0.05709659116362493,0.06144768199470647,0.06478550522036107,0.06643555898881082,0.07512633740249963,0.06765536012225468,0.06276396788037178,0.06778710168547869,0.05933376171448111,0.06717900561320309,0.05246481032986291,0.05715782453151319,0.06005972795833293,0.06515167575363753,0.06945323527426342,0.0556612314588225,0.059637250083258944,0.06699410565802676,0.06375265364730683,0.06593058664575074,0.0714946722741357,0.045356629372906826,0.06106077782921593,0.06451502226946434,0.060772595575484936,0.07192370940528298,0.06863506822377768,0.060246430974799155,0.05934295256965958,0.0563921498749846,0.06784249490121909,0.07109846359001543,0.054749421139501916,0.049443235454709064,0.07467745358793161,0.06840274690623234,0.06395148486384396,0.0607635712530274,0.0538788775615193,0.06626027541302221,0.062246290957513765,0.060295981336881466,0.05214083291291658,0.05630895038611894,0.054601556467723036,0.06469919385293171,0.06842705178017779,0.058944288389115564,0.06698606335358857,0.06653339603927028,0.05392976152638554,0.054959521711638454,0.06213483564239826,0.0491165796336684,0.0666700587518741,0.06210230622102116,0.06238223216480813,0.06262178712896943,0.06592887484059232,0.07127107489297263,0.07047267189725956,0.06310021732089811,0.05611655793470269,0.06610370552012533,0.06984490544888503,0.06617063129279802,0.06191593585690653,0.06351776298651804,0.06049883042600208,0.05513562586151581,0.06093019009437363,0.04730248127761119,0.06791218998619114,0.057006716587377416,0.06840822080698525,0.060267665971842334,0.06681965570204804,0.05928951533438975,0.07211849276357167,0.07406616990678253,0.06095148441935476,0.053268853675040834,0.057164661052606924,0.05461795377179691,0.06737075309225904,0.054199759546732416,0.07065403062104929,0.06045982499881346,0.07109276481304827,0.06892550971798886,0.06891936064743735,0.06413860553460257,0.06744178373234676,0.061327497946083304,0.06688842035423831,0.060767936632404596,0.06841833360582228,0.05084462579127781,0.0634014358196499,0.062358102386423496,0.0626320217207112,0.0713760165428336,0.06447344817282158,0.056578707625900976,0.06285395332002885,0.05614395376189567,0.06465331560802295,0.0748638187620509,0.06889305575289925,0.07027718350878137,0.0620705934041382,0.06449196578182365,0.06351606189202241,0.06974751314005005,0.05499111305297928,0.05371439117728503,0.07228629179126701,0.05875354488496551,0.05716517756195792,0.0631909293374994,0.062193691205283584,0.05566387766674797,0.06316649529702652,0.0663876111794964,0.05256778616755134,0.06609775358881573,0.0624524280329921,0.06810424146552511,0.06823747026729979,0.056310612579754685,0.0612775924975562,0.06937387711403614,0.061101862376574154,0.06112435119363929,0.05225756967803828,0.058425820151136024,0.05572532883288226,0.06029488762404221,0.061243625285794474,0.05795578561181172,0.07190259134462798,0.06720159783299802,0.056290729597196415,0.07539859165200888,0.05505221112768763,0.07327694080455738,0.06675966920207618,0.059369846221239864,0.06606623917664695,0.06403480931116451,0.0553579599832567,0.05985667091888788,0.0620394270573091,0.059627493588654865,0.05378522154976806,0.05616549637456719,0.0680404232240473,0.06577411345561648,0.05033815059668845,0.06190580901359048,0.06070971490022013,0.05674904910821994,0.058356251280165916,0.06071369871254713,0.05718396721801034,0.06689583057041637,0.06679317636410018,0.05844145941858775,0.06393829673124872,0.06340828291974003,0.06184220912298507,0.06773189978202074,0.04695904006494754,0.06552631747057831,0.061967589096737094,0.06392644006439956,0.052114959598345334,0.05822200806562636,0.0642231614298641,0.05994032131997929,0.059401301771376064,0.05559786578431736,0.06441862801618649,0.05384189343846617,0.051568537225095815,0.07464387604814304,0.0637260633630278,0.06604419881889396,0.06948091532416961,0.06640325260010121,0.060809861562643286,0.055288676293842,0.07314807848544028,0.05739082941978066,0.07567488843134441,0.059847960196571987,0.0638875331645089,0.0658677500545963,0.0618867065996578,0.05724042118437346,0.06577210778988801,0.061253805841522214,0.06378899161728363,0.05926395917667943,0.06630900308959098,0.06631469010587883,0.06829172790254313,0.06822542875404708,0.064292485221219,0.04930390188295648,0.06578633398879347,0.06887058043137714,0.05805401833384856,0.061162752641394286,0.0698739769511665,0.05682046295518378,0.060924721388149035,0.0601122182189093,0.052508040928260266,0.06317752514091518,0.06983083725914686,0.04957223980675411,0.06060547810229326,0.06950792155467186,0.06665728749161268,0.06220946888418638,0.055204360002749565,0.06790875229757776,0.05224123619863968,0.06277796400753957,0.05643560425777565,0.06716996946580117,0.07253280476709664,0.05905481642557442,0.07064064806140125,0.06864923689289786,0.06378549908263367,0.06989792350851902,0.06315848723227857,0.06747829319858013,0.07055162944491439,0.05823361583896998,0.06983961239924755,0.07138082590231198,0.07068670525961057,0.07149615361754406,0.0642975984448923,0.061749386243596646,0.061389361661884974,0.06499622010509154,0.05742862275821706,0.058036223483764623,0.06555481116769876,0.06673103656839183,0.062015457797381345,0.05774577260304276,0.06484842885657925,0.06772718968686309,0.06865736436329412,0.06584336756474976,0.05748275345328317,0.05355413022346964,0.06318739787775429,0.06718563850542458,0.05956987838705619,0.06184806739237061,0.06743978125359246,0.059625625936965784,0.0626759018661184,0.06412084815981102,0.057004449900106696,0.06777330293547362,0.07041939366375621,0.06780827617687729,0.05732183068811096,0.0628967695591722,0.07164907234300502,0.0631419319058803,0.07262833419841072,0.06336872658279127,0.06189464122112927,0.07089518936395163,0.05639222326570037,0.06118230627861488,0.0593715950164196,0.054222059558023845,0.06422761561661405,0.07206145116999049,0.062204534023344776,0.05932297075117539,0.06340229436687053,0.06849676396242578,0.0614025793953622,0.0715702496582151,0.05938542208122159,0.049206060112737454,0.05976845203720965,0.058585645398477215,0.06790000636188412,0.06437026059226975,0.06853888911097838,0.056227081114030394,0.06937869454580042,0.05737967482867172,0.050614156846974366,0.07256089170909155,0.06368245549614433,0.05747374979395353,0.055104977932496524,0.07095537060429594,0.06418568331612917,0.06585287057513085,0.06410531207687178,0.05278707868625493,0.06523411643757732,0.06451741182092882,0.06363516597526392,0.058443233906093996,0.06362601893639869,0.06295438648188215,0.062481136475632505,0.071091074202174,0.058298155432355225,0.06211826367743113,0.05348834790462803,0.061765053717638484,0.06054163461986997,0.060390390185397894,0.060063005079725354,0.06330534150127362,0.0530102961226354,0.06115306726602099,0.0600729495393681,0.06060499993169922,0.06532429141756427,0.069183035487288,0.06726411476609886,0.05743662916120437,0.06565213961650493,0.06245436188246646,0.06196013328343504,0.06461999517050745,0.05871724199279757,0.06332769182621442,0.06646781328371412,0.05558322429689161,0.06504884281866757,0.06153048347320294,0.06051965367028002,0.061012546606792825,0.058406152202626876,0.06696918604337379,0.07163507243014144,0.05365626766215386,0.06271891874443217,0.0654153492786928,0.06734609257435017,0.05762523433488902,0.05015556913266582,0.0656265294255676,0.06867789949076014,0.06694093088250208,0.04410145532324898,0.07079626097218172,0.06125553841848257,0.07127703566156755,0.06575123315736556,0.06338569242437211,0.04932994227163931,0.05986956753746844,0.062124530424555166,0.06276032902234527,0.05663777928620631,0.05211365639654785,0.06719007355204887,0.07231549330279757,0.06950552709098037,0.062327089764385696,0.06917658616744637,0.06487451938287142,0.06055739697605931,0.06891323230945481,0.07374627558488839,0.07475497045506244,0.06324253843777469,0.06046390483679005,0.04780115528162791,0.0600067807459921,0.06972974216051199,0.06350708168215649,0.06778571981423026,0.06038571880849237,0.06953338771643228,0.06598642926726483,0.06806772330238683,0.0593735972017013,0.05833766830369512,0.06234883498037797,0.0709903599134642,0.06434440993746005,0.05645360903614456,0.06443726059101482,0.06283969956585567,0.06340774646493719,0.0737470185198114,0.06503165382498605,0.061540285796675365,0.061980102579805345,0.043697078344652446,0.06727870207934172,0.06798831238344676,0.06368240844543008,0.06730999253876725,0.05645925097728563,0.07076668156835397,0.07029282160907613,0.0559922717255463,0.06738435317154731,0.06382214776135166,0.07048231460029612,0.06275630264397464,0.05540950011601293,0.04896826028835738,0.06723860682822835,0.06535732153863219,0.06721135170543657,0.060544257060486315,0.0634003179272963,0.06061259605740117,0.05355713125723863,0.06346871649419211,0.0653631487948437,0.06887433418410763,0.06380261294108196,0.05945675838804207,0.06838056393484479,0.062302191488924,0.05865201794072854,0.0670161004432957,0.05736351470646617,0.05824837053518814,0.05803734436307102,0.06855319428428044,0.0603718828161336,0.06403008007980623,0.07518240132566084,0.0586101839796976,0.06431340796153306,0.064692467133051,0.06754238901643181,0.06600528058720849,0.06227921525282193,0.07099317954215514,0.06544759849229653,0.07101302865377948,0.07312479915439314,0.06908934783979043,0.06501613811907653,0.07469414611649772,0.06297571132489554,0.06757628347107864,0.07039027135636818,0.06676746098171188,0.07062929667048831,0.06233666587580768,0.06393224750467101,0.05512989272752337,0.0606429983921909,0.07230161475682931,0.06733573904373502,0.06429156931746166,0.07139042701140591,0.06622571556151902,0.06872517654910962,0.0616658897533186,0.061450847465966414,0.0746118400733933,0.0722286399567302,0.06154232365931645,0.05835720786933454,0.06869560028865765,0.060054829478009705,0.061382781552615444,0.06553462450302736,0.06568396800392348,0.06078733015619507,0.062327067338931154,0.05722762769196846,0.0573965207625149,0.05888210086867399,0.06637797204851625,0.05165868393653187,0.05742406696592959,0.06455027673768407,0.052267850290061785,0.05656358894863895,0.06566599700992765,0.05465527907242904,0.07019841544271645,0.06413435577371268,0.06363203807479909,0.05658632279367011,0.05665991190296028,0.07163393959150852,0.06170206360426348,0.05522296974906067,0.06769951035057012,0.0695517442399611,0.0681686330460784,0.062494625497089555,0.05983202920067851,0.07313464379274465,0.06462997553808986,0.0556609563674237,0.05928747499664181,0.06591012724610881,0.05555486319407778,0.06676950348282967,0.061728302743151636,0.05262194056210401,0.062352283612841446,0.06252061469048752,0.05898270702210906,0.06317740314711107,0.06748764889192925,0.0706351621902798,0.05982662362326797,0.06896046491575,0.06633384857822457,0.06965550127312334,0.07204469657930734,0.06179194764096754,0.06321556609308403,0.057436277642057484,0.054799898999364355,0.05975083405648057,0.0650872323021559,0.05955102893653687,0.07123956900406284,0.05410715460382376,0.0514338096537293,0.06408337961853866,0.0663939026666088,0.06309287363802009,0.06700643758232437,0.05665802209495219,0.06786888261006249,0.06946780375523455,0.06292567632258543,0.066167721043946,0.06883820151157793,0.07141693250700243,0.06857550510763501,0.06540046706767974,0.06559474677887567,0.07202703885996165,0.07080387742095837,0.053913271342751645,0.0692685611050472,0.058704235392278634,0.04637135905112975,0.053525898085560436,0.06066602626301837,0.06709747765358176,0.06267921396822208,0.06775488262483037,0.06440863844085518,0.06805583847080601,0.06529174554731991,0.0650460927755496,0.06029359250879328,0.06603388948567515,0.060626002206171675,0.06778242512946252,0.06362589995979254,0.06847652981019939,0.05292454516556419,0.057812226983444764,0.0668988494041742,0.05316016043201946,0.05969230884665333,0.06189230718599333,0.04924509000999351,0.060441168770863085,0.06355191868274242,0.06657134311369076,0.05778421099414399,0.06693243078578487,0.0733840380858079,0.0637981795184124,0.07166973015029447,0.06511764872720112,0.07095580274336134,0.0602719078279096,0.050916733427360306,0.05678180929136968,0.06469229810803381,0.05744391702174499,0.06523446971944438,0.06200204468507418,0.06513430975804421,0.05966699206582849,0.05276852185875974,0.06632832165220146,0.05652977762888735,0.05852584688013639,0.0635661708986387,0.06549626516822162,0.07318621240215825,0.05398011951805144,0.06359868255772337,0.0464317241377059,0.06894086390901824,0.054893754815926656,0.07155932144466667,0.061821297995872354,0.05877583760441621,0.05728058256288317,0.07432966866598817,0.06806039219636847,0.06705785982382365,0.06567806775148474,0.06331274270258812,0.06679551103881973,0.057004119139221196,0.056845224014657816,0.057982264865976824,0.059117554417148666,0.06230485130730525,0.07448377344752372,0.06380300263973487,0.06935510413711915,0.05868845715143959,0.06957278720561377,0.06652402390235224,0.0580932309259896,0.0662824005993698,0.055797127851787344,0.0524700834149573,0.07031952705811556,0.05807753875330515,0.07001370147585198,0.0640599087425596,0.06819204407482526,0.06624284979454009,0.05715220738363999,0.05493617582243525,0.06327265304611557,0.05755669452029111,0.06908710346595787,0.06772796363221759,0.05996625477987104,0.06815701182676145,0.06460266333109843,0.05594018923817426,0.0637528135841963,0.06318726166146527,0.06784694045002013,0.055496391084477695,0.06734626361592402,0.059900385559390465,0.06776445809696327,0.06373867429885499,0.05933092868617776,0.0637715829448294,0.07081907253930621,0.0736835792826088,0.06980547288699748,0.06126069274806406,0.06176700107777239,0.06033385441249939,0.06377226990444607,0.06157449936446981,0.06345088250785666,0.05692540084028705,0.06464106314148495,0.06560141890766741,0.06483365975694708,0.0663284708995837,0.06694090864144579,0.0667029874939069,0.0657549413926907,0.0633402441997056,0.058720486051439154,0.06773423944948385,0.06769207334873276,0.06443087975449488,0.058908974616600315,0.06626427866369286,0.06335453532798771,0.06239743198316177,0.04962259899344555,0.05680516902915425,0.06155347351764928,0.056413358205362425,0.06625358090126313,0.06271533934153684,0.06691191763330219,0.05830414292745179,0.06309458705719505,0.06515786998855326,0.06211192659022902,0.06325167561437126,0.05635625816941574,0.06497397155162865,0.06081277182258192,0.0700012765135072,0.06842162076064756,0.06542252733713236,0.06220142888641633,0.07072853891847551,0.055487203587350015,0.06376863411354428,0.06007690856427951,0.07065471934985323,0.07269575395518812,0.06718420178534584,0.05899102664162112,0.06837527434819937,0.07009003166280042,0.07121348412792697,0.06109604722635317,0.05906401802771151,0.06704441817195372,0.05868470039690893,0.05595736086251369,0.06730063232448852,0.06203698173639723,0.0703544098292522,0.06690891491304754,0.07160386756636124,0.06368632738769234,0.060737116302862784,0.06370006014634866,0.05731613720938367,0.05607080398734735,0.06522580251815335,0.06487538271032789,0.06139284689902242,0.06384219186666568,0.07268591497695459,0.062040372595839645,0.07100582769905456,0.06927462408464065,0.06202478847591784,0.05487667057493753,0.06846975492809472,0.07242688105604125,0.05755032719940717,0.07253712695422193,0.06828852091346277,0.05816144002483527,0.06219601892566001,0.057068626723359946,0.06040794862009918,0.06434959047465967,0.05933286786437828,0.06497309819520333,0.062003841113922584,0.0520956319776406,0.06244229633268771,0.06856153242594928,0.06550408567355562,0.058249751440893435,0.06625378987911426,0.07226685925533757,0.0676934261047833,0.06731275862342145,0.06544265316095514,0.06249327935783376,0.07302680840455572,0.055125750591252506,0.06785287691326164,0.06351499211095096,0.05904292439880991,0.05958615818790504,0.06664690982048556,0.06439398540625987,0.0652968847423743,0.07110549513697811,0.058602973016797344,0.06545154373949366,0.06911179800505152,0.05792107187611287,0.07066901539123382,0.06758258878096732,0.06871505358127089,0.07646424730207121,0.07358977817540643,0.0653148552756445,0.0631859306898484,0.06387182715955714,0.06121348107437147,0.06531960602705433,0.058181738694898996,0.054348082817114396,0.06649682996344333,0.056319371148638164,0.07168526095356875,0.052893587057799535,0.06850693138777,0.0717592915479518,0.07051479281141639,0.07067281164607991,0.06499491710110336,0.06836038346222215,0.06657644576019833,0.057488746465011245,0.06510385277209912,0.06729868714470066,0.05217792419546782,0.06427148847848185,0.07061877231281732,0.06010952217509416,0.061498531935296474,0.06179167941205516,0.06172306662303708,0.0609505588540491,0.07055782680412548,0.053802326663895256,0.06671622643354927,0.07023632500447544,0.06607155294087096,0.06275386287473818,0.06322034506840962,0.04782086723389268,0.06208907383050306,0.06718866580923055,0.06345329593326074,0.0710492658919796,0.06296427358365078,0.06564080435583022,0.059207468378357705,0.056893793699462014,0.05637071342691326,0.059075230096308246,0.07190339315467284,0.06443985455302367,0.06651934852889621,0.05133685894345456,0.0615392182510293,0.05284960141073076,0.0640616921168332,0.06988635779315946,0.06306337067187612,0.060936919546393395,0.06083837448127175,0.06985264286754285,0.0721085442418678,0.07112033015243252,0.0655050767912621,0.063172945485154,0.05614758617948197,0.06376710243035852,0.05412277357025659,0.05099528715572932,0.06430945901664001,0.062425593088801856,0.06579282732076519,0.06858317452728228,0.06631108829328244,0.06633191620912526,0.0669144807329075,0.06960113265850414,0.0660730780308332,0.06231741477977112,0.0678029423286362,0.06554756738168478,0.06270213793459992,0.06967211033857093,0.06343006172213275,0.05542841938802136,0.06439490569914415,0.06297481004988889,0.06450494196684223,0.057632298028715154,0.06468458527906915,0.06554442279685394,0.06743753839984326,0.0659008667006476,0.05355862517585342,0.05316091631507765,0.0590833600433192,0.06990984973178124,0.06162139108254416,0.07085837516827553,0.054636629428075456,0.06104927577859111,0.06823736628733612,0.06297790225223328,0.05537187210645042,0.06233059344673999,0.06338523806721356,0.06650007878709055,0.06681944175022969,0.06046848422703087,0.06247048096628176,0.055329400980111396,0.05945205025287742,0.06955159120425707,0.06274244561317854,0.06087356063311648,0.05798388005220763,0.058066929456435476,0.05469469308649339,0.06474432044543482,0.05539500665140291,0.0668540738127398,0.06404915732798594,0.05689030299204593,0.06229575577678611,0.0658624738525157,0.06310483584075074,0.057896725238032506,0.06550025467152426,0.06717297439813488,0.07265102649960006,0.06550553560890086,0.0714758621798117,0.06253019365524201,0.06250815728970141,0.07266012423607913,0.06376423283777605,0.06160950240737047,0.06460223368176596,0.0631620595405222,0.07156127187077224,0.06345932873309053,0.06229854230084568,0.06442791870093931,0.06112704278448207,0.07211458772011686,0.06234169399725043,0.0659425902242376,0.0663980848541186,0.0650267479434589,0.055628444851783594,0.05477493669905302,0.06477096245941033,0.06383553526302951,0.05081465405225086,0.05660208627200336,0.06279055356313526,0.06733844581259431,0.06826190858632553,0.052763742208094905,0.0710014924529298,0.05533865215672347,0.060574528415619305,0.07024795551443419,0.06633869574275666,0.06250711719692947,0.07086038571709194,0.0693794264551538,0.0656446075378905,0.06527379399578662,0.06349852837001628,0.06318818337903809,0.05100827165479945,0.05900580594120915,0.06010666097794737,0.06827753281549392,0.06096469868913763,0.058396397096829464,0.05938941301201131,0.05148961769199027,0.066298609367512,0.06159333007791606,0.061643129850033326,0.06732515989704455,0.05360749678869291,0.059584249821929156,0.054367517089458584,0.054001603793473116,0.05736844625327055,0.0591167050422884,0.0663023066614522,0.05561390404622227,0.06290324186936455,0.06271293752684137,0.0656536962218972,0.052095905177643176,0.07118121733458312,0.0687016689258437,0.06280358789973628,0.05849073513992403,0.05253507927665292,0.053904670033126836,0.058070177448085736,0.05361257392519595,0.0589668636225208,0.05419126972932316,0.06378370219568039,0.06058596676902362,0.06922532108625862,0.057500254362973985,0.06452061162868025,0.05831448048460927,0.0716485412698264,0.05845788837889794,0.06280491254758178,0.061802337756010794,0.07286912769813375,0.0662623992426217,0.05623018707443134,0.07172906849929253,0.06794258947130336,0.06095858791177196,0.056669933264170025,0.057864151971452046,0.07424266913259167,0.06364573068972071,0.062427007283497,0.06406646987341531,0.0645875672760727,0.06920059254456626,0.06315959333981738,0.0603369236120023,0.05649531903990222,0.06924130456393451,0.06975493321019807,0.0641290271681273,0.06932378029190234,0.06677495556989543,0.06440347065789215,0.06933877943924438,0.054122488116199366,0.06153320901549556,0.05510281121081468,0.0537595211176721,0.058619456412868684,0.058748786608717365,0.05452342279911439,0.047100116279322785,0.05833604637827369,0.06707087337754852,0.06519795453136422,0.07078014575599546,0.06785121170673972,0.06669870843584152,0.06587697570920747,0.0748707121343809,0.05408562157212156,0.07035801001335762,0.06307640943185325,0.06629394316452439,0.06998967820503901,0.05889047668147124,0.06818703995618866,0.06476742354998358,0.05503255155489912,0.05078708895536761,0.06735757501197641,0.07146910485378699,0.06753667652447418,0.06144107017363757,0.05914729071890677,0.07178733212259635,0.0674810491168954,0.06301386811135251,0.05789830318406118,0.06984256535937641,0.06407789427280348,0.06427778030446388,0.061025570455251296,0.05841313053221303,0.07343295918956744,0.062099185080307234,0.06077296758438247,0.0656617766647897,0.07004508986460085,0.05402442004266769,0.05951187028655153,0.0681521819887958,0.06598613492211841,0.07008312415232662,0.06336114777381918,0.06028444968271411,0.05766432755597402,0.06157628590488634,0.06412087792626414,0.053894881736713585,0.0708735137425448,0.05311740041278676,0.06634077866121718,0.0624462131191279,0.0683492096698842,0.063032695355288,0.059887190401250436,0.06899401185459361,0.06324982484111016,0.07301081951688901,0.06453000182004755,0.06783494922915892,0.07018955546426461,0.06918860197101781,0.06034291853263461,0.05847794566559087,0.0652696927794016,0.0641208510412043,0.07540413590468556,0.06818522093208579,0.07162666288698874,0.06726847158106071,0.07433219561417412,0.07368942194128417,0.0685226695675803,0.06817087766358498,0.06225567046500626,0.05775192357890743,0.06537233252787045,0.053162269654601135,0.057311700656951806,0.05969336572804758,0.06400446321762712,0.06755835596937054,0.06086174291857299,0.06693669104263678,0.06935506249735418,0.07146493293803008,0.07132726281378243,0.0669089920344617,0.06741070235945437,0.06465159672379686,0.07326984966320846,0.06480072412875174,0.06071569224288359,0.06865823441287575,0.07011603541464248,0.060976667976883434,0.06451057416943207,0.05951282026501739,0.06791829784238802,0.062166066094818055,0.06727044413252325,0.05899444267439003,0.059655676851619106,0.05943126256491045,0.06081707294542629,0.06107866616980083,0.06273206808640894,0.06453676694908442,0.059831010277921995,0.0646925036116916,0.06700687622340906,0.07232181976769453,0.06468923458399353,0.0732557510557713,0.06553170329651778,0.0628717730339121,0.06536265347021342,0.059211076913672,0.06397353507698941,0.05099570585798248,0.05893609372989908,0.062455952251701745,0.05892692847889203,0.053501749480996584,0.05739796476383985,0.0580974172080278,0.06993928679106323,0.06843456164158568,0.05590874215674606,0.05563184741258798,0.06764564679578701,0.05819326295102907,0.06672153605607237,0.0618686128216245,0.06691416003725623,0.054665534620111984,0.05490044475096996,0.05705191708862589,0.05883810086028629,0.06117576032138666,0.05869680146724193,0.06121057637468848,0.06453467427814844,0.06927052185532376,0.06162246503789792,0.06990016050425521,0.06555439536565752,0.06617452451901734,0.05103800840411361,0.060493829171097246,0.051169270476658174,0.07010635173222227,0.06628268806716088,0.05230033870998852,0.06845296016325939,0.06337134456329029,0.0680172354427602,0.06259366582467749,0.07093591146103354,0.07471814260917017,0.06400658049881042,0.06139579677937437,0.05885494902835596,0.05632936979250748,0.05596622132147381,0.057426846554171164,0.05944372494345476,0.06196153212256229,0.06571370834432512,0.04936424336523927,0.05714832623972671,0.060161456710892666,0.057800253074321684,0.07268620209384892,0.0694368443481368,0.06620308998652377,0.0709927971120295,0.07396995556912866,0.0609399368528914,0.06889626165638672,0.062392251792981114,0.06847352784729033,0.053188543823537285,0.06068993113354085,0.06638322371301313,0.06652653515345168,0.06501312355115126,0.07525560202651185,0.05084219421966994,0.06852672835231033,0.06403565440466372,0.06765013422549368,0.07165926041424672,0.05552422896912316,0.06600191045101926,0.06936258592978103,0.06905260294593511,0.067550687884749,0.07295612429828541,0.06351180416115361,0.06758878294830872,0.060121345595954626,0.06906503599854816,0.06304872946666115,0.07018735223874825,0.0683502014717326,0.0677057183965202,0.06481968058148034,0.057492405014610656,0.06876212259862413,0.07023623006773624,0.06056393938965702,0.062494816558180365,0.06577391709885992,0.06426820859804827,0.06706991283987437,0.06275364820429555,0.06239986356686072,0.06555033463118379,0.06856631098869825,0.05369176675433407,0.05946996935723442,0.06538853867444244,0.06630427207446643,0.053908070135941455,0.06120819682809643,0.06342512168628281,0.05592984203615826,0.0742680309837164,0.06821218950860267,0.06383395163448236,0.059059069240880536,0.07406480026414852,0.06187511075894931,0.05933102756188387,0.054451403534573506,0.06934394527334632,0.06543230883826771,0.051628386564535556,0.0603094880062011,0.07189600955840925,0.06669457797194478,0.06708348764396316,0.07070540388549515,0.06851138864350458,0.06032654368365625,0.05591172894571487,0.064928134486996,0.05763931929878906,0.06689862706138071,0.06012794717842945,0.06604162555662918,0.06851289730363451,0.05736370995899887,0.05386220633219842,0.06189361825972691,0.06603706713291621,0.06752310450762919,0.05723328442462719,0.06236254095052707,0.06668544989817687,0.06395727474367333,0.0680792328058688,0.06971130443888013,0.05874810712803291,0.06235706350919197,0.05403747160825821,0.058614976981994235,0.06706665524380134,0.06116706718801809,0.052415462386861886,0.07109476466716826,0.05915659723517855,0.06255761181824901,0.07206095894984987,0.06976851428127502,0.06021221880108503,0.05494660878753951,0.06840842031918826,0.05034408827751665,0.07460542476608385,0.0703278677584582,0.0674396530589639,0.0512686021184436,0.06028885282977649,0.07225656049666268,0.06931835813950703,0.06514432158975492,0.07337597211088923,0.057133317034614224,0.07356910486117327,0.06416104392645268,0.07459719983032258,0.07060689763313437,0.0674990556487827,0.06690267226654659,0.06525546130787337,0.06924343692311449,0.06816089922235288,0.05813398719015575,0.059905011087683493,0.0649011697357755,0.0695528820218888,0.0599789362299164,0.06173761345049116,0.06809761459747679,0.061490733025757686,0.060235133743282,0.06314265272972956,0.07020728561749777,0.0664600592094215,0.06438284297745253,0.06460953192735014,0.06671929482837582,0.06195998396293732,0.07107043539869286,0.04910307110623914,0.06676280626643379,0.06309119826514284,0.06589764905197015,0.06969340343201999,0.05512122954707914,0.05339155195291624,0.07321461407563223,0.07024908198469741,0.06662980549202288,0.06642168363229262,0.057137677935953715,0.05336065133325805,0.05980186438877045,0.06369384747057319,0.06804708489638958,0.06898569247724815,0.05995261880128265,0.0628261774337312,0.06123772789379378,0.05213011174493992,0.06885821297858646,0.0653991787969125,0.06471578153005911,0.06842412691206035,0.05679355185595148,0.0704730581748859,0.06979561543603255,0.06024232265324657,0.06372848474282364,0.05848673888669864,0.05637714426176597,0.06765386151729877,0.05864085312964466,0.0590284017488855,0.06156102489238835,0.06729415496876745,0.06493556641716106,0.0653669086390106,0.06058098443850078,0.07007695632100752,0.06501346836156877,0.07272607510032571,0.06677424384790025,0.06568663611007111,0.07503698855267596,0.06550535585310861,0.0666788843312418,0.06498396509158003,0.062148707090138935,0.06631762444496801,0.0663758289124079,0.058356187260187264,0.06242238488573825,0.05346293461522446,0.05463541494729213,0.07603749367846643,0.05249378392409171,0.06025333759921526,0.0649634986132212,0.06893503396190666,0.06289226249194996,0.04989393850241601,0.06774571047037845,0.06652685901691467,0.05274230478859516,0.06931800260028254,0.05563796223278267,0.053188501025577664,0.06642884696503853,0.06445853042023154,0.054901368169857793,0.05418261594898932,0.054627111934825966,0.06742566400422692,0.06844708547521108,0.06071427552392314,0.06250736456774426,0.061881481535157226,0.07313005497980887,0.06741241656584718,0.0597441627616214,0.05578022272183603,0.06116172639674944,0.06472856979641312,0.05456695010002699,0.0562814450153199,0.060231725961814975,0.06492393920775553,0.058487374061424686,0.06305509048691665,0.05434614413888013,0.057485630059929745,0.055604492341286005,0.053352899117064166,0.07406686394394826,0.05675930196168506,0.05768712673619432,0.052917548077651214,0.07339661659356597,0.056525517605265654,0.06435484346660908,0.06545582629570187,0.07037024935895851,0.06249653787668132,0.06686122674010793,0.06314253236720095,0.06285306045521935,0.06206005210744941,0.062189640319396715,0.05964354545049062,0.0755215879993648,0.0667173488432532,0.07110210363155015,0.05978792474129431,0.07138907622472682,0.06394434420062695,0.06683616366560177,0.0648824351901261,0.06086101435929614,0.06161623318157682,0.06329921839948466,0.05447050353037795,0.05828987900824775,0.06486930926986434,0.06740885966909639,0.0665828953302158,0.06015979064340282,0.06061556438759448,0.06819525562741224,0.07026986208738381,0.06039774530769902,0.07103801738708665,0.06689503584797708,0.060350810816142325,0.0662304349743113,0.07038574498138184,0.057921132608286865,0.06564271925101503,0.06228700359339863,0.06393675017888975,0.07021588070609651,0.055314097019489736,0.06181768174923246,0.06155248428620907,0.06577625345678889,0.05958674123089379,0.05799569369875317,0.05548105338382095,0.0565607017254623,0.06766922525829817,0.06360329742972445,0.0490611225348002,0.06792081908050504,0.06640660782189192,0.06637025167321964,0.06505911121478003,0.0703359631445917,0.057462508022542616,0.06786993913523642,0.05561462119067502,0.06630465952306255,0.06285540746189464,0.06792425261121703,0.06544271377349625,0.06396141380240304,0.06200173061254755,0.06415768892941082,0.06748069076108183,0.06932259925277875,0.0646786617063659,0.07172848149871133,0.06687536266252392,0.07100968232480588,0.060709653916254464,0.06339247567178039,0.07099084736864707,0.0646112085622243,0.06787101831137216,0.05992399742212713,0.06202666524526967,0.060037147301401486,0.06514046314272853,0.06631592554750113,0.06684855346266529,0.0578699408653951,0.06672059409380098,0.054874830374655834,0.06065553516062766,0.06583763244080035,0.05812022235303958,0.06536137514125112,0.05791645376265807,0.06242428765916707,0.06960466124082049,0.06410581718897437,0.05165880181620482,0.06722840236427968,0.05748469386698672,0.05692535909232083,0.06593176749689522,0.06553105252553126,0.05847216070234475,0.06867742961351603,0.05994741217119616,0.05112465355332649,0.06335567200516748,0.0686170561393322,0.06061084940570039,0.06571537959425591,0.06606351699241092,0.05774574311517075,0.07203356421102305,0.05821493193190952,0.06394518697909543,0.06537713888200442,0.06344858093563428,0.058974473147253334,0.06496431998476238,0.06567251292409909,0.06387873914465825,0.0657345659054302,0.06093010212747702,0.06740527133135514,0.061055577457163765,0.06761182038802845,0.05902633254786846,0.06340525252832259,0.058317580995269805,0.06785484497091962,0.059264313645233116,0.06915632302729544,0.06783614143135439,0.06726608369058748,0.06089797607567941,0.06595484643192592,0.0693471709365843,0.060248225545908196,0.06339831997507825,0.058907724566858555,0.0730063058197607,0.06638783486111592,0.06163915726359592,0.061653057567490246,0.058416339419784746,0.06329079680312247,0.06846680718157792,0.06567255373678166,0.06641488003586954,0.0590487539027295,0.052942706093762344,0.05983760981088452,0.06735748677046953,0.06177599769429335,0.06196203407945707,0.06582012831325637,0.06637125250510766,0.06567488776547797,0.06877737339574858,0.06099088650061478,0.06305223522383707,0.07500651906037364,0.06424902141241721,0.060569877217418314,0.05939172080870804,0.06640607697514014,0.062072496515932156,0.06165396059644134,0.0656991221346657,0.055627853943654645,0.06736274705911183,0.0631248198140301,0.06772502459987778,0.0706432368826661,0.07170181996526531,0.0738969381420127,0.06166147451555224,0.05464434383306978,0.06872619194808599,0.06558077107677536,0.06893712992303017,0.05733213775920719,0.05909300463133911,0.07007416215387907,0.06195831185156845,0.07100096233506839,0.05513702951947244,0.06763989497087336,0.06556667109351058,0.07249777971947394,0.06642548520149796,0.06364237219208274,0.05303665229002826,0.05800727028896924,0.07288211381612987,0.06625160892996598,0.06023976250193615,0.07386546549662315,0.06837370610757673,0.06683254251303938,0.05554911174123796,0.06180580393397783,0.0595834966701771,0.05804415103964679,0.0688678828272472,0.06811290038470164,0.05115772882616171,0.06259787659780491,0.064037113221014,0.06340421561802131,0.06649484389681046,0.06407380722210762,0.05948105205126448,0.06261975277110945,0.06881231200371618,0.06127818391525037,0.06425853627311595,0.06389481063061986,0.06102650223698923,0.06106474040885232,0.07268809560293629,0.06771091837242138,0.05793822827873561,0.05802012984555055,0.0698403279332857,0.06668847771187186,0.07058126495955355,0.06118024749095652,0.07184919141790394,0.0702623468969982,0.0582422475107868,0.06354341771233843,0.07135301890074723,0.058834230570414744,0.06517326279189972,0.07062242804624866,0.06352866985653603,0.06210010157278841,0.06693273910623432,0.06434655053812216,0.06422093036438037,0.0571884139138816,0.07217980483372384,0.06642754542358577,0.06593900975069938,0.0692218932694976,0.05633502686353001,0.059040342451440936,0.06812498243794222,0.0592192089236913,0.06950879133889085,0.06923448779391381,0.06525258366626208,0.05946774205001476,0.06598671604251101,0.07527955968276617,0.06151737153185969,0.0638989856476721,0.05744780424964052,0.06680703695028561,0.06244281455184518,0.059095993558000215,0.05638848555475309,0.06159301512896415,0.06945412549225567,0.0684725824059352,0.07154639230412742,0.0672822645300099,0.05330144652738565,0.06517718521563232,0.06686512491327205,0.06571404996873714,0.07195909871939804,0.06923061802921054,0.05489508249465555,0.07358402550436338,0.057317221069040844,0.061763825794109044,0.06204195053294713,0.06845490749253565,0.04797098982100274,0.06730561687245636,0.07045920417260315,0.06701758340243491,0.05820879740259855,0.06433454889661736,0.06826194094630647,0.06165324075096508,0.06505353324495908,0.056129727857490055,0.07086220629418932,0.06087998929866921,0.05688469133878467,0.057843044834944415,0.06678280180643953,0.06419820018867362,0.06423679413763028,0.05870852060689044,0.06746886859447225,0.07346644133379615,0.0575639744921608,0.06819974049115285,0.060377166081380954,0.063199001290987,0.07214808107104054,0.0599761501624768,0.0710948101777068,0.0647596945564707,0.06609199723439471,0.06782290035582392,0.06096956588650357,0.06715705690079674,0.05313726419740078,0.07310388508155863,0.06644438523753912,0.05693750053455228,0.058298743328261744,0.06117033028881451,0.06487220011607533,0.05924048965333767,0.06430498270300089,0.059409939565068626,0.05926154954916875,0.05082468419332274,0.06878954036288745,0.06433361108345714,0.06436363306804269,0.0544723693855656,0.06160405903110949,0.061086626877079886,0.05723551841919431,0.05994415071833477,0.06918260075856833,0.07096049608489977,0.06655534217714698,0.058888771626380024,0.06366415810639323,0.06128480621327544,0.062097068029518435,0.06844623757469323,0.06998284675568973,0.06054286273766672,0.06412036282055225,0.06858676550498902,0.06436716005362492,0.06374827159680771,0.06936496362933112,0.06871925139698398,0.05916325072695727,0.06010436522337184,0.06049197128033908,0.06485280413522101,0.061987874949171545,0.06465512757008104,0.06414859149185341,0.07387409720134253,0.06565838446959214,0.05975396346356796,0.058979364294255654,0.06276328995091573,0.052875561823842315,0.06786238380567125,0.06582602595040521,0.07402120249544167,0.06898103613797461,0.055474582622915505,0.05595021702004707,0.07415848287842439,0.06088967257945768,0.059334718654432225,0.06070704914543343,0.05759617773121728,0.06631888980012351,0.055674036928331075,0.06847336262318497,0.06106562437373003,0.0544293668686858,0.07657326097013203,0.06383970618896309,0.06558588590619599,0.07009475592527903,0.06297961660219409,0.05934437487781907,0.06995358857744696,0.06463526606774647,0.04670141967159454,0.06973577070040118,0.06686334600496804,0.06337022060507129,0.05801072658053877,0.056222603168124274,0.06035294840475207,0.0627138604674759,0.057516251349496644,0.05955629519070882,0.06580831644048465,0.06584530847394254,0.06666725661684608,0.06039698027345371,0.05033132732243918,0.05700810288387931,0.05547050430290322,0.05701648250469586,0.05670278039396623,0.0706694612057926,0.06043663374390139,0.06430950754616518,0.06917812931520725,0.04692920241159977,0.04824794197486438,0.05816106945716151,0.06140405532362305,0.0581974554125857,0.06848796331642165,0.0643604662895059,0.06298210627873138,0.06341441058284086,0.05991770569862917,0.0700358370598459,0.06162725697397018,0.052274816865513024,0.06394280452242926,0.06405957974966717,0.060194066058298606,0.05855436511625704,0.07143445127089887,0.06587040422005508,0.073963800629301,0.07448443883210547,0.059641642077175774,0.06876950729818573,0.061316385434461114,0.07148434045042144,0.05414636483366931,0.05712129482225448,0.06837429323661373,0.06145815443234015,0.0666769237121776,0.06785774457684446,0.07346856194065227,0.061398470413356145,0.06492319438420707,0.05553755908513061,0.06271482521499813,0.05458899567740744,0.06559366095038499,0.05621908879147347,0.057197510335066454,0.061552095717661945,0.06281880035859247,0.06580683763738665,0.06415115491324119,0.06339642831223255,0.0627589814231959,0.05840176556833973,0.0699594005638731,0.0696995473260093,0.06435148358493986,0.058409736640022955,0.06419826314118787,0.06081576082776716,0.056060508728007864,0.0737441835710709,0.06180982600880865,0.06426910423238302,0.06590423467842754,0.06458355250194817,0.05984133984553673,0.07346859741134239,0.07022103328401985,0.06482226667536412,0.061088783516995565,0.061775250660393695,0.05927426628957525,0.05743932389908199,0.07414902659354228,0.06158824457880026,0.058765150011385214,0.07568599697087251,0.056379233018602676,0.06533223573981747,0.056321487788966645,0.06070282207317536,0.06901752962870225,0.07208595804178047,0.06866728364066163,0.06024242616184626,0.06614395320076416,0.058378758651845664,0.05970059238132487,0.06352026291721377,0.07042580016567321,0.06466703151535651,0.06002560562551751,0.0673068442318216,0.06382664985237992,0.06506334086021996,0.06416365337651285,0.06709583416253972,0.06764298970483387,0.06938327969118163,0.06290316186280956,0.06607834814569988,0.06411841732161888,0.06445779700293006,0.06910192728423423,0.05629523091668244,0.0708217505403475,0.06462183871125254,0.06264098443329331,0.06623968082291962,0.06143926308348709,0.05784041816209201,0.07031045931170155,0.05615237304065035,0.047492250993512786,0.06565046670039473,0.06381899279672126,0.04991074034014742,0.05919175122094754,0.05610279135331692,0.0642046253575559,0.04997114421855772,0.06854646419588108,0.06849266979886033,0.06117262975221733,0.05910538557702473,0.06277241123019717,0.06416855525838452,0.06192712316041665,0.06978280993645661,0.05455591173553197,0.05745797867002923,0.06941945040139219,0.06483412236135191,0.06708277144654101,0.0655224538377496,0.06919110448255049,0.06712294531473459,0.06832190375292266,0.055121904821136936,0.06485568664857538,0.07510294325140476,0.0647819337086151,0.05188851986326476,0.06294424349045077,0.07293263646186063,0.05791973326012951,0.07078165672104533,0.06646758385340218,0.055704312305179804,0.06885395316084564,0.07101274873034108,0.060104882179366166,0.05536169683019446,0.05487502569000054,0.060177781129966496,0.05789469977088503,0.0728584128445251,0.066398493907254,0.06723364940135715,0.07290357609484638,0.06382069133855132,0.06234722899986558,0.049670106470161714,0.0607393384258193,0.0580631216377898,0.06330688490926911,0.05641823103028393,0.06599384523136811,0.06164879756492758,0.07120359953670113,0.06675172016874104,0.0533064263798521,0.051974854053743955,0.07142630727051918,0.0692908384307927,0.06703717805300458,0.06512943940040689,0.05983950927344649,0.0577205908813373,0.06696845865675444,0.060184110070950016,0.07259305827785198,0.06149239100567055,0.055290795632015347,0.06459234564576734,0.05711605366826965,0.06791972761656137,0.06969719306477884,0.06397670129576227,0.057350058057858005,0.058382723896276426,0.05987992228989662,0.0673639676941891,0.07309586673554525,0.06792211322208776,0.06411263083114874,0.060776118021670766,0.06105176803349378,0.0647123201533142,0.06329905453573884,0.06250222188431301,0.06124515131556994,0.07319354213638443,0.0584144742990568,0.06813395041754478,0.06449242554301388,0.0695825423502728,0.06224295482805103,0.06456554586243696,0.05995814195881029,0.0658815586356542,0.07319534209004594,0.06467662696422177,0.06707349617205413,0.062091196679210474,0.06474875317700476,0.0518186285072137,0.06514476706118237,0.07322228514343379,0.06586494329819428,0.060804232439142084,0.06194897857613856,0.06454011109603194,0.06844949909432152,0.06335928746532593,0.06539435458667019,0.06328765122011458,0.06143644588876208,0.06556062019376477,0.06560400863951779,0.07214102133689995,0.06109424710232504,0.06530271265826867,0.0651529717226923,0.06403145181814329,0.07651712831820932,0.05697233866279048,0.06638239784257535,0.06233648604046204,0.05714205793055148,0.06347490236304068,0.06854121108265324,0.06667461430494283,0.06006335182878801,0.0747128097032013,0.06672229411283587,0.0595462060682007,0.07042767499546723,0.06077725445774419,0.06012882314814314,0.06814369737800666,0.06385036606169567,0.061978042129096286,0.07105899284669044,0.05764322713541145,0.06749226781087514,0.0678933312383549,0.07144882613600476,0.06368098465422503,0.0664066824034753,0.06036236103249311,0.06889109834665881,0.07129476573448701,0.0667887230540106,0.05482663069127541,0.0642183025337491,0.06298558392637936,0.06336397938380497,0.07321570394744864,0.060609100341875145,0.060116337677295684,0.053430797048336004,0.06361234681268688,0.06555222526017296,0.06936466050642842,0.06676554094832621,0.06829012148643943,0.06978061437171147,0.06217603535861404,0.06376868490102464,0.06592652574699345,0.06271471136777254,0.06577948918027086,0.06981003927264265,0.06422060125991425,0.06662412934223055,0.0659406132008766,0.0598712317828531,0.06425326366942391,0.07090695401532318,0.07085613643053487,0.06136932463549689,0.06200387196210924,0.06611413917435358,0.06068952843495962,0.06157069264836345,0.05885410440701006,0.0656225161847041,0.05968857139941626,0.06394677756311648,0.06546223268514825,0.06571280902754614,0.057502368922405206,0.056233172154429384,0.05901257721933144,0.060321777744615675,0.06502942700511666,0.05626002641445342,0.064388420988944,0.0686021140086498,0.06252723288947708,0.057960865704026185,0.059502255170753,0.050269724199367274,0.05993412450552743,0.06673331687850101,0.07083572016998255,0.06049775085328121,0.05582338201092789,0.06340786235428561,0.0566217369848957,0.0631912546849013,0.050173422744100415,0.06729642271524114,0.06548681925215263,0.06142271522312211,0.06634187817286698,0.06890343306269114,0.06287002175659459,0.0628656772251828,0.05158298098753451,0.06204165503690879,0.04882380085179997,0.06225994721901222,0.0666080699707122,0.07072676174522197,0.057153403521060256,0.060780224190855325,0.06341894983909213,0.060368415814444104,0.06507341119933238,0.0631210185257213,0.0661677759492714,0.07006657460925109,0.06433205193169411,0.059710257685596485,0.052196247154171496,0.06450521189984515,0.07068136985833769,0.0542445584843939,0.06505624926586845,0.06443585164975277,0.05942282922261072,0.06644141219910313,0.06837735362317188,0.06683858479077373,0.06247793938143163,0.06898986072010341,0.06715741495254637,0.06523236980569577,0.053342222848761056,0.05698141408833099,0.07242258952592442,0.06757173784990453,0.06575002893643439,0.06624750746542563,0.06902697453003301,0.060912227104527525,0.05300105995311506,0.06857199311864542,0.06236142776173768,0.06686031776950516,0.06894205133110753,0.06429646089580252,0.06778937236593158,0.060671498844977925,0.06533114963531075,0.05961247910728237,0.06101231688334287,0.07020799695738682,0.06784039153688873,0.06015245518797001,0.05942702456147726,0.06716739877627959,0.07445500106457709,0.0653287671294542,0.06312839505974317,0.062277034071945674,0.07117328764015171,0.0653621021317517,0.06558198323710003,0.06680403195618236,0.06571006079862285,0.0650165752223236,0.06560369375264677,0.05702474438603096,0.052942743221912045,0.057904982825699836,0.06459089046944146,0.0685804251766304,0.06026672971127944,0.07708884093885304,0.0647067042393506,0.0661567732886251,0.06665291980059791,0.06148671363538124,0.06546207696186253,0.06562998365117827,0.06743006760866038,0.06972980355708754,0.05595878386455955,0.06029477523268649,0.05683306330761074,0.07228792822700361,0.06885258577620003,0.07176507269510263,0.06965886621492051,0.060544964316894996,0.054746324326798615,0.06868969217186333,0.05968586747094767,0.06185259001339747,0.07451702396417306,0.06842537699660334,0.05268702799669827,0.07161915359957427,0.05069891680322322,0.06351425632297,0.06752650554564783,0.057178561197820756,0.06399905181002344,0.06860595394588512,0.06108984950833502,0.0672273944237186,0.06702324683617646,0.06166791245772301,0.06571997462009088,0.07225334275102097,0.06110948990843843,0.06723959926199419,0.06572553057221059,0.06908802191536101,0.06511159740917882,0.0682758248986694,0.07057925033884603,0.06636472076954764,0.06577272891933239,0.06459078888423324,0.06162199634185935,0.06767035229747914,0.07097128393682087,0.06404430003319336,0.0616660500535151,0.0685689576061426,0.0709569270952301,0.06688845239912838,0.06531102683099503,0.07056991872943896,0.06227355981178219,0.07315615203605055,0.06754947880863063,0.05988196286853008,0.05625261731972762,0.0652267889359053,0.06256806736533341,0.05824936756280006,0.06483453669019737,0.06578282954505482,0.06195614090077383,0.07031492136591246,0.06739633350696442,0.0637717048550775,0.07138473682253113,0.06360120410742356,0.06317375549928095,0.07123551600822875,0.07202008250735503,0.055500041783377696,0.06087948909563721,0.07036244373634436,0.06695810205549663,0.0696913960794126,0.06738566662143855,0.070668522925333,0.051277561161990316,0.0705903143316052,0.060504745014340555,0.06803902994979952,0.06347427258693167,0.06253615071976709,0.05632780118410273,0.06586238224562407,0.07413286327367126,0.06489080771778977,0.0690516370413212,0.05771615315598389,0.0688604019428157,0.04881396542728003,0.0667267637680162,0.05284335779477432,0.06473966554478529,0.06497018333047716,0.07237041140748238,0.07190440648438759,0.061930215845942375,0.07152432029610178,0.06521218577290507,0.060461716292772416,0.06788650024028905,0.06598552642055827,0.06122944843075988,0.06580342448869264,0.06271438588772815,0.058063211926871194,0.0714916689125053,0.060437482080735024,0.0675116906910253,0.06519077835772964,0.06756153748273572,0.05846348387280873,0.0679341569468471,0.0633187067239423,0.06897617074183932,0.06107505330731725,0.06168548165055187,0.06616210444580084,0.062223611884314654,0.0707020662433746,0.06138130369100069,0.06061204120856466,0.06155597463163867,0.06931501360396716,0.06188710087378576,0.05852810042313737,0.07008999504166943,0.0698597647128541,0.07355122632352315,0.07329640549072733,0.05036355865386773,0.06687675379645765,0.062215081528749494,0.054286700190892025,0.0629013711553955,0.065671541672226,0.07041120941540721,0.06261701361661062,0.05982384091203974,0.06325388154753173,0.05923448392602215,0.05583536485814351,0.06249795576120192,0.06761511898654332,0.05462230701952643,0.06728459355731009,0.06928183855611611,0.06816685612338269,0.06427800970242803,0.059852511862567334,0.07023128978496798,0.07289558541419784,0.05783785012053588,0.06310749284894018,0.06331454010939144,0.056906755005752666,0.06258385741527364,0.052421904454104926,0.07257245898627282,0.05121623945167618,0.07413136943400733,0.06949873384845423,0.053346254526830464,0.06098523503168597,0.059003408634814014,0.062041807188549354,0.06499605872429684,0.060882525396904905,0.060398989630766425,0.06917665658295644,0.06554253882408603,0.06907550240237037,0.06085825234987686,0.0646034398485893,0.052704293923854846,0.06074462531600956,0.06554232486021164,0.0717672392498741,0.058935332431150456,0.06091377614656038,0.06735306039900746,0.06475245508919297,0.06085033925500614,0.06640172099826858,0.05474191439910061,0.06910514296453468,0.06526375331731044,0.06406952340985086,0.0619026689668122,0.06780411707149633,0.060162820737504216,0.05680377383505403,0.05819012326735172,0.06336438616637106,0.06372387995919651,0.06415917536730667,0.05764213095447923,0.05229737929133453,0.07465951580581559,0.07283094842820564,0.06121960295166235,0.0746732989199164,0.06661634115111051,0.059645373870095116,0.0637098667593039,0.06051297362903399,0.06348039750798395,0.07064125813142211,0.06475721456234218,0.06341243963351105,0.0557706149960817,0.06720550981155726,0.06452428777300379,0.06037187682327294,0.07494172753690402,0.07017212167331882,0.06842387829255558,0.05966648798852604,0.06488916616904773,0.05785092107175932,0.06682438236626786,0.06646997479816907,0.07137018049330854,0.06850442556899022,0.05647463810281528,0.06813469746315029,0.06674105177214978,0.05629340287898936,0.05272509081129175,0.06505518512030808,0.05483036253482017,0.06915613751104357,0.06538647511263485,0.06782949764494692,0.05535139450014976,0.061836462639423696,0.06762362424718824,0.05586472737183498,0.0656987664629361,0.07561677680354423,0.07081716134257064,0.06892041179988725,0.0627042408976041,0.06715340250584431,0.054339264083516844,0.06591920809223314,0.05543322903042702,0.061892345440135554,0.05301242005299101,0.07231228436661871,0.07329809479860924,0.05940426797230937,0.063998759037331,0.05522438190629411,0.06376211047997865,0.060906919381136765,0.05696151645749005,0.06402267298337075,0.061666092903317794,0.05642075315775198,0.06324873062074657,0.0647124559291116,0.06397315776645647,0.05955879720893535,0.06478276759640272,0.056374703842848026,0.05684315496013079,0.06100721864981064,0.06199003214286097,0.06378645848070928,0.06633794841511037,0.072038751128364,0.0695288369046714,0.06687791954735181,0.05418749955212444,0.05689806248768206,0.06958358990790428,0.06838489351929483,0.05169519220812617,0.06705293577988947,0.0694475389155573,0.055763690117353576,0.06664467968136384,0.06248354102635282,0.0696335448342565,0.06807559000509458,0.055636902170658184,0.05177807016548878,0.06399996318559566,0.06508500253630826,0.06193923422185248,0.0635108136655862,0.05900101046339166,0.06616274266611458,0.06652500663815233,0.060861074574188774,0.04812811417308102,0.06995866035640882,0.06681842360606968,0.054712699672978135,0.0689151832946433,0.05685743866114634,0.06016238602038014,0.06643808185289868,0.06288370182608302,0.06496406238587951,0.07028917001089302,0.06385791041574557,0.059979852600076905,0.06003921062389235,0.05818777737957573,0.06205356159172215,0.06526000684375571,0.06972271024505247,0.0748821497719531,0.06086074012376766,0.06514350480048461,0.06499616765025881,0.06840363600356586,0.05808713618761972,0.059849496286087336,0.06771793182217045,0.06352829261817391,0.06359131771722688,0.07286993689636952,0.05847647847205145,0.05889178921525917,0.06603807811575287,0.07138946161245517,0.056665385778842864,0.06712219751801324,0.060194816894766454,0.060475868656724205,0.06286188090476862,0.05842823946277363,0.06599090354318274,0.06715467669806321,0.06071680892491432,0.05649050068113251,0.05588903610845587,0.05642431293010628,0.06462637113435957,0.07165726220360608,0.06039291852895307,0.05860809704273768,0.052533230768985424,0.04589143586240108,0.06377383177843032,0.055225924112242794,0.04465427838526742,0.057676482877903376,0.058805382346335956,0.0666902999519472,0.07390540669453904,0.060817283154035395,0.055345933421387085,0.06460531691647228,0.05942426021412364,0.060233120280024546,0.05821784335318329,0.06933212733534123,0.06745163017453575,0.06469433846483369,0.0662059171812297,0.06726164495533277,0.07112081641006412,0.06909937520404205,0.06016909364105156,0.06976754436952848,0.06999787172252345,0.05868040838030545,0.05537340524044608,0.07157801098965388,0.05941110583328493,0.06782315209537919,0.06295867134472699,0.04964920399384883,0.06576580537130347,0.07116166334330568,0.07320447850848068,0.0627399289990282,0.05992037207970291,0.06553679137498059,0.05554275427813375,0.0664489366786214,0.06047566159427466,0.04342234589525589,0.05864837309721708,0.06662673019451909,0.056309763293852205,0.06263508066025146,0.07046486351746638,0.0561611440397331,0.05273172488623762,0.062034048626735096,0.0665090155319763,0.06660824295523576,0.058890729748843684,0.06854396640828345,0.05830117967130388,0.06099956109143765,0.06544063949907869,0.07146016620647125,0.0601316038912973,0.05823964436499292,0.07411600389731204,0.05329087756157015,0.057102135586113684,0.06547680977033646,0.056424395098081406,0.061090389730825215,0.06440394974360084,0.059692718868628536,0.056327531332703185,0.06412310950170452,0.05541706323532147,0.05700663519042602,0.05999136071730307,0.07075953228342714,0.054231619206562526,0.06464712223067376,0.056224990717138795,0.06833808887351683,0.06505922486754324,0.06798513557964965,0.06221697520928131,0.061517387605227326,0.06822781905873436,0.05993179791035291,0.0679438255669432,0.0611923811328011,0.06733199184011718,0.06953815281385924,0.0666296857874315,0.05905069098329172,0.06196412642868543,0.0671586597131007,0.06752480909554631,0.06823594997327434,0.05797113208042198,0.05624123140075066,0.054437815248650184,0.05552039682336693,0.061258291102211614,0.06042678447119247,0.05559585412348737,0.055270650391142165,0.06645432090696354,0.06929679105752996,0.07026273383093046,0.06783716016909716,0.05127694142101352,0.06930716123218411,0.06524265515489575,0.060321028818536335,0.053493830934408404,0.06789697971629763,0.06889339378288067,0.052086032914108925,0.05643046759570489,0.05855120665996169,0.07298846586713245,0.06876974765984212,0.057523268254892373,0.06247512000517504,0.057007275178327654,0.0633737152952723,0.05841308595134015,0.061348155713289186,0.05425413028278426,0.057871824988073195,0.062040535003990684,0.06865592954985889,0.0671699010842624,0.05526209265755378,0.05910217910236726,0.054986784808368605,0.058778176559224224,0.06101224648906661,0.06338318158757666,0.061173032067884256,0.06427018334812659,0.058643627906219575,0.04667033691582436,0.056424309063413586,0.06537211257778068,0.06857516923486229,0.05606524634394977,0.06444675335218332,0.06557853636669819,0.05961406543987971,0.0539268945999925,0.06286128271576089,0.06106463973271643,0.06513570498820147,0.049551278013521355,0.06739773578017122,0.06465827216686093,0.06434692020013405,0.06920727320875313,0.07334931480967109,0.07103720234903052,0.06387658029365448,0.06016012599484658,0.05861685564936087,0.06595758802087157,0.06391644482337214,0.06906932336817961,0.06639317426997773,0.0659956423111819,0.06352118676739572,0.06202488407177387,0.07507058399984427,0.06711532048741103,0.04925741388873957,0.059824882368250024,0.06351598226132404,0.06117209518961551,0.06408495580514723,0.06566299412802834,0.06931390771910359,0.06925143316732044,0.06673762114095166,0.055952267646681465,0.04548759251459168,0.06308375969079777,0.06482373122718202,0.06590883477816098,0.06697964755482888,0.07550577043774873,0.061348059918417165,0.07233634651432322,0.06654834385033395,0.06737867268067331,0.06606013704457817,0.06596190755012796,0.06751949583747138,0.060880786272522455,0.05986990828562601,0.05962157233436388,0.0728627680632408,0.06261955918593423,0.07118423424306286,0.06940694673418608,0.056121479320886514,0.07126613771348064,0.06779553665020399,0.06432095013003658,0.06717822978294793,0.060862815510834056,0.05936552734888556,0.0670780234810584,0.06520360220851415,0.062169685056276944,0.06440177665954044,0.062274634710834255,0.06739342167853662,0.060297165654629906,0.0673741072552124,0.06889828759885586,0.06223097057649803,0.05420261948105836,0.05288721543657642,0.07290822610855813,0.055559629182784814,0.06540281510925403,0.06050129788248988,0.06459950761845668,0.06751853387337459,0.06765409715283513,0.057401786074572345,0.06569261249458919,0.06334267029190213,0.06779007569087991,0.06404052347580358,0.05897297371646423,0.05527366291621003,0.06396699429685837,0.05199073350468374,0.06048653521302045,0.0695922705583717,0.059208624026260426,0.05792013813169267,0.06545080532776366,0.06684403412552366,0.06371397989410463,0.06776389729755103,0.05355572579654061,0.0629348365125421,0.06657684133221176,0.06456923031071837,0.0630927418942326,0.0623101662841865,0.05926255571789814,0.0723179848478514,0.06656390635918834,0.06363691254021314,0.062443970277514964,0.05278843430540488,0.05526916007866726,0.06590795540733828,0.07147113094818314,0.06907561567109888,0.06492843843258861,0.06599787821930592,0.06247295975793584,0.06486466323617918,0.06726790051949909,0.07046521955337344,0.057130127574223595,0.06890266959166799,0.06384852926253781,0.062240021146361645,0.06342311257022477,0.062118590688967336,0.07700828924073877,0.06949645432909085,0.0641636683979824,0.07282974738157055,0.05976164824883304,0.0692248612577692,0.05500701804026979,0.05627230857145868,0.060404705703546516,0.06475746462262749,0.0713972117931338,0.06501084693503609,0.07144584723129127,0.06705189786771203,0.07098954171083706,0.055057908003798686,0.07215954175777622,0.06526603450461192,0.06563411832322211,0.06104389442651677,0.05548988695323716,0.05009048430965585,0.06523331899146892,0.05593153124786641,0.06275977395028323,0.058989344295416926,0.06603651107208489,0.05998460595148057,0.06882457809183781,0.06557771007715632,0.06412035456970114,0.06936918886848037,0.06884627569977236,0.0609835002067453,0.0627409802815694,0.06254774256319354,0.058787221253377465,0.055478276290409356,0.06868246892546159,0.05950845815111737,0.05701150234094432,0.06455231373975509,0.06440414270435674,0.06249810808033531,0.06475469010642713,0.05632891427947434,0.05798640779942155,0.06629371937719047,0.06292226401193804,0.06182880138592492,0.05793486126772418,0.06851701814208472,0.06553989763274481,0.06971313553297423,0.06321168925150242,0.056816042388062385,0.06855896204504597,0.060768898301045766,0.06464013907695088,0.05947073867460405,0.0643397172660305,0.061987875037514975,0.07113337324131719,0.06698780798774026,0.06211855149556946,0.06344892259989432,0.05887192323451143,0.06608011369199834,0.05782304544991207,0.05563809417422834,0.06526452750542577,0.05823263000461236,0.05279064169578984,0.06665481946622576,0.06460431588721574,0.057414222833925055,0.06943221856834242,0.060929220502928,0.06723124769282682,0.060308461896082674,0.06380835565144412,0.056093038596070885,0.06061643517547542,0.05005506851067988,0.06432610321989526,0.048290788619135,0.07074219678288865,0.06640124149415608,0.06720375865115046,0.059727800102154754,0.0631130644007998,0.05890025615029992,0.05936894581090249,0.0677769890523052,0.06086314777616901,0.07359822173099126,0.07076576616267877,0.06453638462568578,0.04957370134931755,0.057640215044869506,0.06857657457492991,0.0656367914931999,0.0701900139060328,0.07190601371797783,0.06369176563690393,0.06489864899576718,0.05602438538043565,0.07191463968788492,0.06617611545202107,0.06576679114616246,0.06410459926238805,0.05919048176421789,0.06447771031354729,0.0730558554096364,0.05996882094793214,0.06470198438638881,0.06570708460440533,0.06140639568916891,0.07172969436048109,0.05871887934479457,0.06067759130556509,0.058719772894718254,0.061072538517524616,0.06298511416188124,0.06453844494496574,0.06775360470385677,0.07207126049145934,0.07239398387358996,0.05558663007363192,0.058942994520157985,0.07111558519467659,0.061716206227267104,0.06769938612835366,0.06347440432634105,0.06640427951091385,0.06365848873234503,0.05968166202472859,0.061525026636995134,0.06221250281323557,0.05353038677773312,0.059193346895890564,0.07343983556102313,0.05885857036233515,0.06561566139065499,0.058162762277054705,0.05516042390949921,0.06632400878617295,0.0748678643104981,0.06095050117573844,0.055055613092867495,0.04991967003135918,0.05032323226323133,0.06749339602181778,0.06594893119704734,0.07002461853889269,0.06446579179972696,0.05149192944691269,0.06958077532058742,0.05981738470477164,0.06857280957831993,0.057327860052255576,0.05480703071466643,0.06132560691558724,0.05980532676133521,0.06100887149393733,0.06395206293502287,0.05725924750389217,0.06954036445526593,0.06782744925433803,0.06492415060421705,0.0622213387214625,0.06773782719755533,0.06556138545399012,0.06631086801429723,0.05908834158675833,0.06667183793525802,0.05863993379166655,0.06326037019539966,0.0690853958284171,0.05433564926919704,0.06688186026877294,0.06663619386970382,0.06482379916862321,0.07285776650080733,0.07192387998140862,0.059852092597674836,0.07329715865650631,0.06987566402368649,0.06735484408024292,0.06958499062408843,0.0453507407168405,0.04886113892720856,0.07311872164059355,0.057902031457943176,0.06945558656060952,0.051009734978036264,0.06693414288445185,0.05926325481246965,0.06899927109009138,0.0645445273149762,0.0756268831064942,0.062138616534635045,0.07031937156225393,0.07389470663812484,0.06541277049122277,0.05206410293783682,0.06968650145017528,0.0627377783493326,0.06268610872680552,0.06911999229876642,0.0709631604618407,0.056764074837992215,0.07226053423359191,0.06769212965764326,0.0690519801209873,0.06123505098145149,0.05841090470945344,0.04925113337861981,0.06589948125510689,0.05856738543853863,0.0652002743277249,0.058972698138276144,0.06302249515296549,0.07163787506811142,0.06139444059911703,0.062486689921537,0.06481550029772662,0.06801348661528481,0.06921804040794827,0.06947400264178952,0.05360931375659189,0.06665909205211953,0.06537633357154847,0.06421302155065431,0.06203202739143619,0.06377816781556128,0.06502063578955707,0.057968490256807656,0.06874575326557919,0.06287200850527724,0.07133107315171877,0.055643445729274196,0.06347595553281296,0.07115399930567841,0.06124911490812954,0.06443279870712759,0.05969728259760366,0.062424770756967424,0.07115999750100874,0.059772338272422676,0.0631390291783318,0.07049236775042336,0.06809856063474372,0.05984647913395971,0.06599033100553336,0.05774469232069715,0.05712887332758574,0.06295384886993526,0.05842116030175363,0.061838976309256714,0.06128307321831989,0.05935834991833912,0.0616511959080158,0.0729450337937749,0.065875565116485,0.06893307213032573,0.05950987612181721,0.07085633975564234,0.06665272798047653,0.06812265545374505,0.07053015513110167,0.06428472268923002,0.07475056660863405,0.06853470779901993,0.05205774429319848,0.05619444212762491,0.06650621423526086,0.06190685555642465,0.06598356165971017,0.04799007005723871,0.0690641447292416,0.0634045111897941,0.06772896587942145,0.06731559472660587,0.05222097332912619,0.06571233006709909,0.06008866323319921,0.06597470301138274,0.06478068725570882,0.06395901294460887,0.06246399692101469,0.05979160116205176,0.06651083295821586,0.0627726896768854,0.06322941614937133,0.06408020774157229,0.06603198120120679,0.05537428857106616,0.06116140743624825,0.06317572231647747,0.07147649793184721,0.06486232726704237,0.060581090033448706,0.05270595024297587,0.06614391018466577,0.07450377627950126,0.06260946135134526,0.05360154503154514,0.058696718330923076,0.07328287862006512,0.05874693825805427,0.06153231146235541,0.06887171725274849,0.058517663376610775,0.06365998394052885,0.059350333950702674,0.05777531068569353,0.057648892867872566,0.05756654362588194,0.05375728822004128,0.07327046057904207,0.0633966883227743,0.06745480729216422,0.0668564610628059,0.06751554003648062,0.067988411698118,0.06378891433169535,0.061569763947349414,0.0705535054597605,0.06665182729220168,0.06840132993121392,0.05094673118402254,0.05490774904113481,0.0625886249344999,0.06531690581871662,0.06191041113398703,0.06063606034393808,0.0630809087533919,0.05616378462818292,0.07559039065025669,0.0688109534920146,0.06664120476451613,0.06066269068425756,0.06992404972821657,0.05514956132752323,0.05288835697577386,0.06539867603553298,0.06230037671976859,0.05172164166563696,0.06361006273092969,0.06337882476382031,0.05654561374244524,0.06410760951112922,0.06518738260266785,0.0645986113172102,0.05233028917346488,0.053465397248366346,0.07344069780622617,0.05562741022939358,0.06755270373965948,0.07253961088915907,0.0677679379755424,0.06139327579600872,0.060519292746737466,0.05689720737748388,0.04744091522317318,0.05225326230286752,0.06723943192560897,0.0547107392031488,0.0601425604285457,0.06059337955038902,0.07367894981521021],"xaxis":"x","yaxis":"y","type":"histogram"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"value"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"count"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"title":{"text":"Home Growth Rate"},"barmode":"relative"},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

In [18]:
random_kwargs = kwargs.copy()
random_kwargs.update(
    {
        "investment_return_rate": simulated_investment_rate,
        "home_price_growth_rate": simulated_home_growth_rates,
    }
)
simulated_break_even = RentVsBuy().calculate(**random_kwargs).value
render_plotly_html(px.histogram(simulated_break_even, title="Simulated Opportunity"))
# Print the p05, mean, and p95 of the distribution.
print(
    "p05",
    np.quantile(simulated_break_even, 0.05),
    "\nMean",
    simulated_break_even.mean(),
    "\np95",
    np.quantile(simulated_break_even, 0.95),
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="780ca03b-3f75-4cd1-8784-5c8e72a53d2c" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("780ca03b-3f75-4cd1-8784-5c8e72a53d2c")) {                    Plotly.newPlot(                        "780ca03b-3f75-4cd1-8784-5c8e72a53d2c",                        [{"alignmentgroup":"True","bingroup":"x","hovertemplate":"variable=0\u003cbr\u003evalue=%{x}\u003cbr\u003ecount=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"0","marker":{"color":"#636efa","pattern":{"shape":""}},"name":"0","offsetgroup":"0","orientation":"v","showlegend":true,"x":[3108218.317265922,-5163821.983140271,-3481382.7904270906,-11965929.211236069,-4091526.4010519087,4160457.652819015,1910469.6029169671,-5974517.614612583,-5010842.084515071,-3034414.197648138,-5707588.747448325,-7653861.970036881,-6234178.786469163,-1566058.499307854,1094702.2213231008,-6965081.932770293,-117810.00027956255,-6318391.137825385,-2240448.825650219,-12510675.2759805,-6562037.128171718,-2878190.6117641553,-7082878.368860202,-450570.07436099835,-6158471.36396497,-2258354.6064046174,-7299946.727179414,-4870423.678899113,-1571084.6375591755,-10987982.355183464,-11163880.155524667,-1071881.1791814398,-7051636.35424608,-5803536.105823824,-1699263.9349654317,-402337.03835280053,-11746454.569436945,-3815654.70536628,-6110674.1326403655,2234170.9018830694,2702259.484154569,-2103509.3049860466,-282133.9835646171,-3245109.725276144,-3801471.3703249544,-5030145.020141646,2532559.3245572727,-8607843.75289791,-2585590.3627230497,-3787936.029725585,-2354938.8870969824,-4131410.1377359647,-3418340.5694950074,-5221576.846669957,-2725730.0999817885,-4598232.375379339,-845446.6458054464,-3888064.259009259,-6236066.29256255,-11398266.320417728,210031.33191801608,-2443478.827357704,628522.2388971038,-1847052.9390362985,-1130654.2880676016,-3855423.4976666328,-3343203.2545389086,-4083443.3723938726,-4888046.244614318,1191800.1323734447,1199835.908325985,-4281188.640742043,-1145302.5134831686,-9032187.69486242,-10137114.409919323,-715978.5023267027,-1104125.589228969,-10695707.584617807,1959305.0783186127,-4250258.251461873,-1445836.6530254185,-4805759.862231075,137404.37614372745,-5715768.69276329,3071801.8105648514,1084912.8815534767,-10965809.955686627,-7951847.332965173,-5888292.518812204,-1589671.5925617944,-1723617.8652645089,-3574852.512307385,-2922778.896000726,-3805847.597732544,-9910025.0841737,-2241563.331461737,-825693.6604085173,-9051359.687204305,4911236.059931619,-4168196.9567515217,-6271339.479368238,-5121156.677022177,361227.69952367246,-4552698.601130517,-6596638.278073218,2313448.954357356,-3544486.501566168,-4709408.982660411,-4918290.90085702,-3112447.0920636877,-2846633.0076315496,-822644.5496016089,-4028906.943739945,-2453449.414712183,-6579761.353571849,1300692.623283729,-2121820.8369968496,-731893.4094838277,-692665.6558713131,-3503628.0955697913,-3488030.198556792,-3212087.354180039,-1752181.1319303084,-8113635.304522183,-3461942.3782634046,-8487674.182712937,-2088228.4432336148,-3297643.3702356983,35899.54029595852,-4255869.118104259,-6474252.147676718,-2920221.6247277036,-2668206.3235321157,-4056529.028334221,-4432649.54107449,-9654914.206916478,-6466409.941368993,-1215439.2858955152,-2498309.1330368947,-7184282.076569166,-1689625.9194570985,-3271234.5598152503,2385446.5369118117,-4438986.3861096315,-3814339.091576388,-5053648.16015964,-6929027.914583208,-8378476.166364364,223552.64173659682,-1630145.3626817297,-8027408.697864497,-5672755.6241293475,-8351568.2260939665,-3130516.9885730194,-4553856.170472559,-4413870.519769808,-1253157.8840879109,3804320.015189437,-1149146.5810469482,-938656.2050797436,-6883838.124801533,-776771.2195610069,-3831353.776689523,1576042.0077337604,661117.147867512,-4063843.455893874,2391673.2291518543,-7060089.474787883,-2174.0569328852,-4186067.201078186,717257.7633630913,-8031216.177671646,-15149648.136828946,-3894460.4849508237,1196800.6884802934,-2686986.790382374,-7935185.5623521395,-5342691.7049258575,-2420365.6707172077,-2055187.652336022,-780483.001251094,1021347.494795613,-4354196.113267486,-1587097.359211104,-5434373.256929912,-3231694.2728971653,-6395167.121162796,-2762938.4935882557,3047666.410868263,-6130768.599220596,-7775469.21268471,-4684941.68817037,-3896734.129051782,-1724880.4980106074,-438415.6932355948,-5419603.807991082,-1533571.358912129,-5871050.91442525,-9099978.146972328,-10323086.512351457,-4396366.022246785,-1398243.2540917397,-3125313.5091795456,-2722841.209637154,-5373797.781827087,3477714.498140568,-976871.2940649139,-1425160.1125488002,-13739637.94160431,-8805897.443575012,-159956.05923466198,-3562588.643979918,-2600625.8674568646,-6502549.141943313,-2180538.524475964,1229912.0122371279,-1272593.6668575555,-6262518.007687381,-2472067.9890210107,-841342.0772638284,-9335058.23524982,1338705.0515354592,2058148.0900454894,-876532.2101895567,-2168110.217180131,-3966030.222471568,-681222.8819779418,2274044.847926939,-2566434.93356687,-9281577.720501605,-4519878.489599682,155638.234571293,-5057558.797636697,-5949486.551692901,-5160698.128520146,624021.5368692316,-4415837.176199214,-4504873.772343336,-6347208.451657517,-3408112.069834776,-3365927.2317019645,-1831736.4425613396,-1265565.0414901376,-1503984.375135282,2804929.780770542,-5421425.822156433,3252749.9522811696,-5306530.751106158,503844.28376088664,-9297047.7967979,1521291.1637241188,1906491.4311198723,-5639505.938217289,-4385254.243614439,-4553411.720526023,-6749635.28184073,681020.5521676522,-1702630.018100597,-9776094.846424645,-2152000.5125788935,-6177649.443940576,1120279.616601402,-7152299.166286839,-2696837.6479762476,4284113.200432887,-1962046.0049531348,-3441874.799736146,-4282397.588817811,-2230126.493520012,-3076234.7476922013,228604.24725913256,-3369951.416055169,-4627580.958573269,-1078374.7427712493,-2644483.6265377495,-3862038.679909719,-4036532.115362047,-4667688.287548926,-8449424.051340654,-1327759.3185110036,-7472519.911121849,1143484.6545364633,-7559668.930494357,-3229324.830893401,-1275169.5288460255,1253806.4231131766,-6137852.723909138,-3346699.5918997265,-6459036.342051409,-2971805.307674462,-5311167.409556517,-7728310.786624273,-8673980.562709527,-3287098.5034362115,4874244.507142661,-11912907.425414633,-6721429.008164953,-5040151.784907523,2107690.4285054393,-3375707.363735484,-3408649.5206446014,353170.45841452293,-3237384.8125851937,-6343512.150584882,-4468367.72897969,656142.5812002998,-6780783.574536864,-4470347.179141894,-842606.9432250578,3289192.925248336,-5134617.153904324,-4702948.786531856,-4958046.406088496,-6092260.651480421,-6023105.249498853,-7027915.377107946,-5120968.910032571,-6738712.108078437,-1904682.7785120187,-6921832.763232776,-2391033.67911708,-9699892.345276294,-413656.47763432935,-3701646.843268642,-8553613.426953288,-5032460.160886383,-242427.97186980397,-610928.2781797983,-5640316.141709367,-6892163.628124664,-2911294.289720189,-5439991.485398814,-4345424.74096076,-3961482.5816004425,-3493345.460472364,-3263505.995655926,-4643834.501919942,-9235800.360269928,-7455296.785666717,-3308076.114698017,-3909115.8800581396,-1072697.9236071017,-5200572.066034175,-8525885.363360317,-8141041.3011342045,-4098294.3238444943,-890954.1431204733,-2014121.8440525662,-1456691.2762549687,-4087807.867931213,888207.3231888376,-2400868.5044256,-1869413.8860021625,-1678281.9524741825,-2903418.2591133323,-1303565.2004026938,-2231583.950134687,-410221.5097076148,-5502503.0798599385,-9961284.606475368,-9157652.552477801,-4868568.990957933,-7547167.402545121,-3103404.843457192,-3463210.5507098883,-2541816.612799558,-2030144.3651905656,-8319112.35311109,-1184008.5585338604,-2286475.739816839,-5206229.9023887105,-3669921.6816714294,-6036161.756240757,-8093415.1705487,-3913477.710662919,-1169556.8694825843,-628241.9930385258,-3525040.1977791414,176270.26255856082,-5589931.605021354,-1750864.5308007374,-3380703.353994025,-1922276.6122812554,3890488.504213907,-1098571.571354486,-755471.8685108833,-3045428.1980979815,721539.5225719754,-2321642.9912748467,-4172290.4726505987,1439043.3817294668,-5163679.563211283,889828.9333988968,-5652570.815466125,-4459581.674373113,-5619040.973728889,-4618632.700964444,-1984512.6228822507,-786195.0359612554,-4139140.728833463,-4409108.719675263,-3431643.0951785278,-4334402.686840795,419152.1107781548,-3170941.8051000386,-5019763.789296141,-1671619.0262997113,-6451709.6712183,-11466933.094528928,988714.2680527512,-3324416.956213873,-3606976.111061331,-7453807.820480678,70814.0965141505,-4983099.840542372,-7502836.96003704,-2467662.3097813595,-2195861.785073921,-2359179.6553743463,-1706994.5960958656,233687.89412630536,-2958533.0127089303,-1038790.2440425027,4723961.351293592,-6556722.532713592,-8380756.261289988,-10173225.685911844,-7830067.978655283,-896756.7909766678,-2845245.0670030415,-1626931.0651005972,-10251551.638978079,1020743.1417834815,-6508632.279173076,-5453800.694491096,-4111755.3897188026,2057552.6742638769,-9300488.156830218,-3861059.7476530634,-6802634.090507427,-1508697.8685784526,-8032170.122051399,-1919521.2855259571,-1856597.0640017241,-8970885.95148983,-6885319.981258703,-2258093.0313681457,-995666.1388367694,-5821687.368197536,-3129156.251309056,2703842.0605120473,825213.0647884719,-5889346.612990063,-8724017.558186647,-5673705.03609725,-5470541.438984178,-2971503.060946157,-5481177.090664297,-10520938.086493433,-3101791.818326749,-4119148.8257891685,-1249861.5543187018,-9226712.165545752,-8663074.47489318,-7979664.134018492,-837595.68575323,-8940371.827160483,-9917459.226993622,3338931.1980585027,-1527837.0039960183,856099.9576512035,-7967907.704178039,-2692380.495889278,-395968.19879843853,2969437.7148325127,-2043268.9356649686,-477308.7895366829,-3939459.7829148024,7031712.091169706,-4072548.397802258,-7224248.975532306,-3083998.9642763566,-9820152.440065412,-49525.26662180573,-450213.07777874917,-4522616.669099294,-1755300.6637415476,153925.7876491975,-11171977.352600962,1266279.910882363,-2546515.087403156,-6431683.356923336,-2782771.0135388505,-5316475.683762249,-1982321.821751644,-5715404.313650809,-5263600.030067252,313122.02368666604,-5653880.5981429,-6525123.383089663,-1602327.1717861667,19687.4338893611,-522810.8019929528,-785347.7993882746,-4456377.046047401,-6320475.832922624,-10639772.798962362,-8018870.782968827,-306977.640911432,-5596092.569025287,-1568551.824568916,-619696.8794263303,-9626615.926881164,958867.6174003016,-6649843.082687806,-536321.979855353,-832882.7288368046,-7771256.476156596,4357938.3112862,-1274487.2362885308,-7453374.354515465,-449488.40675298683,-5611768.79200252,-2014657.5157623943,-2061629.7957977448,-10631447.643405322,-2836389.6639573593,-8737386.375238573,-5383181.2744531315,-6308876.535721656,1195868.6883116066,2157641.008549433,-12489566.637968743,-1719531.3703426216,-5193491.971330563,3247436.5641651936,-1691846.8921287395,-7877908.669166846,-3234756.5816377345,-6203322.061828718,-4359033.887135122,-5068624.544090126,-5874186.949438272,-4570093.2967202235,-656452.072676193,1098662.9304776583,298312.8772283774,-3795814.2093185578,-7418115.997892089,-719892.4669580199,-9811092.757447582,982239.0464890096,-6525852.013144055,-2291468.1161943898,-3304536.761049062,-5062914.144372316,-5771202.326404167,-12047903.168643106,-8901803.615682347,-7333289.899798313,-1240095.0948330518,2016214.293417694,-713720.7944649346,-9096642.930167504,-9979237.326085526,-3875382.287958311,-3980403.6613317803,-5629020.785086533,386732.9810865652,-1414683.839734815,-2498916.566396665,-3651118.0360703934,-410914.04508459195,-6654943.869443655,-1980042.950852031,-487714.8218445275,711014.9032378327,-1311901.4449825287,-4030870.286312448,-3500382.1717311814,-3692455.134656299,-6565383.892068193,-4162957.0992403384,522721.40221566893,-5754724.58396809,-59372.085496056825,-5096940.303485133,-4440219.5262385495,-1098575.567379959,-2200794.4125598148,-8136994.874376126,-9823500.157802826,5168625.02215247,-2271094.8478259332,-6617945.379650027,-6302264.729400392,827934.6445920989,-1771692.2462842967,-3158984.2884736285,-4583361.933247585,45609.449868261814,-5854342.502779758,1247763.4765223842,-6760977.332925953,-5366736.307293629,159900.67018153705,-9527370.790700784,-1843232.371871233,-4793774.621634744,701253.8867279422,-3869347.985316843,-2153816.6020878395,-1939196.4136296753,-7546375.097137505,-179762.08194433898,-8130866.177210998,-9850348.133390225,-1312409.4065553825,-3784352.784156598,1339011.1420744173,-7466215.234419357,-536417.6586512979,-5086764.085325335,4253109.409074357,-4446911.36717736,-3530629.5689608287,-2493392.428625969,-7060877.54596521,307431.31290302426,-7673817.349818705,-5826812.085907042,2041427.3231528755,1627140.726387035,-3478301.4650690183,-4856692.947819119,-4036813.4618588295,-3285037.0274048364,1113953.2926998753,-5678125.425357906,240834.83062322065,-2243556.5435682107,-1173743.3102199174,1436961.7386967428,-4241982.20789196,-6324553.293988086,-506810.40460979566,-1827309.6389075033,-3912248.7107449006,-8681376.325377245,-812761.9430109803,-5531491.657927031,-5975976.608302865,-6279506.740866322,-4202528.275879892,-859826.5630434081,-2076723.3793412652,-387847.2669103835,-1930623.859460475,-5737338.537617432,-4288382.013670975,-5310811.927544393,-1355991.7259642258,-6841039.343004115,-5512448.791144038,-3904416.877218362,-10734306.178089947,-7941965.828925058,-1963683.6328824786,-1003651.6699578334,-2784328.9231909662,-234356.06613641605,-843545.6820562556,-10099450.423310224,4611228.0324552525,-3213095.4202638473,724202.1173412409,1924635.9104272258,-6491167.766725669,-3627349.7186564077,-5849944.250502907,-7496150.516770365,241449.6510362029,-4473268.033771327,2323071.9352198374,-4410641.528012453,-1192420.5210053343,2386272.820611486,-4999734.447411392,-6352641.700611686,-6333630.535256983,2963884.5428985935,-3622384.1703749653,-3200088.3593374677,-6535537.1898293,-8583221.789809337,-1016061.5041910429,-7130266.750855295,-6715298.46421981,3221213.1851052027,-10445645.452385182,-8078491.774239916,-1470809.6679313127,2346638.6107349973,-5939364.113963714,-5842991.262800159,-969270.5952737797,-10785320.835343195,-260563.34793806076,2851954.094611602,-3515240.038781915,-5057116.465968948,-1732667.7701539248,-3932358.2862868477,-2752835.4377617165,-4312032.620248353,-12809828.012951713,-10315287.235977521,-8184441.201570489,-2747558.1727310605,-6961520.135823542,-9665232.944236267,-1891914.712563552,767362.6497909576,-9511657.51737435,-7877561.750366965,-6785104.431416428,-5205096.176765297,-2470743.282106148,-7780461.733535416,-1503940.309677355,-69708.95653317869,-7780994.969934592,-4133451.0606400855,-969267.6692692302,-728348.5302871466,-8173391.041274369,-4086454.621466005,-7091253.750922648,-4638352.657594031,-3372942.5510567185,-4799003.413272724,-2411015.027034465,-2604133.1744847987,-8428391.031293528,-3596458.9595004264,-6316102.657245068,-12216.258794246241,-5751466.656690415,-4390379.983964181,-5813529.861608077,-6360273.940898612,-4569373.071848294,-7993196.40689491,-4635539.675625818,4750363.195216127,-4031819.8678227905,-3660417.6619532257,-5870380.129694182,-5292669.868323393,-74343.83886350133,-4263234.971309029,-3210514.7936193347,-4494268.2378763985,-6185370.10747624,-8632493.080051076,-8885847.98454345,-10794946.263046943,-5566641.678697511,-2282988.1570494315,-4216792.491109616,4270521.541589582,-2437400.8870379273,-3687238.7952805106,-7375624.8994756155,-1373760.182633398,-1774264.3109680582,-1119379.179832358,-6480891.268783597,-5469539.158864684,-2290598.118229788,-1206665.4116022103,1666371.5421632044,-4122004.0888352096,-6515455.00419284,-3239216.0520645846,-5568546.512114961,-4236022.31391895,-5573956.109764318,1158532.605794143,-6421835.246973069,-1186227.321004497,-6583149.985223937,-1321032.631281346,-5027249.658199191,3369893.021675691,-8584057.821428584,-2460343.9569781087,-5568934.647718035,5725551.73243919,-5957449.165507212,-155862.90431227908,-5051778.731188385,-756233.5865358058,-8746737.05167076,-6074165.858822785,-7919176.762961118,-4662450.895047724,-2540902.102597885,-5293089.216565808,101047.36175749078,-4466025.244668851,3643131.424194403,-6831591.311702408,-10702512.854424644,-4116203.301265871,-5220754.024523502,1052189.2924309298,-6163310.134582501,-4047083.7005017903,288403.181610113,-2009289.6786106927,-3799429.0898057856,-9691333.29032312,-1032505.7016388923,3024237.750720784,-5437836.383648697,-9873332.01922065,1311952.6075504199,-1861593.6654841732,1145813.449724136,3405731.35711783,-40745.30405284464,-5873378.722100848,-3663700.1786145773,-6344993.990860052,-8370196.253253496,-6211388.4205629695,-2271126.898450559,294525.8443287462,-3335714.41654896,2921463.260565592,-3729930.3754687104,909879.3459419031,-6839764.570884768,-2068071.008844832,-1085371.1466626693,-4088954.196812872,-1324472.4187325425,-1189752.272037724,-4469684.286174389,-1539375.9439836107,-510006.7027508728,-13774962.763512073,-5982324.331482607,-752950.7294773329,3114845.365346864,-1071265.761835536,-5436864.43667708,-582316.0411175415,-6468131.980780505,-65329.86462671682,-3920808.1801074557,-3744994.392445592,2848803.4319883585,-5610279.844080823,-2830199.692456037,-5362354.42681253,-6659551.05134305,-3443899.8270855136,-6266406.349454278,-2336969.7165887207,-7715635.714724857,-8024054.620286733,-956456.561270358,-7678797.896118386,-3618034.1216134727,-4960987.615400493,166901.5949543044,-10102871.615946677,-6903384.21234603,1178144.448157562,2626611.2794076763,-3867357.727741478,1083141.0256300475,-55679.817075487226,-4170012.3008661736,-1792997.3283288442,1321911.3778857123,4066640.164750848,-3011348.280347718,5236535.796213968,-8640487.345526168,-5100409.102692898,-1821487.2696563862,-5441596.448582823,-5630057.399920401,-3250512.675601432,-7806721.942075262,-4884697.709371459,-9014557.917628903,3178090.632586943,-3124555.2763749287,-3937922.8776995484,234803.5636474043,-4073130.811843574,-6182152.65140529,-4893298.795243282,-2722506.7843672466,-3763245.4374687057,-1443109.267675478,-7925729.996352069,-3157278.734002728,-5304688.24614531,-7161742.272738667,-3335688.830156859,-689606.7058452964,-7617896.415689172,-4368964.803082794,-392595.2487851288,5287652.703934217,2953674.9068644177,-2154815.55004924,-2293774.9867856,3671966.22013369,-1416535.399088977,256883.5505150538,-740040.1019606218,-6287639.978184359,805583.8875403665,-5743140.992076293,-1737402.8862783425,-2703805.1226577777,608301.9992361125,-3783751.782921249,-3692902.917516742,-6081824.0592237245,-3665391.7446673233,-1253676.4406431224,-6640354.313723234,-3368462.578789782,-3871857.2439652532,-1521956.5856495202,-1518877.0857883897,-2039180.5280358437,-645889.8637420591,-13446765.09946457,-4414939.409847181,-3480762.9981130883,2868013.5130763,-3874232.4478585366,5492708.249798924,2540087.2705641314,-1043127.9522303883,-6279268.541500507,-11007350.397912767,-3979300.3904909547,3408551.4538708106,-2697738.5931972843,-1046176.5293266382,-2569933.0831513274,185047.10780784301,-5721759.382790942,-10171371.945093617,-3870056.4723325316,-5564185.561522635,-1508581.6513448507,-9287807.672382625,-3017156.873740537,-76947.86476427875,-5829253.024801657,-1653082.7591753043,-8469753.722408133,-3889526.8835606277,-1410106.924232673,-1729751.3704742864,-2896697.963438591,-5605440.483414881,-10530463.160623472,-1302473.677338332,-1544515.1567467041,-9804375.43106562,-1907664.9901838973,1534713.44076108,3706593.686804069,-4287679.683768582,-8390081.53913222,-1749521.6722848844,-4768062.527378652,-4692377.359520383,-8104527.059077216,2980473.715764571,-2664606.9112800118,-3121097.479831366,689883.6727743112,-2184671.19383068,-2139601.2464576177,-7760885.760707108,-996449.4913285412,-1645109.4344284683,-7476464.244252927,-5206646.677048221,-2564574.864826532,149666.40716457926,-2920006.4948263094,-2174933.0136359315,-478976.8154970985,-3795281.0931595247,-4303581.021839552,-1755318.7632245626,2963627.670481492,-2059624.027801197,-3105497.357720049,-466584.45408406854,-2505509.120579798,1232748.6047144346,-6021077.602206804,-7662196.086201167,-11157438.79295615,-1330368.6201333292,-2861962.4826421645,-3584445.5385665316,-9528734.467473943,-12648030.513707548,-2137911.8040768653,1216367.5213667043,1417788.2342885174,-5615507.482888181,102171.58825115673,-5378193.592710128,-8909823.750728056,-5408985.468535278,-7542220.248891024,-2964210.8687535822,-2821857.239072453,-5584980.495967701,3486926.6655649245,-3873748.8838927094,-7480501.031661737,-6317338.492551303,731073.5339137334,-55545.222880512476,-27363.284183176234,-3560168.8972776197,-5171314.380635949,-4864465.505980225,859499.6165317614,-5311083.371229203,-8866308.081179198,-3708437.2589184064,-7798986.072458878,-5041705.491032121,-6984609.688731207,-8735916.980432823,-4081219.823469287,-5499881.297310501,695997.7876953762,-7810157.2836803105,-6178549.025073914,-695213.5297834948,2492571.060612928,-1997648.9844873436,-6138721.167611111,-2695232.213009105,-5874129.697403075,-824637.5767037421,-4307181.526334949,-2279149.24383015,-7257295.138066364,-2484820.040140046,579776.5759251043,797288.7275463138,-103000.9952135589,-2567483.0189992096,-2141419.826520119,-1199541.8916573916,466156.9336065855,-3399384.832418954,-2601771.947673794,-6327093.3666251805,-2946455.7833356615,-3467600.105580164,-1017567.1981591862,-2720682.164300952,-1040573.9113805704,-170256.22973925434,-873731.1888880786,-7494479.100466847,-6118063.699772377,-1294180.4471211657,-7511386.148822874,-6304184.67259788,-4443953.07329517,-9914289.513503628,111194.01673116535,1003061.3899988998,-6279202.4098220505,-11297515.747846844,-5018163.89858244,-10351334.552493047,-5083458.686649332,-1072135.7890587188,-8249741.915163288,-1598780.3218321055,4013284.3750737347,-5664442.402627482,-1733699.7628240827,-703667.6848167665,-913811.9401617795,1267046.6753658839,2901802.2203140333,-4140348.4528686004,-2315158.7251596525,-5545057.293179451,-5933146.8923132215,-3488857.977397077,-2454355.4191930387,-5866888.301082781,1247763.8976330105,-2237161.099729724,-9788477.543129718,2223453.2383497003,-5583019.175047157,179869.4537102785,-7691783.995879555,-2415506.6422030833,-1384236.408720471,-7241265.444229793,-8373325.145315716,-7396886.332153464,-3923873.7747022603,-1722912.9866806418,-1706289.7175197396,-5012880.663318245,-1001409.3506820677,253.49910897761583,-628517.9528590161,-1936803.9537267014,-1671429.585210232,-830141.4704238474,126274.74622231908,-3698296.621402111,-7614697.3976052515,-7284001.571345715,-4182807.6959155984,-1708330.1824676935,-8569018.025594862,447244.8749877475,-6210091.553657586,-1805473.0148943253,-3433871.2399596423,-4478933.197239229,679628.1937559675,4933813.863226661,-5644288.067488538,-3919903.6454484034,-2928796.248396637,-1456999.4987261053,-3542920.7963769026,-6146971.684740872,2262573.0880734697,-1373225.6184955873,-3611876.393252015,-5673600.110504756,-3741805.3322950974,-7088703.726582563,-3695191.34732916,-2454666.397307994,947299.0206372142,-3995188.0459657125,-807167.7872116901,-7173827.4182946645,-2665220.301344117,-3622536.1360151526,1405197.358571129,3061166.3776106015,-12469129.500852305,-7078885.446458204,-3189859.409804426,-2775632.4995582104,-3269619.619845191,-3348234.4010360762,-4023998.6194298062,-3398918.8303703275,-5355390.102737319,-5797022.257950606,-1671498.2819871642,-3415908.6997962054,-2345545.814444093,-6014842.891970335,-2219003.4475783464,3367707.52726138,-3221937.283874754,-1729158.7451391388,-13366139.340495884,-9224712.043286184,-3470210.6041437276,91776.31388917938,-5171031.533095058,-5540211.3715012465,-9606257.5561278,-3868670.211373266,-1644860.7986913398,-381453.99670082703,-10423237.08464083,-5929856.038813656,34385.789814012125,1790425.2127704006,1727322.3009710722,-4952831.752583856,-540379.8529491872,-6083061.941777891,-8777609.966289723,-277404.06563477404,-8882546.441577548,-2172223.3846799824,-6108923.196763787,-5473974.851316474,-1858772.0126741342,-6924134.763092432,-9428881.318682559,-5339785.856514525,-5746682.031935275,-2158029.476021461,-1504466.7847313974,-1352140.7316712458,-12134646.401428727,-10565399.253867794,-1087386.705842575,-3593313.264177244,-9359619.404218812,-3111730.0895223822,1087037.2710447628,-4452229.049537441,1636697.3278578129,-9791627.718510557,-2061669.441294415,-4699398.194281684,-750131.7192530185,-1485893.1509461086,-2060059.6051813085,-8909268.97828417,-3231521.1703919806,2666690.9368765224,-1288831.5628217962,-10382945.061547957,-4281527.668636175,-8332715.834541686,-5054794.839886488,-1062975.693426922,-1347155.1688257996,-4298467.435406579,-5689722.229643209,-9335110.283999192,-9306640.757231431,-1847661.110260603,-239008.89751281776,1125031.5539474208,916873.0470989719,-13842061.356199328,-4427042.600721788,-6724140.39211558,-6386570.180909984,563473.5717319511,-2104349.9837142397,-5147601.565658661,-4836055.965480678,-4305570.1591870375,-5847956.60564651,-2979722.1819619928,4135947.996976575,-8638344.411128864,-6112928.312257612,-2324915.949772408,-7511651.1795783825,-10210326.025697272,-8537438.422793223,-1446182.3850251175,2230329.6954251286,-1282182.6249376163,-3113116.2834367324,-4232058.513487736,-1442430.1916769352,-4199875.8402104825,-8134275.333994212,-4303740.994156186,-897506.4857917363,-6558311.429842588,-3718914.722937368,-3699197.199922722,-9314378.44765396,-7714332.770415952,-884660.9540085346,-5008840.75936839,-1060975.0670283698,-9016045.934423994,-586804.9284556899,-2116887.5452977046,-10259919.665687112,-13818029.404607799,-8336549.52643504,-2760064.701798279,-5729837.207184399,-4536013.395451777,1330128.9322649576,-1290780.0047714468,-2055852.1599322706,1252579.109085897,-10214352.411352873,-4692952.534938285,-4591876.515379887,-12969286.876551876,259350.97498759162,-4159236.6808144264,-516064.2814928051,288080.3687097207,-8874366.521885403,-1564739.6573889367,-1024391.8181735426,-1602653.0668714363,-1340528.3241721373,958457.1782717425,1369839.2419308666,-3423741.776364673,-2366017.615843728,-139671.81797099113,-4600978.487722453,-6564088.433377916,-10685742.823357519,-11347615.642031873,-2284810.0115697086,-1723509.2935980577,-2831209.7116116285,-2721601.2877518367,-7711549.801583933,-7745189.324845264,-3657193.031720184,-11571426.577524949,-885380.9774576686,-2538019.7464139983,-2703506.799830988,-3681968.365639372,3836493.490033254,-4911104.532594131,1429403.337101629,-4487212.792185215,-7339168.198183989,-1624108.1533926,1485445.6147764586,678157.326821059,-2736229.365457123,-5177952.912831763,-9072296.523550965,-7176153.021959847,-13519363.272424862,-1263510.672449438,-1443809.4879582804,-2303470.465685526,-78803.6880088374,301407.62843868136,2257787.1478053164,-5263657.482049532,-2113100.393574776,653340.3062550351,-1526105.5595345385,-6037933.398009092,-2332430.299438662,-7943950.747396525,-4554593.237155376,-2447215.043927772,-13047641.338642487,2375542.5354284085,-4661668.130642032,-5208329.453987217,-11489538.307431122,-2983338.7348177712,-4063117.8808124196,-7248781.083454782,-4099539.733595604,-206165.41470966116,7331.98824935779,-7177658.164233504,-604478.8810446691,-3773994.120879559,-9006038.551991934,-2333149.854517298,-4389221.643809,-825253.7006152626,-3180489.2642432526,-3273968.5581458136,-11490854.895798951,-1471220.3467784561,-2833938.9517421536,-1959024.4752151184,1423725.3153237477,-2564587.78198646,8483.04846999608,-57959.20216562599,-4872607.154929174,1217263.1441921704,1035523.3336675689,1840408.4343413562,-2217526.0043730363,-3514691.59309691,-2959400.4023388103,-2695848.921549592,-857294.604569478,-956154.9303799123,3028691.005139243,-10082252.970076857,-5507940.27260596,3402892.8262671605,-1834179.1237083655,-6756803.029654518,-5553896.417619668,-429618.9986309763,904673.2604303211,-9743963.418371819,-2357716.13849004,-4076131.626627581,-1847953.1819326784,-6626206.0280709565,-4990309.037397632,-1881186.5379797556,-8614857.283658613,-518766.1999886669,-1119386.806040164,-6692706.175722593,-7863834.435559757,-7377410.917754404,-79355.36186871491,-6248110.803008348,-4700223.818336394,2808148.1738711894,-849123.1594444066,-1218105.4459679201,-7021584.082966125,2660215.8396679126,-5329648.8213069625,-1327160.7058638297,254349.77723045833,-3368683.4145466853,-4675794.026591584,-1648535.893201951,-1067914.7982092574,-7353712.433583071,-2720325.3093814068,-243504.33916291967,-3629999.964681994,1216418.779725952,-581228.1251003444,-271664.57290159166,-6840899.614490418,-357169.77519333735,-8591520.115805201,-5227750.3187999,739899.7651863601,3098651.890810786,-6857159.132080457,-3011473.4845914934,-10457391.694048114,-1988611.6734202858,-6854371.4420944955,-8998550.17682583,-3800015.6903307503,-4404564.048555421,335903.14183331095,-4703946.001539795,-8566772.329264566,-4434972.231485561,-5031727.972214222,-2935247.0874135625,214564.36876276694,-5053602.516565114,-423156.84323686175,-1900085.28658422,2710374.794652058,-2947402.5295063686,2841940.0110635217,-1362908.7666637264,-3714395.9384507947,1525922.005835304,-503408.280394556,-4780546.202461287,-8358124.596484259,-4025038.1346843503,-2761940.545442637,-5795550.709974065,-3733426.420106804,-3183221.9689249806,1565747.1552181393,-3532445.795080526,-4436887.512131877,1517188.5016396586,-11211118.135025065,-1909945.394464843,-6896627.519549239,-444764.8459381573,-1270935.5683193058,-8387643.368403051,-6153629.494314941,-7031181.815489687,1670487.1932241283,-538657.9393192492,-1363204.0304687694,-2343079.406220369,-4752572.127445929,-3682718.9127946757,-8730872.734992385,-6190849.522675402,-8174002.470555586,-3882938.0955544207,-7846430.940606756,-8987255.051550807,-7257820.660276419,-3547544.4339303635,-317823.48566950206,-1581551.270919092,-4061183.6491693165,-5803188.300823668,-3480903.0533765834,-1016187.4380662944,-8661444.646686729,-4279776.638257045,-6715596.4940133495,-8902762.244030211,-4796537.841897992,-9112817.418381413,-4410054.0462617595,-7917465.4658558,-714363.5382693261,-10363555.605138993,-2211836.2257155534,-6810110.553031795,-8442853.16489943,-739.44354554452,-10828259.669399142,-4542455.67771058,-11378892.229539752,-5338924.540980365,-2043259.034567915,-6134244.756930834,-8853122.01645698,314124.0212211888,-5035294.835608188,-6380573.398223996,-5501682.020106429,-246825.4010070078,-4711847.054185141,-2651159.1564556994,753954.7450724356,-11290293.583244985,-7255417.672931824,-3835829.6206408944,-3127320.0051491056,6168031.384472158,-602750.0436497591,-6599921.896820957,-7252061.411402738,-5176456.237464454,-13281410.621282868,-1182218.9420662783,-38575.51785828732,-5063316.073633362,-2124610.1478527524,-1073039.418650752,-2765290.9860252254,-1983249.9683939014,-591914.8576038331,-2489992.2209244687,-2326861.2294477914,-5249435.070371889,-1985777.4627877623,-10423836.752007287,-824887.3083212283,-6468825.1895711925,-452429.1403192505,-8830762.439959954,-2220058.9892889354,-6277118.439662351,-6047029.984940514,-3893265.397590302,-1891290.5841748817,-2324812.597523369,-1859750.8564635143,-5064211.533020249,-2595479.895414822,-7519278.004978764,-975443.234884182,-7483333.278609894,-1541052.4407327315,-4894883.346259201,-7567392.135532189,-3477909.425180953,1188649.3817687668,-3757442.5809416575,-2208194.0672385786,-2703100.18775562,-5875547.869100528,-5593237.472812081,-3713783.5638123564,525081.1601375286,-3174498.5499740504,626139.5340185147,-8675950.158681713,-4135937.844980685,-928088.2194428239,1163466.650916472,-5839268.750469709,-4200786.256229732,3498595.5445778873,-8051575.210158592,-6308795.433183547,-2104898.8268129826,1129507.7546422724,-5658894.073227787,-5979113.800511228,-2461257.6592071075,-6907244.80123182,-1177798.606542347,4029530.3379147016,-2690430.8814991303,3281593.656553045,-3392986.9547602553,-7567329.503956309,-9116599.88811127,-8824769.908495482,-3032520.892463168,1687507.1405557971,31528.982744764537,-10284740.393497635,-1666072.4808099363,-2715111.5366745368,-4803158.156362953,2318879.70256334,-4460957.600605536,-2241173.1264295112,1752025.8431311008,-2034069.8032493256,-4541236.967971375,-3055199.0095964633,-8601693.550735863,-4217540.946212359,-3978738.390713813,-3389241.7455721796,-1235548.2404136322,-820825.969132835,-4721377.029504275,2077692.2592133265,-7199676.80165463,-3151437.752476452,688806.4539060667,-5291446.109136909,-9240117.730776723,-5634707.125718655,5011250.318165116,-7018780.777807957,-241447.4422106091,-146507.01961453632,-11151955.019045334,-6491316.358393071,-5569811.990610223,2628670.601431165,-1289689.807516301,-4977110.166117622,-6231474.812219286,-565250.6171381418,-8184671.614770224,-6701597.281968676,-3393222.3817551266,-11468612.073902618,-2872920.912766846,-3911442.0264565963,-5474245.17377642,-6789690.292913068,-1504081.9425805416,2256373.587998472,-5203267.357333066,-3214844.465227559,-2377323.8790821377,-10554773.392926838,-488208.4728886243,-6868004.538248939,-3725162.2763738986,-5695647.502213728,4364479.557043044,-4098762.40591811,-4093308.0760314856,-8694865.182751812,-6653122.8887471445,-2323437.6684819646,-5956284.120008256,-6868515.235075058,-2403342.1002655756,-466561.1345419828,-4279925.806231832,-1115981.900759384,1514827.662676189,1579741.036955282,-2615.1149481851608,-812213.680020785,-4639853.908709452,-6410672.839443072,-4605170.120480467,511988.0507670697,-7518966.444036836,-9357376.646948095,-1795475.895691175,2071477.2990479935,-503433.06843806244,-3375453.3651458845,-10312314.148291359,-6579633.365889708,-164524.6242833659,983385.7096282449,-5336680.807192346,-10176992.674109092,-8000445.847778445,-5662789.458601637,1459078.7965891603,-1898887.899759816,-1919925.087540593,229606.72857794724,431690.0587326754,-4392827.743479693,-1249228.2075436935,-627901.456015002,-7697965.189452433,-1550507.4224176127,-1748424.815622529,-11046673.319458025,-5736359.266736124,-2331496.9283928685,-3658852.29538507,-1544083.1361230677,-2281178.4324710555,-2674426.4614309436,-10827055.321679186,-1347400.2313621268,-7944130.059983747,-1963671.4118067771,-873317.5539588798,-7522441.683920784,-333938.4913444426,-6618553.200162206,-4908021.292412778,-955001.6024442054,-4952268.958224449,-5633891.496368777,1041853.7346118763,-2270287.521520216,-815762.9548671953,-8235857.54239282,-3834132.6843049023,-40655.55640571937,-3368021.8676530086,-4306949.856991239,-2620566.070924524,-7975163.058930766,-5994899.961239513,-3192070.253924012,-2345831.906471228,-5516002.893894596,-8767392.528957238,1016549.1588747073,-6513681.302959906,-6497495.586767178,-2600961.5180472285,-8819737.17519491,-43922.743346848525,-5036010.911269713,-5864716.417338558,1438049.7717401478,-6237473.654651258,-6836830.904088372,-1633721.0349139813,-5639449.707083402,-1253537.8208446745,-2045415.818137642,-5032339.54009646,-3926083.8720494322,-5347872.736169023,-8186608.504590537,-9118714.492585981,-8911681.627248274,-5281238.450395873,-4284356.761496071,-3437131.056781858,-517866.3766505737,-171946.8792253267,-9302873.053908132,-5800275.099756911,-3414387.6747035217,522125.58352580667,-5331064.847140471,-2952078.109363368,-6583084.071053166,-8244725.78273784,-9843530.386537928,-6759494.912742173,-5460189.484310007,-6880846.673443213,-5132741.891310798,-910488.2210737802,1750896.362692954,1189699.68863957,-471580.70923011936,-12549039.664312575,-7180478.705813383,-9593700.46324917,1940794.201697925,-8763265.529076103,2254096.6504409835,-5099561.04505112,1273378.8301215526,-11353016.1374566,-5408374.587774232,-2802325.831626145,3471931.837187821,-9132886.625932122,-516789.2271050159,-3150285.744340038,-5228218.875252558,-3909936.8682491425,-1588921.6588927843,-3167936.699350603,-6263707.056960572,2557507.539915232,-9842721.12976499,-3351532.632691048,-5362713.482612623,-5015724.247824792,-6125610.781584956,-913945.6035547797,-6669558.052633157,3418249.6916577835,-721890.1411885675,-9761113.756741926,-2015788.9419276137,-2494705.060751211,-229625.71981505118,-288874.03206985444,734603.1702784952,-2672729.55596569,947747.0845800005,771390.2822484961,-6729385.029216036,-1265462.0461952742,-3064191.226168845,-5420786.3391038105,-1955078.2754096314,-1691741.2818461992,-1946038.137852436,675687.1526905578,-317345.77509186603,-3622706.602312509,489466.1666558776,-110362.65679456107,-9424520.000009378,485585.19979056716,-6577226.784310937,-8000921.802906318,-545836.2969290633,-8756328.333801102,-4858142.432709241,-8017572.573047683,-859566.6529534608,-7239902.491842106,-5170870.25034982,-3454033.2081915718,-10519039.316551015,-9290719.329091562,-7193315.576049763,1318215.474434033,-5922236.913365798,-5938039.526980657,-9786391.959500087,-610490.0526549816,-2184174.982833801,1650434.1371211614,-1170150.5003519058,-3101648.8895526957,-3771014.7917501386,-3230616.967879016,-5225882.974359166,-7905888.1572348345,-5917081.918557687,-1585207.0924098212,-5166578.322849911,-1758940.9275472155,-7967037.202019095,-3104094.0080358554,1282836.9825794045,-3405571.9229213297,-669990.7527044546,-4126723.4152473807,-1813102.8370041642,-4600764.100315124,-1037960.2758000139,685555.3575432263,3121184.5666770265,-2384957.462698307,-4450046.537703395,5753140.544913664,-3539317.0438670665,-4549252.210748691,2044900.9461911954,4328189.327424135,-6569118.122935435,-1751175.3560575526,-7431074.86444645,-9612430.664379567,-8117092.891179191,-6319260.735908685,-2905818.2810726734,-5192863.30599601,-7605480.299969155,-3202269.195533909,-226686.1530317515,-10891270.754361304,-2254544.379082525,-5514151.510510456,-5952176.779990889,1864471.9745324906,-2470302.492021408,-4594493.717455823,-472307.7834804058,-6594497.622232132,-5318379.410385227,1125248.0903351977,-2386190.0031534396,-993581.4644505549,124151.56017820444,-6751054.504555048,-4977635.445332432,-4469512.906489696,-8174330.283663563,-4706782.07665249,7056227.457954837,-1858664.8664591983,-4324100.194445355,-2804379.673522856,-4467763.618416425,-3789209.837543927,1828681.5629520044,-2332182.361721186,-1413318.9699512552,-6409650.933238696,-6652733.191143749,5510264.660765115,-4878909.311775349,-5242813.643223697,-3122946.848493919,-4787537.776030516,-2992592.8902417384,-5175111.171416072,-4474102.014822422,-6015199.286796397,-12614381.879629489,-5006372.20629405,-6331317.510894302,-2874711.555246616,2355938.455464827,4094279.559114063,-4739665.421990037,-7043726.751141444,-8471219.300841292,-4021816.592745712,-4355512.027874326,-12443113.99118023,-9301187.761541065,-2692850.3422012124,3978892.6275200956,-5446140.489260199,-2918534.0684237853,-1239993.5564472843,-1116225.520747656,-7203662.362618225,-7314188.474781122,-10340604.972846478,-7574151.414408395,-1717441.2008751985,-3799875.188607108,-3080963.601177251,-9481193.997836214,-4638978.973491533,-6804100.172631964,-2822755.7280926555,-4972948.5702116,-2776404.8050457183,-4612219.778738549,908182.7847768255,-1541354.0821397267,-5220760.0141759515,-6690917.700827925,-927169.5615986008,5226395.492002012,-2064273.0470980238,-9943292.558476347,-4922824.073883219,-4840760.145113278,-9494514.339738434,-8713433.406095648,-8177986.269056765,-6115124.697316173,6146854.759896014,1953452.0668904912,948306.2553524375,-4004673.2864420023,4650749.48316386,-5723614.401512912,1201000.1800049134,-2061121.934303606,-3869864.640920745,-3586869.7242813744,809871.9992201272,-2964067.6410685666,-4141361.8504625745,-2696393.4454588145,-5655385.719350813,-477940.479723027,4432287.363299413,-5228494.209949273,-8788123.416524254,-9830217.86632546,-1596168.953843126,-4502194.804912722,915689.4428636245,-3367793.0024475437,-1436371.3960050885,-3713358.5416172184,-1766523.7441118658,-1957507.4480424915,-1631552.5190305784,4450299.982966371,-4288466.35390039,-1231596.0642328188,-7040809.9956943225,-5779319.121823855,1030250.1216435917,958863.2541785389,-2458908.6786444187,-4161949.6348722614,-8284695.168325411,-7212760.9139990285,-4594686.963383568,1177880.430783406,-4010987.2529644426,1442235.615358973,-3098654.318291651,2131514.8149741255,3769949.3216292653,-480324.9951196816,-4825618.640068609,-3736269.441194076,552111.9636080004,-6472096.838279251,-12581544.315755546,-3330888.234438015,-521109.28581523895,-7885706.741459798,-4984042.393771861,-10064583.428526014,-1190567.7940404303,-2087910.1962712146,-3062228.0349108055,-8736486.840592109,-9413395.737625271,-9946332.854479466,-1702977.2185163815,-11360208.286498372,-2087520.6710464135,-2099749.730848307,-2825996.5937148677,-1778549.464802837,-12694196.31944336,-749080.8621402215,-1621840.9862417951,-4201302.865949482,-5526663.200689539,-995102.4592537582,1539460.86195012,-5585488.264635198,1534543.5711265542,-5923921.002980774,-3058075.9698607735,-6412037.595846359,-4118110.4252566844,-4782809.705267474,-8403568.107825581,-11449373.042763956,-6816398.155776819,-2115293.5563395936,-6370012.069679484,5559903.31973879,-1589305.8147793077,-4524195.31586572,-6604599.4227439035,-1714319.9292830564,-2980473.3152662097,-6097777.601743771,-323718.08954902925,-6569757.162628556,-3500735.8531005215,-1892812.3671975005,-1301081.6122801527,-910544.8386154659,-4546218.180008137,-3188707.4595002383,-2435170.067907935,-2895292.88984018,-7982526.859967537,-5079593.128341081,-5521477.938330317,-5934179.0064824,-3664898.049940059,-6763317.044536982,3471755.675095575,-6754361.49235169,2051710.7309750784,-5735882.0783426985,2152503.3871174883,1581343.6737218257,298222.77996886894,-4761296.0855841525,-7261912.970931791,163685.1046728175,-1957047.179027643,121299.25134298019,-2383593.4053841997,-7924170.330052248,-9888087.056346415,-3143446.791441215,-10583419.583229592,-1456012.7621571422,-2474346.9974061716,-6393456.4813919775,-3497752.9923207443,-1949067.260123616,218056.8623091057,-10484858.327039886,-1375616.5221322328,-6484478.325296063,612680.2849882655,-3889280.3449606895,-9051450.584098496,-339773.0063987449,-9132572.858372286,-6523671.4442934375,-3893053.5993398,-4125140.5412201723,-1643010.7134442609,-1948384.9777225684,-3447534.6321780514,-15872.236969023943,-891041.6346636117,2310982.862342598,-2657656.108647626,-361301.68640210666,-3284439.2639894597,-7518312.999679873,375095.97886717506,-1453161.0288672335,-6458620.006442793,-3873433.0976326093,-3385116.3289747164,-1779311.1428804658,670592.7576910332,-2436042.434236087,-8117597.02410426,-1987147.3835457414,-447366.1497171018,-5763839.079193318,-1917942.624083938,-5380148.292567842,-3279343.243782101,-398461.8859239854,-4301017.868151564,672843.4608980604,-1756016.8345921151,690939.5061097108,-8568662.333196161,-11538920.385658864,-2787028.321104641,-1426667.6421616543,-1894476.3392924108,-7360824.033246145,-2101753.2517899126,-2900087.0426138286,3529628.517956173,-2009045.2386241686,-698285.0015225373,17975.19146693684,-2292494.853845982,-3542952.9845078215,-4076543.8632863164,-1392747.4259789493,353033.9090798814,-4907384.743118685,-9232798.427079175,-6002025.139147211,-4907030.602457349,-2968174.7300712727,-7326829.024508093,-4801979.273023143,-8878748.524351832,-9344709.539681008,-2652764.366758246,-2549821.3102672333,-6005324.549010064,-5136912.281897349,-3434469.5864693616,1122478.2193306275,-3682001.2260495797,-5508670.405441832,-5131797.862840172,-3231250.7811455764,-5975787.517594639,2657181.0637648515,3047759.429902548,-4091536.627558632,-8795454.603425536,-1804516.5545991194,-8366982.608196866,-4587859.055729138,1931753.6837612353,-7207250.170204099,-1942609.2466435097,-7950568.508106746,-7445222.341575847,-6253373.443176968,1359375.8956428394,-2829757.4051632714,1412791.0676790196,-6556280.430686783,-6786485.987248935,-6952040.343963954,-3389866.2680001445,-3544268.6051152907,-6026744.229178386,-5744114.513403481,3778769.7598497774,1385412.9499026388,-3620030.224212019,-2798749.2522884663,-2156538.533704076,-3926776.2713737786,-11611155.856966397,794717.4247753657,-7307561.29156973,-3971801.506898228,-4988729.081735393,-8681642.613149822,-8074248.235387392,-8528991.304911798,-2570462.855503397,-7478718.411931859,-4293265.020549862,-815493.8387810066,115046.99349001423,-2838.3769764453173,-4501319.766808562,-6938511.668775491,-2134106.7050335407,-2812528.8263827413,-4345988.055031531,-2270119.194286423,-6182060.896121845,-8978239.136898372,-1849196.1417417433,-1910975.1819845326,-5772157.574665006,-7429424.230932146,3577410.1933673657,909953.5307927988,810658.066016905,-9599059.545515774,-4756365.326199671,-1159350.2619992215,-5790818.551146012,-550869.0218467806,2935443.929106405,912098.3122141995,-6405233.916080387,-1528747.480379466,-5813319.006122217,958638.8506198991,-1305297.8575862776,-2241475.87550148,195838.1875605546,-4804368.632153366,-1719460.0675996263,-4413657.8585009305,461337.4125172887,513435.5666958019,1478506.8830139209,-3139525.1435646005,-4871579.829215676,-3199963.006993355,153270.20440054312,-2733237.478955576,1405996.9588408694,-4810535.980959421,-1713494.1801826106,-6892795.949345086,-6248310.488748411,-264083.71014178544,-1073591.2211504728,-4642677.102303453,-1938868.312722193,-6480372.368331131,-6073864.414105905,2658314.993438082,-5802662.223859788,-4533050.233312266,-3431989.562589042,-6747342.241064947,-5032363.709071944,-2745608.821694508,-8382208.347181156,-8724620.402551193,506947.6540745478,2491194.1163238212,-6120580.451985555,555143.4205489848,1017291.5960940626,-6057457.791985109,-31566.14925646037,-1393263.3469283842,356616.5185407996,-2854225.323496133,-3690144.644316919,-6069905.898316836,-5446323.670549812,398575.180331165,-14012277.134419898,-6499329.90032763,122673.20498575829,-5241577.84637569,1943875.7939235903,-2568772.6863082275,-299065.71191367693,-8931108.268552005,-4880454.318607781,-5690060.729280384,-5185460.778700976,-6539376.407568305,3611436.109328417,-3659569.011403351,-10323112.686927872,-5878718.126438776,-7838381.96800508,-945017.5350284297,1872260.3998621088,-174506.3162593916,-347953.10771097057,-8494720.616984604,-5224365.216572957,-4515347.676388638,3022629.6538557857,-2090313.8478056435,-3319002.7048917096,-2715190.678597722,-2393778.130731873,-5874371.762563128,-971613.2262851745,-4803795.768668829,-5896161.772703212,-1331282.6231693756,-1390053.0405957978,-6793814.038062919,-7206020.055259234,1638727.1459183022,-1684132.3668488134,-2030015.1581373364,-3867051.890660436,-270970.93397301435,-7581555.045240665,658486.772733828,-3539586.973637527,-6432848.557623733,-1236601.6545492709,4107.709631564096,-8942442.354985107,-4179673.8389232326,-2794881.1489303336,606676.8926504087,-3003875.413358208,372270.58890966885,-7331047.062987402,784168.1653356906,-5837777.46757465,-2800290.1452396447,2126661.5915412568,-3018460.8324935865,-110183.85843096487,-6765469.841052068,-7111325.897934638,-6289815.964689201,896581.0197037924,-5094885.391847897,-4489265.746210379,-4040936.0118241236,2227122.1580742057,-1597786.280862704,4306919.086679542,-3672272.6645685714,2782224.5271805786,-396193.53273979947,-967662.5909488928,-2988777.181305183,-4940981.062218251,-418275.061887363,-4936763.74622157,-5205983.483493704,-3100381.2036801334,-1974147.7092954516,1556563.7317266688,-5673034.043442596,1745546.9545113035,-7925855.312259907,-4642952.578561293,-4359805.298210971,-6435495.0263881385,-7017755.71885813,-1616663.4977088086,1405835.111266654,-468429.9477825407,-11388917.044830251,-10891687.170205351,-7563810.679476239,-2076909.014387412,-9018199.401253004,254540.86044992693,862678.3715809826,-1852894.5398641936,-3349772.8812756147,4128023.8295111684,-2480598.2052099966,-3615378.6926893033,576406.0481133293,-7337894.244005669,-805557.4577252045,-2468347.8088907655,-4046118.7161275037,2534979.5469383504,-1855733.4096166883,-8058677.194878023,-4425379.997755416,-629780.5705885049,-1013805.2935900576,-5032559.926578602,-761405.2664159909,-3768187.022525342,530745.1309884433,-6802712.091377573,4390242.354344219,934178.294654876,-4393188.646371394,-7403121.827923574,-7104666.105100435,-2999977.555257585,-157629.59191972762,-3606866.319107961,-10486740.325983975,-3524569.408151094,-5490069.229105936,-7494584.151610959,-2221282.848396376,-5201355.065966185,1071663.8338514194,1680549.9002377037,-5179325.631379737,-3835427.381784236,-8186040.3181570545,-1006407.4867059812,-4845254.802928947,1839750.2506853193,-2084868.9567735698,-8809109.140934322,2838043.672338468,1243370.0466514137,-1860661.746022325,-2241532.372652922,-8248452.664359994,-7194391.109452466,-8443641.656316167,-1935057.4883444346,-2611326.9121262077,-891991.5152606294,-1766488.2837156989,-2461692.3456131183,-4882025.928897299,-5096168.99217711,-3969937.1130246036,-5886184.092741899,-1807753.7439190913,2085434.0893020816,-1804548.9811498355,-67246.35597504117,-6627611.867892906,-4770599.378553083,-2705345.049860606,-2746965.086625945,-4956817.917673957,-81330.29680848494,-841776.3213709388,-2246072.5423980877,-5597461.236778464,-1670307.6955868863,-3974338.9444869105,-429710.353620179,-7331706.291274138,-12665535.590237994,-6216059.3720849715,1377105.736078998,-4425048.277439125,-7419041.146243496,-5671628.578520102,-4681.383126066998,-6045337.0116854105,-1172084.3125325497,-4259583.607169885,-4671454.387041917,-4509208.352045905,848681.1867687423,-1658332.6384533811,-8192809.05954664,-316775.98140426166,3717570.8411881197,-2601833.0802224427,-10516957.159189982,-8515956.742727825,-9061844.739022378,-1486983.5838501193,-1428787.7755595855,-6942994.213415405,-6756490.114599358,-6597479.558557356,-894788.2687630579,857856.1696878709,-5529550.476501247,-3312654.419745138,-1457482.8274062537,-7022373.716120016,-4228607.827611703,-6267926.856445657,-2901826.1172569096,-5921473.646738099,-6197304.165452447,-4871051.590115204,-2379086.2917863764,-5808855.142622514,3483782.180279603,-9732880.513705308,5325518.402780846,-3603290.0513640977,-4737348.809223851,-8703679.627986098,1840260.9185552746,-3536540.8562759515,-4664269.734672784,4814852.969633818,1188138.3437127508,-439201.12299723364,-1052283.403820267,-7371876.55368544,-1291453.238813443,-10194829.292297695,-13877421.438166315,-5950929.21481364,-1076495.3671048712,-2548281.697161289,-8326691.800389258,-3853013.246030107,-4623578.895668119,-12287527.439179173,-4102889.3369573373,2448775.905787699,-3702508.1172836367,-2908583.829714369,-6051405.76391894,-5059944.512299063,-2370271.10169884,-3535092.1585773956,155569.31516226754,-6659518.713026538,2568047.9460310824,-92943.52049270272,1055840.978247162,1048046.652672533,-14451915.983132174,-13210104.917518377,1547700.4134095274,-2632295.3437701613,-2223315.24478022,438588.2091528103,3087891.445254121,-4928966.473793963,-1818391.9534294866,-7149798.2564733345,-1484046.4141879939,648619.8138851374,-6478418.863573536,2304964.929274697,-5134786.803446766,-5663850.360471442,1770414.540041646,-1012109.4893451668,879939.2483229209,-7753810.192694912,-3001207.1574328635,-1531850.8824895471,-6678833.281738384,902263.713057098,-561950.1979697132,-33445.21032372676,-6773906.539217824,-1959874.4846140575,-6659092.338241762,-3040053.650100071,-7753514.1609995635,-806543.0796600338,1144262.5640640426,527823.631209882,1877183.0055668242,629368.0416554473,-3460497.6801198833,-3539126.2727171965,-1454687.091767013,-735876.3101791777,-1941333.9147813767,-4641950.930085832,-8890183.47186677,-3733795.4363951944,-2394080.6362274177,-8384876.572959617,-4765370.93633968,-1375850.4128897395,-8981987.405873017,-9277679.430524683,-7891523.830645108,-1843106.2336154021,-7154888.100624076,-2927595.5526096504,-12114730.773395827,1095843.4910813365,-10240916.03478117,-5988400.0425927155,-6617758.2284043655,-5434877.54516975,-10082975.82439892,-6682227.954286665,-2338096.9341754913,-4508950.882190093,-1867441.3526142575,-8712137.143387914,-154175.74186080322,-6544526.834324352,-9964294.058887495,-6710724.399054864,-9918136.03162192,-1392088.20498172,-2213318.73093256,-1980596.0199462622,-3427680.467441583,-5429132.470448691,-4329028.041289214,-7922408.918205366,-2735343.2655123323,-1373618.40906379,-5819760.469443772,-768115.3876475394,-5897358.569262072,-4930445.764245523,-5267342.25083681,-2219432.329614999,-2218271.749787761,-7043736.259193869,-147909.7406279482,-370790.29880601354,-803795.0151919313,-1164411.8829935975,286659.7383696381,-8829517.387359474,319645.05185025744,-5292272.84455814,-8883357.12105122,-7494031.997759981,-6405153.673496088,-5070619.091549519,-976566.8007364515,-6708196.829439574,-2102936.6801854614,-5559077.542079637,-4808354.059786169,-7110506.866884358,4216444.091553999,-7361862.6268271245,-5993546.688548364,-4044196.958206227,-4091619.781197017,-6376771.088978637,-7969285.711506145,-6927255.476023873,2217000.6677489225,-6678880.929941893,-1276299.2450463604,-3511899.6776178963,-6547123.774145465,-5706442.892752761,-1748926.0099453386,-83286.8579242006,-1556881.0040738806,-5454505.1352689415,-3161859.349735031,-4494499.591334576,-2534694.3152243234,-9224763.192420974,-3645781.344433449,-3371011.481190875,797487.5539655313,-5262528.31791063,-6873566.3184371535,-4170176.501156602,-3761020.7307085283,-4424513.119916057,-3849363.2094864734,491539.72975822724,-538836.3993439134,-3819525.7875086013,-6162930.627841106,2407486.9784241263,-264883.0396324247,599525.1549918652,-5105708.940898988,-4818386.494610649,-3091722.974018609,-2341466.757068839,306076.2650313545,-7867452.647614725,-2634413.8682800457,-4835285.160685472,-2611934.3172586057,-5767004.811500179,-2178175.1716186963,-5404733.131438723,-1043319.3527380619,-2618309.9891717937,667122.9984661173,-1875450.0718142204,-749848.097744545,-911364.8225986585,2001782.9452938903,-8963080.237642726,-2344298.099261556,-8883320.801380288,-4352933.949618157,-10201674.36249121,-3849698.9489096384,-138358.5265388023,-10698946.25961436,-4377836.550189282,-2209312.5103746224,-4290836.333326278,-707620.9918941092,2307667.8487244286,-13897760.99356958,-3645959.129366737,-3404167.751525456,-4110173.224476734,-1722408.1975532211,-5905216.300277945,-5049100.051049698,-864858.9549356718,647671.6317198481,-8340587.879913056,-5046729.865735495,-794098.6616733056,-1387363.4056440983,-358100.5975179598,-1919518.5914418306,-5943109.602407679,-8611975.25874447,2245847.149426708,-4121393.859747492,-8743068.78198551,-2822520.488553405,-3819239.991027586,-3630968.799523376,-2082751.3601192273,-2504752.604628442,-7929202.313573597,-10236114.422018727,-5808605.966208024,-1665304.1467504464,-5173471.848722752,-2760529.2468468472,-9553465.27375646,-567020.867321251,-5834223.971084245,2613660.666221088,-4123662.5009744465,-9888307.67841852,3008644.358593207,-2475566.818782619,-806682.1445848085,-4075837.158269502,-3055355.6466239914,-3879021.9210114386,-2426705.7043288946,126981.21218374744,-96037.97511395998,-4618331.212100685,-1665241.91079613,-9207179.246402025,676420.580480529,646594.3489785604,-3904884.894320203,-8574045.743734958,123444.97880957648,-5864428.730884541,-4691096.912367643,-4747222.039368432,-5948491.513745507,-9191372.84177875,-3309292.3473512214,-1825906.5032229535,-1586320.165206151,-6824470.401984908,-378052.7993174214,-5329475.6166452505,-4898448.207313164,-2308353.710203849,-391325.02423964534,-1219889.424848929,-3263371.2300101835,-3598758.1474011187,-4409255.646378282,-6181305.4171537645,-10323213.191008097,-2963504.7089147437,1646102.7984892353,-5497440.683039557,-6638142.623281449,-9007208.591994822,-1562851.1777391694,-2272481.3382409047,-2362632.389208287,-3140717.7854547035,19057.917707053944,-2078540.623966314,1435389.585189052,-3730842.9252425395,-1967111.3987971079,-1967311.0656886213,-3966107.439038584,-3167253.2294740435,3706167.484862324,609197.6702444274,-960638.2081674524,-6519538.171941832,-7392891.963306002,1356672.6316566635,-6388255.477879973,-3653982.566304978,-2512698.3734463025,-1479369.6540778354,-7526073.973559964,-1663091.9285495635,-607974.5806705765,-3382900.3491502535,1658501.4927946534,-6635964.315478969,-2339588.5374312215,-8467639.872163264,-7286157.098105824,-7607191.296191974,-4862774.238160845,-8398728.07418388,-8542631.053177824,-4560799.143027272,-735398.1550170183,-3991250.891411165,-4389076.910862917,-4353061.243527144,-6003951.0628982205,1331080.4772890564,533089.0292279851,-6720379.123670161,-1419219.6900012046,1707109.750339076,-1086049.1280120239,3185663.3271309473,1560412.4352868292,-2271451.0899443515,355190.37537286617,-10070899.300487049,-8679948.670650676,-1470696.9281677231,290765.7509916704,-4675864.119365262,-1252099.5813998915,-5622340.566240888,4079117.566896377,-1552440.1791706383,-137540.12996249646,-9649502.262853054,-7867776.294733003,-1874029.2887481786,-1762726.952813981,-8801713.305593122,1277919.0470145084,723716.365351174,-6388615.5333671905,-3418197.533308074,-7346384.439285483,1464195.4735606238,-4671355.953379845,-2890269.468312336,-1293434.8981722407,-8450939.57167105,-3756987.9684338085,-9816170.006919328,-6202970.704371665,-8705397.032664247,22707.73810807988,-7067358.806164874,-6963126.725335481,-934000.6407103501,-7903565.8458393775,-6680118.018826444,-1920369.0413656067,-1697166.0757375117,-2540795.1006183494,701521.7264256738,-1035872.8802474402,-1539766.3164616656,1803431.2480126899,1406208.2016070057,-4987817.1724507045,-5120668.239988195,-4638765.587364832,1425537.0327229705,-2583018.6275835093,-3857429.727684263,-188754.78101149388,-1410991.3605529573,-8648159.462063272,-2395279.646432711,-5402155.407384565,-4288895.851524863,-6499901.232517807,2509236.1102841906,-5592886.385852556,-4550020.252960138,-6498592.892526632,-2864866.6066079866,-11592775.471134203,-6699949.845416203,-4539592.919040082,-3877055.195528362,-6417958.896888956,-2437238.3730525505,-4075858.357025614,-12816605.454513967,1137269.8410300817,-4171825.090546053,807737.8576328978,-5679682.388419693,-474835.3565691598,-303238.68715037405,4308117.035827074,-6509562.80216473,-7068977.658627914,2034.9602549634874,2269388.2154005,-4688040.342928063,-7578742.359692233,-3107850.2389365397,401089.24447403476,-10601611.093567174,-5208315.760507856,-9521120.75697411,-3998027.1305836374,-168966.9608635474,2432701.041795049,-3448571.7890997417,-735318.7571601141,-4602058.372813996,-2040163.683720518,772544.7349743657,-3569005.9320626697,618960.8255870827,-6187779.982445465,-6922227.967852615,3966439.6062318813,-7250169.052766789,833640.6809589211,-1736372.8753496893,-4072244.5652726647,-6366693.436477726,-4263670.7678429615,-7420982.833310447,-4064619.2216145266,-7113592.011923116,-866316.8475194676,-5508142.515783342,-8276087.896973404,-3869941.615872696,-3246631.0395565573,-3011286.0549089108,-5160523.801226821,2931300.604848819,-894662.8066444863,-3230026.42660058,-7372193.302335933,-1929258.387920184,499769.5653869249,-2065702.5637019947,1606488.9962910283,-11853907.728694046,-5478953.303134399,-3180165.7164705936,-1656818.4625170436,-9017945.169010373,-1979201.3354450986,-2036426.514135033,-1369158.2259554584,-2889073.4240214564,-2160141.3409295585,2010287.8835432213,-7745490.592219368,2160280.9533373564,-3721214.9848694503,-11132112.648915166,-8125120.568889273,-5626808.415981373,-2429280.851251485,-3438863.5188585464,-1325510.2060656697,-1743904.7510145325,-2357790.995961616,1729359.3623622768,-1838702.554966731,1018645.1041531842,-7733642.817682441,-10014514.073472742,-2035301.1933085416,-4534496.519541886,2485634.760548547,-3555425.424276402,-6514452.073342489,-4434015.632313112,-3765757.293464821,-7881185.063632654,-671736.756785566,-2432775.7159185857,-13102529.029573606,-2953824.8056184016,-5523545.898863841,-2655075.6463948227,-3239586.6778237876,-5960913.039807928,-9063059.851633292,-5093241.867291155,-8410331.001559598,-106546.50984241813,-12603369.257027404,-7962217.6362259425,1878591.9383791145,-2125976.918699261,-7905559.594242366,-3060558.3880467787,-6385078.717319261,529369.516641954,-6643849.245674968,-2237616.2321106177,-3170306.2902098,-1580793.5195039194,-2174600.7604744565,-5103906.441827392,-9587082.070867885,276495.2865916025,-5198402.772992674,-1866178.9407434035,-5164397.183602473,-6103140.167942025,-1365126.8783896938,-9277243.090051515,-4059418.161856236,-10413652.43057934,-3990177.382626405,1793625.1716015022,-9143635.392992282,-4896747.767702403,-859862.8266268037,-4539052.539138028,-2756610.0499222875,-7516164.984038239,-2716586.0763068777,597501.8455450032,-7539920.293101351,-7715682.737878699,873537.4939793758,-4296246.202624684,-8839869.694539469,-5695626.002028458,1618263.6138307229,2671864.925924076,-7856546.72111138,-8489800.738388598,-5958035.990134725,-6568605.3940609675,-7640445.344007188,-2830995.647348685,-13452529.070476083,-8933416.351128442,-6475984.887579594,-5370722.611116257,1499452.0709024593,-3719527.3563795052,-1999803.786168037,-4075815.5973048825,-2832087.420229882,-2605282.8381916583,-426384.6543389,-54484.68342786096,-3188618.0331348404,-2954885.7590432055,671455.2978898492,-5274144.954458103,-6621367.744327497,-6739008.9126825575,-2812245.8128934633,-7036245.396363659,-2836751.4125132207,-3852425.2605568133,849594.9040691704,-6517959.508683456,2422932.9644513857,-650783.2563017756,-7442976.842706082,312628.6451729294,-2568114.413943786,-4046931.857804483,1015944.8181821695,-4162708.255493582,-4363703.0771339685,-5676090.386173047,-3197112.468787349,-6494742.893033495,-2259174.562353488,-3664537.485413924,-4599735.576703832,-2269014.38680307,-932926.5668604076,-849923.4706201032,1619738.049862586,-2910919.9835884217,-7844384.032241058,2759345.472349912,-7720697.965906806,-3301227.4830744658,-12749956.202637438,-5330592.026886007,-7345025.38639313,-5629751.378654638,-1947470.7010782175,-3990284.47879616,-2765555.0398037024,147445.62442531623,-6045125.580507092,-8290963.018892739,-8190429.199456686,-700610.6337989699,-11188956.845462061,-2863185.3055177433,-10784583.832843073,-1991076.6691104844,-5790502.491764877,-4486635.295763578,-3784018.175876742,-4165192.2063717954,-4866839.0732061155,-12022530.225478891,-1938411.5154468622,-12253118.206114179,1138769.343291657,-9340283.168553496,2384009.686063409,-2441540.9901406094,4115830.8181621786,-2974279.4772447385,1165848.4261172507,3222753.955841355,-2569114.7917217053,-4613966.218878549,-8481061.558741681,-14036.389731157571,-6970341.109978987,788369.7776334304,-7965059.803067278,-1680545.7851033956,-4024709.945991872,-2495672.208342121,-6633771.977761278,-4463.656613184139,-5820442.970215581,-5398927.009888899,-6233090.379419161,-4628893.750442466,-305439.369874211,-4323521.131069135,1645258.2313778587,-2397626.654791508,-3590719.2105963994,-10143181.470732741,-4897205.849628054,-5431574.273178382,-203862.96962193027,-593661.5143773705,-4609786.020552145,-8962986.10231473,-3287366.1423163284,-3829984.5783013282,-7368309.492951924,-1690400.770256875,-3711082.1654491965,1102946.7469415646,-975614.1115753315,-6824334.989004721,-6927653.056422485,-5222002.402412726,-4839143.0744181,-2570871.472917797,-390734.4576937724,-2006294.6137560029,-7158085.776111882,-4497725.137000961,1153928.6872273497,-4007293.7952923793,-3559536.550165875,-3192407.5730763115,-6198720.855619827,-6933542.969963424,-3410123.378087232,-4672915.876810577,-465931.7384076193,-1353059.4497514274,-5889012.976407045,-8905126.47684523,-901195.1732911151,-9162919.297236033,-8972529.334524963,-8717649.226345498,-1165294.6394156497,-3987713.5281364005,-5819896.917940574,-8513.632853237912,-6517643.675513072,-1516888.0013055103,-3872282.6525560673,-1395355.555299228,-4412324.875289287,-6551186.503330329,-3284764.9042117503,-10908354.064521207,1782975.6941383798,1264170.0728973672,-7920632.571081927,-6539041.326305335,-4418746.261487419,-8254953.682778705,-5720479.0460356865,-1181798.7942540925,-9897133.633267285,-1718571.7788607627,-11287051.476497298,-2889676.031931285,-10102185.906899262,-8394079.06873257,-3792562.3513136096,-3833062.873488905,-10519632.231035946,-972759.2938526738,-11519165.343274526,1297783.159643311,-7603877.7406305,-866237.4966626633,619490.1328369956,609419.0087147951,-261457.8500212934,-9310847.557398608,-12881102.049568646,-8545889.837485444,-7467118.157644227,-7991318.523356847,-530635.2973655909,-1091616.7100545997,-2022004.763501916,-5456035.0155991465,-5080431.814366486,937994.9665585309,-4525714.478762852,-5958454.492329909,1398114.2755256016,-1783412.994998565,-2530961.5043255966,-4134367.2033316717,-1704810.3998205867,-4329708.6570147555,-1642241.2343071103,500149.45648562163,-1908928.4930747002,-2146622.268668955,3724258.62823621,-4077658.0495449845,-3706805.0216368474,2827762.7515359297,-486274.5889665466,-2202201.706016464,887250.1255313698,-1597663.3116884697,1593343.1296102423,-3927184.104531551,943392.1994024236,-7687362.42619236,-3885711.522747569,2077943.0109204073,-8713006.376227885,-5668080.2067254195,-2081963.9395523313,-5190502.896515679,-5744219.77422687,-132882.87486900017,-6063173.13113633,-6204212.240371429,-2498587.0378064513,-1153866.0625220612,-8666256.413399916,685298.7721312027,-2860354.8832179178,-5572426.571195146,-166940.67188581638,-1921132.419210285,-241791.2266102992,-6076159.261778543,-3711943.1319457553,-6346377.99529703,-6104044.057955179,203373.3721323125,-7902980.486686764,260024.09440527298,-1518033.8605708778,1591503.6468530763,-252729.6890261434,-2753860.8988001607,388987.02051456645,-10172330.387593986,-6084029.089987986,-10193856.505852392,-5376899.340066303,-4320226.087367572,-7690868.093906896,-3096370.9981953166,-2611557.6739140097,-6651071.122317512,-6001390.725306295,-189138.1912695989,-4925847.121718207,-1614870.8367331773,-4469872.825005053,-5966866.366079071,1191971.2021909095,-4665922.390292181,-3695265.123062469,-5184887.244115343,-155157.23798627965,-6320195.894289503,-1192235.7133639138,4684165.051327273,-7092507.198350543,-3862693.4114528503,-1135774.2962815203,-11255083.549016021,8302.762887496501,2892298.738388406,-916800.7296329103,-6442850.14560377,-5488709.8010117095,-10773664.786704645,1753737.7629599627,84519.26883055456,4168637.283666417,-665731.7390296906,-5205772.134439994,-2732178.1416593436,-3921003.769509036,-6339500.918011839,-5739071.155267872,2441154.5935786422,-3386028.3473180644,-2612881.8227602,-4886575.109894751,-8988354.014810905,4963271.160595434,-1882898.1590970885,-8110193.20816184,-5927986.539504075,-6721808.2933759615,-2278352.9969980046,-5403884.695778938,-8381966.264709504,-2623038.370391166,-5257799.983245207,194516.38111448847,-4555477.762138404,-919584.9263465889,233587.1785542071,-732041.7003019955,-7979137.763478726,-10361276.136447484,-7486457.095738314,2338178.8592314664,928416.0403535794,-2684895.9209439177,-952858.235698238,1120937.0993401054,-999771.2285046559,-6011524.262470525,-991836.9964328744,-4865452.170022484,194164.2955511473,-2226087.9503969327,-2681666.1520384327,2098279.1856980696,-7721349.933543504,-10625224.073916342,-10184238.740384983,-9395064.545247938,-3561189.754904378,-7697739.4337393865,-10171957.71477947,-3606264.9848321564,-3433891.2294550035,-1215095.1871934123,-11399434.294480706,-1093814.1147479042,370349.0473200325,-9707637.080651985,1987630.0864203405,-1204409.9091377072,2716095.179911418,-3498714.624175233,-4247853.099410184,-1109291.4662446044,-5177492.473484522,-1642200.4704007953,-1190985.2207576763,983198.7614765,-3719860.90168776,-6305251.44767835,-5703978.47773082,-3345158.324288208,-8697097.109478759,-6554420.746727081,32917.11771666072,-6383270.353907462,-6791734.980820181,-2279050.676625401,1057955.8455342408,-6166612.064173466,-2914390.567961648,-9648614.396830194,-4191908.1885516066,-4543066.794656863,-7526994.175570648,-4568237.406490892,802213.0293170456,400428.8024508245,-3814956.111967316,-5136124.883718537,-100684.0087678209,-7161932.644517206,-3121353.7455410715,-8153997.646199778,-2392784.9113915097,-4133922.4715813864,-3236658.5027113147,-6088262.535658821,-6210606.726949146,-7697539.925575787,-905455.3168366794,-6206722.616155796,-2246780.1638648678,-2035749.1031133477,2788528.5009326544,1959362.9481221614,-9627547.792854754,-8462137.879654614,-5836330.7080249805,1685511.2587175742,-2539605.760320777,-11164654.114933472,-1585046.7669183481,1057613.848064296,-812324.3751782048,-2203498.941252373,-4942872.565655317,-1453403.8853917047,-6706159.906722938,-1794168.9570573028,754409.1213641763,-12246455.569587547,-4957554.245995585,-2378195.2821349204,-2936128.579687141,-9108193.924133407,-9118634.44872345,-6712996.031697767,-4878847.246482633,-2095306.3958975878,-6107728.3674984835,-9188857.262539834,-7809145.312939417,-5089635.976763174,-1555610.1989380997,352557.1059870068,-6934140.732987003,-6967129.821030388,-4424777.429425579,-2785760.747312771,-8509084.449151682,-8407582.181878379,-5124753.726111896,2763725.5579032637,-4512336.875403477,672114.8734693769,2496656.652805267,-3059847.2839916516,-6297940.227882091,-1386049.0154213216,2201895.160275772,3898991.710632777,-3517018.054036632,-11662418.496492412,-562743.6120794099,-3705923.482986482,-292630.7116558403,-6210495.364469983,-3444407.039368514,-3557742.473992089,-10072049.368690712,-5212297.291031387,-2579043.3482257463,-2437500.683377711,-4090619.555834774,-3298434.602871198,-5629509.737753645,-4365451.502134839,-645094.1052031685,-2826513.271221433,-1167486.2838587891,708167.3973306818,-4681257.684605446,-3710384.9685826525,-3847434.332817496,-2201444.1208334398,-9057032.556379905,969740.0544023477,-2322671.807653025,-5423690.64077737,-2131272.8441199027,-1855764.4762451053,-6288449.26782739,-6932680.285452038,-1387032.7441889793,-3594764.87778762,-834242.0801003426,-752402.9239348713,-2532849.1276216675,-3228764.7183315046,-9703924.033189483,-3539893.3742126822,-2486530.880081078,-955741.5638127141,-6607503.241311433,1336174.3348015305,432342.3232097104,-4320545.007965891,-5819242.526301216,-100858.44601617381,-5329086.208231538,-2111101.5138407405,-4810579.342990022,3852055.4740587585,-5456364.597573836,1101043.1166587397,-11616328.578799972,1767178.9920368306,932753.0348985158,-2512968.94255537,-3008864.8146304395,-231126.02646944486,-5385248.557529511,-10641611.244554384,-11521039.017825793,-9179315.05743062,-12740576.574147124,-4110563.8009552844,-4838500.693201881,-3722586.5157900155,-2309202.4943246692,-2168317.554486554,-3264321.3379047643,-3930743.1025110073,-9805798.15719674,-9696660.341077518,-6205676.829206197,-4302122.418226052,-5673307.551857721,-1017064.412771672,391354.99618698284,-10187983.369973985,-7485710.925426552,-7380373.20614544,-7827984.887400763,-2184296.688750252,-1544335.3505484816,-2286206.790774651,-4452774.75593592,-2670916.205082299,-2060502.3467609994,-3069353.3068863396,541774.4970185459,-6825851.358714474,191711.05975265894,-1863908.1965137497,-3538359.295189025,-4455655.352095611,-1534577.878735492,-4396053.511908155,-11061898.959404713,-3494070.7128279824,-3058368.9187602773,-4286780.35083,-125005.96361437254,-2862128.130954301,-4575604.653247099,-7675093.231105136,-4600978.360170439,-2845655.5931356754,-5893456.716896048,-2708075.3336525243,-10725538.419183586,-3756579.295115931,1262792.0984889362,1446215.3901636712,-3543418.13945705,-3654875.4916324466,-2198577.2726614233,-3855153.4247873686,942263.6294188909,-7254347.359787812,-3015414.5503286216,-6760377.540746773,-155331.33026000485,-5785364.895489126,-1728526.9074147455,-5850877.389114469,-276592.6728930045,-6088647.879227063,-5456196.314418456,-7738159.952609945,-7554574.333612416,-8682325.565562949,-2987121.83185756,-1698254.6340884063,1317368.5671843179,-8881070.806267664,-2488514.9757920243,-2173696.7575519495,-2495493.4972078744,-2313054.7782151904,-5965048.237224404,-3529784.786735516,-3690975.0067948475,-634149.4905410074,-6570994.764333108,-6572379.955696797,2675717.954147514,-4949668.391800728,1655735.040502822,1810734.505686069,-2224189.912527876,-5224105.597306585,-3782551.143926529,-4167549.847990254,-2466259.0627786275,-3438707.3576124124,1817084.3955585454,1273827.5808592662,-4563913.911334703,-147226.63647673465,1699165.1372444443,-5855976.826146705,-10562846.07667558,1917462.9164585397,-3831530.5604371093,-5500798.842714746,-4393426.903033324,-7485226.120971842,2785926.4762028083,-7792108.8524244595,-12747227.818026181,-5529175.69128296,-1914905.5252294987,-2734025.1441118196,-1572854.0756442975,-748016.1368800141,2535443.143122278,-2521430.4689165093,-1939083.4382639397,-3721075.113026157,-6041811.314937836,-550930.0727462657,-3151433.203546213,-6853422.16325273,567728.0826566741,1609678.151730556,-7634859.261575018,1776110.230111014,-4863204.626869375,-11243756.029222913,1300546.4499482457,-3106280.9732974917,-3593024.4314602353,1758198.9334573466,67171.48955114186,1377505.3248133864,2618968.399089696,-7731419.542754091,1060250.8885152992,-3769011.171774145,-3956091.0588903073,-1897003.503132809,-5195414.197963761,1479026.5161248632,3067250.6415832117,-9966454.3868099,-876359.3933015615,-10158642.78306918,-1307361.4858157504,-99823.9556377437,-2430631.8341749255,-1223334.0109161902,4493652.269749559,-802321.975572262,-2053243.6394441482,-949271.272407623,-6009489.411992012,-4757147.21764281,-7679471.662284095,-1968412.1643247325,-9339405.358270489,-6222962.865890315,-1828882.6456565764,-2588916.9679051675,-4035922.230090442,-3570045.211397717,-4572557.928849615,-6693361.431107458,-5036530.45730325,1896.0588581748307,-6909867.4209735375,-4681426.771253798,-2634919.241814155,-3783287.3257940747,-800540.608301105,-1917007.597238101,-2404188.5521573164,-3274411.0848080516,-6368278.502119992,-4574616.14370049,-5797274.209618695,-3051681.056166524,3559997.394000683,-4797859.396205816,-2127078.8775423784,-6352563.045450006,1887519.0601938684,-1609670.4347534645,-1951701.1352945622,-3577064.994993046,-5075923.181349525,-3547657.4168219008,-9966817.922603877,3217913.635819437,1717372.660522772,192583.06630720198,-2884960.130416956,-918130.9351784065,-1688784.10942382,-539507.5343591981,-10203363.59374308,-2557194.3630585223,-5701859.662084628,-5122397.05865767,-1348959.0872631744,-4278997.494608931,1783781.5095764734,-9103250.906704647,-4392608.599218786,-1976543.013339661,501043.67303313315,-8095698.421604884,-4236481.385607183,-2713646.2349967286,-5234123.255551618,2226553.242048245,1122675.728499975,2205542.7019534167,-5080431.694847032,-7916127.3822099995,-6006483.155575762,-2550709.350507226,-4403697.086150141,-6245132.198689871,-2352739.977835655,-8158571.3044856135,-3854142.5751763284,-8288060.130015499,-92067.68469440378,-5675278.582491539,2152977.4467048626,-9347029.775631253,-7055211.763542566,-634179.6154835457,-13584494.961631086,-7002371.996757615,-2454666.8123324253,-3588338.17895429,-5386647.779011581,-4596111.800596988,-4810802.977548052,-9604377.219508927,-1684528.271522345,2663757.9094458725,-4601639.231427044,-6543784.387418464,-4442966.163290124,-5850676.007173698,-7401192.759329896,-7481065.089214473,4770588.890507473,-1694476.7668424305,-3196305.8580808993,3226697.787693551,-6242875.593696587,-5950679.436304221,-854990.2199724969,-3090442.1064513195,-3766336.363128895,-4288992.568254661,-3857572.471816048,-6610979.639844159,1262725.812369071,340370.8649610542,-3414540.862060705,-4363002.5834092125,-331738.8400521334,-2594158.7856555916,-3590589.760334201,-11569545.752146345,-3851151.61944565,1107531.714407729,-7312367.883991579,-5870881.247673055,-5204822.338385738,-5677773.4408886675,-11831146.955099823,-5120536.180393118,-8296323.507050547,2094785.9563327748,-2229972.4541388825,-3759166.324869884,635932.6817390621,-6517771.950377824,2571441.4968095236,-3360419.6411241274,5406913.339288976,-4652660.604904333,-958953.0523680057,-2956755.183996469,-5394358.377636148,-10946747.160620876,-1994359.1972436458,-2234161.610893393,-2173123.770560395,-2510370.055176813,-1844522.1151559642,-545184.9246449918,-1520837.7566040084,-6211915.400187146,-580244.845125528,-6835559.732575242,-11814683.56323271,-3890303.2319003474,-1099875.171872355,-4817037.065823225,-2493655.777820141,-13307736.10465853,-6332224.387020068,-5895086.844137594,-4553671.819139352,1335283.3415722419,-4243434.174075119,3693077.1978903003,-906436.2314737607,-2035767.1906954981,-11568310.60503186,-7826199.172872417,-2843127.2944159256,-3694592.620050084,-2862083.9062342644,-2182426.360788863,-7558402.94112858,-3254491.1494299937,-4836321.113367764,-8964586.353744524,-2334233.8429874554,-3139669.695993552,-6445992.653220778,-5840437.158430727,-6986226.855260044,-2634927.085960258,-2547165.800752636,-3866265.2878610007,-1371950.0653073713,-5373500.209004132,-6783042.320533084,-1796934.9631219134,-1919635.1670639794,-10417676.571184475,-9629802.570216756,4017017.174894139,-4193499.7034219485,-2834299.3766923994,-9142420.85806663,-4919830.311931545,187135.0787743982,-2584735.991511978,-9418297.033063617,-2671540.9309731927,4831232.545919318,-6085532.5022867005,-9335040.583046503,-634884.607638834,-1007648.6545027513,-4322009.237873364,-5854030.48921364,194233.30903635547,-3290522.875011606,-727239.569262851,-9002744.430180142,-3308779.239907449,-3870698.5855900384,-9440691.694615267,-7817278.169116193,1259128.3432128523,1144998.9508370105,-8423444.539100789,-3722265.0584084326,-4201680.587992932,430061.9828719273,-3299391.980307931,-3340658.9726805,-3325036.998835575,-6964550.883264737,2524511.975460397,697922.7373228073,-4658631.192125687,-4522198.210729172,-10402772.75640945,-8633028.476335084,-1686560.7352568842,-3724823.26223032,-2968143.630216703,-7829119.73841726,-9107392.436084187,-9117762.327767111,2669125.9743799213,-9125136.048103416,-1605609.7570249625,-4496431.798611227,-9164294.386497937,4381921.871050976,-4020658.145364198,-8177281.358504124,-6645216.4179391805,-2201963.0223254804,-1272360.548013432,-4277974.211779246,-5557896.294694154,-306659.52835578285,-2671272.825106092,-4056379.1255746987,-4601091.851347346,711003.8811274972,-3220915.596032176,5100554.289561266,-1035046.8802408706,-1880238.8903941214,-1823586.0718603227,-6573828.286039511,1389.777804536745,-6247759.708925268,-5318529.253747709,-5202209.437828658,-4342832.04695938,823736.6548017357,-10825940.600863034,-6659613.70866937,3083456.723639734,-6087694.967913408,-2075051.6724623665,-7007616.732931539,1426938.1047010832,-5874066.604579858,1342576.374919571,-4264449.596935671,2400755.552068388,3398600.963857268,-9782530.288940417,-6523552.527070653,-13449841.091788184,-3925150.420043526,-2319246.75650022,-7708739.759345841,-3028143.853533905,-7492060.326180154,80424.25725166313,1449117.2099140342,-6027884.933234328,-909911.3241429422,-3972296.6709833946,-2980453.1654586345,-9070499.726581983,2059840.0993846562,-163916.0665207822,942866.3215317093,-584817.1940487064,-1978778.6410769708,-5271638.14967796,660505.4132959619,-5235956.781443727,-6991861.646158686,1647477.9850889053,-5736351.694356475,-2587391.064773543,-4391828.238053253,-11053561.340389548,-6395694.37908995,-7003972.516559415,-556372.5476289578,-2774816.5785334017,-7681157.705832325,-2630600.118937919,-397871.07132269256,-331262.87143641524,-3376899.3323544357,-3660890.0528354123,-4547870.158060996,-4778608.791002758,-9208248.119176853,-2486244.714118544,-9042592.238581846,-2296552.315643316,-2254203.565487677,-5087081.968582474,2835524.3426885847,1676143.8821544405,-1834021.5970613416,-12451773.728933197,-839416.6266830657,2708614.674356144,-941680.6454930212,-1801158.5466074292,-1179474.2091163285,-4963614.804049345,557175.2932457104,-5086278.148759289,-6278720.520262513,756023.9131303858,-2574034.286453914,-53514.54434166849,-8000617.608658791,-6835658.267894434,-6995209.600569082,-2226580.635272639,-154641.22062919848,-4234377.726709172,-4701534.121178543,-4000525.988131549,-234787.8449567668,-13074.15363956429,-945031.9759886693,-9309175.780167645,-4553062.558493888,-5754593.662092324,-3036613.2256954405,-6322941.517053438,-4549321.137535701,1487386.4995469954,-6034082.264020536,-2733907.7802652977,-8973816.167045068,-5094262.994482268,-396110.9622230809,-7046587.2530128695,2326425.820315145,-6703037.639072638,-1840760.39765371,-6568469.218870938,479879.02678122185,-6428490.576892277,-6289402.047607217,-3751613.36408455,-3644464.6112654638,-5256807.9057386,-2727135.641324749,-2452840.8363747373,2099003.48971238,-3533156.1845425013,-6321313.779697718,-4069843.699638363,-4669944.1143216975,1330258.6863519773,-2421892.569741046,-4947537.352467315,-6718730.715409905,3247613.7536472417,4965182.839336034,-7038627.321636606,-6848637.021995733,-5382900.734716067,-3474235.0016748495,4827675.297027655,-1591070.2167977411,-6959384.70927146,-4014642.54026331,-1396920.1741005965,-1800831.3172714226,-7386407.027811075,-4952621.8284905255,-13076505.326862656,209154.2156609092,-788979.4050682876,-1652280.0080645718,-2228634.4721496627,-3515471.132587759,-5684649.697044589,-5028556.969530484,-7644449.242046898,-1020716.7721037176,285972.9271661248,-11895198.939704679,-6122074.575967232,-3256006.0814790856,-4842269.783280371,-6338015.918765254,-3577855.4480075855,-4169350.317898046,-5756163.896572754,-4790891.27256076,-1303810.9650088362,-5331226.7180357445,-5438891.887734753,-8801621.308137432,-12298.563791148365,-3793063.3781512566,-5073135.2083813995,-2306835.0037578754,-11174890.442560896,3985736.6127576698,-5563672.983167578,3853374.0963009745,-13468013.480914794,-20910.534078823403,-8795103.400024902,-1398965.495164942,-1551914.5086600613,320178.1026613824,-5752896.80867734,-3985826.5777547415,-7327508.346448975,-675359.7695494369,4832773.377648199,-5082677.726475072,-262606.94421496615,-5090560.158764077,1023084.5293830335,6398380.219807435,-6488300.936089984,-11979871.75172574,4495280.878464414,-716607.1278913617,-2435167.166316455,-8789003.302622821,-652511.5277154222,-11278756.80449655,-3801199.2729704604,-4867448.419010719,-218659.02427289635,-1632850.9512279965,1778089.066980265,1353844.5584936198,2267013.5456728637,1758699.4295304697,-827771.1453251289,-6660854.538347842,-3707193.768890079,-6274118.464989544,-2555268.6995776687,-10261711.200398222,1285454.2026233878,-570152.7315787133,3957427.8183864206,-5320674.819454849,-3571197.7774015497,-6882731.424870053,-3552949.1735894345,-8403734.68199431,-6155083.071553942,-1689899.7027124874,-7219186.073833587,-2817017.8978909515,-2241590.664088059,-5237051.233583897,-2265280.7951472905,-3947139.398991732,-3482845.889366014,-7424585.356236633,338496.7726927474,-2144114.2770838197,-5797482.143104813,-5096514.7275647605,-8385720.994042631,-4354149.04565124,-48040.546925487,-7359791.414594322,-1247823.5943381265,2097609.1214127606,-732635.0031500347,-6105924.016986668,-5393837.088098327,-5885391.410400923,-2446751.5213208627,-11500532.824366128,-1184352.0734395161,-2483374.2425997294,-715477.5886909906,-4736399.013141433,-4097221.98205889,115473.04383677803,-7121518.450915672,-5760979.011355843,-2231066.8062192313,785189.0415650047,-1042488.3137744348,1032204.934530871,-8566505.516220875,-10421833.844843045,-6471873.816352712,-1720808.0572731532,2450390.0610379633,-5537816.876536917,-3160844.476786645,-3332890.795532828,-26003.809963716194,-964486.9638324697,-3174987.4151682854,-8482650.055653747,-2396938.705524167,41398.27129514143,-8133268.691028552,-4260197.028100247,-3498198.937747511,-5122658.8056025375,389434.5258013774,-2936193.4150949363,204730.65469107963,110690.11357448809,-6534493.973257948,539345.6200542282,-4676877.861897413,-7600396.080156907,-1437303.7207612116,316127.9698290806,-2249236.6809416935,-7389617.304473255,-8478763.681690568,-8696558.87455073,-5301838.850807862,-5919175.958195462,-2508022.613475876,-5887235.401790002,-4377672.513493821,-345449.4859331716,1019745.9330551792,-6413361.624713354,-5827486.527318656,-6109158.404474871,1962047.3086165395,1639785.1552618816,702339.0123638771,-6912973.712901013,-4266765.592363581,-6265624.417918777,-8651741.741600897,-4152594.6844998132,-2219388.107590651,-3046944.3060521446,3824104.494892883,-5577931.60454255,-4274950.073913531,-9518143.417013716,-8899235.473164694,60914.052693163976,-1670726.355979504,-757628.8375008851,1100281.395803729,-3086680.7523076795,580587.4342913311,4962606.453863252,857764.4630130008,-5965606.227947723,1454235.2856869828,-850366.1662260927,570094.0451080203,-7823873.997927694,-2917697.351040723,24882.721000872552,-2773047.9356873855,-2285233.8865891583,-6414729.981412487,1644331.1055404078,-2678650.3111627866,-3869987.61166298,-10574415.915816672,-4396588.423861975,88681.89240718819,4427842.236398248,-7984232.241926566,-10004637.398719367,1072383.7174814008,-10306863.20479858,-9236461.94116482,-5683520.40220329,-3788237.825032193,-518774.58644824475,2168465.640069859,-4975416.748547921,-2708635.2523414465,-4469655.478901392,-282453.65579134785,-6336894.379991954,-3914689.5023932196,-2182300.3988988735,-10798060.636442391,-2132265.9815586824,-6974140.33149128,-2563945.9660164397,-8606103.199964654,-524985.836559603,-2188287.70331257,857682.0728758741,-3615191.265014246,-6024275.19628299,-6836407.686573837,-5755138.297582775,-1322044.6004044414,-5375658.059408952,278569.3497653436,-2855837.745993104,-7246264.4908997025,-2412144.850688599,-7825546.971550706,-2055810.2200879548,-2325500.317738861,-9074965.342419423,-3154858.679096384,-3314765.5742619913,-3558349.5078288335,-4533045.303133348,-3461184.7610602453,-339651.79000749625,-5703010.829457087,-6153002.335724184,-328506.0181378331,-10042831.20278074,-2693544.692564631,-2408089.6947251763,-4939171.325388679,-7961469.531596063,-4782285.72137654,-16095010.189475602,-8206917.535160597,-6085746.14281328,-604553.5601612534,-5302035.248026719,-12479594.361855885,-3321545.3124896083,-9372447.594692856,5490217.886141131,-7353476.518790648,532893.3621922657,-6521896.650080616,-9853380.670376899,-1129316.2615460586,-10951426.313469626,46075.7109738905,942056.5067598689,877560.7435399964,-4709945.403627217,-1368854.3697214406,-1425864.6284735072,-5950913.083912209,-1421030.4664462693,-3310779.2583027557,-4122237.038358435,-1986056.199616233,1919916.4587241989,-8583667.618663251,-4060535.973541841,-1379324.8070116192,-4508425.4312931225,-4229700.770755403,-2330899.400903607,-3807816.5585817425,-4205516.110857703,19522.994752738625,-5359803.683029848,-2317317.8416786343,-4647124.31178412,-9435687.644946732,-431754.84257589653,-4986298.725170502,-966302.456461532,-5434282.426925946,-305187.2766627576,-3578396.744242184,-2633953.714638816,-5540679.13557752,-8356284.296762981,-5807415.923783319,-1676388.554527942,-1333038.3289607093,-2089661.6505323453,-1112212.790200565,-1256291.336371554,-4012935.3204384577,-6991641.341020532,-3783639.6316244937,-2894829.957226906,-5759333.191754194,-4465873.626898799,-1278240.7035127804,-5269739.299348986,-4505013.834370247,-4941516.560165953,-1034709.1471793149,-1186773.3484618776,314600.5507429438,-4169169.0897639617,-4437214.56360617,-212736.24125519395,-841290.8910563514,-2885712.7429645397,-6567698.036878398,181535.42061568238,-7116930.174164232,-2313236.0814827215,-398120.8040857613,1140792.6063813213,-4442482.072822211,256016.35192070343,-2849016.7941199206,-4417859.883395206,-2944674.173028428,-7070834.566195181,-4083356.551245412,-220917.88793871552,-7702618.4538672995,1407988.3179822452,-4134572.8117137253,-3090962.8165763207,-6181708.140672024,-3991187.911820218,-1733302.2420861404,355809.16238448024,72167.12833245471,-1125290.9169301875,-2966407.5269785207,-3492836.520217512,-5847672.68230518,-1842376.1618608683,-894733.7319946103,1566255.1595054548,-3596933.3173881397,-3105898.4200487193,-4411301.243260112,-12418538.062505508,1615254.5960054062,-2854590.5364068765,411063.9001457337,-3453237.945671575,-7059854.777826438,765185.5650681145,-6077263.829858167,-6453929.676322658,-5883329.645397907,1871847.4338299893,2841319.9341958836,-2279875.05045834,-10466449.730944805,-2228786.2871030644,-5037282.312101206,-548624.9967504051,-5398253.760176919,-1627684.2488764785,-5580957.016422004,-6460605.206246427,1483869.1917207278,-4924741.941744233,3004382.3932045903,-5238237.525734225,-3135643.2633597385,-2512144.9785638247,-1373873.6728841513,-4608880.236303922,-5928394.2725268155,-1183567.899671615,-6189566.3016500715,-2268714.1498045307,-881026.0229374375,-875362.9818746075,-6534678.370090408,-2295751.2176012024,-10821946.596157894,-4450826.430101355,-4049507.5645669876,-8341746.416500198,-5829675.545079001,-3729993.47533717,-981446.5224776231,1798860.6640975755,-3716263.73028069,-7933062.995805293,-5886356.53252214,277897.117997963,110291.73793570884,-4356385.313935069,-560951.1379657816,-6186751.541683676,1713011.9381763898,-5513673.791509721,-6571063.479754338,-3288357.237103747,-5201583.637061609,-7026521.2839208525,-3411681.9001231287,-7094898.871342679,3063247.9363572635,-6110465.034646415,-10039784.820313945,-1146458.411797205,-3973543.0040567834,-1346421.8038193714,-4706623.910545802,-2504879.372969467,-2083615.4393560514,-12692168.724421754,-11325821.984106129,-6681326.787502108,-6589345.45813445,-4585269.752840925,-1813529.566016173,-2160046.5572389886,-10115095.789783893,-403444.95924185496,-3001036.7419466004,-1533906.6653989702,-6769102.529260637,-9542810.862756608,-3258982.7272416595,-9095281.6862368,-6300979.282794813,-4307722.365920324,-8524033.511261377,1225843.0969537534,-1620199.8553514723,1030245.1590330675,-3671460.3259434775,-7216014.967760883,-1321582.7048234213,2668335.8732499983,-456539.95436335914,-963571.790285619,-5821247.150821758,306682.66381485946,-795060.5305206329,-1229901.1494290456,-73268.21952968836,-3379087.2587849703,-6375226.694814453,-3803282.8153231163,-5423786.288591302,-4934878.454700489,-4046991.943676576,1848790.3767115735,-1860452.6159912609,3861552.9097866397,-2729310.585210845,-9775196.866626307,-520211.52250259183,-2876674.844862761,-5834822.060941942,-3918789.0984989237,50699.158471312374,-1847202.8106371798,-2328202.990710237,-547828.8651945014,-1999766.2587101124,-6584119.80744007,-3037642.490041647,-2084729.9551316686,-1285029.0757230353,-2483311.6491168067,-6184902.977878895,-6464328.555624314,-10430732.475491203,-7285808.602454012,-7172657.559878009,-4687705.944541125,2910520.4003032595,-8013244.144119518,-6358652.725656688,-11618203.137468003,6032221.574865114,-4611602.59843415,-1040659.5183015876,6586271.05781745,-7938687.039357346,-4946857.662003566,-653235.5895912535,-1626346.3972016163,-5289694.85813335,-8508029.32794444,3622547.165811291,-8716512.25146617,-6945640.848015873,-1103471.264632225,2629230.5728229415,-5948786.015940497,-3568322.630296547,-1419813.6572020873,984881.7669079024,-973482.2859406387,-1478121.4125780072,-3146234.458169218,-4282975.5421921145,-770968.452682633,-3104360.1167231556,-7560096.725654932,-160940.3198268637,-753933.991474919,-5740891.412100242,703521.8526849691,-1661488.575047467,688308.0514966883,-4374954.6721062325,-5342248.225260526,-8577025.165405251,-8221383.411960421,-2477905.8317874875,-6729177.6934335865,-1193939.5769040268,-5291600.440560628,-8442280.531898476,2676898.9366094135,-4185031.8532562517,-2934201.7660179483,-4626371.044627421,-3820338.3124754876,-5098521.506148268,-300469.6710796114,-7412535.25957166,-8570286.288283821,-9691226.57913182,2027287.6331851967,-3296446.156743994,775969.3345387336,-596532.3693374693,-1643183.6433037277,113510.0428224951,-5966934.385447917,-4404001.68412694,887105.2154405778,-6888508.559899836,-9635112.199759742,1715445.9550238159,-11088220.235813616,-3186836.961651914,-2865269.137375653,-6784297.650953048,-1079742.1536705894,-3301720.557702884,1383442.0251778373,-2259311.2199641615,-16369.333421779796,-7374835.823266633,-310389.3354722895,-8822539.772900142,-2548496.038574958,-4271392.682798801,-3987752.96718874,-1232132.1112772226,-1149867.719722746,-10478231.868086457,-58265.80273721367,-3792820.5797634292,-7022955.016542461,-3967871.4602577463,-9876724.226401437,-7152045.799482996,-4465716.043985875,4474495.916985173,-5376414.592187861,-4431075.390564041,-5044145.16149367,4111444.5834812,-1258827.9755372424,-2496360.8175628874,-4741849.971831936,-2368936.760110667,-6037451.785678107,1956653.8609420657,-4583971.486630542,-5491775.596473331,-639227.8751239851,-4630241.090862462,1749497.7121099494,-6273807.42364238,-5494578.944918694,-2559549.9418630265,-1104292.0720989555,887674.1052596904,-5995766.750487249,-3807108.935439067,-7403361.817584965,-5774823.841018714,-9336377.605039809,-6608329.709771294,-3263339.206402472,-1990447.8567567132,-4010045.954946371,-3738868.8786809333,-6535498.120407239,-1190462.8959752787,-1666300.350379344,-5817443.039337283,-1436136.5869144313,-4183599.298184838,-4039415.5349730365,2741227.389883347,-6905615.016249383,-74229.36922163144,234594.93394110166,-827976.4793277141,-1648933.2726542838,-4200645.102693353,-995276.4378495812,-2818404.8893639706,-295414.1192257218,-1839528.5979414023,-1718273.7000931893,-643482.927174611,-6637933.389981056,-8986920.46067328,-5509970.4800222255,-3555292.9718597494,-1969515.7052261438,-7988779.119265556,-1942327.0577065349,1261093.8859302085,368243.02292716503,-5444749.577401474,-2430562.532546727,-2024218.0776180867,-6408117.811818652,718414.8341403287,-2152113.376417443,-2147938.4530569203,1866865.7508442476,4067283.6874868944,-3281603.3210248854,-2129877.1305015627,-11244921.79582535,1431616.5068282858,-3001103.010827884,-811692.1571212113,937897.0340285879,-3414314.6504356246,-5014379.175968904,772381.4157126416,-6143234.085331688,-23619.82432122901,-6180448.298796347,2360429.351733025,-3422685.1467764936,-1428605.0587158278,-301472.18474214897,-2953205.700909784,-2737756.6495938115,136899.6418510843,1485572.8092178833,-1681076.7675523814,-2231647.1639956012,-6493985.6357930675,-913824.6173209678,-2307257.3005813137,-2589952.551348923,-5628892.584799301,-4807834.380798604,-4260873.532510949,-14116426.174921013,-298095.52387663536,-3325250.429383518,-2035633.2056565173,-2203016.7356532887,-5404456.466679683,-5707854.538679667,-1017208.240781879,-9645430.280285757,2172454.965458326,-5267463.095975529,-5524030.310009934,-9877892.388014337,-2875768.2838727618,-250028.27650196664,-3797231.5091344714,653220.089556992,-6115167.414475637,-1287385.5870357584,-3632060.9090751,-5119207.051653428,-5344437.059024841,-369739.7188988738,-2518336.1602948625,-5661506.5309414305,-7641566.540652314,-9610118.606909743,221273.29932153597,-3345847.9436663594,-4884186.7867565695,-4023270.763472788,-1896626.3380789775,-10169460.137679977,-3202382.3453581464,699454.1686350908,-2545664.43932632,-8068524.7933326475,-4015416.8682631617,-1848021.376177432,1542505.7991198748,-7294308.924523352,-5775117.723643605,-2793412.7220516577,-89914.93504662625,-4458329.003545946,-4673263.852161113,3116193.750566045,-1840339.103944894,-10961871.201654632,-4674303.040365595,-7158212.444378022,-3803321.2544118017,3343315.6704696007,-10083372.724361917,-5615998.100851597,-7526421.25704306,-6471767.768989822,3450754.3637350425,-10386236.397744387,-5320996.436893172,-5944621.892686315,-761003.5684904028,-2629324.5656420365,-8306557.317976693,492620.272716634,-4124305.9775630105,5491434.630389128,-6454730.002862655,-1993624.7056635767,396138.321602853,-7767935.546967035,2299347.2892231643,1391389.8885249887,-1079756.8164271545,-5808594.391184956,-881008.6267483421,-1708247.4520311262,-9355440.542335747,-1872280.1394451763,1419066.9804813694,-62823.37807724625,-2513492.5367257465,-4489499.190636763,-1752130.1269192845,-234921.23975188285,-3845799.3054570127,1201613.000158025,-4316142.505047455,-5256385.697133146,-1205922.6794160604,-2857933.5157847498,796269.4836128093,-12528895.212179303,-3139804.138394068,-4548565.167913517,3127342.079563394,590202.0299604423,4786345.019159937,-1185294.7849736437,6003.21290021576,2727588.670651082,403655.2805677131,-3667572.525897652,952075.7422389835,2155602.305211954,-1857700.54224753,-6828594.8102073055,-1260087.7499900777,-3413104.4844516274,-7829424.78957184,-3423580.698969241,-5959302.833535325,-5295601.276292698,-9698933.06768556,3207095.5405683815,1398284.794989001,-13258601.518867869,-5620070.351595389,-4487571.632386705,-7648417.473982658,-8662956.516342616,-4100808.7140592206,-9054281.167577425,-8279155.233530184,-6026082.646022884,891327.8705006503,106896.41852522641,3691981.7109550517,261601.6617347356,-4842757.4076210335,-4468093.917431409,-1123846.2319136467,-3269339.3133496493,-1760932.9880993403,447801.94646951184,-2208450.056897685,402498.7219290808,256640.46046257392,-2596238.903119282,-5007153.093819417,-3710631.8013416603,-3491855.860753905,-4392879.323516374,803614.6388540957,-5151020.916791646,-5150217.6367969215,1915671.3494054005,-8836334.130560793,2682621.896521421,-477015.42007996514,-7869566.969922917,-9231130.234392779,2247408.2641263027,-3581053.452895833,-5747166.582726367,-4775510.003953092,-1630998.060506478,-3618315.353080077,1595172.3097159099,-8491941.361928739,-899539.731815163,538127.0677886773,967967.5317490622,-1157784.0149954036,-6063299.95520696,-5769337.3385859355,-2517165.593352869,-3526770.198514035,-462380.94025831297,-2311891.0872039143,-5607104.499529693,113364.86444588564,-6297056.337859584,-6284409.280670574,-2251711.5256787054,-5854818.194008771,545063.0318870358,160494.9088982828,-607786.0484520998,-2967143.872670479,-3146811.993448114,4724632.382785175,3131174.385453604,-3245240.3567963326,-6380999.482618865,-341376.69729506224,-4439715.5077368915,2534489.757415412,-912158.6908504386,-1132470.2827887181,-5622260.086599879,1499049.7348535862,807899.7985492181,-4619311.913646972,-4304113.80339404,-3768525.2885467354,204526.8802082166,-7746139.204936974,-1178819.4567479305,-4962609.2724150885,3407424.758444652,-6423360.571491947,-3492854.851077879,-5654523.193724655,-696475.1247500293,-2266924.4613059126,-7149281.563215833,-5310466.456347587,1029484.08755612,-2630116.05097669,-633850.1478808653,784680.970642563,-6670665.029136229,137022.61130617,-919453.8650523722,-6244248.687280271,-3318944.053712858,917881.3469076101,-97017.04266672395,-429426.3280786965,-8716623.08533828,-9393201.674120968,-9057503.067070678,-3544212.212653011,4415518.043301979,-11157088.192273974,-4463080.264652666,-3582534.893709192,-1950499.6582930535,-4645247.984055897,-6591549.912075469,2855350.035575174,-1154935.1720252149,-6005394.506141344,1951570.60558727,-2778138.670263989,-9696751.489322687,-8283672.980599798,-4816922.628948839,-4576027.045766976,-7071590.587931467,-3597564.075134109,-7647745.759233618,-4653127.885484511,1168674.5886355918,-354629.74464418367,-5218312.303867362,-10877985.150462896,-1628476.6163966302,-3173953.9834493417,-5747425.79872163,-4596822.392484004,-6975189.511777263,-3095190.618128864,-3991410.92903807,-4297262.422293747,-9385969.810737118,-10475953.53643097,1466950.5992235187,-5020369.667541256,308489.3073255792,-1688538.7026537582,-4602428.0780854095,-4059827.655259028,-332829.1143507529,-3748237.837290397,-1113699.8090952057,-3739184.7143083327,-2614016.9994592723,-3549564.9737052154,-1630501.4561045077,-4634091.486383666,3610361.0193083864,-2436161.6691858806,-7466990.303041428,-2053624.5541989189,-3338074.2505967803,-3393435.2071193177,-8219561.803728966,472545.8850493673,-5038035.124120604,-4627190.124392055,1797561.1464418694,-1721307.337586319,-6574206.880721619,-2545687.223416308,-315184.6741671795,1541582.3154182006,-4006078.8904224904,-475574.50303753465,-1735772.1021624673,-6241786.225549493,-6214078.246619077,-4619003.609245595,-3749922.6013500597,-5493536.757874368,-2246326.352458326,-932061.524483921,-255879.3445759304,-5943350.139209636,-5964076.336452233,537839.2893176526,-7558068.21739286,-11887226.282311834,617832.2397899013,-4920351.174588431,-4095147.149987858,-8707001.30184437,-5558682.873292755,1468965.4942639302,-12004589.909165401,-1463716.4788502268,-607678.8254270423,1146202.573490452,-3699125.311904542,-6555032.929141169,-1453490.1462602187,-3926925.2086232137,4020040.638500167,-8659955.71231031,-5272624.975907801,-2327190.4148846734,-3386048.5063026343,-10368255.262395974,-1267390.4438783918,1551520.4535076674,832379.9987226315,-432359.79476975836,-5438384.363372682,1102403.3592517935,4430339.339814529,-4240687.55633259,-5067158.79289519,6367759.431819949,-1005817.6567504928,-4048350.738123322,-4868931.193637576,-2767912.9584425334,-5109590.6427445095,-7685655.628514688,93685.61439291202,-3113437.017610032,-1977011.7227906082,-6300958.97541205,-874238.4988699425,-6300718.790164664,-3743913.5445765965,1000134.6550265439,-6394181.6771826,-14591684.704562042,-4482946.188935822,-7674527.440549095,-9674001.696367051,535429.8730331343,-6017794.76910164,-51971.18638910912,-7628297.041747582,135876.89325164445,925321.8085095054,-2600606.5025242083,-2033808.2622477692,1021898.9586688541,-9196462.716985807,-2243919.114998499,-7161521.361517062,-1365927.684509758,-200975.889052812,-1391006.514033881,-2408458.323563181,637025.2090311609,-6692626.904057259,-6124673.875803975,-6234003.84291059,-982504.4271943774,-6691741.884922344,-7317439.303059999,300872.55643868446,-2232086.2734506354,-5424367.088767378,-9170528.2918284,4704477.796115681,4882964.605744462,-7038802.184571216,-2003597.765443381,-1578959.2523533646,-1099699.5654878914,-6904039.964528879,-1228570.770931758,-2428362.665212471,-929973.1046769712,1161088.4042670652,-2369431.519398289,-1139634.109465804,-3446842.829843458,-9814124.022054527,-654951.9662672095,-3366771.602264665,-3516438.173984362,-1487450.2820433937,785906.5947286263,-6671143.372055218,-4742607.472756319,-2026754.389675323,2466215.6739242505,1866046.5742927343,-657797.500010197,1371752.922367612,-7994663.659137974,-2589496.8641311936,-8735250.327454282,-3400416.148744598,-5055628.138226634,-3074536.567654928,-7812939.74852841,-2083579.1645917892,193873.73190343194,-455296.0004022792,-9319560.203913735,-2677309.270000113,366845.5734868515,-2545004.229111301,-2751617.0899176206,-5007496.769988576,-2235394.9511295166,-5129018.708254611,248094.19958742522,3424396.5939684436,-4488681.920663511,-505771.8366670981,-5805333.180603553,-4573284.272683775,-8039286.377679829,-8776183.129600905,-6053352.715116855,-8656347.39137933,-6238096.907338973,-4689144.925792873,-3255541.691069141,-14696515.997679925,-6997344.020649377,2744095.00558009,-3451603.447286345,-7956951.239291327,775174.6993302107,-5124402.091344947,687281.3554333504,-3015097.532036975,-9825654.184456263,-5980795.364024721,-4668813.812184388,-4259762.055215819,-3883119.957727652,725936.6500864942,-9105420.149149265,2107642.8315506335,-1636656.0264136326,-512336.13276747055,-4874590.2115494255,-3209278.328987507,-4564.795740026981,-1300800.2776569873,-2913988.246509956,-7179501.715246543,-7031605.242696896,-1908542.229172837,-549290.0750874924,-5839424.091556298,-6719353.900049524,-7156856.1590430355,-8863438.446031306,-2681594.29261849,-2552702.6962744854,2081404.8145530093,-3258861.788912285,-15161337.710381582,2478171.783329537,-7102846.021018935,-1441882.4154907763,787203.2870100215,-5770701.223202869,-2773755.5464526303,-11989232.127059782,-6373504.510824041,623770.5085674785,-4638540.89494938,-3073716.311641872,-3746520.4136653673,-1603847.9472083226,-9321465.80286858,-8118882.735621048,-3130807.214017449,-2117549.3303817883,-5337463.830430198,-525386.0819352567,-4308411.243785098,419037.9635743946,-171082.2561866045,-7759263.849684969,6404816.556957353,-5227285.738665536,-5127654.439861266,2256910.3040777463,-2615738.789659558,-2795206.284501847,319881.3538003899,-3123302.2636650763,-931269.2054750919,-3491857.455385385,-614511.0612060465,312477.92967624776,-294616.0316290669,-12795400.610066997,-3322436.0080365352,-90473.66490388475,-5062468.154164668,-5859420.313321203,910012.4397001527,-2931770.178226907,-6441439.603971211,-1193687.0254777353,-7475652.108983383,-9578223.8539484,-3650309.4616522994,-5402063.04235399,-836325.5160412844,-3035126.154156536,-8263570.065157449,3964833.6178677212,57305.23057255149,4898284.408666678,-3293570.5581434444,3985587.085179167,-9123184.303513573,-4625555.6579333395,-3932436.9443873707,-8540268.90756562,-2134899.580596449,2732979.161426695,-78459.97451960854,-3324445.278667218,-3628798.3369502276,-3744965.0878513902,-3959682.737671232,-3892372.8310608305,506709.0277818572,-3806485.923002057,-90846.87459584512,-7470678.997709025,894210.4683407899,-7147764.11433525,508955.0758945029,-2826641.2840341423,4333642.677181991,-10896459.52837272,-10623973.858915847,1642318.5522176325,-2905785.7110916097,95925.07988436334,-2491676.9247298352,-6195272.096421439,-7208163.740290953,-1254859.6094031837,-3581325.2642207183,-5044722.2372284345,-2986006.3302369993,-4339223.428836498,-4594904.675301611,-4731915.470657662,-11387857.998194981,-5770216.942167314,1632532.8705747249,-1551173.2435157932,-6020533.379927997,-12031913.516465453,439129.46304654516,-2094768.0886109527,-8124801.938578885,3152423.2859663144,2608702.2041297294,-3003790.732971064,-8670877.543698488,-5568403.487991903,-4502938.243347412,-317748.98728262074,-1594319.007696189,6320119.290537418,49147.555357955396,-10450451.102596153,-7616472.653978373,-2522598.802784521,-7312970.270553807,-8103487.942443756,-13025689.425358808,-1285810.2053631376,-3549717.379231347,-9127703.664428294,-967369.8685690556,-4087748.1277316827,-1606792.7311282959,-11273332.973479044,-7245554.8014015965,2621397.6367427036,309368.7551865913,-8972404.587306745,-8086726.310250403,-5611361.090918193,-5259894.566284483,-3527243.073275963,-100056.02668916434,-1893770.0485410448,-10114266.392197298,1162531.650140604,-2599789.356602935,-5082692.941396968,2259125.4157838523,-927558.6315538865,-6194480.881590087,751153.5644435063,-4250782.830539912,-6261134.886955317,-836452.6285774596,517824.73741864413,1163.0601327829063,-4951220.033045001,-6571746.570915515,-8651800.564692516,-4018368.273907166,-4215909.904472843,-431501.5739984177,715571.2839912716,-10860208.303437846,-7918004.765980123,750609.577182021,-4040456.8790572984,-7800926.29522955,428261.65671277605,-3060204.9256449845,-5542846.415313991,-1790388.4658300728,-5514596.426387673,-2357416.5356518384,-7374893.316518528,-3002392.153731497,-825766.5054834522,-862875.9034087341,-3474813.109902721,3122242.01031271,-6397584.332329918,-4600961.120634887,265596.291933788,-10890908.839937843,-8401696.61252694,-9829108.99965473,-470146.87526822835,-6289184.903761972,-4280384.091079392,1535616.7146694213,-6828441.458489696,-3708514.281367587,61220.08274740912,-6545837.972293508,-5471576.605490934,-8763923.187640855,-2219764.1759458836,-4694817.06524883,-1471455.9946460705,-2977405.9894436523,-9809895.378140396,-8917801.43551527,2775822.515408056,-3137133.9651144706,-3348456.2961246185,-8802264.742615476,-9674925.97404772,-5777520.17131771,-2456321.0662747715,3311094.6551943775,-7080738.246778952,-2862481.487156572,-1925885.9501065128,-2459128.440178735,-1025019.9825550299,-8397593.74468891,-5927403.153764583,475771.6967704799,-2588735.0631633233,-1098580.2056850847,-10479247.45344867,-522138.0871979939,-13067275.309062801,-1576589.9656623937,-10659021.176287482,-5350205.017850373,-1391621.729030257,-4881297.067227952,18320.368554424495,-1081937.210961448,-8815721.763099931,-2725507.903831467,818229.5069093741,-2867059.269237198,-1539499.1828170829,506234.2125900574,-852499.8435094226,-11099233.359610617,-2734481.333509814,2581822.7617387734,-10437680.685925884,-1025257.2757023703,-3335726.1908554435,-3132630.9617007896,-4628406.59259228,-3715380.406287711,-3106917.728583526,-7463039.431365717,-7743699.319190234,-4329459.12538949,-5119247.844397521,-7453478.566220539,442891.5740344506,-10252175.7847425,-1660396.8439819906,2120180.706775749,-4819436.453776501,-2030509.194076687,-2526615.757441964,-7069008.634105156,-5861451.998758245,-2452776.979338588,-6440285.151629124,-2241572.37359678,-6920319.230573248,2354180.729973806,-7773853.986425262,456986.48844655976,612531.906311119,1532532.0857718345,-4249789.659111677,2012341.4630096126,-5715632.642219542,-7390924.158080058,-8599179.752626473,-2728719.9426703844,-6173406.001781916,-8072665.4062406365,-2011181.2372736,-8719494.743067607,1453908.9246589746,-8674371.45237666,-1173487.5041728634,-5920221.615014195,-3674036.999786251,-2628865.8218410052,-2501623.1341986265,3081564.932850985,-5370734.516295832,-1414817.9035117235,-306536.3546885587,2631413.0818736274,696120.8533489332,-5908481.493833806,-5143947.4863287285,-3811448.8903783485,-1530107.8245152682,-989712.5173846018,-9242753.002671793,939582.2601251323,-3100846.2370767742,-3055413.6776916776,-1558874.371536281,-371903.14354657754,-5583747.408791382,1596502.569066234,-8054213.428131461,85163.65260028839,-8226203.482619708,-2345864.076698022,-6959192.081602801,-3499653.0796692558,-4685917.67027306,685292.5523859393,-1959340.1319882758,-7028272.212156609,-1712943.475971328,-7379913.164674543,-4547202.398809724,-7124573.108955178,-3649799.80065473,-3085969.6054489147,-8951462.51203619,-7148121.663517676,-8147978.124024184,-2646021.930982966,1305275.5453265123,-8665594.290945977,-5836070.836375071,-5595991.110216053,-4035384.7451561186,-4574418.738404563,-2768304.355215013,-1496251.6275285408,-3861731.168874249,-519889.2828403339,619363.635738615,-9353448.813760173,-3503917.914276123,-3031253.053796636,-2619395.4986010212,-2618028.4109156877,-3049755.4822921883,-6154007.129070806,-2701040.8127658013,-4664658.881175559,-5843552.454924485,-2536947.3892129734,2462117.1373676565,375596.8187404182,-12561709.28506404,-1261008.8202562816,755221.2571202535,-4262682.484117366,-1876179.2451116312,-2027454.1751100775,-7422081.008703904,-5154245.895026302,-4703298.43296255,-124304.81783453189,-6841085.823068938,-3791055.1975079346,-405258.4484656714,-5995549.228696218,-1009316.8255585656,-2795357.2754268795,-593425.1914718151,-6579269.186125411,-1366581.5960485302,-1200701.138563592,85695.00513853692,-5348263.270314669,-4887911.009497603,-6640282.687110064,-6168443.674466537,-7250207.813233532,-2041120.4167110417,-4015757.6596538797,-2529714.162836998,-7622758.76342872,-4613812.397344578,-2180917.7312367447,-7276811.451255005,-4506385.514829064,-3938732.4819190037,-643381.4846397247,-7025345.595220301,2114145.5551325846,-1712713.7753182538,173137.40256402083,-11742426.957311597,-1116034.0021895114,-1553945.7862915304,-3012142.6970235277,538905.6718664784,-7394248.8916266635,-449079.53967066854,-4488232.359555071,-1390139.2647743635,903141.2559243888,-1770346.8874022719,-4712060.100733705,-7693977.145448523,714225.8866897635,-3719198.414925427,-8253200.2883974835,-7417195.586002206,-2788520.8278791644,-1138212.9117621388,-8101441.320942493,-6225932.8438802585,-4535460.510986714,-564463.7860830929,-8734878.855155984,-331883.8862574436,-3739246.44913096,-1522537.4445094727,-5000323.966965515,-7761642.152201369,-776928.4815378003,-5048905.4381772615,-3676402.465871034,-463106.9008523021,-1708298.7734282035,1755375.521947423,-4137353.9289918,1752090.6589103974,-3446422.2353327908,-3742445.9547035173,-5704568.444953367,-1911167.88048039,-2190496.3875710443,691869.0730178654,848479.863364121,-2227392.959795041,-2153929.280823443,-5392979.048586898,-5690596.9591124,-8038236.01693305,-2184624.9688453916,-5958288.789707412,-10944275.558089623,-9381001.634331413,-9715961.442668233,-2971781.341316007,-1922355.443150131,-7675576.637453012,1489396.8424782958,-1160514.491831744,-9004190.661652848,-4710118.911273982,-4559696.831152622,-693718.7526128516,-10924574.290543145,-743328.6383760776,-1069949.8600889929,-2030656.17820625,-1880339.8356616702,-7817581.62991515,-5164553.242500475,-4258228.939690245,-2979371.7587698344,-4876328.942264361,-3145324.477008259,-11772034.942226296,-6575677.945084143,-2280469.816973947,-1527092.44597603,-5453609.083543986,-2547659.8410073947,-4736059.391290581,-6887326.81782818,-6406162.922058109,1928774.4132609814,56515.63265321031,214644.6373909004,-2946341.59119251,-5362812.242800387,-11248353.833399212,-1920128.2701833118,-2556723.0055902135,-5496762.100968612,3628221.632821424,-7808060.944116149,-5757531.959327973,-10448396.711291322,-4400377.2922530025,-4152355.4285312444,-2614502.441839002,2534053.290486589,1581048.7279084548,-721132.5900397003,-6071148.622176496,-9651679.906971049,1221158.4323277809,-4467252.908992127,-5447799.855685518,3177345.4769944753,-7175231.6471874,-8451373.30275762,-1872355.8140866011,-5237217.512361029,-4536299.256308828,-5150227.772729509,-2731834.742389407,-2740727.4522208534,-2003048.585217731,-4604064.414835884,582098.8723287825,4196107.426750841,-1683127.8343904167,4780176.494616613,-706977.3684164975,-2277155.3603985272,-3850859.526011696,-3419105.1344273165,-1197753.767912455,-4885277.377175703,-8435949.123841818,-1198239.7577137742,-2518581.280330347,-7430917.3307895195,817551.888331484,-1293014.5749592148,-2417436.715427518,-6595018.592838695,-7466114.352654561,-8391469.218852649,-5915032.360954925,-1128634.4186088834,-9166908.160947263,3618039.6160839032,1203744.6953297332,-545767.0999603793,-8009555.119998279,-437099.0591399316,5345057.842841011,-5524617.576178489,-6165202.22396653,-7838468.805669542,-2637918.8601699322,3692462.1026419494,-4797665.070754511,-9652839.082362084,-2333519.2615725007,-3159708.58100784,-35514.483676787466,-7547049.468055407,3245373.0324294902,-4374255.929985056,-3812138.6087976545,1932373.788809482,-6888536.90726532,-4118829.7098789075,-2983347.2437009662,-6998757.601482039,-428424.6023270767,-4022146.8956640232,-6917637.892686175,-3275169.915148383,-2611196.6505202595,-2193603.1334758606,-4898323.991951123,-3697629.9814222883,-7770351.746598784,-1105201.3849557452,-6404293.082631815,-8424958.183635563,1443448.1887945998,1709695.312502848,2498121.63384472,-3871315.6706740037,-3617214.0258660093,3500542.604709575,-9129195.598389298,-4219384.150035258,668662.8593417276,-4150098.3727552183,254848.9152774103,-4842144.31797562,-7743753.821145294,-3978159.8643083572,-7540360.684312688,-1176602.8802224956,-1444435.6281811297,1282502.8802059963,-4664724.410071049,-3167997.6465460826,-4218906.300530728,-4146931.1557842866,-6076529.2386281695,1146070.8572779149,1871502.71324474,-8160338.195498157,-9298318.549365578,-1024686.4459559433,-6148009.637102049,-4568108.770725254,-4377757.559484642,-1434998.0838439278,-5152603.143944563,-3194989.7388502527,-2560946.839535836,-7201491.480789859,-1932009.16323167,-1545077.5360620432,-8629236.723473527,-8456547.96401121,-2770766.5085094254,1679613.0418130197,-8919066.406351458,-9877815.530312678,-5085897.517049564,-13507280.863703147,-1303042.675465932,-795683.9931209162,-5134985.497590836,-8900301.826856436,-3286157.101064557,-56221.52472257428,-11811355.864070278,-3343911.73443749,-6597572.632679099,-134624.1021468714,3289611.3252002727,-5206852.7457700595,-5796106.733777488,-6533429.748780489,-8067370.756549539,-2155808.3061014637,-5108609.83621174,-8703189.873164548,-111065.26818645,-625380.8618615568,-3610469.466859022,-4478389.744730718,-960074.5357833635,-9425280.328098277,-7310080.9380365405,-3724881.539714087,-2315155.5741929337,-5007431.651104258,-7502471.039539443,-1228057.289633859,-4066285.186166797,-5803180.138653351,1734992.1269045454,891252.0825913735,-4611640.118767984,-12653693.108592963,-897215.0949374158,542778.9520377126,38904.26569556631,-2798965.068640046,-1697517.2567395642,-4066873.352516208,-8534380.122905204,-2193239.738330452,-5293206.857636409,1075428.8929121327,-5535349.204087051,-10473372.166850254,-2237150.397737181,-10741209.441974994,-3544720.424197579,-2255712.275607532,-3578887.0849690586,-4910763.7873092145,-4972353.183089215,-2672958.7588583548,-3245249.22405399,3018271.5694374386,-12991061.072486624,-8659510.570592366,-1350396.011582775,-7971178.752891526,-8298954.1209219005,-677017.9555047601,-456469.51703130826,-4145992.268733645,-333864.2694998197,-1649786.9940036796,-4599802.689621214,-1668431.0873881858,-6944198.246278057,-2839084.8685649186,-4367572.032906638,-3834667.1054457063,-6019801.102690261,435360.2082454972,-5767542.241471939,-5510026.820508881,-378437.8383235913,-4709851.104267508,233889.13335530646,-1485186.3872969877,-2049232.813418951,-5548003.133834096,-4787251.654375689,3034821.377397483,118462.61493765749,-385190.08214771934,-7255696.28979284,-3431987.1350723505,-4981742.073080622,-1838515.1233674232,-7216423.098409497,-6718457.686306678,-6324055.68330696,-4137671.751799466,-5679797.229787731,-2763876.89126038,-8832777.865535993,-345272.1678878721,-5679377.4310398735,-5892189.531124238,-2333055.355926508,-3309781.0890845917,-1718087.629773993,2493422.1844208334,306152.6509056818,-8267384.95092345,-803822.5046022162,-8126321.373983322,-4968393.915366152,-2123935.5816583447,1042996.2336061001,-7253589.484033601,-417545.4809604883,-1445787.422259882,-1866465.6691309325,3909119.9644206967,3229351.1025138013,-3444123.4319869354,-3999581.8881568285,-8011354.93730053,786016.5771060307,800944.9352131411,-4599646.487332877,-699726.6439862214,376508.84984894283,-1029350.5557284355,-9610225.15016593,-2471157.1746835597,-8401382.746608105,-3507681.3087917008,-10032294.33316415,-12384436.551544588,-2831656.52560807,778008.6699804775,-5496883.708185634,2501409.482763348,-3958521.2433436047,-4942460.522130683,-9026165.433579555,-279215.0222239047,-6542757.52174741,-2197719.8192182146,-490048.1769868508,-3244096.898899682,-2458525.8470417745,5209209.075232334,-4611352.62171473,-1221215.6908416133,-5514139.948364846,-2200617.4783582147,891947.3337691519,-1842320.052161213,-4494272.291217335,-7139504.735749744,276190.5136447195,-6560815.780035138,-4426832.014858738,-5412637.372264495,-520922.17213297077,881327.2957709506,706789.4260786623,-6701168.044697532,-1389655.3212448116,-783811.9668049812,-6097513.936289856,-8015056.111749432,-4556188.538575254,-1276026.4811013602,220604.9293089807,-2428781.254161766,-1728985.8448007964,-5352419.305957868,-5555552.46951909,573129.8044238295,2533649.4375789985,1418064.2789813317,-2252191.294148719,-6810343.348288154,1515758.2070921455,774339.3558267262,-6346254.44427504,-1803285.4310671277,-8980224.048015472,-1387963.2861249987,-2418579.937325809,-4330597.420621609,-4659996.384452896,2289.3688101023436,-915232.0209400598,616014.9295054935,-2972598.460006479,-2268159.125991177,-1277391.1367755458,-60712.59692647681,-6633473.817337865,-639597.969977444,-8547182.822890101,-5658206.015924146,-4086382.4340545535,-424199.5955623612,4428824.877907548,-6486465.082245166,-604830.3144789413,-8162281.641926153,-3788870.177288031,-5625987.555955242,2659192.734930589,-632901.3565452285,-6286248.541744847,3231251.443234699,-1060350.6646080073,-6720500.669913042,-1948340.975879727,-6030298.983602827,-5980115.231260287,694936.6112499535,-1303247.4259383269,-3379885.9368252438,-2519975.8295336775,-3725788.1160477977,-4472947.519516414,-5786288.77565003,-683525.7071088552,3990833.6358722653,-2727141.9718678966,3529516.110282365,-4503693.691810852,2595291.9498253483,-1643090.2607801873,-9787168.538250899,-9788253.831791403,-5845696.48944599,-553834.3102032077,-4875361.335935516,-2534832.4722024594,-2694973.8237346355,-4363253.428864025,-3417959.371476244,-906598.9821185432,-4263834.728907034,-8876610.684370402,-6168373.598288726,2057573.6444973499,-2399308.861094434,-1797410.095082797,-2420715.112966012,-2373396.9901461303,-703562.1139980908,220211.15304782428,-4074937.656171659,-3319142.6018734686,-1293697.446338593,-4161817.8991243783,-9589429.187813431,-3266653.1200577766,-3082408.3204657286,594942.3292304277,1397200.3770352602,-717152.8291891646,3652947.3274439257,-7001936.128744578,-8260851.721183691,4590731.868275184,1852973.9212650657,923510.7241996285,277363.8065046314,4127466.3618162777,-2611083.4506347664,-11915466.773975885,-2717019.1875633486,-2965043.191445682,-5600666.08107866,-6977312.320762541,-5115565.686535429,-5979913.781771276,-4221418.055156555,-9627722.937058734,-8302340.505881835,1557826.3465012051,814451.9252021499,-1994105.3429753575,-2686914.3726434875,-5351078.9408547655,-3064542.9142155014,-17061.393918996677,838501.3260131152,-5732536.719769951,1426089.8820144292,-4619386.119827944,-12011893.02678199,-8209390.241632316,-5291671.398101401,-2446619.086642242,-3059714.97271315,-8918967.793890895,-2626100.742762007,-3563954.9949921984,-2226961.5724052843,-4608546.619981412,-2148578.860049084,-6824229.529030198,-786060.8631258477,1985610.085615063,-413611.4639291242,281533.8393418044,-5482791.528956375,308642.9440079611,-4007702.080659559,-6308824.335411934,-1237855.4202431906,-2699987.95453112,-3766393.705171341,-11025313.128338723,-5608358.087139992,-6421556.943778984,-2322268.651433507,2113076.718310252,-5237234.782536199,-1624489.9592106808,-252169.18905335106,-7434074.159716254,-4027887.0235812757,-2589815.687326474,-9402858.585165545,-5174248.700233571,695256.5764402822,-3264704.4769416293,-9588704.843681347,-6622331.731033515,-4994811.750992293,-12208886.779381782,-3270193.4577140408,-2290390.9006328825,-7439650.1567114815,-5033992.856538024,-5491245.134661751,-5659863.164404804,-5018154.264260011,-7811991.377578761,1411548.0780170597,-6691843.582328458,449053.04872887395,-9556411.037234072,-2032698.99713826,-4273690.834182113,-1888432.8576931357,-4849176.249842519,-4639996.058651276,-881581.6047794316,-2146890.1917151045,-12702686.902940676,-2971978.3860249817,-717195.9872813132,-9170028.942622183,-2517416.209882753,-3679995.494968474,-11517393.383044565,-1403386.3959095012,-6723016.376845999,-3565239.444167502,-8350798.5018412955,-2496607.8525437843,-4395929.760274934,-6742852.747680556,459758.4627861902,-1833221.5340049444,-1009510.4072881304,-2659162.714579152,-307599.04588778317,-4378815.176379414,-9148068.24609772,-1950318.5070857573,64742.76390400343,-1070501.312771203,-1337976.5598325375,-944469.3902528454,-2033935.213540174,1645402.9298958294,-589853.5014167428,-5035182.468039848,-4570609.748677138,-1989100.824286094,-5796906.533460719,1790315.078538347,-2880891.8875360433,-3613606.0182971787,-275553.414863402,-1274653.6888732594,-3390568.628220454,-10164331.077769708,-2968782.887827659,-1209916.9582659695,-5836166.288492219,-1858923.7075342722,-7338398.490635378,-5209143.264968751,-3218244.143376151,534637.9244823381,2354128.59650366,-2093415.7257092,-2722893.5188462734,-2180221.4530822188,1061287.541974945,-6983568.8992479965,-826562.2295010304,123992.93729653396,2202382.79267261,-2477832.418631682,-3624622.2031186894,3310196.0688706413,1916016.0730432663,-4388080.112006655,-5530318.572906109,-3742663.1047908645,-5386665.761300139,-3558026.5516374744,-5009985.945432225,266229.4163714498,-6365095.651878426,-5916531.414351698,-4877245.489857396,-5932289.4202723075,-5661125.656320641,3317789.09985316,-1695756.5736783482,-239618.91086104885,-516067.5081701819,-7628418.65031911,-3769931.933700403,-982459.001080811,635704.5944018904,-3484424.67010965,-5471995.234752331,-132512.2965769861,-1109171.4363798387,-1118730.4942268413,-3971299.8485949747,-12287263.491448576,791943.807327697,-8025286.455216136,-3438757.747895414,-6893364.092637872,-361780.78415931575,-279894.5784204472,-3379343.9359215256,4647372.069467982,-3546150.1912029404,247245.42718130723,4487305.723612858,-525249.8896021917,-5710501.91811396,-7554052.599058762,-1041113.3017682862,-4379402.016912304,-6135596.336728621,-9779673.709157823,-6533973.2621307485,1144402.246390853,-4627031.3227151055,-5972310.515003771,1203047.2194864582,-2693821.757098561,-5674891.41629385,-5416767.101263057,-9583440.141860921,-2481403.1124815326,-6852028.579349585,-4510697.57616527,-3450380.4777489323,60616.28742870875,-10852773.32073481,-3503604.550989846,-4559976.907808125,607714.2636316177,-4989754.636228429,-3275069.467450084,308656.1976010762,-5588976.427976154,-108286.74308384396,-6364053.870283367,2042690.0714402478,-1721968.9586379193,-7551282.08252714,969601.1956567299,-4772901.327336045,-8354933.60761396,-5891690.9999651965,-12305592.164667059,-1081042.1986824982,-6704863.128161844,-2785743.840110032,-9168514.442597352,-346797.40965360403,-4326814.849693807,-574446.2729626875,-3637439.8321691584,-2412745.4788905103,-3896216.2236789875,-2965160.4595854506,-8135276.14520368,-2284929.4145646133,-3134844.3075023238,-10780830.875116047,-4893059.646457944,-1147054.7837221306,-10165423.397100348,-905307.2485862356,763623.2836200427,-5334788.333758911,-22009.296112017706,-2448535.207613047,-4259286.793123551,-7155945.999407338,-2833502.7099309303,-8036468.011913082,-4074789.188086517,-2886312.5839590617,-12469391.031705867,4128416.905681597,-15499783.485631645,-6958679.08434229,-7286756.716760188,-1387179.8031511195,-6438723.16118701,26463.01309586875,-6519152.015450988,-4988058.747202262,-5269753.276738988,-1869778.4895976987,3247174.8050908837,-2821318.6879866254,-6848866.549050292,-9414681.520978201,6148420.617012372,-5508275.954445383,104600.2910590209,1378753.5453738533,-5636476.942664886,-5075572.9640923105,-6315462.807448238,-4079306.6190819573,-5097843.859144051,-8128018.157090038,-1251451.7263421658,-4130593.5186494254,-1944277.79143418,-809857.3567657992,-598972.5514305178,-10345119.581637483,-2628850.101734534,-2470256.572282889,124737.35863126814,-1429268.9178084973,-598618.7241666224,700047.3745004907,-6510246.189583177,-1131655.8863567896,-1560503.052375678,-3991415.1193643007,-6170190.278758846,-7486777.7184090745,-4374145.759449681,-5573013.237703151,-5410136.234037375,-4735680.3831444215,-10572564.128059005,-3844624.7856557984,-4300898.788864925,-9976520.051534982,-1207932.2126226574,3261986.3893061373,1465654.7329741763,-6932162.553279862,-1811834.7055827398,-978017.1700303555,-2358319.305776816,-2061794.4735599076,-8191401.763652969,-1499413.2897822596,-9684255.21902993,-775666.6108795013,-7629706.620581154,-1286480.9568407964,-1841691.260993857,-1964979.4297990073,-4836488.02114998,2304304.031174492,-717513.2223867942,-5421335.316176131,-8110985.065030184,1104369.4317124337,-7129940.048320286,-2732658.91002352,-4760560.162392216,-8234031.764532022,-12987731.713189302,-6245865.652089972,-2260840.876243202,723968.8174689822,-1627149.3398107402,-5784071.410306884,969362.3384311218,-4580303.19496602,7512180.752312539,-7069475.324800036,-4404095.955028702,-3428898.4079291476,-7233142.187101085,-8177423.953148356,-5011425.03361872,-1240976.688579699,-2091193.1349342186,-3140383.8196495716,-8172265.665210577,837757.5098019503,-9692502.208840763,-5899202.040606007,2319768.2448019143,-8009542.760589538,-1875954.6890584622,-2661052.9654762093,-45377.84013487585,-6854445.449381435,-6555932.6733827945,-14276197.047763774,-3388446.1605968922,-2889411.809226621,-8667798.99170363,-3063285.7881082147,-7004484.84667149,-4695662.239994321,-8340051.316541808,-3188150.771126574,-7327899.906438699,-3650588.205665773,-7199188.737560524,-4665702.823343633,-738338.5345266275,828098.0402050354,-1822096.2170011587,-1417916.5418832898,455919.8753000647,786618.9863441046,-659821.5807988569,-3873373.0109974258,-3327790.886455247,-5337534.925899776,-1709673.995361628,-5452002.708598845,-12037093.748579124,-2214760.1841182094,-3953425.196120806,-837563.9670553785,-3443252.6098439526,2364689.4394278247,-5102725.67874134,-7281549.092429139,-5363563.463119129,-4326602.359652191,-6851957.857859228,-335840.69660517015,-7018854.862462483,-8182654.048398346,-8787678.890989268,-4309910.612772193,-4956159.27374669,-3872305.5976891406,-1706838.3254885823,-5304355.899200704,-8161962.8945993055,2218702.0085283313,-6080699.811501436,-187442.19232709706,-3711731.229114514,-1738141.8989974465,-3267114.003320683,2742328.2201476134,-2931211.0103565715,-3802566.6286890544,-2494284.0207268335,-736950.4076813571,-4682879.27608672,-497626.1620841976,-5902527.905164935,817315.0589111559,-3105555.343063264,-8383420.774713514,-2052127.6202730741,-1454752.371025579,1779266.3016601354,1487617.5773693137,-2528748.8451548424,-1232910.9704249408,-821433.4764823392,3467284.8424558965,-5568301.943073077,-10683284.260612661,-7308117.746138915,-7881274.6913983505,-2726804.515085494,-1007725.2108768839,2229506.5152162053,-589418.8989910837,-1520213.8552847598,-1021915.611088885,-2383752.49211419,-1293569.2232316975,-304494.5723630022,-2208102.8366356175,-7035426.286598917,-352566.65046161227,-10795546.28056621,-1714784.8924036361,623399.6387833841,-6269843.460109223,-4113914.620308621,842630.3546342663,-7799799.974817401,-2920684.741604969,-7353518.477458248,-6099562.28130272,5958561.994178183,-1964623.2623564042,-7789257.072403466,-1135701.3835088573,-9578006.132963343,-537843.95114162,-7522576.497520955,-355600.98798107915,-8658285.093269698,-847373.5392239541,-1438792.2351743262,-6838089.301547021,-2241015.970674293,-3694089.8716799226,-1969592.1026198715,1679198.2202495877,-3824254.5196977966,-1663441.617381433,-1135570.5288733784,-6900736.01779761,-1952416.5270185042,1382609.4525735881,-6297132.57016024,162544.7115886137,-6514829.89746494,-3478471.758416258,-4071226.357698925,-3532972.381135119,-570386.732243143,-5545729.603760531,-6468703.116993802,-4972641.310914215,613369.2814982571,-9516842.817855762,-1265280.9531983435,-1249631.2649194226,-917615.6032754313,-2146406.068550825,-6865279.037167452,-4930941.341528086,2282772.983227728,-9344009.603851799,-4643952.228204215,-2505880.4259712175,972833.4276366215,-7002696.294639023,-2835712.062662786,-1961344.498900976,-10032602.562942648,-778192.0026538372,-5747325.462140575,-15753079.380286332,-6695494.884563914,-2557234.3702447265,-3334492.8125705924,3272821.2373622004,-4203951.589089984,-2476282.783851724,343933.32784586214,-2731888.980706718,-859548.4560518321,-1441161.8967004046,896866.8062488176,-4196975.729142036,-6351969.817314113,-4768728.906232415,-2886881.497448532,-2907109.765283264,6765454.837911757,-411706.22767372243,-1539132.0275256727,-5234520.761597246,-7492961.814051012,-186272.35278829187,-4125663.9266157877,-2919393.5700277686,5990051.084915174,-7222790.015330752,1561471.2351615895,-8142830.027682528,-147665.21423446387,-3653527.1658506505,-1042725.2637652624,-6468593.885602763,-4380193.8171883,-873128.4947493616,-1712462.9838520773,-4398611.640297135,-2687287.519084545,-2162273.662756808,-7228590.312219675,-5335362.639379738,-5785983.8310131505,-3142953.1184171587,3773446.1907140873,2524461.982505575,-2455113.970985811,-3357412.1658716816,-8357969.617488034,-5606789.942812579,-6081180.640885539,-3965067.685911486,-7653085.008642241,201470.48624289595,-1397409.7194484975,-3713841.4362046104,-708469.382637715,-5439495.396424575,-4355008.678605255,2552520.235035127,483389.47324347124,-4171563.978419669,-8068970.561804475,-4679392.020113973,-6244212.819514338,-1835786.2480455507,-1794355.93794916,5263058.714503916,-2713086.251534933,-5382400.141511941,1504465.0262972657,-1502524.8504471816,751303.2377801873,-4447125.984578185,-4382921.741892707,-2924239.37272509,-4271434.897460915,-2175197.5040723253,-8079085.107873585,-7021279.729692088,-4361980.732934015,-4949321.345313828,-3480235.8864811864,4164507.147617558,-10660073.61323127,-4870215.580363676,-6545496.213300543,-4581139.971171757,-2860797.6118905675,-744660.7196260095,-2851629.7442120295,-2638295.691575654,-2223602.289287837,-6255236.234293342,-3811975.428635109,-2954005.80085925,-6082361.480675768,3354426.6830735356,-3243379.8911045548,1661313.8217370994,-3254208.6722917035,-3044089.9373717774,-3656836.0807588045,-971733.7546660062,-3435502.59942979,-7273226.629725401,-10198039.409810929,-5240453.196672614,-2498436.1549139097,-3702423.5278930236,-4371195.748387549,-6415711.435147122,-1648420.6016953029,-8050644.378067149,-8999888.625272162,-3710553.046745673,-3766495.3399691163,-775735.2070448454,-5353448.684143979,-2582922.0257772096,-3744628.8613777664,-515989.83469522,-3650710.919465054,981611.9526765365,-2346488.0986553337,-3724360.1585109383,-1629887.6546120625,-2579400.8518761564,-667094.0278688706,-851511.190499682,-3831385.96352287,-6028579.996255202,-1922893.1585959252,-5631666.589104721,-7320499.934640033,-316257.675794784,-12814674.351574957,-718762.054110188,-3728944.4960451704,-10485528.82575785,-3248598.8475429546,-6985580.56094409,-2635863.574037928,-2927461.4986453857,-9919641.854106009,-2193019.934804825,-1664014.7656540954,-6772743.910285041,-3866762.6873730905,-2221062.9924033843,-2783766.160115104,-5686119.235679194,-5843154.927614011,-5014885.01663414,-3470769.363530319,-4142744.1469403915,-6194487.879091261,-2367593.1096288282,-5967665.478549628,-379590.1894839164,-8380143.430533311,-553466.0686433725,-5112187.938017355,-1160419.126909228,-1182816.899538543,1405051.0371752288,-4648265.700876921,-3667664.794679437,-3891160.1024329225,-9213395.888604175,2076589.7937870156,1309790.1762582771,-337044.0423874613,-1581396.6874147449,-3711671.351167323,-3511084.8917610776,-4046280.1577893402,-5580477.9471010575,-5918357.148857476,-5211100.784821374,-1825389.968947444,-2605867.1825945303,-7417286.633559383,-4431847.48219019,-5113259.555599353,-1833988.9957602806,-3904577.2248807196,-2523780.072007425,-567364.5919264555,3212362.876703144,-66735.01868472807,-421889.80829930305,-289046.7076408323,2649788.797420754,-3392164.086292874,-7643695.83859331,-1975377.3000514023,-9890176.398109466,-11427269.624025183,-1955942.9924511397,-3943379.7201224566,-5773832.686143011,-4142908.1751641445,1645455.357084928,-3738805.31898576,-3087624.3573871907,5161978.4954968,-3037120.0951173203,2069399.00000773,-3513165.408228254,-2514620.4619692992,-5029058.532836381,-889285.942467574,-2666702.750634798,-7837139.396666357,-3887533.8513762094,-573963.1841640528,-6001998.879001096,-3995202.0214791484,-4491030.169083104,-21753.63469075039,-4540400.511055621,-6835617.453906909,-3707517.879577117,-7812602.358588638,-10036876.913983319,-6112880.839758076,-8606521.955924092,-4823792.979120197,-2498143.262255037,-4399179.706232455,-5144625.412615897,-5331834.623431804,-3265041.7039938085,-449695.3835412022,-5947663.667308972,-3879767.989327535,-7076053.947205411,-10717868.152656598,-3972546.794142754,-2443689.8280222453,2539304.8378360756,-2910428.4548077956,-1871019.1790503263,-6604251.541172302,-6124604.960967161,-5931634.666342282,-7454606.335860082,1804239.3338171989,-6275504.692403264,-5525698.274378689,-5280544.818186527,-4541568.55755949,-10900693.244436841,-5881377.034265048,-4942456.514521418,-3651786.234213013,4905262.750866098,207890.61283602752,-13740677.834130218,-6841264.29451251,-2757668.0633360725,-112035.08199729957,-6346768.076513179,-6579670.808738697,868338.9904355705,-1592474.724374866,-5633690.532229828,-10824417.181584561,1791149.5645032711,-253479.60081103444,-1495727.041195549,-6162432.484585157,-201803.63802289777,151546.5961851757,-5750966.122233806,-4046299.363155747,-2218588.9743000716,-9754442.512434604,-2704440.7446560375,-9486894.285802571,-6147002.041894074,-551075.2045398653,644293.2281215955,-3547862.2458354374,-6530733.041377541,-2422683.985175196,-2590474.0641572233,-3616676.92185086,-7007681.974501838,3290896.685946038,-9988346.513721397,-5494978.905169962,-3378723.558428293,1810731.3627494518,-285187.0262578428,-4275582.511589821,-4243594.203929683,-1974127.5323682725,-3095857.786628142,-5100637.319368616,-2259517.4111972563,-79516.12691550143,-4820064.771573996,-158015.2637838144,-4012485.02229953,-3524110.89298141,2886207.581040142,4521892.1807828825,-354222.1757639777,-4238303.176506206,-8401565.400640175,-5907483.617430022,-7327485.078656612,-5731512.671891438,-4251944.466697466,162755.86685169674,-766527.5723539963,-4981655.891950959,-2567649.033288587,-8599542.066183334,-2990640.497472519,445215.3822163474,-9748739.87854645,-8726699.035170607,-3827294.1413245127,805494.2215956151,-5292622.455203876,-3535538.9564899113,-3620456.505322326,-5271560.30294333,-2489668.372195678,-7580995.8617123645,-11213587.694476116,-4249772.638407435,3241166.925640924,834430.0781338252,-4687376.040295621,-8629112.365309177,-67363.85812752508,-6799181.684255181,-4168429.03398774,-6767873.42542031,-1646703.156483503,-3914129.775945464,-7240211.619512357,-3836152.719717091,-7139418.091721658,-6656755.51386252,-8115586.681096045,-1735902.4642005116,-7124944.7590416465,-9935934.25821058,-7159359.095478451,3473382.241487611,-4288784.084620735,1344145.7637244891,-4246412.343093235,-3880616.1921753576,-10005305.930500982,-6044724.490527483,-4759424.461718952,-4554107.090187922,-627978.760096861,-5212507.71949204,-826978.9296881296,-7404350.873051383,-5008022.766558262,836702.8723603021,-2327390.812341688,-5456556.602885427,-2890681.902267807,2566553.4377726875,-1371948.746978363,-3391546.0472051464,-404165.6298532281,-118096.41499807965,-6495700.064397434,-6328172.941634124,1642912.254096968,-4577120.232496638,405711.9746559728,-4231685.058886915,-7335799.573295757,4662893.840052603,-5757048.67314975,-4860115.706168359,-2430076.755498685,-7576965.426776867,-5048866.796691336,-6312522.469644364,-2462759.67376237,-893460.0419617575,-12243305.991188064,-1711703.9615071397,-2244984.079943169,-5851419.779162239,-5615927.552580655,-5835406.594479546,-2660727.4010229167,-9119682.89180894,-6417245.821071738,-10822278.587957282,-8495539.978875136,-1133584.7287970725,-11556196.556336518,-926167.3861382995,252385.78253473528,-6550360.49842088,-7887738.662233254,-4876236.83625116,-2278875.1672836,-4440393.71862475,-7188974.588085741,-1602278.0087814834,-990166.0083568394,-5150661.539309646,-7587025.593057118,-1137534.754503781,1104877.796180077,-3448189.6453229617,-9022457.08577975,-2297089.583291171,-4968166.382755879,-617492.6172191016,-6509401.020175511,4858102.123856951,-4100375.951046774,-8427378.532469587,-4707876.4117799625,-8737432.873358767,-4672670.5881275255,-1396173.4934647735,-5626174.173869038,-5556038.1224786285,-7345867.441550268,-4314579.357440801,-6415044.97214289,-2622232.648970738,-4541434.54049575,-3185522.922850227,-5633859.101347968,-807592.4237333089,-5695025.559313346,-5799066.804269902,-770939.0749897435,-1521.3351248912513,-5567532.3184865955,-5481547.808618739,-5162723.354292588,-7481835.073585488,2190957.627030963,-36812.026719363406,-8042666.340836637,-8577070.058036137,-5944495.512009241,-2458908.8122282512,-2550188.344458172,-10126125.048070917,-4266920.057447152,3186774.424188194,2246256.4159748964,-2341255.9229218718,-12170607.753595483,-2503259.566274287,-7785867.463179756,-2813161.575897591,-6369332.341462426,2832593.8862483706,-4721041.012135241,-4947059.521706084,-1247199.616643425,-2896320.3070664164,-9452881.627576571,-3095546.2259209696,-4249282.2783819325,-7232963.608155839,2734921.9280048795,-350686.3982404005,-4031569.0351104084,-5590087.693906464,-2387127.9341876227,-1560057.2131520137,-2022585.5504368376,-9744213.1338315,-8863725.10983469,-2531097.7007042244,-3778177.7291277163,-2599480.1456985865,-2091063.9407476187,-527602.435160378,-1287174.013255477,-4272450.363379884,-1573424.139493592,-5334856.391020309,2780895.8101323396,-3644151.5643466003,-5501128.2279269155,3911774.189828491,-11761537.83184485,-2380812.367060665,-5751237.556398263,-5232135.635309031,-8739837.539539145,-4936683.379247654,-3322863.861892801,-2555215.4769006222,-2700811.8084542397,-296054.7520060763,-4944761.861150051,-3252298.80182188,748100.5186556522,-2611610.7064720932,-4335989.089290006,709178.4228018951,1459583.0777514465,-8187854.859927598,-3594847.0332592074,-874084.5369025804,2676289.0333634187,-1712792.5897289068,4405005.669939306,-3733859.2514353115,-50481.64671991579,3833184.862071475,-8524826.186265117,-2976757.78310387,-3730738.6711451933,3338305.460284207,-4262813.120511978,-4954189.553810589,-6902851.923485003,-2199914.074767543,-9378004.731862925,-2890277.385363478,2339710.941184886,4357145.003804281,-6849186.888049655,-3398266.0683437176,4881043.352940254,-9755830.512827698,-5410615.403000541,-2795836.8436135743,-2582634.2449229285,-6385076.631867858,-2510144.408158364,-3404032.0515170377,-4292567.12266727,-6194492.550798982,-2430985.4712157696,-8622103.797064805,-5968660.13203918,-3247920.2999269627,-141489.985467704,-3427627.133782477,-7544525.172805499,-9437422.58964726,-4679561.778811236,-4605319.530705979,-3818337.756503109,-1852415.556812319,-513658.01223339513,-223203.36040650308,-11117037.83248607,-6360160.981979514,-5372634.564205533,1556725.9463582877,-5842545.601273648,-4404981.783860019,-2941230.9865169246,686213.3438712321,-2710710.4739029687,-9976423.737705305,-7694032.209041227,2417731.0779637806,-1704727.9334624372,98943.74907804932,-5138594.770409238,-902591.3608947117,-3072075.438074176,-4335080.777428776,-763618.1186685022,-5322952.637100811,-8599081.10611159,-2691454.3766772263,-5948249.44095435,-2411858.186090201,-3550433.3540145867,-3772710.862646267,-6657567.365114382,-2025176.2709987424,-7961588.09260672,664803.1047010701,-1847888.4243587572,-174336.87129778042,-2740038.776747538,-4475874.34187744,-9352661.669098327,-7006177.717121387,1905620.6459634937,-3623583.4768255875,1067865.6288403906,-2923852.551440157,-7270250.6634441605,-4437048.258735577,493294.2493664231,-3505142.05896136,-1696094.587165244,-5932230.454165788,-2066189.7039369307,-3936561.746135203,508525.1197401881,-485200.5711284727,-1879218.440694496,-5616957.30330245,-6474272.259946387,-4132413.5843359623,-4363257.238454912,-2136329.0771088637,-4592188.713681793,-3554652.6395677906,2607900.804749163,-2750921.283285765,-6633495.492426349,-3019963.1237511486,849245.8234628402,2427992.997828016,-7502874.2333414685,-5120845.482547898,-4938251.172113432,-2598465.2050200235,-6857839.899011185,-1528274.4058764689,-3493497.9205351695,-1329263.6762933284,-2470654.981605852,-3729710.360757999,-5573460.842551965,-5420245.8416802455,-3730273.1571438992,-4277971.6706618555,980744.006268872,-3821116.3597159265,-3318216.5554823056,-7251270.964153022,-3013029.42516521,-5736220.083223604,-5097604.352028867,-4670375.757328117,-7088366.027204145,-722226.9077127203,-4281015.408777567,-4510464.736221025,-1889175.9410042856,-7013694.842451146,2494935.7718336824,480522.9064909816,-3949185.9491380714,-4271334.07801695,-3652529.2364632357,-6494905.195252629,-4351004.633326136,-1880132.557788629,-5857672.522072289,-9051970.942763994,-4103140.679310565,-1500829.482541969,-6939079.286129592,-1502972.0895758383,-840137.8946075607,-3013650.1599100027,-31103.251645291224,-9163579.11602991,-6211268.821324382,-6337205.645846374,-1553769.1393559147,-5101373.689876171,3891907.770193778,-2869437.246284524,-5043794.374901127,-6988797.575663079,-1213856.0852264464,-5348143.679526964,2540967.5276120026,605299.2847469542,-3270490.103692606,-1372975.604073882,-9592449.96321394,299366.24391296785,-8266747.552225117,-1377686.1291737128,-865697.8128248658,-5867.296710377559,-1129437.69653132,-1226105.610858092,-4164689.9583225343,-578567.1221985687,246777.24036708847,-9622173.411574833,850921.752369836,-7078357.478042083,-1798677.0073345024,-2270116.661069408,-6578685.326963458,-1324388.5744641349,-7723453.599996267,-7282412.7450791,-6099591.545588873,-7826357.649783505,-2666190.3363992553,-10039550.252267655,-5774376.833115231,1732387.430948995,-3009677.121426353,-11709367.583483038,-18302.887610422447,-8659506.894780297,-4188661.9135004375,-4080255.374666998,-7177430.354309119,1677987.7831036747,-9516821.649091914,-3780416.2317841575,-1651655.966883555,-2319769.8302676454,-11084089.965683978,-6727491.0118881725,-5210017.064844385,-1340198.7587752547,227836.289809661,114521.81503589638,-5674353.570411865,-6551478.265787151,-6297904.653902931,-2919673.431333972,-6558972.493786106,-2147751.2109117284,-1043345.8759929426,-13046660.800198408,248252.48366279155,-1819020.546165038,-5711797.043167546,-2680297.913459828,1369739.3991946857,77172.08183162846,-1679416.7734662741,-85765.28306878172,-983191.2963005155,-8623663.590641895,-2335660.842502238,-4300548.1547409315,563922.4169515222,-4528679.501140701,-5711726.113410799,-2478775.1034786804,-335926.64275575243,-3090894.1780293137,-5364665.190875491,-807770.1486944575,-7416439.089771537,2507577.4704545047,-12810709.265177226,-2152439.914130451,1607581.2352643684,-5150104.966899637,-6534438.044524627,-5640597.003623681,-8029403.330948696,-3058656.063066529,-4914801.00068889,-2613690.4771228433,-4129688.1430536304,-3578227.4758451153,-4041288.4279795047,-3462338.6769368574,-2834266.0456839055,-1232454.7437348422,-7867101.6784131825,-3672497.0511320867,-2445863.45013698,-5720913.014164135,-2666340.8346969057,3212650.7547351923,139307.31217773445,-8632145.19765001,-6813529.186675303,-2708634.5488504358,1234079.602489098,-2168055.9200253002,3257641.0437841285,-1326044.2912807781,-4437246.577553762,-3268198.70098589,1417795.410658054,-7553308.185792837,-4658827.094982091,-3521600.1285476554,-964246.9092159942,-8049187.9802032085,-1980415.9260975402,-2406761.330551496,-7843267.944871457,-10078845.838648424,3956940.910735203,-8651210.733165462,2389412.0711210314,-1821933.3312116656,-5206589.864536935,-13392620.616389642,-4317363.945095701,3495644.9614263587,-2327974.1554477047,-6204623.274912007,146248.85048504546,-2933312.2354522794,-11269270.286004648,-8985830.421278682,-570374.0158533882,-1548348.093253186,-3113940.5144604966,-625185.5941178184,-12335865.437029798,-1922746.790399244,-6229980.148464901,-4624080.0510212,-9230886.204025261,-9912292.525342662,-3158089.2712930497,-2820666.164469026,-1453329.77096292,-7319979.005397961,-43187.56824169494,-4268561.954517163,4350280.708774928,-9163893.98691874,-457735.1286123693,-1360443.7509699836,26962.006556402892,622989.4628798962,-3110538.209513219,-1074227.1367956884,-6069056.668167822,-2248665.16863947,-6098234.79650769,-2784222.661557669,-3055672.140718654,-4754294.055162311,-1424842.7835388668,-7346923.015322076,-2247306.8959273435,1107408.305547312,-4088559.1918752715,-2908673.4790636506,-6603773.828222539,2500971.698169263,-7759418.801768355,837380.7780661415,-5509830.4247846175,-5596344.794815094,-4243632.588403877,-6578519.751902521,-373972.40627112426,-4842675.540477559,2669866.5165080875,-1462176.2570666745,-1793318.8072111532,-3914584.006381847,-252404.75262352824,-3174089.789591724,-3459201.9134727214,167789.83947279677,784719.6963252164,1564552.384125391,-4339749.681726084,-1867699.4177322667,-4331588.2650448885,-7637045.572985845,-98314.47917180322,-7879687.576615252,-4739378.591644067,-8670268.109632578,-8096915.408570655,-7467195.909058623,4510466.3283316735,290108.6135517508,-3161727.4375225846,-2683798.5611366797,-5822012.990523113,-6470139.526539422,-1101430.7904666401,2486169.986142855,-1750834.5432574693,1052241.4409756884,-2470855.2649640013,-2936923.4393682517,-3342731.7725630235,-1167208.2907316554,-8341088.269132657,-2114936.3971792404,-5022084.22156053,-2556932.746922964,-317409.17984243855,-5306136.628259628,-10408346.80366114,-8483942.544693347,-4698848.620461477,-3306274.012166202,-1612495.5815646593,-8708448.991049778,-6251133.712402692,-1266624.681842154,-8409117.908149166,-3470081.0296882484,-7489057.539077632,-3732406.1018841118,-6037036.344457756,-3826057.8149919305,-7837266.846512457,-6208812.133819457,-1803711.1071422491,-2026465.9112654831,-4095287.135394197,411584.9073826205,899868.0682608783,171272.4016788695,-1409175.7853860091,2066469.014902044,-7440012.679113315,-6139609.876371544,-3633395.9404625334,326951.1892179344,-8865320.27741708,-6687878.040375242,-1784329.3473420553,-10745305.255202152,-11085953.14998491,-7800458.769720715,213153.00471088104,-6135981.341713171,-2438824.710874466,-9247336.737335263,-6981395.307787055,-7522036.565037509,-8421888.466629026,-5137334.300150927,-2000652.000271177,-4140228.0844216626,-4264783.961960541,672323.4133556597,-3910728.681051112,-1313837.6023201663,616499.6736116465,-8284447.319130899,-7349492.0672142245,-6423334.428057551,-8115389.639184816,-923287.9277969766,-173748.46810380556,-1505577.0448184088,-7421451.4041294865,-2742650.311444789,-5221111.752692221,-2968067.886730196,-2588860.617734289,-4775746.644978572,-5413148.717570933,-2187778.119160507,-8673922.224018242,-2780421.5969210416,-8893686.490949422,1062493.9948673975,-5572503.538249088,-1283022.4041039944,-255432.76115692034,885565.5055036526,-7241431.150012046,-12270825.624928927,416270.19710569084,-5945511.29990346,-9952302.501860933,-4815576.668718774,-5703806.174094481,-2478448.066161623,-5838753.408983683,1294991.677987907,-7354824.571061086,-4814713.259594293,1553639.5450596195,-1582776.5560620986,159778.02754844725,-9601729.157803755,-9872260.111489682,-4029837.6505312193,-749684.5802255645,-4210445.830136189,-4547792.7770960275,-1047231.2987981681,-2853766.19988871,-5890990.099765763,-4566597.640081378,-12255828.103758685,87473.21982068382,-6705700.065810317,-806873.2814037548,-4542035.2974388655,-3998574.538440341,-2995748.1013519764,-4764697.875714563,-8975658.631283237,-879607.9965095147,-3953377.5898824064,-369416.1719457656,516331.1295097079,-2374351.626769487,-4128975.157538779,5659811.478654072,-1707849.0207886882,-809332.8676269874,-5031439.346927356,1293374.6034049913,-2373091.916098576,-5440584.488298151,-5833442.150127875,-7925095.746871363,-5619310.602744414,-802040.7041179631,-4255848.369682396,-3528020.1106348615,-5611815.358117596,-7033368.182960939,274727.57077114284,-2769180.265756894,-9317729.434993159,-5304038.726670737,-8098348.336921526,-3565467.6545698876,-8425327.440339282,-1534656.612404516,-3480192.2068514246,-7062516.472294984,-7423957.503636165,-1528308.200890148,-571549.5821349081,-5200721.535527151,-4577851.88351729,-410311.4214543104,-3167825.8583051134,-2406496.547925791,-10157030.75538918,-6916949.756009321,-3707399.9123580195,-5138419.3585387785,-9275345.673435539,-4068976.766842339,-9661665.773269225,-13575564.12808193,278410.90399633534,20411.5212869104,-4034290.641247295,-2171382.769986907,-5558080.495130878,-17571.83052795194,-3493074.160627056,-12216299.68286007,-1151635.9532071333,1084426.3764618263,-3600522.0171537045,-8016758.3810376525,-971100.0167173669,-7879420.221601605,-922476.5563443359,-5368871.784445399,-6014563.277903089,342440.73089239,-1112197.3568449356,-7982176.451418849,-762022.2543966137,-1559264.9306096304,1817402.4152074885,694748.4529814776,-10492933.391354522,-10544949.758451229,-1287424.5175159127,-4870154.477414088,1721195.9466232192,-5455374.207278131,-3650737.0327677764,-7276548.89422185,-1440089.6072375756,-6597730.598942703,-6174660.172483973,-4782921.006946778,-7060892.776370661,-6209529.315295551,1205771.2788032992,-6978150.871078059,-479040.94140738994,-3515713.106496392,-5364055.772462517,-1173245.8674780466,-3129657.8348366264,-46192.23067913577,-4600619.956604809,111377.82730597071,4210769.670027953,-6359911.539904855,-823955.3271914795,-11818.707180839032,-310991.1315648351,2372547.1967181843,-4232860.090535095,-3549798.9692283496,-7450045.250071969,-657867.0996822184,-7816205.095460313,2955915.964754477,-4353796.347007763,-3663935.0063319877,-6029560.294067521,-2127972.681576468,4005071.893146841,-4648905.2900465485,-3609069.31225002,-2519224.4828574974,-3376568.185269313,-4613974.949261276,-3141436.2284667883,-3903583.316948384,-9506190.640542567,-1529270.4566703942,91.70943682454526,-4151350.5121110715,-2798620.983735826,-3024.0955878049135,-1501244.0529180625,-7707790.105136147,252325.22994477488,-8192816.454464849,-2785186.6343675256,-1688377.2966188043,-3425155.805645749,-7887179.522683933,-6463013.761984471,-5429044.367410189,-6757940.731769274,-1882568.2565192934,-4972030.438762955,-1598243.9449801072,-8330369.305613501,-5652991.823726531,-6802600.921773378,-3989525.0933283586,-4068176.4632193036,-6285428.066604942,-537605.1112037655,-3331161.6575179137,-6178311.733822219,-2164354.011169104,-2452609.1790983044,-5119508.563479213,-594114.7069221623,-1597088.759620849,-7960047.30503628,2487230.364516629,-303749.19186030887,-2206027.4192095995,-2851761.3041147124,-856285.6636833008,1164049.2486987635,-5571732.227643376,3575715.132801594,-5493586.056996016,-498281.60191848315,-706325.6437573712,-9150295.583741382,-1231947.696914075,-6344176.112518484,-2721043.477153901,-2454598.4330136087,-6485510.999073604,3422156.522598289,1453027.9239349887,-4402353.944008706,-8356606.473893996,-10046040.905105406,-991896.8872347064,-3228479.901910262,2132858.253846377,-2095649.1753973905,-4042370.774774654,-451328.03034708276,-4886317.539941544,-1440752.507198304,-4073115.4259202685,-3120571.145631423,-7092616.549663153,-4550756.896300061,-9542256.88108536,-1683130.962286869,-3970527.6070393957,-6391807.2930823825,-7204462.3419180475,-7746662.581013823,-1624972.3987220395,-2118983.0652502365,-7363118.618142376,-1146785.1263595019,-4967095.733428132,-1011207.9339666385,1701454.5526777431,-1395219.9551338013,-6129054.247684438,-7711052.934247676,-1662277.060250694,-3154839.0337506365,-3525425.705034435,-1513133.4237628626,-5195196.02137804,-7303560.8180418145,-5655580.273477549,-3178424.7916275533,292700.6796677485,-4509670.808128532,-588168.5056135692,383798.5742323771,-3636516.255294474,-4500668.458756586,-10320406.9248697,-1610215.1049583629,2212968.1468948666,-1439141.953453891,-417312.1244329363,-3675965.152204238,-4855135.400467472,-6266643.2357734665,-7814094.39712763,-3455498.9696281217,-2791088.275025476,-10907411.602605104,-1355717.1538792271,-1286297.558560377,-2422349.0221386533,-4662323.500616435,-5545026.603550717,-2607295.649070397,-729936.7554876916,-8450932.890985446,-4431770.327753259,-5866847.003728282,-3918049.437463101,-10020322.040838938,1285550.6876436286,-6208398.865241645,1717483.423683066,-2245429.9416000023,-3273803.5808406267,2818318.217512721,-2064163.5567185767,-3500859.088897044,-82065.6481450852,-7509169.034981145,-2771965.444524357,-5607160.967512034,-7478258.607962502,2683967.4974781787,-5586819.63325518,-4987376.337860129,413913.81367893796,-1894115.9920247253,-8081918.230502034,-358574.48459675536,1437568.8160018977,-5482703.544995256,-1141719.898213055,2410967.4173472393,-4661659.457882896,-5632297.4955555145,-3182450.164661426,-961284.3432549909,-4246485.607730162,-10842028.089020653,-4333726.21231319,-9589130.395071378,-7331380.649065008,4605002.237062888,-240897.44983833376,3413575.2826182134,-3726549.6235503256,-4911621.094225535,-6559170.911813553,-5267545.102905341,-1689923.2176526114,-5878279.095283711,-6144415.441750284,-3503971.49671302,-3156949.547024548,-8310756.8673058,-2259467.6164161973,1515118.558577572,-990586.1154187117,-3270611.2403808497,139663.34079082683,-5784682.440021098,-4195268.266335996,1612380.8726508636,-4502926.876753103,2987354.227631688,-5685176.468518417,-1067683.207846092,-4352201.587978434,-6292481.392326269,-3386205.1297376193,-5080826.367717044,1882349.8713222872,-8044845.961823359,-644892.526812464,3478552.517508641,-3596407.522330815,-2958882.320064647,-776284.1945492439,-2252776.060174525,-3683190.156575878,-4858591.17442812,-7944488.412447395,-3967275.834834813,980937.1497439295,4693901.121639701,-4205266.736763591,-2046596.8639957476,-244419.22167936526,-2125156.7569742855,-163656.19210264087,-5264966.90475753,-503329.0873226058,-8585936.183657676,-7230656.54953834,-5106569.610481542,-1981438.13434021,209573.85263761133,-4674097.559860319,82578.12672167271,1443484.7320475206,-11778848.06038664,-5680538.363909508,1286595.3267683033,-2012228.9061174504,-7236692.314893551,-9800608.913253985,-8150779.349572523,-404598.67267942056,-1601446.5768766347,-2573737.541729194,-1659260.8659409732,-6441401.947664514,2058324.9663932994,-5693117.28862487,1483298.0927514117,1118338.598313719,-1276586.8489503488,25263.959694979712,-2389299.1553696487,-8388598.800732981,734410.8816982601,-4168404.2035124013,-4448936.541927634,1487356.912724506,-4441233.789448837,-2295567.350713797,1324682.6117380615,-2117506.216526706,-7015018.567336451,1811969.941777993,-4658070.848707363,813320.6399126146,-1344550.430930459,-3698137.414309008,-879220.6177411638,-1793367.9866607934,-4734597.285906646,-8211284.107865658,-7051386.508396468,-7455513.761999834,-4881112.092799146,2514443.9040723946,-5050809.79165783,-2927500.865280701,-4809276.318450881,-1539443.6938080061,-2126760.5349627957,6486857.947119392,1777835.9693052769,-10455429.16033072,-2591144.145916138,805599.632722808,-4722936.046628382,-1458416.4314188007,-7827357.435467064,-2240795.2186687756,-5202004.993425772,-2636986.519685203,-9154365.112407118,-6783929.508682575,-178408.98115610518,-3986126.697974447,-5159565.912877707,-3000550.9135004077,-2384643.382091243,-3597615.5420960505,-6141756.144899262,-4529125.0361809805,-6644515.324452341,-6140852.215664983,3028584.590205835,-5217323.523217981,-5163240.2799972,2731399.632220843,-1347396.9382144865,-2775449.094761722,-5401601.8041558,-11314663.47373965,1597071.6241268087,1503360.8321669362,-4506091.2685648445,-3142898.3480271865,-2861931.329889618,-9998819.940283602,-5536972.216549752,-2033725.5647019017,24454.304045198485,2541693.3327246923,-720634.3409124538,-3600144.00779731,-865373.4907645751,-3876775.6618133215,-6496970.607101046,-3161246.1883295793,-2612010.468624819,2516570.1873701625,-3186483.565151222,1116246.4226677548,-1731077.2053807564,-2651979.6273530796,-5263967.062849417,1899510.8746382203,-5130550.429230519,-1736346.3614629004,-2440640.947326755,-5579082.610726008,-5296291.286507428,-2881995.016493896,-671674.9259183742,-6424347.9208121225,-850378.2096957788,-2145764.2207945324,-5693039.793282317,-3592757.3822419196,-5899666.150429944,-8815663.882212363,340170.47775843926,-6708358.362862263,-1295151.1848407686,-1514136.9093423914,-545972.963730298,-5959396.195317961,1340910.6687123459,-3482142.5280586947,-7379572.083414139,-8471804.399342258,-3230541.4840067495,423153.1958743483,-5974938.990960785,-3238867.8696557414,-5554609.709619742,-5151625.265139487,-6561561.11711378,-6693667.686015901,-8076290.439329602,-5222070.500324812,-2214202.833468968,-7692301.875773475,-8160570.688633311,-2752400.3322851993,1949767.7126739454,-7550663.390925266,-3960041.313084568,-7002477.685514822,-4487992.088794925,-431572.6870951019,-2826767.8240091708,-2575269.2098881844,-11585499.851555267,-4220206.701347709,2886839.4107099604,-2369400.6424585227,-9572419.584349804,-4894312.066872299,-10061136.146421473,-3092693.042936586,-7790224.585369568,-1903647.4565494861,-2315609.7369430494,-6127608.13670936,-2846076.130747102,-10192007.403912984,-4951728.870788168,-7916963.160647547,-3358771.358356244,-5354021.996147692,-1255741.7956802603,-5700261.4919098485,-6415178.3616843615,-3524824.299186047,-1687107.7298951317,-12909555.718645485,-264028.80533535033,-342724.58372089453,-1907212.3816929515,-516714.2430488337,-2547724.1472175773,-4198766.965270331,-3510715.7629566584,-6870083.252711568,-1279930.2889405116,-5226894.360792831,-1972609.5377866067,1108939.8315936495,991959.2517098635,-5658103.872494202,-1116462.951657474,-1799070.218383789,-3837634.381333912,-2007218.5786089674,-6141490.445318926,-7482582.709931131,-3680139.155299321,-2783807.144068923,-3825273.510101987,-5622525.802761337,624270.7981809545,-1586965.2354363408,-3426957.714912193,-9471577.243137456,1894755.4832832366,497838.4795019813,-8580252.955849398,173452.71949545853,-2071367.7810099237,612734.6432569642,-5756129.252252594,-615196.3292753026,-5439193.8608211875,-7830755.3813797925,-6000885.532180669,-8880117.193392096,-986185.5154945664,-3487515.725101645,-8412837.171705978,-1185461.2916481812,-2545206.3504216634,-7363265.478317397,-4943327.301906675,-8209980.049200737,-3151911.054661466,-6884323.399152292,-3493611.9817539807,-6181274.610390676,336757.73752111197,-12621016.234330121,-4241598.04215722,149413.81939480826,-4816538.355246246,-1124463.6166448593,-1787268.5269495994,-3647133.8979095453,-3034307.25393698,-5220613.191722238,748117.7380264718,-3974538.6972042806,-1747359.057786055,-9645289.97928299,-4210853.259128064,-5344645.880860014,-1542115.5092418417,-1420283.8019334972,-3336333.2190277763,-5461218.345118381,-5113634.765536543,62230.529869617894,-3412087.0202626716,-5939246.508122485,-3275496.017235942,841710.8714608345,-4038982.42389524,-4557447.949836638,-3362963.649278691,-6098045.37967095,1093574.0162284859,-6907630.363918435,-5977312.00822128,-8426955.032255726,-3915190.3566975268,-260911.5200249944,76582.4696441926,-11320403.572503548,-3032431.3954786547,-3301990.854739923,-2581213.5411150698,-3070088.1315262616,-1570092.8756321631,-4174472.7577936454,560974.4659252651,-10570683.140271645,-3381269.8867582455,-3439672.9858766105,-2542535.2987128645,-6571554.190899132,-2139925.3960892353,208847.49298277497,-7150474.277414107,-4339971.736675665,-6476752.255256629,-2985627.039140051,-4220903.913035054,1740912.9337453842,-1263483.8326223474,4954953.146917203,-6533729.988104502,-1748964.8627935033,-111845.08469767496,-2310749.665825352,-3468675.378989268,-6968049.438993897,-3800707.7647174485,-8684729.924626576,-746791.7546024676,-6056950.257903593,-4875814.6314862,-686943.3442308661,-9730970.048877146,-12461152.973718613,-6080038.531577226,-3922272.3827173226,-6263080.725745747,-512120.5574935451,-109260.31856992655,-7753300.485465562,-436604.0517008249,-6705535.590764692,-2764099.4017568864,886485.1345645301,-1326331.3641838785,-232209.77382772043,-5605440.509009808,173929.3981286958,-4354095.0299780555,1741467.1415294874,-1501295.2691631336,-2989724.5089073833,-4389724.317042828,-7037206.185387168,-8725347.444723833,124482.34074714407,-7532405.866711091,-8860431.309949223,-1661208.426936593,427519.9280559402,-4239110.043194858,-8546687.182325998,189138.2209190149,-2660243.120758459,-4811261.508205149,-4631002.8123641275,-9696160.640473958,959422.8028603718,-723941.6925394237,-364394.8485189341,-6940343.610362349,-2848696.509705495,-3901106.774296783,-1740747.4574135467,-2022793.962120697,-2544978.0949912127,-6423300.153101984,-11022108.991046377,-5125696.96500962,-2824514.2997617237,1160565.7797437273,-3128454.737289466,1752943.0914809126,813027.3196011875,-3574790.7382514374,-4250510.593303541,1788184.3530316371,507151.5374401286,-14641016.486707862,-3860513.591034645,-4023392.284187369,-4881778.115152184,1986923.6783886217,-3689157.975127821,863908.6346507985,-868718.7853664253,-2110072.150249377,-7493145.662643457,-1790032.257853683,-6551918.142717618,-3649052.9103206433,2294587.6658824645,-3303493.0705892816,-3958296.639945686,-3548764.27531565,-4849015.901128579,-1059845.1122945603,-5906828.918456046,-1127694.648113003,-8757223.060379932,-970820.425809361,-1052619.2025479265,2194779.893560525,-5945158.310788812,-6902022.522790117,2251764.592485346,-4852815.563832883,1295261.26138377,-333655.3864943776,-4901796.4414571095,-5992343.856355386,-2167618.8728666995,-6794158.941663476,-2579306.537634732,-7149285.833408326,-5645087.632771511,-4762668.114781007,-3545072.664187275,440429.897757547,-1790123.0168906562,-2675907.9751785025,-3022215.42397123,-164508.16567310132,2110877.7504882403,-3532883.5928080883,2443098.6409510765,-5971242.393383117,-7665749.012978181,-5903328.898067925,-1107863.2057985738,-4210068.610566676,-137967.96143930778,-2966774.3729184717,-3686913.3053989355,-6594779.312242731,-2527958.956208337,-1139291.6328570154,-9944945.557172105,-3670281.834584633,-4583051.204422707,373201.9403155409,-2043837.7510065064,-2768890.6607465576,-5619014.667888112,-4870296.483609417,-3108611.0240910463,-4413117.6154735945,-7367217.157476785,-2540281.282340193,-6216838.055028234,-998360.5251806471,-6421109.714277688,2694722.0722548645,-8024283.054864045,-5515397.433216788,1633791.480649501,-659406.6056934651,-487876.20295543224,-2892224.621481348,-6794805.957059087,-4561831.11926339,-3913499.924439123,-6646896.244569449,4037717.2320997994,-6236751.124003025,-2894750.714123871,-1865960.093034789,-1949545.4047443904,-3878539.899496816,-5289165.636258081,-6144423.722971989,-2978126.951094387,2426731.267151298,-3775886.219797425,-1762569.8546728902,-6086100.642240511,-3668605.139827123,-11339641.445783462,-9277697.698500546,-6477742.719518244,-5271227.821532715,-3217395.86370535,-2728902.2239632662,731280.5295264199,506864.32224139944,-4880573.617652651,-6953503.538201745,-4797441.284766924,-4235969.666731648,-2550937.809718838,-1986577.0837251581,-619214.1785751823,-1554610.9203918334,-2359883.9439622555,-7123955.574435608,-3685359.1402834076,-1819067.2039161064,-8406775.298697788,-3313836.052000651,-10808505.25492646,-2408261.8171423245,-3336820.0422776416,-6841513.834751159,-4906679.145270832,-8836835.651391635,2118967.5732968114,-10217026.83539394,-1014998.1401612572,1676401.709823098,-2237973.758319269,1547335.3752857894,758773.1581082027,-5977607.050768692,-3749748.583772924,-1837611.226526402,-5352186.246259164,-1205188.4751980286,-3130327.7178789657,-4245700.818794914,-11986914.17879225,-3402757.4241911015,-1861763.7646050807,-4598970.629937256,-3230417.9740758,-7982173.608985014,703494.1227512676,-2952872.4622340854,-2926758.9193370156,-8417023.056413172,2079657.9639923833,3503512.7596131787,997906.5702622756,-251050.90282885171,-9259636.835431058,-1943452.2853885554,-2021463.9733018745,-1186931.5060561951,-859774.1644445956,-6283368.811196594,-3409381.1552546993,-4607457.344597951,-14327.708789166063,1591780.991750855,-2009867.753498897,-7632206.567657124,-3357243.394963557,-6037018.248769706,-1494664.6643918622,-4252205.741322324,-4577238.213478928,-5989195.62727659,-5122452.505589312,1620844.3871865105,500054.1432756316,-242313.83378043212,-8084118.820297467,-6606026.438163975,-1975798.828312654,-4196392.077395448,-579182.8740705978,4589154.773499016,-738778.917158898,-7689900.284623743,-3280074.984137834,-1471582.5206199922,-5111140.730469128,-1206580.1609955244,-1056847.889936002,-4720934.954271264,-3665231.5000054576,-6908580.428175719,-1268116.9579969253,-5985419.440085642,-10816837.196765732,-810883.8351814039,-856044.3598766495,-1162636.7827632315,-2807732.591089543,-8763307.19060369,-1499235.1304395236,-925443.6831176225,-6780501.394467095,-7360093.447416851,1533314.3940866757,611680.508231286,-1278950.8651201818,-2508641.5079352576,-7529402.309595092,-6477226.07319597,1692843.0972822662,-4587303.880653841,-6131316.416231664,-5241031.767175153,-3345542.496437641,-3361170.410973763,-144572.67350756377,-3442733.477002266,-7200971.466594094,-9483879.179345578,-4052985.253149813,-10835402.899832165,-3874932.8058869336,-3063530.6223028917,-5528208.692635337,2500230.3971554004,-8202258.953268901,-14266593.733740006,-2546209.2653667685,-13222676.482018016,-3880326.7650515046,2759529.722840516,-3169536.9990116674,-3466896.3315400593,-5193330.060117321,755015.027942868,519420.9839391578,-7645957.68286778,-5327682.653730484,-8140593.346068462,-10184285.874177307,-4535154.533288769,-6296253.929228811,659460.0551716052,-3599554.7252682447,3104278.6301723104,-5475521.074119836,-1290833.0530447103,-3218455.4907626566,-5760641.024138438,-3236593.3929185327,-9641346.41396346,-4295327.344339689,-4862672.9117727745,-1347881.7834713385,-5945986.777971895,-5817664.719689338,2112530.219043892,-3446985.7506614886,-3828572.0088841207,-9417421.921585482,-4778801.869712653,-2839595.8768808506,-5261950.137515213,-4986180.308589645,-9587143.748542488,-3904882.385946624,-3757840.61446774,103050.79426598549,-1194488.2600613367,-1366358.2569884788,-2560866.7829448804,-3672324.5822928175,-755264.5919400416,2248083.7190307677,-2483458.5132940635,-13312611.54711047,1401952.16031399,-47649.16082581878,-3742986.3690369697,3961383.5108877923,-3068992.285452852,-2033310.2927566692,-4841324.493062925,-3468097.934990637,1535325.7168745538,-1921548.5356875192,690926.5648939256,-2600801.9061426204,-2332728.5998709463,-2745396.749880528,-5709735.183654487,-9626099.33322148,-893913.8114773519,-1186400.8482278343,-7991884.151816634,5297644.839848086,-1948965.946565766,-355368.1315859705,-7886349.52946605,-2018471.4139738195,1196340.5687748287,-1179449.5147664305,-5059631.816200515,-4913136.187774563,-2010471.3131047674,-2515878.0426599793,-4025585.06376232,-7455105.599677814,-3005026.0407502484,-3971752.2156820092,-87305.00543873943,-4268329.159143459,-584009.888183333,-2573076.059128303,-2030764.1662941668,-6216976.404340189,1870650.2268977836,-3927009.5732990988,18124.954476159066,-5886515.763134157,-1019072.7897074856,-7387510.666519802,-2362195.3621037565,-3349335.2099701986,5128879.027908627,-8501479.751904428,-4127572.9768778887,-5713608.304855239,-859142.5542698354,153039.11244870722,-3822712.116453778,1722419.8588851541,-8229583.978937199,1592760.7354247048,-4732479.4253832735,-7991668.707920321,-5476512.773496268,-7908347.324285829,-3633167.8277324494,-9788902.70278336,-3835420.204673093,-361764.4112849161,-1495919.4438508134,-4212587.308515361,-1501326.167127546,-5694065.258384278,-11140057.610902186,-6295882.437196944,-12986246.473419696,-10102725.599198855,-6916242.53073868,-1234806.1983127892,-104891.36327913404,-2450012.2396017667,-1177278.2437883765,-3085519.3608628474,-4171243.3450510986,-5241476.266323504,-6081911.588257026,1257305.7574523222,-4822888.780936003,-1821841.5062007885,-4444972.732118037,3401874.3988671526,-2674978.393566055,-467994.5732277613,2215811.6502860375,-5332903.284985404,-102852.1346339602,985834.2730577216,176402.5860747043,-5196383.349467359,-7141510.373951198,-8411759.769850988,-9179048.087232871,2323005.925678864,-11178707.764814913,2599643.485784568,-2971165.5901584327,-2383200.4751208276,-3644181.027454052,-8561532.167985972,-5118258.2582091,-4929779.31191591,-3023801.708096901,-11873684.460234042,-3317080.9507402256,-4771370.269899093,-1071020.970644908,-1250017.1277045365,-6104247.876477397,-2706274.8321723314,-2096148.6518864613,-1153438.106605228,-3129212.4412745256,-3288702.2688811515,-2687831.2833078746,-6148302.724238547,-3010037.9032769557,-13295240.29965467,-3142884.6230701245,-1020905.3842691444,-395805.427222345,-5793205.802103549,-3848810.6178832706,-1197118.2737410031,-6925380.082451917,-3175244.80237063,-9205470.28114006,-6525204.855250152,-1675999.565826334,-873674.646305861,-6289362.608882593,-1004230.2415195666,-534535.0919359084,-8179166.259179022,-4519019.519961467,-5499218.235092279,-3609985.4402682222,-3352148.6048597936,-1588009.245946141,320004.7958482187,-5093119.032217773,-4958485.061781829,-4146104.361439597,-2061951.5323574655,-6719284.840399858,474880.59337121993,-5402649.188723149,-7746675.9965438,-1041876.5417654179,-990666.4667408392,-5890567.045560842,-6577564.701344306,-3411118.316861119,-8924503.341645608,-1187108.7821805198,-861175.9574405309,1525725.8452180028,65180.99029047787,-10109244.827385822,-2549629.409780778,-2166618.844935205,-3212908.50237347,-1013840.138345344,699740.2925241757,-2058928.8594528306,-1797426.7656990234,2065746.0179651473,1076806.1979131382,-2044745.553751098,-768225.5255749207,-2673552.883134691,-741427.6542833727,-1675020.203460522,-6294594.30775892,1459542.868950747,-3786483.0901277103,-3834546.8334159926,-4477075.388149325,-6705423.389919886,-2408875.442749258,-6475309.221133975,4685554.888721393,334190.50966835767,-3770018.779204445,493503.28923400305,-11873497.970818764,-11471340.194572989,-4220214.91258347,-3864549.3852066826,-2647298.854266949,-2462260.528201947,-404130.09041493945,-2227218.7289959453,-3645371.949766148,2145112.1120381076,-10684934.31077962,-1348259.9914049134,-5198912.805162376,-10466790.082056722,2702932.9921612088,-2477973.179547755,-2787894.2745390367,-859776.8170269337,-6560052.88723696,-940636.5190120619,-4793985.677580509,-3821771.698007729,-2896919.4311361425,-10070937.675133474,-6921773.402562205,-6110822.132337138,-4950723.873153806,713482.0878544897,-3772462.012861952,1105334.776405124,-9119626.052036501,-6533996.750429604,-7491172.028948726,-4046164.618208315,-5395876.29809943,618853.9310598914,673544.9323651958,-4166693.1300840285,-898129.3427954689,-3380971.737991115,-3752078.5390722696,1217573.5996109527,-3562819.796115538,-568745.5141752418,1063234.2681040782,-3941761.771193225,1599893.1040339414,-2828145.0228105504,-6519015.134170869,-5517604.325857097,-6830201.079861585,358454.3772693295,-2042660.7419205103,-611534.1983471867,-13686446.908859761,-10799980.499757778,-2288542.1884192005,-5961098.170994038,-5156817.444930224,-4394978.950664457,-7597786.885668933,-139982.55776410922,-1474765.1116974652,-9605844.60505021,-6385761.005094897,-633032.1649874169,-3595105.8788744044,-68817.32331652753,-3856129.4412411563,-5619821.15431384,-8896925.229452586,-9464525.601677874,-1467684.15792338,-3447646.4932026025,-7020399.750349526,552658.8919775933,669020.9586161654,-6132308.525659174,-6816187.599187402,666316.985711772,-2654267.451011814,-9984229.919564195,-5893894.958681055,-2332165.138269007,-5592366.396749966,-2974939.9616535176,-4238247.196447486,-4120847.996140154,-11355470.563442707,-5311877.579711175,-4010304.5056345146,-1902982.6464182641,-1827486.947066076,-2553419.017776465,477062.2789358534,-483835.7308231406,780854.0499575995,-6212813.92961528,-10941646.475636542,-5153922.704747859,-3512526.3842810877,-5026764.907956136,555307.0288397241,-3966362.905687781,-3031914.985702755,-6740642.566973131,-4044619.1458460987,-2223422.0589340758,-3685668.9574170504,-6203839.795565652,-3785726.464997383,-4321191.696723921,-6631490.191444129,-3398663.2541080974,-699760.8989661038,-6717358.827683004,-6569921.317526454,-2702549.3244740665,-2082330.3251962345,-9023924.957135659,-8176792.554402571,-7270774.027555905,-5758803.2594709825,-2059029.5063442737,-6722252.77274601,-1595775.4918691553,-4657398.715092337,866173.1696879659,-1788604.2679252066,-8768442.261138465,-1281459.9352490995,-3167808.705196131,-7234129.667920535,-5485294.262941938,-3990115.535951983,1166782.5949318707,-243507.23732789606,-7909466.200548561,951611.0588786304,-2639881.9834505543,-2262237.1338167097,-5186724.526437262,-8938369.538272293,-343252.0424270481,-6558072.001881288,-2591446.710746372,-7015888.667878872,-3822350.800334871,-1688005.5076051643,1587385.1402258053,-3844819.9761285055,1106908.7767359354,843481.3809030112,-744030.2277794667,-410552.2916226778,-2697422.9490225744,-4548607.550146604,1676919.1994209215,-717337.7345777862,-6317930.806311835,-4454452.91286643,-9101831.72837017,-9042912.010649782,-6161037.754772548,-7916287.92305216,-145024.1476680115,-622994.0507846437,-1367593.8557278495,-5846112.257361861,-10696027.22911594,-9400085.424888857,-680901.7450532727,-2337017.0949362908,880354.3283586949,3099845.132839486,-5147452.357766848,-5895773.975951853,-5552703.383288361,-7353219.976998436,-1407766.192543015,1510238.4098264351,-7576933.119723195,-3233391.841112571,-381478.64324386604,-7151361.344788041,2178266.681094693,1261010.4349987395,-6027330.293722864,-7830950.710334638,-579128.1567238569,-561548.0343762618,-3472549.879376959,-485159.12116319686,-3561773.235173501,-4083204.597218748,-6321715.627248518,-2213720.002597032,-1590340.058629755,-2165523.956612507,-8241233.896015365,-2122724.584282765,999297.9877549261,-5871868.489715651,-4530341.402205892,-6079391.795266053,-1907243.2060252242,-2886119.620217709,94617.66261104494,1461686.18592518,503088.47888003476,-2450560.8646387216,-5758866.187976468,1946101.5118555203,-1086165.8388999943,-8678620.289317694,-4009939.9973848276,-8683234.540438766,-1217384.129914621,-4823477.56684179,-2053034.5638485942,-7586590.195676956,5689365.009520635,-3072901.4760194253,117582.86617894843,112709.66208106466,-4740725.612726752,-2250504.0928242877,-1333559.3080941457,-850820.1426229402,-3127906.49633345,-2928766.104738362,-2360887.800051233,-1314777.8421896938,-1192094.5497589428,-5458533.044576863,-2414205.45233446,-10085505.416805627,-7679785.324717715,-4194142.531746803,4715231.139763683,-2982083.537054809,-8821528.049763553,-8741547.6227815,-8360053.969102846,-627653.4485568963,-5940767.139607355,-4039667.1156576425,-1105834.7596224044,-2326004.860941114,-3302212.7622047625,-84414.98676970974,-3227245.1555751637,-2325570.125755707,-5221150.587283947,5125963.187147882,-831955.5075521413,-10837732.720463078,-4706446.533229144,2350185.8289287426,-1426322.6394344363,-476949.4856183864,-5356323.141361095,-6242376.381531715,-710716.2173116952,-133521.64479792304,-3324150.6912095677,1020827.9554237053,-3795828.402276404,-2195339.2641402185,-2121620.7487352137,-1068799.9838522002,-5333401.852118876,-3159763.2123025972,-4814319.099949632,-4954400.147171672,-10420575.468089888,-7846801.142226137,-8542104.17641577,-1320181.6278255638,-3367902.6554531287,632910.0626487341,-4295380.557449736,-3771340.0946612395,-6201854.810318379,-801340.833452113,-6530207.453345263,2003758.079166172,-1971075.0538230073,1548096.9247052409,1598179.3966695983,-13096733.151800511,1111317.9671888016,-1388110.1855528317,-5054330.614947328,-3615930.157858232,-10062406.964572288,-4935849.841488909,-8479562.514139067,-8116022.260362672,-7649535.58780021,-2628638.1955542527,-4716701.875269195,-1837974.7452379297,-6673884.680490755,-11446392.111778509,-1329747.9342608303,732599.6287051924,-6083646.687569099,35040.13670006208,-2886496.479989186,-2872952.0932387747,2694019.6103536133,-884954.1098465044,-5720297.386639066,-4994829.091685358,-2060603.4333429486,-1618281.6534201186,-7824869.617362545,-4455214.69852804,-9430048.546436932,-4653801.8175343275,-3489374.371855313,-8854667.890668496,-7477100.534137934,-6430423.062285695,442160.95701077953,-4318434.803049423,-5433911.169947857,-3974274.4326940123,-1753607.1609342564,-1534044.6160909552,-1520187.758492466,-1253739.1163826156,2032190.9966802746,-5997515.402516572,-3597186.196363151,1320322.1674619727,-2228325.235140994,654021.1463760324,-2179220.4091370497,954016.3405662049,1171947.7405535784,-5634044.810355589,-1125479.067072846,-450220.42370171007,-1012030.0693791378,-5060393.800707949,-1727082.4332631156,-2547835.6807915587,-4030533.1988507193,-5586252.532335253,-5004384.46544832,-3050432.2887577936,-8547473.512376783,1780965.6960833445,-988323.2394213639,-6321243.036599169,-5839859.810007743,-3170200.2254297677,937690.4423734676,-6962644.800096918,2664933.4188527763,-1086459.6838870049,-11211940.658634715,-4094493.9543981776,-9548794.667469043,-8267795.09463452,1130550.689882623,1649649.9201797321,1075869.76888334,-8891725.036480809,-3866274.798602905,-2938171.6100036204,-7968819.336104311,1256211.8355985638,-849779.9249208961,-3065821.8840682264,-10643852.20227654,-5842502.031863222,-7074401.547558349,-8041801.708314834,-2237318.463907378,-2158235.133031236,-5009331.7715307735,-3092825.451023886,-2314735.946180351,-7171114.449659742,-3950781.6519806273,-3374176.7688762834,-2592791.0299119838,-8489996.351085022,-568064.8737837169,-3973970.4844510797,923542.9767283071,-3459814.2862929665,-2288254.4360837433,-1459424.5792087838,-3086346.4335816745,-4977619.125413986,5312710.176630005,-2955676.7735915016,-341443.28372797556,-6237082.755018564,-3464909.8481627274,-8573471.442735638,1485180.4880459327,-967551.4780084509,-3378597.655255029,-10135242.893461294,3934611.493658155,-1607070.194915697,-845541.5410712231,-1380227.5087697394,-4726413.526193142,483403.5886381436,-2033606.8477102946,-6699866.376328185,499348.1810545102,-3526359.9042284507,-3639170.5978410523,-8382040.79303601,-2196656.9907056466,-5227318.931222685,-1809060.813912671,1059390.2651674654,-9852056.748113628,-4358154.143792953,-5436686.8142597545,-7368172.893039962,-2413232.0694436356,-185173.87369629368,-1619541.9649509974,181885.589882439,-3875936.4800426736,-2892633.2271101363,-9066585.983780216,-3095990.23173053,-2956745.8761398643,-2068427.3546133349,-7373459.226338116,453844.35583793,-9759674.021836989,-4544534.387997989,-7441807.8595418595,-5216872.474767169,-2434556.3449688647,-6837424.08358982,-6424086.589896282,-4407211.756579602,-1126647.7500037588,-3185090.103601726,-2791182.126106048,-4551706.412912827,-6437197.817320984,-8387683.491112145,-5564706.638673155,-4596848.022862125,-2318340.727627784,-101420.48943652399,-10775000.942815844,-4114360.84479375,-5069869.530224299,-4197645.444706492,-6108191.900133271,-5040701.939563721,-3428809.4063237496,2922816.353533298,-2788666.2399355583,-7322764.907156844,-6019687.547380887,-4335600.887146901,-4943678.288328836,-4066358.156052338,-773349.3570001144,4389927.164675454,-380725.7019588016,-4946636.917682057,-7849826.202798169,943403.4219764024,-6225694.369654277,-6458669.854422577,-4901945.5392593015,511230.22595288604,545237.6654987484,-2017201.2861063033,-9165005.30914579,-5203185.492222635,2774545.3420532923,509835.62063754164,-492038.2621003613,719792.9487678222,941275.0146486219,-9339004.69292113,-5141344.155543368,-2235712.0574973524,-4232493.171643702,-1447621.3799093869,-3035547.758797737,-8418007.57658092,-6459823.90271464,-468537.0773454923,-10288927.08241965,3279324.68658896,-2419698.126404451,-5811133.355758749,-1671356.1847436596,-7533289.725518363,-6225649.358983234,-9929430.216533985,-7453731.802124876,-2829543.5075683277,-2205663.4189898483,1430596.8409708682,5218345.301216304,-9846871.892029976,-5811777.900440924,-5032341.720457407,-8218211.206303814,-6905936.505449906,-710517.5552387089,-6668764.0995026715,-402910.5226881169,-5984491.579647439,-12139236.0095843,5545406.845924751,-5809957.451455619,-2937601.2329795435,-4988932.579159785,590257.5337430984,18443.18095666915,-8144598.557410421,-3274936.7848438853,1780035.8678831551,-5122892.562461069,-9924553.753194403,-7069731.896800034,-6980467.893404946,-3932305.743898852,381155.0004256703,2745275.805720305,-2880013.287456872,-3662543.722661441,-3693449.700374756,-14030886.344996959,-4441461.692991784,64805.60240973346,-11241040.1223289,-2270261.044693209,-2463791.95922572,-4127396.636575518,-2473347.8134314474,-4230459.472594917,-2228222.5672165174,-8602877.962783895,-5585258.973293798,-4224973.2475869395,-6478675.732624533,-2218505.4953699205,-47215.71778625995,-8365455.908036843,-4217757.033878772,-68045.79287280701,-6016561.801984273,-7332442.321406387,-4541300.259227727,-6374047.368382171,-8529179.16846412,-5601629.623839283,-3166552.491949126,-5970754.27957719,-7629312.230210364,3424064.6889589224,-5349915.981968004,-97130.83268926479,3140652.753966675,-11105209.358845472,-13824655.449963622,46463.81419892609,2160764.208599003,-7821741.2603298,-5231591.028150968,-6134417.7128300965,-4200602.018589586,-5697348.917434908,1983726.0220731534,266194.36178990826,-3916474.733027136,-260719.95766372792,1745622.6083335625,-4361492.725517621,-5014510.5929363705,-1023220.2899905071,-3413889.257522026,-4315596.151947916,-7738700.442817399,-4248213.808629928,710024.7649006937,-10023768.479694258,1993159.3356487565,-4056835.7660467383,-8150107.474361681,-3173075.4409329575,-8660075.87231559,-6506476.78214732,-5469083.442430602,-4365616.3454771275,-5019978.937856459,730735.0421162006,-4611537.9880364,-180240.14912008122,-2458426.6033520624,-5240236.779437497,-7838441.575089514,-4204574.854792824,314669.0078603327,-5686890.569696157,-2230074.7232813314,-6888542.4665397275,-11305891.380959954,-8844332.463216696,-7107292.288798511,-4517167.093057036,-1030280.9259545151,-6913643.797453271,-3113471.3539635073,-3062062.185492769,-2316599.556967212,432434.08390761726,-3707827.212458724,-2105300.5613561133,455866.4170880746,-6471333.7202359475,-5234432.180611966,-7570573.625419956,-4963229.533477295,-5524804.301518671,-5663626.551345873,-3345444.252209913,-9881684.415139278,-942207.9838868957,-9747267.53645067,-8387070.961419687,-7598574.687019471,-3404204.985991857,1346654.707166599,-4107255.2211209275,-4034640.6174711846,-11205278.4095264,-7077052.499391291,136514.63925348036,-90859.21369643882,-6969459.866816175,-7945525.15399177,-528772.8533075172,2594539.159632778,-11362389.433796313,-6321888.762689965,-6023045.56988791,-6046575.0831176825,738068.313813027,1080242.228769699,-6368481.0171816815,-4333823.706818199,-7942890.742000036,-3866493.7330097407,-3063460.2100656964,-1554500.2574363425,-4246166.634859612,-1948018.869813148,-6550036.251640677,-4908596.542565331,-8523212.421027582,542017.417450957,3624710.3397459425,-7225886.585619389,-3554967.9597922117,309806.2941000946,-1075662.1514714658,-2220097.5646344274,-5244708.421290273,-10551724.25579109,3247638.246754192,-5555234.057006072,-4705271.243757777,-4302085.973683442,-834873.7490460202,73117.69118086249,-6447427.852419846,-11423713.963085249,-692835.5540981479,-2696451.2387280893,-5851553.00752832,-3700542.0654211957,-7354784.251526134,-2031154.106960103,-4067476.9988505505,-3534897.5945625547,-3224920.947215209,-5291926.590058036,-12772183.101740232,1912880.020897437,-5366155.340507882,3271683.74870006,-2111574.953104982,-4832595.431833088,-3640126.636726793,-3098301.5388590973,-2861281.2264061123,-3310839.144656975,-2426779.960226791,-3071770.908351695,3884135.144228954,-1803911.8224321231,-9084459.612302307,-314635.35997238196,-942166.1306473613,-300396.411179509,-2674310.9289037995,-2951580.7558791153,-2091726.7902229,-1241189.5697790962,-8548070.297666749,-10382243.457400745,1321930.1200764198,-5990128.486082929,-3587464.7316745203,-3259601.8262999747,-7349387.667118065,2084484.507446833,-2744844.1508097313,-3788749.591363674,-7185272.185824525,-9550431.957595946,-3638481.23589525,292462.6341998987,-7282287.164344015,-5389163.971009957,1164275.8975382708,-10883081.040043402,-5174483.698643336,-592053.5387283918,-10688393.126836615,-3978298.45029635,-1915627.2514669336,-8068986.906767929,1335053.016042905,-6446731.3780201115,-7081081.2147844415,-2532450.835911477,-1013120.1637190375,1227961.1129239704,-2689958.5976553597,-3782953.9507728014,-3523820.6847822648,818031.047885498,-3072188.0619413927,-1682147.29667365,-7606732.6020915955,-6640350.711303336,-6921535.534664151,-4457994.920378955,-273806.08772504516,-4390352.509132624,-3088415.1352527384,620174.6589759253,-4581072.085009787,-7798124.869933572,-4883312.341714811,-3352420.401139293,-5502538.208498539,-2868479.5751552116,-3939467.4837092888,-5606455.703423813,-4429044.679533197,-2243089.991798127,2977474.6071166843,-1694731.8950581755,-3449405.9346053284,2934842.7423185147,-3899938.693340173,84148.37558669783,-2483267.425179531,-8828574.567070741,-5064396.907807365,-4171712.2692144345,966073.968883479,-6217789.50190343,3256394.567298202,-3633545.4782207925,1870091.7283847444,-6306870.809131914,5108027.333851153,-2244719.4408226833,1695989.3820121381,-3583444.1325513422,-7989855.557398362,-6197018.361776925,2304214.23137022,-3840063.050178202,-6021955.8723359555,-1892671.5042390171,-2207115.9673951697,-4365545.639650935,-356002.8684823811,-4784942.490579275,-6320709.689303404,-132741.66861990467,-106318.21749955975,-4385744.766652238,1343443.9936865848,-2700143.2566238623,-1533568.5072181504,-4561888.737333093,-6842102.196863942,-2282107.472520007,-2777551.3497482967,1444821.5046446472,-2693312.7809424084,-6508788.245068088,471163.37878025323,-4042544.4500037543,-8216370.247049134,-2680907.8884642553,-5561112.996164875,-297769.468404226,-6193340.346466735,-135663.39000935107,-6779717.7420874275,-1010189.1032533254,-4161607.8333312534,-6241251.190150369,-8060436.334786458,-3729119.0225738715,-6467617.176176773,-894814.9277370386,-1443577.5905412324,-3663022.2563584335,-7288111.598737031,-1791318.9875831567,-5557455.251679249,-1605820.432907734,-1669445.8073245902,-2650506.6211243123,-9755244.997978358,-1772079.2254107744,-354877.0997304786,-784481.0616788939,-2111552.3537241174,-5879092.393851567,-1886383.8835622333,-5938287.456844548,3060658.2469200008,-4287444.868378583,-571438.1004558168,-2943747.679423861,-8159196.953330619,-2791096.201196607,-7379172.367712453,-5691612.087090613,-3564322.711461,-8882428.219683355,-1412804.108111199,-687850.9291028902,-7638515.742872769,-3716198.7681160476,-6611941.529986452,-2778390.8489407413,-6359451.776459444,-698685.6495850068,-6531710.735894473,-206898.75824359804,-4543142.011815999,-3779354.3165042866,-7301785.612962796,-4091980.5875574704,-5765077.758341316,-469758.0004124455,-6632330.609595623,-5706802.211085331,-1486895.7774617989,-3379296.063447302,-6546852.293945981,422911.26992868073,-4451778.903114639,-2759715.19684943,-167908.91254238598,-3763956.8193878625,-2265255.571181625,4228405.960842356,-3102704.146504717,-7831111.534077808,-9741230.21048624,-5984519.379730444,-2574538.3240681887,-4276376.691610878,-8180723.509993346,-2799228.4710733425,-9803107.21617177,-1611749.359919509,1275134.6441778876,-3102382.2911573015,1504024.2211153693,-3921287.7335944567,-7201750.650846944,-8750871.352267304,-1442228.3022678941,-9900470.20646897,-7345971.291905645,-749346.3656832147,-2648550.7128379997,1312064.9968041033,-9821257.847755421,-3576103.261552751,-1192791.3171934467,-2973382.368300489,-7710918.784711299,-1208965.297572691,-2210552.5333561245,-1554959.1038952041,-2055496.0900869668,-5774810.291467071,715192.3478038106,-5150799.406577069,-6112860.66584008,-3733522.3009870686,-11220643.424839858,-5837530.059897758,-6933972.603031049,686031.6915793009,1535976.8543514796,-6650240.91974715,-6981638.715235578,-1786821.8484965675,-6041170.566762628,1313883.462424459,869276.773831903,-5483235.5693776095,-1743045.4958539326,-6503710.116661184,-3777591.5492311474,-1026787.0496391356,-2129358.2784610614,-6534043.628239673,-2980989.552836057,-2835260.5359230116,-7600049.6316216905,-1256775.3540299144,-5416310.917251429,-441765.86594658345,-5758663.015807172,-7922838.414454218,-3114989.280757852,-4070452.950477086,1443194.1628118958,-15220876.66608265,-3860585.4247916993,-6988190.942029703,-5423028.596298892,-3862376.731345305,-732592.1813190095,-3827661.590742441,60678.07413015328,635585.1477107964,-8840424.603532016,-3131597.390784247,-11383157.385169033,-5596316.114417672,4125520.3213111516,-5963398.978251258,-2049734.1386341676,-9602085.937575724,-8425932.084661186,-2362089.617970137,2377236.505615197,-7760311.834354324,3633991.4059222694,-2737704.3937971536,-3672975.8201991096,-3879106.8004290946,-4793921.905219069,-6547093.134103634,-4603148.657746263,-2604575.0844615623,126128.62586355582,269878.61460216343,-2344479.687823126,-4306472.976285234,-3053252.9920985103,-5160505.065736664,1564548.8472740725,-7638109.130723087,-1851000.8861940205,-5011759.417510661,-3031441.088917235,-9260333.763650363,-640103.3812817205,-954697.7305263206,-5981035.564451089,-3397892.7029076684,-3828543.9076619316,-3531580.5820805,-8931546.741965016,-1233818.7454490773,-6812034.322251949,3236534.042883586,-4684849.836720898,-3180115.262919493,-6093237.275615713,-6547577.163827173,-5470924.885103224,754234.7100479435,-83302.97646421567,-7714961.337948889,-2201630.564213341,-2055696.9155400656,-8573425.508840645,-7790512.138231636,-5981458.135704353,-510446.9111913461,-3646395.382864617,-541919.5644748174,-2038940.3156885952,-4360458.790925976,-941770.6019307747,-5165335.100041624,-2463451.319885364,-5837065.644858453,-9267780.396202905,-8597429.944198322,-2294154.136690611,-2777589.4660765007,-10260799.879296869,-6370096.373785716,-1082079.9780426938,-1665616.6063233,-1045361.6982635725,-8852084.673018312,-2958717.9689440504,-1318910.2885344084,-5819693.829757752,578254.4222214799,-5809568.862052629,-6785870.439744439,-247787.4391626101,-5006057.676939677,-415837.6457458157,-4810875.999463746,4222654.735188846,-1184871.2745470703,-7144633.802686092,-3904818.217689827,-2371142.361375047,-2618170.6983009204,3145405.2601169404,-5507254.101665761,3492465.499454377,-5736815.836425137,-2222457.606835693,-2891979.8270370644,-9854405.72016067,1207394.297922317,-5788991.764078068,-2635039.569459945,2643103.1936059445,-6899129.82012937,-2773474.2871997934,781865.1928119678,-4746501.416394351,-1593044.886094382,-5598799.191137316,-799383.2166787218,-1245549.2962287255,-9724205.37883132,-2133336.6514995415,-6652208.261533078,-4146643.253554249,1071030.798256196,-10312169.014224783,-9269064.810922332,-5528296.402302399,-3304690.454661833,5059.322540313005,-10666680.026964799,-1303490.8656748962,-1979045.377646992,-2767935.6676501073,-2237989.0293034185,-7391736.909092272,-8276406.373291567,540006.9774356969,-5105977.284868281,-1678699.0637774933,-5659818.582520418,-8018819.885223059,-3429070.583250491,-4934861.660232324,-593234.7985990997,-1469738.5198503342,217114.69624309056,-5214150.889498241,-1060607.9620718993,-9677906.315698812,-6246127.645126,-2800384.168518152,735845.4047352914,-94446.86465275101,-2274911.9499393394,-4138800.244373098,-8265872.952219162,128394.40983785689,-6055148.011004195,-6440394.6434040405,-1716600.35421703,-7944345.328899763,7070937.858324168,-3530384.5773898326,1629322.0281686913,-3203983.7208697274,3352136.3266023155,-10039923.819571909,-5485128.680394219,-4588665.841970975,-5856216.44751893,-8175033.466369206,-3103744.290607134,-3672361.8631026577,-7817765.666446598,-2394766.7324095555,-7057216.0582431555,-6193227.473931555,-5358416.875107866,-4930496.019764928,-3334773.914680997,-5851116.667070926,-5653911.069570851,-2885226.825716298,-3245269.5646392927,215020.02917139418,-6447374.053252913,-2933550.9882185776,-12782890.544211628,-3151405.34731997,-6310382.362275584,-3605080.8485092837,-7021752.183538875,-3255970.313126091,-343817.6427600272],"xaxis":"x","yaxis":"y","type":"histogram"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"value"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"count"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"title":{"text":"Simulated Opportunity"},"barmode":"relative"},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

p05 -9622369.707657166 
Mean -3561089.390384372 
p95 2078028.7585740048


In [19]:
# A little magic to automatically write my blog :)
import subprocess

subprocess.run(
    [
        "jupyter",
        "nbconvert",
        "--to",
        "markdown",
        "--output",
        "~/Documents/jakee417.github.io/_includes/markdown/rent_vs_buy_blog_post.md",
        "rent_vs_buy_blog_post.ipynb",
    ]
)

[NbConvertApp] Converting notebook rent_vs_buy_blog_post.ipynb to markdown
/opt/anaconda3/share/jupyter/nbconvert/templates/base/display_priority.j2:32: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.plotly.v1+json']) is not able to be represented.
  {%- elif type == 'text/vnd.mermaid' -%}
[NbConvertApp] Writing 1238807 bytes to /Users/jakeetaylor/Documents/jakee417.github.io/_includes/markdown/rent_vs_buy_blog_post.md


CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'markdown', '--output', '~/Documents/jakee417.github.io/_includes/markdown/rent_vs_buy_blog_post.md', 'rent_vs_buy_blog_post.ipynb'], returncode=0)